In [7]:
#STEP1:获得最终分词和标签
import re
import numpy as np
from nltk.tag import StanfordPOSTagger as POS #词性标注
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from nltk.tag import StanfordPOSTagger as STFDPOS 
import time
import os #POS需要java环境
java_path = "/home/data/liuchang/jdk1.8.0_191/bin/java"
os.environ['JAVAHOME'] = java_path

from zhon.hanzi import punctuation as cnpun

def is_pun(char):
    return (char in cnpun)
        
zh_pattern = re.compile(u'[\u4e00-\u9fa5]+')
def contain_zh(word):
    '''
    判断传入字符串是否包含中文
    :param word: 待判断字符串
    :return: True:包含中文  False:不包含中文
    '''
    #word = word.encode("utf-8")
    global zh_pattern
    match = zh_pattern.search(word)

    return match



print("processing data......")

with open('/home/data/liuchang/data/newcorpus.txt') as  f1:
    f11 = f1.readlines()
with open('/home/data/liuchang/data/mycorpusSeg.txt') as  f2:
    f21 = f2.readlines()
    
#f3 = open('/home/data/liuchang/data/final.txt','wb')
#f4 = open('/home/data/liuchang/data/tag.txt','wb')
textlist = [] #在一起的分词结果文本，有标点
classlist = [] #每个中文词的停顿标注
textwordslist = [] #文本的分词列表，有标点
wordlenlist = []#当前词的长度 
POSlist = [] #当前词的词性
cnwordlist = []#中文词标点
for line in range(0,10000):
    #x = x.encode("utf-8")
    #x = x.decode("utf-8")
    str0 = re.sub(r'#\d',' ', f11[line])#这个可以当作分词
    str0 = str0.strip('\n')
    
    str1 = f21[line]
    str1 = str1.strip('\n')
    classpattern = re.compile(r'\d')
    tag = classpattern.findall(f11[line])
    index1=0
    indexT=0
    for index1 in range(len(str0) * 2):
        try:
            if str0[index1] == ' ':
                if not(is_pun(str0[index1])):
                    indexT = indexT + 1
            if str0[index1] == str1[index1]:
                index1 = index1 + 1
            elif str0[index1] == ' ':        
                str1_list=list(str1)
                str1_list.insert(index1,' ')
                str1="".join(str1_list)
                index1 = index1 + 1
            else:
                str0_list=list(str0)
                str0_list.insert(index1,' ')
                str0="".join(str0_list)
                if not(is_pun(str0[index1-1])):
                    tag.insert(indexT,'1')
                    indexT = indexT + 1
                index1 = index1 + 1
        except:
            break
    
    
    textlist.append(str0)#有空格分词的文本
    classlist.append(tag)#只留标签（去除#）

for text in textlist:
    textword=[]
    textword=text.split()
    tempcnword = []    
    for word in textword:
        if contain_zh(word):
            tempcnword.append(word)
    textwordslist.append(textword)
    cnwordlist.append(tempcnword)
print("processing complete!")
###STEP2:特征提取

##feature1:句子长度
feature1 = []
for words in textlist:
    feature1.append(len(re.sub(' ','',words)))
#print(feature1)

##feature2:上下词和本词的字数
feature2 = []
for words in textlist:
    templen = []
    words = re.sub(r'Ｐ','',words)
    wordlist = (re.sub(r'[^\w\s]','',words)).split( )
    for word in wordlist:
        if contain_zh(word):
            templen.append(len(word))
    wordlenlist.append(templen)
#print(wordlenlist[0])
three = []
mix = []
for lists in wordlenlist:
    three = []
    lenth = len(lists)
    lists.append(0)
    lists.insert(0,0)
    for i in range(0,lenth):
        three.append(lists[i])
        three.append(lists[i+1])
        three.append(lists[i+2])
    mix.append(three)
step = 3
for feature in mix:
    feature = [feature[i:i+step] for i in range(0,len(feature),step)]
    feature2.append(feature)
#print(feature2)

##feature3:后面是',' '。' '?' '!' '""'(one-hot)
feature3 = []
dictpun = {',' : 0, '，' : 0, '.' : 1, '。' : 1, '?' : 1, '？' : 1, '!' : 1, '！' : 1, '\'' : 2, '‘' : 2, \
           '’' : 2, '"' : 2, "“" : 2, "”" : 2, ":" : 3} #1逗号，2句子的结束，3引号，4冒号 （其他符号 用try归到5即可
#这里先把wordlenlist复原，之后可能用到
for lists in wordlenlist:
    while 0 in lists:
        lists.remove(0)
#分词
    
for i in range(0,len(textlist)):
    temptext = textlist[i]
    temptextwordslist = textwordslist[i]
    truewordslist=[]
    for word in temptextwordslist:
        #count = count + 1
        if (contain_zh(word)):
            truewordslist.append(word)
    wcount = -1
    tempfeature3 = np.zeros((len(truewordslist),5))
    
    for word in temptextwordslist:
        wcount = wcount + 1
        if (contain_zh(word)):
            continue
        else:
            wcount = wcount - 1
            try:
                tempfeature3[wcount][dictpun[word]] = 1
            except:
                tempfeature3[wcount][4]=1
    tempfeature3 = tempfeature3.tolist()
    feature3.append(tempfeature3) 
#print(feature3[9770])

##feature4:上下文的词性
feature4 = []
#先用枚举试一试，要不行最后还是得
dictPOS = { 'PU' : '0', 'VA' : '1', 'VC' : '2', 'VE' : '3', 'VV' : '4', 'NR' : '5', 'NT' : '6', 'NN' : '7', 'LC' : '8', 'PN' : '9',\
          'DT' : '10', 'CD' : '11', 'OD' : '12', 'M' : '13', 'AD' : '14', 'P' : '15', 'CC' : '16', 'CS' : '17', 'DEC' : '18',\
           'DEG' : '19', 'DER' : '20', 'DEV' : '21', 'SP' : '22', 'AS' : '23', 'ETC' : '24', 'MSP' : '25', 'IJ' : '26',\
           'ON' : '27', 'JJ' : '28', 'FW' : '29', 'LB' : '30', 'SB' : '31', 'BA' : '32'} #没弄错的话一共33种词性
#POSlist获取,获取结果可在缓存区找到
'''
import numpy as np
from tqdm import tqdm_notebook as tqdm
from nltk.tag import StanfordPOSTagger as STFDPOS 
import os #POS需要java环境
java_path = "/home/data/liuchang/jdk1.8.0_191/bin/java"
os.environ['JAVAHOME'] = java_path

cntagger = STFDPOS(model_filename = r'/home/data/liuchang/data/stanford-postagger-full/models/chinese-distsim.tagger',path_to_jar=r'/home/data/liuchang/data/stanford-postagger-full/stanford-postagger.jar') 
for text in tqdm(textlist):
    text = cntagger.tag(re.findall(u"[\u4e00-\u9fa5]+",text)) 
    i = 0
    tags = []
    for words in text:
        tags.append(re.sub(r'^.*#','',words[1]))
    POSlist.append(tags)
print(POSlist[0])
'''

zerovec = np.zeros(33)
zerovec = zerovec.tolist()
POSlist = [['NR', 'VV', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VV', 'PN'], ['NR', 'VV', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'CC', 'NR', 'VV'], ['NN', 'NN', 'CC', 'NN', 'VV'], ['VV', 'AD', 'CD', 'M', 'CD', 'CC', 'PN'], ['NR', 'AD', 'NR', 'VV', 'VV'], ['NN', 'CS', 'VE', 'NN', 'AD', 'VV'], ['P', 'VV', 'NN', 'CC', 'JJ', 'NN', 'VV'], ['NR', 'VV', 'NN', 'NN'], ['NN', 'VV', 'VV', 'P', 'AD', 'AD', 'VV', 'VV'], ['NR', 'NR', 'CC', 'NR', 'VV'], ['NR', 'VV', 'NN', 'NR'], ['PN', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NR', 'NN', 'VV', 'NN'], ['AD', 'NR', 'AD', 'AD', 'P', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NN', 'VA', 'CC', 'VA', 'VA', 'CC', 'VA'], ['NR', 'NN', 'NN', 'NN', 'NR', 'NN', 'NN', 'VV'], ['NR', 'VV', 'NN', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'P', 'DT', 'NN', 'VV', 'IJ'], ['NN', 'NN', 'NR', 'NN', 'NN', 'NN'], ['NR', 'NR', 'NR', 'NN', 'NN'], ['PN', 'P', 'NR', 'NN', 'NN', 'NN'], ['NN', 'VV', 'JJ', 'NN'], ['AD', 'NR', 'AD', 'AD', 'P', 'NT', 'VV'], ['NR', 'CC', 'NN', 'NN', 'NN', 'VV'], ['NN', 'NN', 'NN', 'CC', 'NN'], ['NN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'VV', 'NR', 'VV', 'NN', 'VV'], ['NR', 'NN', 'CC', 'NN', 'NR'], ['NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NN', 'LB', 'NN', 'VV', 'VV', 'SP'], ['NR', 'VV', 'NN', 'NR'], ['NN', 'NN', 'VV', 'VV', 'VV'], ['NN', 'AD', 'VV', 'AD', 'VA', 'AD', 'VV', 'AD', 'VA'], ['NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV'], ['JJ', 'NN', 'LC', 'DEG', 'JJ', 'NN'], ['NN', 'NN', 'NN', 'CD', 'NN'], ['NR', 'AD', 'LB', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'NN', 'VV', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV'], ['NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'VV', 'AD', 'VV'], ['VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['NR', 'AD', 'AD', 'AD', 'VA', 'SP'], ['NR', 'DEG', 'NN', 'NN'], ['NN', 'VV', 'AD', 'VV', 'CC', 'NN'], ['NR', 'CC', 'NR', 'NN', 'NR', 'NR', 'AD', 'VV', 'CD'], ['AD', 'NN', 'VV', 'AD', 'VV', 'VV'], ['JJ', 'NN', 'NN', 'NN', 'VA'], ['CC', 'VV', 'NR', 'NN', 'VV', 'NN'], ['P', 'NN', 'VV', 'VV', 'NN', 'NR', 'NN', 'VV', 'VV', 'VV'], ['VV', 'JJ', 'NN', 'ETC', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['P', 'NR', 'VV', 'VV'], ['NR', 'AD', 'NR', 'NR', 'VV', 'NR', 'VV', 'P', 'NR', 'NN', 'DEG', 'JJ', 'NN'], ['NR', 'NR', 'AD', 'VV', 'VV', 'NN'], ['PN', 'PN', 'PN', 'PN', 'PN', 'PN', 'P', 'NN', 'AD'], ['NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'CC', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'SB', 'VV'], ['NR', 'NR', 'NR', 'CC', 'NN', 'VV', 'NN'], ['NN', 'NN', 'JJ', 'NN', 'NR'], ['AD', 'VV', 'CD', 'CD', 'CD', 'CD', 'CD', 'NN'], ['NN', 'VV', 'VV', 'NN'], ['NN', 'VA', 'MSP', 'LC', 'NN', 'NN', 'DEG', 'NN', 'AD', 'MSP', 'NN', 'VA'], ['NN', 'CC', 'NN', 'VV', 'NN', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['VV', 'NR', 'NN', 'VV'], ['AD', 'NR', 'NR', 'NR', 'NR', 'VV', 'NN', 'VV', 'VC', 'NN', 'VV', 'NN', 'NN'], ['P', 'VV', 'PN', 'P', 'PN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NN', 'LC', 'VV', 'VV', 'VV', 'NR', 'NR', 'NN'], ['P', 'VV', 'VV', 'VV'], ['NR', 'AD', 'P', 'NN', 'NN', 'NR', 'VV', 'NN'], ['AD', 'VE', 'NN', 'VV', 'VV', 'PN', 'SP'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'P', 'NN', 'SB', 'VV'], ['JJ', 'NN', 'JJ', 'NN'], ['NN', 'P', 'NN', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'NN', 'NN', 'NN', 'CS', 'VV', 'NN'], ['JJ', 'NR', 'VV', 'NN', 'VV'], ['DT', 'JJ', 'NN', 'AD', 'VV'], ['NN', 'NN', 'NR', 'NN', 'VV', 'NN'], ['AD', 'NR', 'VA', 'AD', 'SB', 'AD', 'NN', 'VV'], ['AD', 'VV', 'VV', 'NR', 'VV', 'NN'], ['NT', 'VC', 'NN', 'NN', 'AD', 'VV', 'AD', 'VA', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VE', 'NN'], ['NN', 'NN', 'NN', 'AD', 'VV', 'NR', 'VV', 'DT', 'CD', 'NN', 'VV', 'NN'], ['VV', 'SP', 'PN', 'AD', 'VV', 'VV', 'PN'], ['NR', 'CC', 'NR', 'VE', 'NN'], ['VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'LC'], ['AD', 'VV', 'NN', 'NN', 'VV', 'PN', 'VV'], ['NN', 'P', 'AD', 'AD', 'VV', 'VV', 'VV', 'CD', 'NN'], ['AD', 'NN', 'CC', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'CC', 'VV', 'JJ', 'NN'], ['NR', 'CC', 'NR', 'ETC', 'VV', 'NN'], ['JJ', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['VV', 'NR', 'NN', 'VV', 'CD', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'VV'], ['NR', 'DT', 'P', 'NN', 'VV', 'BA', 'NN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'NN'], ['OD', 'VC', 'NN', 'VV', 'NN'], ['VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NN', 'NN'], ['JJ', 'NN', 'NN', 'VC', 'NN', 'NN'], ['PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'JJ'], ['NT', 'NN', 'NR', 'P', 'NN', 'NN', 'VV', 'NR'], ['NN', 'NN', 'VC', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'CD', 'CD', 'CD', 'AD', 'VV', 'NR', 'NN'], ['NR', 'NN', 'NN', 'JJ', 'NN', 'NR', 'NN', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['PN', 'VV', 'P', 'NR', 'VV', 'NN', 'NN'], ['NR', 'NR', 'AD', 'VV', 'AS', 'JJ', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'SP'], ['P', 'NN', 'MSP', 'VA', 'DEC', 'NN', 'NR'], ['VV', 'NN', 'VV', 'VV', 'NN'], ['NT', 'VA', 'VA', 'VV', 'VV', 'NN'], ['VV', 'VV', 'AD', 'NR', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'VE', 'JJ', 'DEG', 'NN', 'NN', 'P', 'NN', 'ETC', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'NR', 'VV', 'NN', 'NR', 'AD', 'VV', 'VV', 'VV', 'SP'], ['AD', 'VV', 'NN', 'LC', 'DEG', 'NN', 'SP'], ['NR', 'CC', 'NN', 'NN', 'VV'], ['NN', 'VV', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'NN', 'VE', 'CD', 'CD', 'CD'], ['PN', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'NR', 'VV'], ['NN', 'VV', 'NN', 'VV'], ['AD', 'NN', 'NN', 'AD', 'AD', 'AD', 'VA'], ['VV', 'NN', 'NN', 'NN', 'LC', 'DEG', 'NN', 'SP'], ['NN', 'NN', 'AD', 'VV', 'NN', 'JJ', 'NN'], ['NR', 'AD', 'VV', 'NN'], ['PN', 'VV', 'VV', 'AD', 'P', 'NN', 'VV', 'VV', 'VV', 'VV'], ['P', 'VV', 'VV', 'NN', 'LC', 'AD', 'JJ', 'VV', 'NN', 'NN'], ['PN', 'VV', 'VV', 'NN', 'NN', 'VV', 'VA', 'DEC', 'NN', 'VV', 'NN'], ['AD', 'NR', 'NN', 'NT', 'AD', 'VV', 'P', 'JJ', 'NN', 'VV'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['PN', 'VV', 'PN', 'AD', 'VA'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NR', 'NN'], ['NR', 'NR', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VV', 'VV', 'DER', 'NN', 'P', 'NN'], ['VV', 'PN', 'VV', 'NN', 'AD', 'VV', 'VA'], ['NR', 'NN', 'CC', 'NR', 'SB', 'VV', 'VC', 'NN', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'AD', 'AD', 'VV', 'AD', 'VV'], ['NR', 'AD', 'LB', 'NN', 'NN', 'VC', 'NN', 'OD', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NR', 'P', 'NN', 'NN', 'VV', 'NN', 'P', 'NN', 'VV', 'NR', 'NR'], ['AD', 'VV', 'NN', 'PN', 'AD', 'VV', 'NR'], ['NR', 'VV', 'NN', 'NR', 'NN', 'VV', 'NN'], ['VV', 'VV', 'NN', 'NN', 'PN', 'AD', 'VV', 'JJ', 'NN'], ['PN', 'AD', 'NR', 'JJ', 'NN', 'JJ', 'VV', 'VA', 'DEC', 'NN'], ['VV', 'VV', 'NN', 'VV', 'NN', 'NN'], ['VV', 'JJ', 'NN', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['AD', 'AD', 'VV', 'CD', 'NN', 'VV', 'VV'], ['NN', 'NN', 'VV', 'AS', 'NN', 'VV', 'VV'], ['NR', 'NN', 'PN', 'VV', 'VC', 'VV', 'VV'], ['NN', 'AD', 'VE', 'CD', 'CD', 'M', 'NR', 'NN'], ['NR', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'VV', 'LC', 'DEG', 'JJ', 'NN'], ['NR', 'NN', 'NN', 'AD', 'VV'], ['VV', 'DEC', 'NN', 'VE', 'NR', 'CC', 'NN', 'ETC'], ['AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NR', 'NN', 'NR', 'NN', 'VC', 'NR', 'NN', 'NN', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'AD', 'VV'], ['VV', 'NR', 'NR', 'NR'], ['NN', 'VV', 'PN', 'VV', 'NN', 'VV', 'PN', 'NN'], ['VV', 'NN', 'VV', 'NN', 'SP'], ['NR', 'NN', 'NN', 'NN'], ['P', 'NN', 'LC', 'VV', 'AD', 'VA'], ['NR', 'VV', 'VV', 'P', 'NR', 'VV', 'NR', 'NN'], ['NR', 'VV', 'AS', 'VV', 'VV', 'VV'], ['AD', 'CD', 'CD', 'NN', 'VV', 'NN'], ['PN', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'NN', 'NR', 'LC'], ['NR', 'AD', 'VV', 'PN', 'MSP', 'AD'], ['AD', 'VV', 'VV', 'JJ', 'NN', 'VV'], ['VA', 'VA', 'AD', 'AD', 'VV', 'SP'], ['P', 'NN', 'JJ', 'NN', 'AD', 'VC', 'P', 'JJ', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NR', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'NN'], ['NN', 'VV', 'VA', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'VC', 'NR', 'NN', 'VV', 'NR', 'NN'], ['P', 'NN', 'LC', 'AD', 'VV', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'P', 'NN', 'NN', 'VV', 'CC', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'ETC'], ['NN', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN', 'P', 'NN'], ['JJ', 'NN', 'NN', 'NR', 'NN', 'LB', 'VA', 'VV', 'AD', 'VV'], ['NR', 'NR', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'VE', 'JJ', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['VV', 'DT', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NR', 'LC', 'VV', 'NN', 'LC', 'VV'], ['NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'NR', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VV', 'AS', 'NN', 'VV', 'NN'], ['NN', 'P', 'NN', 'NN', 'VC', 'NN', 'VV'], ['NR', 'NN', 'P', 'NN', 'P', 'NR', 'VV'], ['AD', 'NR', 'AD', 'VV', 'NN'], ['NR', 'P', 'NN', 'VV', 'JJ', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN', 'NN'], ['VV', 'NN', 'CD', 'VA', 'AD', 'VA', 'VA'], ['JJ', 'NN', 'SP', 'MSP', 'VV', 'NN', 'VV'], ['NN', 'P', 'NN', 'NN', 'NN', 'LB', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NR', 'AD', 'VV'], ['NT', 'NR', 'NN', 'VV', 'NN'], ['NN', 'NN', 'CC', 'NR', 'NR', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'NN', 'JJ', 'NN'], ['NR', 'VV', 'PN', 'NN', 'NN', 'NN', 'VV'], ['AD', 'NN', 'VV', 'NN', 'SP'], ['AD', 'P', 'VV', 'NN', 'MSP', 'VV', 'NN'], ['NN', 'P', 'NN', 'VV', 'NN'], ['NN', 'P', 'NN', 'VA', 'VV', 'NN', 'NN'], ['AD', 'NN', 'VV', 'AD', 'VA', 'AD', 'AD', 'VA'], ['NR', 'VA', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'CD', 'NN', 'VV'], ['NN', 'VV', 'P', 'NR', 'NR'], ['PN', 'AD', 'PN', 'P', 'PN', 'VV'], ['DT', 'NN', 'AD', 'VV', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN'], ['VE', 'NN', 'VV', 'NN', 'VC', 'NR', 'NN', 'NN', 'NR', 'AD'], ['NR', 'VV', 'VV', 'NR', 'NN', 'NN'], ['NN', 'NN', 'NN', 'P', 'NN', 'NN'], ['VC', 'NN', 'IJ', 'VV', 'SP'], ['NN', 'VV', 'VV', 'DEC', 'NN', 'VV', 'NN'], ['P', 'P', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VA'], ['CD', 'NN', 'VV', 'AD', 'VV', 'CD', 'VV', 'VV', 'NN'], ['NR', 'VV', 'NN', 'VV', 'NN'], ['AD', 'AD', 'NN', 'AD', 'P', 'NN', 'VV'], ['JJ', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NR', 'NN', 'NN', 'NR'], ['NN', 'AD', 'NN', 'AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NN', 'VV', 'VV', 'DEC', 'VC', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NN', 'AD', 'VV'], ['NR', 'OD', 'M', 'NN', 'NN'], ['AD', 'VV', 'AD', 'VA', 'SP'], ['AD', 'VV', 'NR', 'NN', 'NN'], ['AD', 'AD', 'PN', 'VV', 'VV', 'NR'], ['VV', 'NN', 'NN', 'VV', 'NN'], ['NR', 'ETC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'LC', 'AD', 'VV', 'VV', 'NN'], ['NR', 'NN', 'CD', 'VV', 'NR', 'AD', 'VV'], ['AD', 'NR', 'P', 'VV', 'VV', 'DEC', 'NR', 'VV', 'NR'], ['NR', 'NN', 'NR', 'NR'], ['AD', 'CD', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NR', 'NR', 'JJ', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'JJ'], ['NN', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['NR', 'VV', 'NN', 'NN', 'NR', 'NN', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VV', 'CC', 'VV', 'VV'], ['NR', 'VV', 'DEC', 'NR', 'CD', 'M', 'NN', 'NN', 'CC', 'NN', 'NN'], ['NR', 'CC', 'NR', 'VV'], ['PN', 'AD', 'VA', 'VV', 'CC', 'VV', 'NN'], ['AD', 'VV', 'AD', 'AD', 'VV', 'NN', 'VA'], ['CD', 'NN', 'VV', 'NN'], ['NT', 'NR', 'NN', 'VV'], ['VV', 'NR', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'NN', 'AD', 'VV'], ['NN', 'NN', 'VV', 'VE', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['P', 'NN', 'AD', 'VC', 'CD', 'JJ', 'NN'], ['JJ', 'NN', 'VV', 'VV', 'CD', 'NN'], ['NR', 'AD', 'AD', 'AD', 'VV', 'AD', 'CD', 'NN'], ['P', 'VV', 'DEC', 'NN', 'LC', 'AD', 'VV'], ['AD', 'BA', 'PN', 'VV', 'AS', 'VV', 'NN', 'NN', 'VV', 'NN', 'VE', 'NN'], ['AD', 'NR', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['PN', 'P', 'NN', 'VV', 'PN', 'NN', 'PN', 'NN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NT', 'NR', 'NR', 'VV', 'NN'], ['NN', 'P', 'VV', 'AS', 'NN', 'NN', 'MSP', 'VV', 'AD'], ['AD', 'NN', 'AD', 'VV', 'P', 'NN', 'VA', 'NN'], ['AD', 'VV', 'AD', 'VV', 'AD', 'VV'], ['VV', 'NN', 'LC', 'AD', 'VV', 'CD', 'NN', 'NN', 'AD', 'NN', 'NN', 'CC', 'NN'], ['NT', 'NR', 'DEG', 'NN', 'AD', 'NN', 'DEG', 'NN', 'VA', 'VA', 'AD', 'VV'], ['VV', 'VV', 'P', 'NN', 'AD', 'VV', 'VV'], ['AD', 'AD', 'AD', 'VV', 'SP'], ['NN', 'JJ', 'NN', 'NR', 'NT', 'NN'], ['NR', 'NR', 'AD', 'VV', 'VV', 'VV', 'NN', 'NR'], ['NN', 'NN', 'NN', 'VV'], ['VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NR', 'AD', 'SB', 'VV', 'CD', 'CD', 'M', 'DEG', 'NN'], ['VV', 'LC', 'VV', 'NN', 'VV', 'VV', 'CD', 'VV'], ['NN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NR', 'P', 'NN', 'VV', 'VV', 'NN', 'MSP', 'VV'], ['NN', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'VV'], ['VV', 'VV', 'VV', 'VV'], ['AD', 'NN', 'VA', 'VA', 'VA'], ['NR', 'CC', 'NR', 'NN', 'VV'], ['VV', 'NN', 'VV', 'NN'], ['NR', 'NR', 'AD', 'P', 'DT', 'NN', 'VV', 'NR', 'NR', 'CC', 'NR'], ['NN', 'VV', 'NR', 'JJ', 'NR', 'NN', 'NR', 'NN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'NR', 'NN', 'NN'], ['NN', 'P', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NR', 'JJ', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'NN', 'LC'], ['DT', 'DEG', 'VC', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'ETC', 'NN'], ['NN', 'P', 'NN', 'VV', 'AD', 'VA', 'NN', 'CC', 'VA', 'DEC', 'NN'], ['JJ', 'NR', 'VV', 'NR'], ['JJ', 'NN', 'VV', 'NN'], ['NN', 'VV', 'DER', 'VV', 'NN', 'SP'], ['VV', 'NR', 'NR', 'NN'], ['PN', 'VV', 'VC', 'VV', 'VV', 'NN', 'NN', 'SP'], ['NR', 'AD', 'AD', 'NN', 'SP', 'AD', 'VE', 'NN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'NN', 'AD', 'VV', 'SP'], ['NN', 'P', 'NN', 'LC', 'VV'], ['P', 'NN', 'SP', 'NN', 'SP', 'VE', 'NN', 'SP'], ['AD', 'P', 'VV', 'VV', 'VV', 'NN'], ['NR', 'NR', 'NN', 'VV', 'NR', 'NR', 'NR', 'NR', 'NR', 'AD', 'AD', 'VV'], ['NR', 'AD', 'VV', 'VC', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VA'], ['NR', 'VV', 'AD', 'JJ', 'NN', 'AD', 'BA', 'NN', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'PN', 'VV', 'PN', 'VV', 'AD', 'VV'], ['NN', 'VV', 'VV', 'P', 'NN'], ['VV', 'NN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'NN', 'CD', 'M', 'NN', 'NN', 'MSP', 'VV'], ['NN', 'VV', 'NN', 'CC', 'NN'], ['NN', 'AD', 'VC', 'JJ', 'NN', 'P', 'VV', 'DEC'], ['VV', 'NN', 'NR', 'AD', 'LB', 'NN', 'VV'], ['IJ', 'NN', 'NN', 'VC', 'PN'], ['DT', 'PN', 'AD', 'P', 'NR', 'NN', 'VV'], ['VV', 'NN', 'CS', 'VV', 'VV', 'VV'], ['PN', 'AD', 'AD', 'VV', 'VV', 'NR'], ['NN', 'VV', 'NR', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN', 'VV'], ['NT', 'VE', 'NN', 'NT', 'VV', 'NN'], ['VV', 'NR', 'AD', 'VV', 'CD', 'IJ'], ['AD', 'NN', 'NN', 'LC', 'VE', 'NN', 'VA', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'VV', 'SP'], ['VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN'], ['AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'VV'], ['PN', 'VV', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NR', 'AD', 'VV', 'AS', 'NN', 'NN'], ['NN', 'VV', 'VV', 'VV', 'NT', 'VV', 'NN', 'AD', 'VV', 'NN', 'P', 'NN', 'NN', 'VV'], ['NR', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VA'], ['AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'VV', 'PN', 'VV', 'PN', 'VV', 'NN', 'PN', 'AD', 'VV', 'NN', 'SP'], ['NN', 'VV', 'NR', 'VV', 'VV'], ['NR', 'P', 'VV', 'NR', 'VV'], ['DT', 'M', 'NN', 'VV', 'NR', 'NR', 'NN'], ['NN', 'NR', 'AD', 'P', 'VA', 'DEC', 'NN', 'CC', 'NR', 'NN', 'AD', 'VV', 'NN'], ['NR', 'CC', 'NR', 'AD', 'VV'], ['NN', 'AD', 'VE', 'NN', 'NN', 'ETC', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NN', 'VV'], ['NT', 'NN', 'VV', 'AD', 'VV', 'PN', 'VC', 'P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NN', 'NN', 'VV', 'AD', 'AD', 'VE', 'NN'], ['NN', 'VV', 'NN', 'VV', 'M', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'LC', 'VV', 'NR', 'NN', 'NN'], ['AD', 'VE', 'NN', 'VV', 'VV', 'VV'], ['VV', 'NN', 'AD', 'VC', 'NN'], ['NN', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'PN', 'VV', 'AS'], ['NT', 'JJ', 'NN', 'VV', 'NR', 'AD', 'VA'], ['P', 'NN', 'LC', 'VE', 'NN', 'AD', 'VV', 'NN', 'NN'], ['VV', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NR', 'NR', 'DEG', 'NN', 'VV', 'AS', 'NR', 'NN'], ['NN', 'LC', 'AD', 'VV', 'NN', 'VV', 'NT', 'DEG', 'NR', 'NN'], ['VV', 'AS', 'AD', 'VV', 'VV', 'PN', 'NN'], ['AD', 'VV', 'P', 'NN', 'AD', 'VV', 'JJ', 'NN', 'P', 'NN', 'NN', 'M', 'VV', 'MSP', 'VV'], ['NN', 'NN', 'NN', 'LC', 'VE', 'NN'], ['NR', 'NN', 'NN', 'VV', 'PN', 'AD', 'SP'], ['NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NR', 'NR', 'NN'], ['PN', 'NN', 'VA', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NR', 'NR', 'VV', 'NN', 'VC', 'NN', 'NN', 'MSP', 'VV'], ['VV', 'NN', 'LC', 'NR', 'VV', 'AS', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NT', 'NN', 'NN', 'VV'], ['VA', 'VC', 'PN', 'VV', 'LC', 'VV', 'DEC', 'VV'], ['NR', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'DER', 'AD', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DT', 'NN', 'NN', 'NN', 'VV'], ['AD', 'JJ', 'NN', 'NN', 'AD', 'AD', 'VV'], ['VV', 'NR', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'VV', 'P', 'NN', 'VV'], ['VV', 'NR', 'NN', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'ETC', 'JJ', 'NN', 'LB', 'NN', 'VV'], ['AD', 'PN', 'AD', 'DT', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NR'], ['VV', 'AD', 'VV', 'JJ', 'NN'], ['P', 'AD', 'VV', 'PN', 'VV', 'PN', 'SP'], ['NT', 'NN', 'BA', 'NR', 'NN', 'NN', 'VV', 'VV', 'NR', 'AD', 'VV', 'AS', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['CD', 'NN', 'VV', 'VV', 'NN'], ['AD', 'PN', 'AD', 'P', 'NN', 'NN', 'NN', 'ETC', 'CD', 'NN', 'NN'], ['NN', 'NN', 'P', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NN', 'P', 'NN', 'NN', 'NN', 'NN', 'VV'], ['NR', 'AD', 'AD', 'VV', 'DEV', 'VV', 'VV', 'AS', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VA', 'CC', 'VA'], ['AD', 'VV', 'VV', 'NN', 'NN', 'VV'], ['NN', 'NN', 'VV', 'PN', 'M', 'NN', 'VV', 'AD'], ['NR', 'AD', 'VV', 'NN', 'NN'], ['JJ', 'CD', 'CD', 'NT', 'NR', 'NR', 'VV', 'NR', 'NR', 'VV'], ['P', 'NR', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'CD', 'NN', 'CD', 'NN', 'AD', 'P', 'NN', 'VV', 'SP'], ['NN', 'NN', 'DEG', 'NN', 'LB', 'NN', 'NN', 'VV'], ['NN', 'NN', 'NN', 'VA', 'AD', 'VV'], ['P', 'PN', 'CD', 'NN', 'PN', 'AD', 'VV', 'VV'], ['NN', 'PN', 'VV', 'AD', 'DT'], ['NN', 'NR', 'NN', 'NN'], ['IJ', 'AD', 'VV', 'PN', 'P', 'NN', 'VA', 'VA'], ['NN', 'CD', 'NN', 'VV', 'NN', 'MSP', 'VV', 'NN', 'IJ', 'NN', 'VV', 'JJ', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'SP'], ['NR', 'NR', 'NR', 'VC', 'P', 'NR', 'VV', 'NR', 'NR', 'NR'], ['VV', 'NR', 'AD', 'VE', 'NR', 'SP'], ['NN', 'VV', 'VV', 'VV'], ['JJ', 'NN', 'NN', 'VV', 'NR', 'NN'], ['VV', 'NN', 'VV', 'NN', 'NN'], ['NN', 'AD', 'AD', 'AD', 'P', 'NR', 'NN', 'MSP', 'VV', 'NR'], ['PN', 'VV', 'VV', 'VV', 'VV', 'VV', 'NN'], ['AD', 'PN', 'AD', 'JJ', 'VV', 'VV', 'AD', 'AD', 'VV', 'AS'], ['NN', 'DEG', 'NN', 'AD', 'VC', 'NN', 'NN', 'NR', 'NN', 'LC', 'DEG', 'JJ', 'VC', 'NN', 'VA', 'SP'], ['AD', 'P', 'NN', 'VV', 'DT', 'AD', 'VC', 'P', 'NN', 'NN', 'VV'], ['JJ', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'ETC', 'VV'], ['NR', 'AD', 'P', 'NN', 'VV', 'VA', 'DEC', 'JJ', 'NN', 'VV', 'P', 'NN', 'DEC', 'NN'], ['NN', 'LC', 'NN', 'VV', 'VV', 'NN', 'AD', 'VA'], ['AD', 'VV', 'NN', 'SP', 'VV', 'JJ', 'NN', 'NN'], ['CD', 'CD', 'NN', 'VV', 'NR', 'NN', 'CC', 'NR', 'NN'], ['VV', 'NR', 'AD', 'VV', 'SP'], ['NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD'], ['NR', 'CC', 'NR', 'NN', 'VV'], ['AD', 'VV', 'AS', 'JJ', 'NN', 'AD', 'VE', 'SP'], ['P', 'PN', 'VA', 'VV', 'NR', 'NN', 'NR', 'AD', 'VV', 'VV'], ['NR', 'AD', 'AD', 'VV', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VV', 'AS', 'NN', 'DEC', 'CS', 'NN', 'NN', 'AD', 'VA'], ['NT', 'NN', 'AD', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VV', 'NR'], ['NR', 'VV', 'NN', 'NN', 'ETC', 'NN', 'P', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NT', 'P', 'NN', 'NN', 'VV'], ['PN', 'NN', 'LC', 'VV', 'NN', 'P', 'NN', 'NN', 'AD', 'P', 'NN', 'VE', 'M', 'NN', 'NN'], ['NR', 'NN', 'NN', 'NR', 'AD', 'AD', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV', 'SP'], ['NN', 'NN', 'VC', 'NN', 'NN', 'NN', 'NN', 'SB', 'VV'], ['CD', 'NN', 'CD', 'NN', 'VV', 'VV', 'PN', 'VV', 'VV'], ['VV', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'P', 'NN', 'NN', 'LC', 'VV'], ['P', 'VA', 'DT', 'NN', 'CC', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NR', 'NN', 'VV', 'AS'], ['NN', 'AD', 'VV'], ['NR', 'VV', 'NR', 'NN', 'NN', 'VV', 'VV', 'VV', 'NR'], ['NN', 'NN', 'NN', 'NN', 'NN'], ['NR', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NR', 'VV', 'NN', 'VV'], ['NT', 'AD', 'VV', 'VV', 'SP'], ['NT', 'AD', 'NN', 'JJ', 'NN', 'NR', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NR', 'NR', 'NR', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'NR', 'NN', 'AD', 'VV'], ['VV', 'NN', 'VC', 'NN', 'NN', 'DEG', 'CD', 'NN'], ['P', 'NR', 'DEG', 'NN', 'NR', 'NN', 'VV', 'VV', 'NR', 'VV', 'VV'], ['NN', 'P', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV'], ['NN', 'NR', 'NN', 'NN', 'NN', 'CD', 'NN'], ['AD', 'NN', 'AD', 'NN', 'SP'], ['AD', 'AD', 'AD', 'VV', 'VV', 'NN'], ['CD', 'JJ', 'DEG', 'NN', 'P', 'CD', 'M', 'P', 'NN', 'LC', 'CC', 'VV'], ['JJ', 'NN', 'P', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'P', 'NN', 'VV', 'NN', 'P', 'NN'], ['NR', 'VV', 'AD', 'JJ', 'AD', 'JJ'], ['NR', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'NN', 'NN', 'VV'], ['PN', 'VV', 'CD', 'NR', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['PN', 'VV', 'PN', 'PN', 'VV', 'DT', 'NN', 'NN', 'AD', 'VV', 'VA', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'VV'], ['NN', 'VV', 'NR', 'DEG', 'NN', 'NR', 'AD', 'VV', 'AS', 'NR', 'DEG', 'NN'], ['P', 'NR', 'NN', 'VV', 'VA', 'MSP', 'VV', 'NN', 'VV', 'NR'], ['DT', 'NN', 'VE', 'JJ', 'NN'], ['AD', 'NR', 'VV', 'P', 'NN', 'AD', 'VV'], ['PN', 'AD', 'AD', 'VV', 'AD', 'VV', 'SP'], ['AD', 'AD', 'AD', 'VA', 'SP'], ['NN', 'VV', 'DEC', 'P', 'NN', 'VA'], ['NN', 'NN', 'NR', 'NR', 'VV', 'JJ', 'NN', 'PN', 'VV', 'NN'], ['NR', 'P', 'NN', 'NN', 'NR', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'VV', 'PN', 'VV'], ['NR', 'VV', 'NN', 'NN', 'VV', 'AD', 'P', 'NN', 'VA', 'VV'], ['NT', 'VV', 'AD', 'VA', 'AD', 'PN', 'NN', 'VV', 'PN'], ['NN', 'NN', 'JJ', 'JJ', 'NN', 'AD', 'VV', 'VA'], ['JJ', 'NN', 'NR', 'P', 'P', 'NN', 'NN'], ['VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['AD', 'AD', 'VV', 'JJ', 'AD', 'VV'], ['AD', 'NN', 'AD', 'AD', 'VV', 'VV', 'AS'], ['VV', 'P', 'PN', 'AD', 'JJ', 'NN', 'NN', 'NN', 'VV', 'NN'], ['PN', 'P', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['JJ', 'NN', 'AD', 'VV', 'NN'], ['DT', 'NN', 'PN', 'AD', 'PN', 'VV', 'AD', 'VA'], ['PN', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VV', 'SP'], ['VV', 'VV', 'CD', 'M', 'M'], ['NT', 'NR', 'AD', 'VV', 'CD', 'CD', 'AD'], ['NN', 'NR', 'VV', 'NR', 'VV', 'VV', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV'], ['NR', 'SB', 'VV', 'VV', 'DER', 'VV', 'NN'], ['NN', 'VV', 'AD', 'P', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'P', 'NN', 'LC', 'VV', 'VV'], ['NN', 'AD', 'VV', 'NR', 'AD', 'VV', 'SB', 'VV', 'NN', 'VV', 'VV', 'AS', 'SP'], ['NR', 'DT', 'SB', 'VV', 'VV', 'NR', 'NR'], ['DT', 'NN', 'NN', 'AD', 'P', 'NN', 'MSP', 'VV', 'SP'], ['VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'AD', 'VE', 'NR', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV'], ['P', 'NN', 'LC', 'VV', 'CD', 'JJ', 'M', 'AD', 'AD', 'NN', 'NN'], ['VV', 'NT', 'NN', 'VV', 'NN', 'NN'], ['VA', 'DEC', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NT', 'NN', 'NN'], ['NR', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'P', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN'], ['PN', 'P', 'NR', 'VV', 'NN', 'NR', 'CC', 'NR', 'AD', 'VV', 'NN'], ['PN', 'VV', 'P', 'PN', 'NN', 'VV', 'NN', 'SP'], ['NR', 'AD', 'CC', 'VV', 'CC', 'VV'], ['NN', 'NN', 'VC', 'NR', 'NN', 'CC', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'AS', 'VV', 'VV', 'NR', 'AD', 'VV'], ['JJ', 'NN', 'AD', 'VV'], ['NR', 'P', 'VV', 'VV', 'SB', 'AD', 'VV', 'P', 'NN', 'AD', 'VV'], ['NR', 'NN', 'VV', 'AS', 'NN'], ['NN', 'AD', 'NN', 'SP'], ['AD', 'VV', 'P', 'NN', 'NN'], ['AD', 'VV', 'NT', 'VV', 'LC', 'VV', 'NN'], ['NR', 'NR', 'NR', 'VV', 'NN', 'NR', 'NN', 'VV'], ['NN', 'VV', 'LC', 'NR', 'NN', 'VV', 'AS', 'NN'], ['NR', 'VV', 'NN', 'NR', 'P', 'VV', 'VV', 'AD', 'AD', 'VV', 'VV'], ['NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'DEC', 'NT', 'AD', 'VV', 'PN', 'SP'], ['VV', 'NN', 'NN', 'NN', 'VV'], ['NR', 'AD', 'AD', 'NR', 'VV', 'P', 'NN'], ['P', 'NN', 'LC', 'DT', 'M', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['JJ', 'NR', 'AD', 'P', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'NR', 'VV', 'NN', 'NN'], ['NR', 'VV', 'PN', 'NN'], ['VV', 'VV', 'NN', 'VV', 'VV', 'NR', 'NN', 'LC', 'NR', 'AD', 'AD', 'VV'], ['VE', 'NN', 'NN', 'JJ', 'NN', 'NN'], ['JJ', 'NR', 'VV', 'AS', 'NN', 'NN'], ['AD', 'CC', 'NN', 'NN', 'CC', 'NN', 'SP'], ['NR', 'VV', 'NN', 'VV', 'AD', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NN', 'VE', 'NR', 'DEG', 'NN', 'NN'], ['P', 'NN', 'VV', 'DEC', 'NN', 'VV', 'NN', 'NN', 'VA', 'NN', 'VA'], ['VV', 'NN', 'P', 'NN', 'VV', 'NN'], ['AD', 'NR', 'AD', 'VV', 'NR', 'VV'], ['AD', 'NN', 'VV', 'NN', 'NN', 'AD', 'NN', 'VV', 'PN', 'VV', 'VV', 'SP'], ['NN', 'CC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['P', 'NN', 'VV', 'VA', 'SP'], ['NR', 'CC', 'NR', 'NN', 'NN', 'VA'], ['AD', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'VV', 'NN', 'DEC', 'VV', 'NN', 'ETC'], ['NN', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'LC', 'VV', 'NN'], ['NR', 'NR', 'AD', 'P', 'NR', 'NR', 'NN', 'VV', 'NN', 'LC', 'VV', 'NN', 'NN', 'NN'], ['PN', 'VE', 'PN', 'NN', 'VA', 'SP'], ['NN', 'VV', 'LC', 'NN', 'AD', 'VV', 'NN', 'DEC', 'VA', 'NN'], ['P', 'VV', 'PN', 'P', 'NN', 'BA', 'NN', 'VV'], ['VV', 'SP', 'AD', 'VV', 'AD', 'VA', 'SP'], ['NR', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'NR', 'NN', 'NR', 'VV'], ['NN', 'P', 'PN', 'NN', 'AD', 'VV', 'AD'], ['AD', 'AD', 'AD', 'VV', 'AS', 'P', 'NN', 'VV', 'VV'], ['PN', 'AD', 'VV', 'VV', 'PN', 'SP'], ['NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NR', 'VV', 'P', 'NR', 'VV'], ['AD', 'VV', 'NN', 'AD', 'VE', 'VV', 'VV', 'DEC', 'NN', 'CC', 'NN'], ['NR', 'VV', 'VV', 'AD', 'AD', 'VE', 'NN', 'NN', 'VV'], ['AD', 'VE', 'NN', 'AD', 'VV', 'NN', 'VV'], ['VV', 'NR', 'NN', 'NN', 'CC', 'NR', 'NN', 'NN', 'NN', 'NN'], ['VV', 'VV', 'NN', 'VV', 'NN'], ['VV', 'VV', 'MSP', 'VV', 'NN', 'NN'], ['AD', 'NN', 'AD', 'VV', 'NN'], ['NT', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'SB', 'VV', 'VV', 'P', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'AD', 'VV', 'AD', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'P', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VA'], ['AD', 'NT', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'P', 'NN', 'VV', 'VV'], ['NN', 'NR', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'LC', 'VV'], ['AD', 'VV', 'JJ', 'NN', 'NN'], ['NN', 'NN', 'JJ', 'NN', 'AD', 'NN', 'VV'], ['NR', 'NR', 'AD', 'AD', 'VA'], ['NR', 'VV', 'NN', 'NN', 'VV', 'P', 'NN', 'OD', 'M', 'VV', 'VA'], ['NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV'], ['NR', 'AD', 'AD', 'AD', 'VV', 'AS', 'NN', 'CD', 'VV', 'VV', 'NN', 'AD', 'P', 'VA', 'VV'], ['NT', 'NN', 'NR', 'VV', 'NN'], ['NN', 'NN', 'SB', 'NN', 'VV', 'VV', 'NN', 'VV'], ['NR', 'NN', 'VV'], ['PN', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'VA', 'LC', 'NN', 'AD', 'VV', 'CD', 'NN', 'VV', 'NN'], ['NT', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'AD', 'AD', 'VC', 'NN', 'NN', 'NN', 'NN', 'VV', 'NR', 'NR', 'NN', 'NN', 'VV'], ['NN', 'CC', 'NN', 'NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'VC', 'NN', 'DEG', 'NN'], ['AD', 'DT', 'M', 'NN', 'NT', 'P', 'JJ', 'NN', 'P', 'NN', 'VV', 'VV', 'NN', 'MSP', 'VV', 'VV'], ['VV', 'NN', 'NN', 'NN'], ['VV', 'NN', 'VV', 'VV', 'PN'], ['VE', 'NN', 'VV', 'JJ', 'NN', 'VV', 'VV', 'AS', 'NN'], ['NN', 'AD', 'P', 'PN', 'VE', 'NN', 'SP'], ['NN', 'NN', 'NN', 'NN', 'NN', 'ETC', 'NN', 'NN', 'NN', 'VV'], ['NN', 'P', 'NN', 'BA', 'NN', 'NN', 'VV', 'NN'], ['VA', 'DEC', 'NN', 'NN', 'PN', 'AD', 'AD', 'VE', 'NN'], ['VV', 'NR', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NR', 'NR', 'VV', 'NR', 'NN'], ['NR', 'VV', 'AS', 'PN', 'DEG', 'NN', 'NN', 'VV'], ['VV', 'NN', 'P', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN'], ['AD', 'AD', 'VV', 'NN', 'VA', 'DEC', 'NN', 'NN'], ['PN', 'NT', 'AD', 'VA', 'AD', 'VA'], ['VV', 'NN', 'NR', 'VV', 'NN', 'DEC', 'NN'], ['P', 'NN', 'NN', 'LC', 'NR', 'AD', 'AD', 'VV'], ['DT', 'DT', 'NN', 'LC', 'NN', 'VV', 'PN', 'AD', 'VV', 'NR'], ['P', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV', 'SP'], ['NR', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NN', 'AD', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['AD', 'VV', 'JJ', 'NN', 'SP'], ['CD', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'LC', 'VV', 'NR', 'NR', 'DEG', 'NR'], ['AD', 'NR', 'NR', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VV'], ['VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'NR'], ['BA', 'NR', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VC', 'NN', 'SP'], ['NR', 'CC', 'NR', 'NR'], ['AD', 'P', 'NT', 'VV', 'NN', 'SP'], ['NR', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'DEC', 'NN'], ['NR', 'NT', 'NT', 'NN', 'NN'], ['AD', 'VV', 'VV', 'NN', 'NN'], ['AD', 'AD', 'AD', 'NN', 'DT', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'AD', 'VV', 'NN', 'AD', 'NN', 'NN', 'CC', 'NN', 'DEC', 'NN', 'VV', 'VE', 'SP'], ['NN', 'VV', 'DEC', 'AD', 'AD', 'VE', 'NN'], ['NN', 'VV', 'NN', 'AD', 'JJ', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['DT', 'NN', 'P', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'NN', 'NN', 'VA', 'SP'], ['NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['AD', 'NR', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VA'], ['AD', 'P', 'AD', 'VV', 'VV', 'AD', 'VV'], ['NN', 'NN', 'NN', 'AD', 'VE', 'NN', 'NN', 'NN'], ['NN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['NR', 'AD', 'VV', 'VV', 'NR', 'NN', 'VA', 'VV', 'VV'], ['BA', 'NN', 'NN', 'AD', 'VV', 'VV', 'IJ', 'NN'], ['DT', 'NN', 'VV', 'P', 'NN', 'VV'], ['PN', 'VV', 'VV', 'NR', 'VV', 'NN', 'NN'], ['P', 'NN', 'NN', 'NN', 'NR', 'VV', 'LC', 'NN', 'VV', 'VV', 'NN'], ['NN', 'LC', 'CC', 'VV', 'DEC', 'NN'], ['AD', 'NR', 'DT', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'VV', 'AD'], ['JJ', 'NN', 'NR', 'AD', 'P', 'NR', 'NN', 'VV', 'VV', 'NN'], ['CD', 'NN', 'JJ', 'NN', 'NN', 'AD', 'VA'], ['NN', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'PN', 'AD', 'AD', 'AD', 'VV', 'DT', 'M', 'SP'], ['NR', 'NN', 'NR', 'NR', 'NR', 'ETC', 'AD', 'VV'], ['VE', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN'], ['NT', 'PN', 'VV', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'VV', 'SP'], ['AD', 'VV', 'AS', 'NN', 'VV', 'VV', 'PN', 'VV', 'SP'], ['VV', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NN', 'P', 'P', 'DT', 'VV', 'VV', 'VV', 'NN', 'P', 'PN', 'NN'], ['NR', 'CC', 'NN', 'NN', 'VV', 'AD'], ['PN', 'VV', 'VV', 'NR', 'VV', 'PN', 'NN'], ['AD', 'NR', 'AD', 'VV', 'VV', 'DER', 'VV', 'NN', 'SP'], ['PN', 'AD', 'PN', 'VV', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'VV'], ['OD', 'VC', 'NN', 'VV', 'NR', 'NR', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VE', 'NN', 'SP'], ['AD', 'NR', 'VV', 'NR', 'NN', 'AD', 'VV', 'NN', 'NN', 'ETC'], ['PN', 'AD', 'AD', 'VV', 'NN', 'CC', 'NN', 'NN'], ['VV', 'OD', 'AD', 'VV', 'AS'], ['VV', 'NN', 'VV'], ['P', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'PN', 'AD', 'AD', 'P', 'NN', 'NN', 'NN', 'VV'], ['NN', 'CC', 'NN', 'NN', 'NN', 'AD', 'VC', 'DT', 'NN', 'NN', 'NN', 'DEC', 'NN'], ['CD', 'NN', 'VV', 'NN', 'CC', 'VE', 'NN', 'VA', 'DEC', 'NN', 'CC', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'VV', 'AD', 'NN', 'AD', 'VV'], ['P', 'PN', 'VV', 'CC', 'NN', 'LC', 'NR', 'NN', 'VV'], ['JJ', 'NN', 'VV', 'PN', 'AD', 'P', 'PN', 'NN'], ['AD', 'P', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NR', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VA'], ['AD', 'NR', 'DEG', 'NN', 'VV', 'VA', 'CC', 'VA', 'DEC', 'NN', 'AD', 'VA'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['NN', 'VV', 'NN', 'VV'], ['PN', 'P', 'NR', 'NN', 'NN', 'VA', 'VV'], ['NN', 'NN', 'NN', 'VV', 'VE', 'VV'], ['DT', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN'], ['NR', 'NR', 'VV', 'PN', 'AD', 'P', 'NR', 'NN', 'VV', 'NN'], ['AD', 'VV', 'PN', 'AD', 'VV', 'PN', 'VV', 'SP'], ['P', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'VV', 'P', 'NN', 'VV'], ['PN', 'VV', 'AS', 'PN', 'NN', 'CC', 'PN', 'NN', 'VV', 'VV', 'SP'], ['NR', 'VV', 'NR', 'NN'], ['NT', 'NN', 'VV', 'AD', 'DEC', 'NR', 'AD', 'VV', 'AD', 'VV'], ['AD', 'VV', 'AS', 'P', 'NN', 'NR', 'AD', 'NR', 'AD', 'VV', 'P', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'VE', 'DEG', 'NN', 'VV', 'VE', 'DEG', 'NN', 'VV', 'VE', 'PN'], ['CS', 'NR', 'VV', 'NN', 'NR', 'VE', 'JJ', 'NN', 'VV', 'NN'], ['NN', 'P', 'PN', 'VV', 'SP'], ['NR', 'VV', 'VV', 'NR', 'AD', 'SB', 'VV'], ['JJ', 'JJ', 'NN', 'JJ', 'NN', 'NN', 'VV', 'VV'], ['DT', 'PN', 'AD', 'P', 'AD', 'VV', 'AS', 'NN'], ['VV', 'NT', 'P', 'NN', 'VV'], ['NR', 'DEG', 'NN', 'CC', 'NR', 'VV', 'VV'], ['NR', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NN', 'VA', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['VV', 'NR', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['NR', 'VV', 'AD', 'VV', 'VV', 'NN'], ['VV', 'DEC', 'NN', 'ETC', 'NN', 'AD', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'P', 'NN', 'VA', 'LC', 'VV'], ['P', 'NN', 'VV', 'SP'], ['AD', 'JJ', 'NN', 'VV', 'NN', 'DEC', 'NR', 'AD', 'VV', 'NN'], ['VV', 'NN', 'PN', 'VV', 'NN', 'VC', 'NN', 'SP', 'NR'], ['NN', 'NN', 'VV', 'NR', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VV', 'NN'], ['P', 'NN', 'VV', 'NN', 'LC', 'NN', 'VV', 'P', 'NN', 'AD', 'VV'], ['NN', 'AD', 'P', 'NN', 'VV', 'NN', 'VV'], ['AD', 'AD', 'VV', 'VV', 'SP'], ['VV', 'LC', 'NN', 'VA', 'VV', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NR', 'VV', 'AD', 'AD', 'VV'], ['NN', 'AD', 'NN', 'AD', 'JJ', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'AD', 'VA'], ['AD', 'PN', 'AD', 'VV', 'VV', 'NN', 'NN'], ['NR', 'P', 'VV', 'JJ', 'NN', 'VV', 'VV'], ['NR', 'NN', 'NN', 'NR', 'NN', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'NR'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'P', 'NN', 'VA', 'DEC', 'NN', 'VA', 'AD', 'SP'], ['VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VV', 'NN', 'VV', 'VV'], ['AD', 'PN', 'JJ', 'NN', 'AD', 'AD', 'VV', 'AS'], ['NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['NR', 'NN', 'DEG', 'NN', 'NN', 'SB', 'VV'], ['VV', 'NN', 'NN', 'NN'], ['NR', 'NN', 'NR', 'NN', 'NN', 'NN', 'VV'], ['NR', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'JJ', 'NN', 'NR', 'NR', 'NR', 'ETC', 'AD', 'AD', 'VV', 'NN', 'NN', 'LC'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV'], ['NR', 'VV', 'LC', 'VV', 'NN', 'VV', 'PN', 'VV', 'VV', 'NN', 'AD', 'AD', 'VA'], ['VV', 'NN', 'NR', 'VV', 'NR', 'OD', 'NN', 'NN', 'NN'], ['OD', 'NN', 'NN', 'NN'], ['VV', 'NN', 'SB', 'VV', 'VV', 'NN'], ['NN', 'SB', 'NN', 'NN', 'LC', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['VV', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VV', 'PN', 'VV', 'VA', 'SP'], ['NN', 'VE', 'NN', 'NN', 'VE', 'NN'], ['PN', 'P', 'NR', 'NN', 'DEG', 'NN', 'VV'], ['PN', 'VV', 'VV', 'NN', 'SP'], ['VV', 'NR', 'AD', 'VV', 'PN', 'AD', 'SP'], ['AD', 'VV', 'CD', 'NN', 'CC', 'JJ', 'NN', 'ETC', 'VV', 'AS', 'VV', 'VV', 'VV', 'NN', 'JJ', 'NN'], ['NR', 'NN', 'AD', 'VC', 'NR', 'NR', 'NN', 'NN', 'DEC', 'NN', 'NN'], ['NR', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VV', 'VV', 'NN', 'DEC', 'NR', 'VV', 'P', 'NN', 'AD', 'NN'], ['AD', 'AD', 'P', 'NN', 'NN', 'NN', 'AD', 'NN', 'VA', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'NN', 'VV', 'VV', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'NN', 'DEC', 'SB', 'VV', 'NN', 'VC', 'NT', 'CD'], ['NR', 'AD', 'DEV', 'VV', 'NN', 'NR', 'VV', 'P', 'NN', 'LC', 'VV', 'VV'], ['CS', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NN', 'PN', 'VA', 'SP', 'AD', 'SP'], ['NR', 'AD', 'BA', 'PN', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NR', 'NN'], ['NR', 'VV', 'AD', 'VA'], ['NN', 'P', 'DT', 'NN', 'CC', 'NN', 'VV', 'NN'], ['AD', 'P', 'NR', 'NN', 'LC', 'VV', 'NR', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VA', 'NN', 'AD', 'VV'], ['NN', 'LC', 'VA', 'NR'], ['NR', 'AD', 'VV', 'NR', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VE', 'NN'], ['DT', 'NN', 'P', 'VV', 'NN', 'JJ', 'NN'], ['P', 'JJ', 'NN', 'NR', 'CC', 'NN', 'AD', 'P', 'VV', 'AD', 'NN'], ['NR', 'VV', 'NR', 'NR', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'AS', 'NN', 'AD', 'SB', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'P', 'VV', 'AD', 'VA', 'AD', 'VA'], ['NR', 'NN', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV'], ['NN', 'LC', 'PN', 'VV', 'VV', 'NN', 'CC', 'NN', 'NN'], ['NR', 'VV', 'NR', 'NR', 'NN'], ['PN', 'AD', 'VE', 'VV', 'CD', 'NN', 'AD', 'NR', 'AD', 'VV', 'PN'], ['AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'AD', 'AD', 'VV', 'AD', 'LB', 'NN', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NT', 'NT', 'NR', 'AD', 'P', 'NR', 'VV', 'VV', 'VV'], ['NR', 'NR', 'AD', 'P', 'NN', 'VV', 'AS', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'NN'], ['P', 'NN', 'NN', 'LC', 'NN', 'AD', 'VA', 'VV', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'LC', 'NN', 'VV', 'CC', 'NN', 'NN', 'ETC', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NT', 'VV', 'NR', 'VV', 'P', 'NR', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NT', 'NN', 'VV', 'VV', 'NN', 'AD', 'VA'], ['NN', 'VV', 'NR', 'NN', 'VV'], ['P', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'AD', 'P', 'NN', 'VV'], ['VV', 'PN', 'VV', 'NN', 'SP'], ['P', 'NR', 'VV', 'NN', 'NN'], ['PN', 'VV', 'VV', 'PN', 'DEG', 'DT', 'CD', 'NN'], ['NN', 'BA', 'NR', 'VV', 'AS', 'NR', 'NN'], ['NR', 'VV', 'PN', 'P', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'AD', 'NR', 'NN'], ['NR', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV', 'NR'], ['NR', 'NN', 'P', 'NR', 'NR'], ['AD', 'VV', 'PN', 'VV', 'VV', 'AS', 'PN', 'VV', 'VV', 'VV', 'AD'], ['NN', 'AD', 'AD', 'AD', 'VV'], ['AD', 'NR', 'VV', 'DEC', 'CD', 'NN'], ['AD', 'JJ', 'PN', 'AD', 'VV', 'PN', 'SP'], ['NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NR', 'VV', 'VV', 'NN'], ['AD', 'P', 'NN', 'LC', 'NN', 'AD', 'VV', 'M', 'NN', 'NN', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'AS', 'BA', 'NN', 'AD', 'VV'], ['AD', 'AD', 'AD', 'VA', 'AS', 'AD'], ['PN', 'AD', 'AD', 'VV', 'CD', 'NN'], ['NN', 'AD', 'VA'], ['AD', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'AD', 'AD', 'VV', 'AS', 'NN'], ['NN', 'VV', 'DER', 'AD', 'VA', 'VV'], ['AD', 'VV', 'LC', 'AD', 'VV', 'NN'], ['NN', 'NR', 'NN', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN', 'VV'], ['PN', 'VV', 'P', 'VA', 'DEC', 'NN', 'LC', 'VV', 'VV', 'NN', 'P', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'VV'], ['AD', 'VV', 'VA', 'DEC', 'NN', 'AD', 'VA'], ['PN', 'VV', 'NN', 'NN', 'AD', 'VV'], ['AD', 'CC', 'NN', 'CD', 'M', 'CD', 'CD', 'CD', 'CD'], ['PN', 'AD', 'P', 'NR', 'VV', 'CD'], ['NR', 'VV', 'NN', 'LC', 'AD', 'AD', 'VA', 'VA'], ['NR', 'AD', 'VE', 'AD', 'JJ', 'DEG', 'NN', 'NN', 'CC', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['NR', 'VV', 'NR', 'CC', 'NN', 'VV', 'NR'], ['VV', 'VV', 'LC', 'VE', 'AD', 'SB', 'VV', 'DEC', 'VA'], ['NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NR', 'VV', 'NN', 'NN', 'VV', 'AS', 'VV'], ['AD', 'NR', 'VA', 'VV'], ['NR', 'VV', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'NR', 'AD', 'VV', 'VV'], ['AD', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'AD', 'P', 'PN', 'VV', 'VV', 'NR', 'LC', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV'], ['AD', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VV'], ['P', 'NN', 'VA', 'MSP', 'SB', 'VV', 'DEC', 'NN', 'VV', 'NN', 'NN'], ['VV', 'M', 'NN', 'AD', 'NN'], ['NR', 'NN', 'VV', 'AD', 'VV'], ['NR', 'VV', 'VV', 'CD', 'NN'], ['PN', 'BA', 'PN', 'VV', 'VA', 'SP'], ['PN', 'VV', 'VV', 'PN', 'AD', 'VV', 'PN'], ['AD', 'P', 'CD', 'M', 'NN', 'AD'], ['VV', 'NN', 'SB', 'VV', 'NN'], ['NN', 'VV', 'LC', 'VA', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV'], ['AD', 'P', 'NN', 'VA', 'NN', 'VV', 'NN', 'NN', 'VV'], ['VV', 'NN', 'DEC', 'NR', 'NN', 'VV'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'VA'], ['NN', 'VV', 'VV', 'NN', 'NT', 'NN', 'NN'], ['VV', 'NR', 'NR', 'DEG', 'AD', 'PN', 'NN', 'VV', 'VV'], ['AD', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'VC', 'NR', 'NR', 'DEG'], ['PN', 'VV', 'AS', 'VV', 'P', 'NN', 'DT', 'NN', 'VV'], ['PN', 'VV', 'AD', 'NN', 'AD', 'VV', 'DT', 'AD', 'VV'], ['P', 'DT', 'NN', 'DEG', 'NN', 'NN', 'LC', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NN', 'LC', 'AD', 'VE', 'NT', 'VV', 'NR', 'NN', 'DEG', 'PN', 'NN'], ['AD', 'VV', 'P', 'NN', 'VC', 'P', 'AD', 'AD', 'VV', 'AD', 'VA'], ['PN', 'AD', 'VE', 'CD', 'VA', 'DEC', 'NN'], ['VV', 'NN', 'VV', 'NN', 'ETC', 'VA'], ['CD', 'NN', 'LC', 'DEG', 'NR', 'AD', 'VV', 'NN'], ['P', 'NR', 'AD', 'AD', 'AD', 'NT'], ['JJ', 'M', 'NN', 'NN', 'NN'], ['VE', 'VV', 'NN', 'VV', 'AD'], ['VV', 'NR', 'NR', 'NN', 'CD', 'CD', 'CD', 'NN', 'CD', 'NN', 'NN', 'NN'], ['NR', 'OD', 'M', 'AD', 'VV'], ['NN', 'VV', 'NN', 'VV'], ['AD', 'NR', 'NN', 'AD', 'VE', 'NN', 'VV'], ['JJ', 'NN', 'MSP', 'VV', 'NN', 'VV'], ['CD', 'CD', 'CD', 'M', 'NN', 'VV', 'NR', 'VV', 'NN', 'NN', 'NN'], ['P', 'NN', 'VV', 'NN', 'VV', 'NN'], ['AD', 'VV', 'JJ', 'NN', 'VV'], ['NT', 'DEG', 'PN', 'AD', 'VC', 'NN', 'VA', 'SP', 'AD', 'NN', 'VV', 'SP'], ['JJ', 'NN', 'VV', 'CD', 'CD', 'NN', 'AD', 'VA', 'SP'], ['P', 'NN', 'NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['PN', 'VV', 'PN', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'AS', 'VV', 'AD', 'VV'], ['NN', 'NN', 'PN', 'AD', 'VV', 'SP'], ['NR', 'VV', 'NN', 'VV', 'NR', 'NN'], ['PN', 'VV', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN'], ['AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['P', 'VV', 'NR', 'VV', 'NN', 'NN', 'P', 'PN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['AD', 'VV', 'AD', 'VA', 'MSP', 'PN', 'AD', 'VA'], ['NR', 'DEG', 'NN', 'P', 'VV', 'NN', 'MSP', 'VV'], ['NR', 'CD', 'NN', 'AD', 'VV', 'NN', 'VV', 'NR', 'NN', 'NN', 'ETC', 'NN', 'VV'], ['NR', 'NN', 'VV', 'AD', 'VV', 'NR', 'DEG', 'NN', 'PN', 'NN'], ['NR', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VC', 'NR', 'NN', 'DEG'], ['NT', 'NT', 'AD', 'NN', 'NN'], ['VV', 'SP', 'BA', 'PN', 'VV', 'VV', 'SP'], ['NN', 'SP', 'PN', 'NT', 'AD', 'VV', 'PN', 'PN', 'P', 'PN', 'VV'], ['NR', 'NR', 'AD', 'VV', 'VV', 'VV', 'M'], ['NN', 'VC', 'VV', 'AS', 'VV', 'DEC', 'AD', 'VV', 'VV', 'SP'], ['NR', 'DEG', 'NR', 'CC', 'NR', 'NR', 'VV', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'LC', 'VV', 'VE', 'NN'], ['NN', 'P', 'NN', 'LC', 'VV', 'NN', 'NN', 'ETC', 'NN'], ['P', 'JJ', 'DEG', 'NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'VV', 'NR', 'NR', 'NN'], ['PN', 'AD', 'AD', 'PN', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['NN', 'VV', 'NN', 'LC', 'P', 'NN', 'VV', 'VE', 'NN', 'ETC'], ['AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'AS'], ['P', 'NN', 'VV', 'PN', 'VE', 'NN', 'SP'], ['NN', 'NN', 'VV', 'VV', 'VV', 'M', 'SP'], ['P', 'NN', 'P', 'NN', 'NN', 'NN', 'VV', 'AD', 'P', 'VV', 'JJ', 'NN', 'VV'], ['JJ', 'NN', 'VV', 'VV'], ['NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'AS', 'CD', 'NN'], ['DT', 'M', 'NN', 'NN', 'AD', 'SB', 'VV', 'NN'], ['NR', 'NR', 'P', 'NN', 'LC', 'AD', 'VV', 'VV', 'JJ', 'NN', 'NN', 'VV'], ['AD', 'NT', 'DT', 'M', 'NN', 'AD', 'VV', 'AD', 'VA'], ['NN', 'LC', 'VV', 'VE', 'NN'], ['PN', 'P', 'NN', 'VV', 'NN'], ['PN', 'M', 'VV', 'AD', 'VA', 'SP'], ['NR', 'AD', 'P', 'PN', 'VV', 'NN', 'P', 'VV', 'NN', 'NN', 'VV'], ['NN', 'NN', 'NT', 'NN', 'CC', 'NR', 'NN', 'VV', 'AD'], ['PN', 'P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['VV', 'AD', 'NN', 'AD', 'VV', 'NN', 'SP'], ['VV', 'P', 'NR', 'DEG', 'NN', 'NN', 'P', 'PN', 'VV', 'VV'], ['NN', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VV', 'JJ', 'AD', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['NT', 'NN', 'NN', 'VV', 'AD', 'VV', 'VV', 'SP'], ['NR', 'AD', 'VA', 'VC', 'NR', 'NN', 'JJ', 'NN', 'NN', 'SP'], ['VV', 'NN', 'NR', 'AD', 'VV', 'NN', 'AD', 'NR', 'AD', 'VV', 'DER', 'VA'], ['NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VV', 'VV', 'VC', 'VV'], ['NN', 'BA', 'VV', 'NN', 'AD', 'VV', 'VV', 'AS', 'PN', 'NN'], ['VV', 'NN', 'VV', 'NN', 'NN', 'NN'], ['AD', 'NN', 'VV', 'AS', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN'], ['JJ', 'NR', 'AD', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'VV'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NR', 'VV', 'P', 'NN'], ['AD', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VA'], ['NR', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'NR', 'NN', 'AD', 'VV'], ['AD', 'NN', 'P', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'P', 'NN', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN'], ['VV', 'VV', 'LC', 'VV', 'NN'], ['P', 'NR', 'AD', 'JJ', 'DEG', 'CC', 'NR', 'NN', 'NN', 'NR'], ['VV', 'VV', 'VV', 'VV', 'PN', 'DEG', 'NN', 'NN'], ['VV', 'P', 'NN', 'NN', 'NN', 'NN'], ['P', 'NN', 'VV', 'DEC', 'NR', 'NN'], ['P', 'P', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NR', 'AD', 'VV', 'AS', 'VA', 'DEC', 'NN', 'NN'], ['PN', 'VV', 'NN', 'NN', 'VV', 'AD', 'VA'], ['NR', 'NR', 'NN', 'AD', 'VC', 'NN', 'VV'], ['CD', 'VV', 'CD', 'VV', 'CD'], ['NR', 'NN', 'NN', 'NR', 'VV'], ['NR', 'VV', 'DEC', 'CD', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN'], ['VE', 'NN', 'VV', 'VV', 'VV', 'NN'], ['VV', 'VV', 'NN', 'SP'], ['NR', 'VV', 'VV', 'VV'], ['NR', 'NN', 'AD', 'VC', 'NN', 'DEG', 'NN'], ['AD', 'CD', 'NN', 'VV', 'VV', 'NN', 'VV'], ['NR', 'AD', 'VV', 'DT', 'NN', 'VV', 'AS', 'NN', 'NN', 'CC', 'VV'], ['NN', 'AD', 'VV', 'VV', 'VV', 'JJ', 'SP'], ['CD', 'NN', 'NN', 'JJ', 'NN', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'VE', 'SB', 'VV', 'NN'], ['VE', 'NN', 'VV', 'NN', 'VV', 'PN', 'VV', 'NR', 'AD', 'VV', 'SP'], ['VV', 'CC', 'AD', 'VV', 'AD', 'VV', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV'], ['AD', 'NR', 'VV', 'NN', 'AD', 'BA', 'NR', 'VV', 'NN'], ['AD', 'AD', 'VV', 'NN', 'VV', 'VV'], ['AD', 'VE', 'NN', 'NN', 'NN', 'NN', 'NN', 'ETC'], ['JJ', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'PN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'NR', 'NN', 'NN', 'NR', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['AD', 'NN', 'AD', 'VE', 'AS', 'AD', 'CD', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VE', 'AD', 'VA'], ['NN', 'AD', 'VV', 'NN', 'AD', 'NR', 'NN', 'DEG', 'NN'], ['VE', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'NN'], ['NR', 'AD', 'NN', 'NN', 'VV', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'AS', 'NN', 'VE', 'DT', 'NN', 'SP'], ['NN', 'NN', 'VV', 'LC', 'AD', 'VV'], ['PN', 'DEG', 'NN', 'VV', 'NN'], ['PN', 'NN', 'VV', 'AS', 'AD', 'AD', 'VA', 'VV', 'AS'], ['P', 'NR', 'NN', 'NN', 'NN', 'NN', 'AD', 'VA'], ['AD', 'VV', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['VA', 'VV', 'VV', 'AD', 'PN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'PN', 'VV', 'NN'], ['NN', 'VV', 'P', 'P', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VA', 'LC', 'VE', 'VV', 'NN', 'DEC', 'NR', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NR', 'VV', 'NN', 'AD', 'NN'], ['NN', 'NN', 'NN', 'AD', 'VV'], ['NR', 'NT', 'P', 'SB', 'VV', 'P', 'NN', 'VV', 'ETC', 'BA', 'VV', 'VV'], ['NR', 'CC', 'NN', 'VV'], ['NR', 'P', 'PN', 'NN', 'VA'], ['NN', 'VV', 'AS', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['NN', 'LC', 'VE', 'NN', 'CC', 'NN'], ['NR', 'NN', 'VC', 'JJ', 'NN', 'NN'], ['NT', 'NN', 'AD', 'P', 'NR', 'VV', 'AS', 'NN', 'VV', 'NN'], ['VV', 'DEC', 'NN', 'NN'], ['AD', 'VV', 'SP', 'VV', 'NN', 'VA', 'VA'], ['JJ', 'NN', 'NN', 'CD', 'NN', 'CD', 'CD', 'CD', 'CD', 'VV', 'VV'], ['P', 'NN', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'P', 'NN', 'NN', 'ETC', 'NN', 'VV'], ['NN', 'NN', 'VV', 'VV', 'VV'], ['PN', 'NN', 'VV', 'CD', 'SP'], ['NN', 'NN', 'NN', 'NN', 'VC', 'P', 'VV', 'DEC'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VA', 'CC', 'P', 'NN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'NN', 'NN', 'IJ', 'NN', 'AD', 'VV', 'SP'], ['NN', 'P', 'PN', 'VV', 'NN', 'NN', 'VA'], ['NR', 'JJ', 'NN', 'NR', 'NR', 'NN', 'NT', 'VV', 'NN', 'VV', 'VV'], ['NN', 'NN', 'AD', 'NN', 'VV'], ['NR', 'NR', 'NR', 'NN', 'VV'], ['NN', 'AD', 'VV', 'JJ', 'NN', 'JJ', 'NN', 'CC', 'NN', 'P', 'VV', 'VV', 'VV'], ['AD', 'VV', 'VV', 'VC', 'M', 'NN'], ['PN', 'NT', 'AD', 'VC', 'AD', 'AD', 'SP'], ['VV', 'LC', 'NR', 'AD', 'VV', 'VV', 'AS'], ['AD', 'VV', 'AD', 'VV', 'P', 'NN', 'AD', 'VV', 'NN', 'LC'], ['PN', 'P', 'NN', 'NR', 'DEG', 'NN', 'NN', 'AD', 'P', 'NT', 'VV'], ['NT', 'NN', 'NR', 'VV', 'NN', 'NN', 'VV', 'NN'], ['VV', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'AS', 'NN', 'NN', 'VV', 'NR', 'JJ', 'VV', 'NN'], ['NR', 'P', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'NN'], ['NR', 'CC', 'NN', 'NN', 'AD', 'VV'], ['NR', 'DEG', 'JJ', 'NN', 'VA', 'VA'], ['P', 'NN', 'ETC', 'NN', 'NR', 'P', 'NR', 'VV', 'NN', 'AD', 'VV'], ['P', 'NN', 'NR', 'VV', 'NR', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['AD', 'P', 'NN', 'VA', 'NN', 'NN'], ['NR', 'VV', 'NN', 'CC', 'NR', 'VV'], ['NR', 'NN', 'AD', 'P', 'NR'], ['CD', 'CD', 'JJ', 'NN', 'AD', 'VV', 'VV'], ['P', 'VV', 'NN', 'SP'], ['P', 'NR', 'VV', 'VV', 'NR', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'PN', 'NN', 'P', 'VV', 'VV', 'DEC', 'AD'], ['AD', 'BA', 'JJ', 'DEG', 'NN', 'VV', 'P', 'NN', 'LC'], ['AD', 'VV', 'NN', 'NN', 'NN', 'CC', 'NT', 'LC', 'JJ', 'NN'], ['PN', 'AD', 'VV', 'SP', 'PN', 'AD', 'VE', 'NN', 'SP'], ['NR', 'VV', 'AD', 'NN', 'P', 'AD', 'VV', 'VV', 'NN'], ['AD', 'NN', 'NN', 'VA', 'DER', 'AD', 'P', 'NN', 'NN', 'CC', 'NN', 'VV', 'VV'], ['NR', 'NR', 'VE', 'NN', 'VE', 'NN', 'JJ', 'NN', 'NN'], ['VV', 'PN', 'DEG', 'NN', 'AD', 'VV'], ['NR', 'NR', 'NN', 'NN', 'VV', 'VE', 'DT', 'AD', 'VV', 'SP'], ['VV', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['NR', 'VV', 'AD', 'VV', 'SP'], ['M', 'NN', 'VV', 'NR', 'NR', 'JJ', 'NR', 'NR', 'VC', 'NN', 'NN'], ['NT', 'JJ', 'NN', 'VV', 'AS', 'NR', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN'], ['VV', 'NR', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN'], ['PN', 'VV', 'AS', 'NN', 'VV', 'AD', 'VV', 'VV', 'AD', 'SP'], ['NN', 'CC', 'NN', 'P', 'NN', 'VV'], ['AD', 'NN', 'VV', 'AD', 'JJ', 'NN'], ['AD', 'AD', 'P', 'NR', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'NR', 'NN'], ['NR', 'VV', 'NN', 'NN', 'VV'], ['AD', 'NR', 'JJ', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'NR', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VA'], ['AD', 'VV', 'VV', 'CD', 'NN', 'NN', 'SB', 'VV'], ['NN', 'NN', 'AD', 'VV', 'MSP', 'VV'], ['NN', 'VV', 'LC', 'PN', 'AD', 'BA', 'PN', 'VV', 'NN', 'NN', 'NN'], ['NN', 'NN', 'JJ', 'NN'], ['NN', 'VV', 'LC', 'AD', 'NN', 'NN'], ['NN', 'NN', 'NR', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['AD', 'NR', 'AD', 'VV', 'NN'], ['NR', 'VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'VC', 'NR', 'NT'], ['NN', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NR', 'NR', 'P', 'NN', 'AD', 'VV', 'AD', 'VA'], ['VA', 'DEC', 'AD', 'NN', 'PN', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN'], ['VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'BA', 'NN', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV', 'DEC'], ['NR', 'VV', 'DEC', 'NR', 'NN', 'VV', 'NN', 'VV', 'P', 'NN', 'NN'], ['VV', 'NT', 'NN', 'AD', 'VV', 'M', 'JJ', 'NN'], ['NN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'AD', 'VC', 'P', 'JJ', 'NN', 'VV'], ['NN', 'VV', 'AD', 'VV', 'DT', 'NN', 'AD', 'VV'], ['VV', 'VV', 'AD', 'VA'], ['VV', 'LC', 'DEG', 'NN', 'NN', 'VV', 'VC', 'NN', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'SP'], ['AD', 'P', 'VV', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN', 'AD', 'DT', 'AD', 'VV', 'NN'], ['VV', 'NR', 'NN', 'VV', 'NN', 'AD', 'VA', 'NN', 'AD', 'VA', 'AD', 'VA', 'AD', 'VV'], ['P', 'PN', 'DT', 'JJ', 'NN', 'LC', 'LC', 'VE', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'SP'], ['NN', 'VV', 'PN', 'VC', 'VV', 'AD', 'VV', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'AD', 'VV'], ['NT', 'VV', 'NN', 'NN', 'VV', 'CD', 'M'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN'], ['VV', 'NN', 'VV', 'NN', 'NN', 'NR', 'P', 'VV', 'VV', 'VV', 'NN', 'NN'], ['NN', 'NN', 'P', 'NN', 'NN', 'LC', 'VV'], ['VV', 'NR', 'SP'], ['NN', 'DEG', 'NN', 'VV', 'CC', 'VV', 'CD'], ['DT', 'JJ', 'NN', 'NN', 'VV', 'VV'], ['NR', 'VV', 'NN', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['VV', 'DEC', 'NN', 'VV', 'LC', 'VV', 'NN', 'NN', 'VV', 'AD', 'AD', 'VV', 'NN'], ['AD', 'NN', 'P', 'NR', 'AD', 'VA'], ['AD', 'PN', 'P', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'VV', 'MSP', 'VV'], ['NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NR', 'NR', 'AD', 'AD', 'P', 'NN', 'NN', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VC', 'VV', 'NR', 'NN', 'VA'], ['JJ', 'NN', 'JJ', 'NN', 'NN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'VV', 'PN', 'NN', 'VE', 'NN'], ['AD', 'VV', 'NN', 'VV', 'VV'], ['NR', 'NT', 'NN', 'VA', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['PN', 'AD', 'JJ', 'NN', 'OD', 'M', 'CD', 'NT', 'NT'], ['NN', 'VV', 'P', 'NN', 'NR', 'AD', 'VV', 'NN'], ['AD', 'NR', 'NR', 'VV', 'DT', 'NN', 'VV', 'NR', 'NR', 'VC', 'VA', 'DEC', 'NR', 'NN'], ['NN', 'NN', 'NN', 'NN', 'VV', 'VA'], ['VV', 'DT', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'NR', 'NN', 'NN'], ['NN', 'VC', 'M', 'VV', 'DEC', 'NN'], ['AD', 'VA', 'NN', 'NN'], ['VV', 'DT', 'M', 'VA', 'DEC', 'NN', 'NN', 'NR', 'P', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['NR', 'SB', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN', 'LC', 'VV', 'VV', 'VV', 'AD', 'VV'], ['AD', 'NN', 'LC', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV'], ['DT', 'NN', 'VV', 'NR', 'NR', 'NN', 'VV'], ['NN', 'AD', 'VV', 'CD', 'NN', 'VV', 'PN', 'NN', 'AD', 'VV', 'PN', 'VC', 'PN', 'NN'], ['NN', 'BA', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'PN', 'BA', 'PN', 'DEG', 'NN', 'VV', 'NN'], ['AD', 'NR', 'NR', 'ETC', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'NN'], ['NN', 'NN', 'VV', 'PN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD'], ['NR', 'NN', 'VV'], ['NN', 'VV', 'NT', 'NN', 'VV', 'NR'], ['PN', 'NN', 'AD', 'VV', 'DEV', 'SB', 'VV', 'VE', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'DT', 'NN', 'AD', 'NN', 'DEG', 'CD', 'NT', 'CD', 'NN', 'VC', 'CD', 'CD', 'CD'], ['NR', 'CC', 'NR', 'JJ', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'NN'], ['NR', 'NR', 'AD', 'P', 'DT', 'NN', 'VV', 'NN'], ['CD', 'NN', 'NN', 'SB', 'VV', 'NR'], ['AD', 'PN', 'NT', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['PN', 'AD', 'AD', 'VV', 'P', 'NN', 'CC', 'NN', 'LC', 'DEG', 'NN'], ['AD', 'NN', 'AD', 'VC', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VV', 'PN'], ['NR', 'AD', 'NN', 'VV', 'VE', 'P', 'PN', 'VV', 'VV', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'AS', 'VV'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'JJ', 'NN', 'LC'], ['NR', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'AD', 'NN', 'VV', 'AD', 'AD', 'VV', 'VV', 'CD', 'AD', 'VV', 'SP'], ['VV', 'AD', 'JJ', 'NN', 'NN', 'AD', 'VV', 'SP'], ['NR', 'AD', 'NR', 'NN', 'NN', 'PN', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'NR', 'PN', 'AD', 'VV', 'PN', 'IJ'], ['P', 'JJ', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'SP'], ['NR', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'LC', 'NN', 'AD', 'VV', 'AD', 'VA'], ['P', 'PN', 'VV', 'VC', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'SP'], ['NR', 'P', 'NN', 'VV', 'NN'], ['PN', 'P', 'NN', 'NN', 'DEG', 'NN', 'AD', 'PN', 'AD', 'PN', 'AD', 'DEV', 'VV', 'VV'], ['NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'AD', 'AD', 'VA', 'SP', 'SP'], ['PN', 'VV', 'AS', 'AD', 'NN', 'VV', 'NN', 'VV', 'DER', 'VV', 'VV'], ['PN', 'NN', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VV', 'VV', 'NR', 'NN'], ['NR', 'VV', 'VV', 'VV', 'NN'], ['PN', 'VV', 'VV', 'DEC', 'VC', 'NN', 'NN'], ['NR', 'VV', 'AS', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'VV', 'NR', 'NN', 'NR', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VA', 'DER', 'P', 'NN', 'VA', 'VA'], ['P', 'VV', 'NN', 'NN', 'JJ', 'NN', 'P', 'NR', 'NN', 'NN', 'VV'], ['JJ', 'NN', 'AD', 'VV', 'NN', 'P', 'NN', 'VV', 'VV', 'NN'], ['NN', 'VV', 'VC', 'NN', 'VV', 'NR', 'NN', 'NN', 'AD', 'VV', 'VE', 'JJ', 'NN'], ['NN', 'VV', 'NN', 'NN'], ['P', 'NN', 'VV', 'NN', 'VV', 'VV', 'NT', 'NT', 'NT', 'NN'], ['CD', 'NN', 'NN', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'VE', 'NN'], ['NR', 'AD', 'VV', 'AS', 'NR', 'NN', 'NN'], ['VV', 'P', 'VA', 'DEC', 'NN', 'VV', 'VV', 'NN', 'NN'], ['JJ', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NN', 'LC', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'VV', 'VV', 'AD', 'VV'], ['NR', 'VV', 'VV', 'VA', 'DEC', 'NN', 'AS', 'NR', 'NN', 'NN', 'SP'], ['NN', 'PN', 'VC', 'SP', 'VV', 'P', 'NN', 'LC', 'AD', 'VV', 'NN'], ['DT', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'CC', 'NN', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'NN', 'LC', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'P', 'PN', 'VV'], ['NN', 'NR', 'AD', 'VV'], ['PN', 'VV', 'AS', 'NR', 'JJ', 'NN', 'VV', 'SP'], ['VV', 'DEC', 'JJ', 'NN', 'NR', 'CC', 'DT', 'NN', 'NN', 'AD', 'AD', 'VV'], ['P', 'NR', 'LC', 'NR', 'AD', 'VV', 'NN'], ['AD', 'VV', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV', 'NN', 'PN', 'NN', 'VE', 'NN'], ['CS', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN'], ['CS', 'VE', 'VV', 'VV', 'AD', 'JJ', 'NN', 'AD', 'VE', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NN', 'VV', 'AD', 'JJ', 'NN', 'NR', 'P', 'PN', 'VV', 'NN', 'NN'], ['AD', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'NR', 'AD', 'VV', 'SP'], ['NT', 'NR', 'NN', 'NN', 'VV', 'VV'], ['NN', 'NN', 'VV', 'NN', 'VV', 'SP'], ['VV', 'AD', 'PN', 'BA', 'VV', 'NN', 'LC', 'DEG', 'NN', 'VV'], ['NR', 'NN', 'NN', 'NN', 'NR'], ['DT', 'NN', 'AD', 'VA', 'VV'], ['VE', 'NN', 'VV', 'NR', 'AD', 'VV', 'M', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS'], ['NN', 'NN', 'CS', 'AD', 'VV', 'VE', 'CD', 'AD', 'NN', 'VA', 'NN', 'AD', 'VV'], ['NN', 'NR', 'AD', 'VV', 'BA', 'DT', 'CD', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'NT', 'VV', 'AD', 'VV', 'NN', 'VV', 'P', 'NN', 'NN', 'NN', 'VV'], ['NN', 'NN', 'LC', 'VV', 'VV', 'VV', 'JJ', 'NN'], ['OD', 'VV', 'VV', 'JJ', 'NN', 'CC', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'NN'], ['NN', 'VV', 'VV', 'NN', 'VV', 'PN', 'AD', 'VV', 'VV', 'AS', 'NN', 'VV', 'VV'], ['VV', 'DT', 'NN', 'AD', 'AD', 'VV'], ['VV', 'NT', 'NT', 'CD', 'M', 'CD', 'VV', 'NN', 'VV'], ['PN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'PN', 'AD', 'VV', 'NN', 'VV', 'SP'], ['VV', 'NN', 'AD', 'VC', 'M', 'SP', 'NN', 'NN', 'NN', 'SP'], ['VV', 'NN', 'NN', 'JJ', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'AD', 'VA', 'P', 'NN', 'VV', 'VV', 'VV', 'JJ', 'NN'], ['DT', 'NN', 'NN', 'VV', 'NN', 'AD', 'NN', 'AD', 'VE', 'NN', 'VV', 'VV', 'VV'], ['VV', 'VV', 'NN', 'SP'], ['NN', 'VA', 'NN', 'NN', 'AD', 'VE', 'NN'], ['NN', 'CC', 'NN', 'AD', 'VV', 'NN', 'VV'], ['AD', 'AD', 'NN', 'AD', 'VV', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'VC', 'VV', 'NR', 'DEG', 'NR', 'SP'], ['NN', 'VE', 'NN', 'SP'], ['NN', 'VC', 'JJ', 'NN', 'VV', 'SP'], ['NR', 'NR', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'PN', 'VV', 'VA', 'DEC', 'AD', 'NN'], ['PN', 'VV', 'VV', 'VV', 'NN', 'SP', 'NN', 'VV', 'VV'], ['VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'VA'], ['NR', 'NN', 'NT', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV'], ['NN', 'DEG', 'NT', 'NN', 'NN'], ['OD', 'NN', 'NN', 'VC', 'NN', 'VV'], ['AD', 'VE', 'JJ', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN'], ['AD', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'CD', 'JJ'], ['NN', 'VV', 'VV', 'NN', 'NN', 'NN'], ['NT', 'PN', 'AD', 'VV', 'VV', 'SP'], ['NN', 'VV', 'VV', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['NR', 'VV', 'VA'], ['NN', 'VV', 'NN', 'JJ', 'NN', 'NN', 'NN'], ['NR', 'VV', 'LC', 'VV', 'NR', 'NN'], ['NR', 'DEG', 'NN', 'VC', 'NN'], ['NR', 'AD', 'VV', 'NT', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NR'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VE', 'DT', 'NN', 'NN', 'VV', 'VA', 'DEC', 'NN', 'AD', 'VV', 'BA', 'PN', 'VV'], ['VE', 'NN', 'VV', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'NN', 'BA', 'NN', 'VV', 'NN'], ['P', 'PN', 'NN', 'AD', 'BA', 'VV', 'NN'], ['NR', 'DT', 'NN', 'AD', 'VV', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['VV', 'VV', 'NN', 'AD', 'VA', 'SP'], ['AD', 'NN', 'P', 'NR', 'LC', 'VV', 'CD'], ['VV', 'NN', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VA'], ['AD', 'NN', 'DEG', 'NN', 'NN', 'VC', 'P', 'NN', 'VV', 'VV', 'VV', 'NN'], ['NR', 'NR', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'VA'], ['AD', 'NR', 'P', 'VV', 'LC', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VV', 'VV', 'JJ', 'VV', 'NN', 'ETC', 'NN'], ['CD', 'VV', 'NN', 'MSP', 'VV', 'AD', 'VV', 'VV'], ['NR', 'JJ', 'JJ', 'CC', 'VE', 'JJ', 'DEG', 'NN'], ['AD', 'CD', 'M', 'LC', 'DT', 'M', 'NN', 'AD', 'VV', 'P', 'NN', 'NN'], ['NN', 'VV', 'CD', 'NN', 'VE', 'NN'], ['AD', 'VA', 'VA', 'DEC', 'NR', 'AD', 'VV', 'NR', 'NR', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NR', 'NN', 'VV'], ['AD', 'NN', 'SB', 'VV', 'LC', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'NN', 'NN', 'AD', 'AD', 'P', 'JJ', 'NN', 'CC', 'NN', 'NN', 'VV', 'NN', 'VV'], ['AD', 'NR', 'AD', 'VV', 'NN', 'NN', 'NR', 'VV', 'NN', 'VV', 'VV'], ['AD', 'VV', 'NN', 'VV', 'AS', 'NN', 'NN'], ['NN', 'P', 'VV', 'NN', 'NN', 'NN'], ['PN', 'CC', 'BA', 'P', 'NR', 'NR', 'NN', 'NN', 'NN', 'ETC', 'VV', 'NN'], ['PN', 'AD', 'AD', 'P', 'PN', 'AD', 'AD', 'PN', 'VV', 'VV', 'AD', 'AD', 'VV', 'PN', 'VV', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VV', 'AD', 'CD'], ['NN', 'VV', 'AS', 'PN', 'VV', 'PN', 'SP'], ['NR', 'VV', 'NR', 'NN', 'DEG', 'NN', 'VV'], ['VV', 'NR', 'VV', 'VV', 'DEC', 'VC', 'VV', 'NN', 'P', 'NN', 'VE', 'NN', 'NN', 'NN', 'VA'], ['JJ', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NR', 'AD', 'P', 'NN', 'VV', 'NN'], ['NN', 'BA', 'CD', 'NN', 'VV', 'P', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['AD', 'JJ', 'CD', 'NN', 'DEG', 'AD', 'AD', 'VV', 'NN', 'CD', 'CD', 'DEG', 'NN'], ['AD', 'PN', 'VV', 'PN', 'AD', 'AD', 'VV', 'NN', 'SP'], ['NR', 'CC', 'NN', 'NN', 'VV'], ['AD', 'SP', 'PN', 'P', 'PN', 'VV', 'JJ', 'NN', 'SP'], ['NT', 'NR', 'AD', 'AD', 'AD', 'VV'], ['NR', 'NN', 'NN', 'VV'], ['VV', 'VV', 'VV', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'VV', 'NR', 'VV', 'NR', 'NN', 'LC', 'VV', 'CD', 'M'], ['AD', 'VV', 'VV', 'NR', 'NN', 'NT'], ['NT', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NR', 'P', 'NN', 'AD', 'P', 'PN', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'P', 'NR', 'NN'], ['AD', 'NR', 'NN', 'NN', 'NN', 'NN'], ['PN', 'VV', 'AD', 'NN', 'VV', 'P', 'NN', 'AD', 'VV'], ['NN', 'NR', 'NR'], ['P', 'NR', 'VV', 'AD', 'NN', 'NN', 'NR', 'DT', 'NN', 'VV', 'AD', 'VV', 'AS', 'NN', 'NN'], ['P', 'NN', 'NN', 'NR', 'AD', 'VV'], ['VV', 'NN', 'NN', 'NN', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'CD', 'NN', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NR', 'AD', 'VC', 'NN', 'SP'], ['AD', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'NN', 'VC', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'NN'], ['PN', 'CS', 'VV', 'PN', 'VA', 'AD', 'VV', 'NN', 'VV', 'PN', 'SP'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'P', 'NN', 'MSP', 'VV', 'SP'], ['VV', 'SP', 'VV', 'PN', 'NN', 'LC'], ['NN', 'BA', 'NN', 'VV', 'DER', 'VA'], ['PN', 'VC', 'VV', 'VV'], ['NT', 'VC', 'AD', 'VV', 'DEC', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'PN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'VV', 'VC', 'VV'], ['NN', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'SP'], ['PN', 'AD', 'VV', 'PN', 'NN', 'DEG', 'NN', 'SP'], ['NN', 'NN', 'NN', 'AD', 'VA'], ['VV', 'NR', 'NN', 'LC', 'PN', 'AD', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'LC', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NR', 'NN', 'LC', 'NN', 'VV', 'LC', 'PN', 'AD', 'VV', 'DEC', 'NN', 'NN', 'VV', 'AD', 'VA'], ['NN', 'NN', 'VV', 'PN', 'VV', 'PN', 'VV', 'DER', 'P', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'NN'], ['AD', 'PN', 'AD', 'P', 'NN', 'NN', 'LC', 'VV', 'NN', 'LC', 'VV'], ['VE', 'JJ', 'DEG', 'NR', 'NN', 'SP'], ['PN', 'AD', 'AD', 'VV', 'NN', 'VV', 'CD', 'NN', 'CD', 'NN', 'VV'], ['VE', 'NN', 'VV', 'NR', 'NR', 'AD', 'NR', 'AD', 'VV', 'NN'], ['NR', 'NT', 'VV', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VV', 'NN'], ['DT', 'M', 'JJ', 'NN', 'NN', 'AD', 'VV', 'AS', 'NR', 'NR', 'ETC', 'NN'], ['NT', 'NT', 'VV', 'VV', 'AD', 'P', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN'], ['NN', 'CC', 'NR', 'JJ', 'NN', 'NR'], ['NT', 'VV', 'NN', 'VA', 'DEC', 'NR', 'AD', 'VV', 'NR', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NR', 'VV', 'NN', 'VV'], ['NR', 'NN', 'NR', 'P', 'NN', 'NN', 'LC', 'NN', 'NN', 'NN'], ['CD', 'NN', 'AD', 'AD', 'BA', 'NR', 'VV', 'JJ', 'NN'], ['VA', 'DEC', 'VC', 'NN', 'AD', 'AD', 'VV', 'JJ', 'NN', 'AD', 'P', 'VV', 'NN', 'VV'], ['PN', 'VC', 'NR', 'NN', 'JJ', 'NN', 'NN', 'SP'], ['NT', 'NT', 'DT', 'NN', 'VV', 'PN', 'NN', 'NN', 'VV', 'VV', 'P', 'CD', 'LC', 'DEG', 'NN', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'AD', 'VE', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['VV', 'VV', 'AD', 'DEC', 'NN', 'AD', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NT', 'VV', 'VV'], ['AD', 'PN', 'VV', 'SP'], ['PN', 'AD', 'CC', 'NR', 'OD', 'M', 'VV', 'NN', 'NN'], ['VV', 'P', 'PN', 'DEG', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['DT', 'NN', 'AD', 'VV', 'NR', 'NR', 'NN', 'NN', 'CC', 'NN', 'VV', 'NN', 'NN', 'ETC', 'JJ', 'NN'], ['NN', 'VV', 'DEC', 'VV', 'PN', 'PN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'AD', 'NR', 'VV', 'NR', 'NR', 'DEG', 'NR', 'NN'], ['P', 'NN', 'VV', 'VV', 'DEC', 'NN', 'CC', 'NR', 'VV', 'NN', 'NN'], ['VV', 'PN', 'P', 'NR', 'NN', 'AD', 'P', 'VA', 'DEC', 'NN', 'BA', 'PN', 'VV'], ['PN', 'AD', 'P', 'PN', 'VV', 'AS', 'NN', 'IJ'], ['AD', 'DT', 'NN', 'AD', 'VC', 'NN', 'AD', 'P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NR', 'CC', 'DT', 'NN', 'NR', 'NN', 'VE', 'NN', 'NN'], ['NN', 'NR', 'NN', 'NN', 'AD', 'VV'], ['P', 'JJ', 'NN', 'AD', 'VV', 'NN', 'CC', 'NN', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NR', 'LC', 'NR', 'NN', 'VV', 'VV', 'NN', 'LC', 'VV', 'AS', 'NN', 'VV', 'NR', 'NN'], ['CS', 'VE', 'NN', 'VV', 'PN', 'AD', 'AD', 'VV', 'VV', 'PN'], ['P', 'NN', 'LC', 'PN', 'VV', 'VV', 'NN'], ['AD', 'NN', 'VV', 'NR', 'NR', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'VV', 'AD', 'DT', 'NN', 'DEG', 'NN'], ['VV', 'AD', 'VV', 'AD', 'VV', 'VV'], ['NR', 'DEG', 'NN', 'NR', 'VV', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VA', 'VV'], ['NR', 'NN', 'NN', 'VV', 'VV', 'VV', 'BA', 'NN', 'VV'], ['NT', 'PN', 'VV', 'AS', 'NN'], ['AD', 'VV', 'AD', 'VV', 'VV'], ['NR', 'AD', 'VV', 'VV', 'VV', 'NN'], ['JJ', 'NN', 'LC', 'VV', 'NR', 'AD', 'VV', 'NR', 'MSP', 'VV', 'DEC', 'VV', 'NN', 'NN', 'VV'], ['NN', 'VV', 'AD', 'AD', 'DEV', 'VV', 'MSP', 'AD', 'VV', 'AD', 'VV', 'VV', 'DT', 'NN'], ['NT', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN'], ['JJ', 'VV', 'AS', 'VV'], ['NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN', 'AD'], ['NR', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AS'], ['P', 'DT', 'NN', 'VV', 'PN', 'VV', 'PN', 'NN', 'SP'], ['VV', 'NR', 'AD', 'VV', 'VV', 'CD'], ['NN', 'NN', 'CS', 'VV', 'NN', 'AD', 'VV', 'VV', 'VV', 'CD', 'VV', 'VV', 'NN'], ['PN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'VV'], ['P', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VE', 'CD', 'NN', 'NN'], ['NN', 'NN', 'VV', 'VA', 'DEC', 'NN', 'NN', 'CC', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'NN', 'VV', 'VE', 'NN', 'SP'], ['NR', 'AD', 'VA', 'AD', 'VV', 'VV', 'CC', 'VV', 'JJ', 'NN'], ['VV', 'NR', 'NN', 'P', 'NN', 'NN'], ['NN', 'VV', 'AD', 'BA', 'PN', 'CD', 'NN', 'VV'], ['NR', 'NR', 'NT', 'P', 'VV', 'VV', 'NN', 'LB', 'NN', 'VV'], ['AD', 'VV', 'NN', 'CD', 'NN', 'PN'], ['NN', 'LC', 'NN', 'NN', 'VV', 'NN', 'VE', 'NN', 'VE', 'JJ', 'NN', 'NN', 'VC', 'NN'], ['NN', 'NR', 'AD', 'VE', 'CD', 'NN', 'SP', 'PN', 'VV', 'VV', 'SP'], ['NR', 'NN', 'LC', 'VV', 'VV', 'NN', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['PN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'P', 'NN', 'NR', 'NN', 'NR', 'VV', 'NT', 'NN', 'LC', 'VV', 'VA'], ['NT', 'NR', 'NN', 'NR', 'AD', 'AD', 'VV', 'NN', 'VC', 'NR', 'VV', 'NN'], ['NT', 'NR', 'DEG', 'JJ', 'NN', 'NN', 'AD', 'BA', 'PN', 'NN', 'NN', 'NN', 'NN', 'VV'], ['NN', 'VV', 'P', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'PN', 'AD', 'VV', 'VV', 'CD', 'M', 'NN'], ['NR', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'P', 'VV', 'VV', 'NN', 'VV', 'VV'], ['AD', 'AD', 'AD', 'P', 'NN', 'LC', 'VV', 'SP'], ['VV', 'PN', 'VV', 'NN', 'PN', 'AD', 'VV', 'IJ', 'AD', 'PN', 'VV', 'NN', 'SP'], ['NR', 'CD', 'M', 'VV', 'AS', 'NN'], ['NN', 'NN', 'VV', 'VV', 'VE', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['PN', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NN', 'DT', 'P', 'NN', 'BA', 'NR', 'VV', 'VV', 'NN'], ['NN', 'AD', 'VV', 'AD', 'NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'NN', 'NN', 'VA'], ['NR', 'AD', 'VV', 'VV', 'CD', 'NN', 'MSP'], ['CD', 'NN', 'VC', 'JJ', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'BA', 'NN', 'DEG', 'NN', 'VV', 'AD'], ['P', 'NR', 'OD', 'CD', 'NN', 'VV', 'P', 'VV', 'NN'], ['AD', 'VV', 'NT', 'DEG', 'JJ', 'NN', 'NN'], ['NT', 'NT', 'AD', 'VV', 'VV', 'PN', 'VV'], ['NN', 'VV', 'VV', 'NN', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'VA'], ['P', 'VV', 'DT', 'JJ', 'NN', 'VV', 'NR'], ['NN', 'NN', 'NN', 'NN', 'NR', 'VV', 'NN', 'VV', 'NN'], ['NN', 'NN', 'SB', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'NN', 'AD', 'VE', 'NN', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['DT', 'NN', 'LB', 'NR', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'AD', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'PN', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'VV', 'VV', 'SP'], ['NR', 'NN', 'NT', 'VV', 'JJ', 'NN', 'NN', 'AD', 'NN', 'VV'], ['P', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NR', 'AD', 'P', 'NN', 'VV', 'NN', 'NN'], ['AD', 'NN', 'P', 'NN', 'NN', 'NN'], ['VE', 'NN', 'NN', 'NN', 'CD', 'NT', 'CD', 'M', 'CD', 'CD'], ['VV', 'NN', 'NN', 'CD', 'CD', 'NT', 'CD', 'CD', 'CD', 'NN', 'VV', 'NN'], ['AD', 'NR', 'VV', 'AD', 'P', 'NR', 'VV', 'VV', 'AD', 'AD', 'VV', 'NN', 'DEC', 'AD', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['P', 'AD', 'VV', 'PN', 'AD', 'P', 'VV', 'VV'], ['NR', 'P', 'P', 'NN', 'VV', 'LC', 'AD', 'VV', 'VV', 'P', 'NN', 'DEG', 'NN'], ['CS', 'NT', 'AD', 'VV', 'NN', 'AD', 'VA', 'AD', 'AD', 'VV'], ['DT', 'NN', 'LB', 'NN', 'VV'], ['NN', 'NN', 'NR', 'AD', 'VV', 'NR', 'NR', 'ETC', 'NN', 'NN', 'NN', 'NN'], ['NN', 'VV', 'CC', 'AD', 'VV'], ['NN', 'MSP', 'VV', 'AD', 'NR'], ['PN', 'VV', 'VV', 'NR', 'PN', 'BA', 'NN', 'NN', 'VA', 'VV', 'P', 'NN'], ['AD', 'VV', 'DT', 'AD', 'VV', 'NN', 'P', 'PN', 'AD', 'VV'], ['NN', 'AD', 'AD', 'AD', 'NN', 'AD', 'VV', 'AS'], ['AD', 'VC', 'PN', 'VE', 'NR', 'JJ', 'NN', 'VV', 'PN', 'NR', 'SP'], ['VV', 'NR', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['PN', 'VV', 'BA', 'NN', 'VV', 'VA', 'DEC', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'NT', 'NN', 'AD', 'VA', 'AD', 'VV', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['DT', 'NN', 'NN', 'AD', 'AD', 'VE', 'NN'], ['NR', 'NT', 'LC', 'VV', 'NR', 'NN'], ['NN', 'NN', 'NR', 'VV', 'NR', 'NR', 'JJ', 'P', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['CS', 'VE', 'AD', 'CD', 'NN', 'JJ', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'JJ', 'NN', 'SP'], ['NR', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'MSP', 'VV', 'CC', 'VV', 'NN'], ['NN', 'VV', 'LC', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'NR', 'AD', 'AD', 'VV'], ['P', 'NR', 'AD', 'VV', 'VA', 'VA', 'VA', 'AD', 'VV', 'NN', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'CC', 'JJ', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'PN', 'LC', 'PN', 'VV', 'PN', 'DEG', 'NN', 'NN', 'SP'], ['PN', 'AD', 'CC', 'JJ', 'DEG', 'NN', 'VV', 'PN', 'VV', 'SP'], ['NN', 'AD', 'VA', 'AD', 'VV', 'VE', 'CD', 'NN', 'AD', 'VV', 'AD', 'VV'], ['P', 'VV', 'NN', 'AD', 'AD', 'VV', 'AS', 'CD', 'M', 'NN', 'NN'], ['AD', 'PN', 'VV', 'BA', 'NN', 'P', 'NN', 'VV', 'NN'], ['P', 'VV', 'NN', 'LC', 'PN', 'VV', 'BA', 'NN', 'VV', 'NN', 'CC', 'VV', 'LC', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'AD', 'P', 'PN', 'NN', 'NN', 'VV', 'AS', 'AD'], ['CD', 'JJ', 'NN', 'AD', 'SB', 'VV'], ['NR', 'AD', 'VV', 'PN', 'VV', 'VA', 'AD'], ['NR', 'NR', 'AD', 'P', 'NR', 'VV', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['P', 'NR', 'AD', 'VV', 'VV'], ['PN', 'P', 'NR'], ['NN', 'VC', 'NR', 'JJ', 'DEG', 'NN'], ['NT', 'VA', 'DEC', 'NN', 'AD', 'NN', 'VV', 'P', 'NN', 'VV'], ['AD', 'NR', 'NR', 'AD', 'VV', 'P', 'NN'], ['VV', 'VV', 'NN', 'VV', 'DEC', 'NN', 'CC', 'NN', 'NN'], ['AD', 'AD', 'NR', 'AD', 'VV'], ['P', 'VV', 'NN', 'VV', 'VV', 'P', 'PN', 'VV', 'CD', 'VV'], ['NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'CD', 'JJ', 'JJ', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'M', 'NN', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'P', 'JJ', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VV', 'DEC', 'PN', 'AD'], ['VV', 'AD', 'P', 'PN', 'VV', 'NN'], ['NN', 'VC', 'AD', 'NN', 'SP'], ['NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VC', 'AD', 'NN'], ['NR', 'NN', 'VV', 'AD', 'VV', 'P', 'NN'], ['AD', 'VV', 'PN', 'VV', 'AD', 'SP', 'VV'], ['JJ', 'NN', 'VV', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'AD', 'P', 'DT', 'NN', 'VV', 'LC', 'NN'], ['VV', 'PN', 'VC', 'PN', 'DEG', 'MSP', 'VV', 'PN'], ['MSP', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NR', 'NN'], ['NR', 'AD', 'VV', 'P', 'JJ', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'P', 'NN', 'CC', 'NN', 'VE', 'JJ', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'VV', 'NN', 'NN'], ['PN', 'VV', 'LC', 'VV', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NR', 'NN', 'NR', 'NN', 'NR', 'NN'], ['NN', 'VV', 'NR', 'VV'], ['AD', 'VV', 'NN', 'LC', 'P', 'NT', 'AD'], ['DT', 'NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'P', 'NN', 'NN', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VV', 'VV'], ['CD', 'NN', 'NN', 'VV', 'NN'], ['DT', 'NN', 'VV', 'AS', 'NR', 'NR', 'NR', 'NN', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'VC', 'NR', 'NR', 'NR', 'JJ', 'NN', 'NN', 'NN'], ['NT', 'NN', 'LC', 'AD', 'VV', 'NN', 'NN'], ['JJ', 'NN', 'VV', 'SP'], ['AD', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VE', 'NN', 'LC', 'AD', 'VA', 'VA', 'DEC', 'NN'], ['AD', 'NN', 'AD', 'VV', 'VV', 'VV', 'AD', 'AD', 'VV', 'NN', 'LC'], ['VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'SP'], ['NN', 'NN', 'VA', 'VA', 'CC', 'CD', 'NN', 'AD', 'VV', 'AD', 'NR'], ['PN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'JJ', 'NN', 'VE', 'PN', 'JJ', 'JJ', 'DEG', 'NN', 'NN'], ['NN', 'AD', 'P', 'NR', 'NN', 'NN', 'JJ', 'DEG', 'NN', 'NR', 'NR', 'NR', 'NN', 'VV'], ['AD', 'VV', 'CD', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'VV', 'VV'], ['NN', 'BA', 'NN', 'VV', 'VV', 'VV', 'VV'], ['NR', 'NN', 'NN', 'NR', 'AD', 'VV', 'NR', 'NN', 'NN', 'NN', 'VC', 'NN', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV'], ['PN', 'VV', 'NR', 'NR', 'NN', 'DEG', 'NN', 'AD', 'P', 'P', 'NN', 'DEG', 'NN', 'MSP', 'VV'], ['JJ', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV'], ['PN', 'VV', 'VV', 'P', 'NN', 'NN', 'VV', 'AD', 'VV'], ['NT', 'JJ', 'NN', 'NN', 'CD', 'CD', 'CD', 'CD', 'NN'], ['NN', 'LC', 'VE', 'CD', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['PN', 'VV', 'AD', 'PN', 'NN', 'VV', 'NN', 'PN', 'P', 'PN', 'VV'], ['AD', 'P', 'AD', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'NN', 'ETC', 'NN', 'AD', 'AD', 'VV'], ['BA', 'PN', 'DT', 'NN', 'VV', 'NN'], ['CS', 'PN', 'AD', 'VE', 'NN', 'VV', 'PN', 'DEG', 'NN', 'SP'], ['NR', 'NN', 'NN'], ['NN', 'NN', 'AS', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'P', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'VV'], ['P', 'NN', 'AD', 'VA', 'NN', 'VV', 'AS', 'NN', 'NN'], ['NN', 'AD', 'VV', 'CD', 'NN', 'VV', 'AS', 'NN'], ['NN', 'NN', 'VV', 'NN', 'SP'], ['AD', 'PN', 'AD', 'VV', 'NR', 'NN', 'CC', 'NN'], ['NR', 'AD', 'VV', 'NN', 'VV', 'NN'], ['AD', 'VA', 'JJ', 'AS', 'PN', 'AD', 'VV', 'PN'], ['NN', 'P', 'PN', 'NN', 'AD', 'P', 'JJ', 'NN', 'NN', 'AD', 'VV'], ['VE', 'NN', 'AD', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'CC', 'NR', 'DEC', 'NN', 'AD', 'VA', 'VV', 'NN', 'NN'], ['AD', 'NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['AD', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['AD', 'VV', 'NN', 'AD', 'AD', 'VA', 'NN', 'VV', 'M', 'AD', 'AD', 'VA'], ['NN', 'NN', 'NR', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'NN', 'NN'], ['NR', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'AD', 'VV'], ['NT', 'PN', 'NN', 'VV', 'VV', 'JJ'], ['PN', 'VV', 'VV', 'NR', 'VV', 'AS', 'VV'], ['AD', 'NN', 'NN', 'VV', 'AS', 'VV', 'NN', 'NN'], ['VA', 'DEC', 'NR', 'NN'], ['PN', 'AD', 'AD', 'VV', 'AD', 'NR', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN', 'CC', 'VV', 'NN'], ['AD', 'VC', 'PN', 'VA', 'AS', 'VV', 'AS', 'PN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'SP', 'PN', 'SP', 'AD', 'VV', 'AD', 'VA', 'SP'], ['PN', 'VV', 'VV', 'PN', 'VV', 'SP'], ['NN', 'VE', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'VV', 'NR'], ['NR', 'VV', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'NN', 'LC', 'VV'], ['VV', 'VV', 'NN', 'P', 'JJ', 'NN', 'VV', 'NN', 'IJ', 'JJ', 'NN', 'NN', 'AD', 'AD'], ['AD', 'VV', 'P', 'NN', 'VA', 'PN', 'AD', 'SB', 'VV', 'NN'], ['CD', 'NN', 'VV', 'DT', 'CD', 'PN', 'AD', 'AD', 'VV', 'NN'], ['PN', 'VV', 'AD', 'VV', 'VV', 'CC', 'P', 'NN', 'NN', 'VV', 'VV'], ['PN', 'AD', 'VV', 'DER', 'AD', 'VV', 'P', 'VV', 'NN'], ['VV', 'PN', 'P', 'NN', 'LC', 'VV', 'AS', 'NN', 'DEG', 'NN', 'PN', 'AD', 'AD', 'VV', 'SP'], ['JJ', 'NR', 'NN', 'NN', 'NN', 'NR'], ['NT', 'VV', 'VV', 'NN', 'VC', 'NN'], ['CD', 'NN', 'NN', 'NN'], ['AD', 'VA', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'VA'], ['NR', 'NN', 'LC', 'NN', 'P', 'NN', 'LC', 'AD', 'VA', 'VA'], ['VV', 'SP', 'PN', 'NN', 'SP'], ['VV', 'VV', 'NN', 'NN', 'VC', 'VA', 'AD', 'NN'], ['NN', 'NN', 'P', 'NN', 'VV', 'VV', 'AS', 'VA', 'DEC', 'NN'], ['PN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN', 'VV', 'P', 'NN', 'VA'], ['P', 'NT', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NR', 'AD', 'VV', 'AS', 'JJ', 'NN'], ['NR', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AS'], ['NN', 'LC', 'VV', 'JJ', 'NN'], ['NR', 'VV', 'NR', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NT', 'NN'], ['CS', 'PN', 'VE', 'NN', 'CC', 'NN', 'VV', 'AD', 'AD', 'VV', 'P', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VC', 'NN', 'NN', 'NN', 'VV', 'SP'], ['PN', 'AD', 'VV', 'PN', 'AD', 'VA'], ['P', 'NN', 'LC', 'PN', 'VV', 'AS', 'NR', 'NR', 'DEG', 'NN', 'NN'], ['JJ', 'NN', 'AD', 'P', 'JJ', 'NN', 'VC', 'NN', 'VV', 'NN', 'DEG', 'NN', 'MSP', 'VV'], ['AD', 'NN', 'LC', 'PN', 'DEG', 'NN', 'AD', 'VV', 'CS', 'NN', 'NN', 'VV', 'PN', 'VV', 'NN'], ['NR', 'CC', 'NN', 'VV', 'AS'], ['AD', 'VV', 'NN', 'VV'], ['VV', 'NN', 'NN', 'VV', 'CC', 'VV', 'ETC'], ['VV', 'AD', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV'], ['NN', 'VV', 'PN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VE', 'NN'], ['AD', 'CD', 'M', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'VV', 'NR', 'AD', 'VV', 'CD', 'IJ', 'NN'], ['NR', 'NN', 'VE', 'CD', 'NN', 'NN', 'VV', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'P', 'NN', 'VV', 'NN', 'VV', 'NN', 'CC', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'NN', 'VV', 'VV', 'AS', 'DT', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN', 'VC', 'AD', 'VV', 'NN'], ['NR', 'VV', 'VV', 'AS', 'NN', 'NN'], ['VV', 'NN', 'DEC', 'JJ', 'NN', 'LC', 'NN', 'NR', 'NR', 'VV', 'LC', 'AD', 'VV', 'NN', 'VV'], ['NT', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['AD', 'VV', 'VV', 'NN', 'CC', 'NN', 'SP'], ['AD', 'NN', 'AD', 'VV', 'VV', 'DT', 'NN'], ['NR', 'NR', 'NN', 'VV', 'VV', 'VV', 'PN', 'VA', 'VA', 'VV', 'PN', 'DEG', 'NN', 'SP'], ['VV', 'AS', 'NN', 'AD', 'NN', 'CD', 'NN', 'VV', 'NN'], ['PN', 'AD', 'P', 'NN', 'VV', 'AD', 'AD', 'NN', 'VV', 'NN', 'CC', 'NN', 'VV'], ['NR', 'NN', 'AD', 'VV'], ['VV', 'NR', 'NR', 'NN', 'DEG', 'NR', 'NN', 'NR', 'VV', 'AS', 'NN'], ['PN', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'ETC', 'LC', 'DEC', 'NN', 'NN'], ['NR', 'AD', 'VA', 'VV', 'VV', 'NN'], ['PN', 'AD', 'VV', 'P', 'PN', 'DEG', 'NN', 'LC', 'VV', 'VV'], ['NN', 'NN', 'AD', 'VV', 'AD', 'CD', 'VV', 'VV', 'DEC'], ['P', 'NN', 'NR', 'AD', 'AD', 'VV'], ['NR', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS', 'VV'], ['P', 'NN', 'NN', 'NN', 'NN', 'NN', 'NR', 'SB', 'VV', 'VC', 'NN', 'NN'], ['AD', 'VV', 'PN', 'VV', 'VV', 'PN'], ['NN', 'NN', 'VV', 'NN', 'AD', 'BA', 'PN', 'NN'], ['NR', 'VV', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV'], ['NN', 'DEG', 'NR', 'NN', 'AD', 'M', 'JJ', 'NN'], ['NN', 'NN', 'NN', 'P', 'NR', 'AD', 'VV'], ['AD', 'VA', 'AD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'SP'], ['NR', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'VV', 'DEV', 'VV', 'LC', 'VV', 'NN', 'SB', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'VV', 'DER', 'AD', 'VV'], ['AD', 'AD', 'VV', 'JJ', 'JJ', 'NN', 'JJ', 'NN', 'AD', 'JJ', 'NN', 'CC', 'VV', 'AS'], ['JJ', 'NN', 'VV', 'LC', 'VV', 'VV', 'VV'], ['AD', 'AD', 'NN', 'NN', 'NN', 'DEC', 'NN', 'P', 'JJ', 'NN', 'AD', 'VA', 'CD'], ['AD', 'VE', 'DT', 'NN', 'VV', 'NN', 'VV', 'CD', 'NN', 'CD', 'M', 'NN', 'CD'], ['PN', 'AD', 'VV', 'NR', 'NN', 'CC', 'NN', 'NN'], ['VV', 'VV', 'VV', 'NN', 'NR', 'DEG', 'NN', 'NR', 'NN'], ['NR', 'AD', 'P', 'NN', 'NN', 'VV', 'VV'], ['NR', 'AD', 'AD', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'VV', 'JJ', 'NN', 'NN'], ['PN', 'VC', 'VV', 'PN', 'VV', 'VV', 'VV', 'SP'], ['AD', 'VE', 'NT', 'LC', 'DEG', 'NN'], ['NR', 'NN', 'NN', 'JJ', 'NN', 'JJ', 'NN'], ['AD', 'P', 'PN', 'NN', 'VV', 'VV', 'VV', 'DEC', 'NR', 'VV', 'NN', 'NN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'JJ', 'P', 'NN', 'VV', 'AS', 'VV', 'PN', 'VV', 'AS'], ['NR', 'NN', 'AD', 'VV', 'NN', 'CD', 'CD', 'CD', 'NN'], ['JJ', 'NN', 'VV', 'JJ', 'NN', 'VV', 'NR', 'NN', 'NN'], ['P', 'NN', 'LC', 'PN', 'AD', 'AD', 'VV', 'P', 'NN', 'VV', 'VV', 'SP'], ['AD', 'DEV', 'VA', 'SP', 'NN', 'VV'], ['NN', 'VV', 'CD', 'M', 'NN', 'NN'], ['NR', 'JJ', 'NN', 'P', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'NN', 'VV', 'IJ', 'VA'], ['JJ', 'NN', 'VE', 'NN', 'NN'], ['NR', 'NN', 'P', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'AS', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'VV', 'LC', 'P', 'PN', 'VV', 'AD', 'VV', 'PN', 'VC', 'NN'], ['AD', 'DT', 'NN', 'VV', 'VV', 'AS', 'NN', 'NN', 'PN'], ['NN', 'LC', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'DT', 'M', 'NN', 'NR', 'VV', 'NN', 'P', 'NN', 'DEG', 'NN', 'AD'], ['NR', 'VV', 'VV', 'VV', 'NN', 'NN'], ['VV', 'AS', 'NT', 'NR', 'LC', 'AD', 'VV', 'VV'], ['NN', 'CC', 'P', 'NN', 'VV'], ['NN', 'PN', 'AD', 'VV', 'PN', 'VV', 'AD', 'VV', 'PN', 'PN', 'AD', 'VA'], ['NN', 'VV', 'VC', 'VA', 'LC', 'DEG', 'CD', 'VC', 'VV', 'CD', 'M', 'NN', 'SP'], ['NN', 'NN', 'VV', 'P', 'NR', 'VV', 'NN'], ['VC', 'CD', 'VV', 'DEC', 'NN', 'VV', 'CD', 'NN', 'NN', 'VV', 'P', 'NN', 'LC'], ['NR', 'NR', 'VV', 'VV', 'VV', 'NN'], ['NR', 'NN', 'NR', 'VV', 'DEC', 'NN', 'NN', 'NN'], ['VV', 'NR', 'VV', 'NN', 'VC', 'AD', 'JJ', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'PN', 'PN', 'VV', 'PN', 'SP'], ['NN', 'VV', 'JJ', 'NN', 'VV', 'NN', 'AD', 'P', 'VV', 'VV'], ['NR', 'NN', 'VV', 'LC', 'NN', 'VV', 'JJ', 'AD', 'VV', 'NN'], ['PN', 'VV', 'VV', 'VE', 'NN', 'DEG', 'NN', 'PN', 'VE', 'NN', 'AD', 'AD', 'VV', 'NN', 'SP'], ['NT', 'NN', 'NR', 'NN', 'NN', 'P', 'NN', 'NN'], ['VV', 'PN', 'DEG', 'PN', 'NN'], ['NN', 'VV', 'NN', 'P', 'NN', 'LC', 'VV', 'AD', 'P', 'NN', 'LC', 'VV', 'CD', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NR', 'AD', 'VV', 'NN'], ['VV', 'NN', 'VV', 'VV', 'NN', 'CC', 'NN', 'VC', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'P', 'NN', 'CD', 'NN', 'AD', 'VV', 'VV'], ['NR', 'JJ', 'NN', 'NN', 'NR', 'AD', 'VC', 'NR', 'DEG', 'VV', 'NN'], ['NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'NN', 'NN'], ['AD', 'JJ', 'NN', 'CD', 'M', 'NN', 'CD', 'M', 'NR', 'CC', 'NR', 'VV', 'AD', 'AD', 'VV'], ['CS', 'PN', 'VC', 'CD', 'NN', 'NN', 'AD', 'VV', 'P', 'NR', 'NN', 'VE', 'CD', 'NN'], ['NN', 'VV', 'AS', 'VV', 'PN', 'AD', 'VV', 'VA', 'SP'], ['NN', 'NN', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'NN', 'LC'], ['VV', 'VV', 'VV', 'VV', 'NN', 'SP'], ['NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'P', 'VA', 'DEC', 'NN', 'NN'], ['VV', 'NN', 'LC', 'NN', 'AD', 'VV', 'VA'], ['AD', 'VV', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['LB', 'VV', 'NR', 'NN', 'NN', 'VV', 'VV', 'NN'], ['JJ', 'NN', 'VA', 'AD', 'VV', 'LC', 'VV', 'CD', 'VV', 'NN', 'VV'], ['VV', 'NN', 'VV', 'VV', 'JJ', 'NN', 'VV', 'NN', 'NN'], ['VV', 'NN', 'VV', 'AD', 'CD', 'NN', 'NN', 'VV', 'PN'], ['NR', 'AD', 'VA', 'AD', 'VA', 'AD', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'NN', 'NN', 'VV', 'PN', 'DEG', 'NN', 'NR', 'ETC', 'NN'], ['NN', 'VA', 'DEC', 'CD', 'AD', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NR', 'VV', 'NN'], ['P', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NN', 'P', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['VV', 'NR', 'NR', 'P', 'NR', 'VV', 'DEC', 'NR', 'NN'], ['AD', 'NN', 'NN', 'AD', 'MSP', 'VV', 'VV'], ['NT', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'CD', 'BA', 'PN', 'DEG', 'JJ', 'NN', 'AD', 'VV', 'AS'], ['P', 'NN', 'CD', 'NN', 'CC', 'NR', 'NN', 'NN', 'VV', 'AD', 'VA'], ['AD', 'PN', 'VV', 'NN', 'VA'], ['AD', 'NN', 'NN', 'CC', 'NN', 'NN', 'VC', 'AD', 'VA', 'SP'], ['P', 'NT', 'NT', 'DEG', 'NR', 'NN', 'NR', 'VV', 'NR', 'NN'], ['DT', 'NN', 'DEG', 'NN', 'CC', 'NN', 'NN', 'NN', 'NN'], ['VV', 'AD', 'VV', 'NN', 'LC', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NN', 'VC', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN'], ['VV', 'VV', 'NN', 'VV'], ['NR', 'VV', 'NT', 'VV', 'M', 'NN', 'NN'], ['NR', 'NR', 'NN', 'LC', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['NR', 'CC', 'NN', 'VV', 'NN', 'NR', 'CC', 'NR', 'VV', 'P', 'NN'], ['DT', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['VV', 'NN', 'NN', 'SP'], ['NR', 'NR', 'VV', 'NR', 'VV', 'CC', 'VV', 'NR', 'DEC', 'VA'], ['VV', 'NN', 'JJ', 'CD', 'NN', 'DEC', 'NN', 'AD', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'NR', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NR', 'NN', 'NR', 'CC', 'NN', 'VV', 'VV'], ['AD', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'ETC'], ['PN', 'AD', 'VV', 'PN', 'DT', 'JJ', 'NN', 'NN', 'SP'], ['JJ', 'NN', 'DEG', 'NR', 'NR', 'AD', 'VV', 'VV', 'AD', 'VV', 'SP'], ['AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV', 'PN', 'SP', 'VC', 'CD', 'IJ'], ['PN', 'VV', 'AD', 'P', 'SP', 'AD', 'VA'], ['NN', 'JJ', 'NN', 'NR', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['CS', 'NN', 'VV', 'NN', 'NR', 'NN', 'NN', 'AD', 'AD', 'VC', 'JJ', 'NN'], ['AD', 'AD', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'JJ', 'NN'], ['NN', 'P', 'NN', 'VV', 'VV', 'P', 'NN', 'M'], ['NN', 'NN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'VV', 'P', 'VA'], ['AD', 'AD', 'VV', 'NR', 'CD', 'NN', 'VV', 'VV', 'PN', 'VV', 'AD'], ['AD', 'PN', 'AD', 'VV', 'NR', 'DEC', 'NN', 'VV', 'NN', 'JJ', 'P', 'VV', 'VA'], ['VV', 'NR'], ['PN', 'VV', 'AD', 'VV', 'VV', 'SP'], ['VV', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'PN', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'LC', 'P', 'NR', 'VV', 'NN'], ['NR', 'VC', 'M', 'JJ', 'DEG', 'NR', 'NN', 'VV', 'NN', 'VA', 'VV', 'NN', 'NN', 'VV'], ['NN', 'CC', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VE', 'JJ', 'NN'], ['NN', 'VC', 'CD', 'CD', 'M'], ['NN', 'LC', 'NR', 'AD', 'P', 'AD', 'P', 'NN', 'VV'], ['PN', 'AD', 'BA', 'NN', 'AD', 'VV', 'NN', 'SB', 'VV', 'NR', 'DEG', 'NN'], ['VV', 'NN', 'VV', 'PN', 'VV'], ['VV', 'NR', 'NN', 'DEG', 'NR', 'NN', 'NN', 'AD', 'VC', 'AD', 'VA', 'VA', 'DEC', 'NN', 'NN'], ['DT', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'VV'], ['NR', 'VV', 'NR', 'VV', 'NN'], ['NR', 'NN', 'LC', 'AD', 'AD', 'VA'], ['AD', 'P', 'NN', 'DEG', 'NN', 'LC', 'VV', 'CC', 'AD', 'VV', 'NN', 'NN'], ['NR', 'AD', 'P', 'NN', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'NR', 'VV', 'VV'], ['NN', 'CC', 'NN', 'VE', 'NN', 'VV'], ['NR', 'NN', 'NN', 'VV'], ['NR', 'NN', 'VV', 'VV'], ['VA', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN'], ['P', 'NN', 'NN', 'VV', 'VV', 'P', 'NN', 'AD', 'VV', 'CD', 'AD', 'AD', 'VV'], ['AD', 'CD', 'NR', 'DEG', 'NN', 'VC', 'NR', 'NN', 'NR', 'P', 'NR', 'VV'], ['NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'VA', 'NN'], ['NN', 'AD', 'VV', 'P', 'NR', 'CC', 'NN', 'PN', 'AD', 'NN', 'VV'], ['NR', 'VV', 'P', 'NR', 'NR', 'DEG', 'NN'], ['NN', 'NT', 'P', 'NN', 'VV'], ['PN', 'VV', 'JJ', 'NN', 'VV'], ['NN', 'VV', 'NN'], ['NR', 'VV', 'NR', 'NN', 'MSP', 'VV', 'AD', 'AD', 'VV', 'AD', 'VV', 'NR', 'AD', 'AD', 'VV'], ['VV', 'NN', 'NN', 'VV'], ['NN', 'VV', 'NN', 'AD', 'VA'], ['AD', 'NR', 'AD', 'VV', 'VV', 'NN', 'NR', 'DT', 'NN', 'AD', 'PN', 'PN', 'VV', 'NN', 'NN'], ['DT', 'NN', 'NN', 'VV', 'PN', 'VV'], ['VV', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VA', 'VA', 'CC', 'VA', 'DEC', 'NN'], ['NR', 'NR', 'CC', 'NR', 'AD', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'VV', 'AD', 'NN', 'NN', 'P', 'NR', 'NN', 'VV'], ['NN', 'NN', 'VV', 'VV', 'P', 'NR', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NN', 'NR', 'NR', 'NN', 'NN', 'VV', 'NR', 'DEC', 'NN', 'MSP', 'VV', 'AD', 'VV', 'NR'], ['DT', 'NN', 'VV', 'DER', 'VA', 'CC', 'VA'], ['AD', 'PN', 'DEG', 'NN', 'CC', 'NN', 'VE', 'VV', 'VA', 'DEC', 'NN', 'NN'], ['NR', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['P', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN'], ['NT', 'VV', 'DEC', 'NN', 'NN', 'VV', 'NN', 'VV', 'NR'], ['NR', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['NR', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['DT', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV'], ['AD', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'PN', 'AS', 'NN', 'NN', 'AD', 'VV'], ['NR', 'JJ', 'NN', 'VV', 'VV', 'NR', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NN', 'LC', 'VV', 'AS', 'NN', 'NN', 'ETC', 'NN', 'VV', 'NN', 'CC', 'NN'], ['VV', 'P', 'NN', 'VV', 'VV', 'NN'], ['AD', 'P', 'CD', 'VV', 'NN', 'LC', 'DEG', 'NN', 'VV', 'NN', 'SP'], ['VV', 'NR', 'NN', 'NN', 'NR', 'NR', 'CC', 'NR', 'NR', 'NN', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'NN', 'JJ'], ['NR', 'NN', 'VV', 'JJ', 'NN', 'VV', 'CD', 'M'], ['AD', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'VV'], ['VV', 'NN', 'VV', 'NN'], ['NN', 'VV', 'P', 'NN', 'LC', 'DEG', 'NN'], ['NR', 'VV', 'NN', 'VV', 'DT', 'NN', 'VV', 'VV', 'SP'], ['VV', 'NN', 'SB', 'VV', 'DEC', 'NN'], ['NN', 'VA', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'DEC', 'VC', 'NT', 'VA', 'VA', 'DEC', 'NN'], ['AD', 'NN', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'VV', 'NN', 'VA'], ['NN', 'CC', 'JJ', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['PN', 'VV', 'PN', 'AD', 'VV', 'NN', 'NN', 'CC', 'M', 'NN'], ['NN', 'NN', 'VV', 'SB', 'VV', 'DEC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VV'], ['AD', 'P', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV'], ['AD', 'VV', 'P', 'NN', 'VV'], ['VV', 'VV', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NR', 'VV', 'VA', 'VA', 'NR', 'VV', 'NN', 'NN'], ['CS', 'NN', 'VV', 'NR', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV'], ['NN', 'VC', 'VV', 'NN', 'NN', 'VC', 'CD', 'NN', 'AD', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VA', 'VV', 'DEC', 'NN'], ['NR', 'AD', 'VV', 'DT', 'NN', 'P', 'NN', 'VV'], ['NR', 'DEG', 'NN', 'VE', 'NN', 'NN', 'NT', 'DEG', 'NN', 'AD', 'VE', 'VV', 'NN', 'NN'], ['NR', 'VV', 'NN', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'P', 'NN', 'DEG', 'NN', 'AD', 'CD'], ['VV', 'VV', 'NN', 'NN', 'NR', 'VV', 'P', 'NN', 'DEG', 'NN', 'LC', 'NN', 'AD', 'VV'], ['NN', 'NN', 'VV', 'AD', 'VV', 'VV', 'PN'], ['VV', 'LC', 'VV', 'NR', 'VV'], ['AD', 'AD', 'PN', 'AD', 'VV', 'PN', 'VV', 'DER', 'VV', 'VV', 'NN'], ['P', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'P', 'NN', 'LC'], ['AD', 'NT', 'VV', 'VV', 'VV', 'NR', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VE', 'NN'], ['NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN'], ['AD', 'NR', 'AD', 'VV', 'VV', 'NN', 'VV', 'PN', 'VV'], ['AD', 'NN', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'AD', 'AD', 'VA'], ['NN', 'NN', 'NN', 'VV', 'JJ', 'NN', 'NN', 'P', 'NN', 'LC', 'AD', 'VV', 'NN', 'VV', 'CC', 'VV'], ['VV', 'LC', 'NN', 'NR', 'NR', 'NR', 'VV', 'VV'], ['AD', 'P', 'NR', 'NN', 'NN', 'DEC', 'NR', 'SP'], ['NR', 'VV', 'DEC', 'NN', 'NR', 'NN', 'NR', 'AD', 'VC', 'NR', 'NN'], ['NR', 'P', 'NN', 'VV', 'NN', 'CC', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['AD', 'VV', 'AS', 'VV', 'NN', 'DEC', 'NR', 'NR', 'NN', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['NT', 'NN', 'M', 'NN', 'NN', 'NN', 'AD', 'AD', 'VA'], ['AD', 'AD', 'NN', 'AD', 'VV', 'VV'], ['NN', 'AD', 'P', 'NN', 'NN', 'VV'], ['NN', 'CC', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NN', 'VC', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN'], ['NR', 'VV', 'AS', 'NR', 'DEG', 'NN'], ['PN', 'NN', 'VV', 'NN', 'AD', 'P', 'PN', 'P', 'NN', 'AD', 'AD', 'VV', 'DT', 'NN', 'SP'], ['NN', 'AD', 'VV', 'NR', 'AD', 'VV', 'DEV', 'P', 'NN', 'NN', 'VV', 'VV'], ['VV', 'NN', 'LC', 'NN', 'JJ', 'NN', 'NN', 'AD', 'VV', 'VA', 'VA'], ['NN', 'NN', 'NR', 'NN', 'NN', 'AD', 'VE', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'ETC'], ['NR', 'AD', 'VV', 'CC', 'VV', 'VV'], ['P', 'NN', 'DEG', 'VV', 'NN', 'AD', 'NN', 'CC', 'NN', 'VV'], ['VV', 'VV', 'NN', 'VV'], ['PN', 'AD', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'P', 'NR', 'NR', 'VV', 'NR', 'VV', 'AS', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'PN', 'NN', 'VV', 'NR', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'DT', 'JJ', 'NN', 'CD', 'NN', 'VV', 'NN'], ['AD', 'NN', 'AD', 'VV', 'PN', 'JJ', 'NN', 'AD', 'VA', 'AD', 'VA', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'AD', 'VV', 'NR'], ['NN', 'VV', 'AD', 'AS', 'VV', 'NN', 'SP', 'JJ', 'NN', 'VV', 'PN', 'SP'], ['CD', 'JJ', 'NN', 'SB', 'VV', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'ETC', 'NN'], ['NT', 'PN', 'VV', 'VV', 'VV', 'NN'], ['NN', 'VV', 'PN', 'NN', 'VA', 'NN', 'VV', 'PN', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AD'], ['AD', 'NR', 'NR', 'AD', 'VV', 'AS', 'NN'], ['NN', 'NN', 'P', 'NN', 'VV', 'AS', 'NN'], ['VV', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NR', 'VV'], ['NN', 'VV', 'AD', 'VV', 'AD', 'P', 'VA', 'CC', 'VV', 'P', 'NN', 'NN', 'LC'], ['VV', 'VV', 'VV', 'PN', 'VV', 'NR'], ['DT', 'NN', 'VV', 'VV', 'NN', 'P', 'PN', 'VV', 'AS'], ['NN', 'VV', 'VV', 'VV', 'AD', 'VV', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'AD', 'AD', 'AD', 'VA', 'AD', 'VV', 'NN', 'AD', 'VV', 'AS', 'SP'], ['VV', 'JJ', 'NN', 'AD', 'VV'], ['NN', 'NN', 'LC', 'NR', 'VV', 'AD', 'VV', 'AS', 'JJ', 'NN', 'NN'], ['NR', 'VV', 'NN', 'VV', 'LC', 'NR', 'AD', 'P', 'NN', 'VV', 'NR', 'VV', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['NN', 'VV', 'NR', 'NN', 'P', 'NN', 'LC', 'AD', 'VV', 'VV'], ['CD', 'NN', 'CC', 'CD', 'M', 'AD', 'AD', 'VV', 'JJ', 'P', 'NN', 'VV'], ['NN', 'JJ', 'NN', 'NN', 'NN', 'NR', 'VC', 'NT', 'DEG', 'OD', 'M', 'NN'], ['CD', 'NT', 'NR', 'NN', 'VV', 'NN', 'AD', 'VE', 'VV', 'NN'], ['VV', 'P', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NN', 'AD', 'SB', 'VV', 'P', 'NN', 'ETC', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'VV', 'VV', 'SP', 'VV', 'VV', 'SP'], ['AD', 'PN', 'AD', 'AD', 'P', 'NN', 'VV', 'VE', 'NN', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NT', 'PN', 'NN', 'AD', 'VV', 'SP'], ['PN', 'AD', 'VV', 'PN', 'VV', 'NN', 'SP'], ['PN', 'DEG', 'NN', 'VE', 'NN', 'NN', 'CC', 'NN', 'AD', 'NN', 'VV', 'NN', 'NN'], ['PN', 'NT', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VC', 'P', 'JJ', 'NN', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['P', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'PN', 'SP'], ['PN', 'VC', 'NN', 'NN', 'NN', 'NR', 'VC', 'NN', 'SP'], ['VV', 'P', 'NT', 'NT', 'NT', 'JJ', 'NN', 'AD', 'NT', 'CD', 'CD', 'M'], ['DT', 'JJ', 'NN', 'NN', 'VC', 'NN', 'NN', 'VV', 'DEC', 'NN', 'P', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'JJ', 'NN', 'VV', 'SP'], ['AD', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV'], ['VV', 'NN', 'NN', 'AD', 'DEV', 'VV', 'PN', 'AD', 'VE', 'NN'], ['NN', 'DEG', 'NN', 'NR', 'VV', 'AD', 'VV', 'VC', 'NN', 'NN', 'VV', 'VV', 'PN', 'VV'], ['AD', 'AD', 'NN', 'NN', 'LB', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'AD', 'P', 'NR', 'NN', 'NN', 'CD', 'JJ', 'DEG', 'NN', 'VV', 'AS', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN'], ['NT', 'CC', 'NT', 'NR', 'AD', 'NN', 'LC', 'VA', 'NR', 'NR', 'NN', 'VA'], ['NR', 'VV', 'NN', 'NN'], ['NN', 'DEG', 'JJ', 'NN', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['CS', 'VV', 'P', 'DT', 'VV', 'PN', 'VV', 'AS'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VV', 'P', 'ON', 'SP', 'NN'], ['DT', 'NN', 'VC', 'PN', 'VV', 'SP'], ['DT', 'NN', 'VV', 'JJ', 'NN', 'NT', 'VV', 'VA'], ['NN', 'NN', 'NR', 'NN', 'NN', 'NN', 'NN', 'NT', 'CD', 'CD', 'M'], ['PN', 'AD', 'VV', 'NN', 'NN', 'PN', 'AD'], ['AD', 'AD', 'PN', 'VV', 'NN', 'VE', 'JJ', 'DEG', 'NN'], ['P', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'VV', 'DT', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'P', 'CD', 'NN', 'P', 'NN', 'AD', 'VA', 'AD', 'VV', 'NN'], ['NR', 'JJ', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'SB', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'SP', 'SP'], ['CS', 'VA', 'VV', 'VV', 'AD', 'VA', 'SP'], ['NN', 'NN', 'CC', 'DT', 'JJ', 'NN', 'NN', 'NN', 'NN', 'AD', 'VA'], ['NT', 'NN', 'NN', 'NN'], ['JJ', 'JJ', 'NN', 'AD', 'CD'], ['NN', 'VV', 'AS', 'P', 'PN', 'VV'], ['VV', 'VV', 'VV', 'DEC', 'NN', 'VV', 'AD', 'VV', 'DT', 'NN', 'VC', 'VV', 'NT', 'NN', 'AD', 'VV', 'SP'], ['NN', 'AD', 'VE', 'CD', 'NN'], ['NR', 'AD', 'AD', 'NN', 'NN', 'VV', 'AS', 'VV', 'PN', 'VV', 'VV', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'VA'], ['P', 'NN', 'VV', 'NR', 'NR', 'DEG', 'VV', 'DEC', 'NN', 'VA'], ['VV', 'NN', 'NN', 'DEG', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NR', 'NR', 'NR', 'JJ', 'NN', 'NR', 'NR', 'NR', 'NR', 'NN', 'ETC', 'NN', 'DEG', 'NN', 'NN', 'VV', 'NN'], ['NR', 'CC', 'JJ', 'NN', 'NN', 'LC', 'AD', 'VV', 'DEC', 'NN', 'VA'], ['AD', 'VV', 'AD', 'P', 'NN', 'VV'], ['DT', 'NN', 'VV', 'VV', 'PN', 'AD', 'VA'], ['NN', 'VV', 'VV', 'BA', 'NN', 'AD', 'VV', 'AS'], ['NN', 'VV', 'NR', 'AD', 'VE', 'NN', 'NN', 'NR', 'VV', 'VE'], ['NT', 'NN', 'NN', 'NN', 'VV', 'AS', 'JJ', 'NN'], ['VV', 'NN', 'VE', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'SP'], ['DT', 'NN', 'JJ', 'NN', 'CC', 'NR', 'NR', 'NN', 'VA'], ['AD', 'NN', 'P', 'NN', 'VA', 'MSP', 'LB', 'NN', 'NN', 'VV', 'VV'], ['NR', 'P', 'NN', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NR', 'DEG', 'JJ', 'NN', 'VV', 'NR'], ['NN', 'AD', 'VV', 'AS', 'NR', 'VE', 'P', 'NN', 'NN', 'VV', 'ETC', 'NN', 'NN'], ['AD', 'VV', 'PN', 'VV', 'VV', 'PN', 'SP'], ['NT', 'NR', 'VV', 'VV', 'NN', 'NN', 'P', 'VV', 'NN'], ['AD', 'NN', 'CC', 'PN', 'DEG', 'NN', 'VA'], ['NN', 'VV', 'LC', 'VV', 'VV', 'VV', 'NN'], ['PN', 'AD', 'P', 'NR', 'NR', 'ETC', 'AD', 'VV', 'DEC', 'NN', 'NN', 'VV', 'AS', 'AD', 'CD', 'JJ', 'NN'], ['NN', 'AD', 'AD', 'AD', 'VC', 'VA'], ['VV', 'NN', 'VV', 'VV', 'VV', 'NN'], ['CS', 'NN', 'AD', 'VV', 'NR', 'AD', 'VV', 'VV', 'AD', 'VV', 'VV', 'NR', 'VV', 'AS', 'NN'], ['JJ', 'DEG', 'NN', 'VV', 'PN', 'VV', 'DER', 'VV', 'DEC', 'NN', 'AD', 'P', 'NN', 'P', 'NN', 'LC', 'VV', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NN', 'AD', 'VV', 'NR', 'NN'], ['VA', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'P', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'CD', 'M', 'NN'], ['VE', 'NN', 'VV', 'NR', 'AD', 'VV', 'VV', 'LC', 'P', 'NN', 'AD', 'VV'], ['P', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'NN', 'VV'], ['PN', 'VE', 'PN', 'VE', 'DT', 'NN'], ['AD', 'PN', 'AD', 'NN'], ['PN', 'AD', 'P', 'NN', 'SP', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV', 'AD', 'VV'], ['NN', 'DT', 'VV', 'VV'], ['P', 'NR', 'NN', 'VV', 'VV', 'LC', 'NN', 'VV', 'LC', 'NN', 'VV'], ['NR', 'NN', 'VV', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'PN', 'CD', 'VV', 'CD', 'NN'], ['VV', 'VV', 'VV', 'VV', 'NN', 'DEC', 'PN', 'VV', 'VC', 'P', 'NR', 'VV', 'SP'], ['NN', 'DEG', 'NN', 'P', 'NT', 'NN', 'NN', 'AD', 'VV'], ['NR', 'AD', 'VC', 'NR', 'NN', 'NN', 'LC', 'DEG', 'NN'], ['P', 'PN', 'VV', 'NT', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV', 'AS', 'VA'], ['NR', 'VV', 'VV', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NN', 'LC', 'VV', 'NT', 'DEG', 'NN', 'NR', 'P', 'VE', 'MSP', 'VV'], ['NR', 'NN', 'VV', 'DT', 'M', 'NN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'P', 'AD', 'JJ', 'NN', 'VV', 'VV'], ['NN', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'VV', 'VE', 'VA', 'DEC', 'NN'], ['OD', 'NN', 'DEG', 'NN', 'VC', 'VV', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['P', 'NN', 'NN', 'NN', 'VV', 'JJ', 'NN', 'NR', 'VV', 'VV', 'AD', 'NN'], ['NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'NN'], ['AD', 'AD', 'VA', 'VA', 'DEC', 'NN', 'AD', 'VV', 'AD', 'NN', 'NN'], ['NN', 'VV', 'NR', 'LC', 'NR', 'DEG', 'NN', 'NN', 'NN', 'P', 'NN', 'VV', 'NN', 'NN'], ['CS', 'AD', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'NN', 'P', 'NN', 'VV', 'NN'], ['IJ', 'VV', 'AD', 'VV', 'SP', 'VV'], ['IJ', 'AD', 'VC', 'NR', 'CC', 'NR', 'VV', 'DEC', 'NN', 'VA'], ['NN', 'NN', 'NN', 'ETC', 'VV', 'P', 'NN', 'NN', 'VV', 'NN', 'DEG', 'NN', 'LC'], ['NN', 'VV', 'BA', 'NN', 'AD', 'VC', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'NN'], ['VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['PN', 'VV', 'AS', 'NN', 'P', 'NN', 'VV', 'VV', 'VV', 'CD', 'AD', 'VV', 'LC', 'CD'], ['NR', 'CS', 'VE', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'JJ', 'AD', 'VV', 'JJ'], ['PN', 'VV', 'PN', 'VE', 'P', 'PN', 'VV', 'NN', 'NN'], ['NN', 'NR', 'NR', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN', 'VC', 'NN', 'NT'], ['PN', 'VV', 'NN', 'VV', 'AS', 'AD', 'VV', 'NN', 'P', 'PN', 'VV', 'NN', 'VV', 'LC', 'AD', 'VV'], ['P', 'PN', 'VV', 'VV', 'NN', 'AD'], ['AD', 'VV', 'NN', 'NN', 'DEC', 'VC', 'NN', 'LC', 'VV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN'], ['PN', 'P', 'PN', 'VV', 'VV', 'NN', 'LC', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NN'], ['AD', 'VV', 'VE', 'VA', 'DEC', 'NN', 'SP'], ['AD', 'NN', 'SP', 'AD', 'NN', 'PN', 'AD', 'VA', 'AD', 'VA'], ['PN', 'AD', 'NN', 'VA', 'AD', 'VV', 'NN', 'SP', 'NN', 'AD', 'P', 'NN', 'NN', 'P', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NR', 'P', 'NR', 'VV'], ['NN', 'AD', 'AD', 'VC', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'NR', 'VV', 'NN'], ['NR', 'VV', 'AD', 'VV', 'NR', 'CC', 'BA', 'PN', 'VV', 'NN', 'DEC', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NT', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV'], ['VV', 'VV', 'NN', 'CC', 'VV', 'NR'], ['PN', 'AD', 'P', 'PN', 'VV', 'VV'], ['NR', 'NR', 'NN', 'NR', 'VV'], ['NN', 'P', 'VV', 'NN', 'DEC', 'NN', 'LC', 'VV', 'VV', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN'], ['NN', 'VV', 'LC', 'PN', 'VV', 'VV', 'VV', 'NN', 'VV'], ['NN', 'VV', 'DEC', 'VA', 'NN', 'VV', 'VV', 'JJ', 'NN', 'MSP', 'VV', 'DEC', 'JJ', 'NN'], ['PN', 'VV', 'PN', 'P', 'PN', 'VV', 'NN', 'VV'], ['PN', 'VC', 'VV', 'SP'], ['NN', 'NR', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV'], ['CD', 'NR', 'VV', 'NN', 'NR', 'NR', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'VV'], ['P', 'VV', 'NR', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NT', 'VV'], ['CS', 'NN', 'VV', 'AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'VV', 'VV', 'SP'], ['AD', 'PN', 'VV', 'VV', 'VV', 'CD', 'NN'], ['NN', 'NN', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'VV'], ['AD', 'NT', 'NN', 'AD', 'SB', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN', 'AD', 'SB', 'VV', 'VV', 'NN', 'VV'], ['NR', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['PN', 'AD', 'OD', 'NN', 'DEG', 'NN', 'NR', 'VV', 'VV', 'SP'], ['NR', 'VV', 'AD', 'PN', 'AD', 'P', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NN', 'VV', 'AS', 'JJ', 'NN', 'NN'], ['NN', 'AD', 'P', 'NN', 'NN', 'LC', 'VV', 'VV'], ['PN', 'AD', 'P', 'VV', 'NN', 'P', 'NN', 'NN', 'NN', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'NN', 'NR'], ['NN', 'NR', 'AD', 'VV', 'JJ', 'NN', 'P', 'NN', 'AD', 'VV', 'NN', 'VV', 'AD', 'VA'], ['NN', 'VV', 'VV', 'P', 'VV', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['NN', 'NN', 'NN'], ['AD', 'PN', 'VE', 'M', 'NN', 'NN', 'VC', 'NN', 'NN', 'PN', 'AD', 'VV', 'P', 'NN', 'NN', 'VV'], ['AD', 'BA', 'PN', 'DT', 'NR', 'AD', 'VV', 'AS', 'PN', 'P', 'PN', 'VV', 'VV', 'PN', 'AD', 'VV', 'SP'], ['AD', 'AD', 'VV', 'SP'], ['NN', 'VV', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'VV'], ['PN', 'AD', 'VV', 'AS', 'CD', 'VV', 'NN'], ['VE', 'VV', 'NN', 'NT', 'NT', 'NN', 'NN'], ['PN', 'AD', 'VV', 'PN', 'VC', 'NN', 'SP'], ['AD', 'VV', 'AD', 'NR', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'LC', 'NR', 'DEG', 'NN', 'NN', 'VV'], ['NR', 'NN', 'VV', 'NN', 'VV', 'AS', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['P', 'NN', 'AD', 'VV', 'AD', 'VV', 'BA', 'JJ', 'NN', 'VV'], ['VV', 'AD', 'VA', 'AS', 'JJ', 'VV', 'VV'], ['NR', 'AD', 'P', 'NN', 'NN', 'LC'], ['NR', 'VV', 'DER', 'VV', 'NN', 'VV', 'NN', 'NN'], ['NR', 'VV', 'NN', 'NN', 'NN'], ['CD', 'VA', 'DEC', 'NN', 'VV'], ['NN', 'VV', 'NR', 'VV', 'NN'], ['DT', 'NN', 'DEG', 'JJ', 'VV', 'VV', 'JJ', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV'], ['NN', 'VV', 'NN'], ['PN', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'NN'], ['NR', 'NR', 'VV', 'AS', 'NN', 'NN', 'NN', 'AD', 'VV'], ['VV', 'NT', 'NT', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'VC', 'VV', 'VV', 'NR', 'VV', 'NN'], ['NN', 'VV', 'NN', 'CD', 'NT', 'CD', 'NT'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'NN', 'SP'], ['VV', 'SB', 'VV', 'NN', 'LC', 'VV', 'VV'], ['NR', 'NN', 'VV', 'NN', 'NN', 'LC', 'DT', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV'], ['CS', 'AD', 'PN', 'VV', 'VV', 'NN', 'NN', 'DT', 'NN', 'NN', 'AD', 'VV'], ['NR', 'NN', 'VV', 'NR', 'NR', 'VV', 'NN', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'AD'], ['NN', 'NN', 'NR', 'VV', 'NN', 'AD', 'VA'], ['P', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'P', 'NN', 'VV', 'NN'], ['NN', 'AD', 'AD', 'NN', 'AD', 'VA', 'PN', 'AD', 'VV', 'VA'], ['VV', 'NR', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'LC', 'NN', 'NN', 'MSP', 'VV'], ['P', 'NR', 'NN', 'NN', 'DEC', 'NR', 'VV'], ['NN', 'VV', 'P', 'NN', 'AD', 'AD', 'VE', 'NN', 'P', 'NN', 'VV'], ['AD', 'NR', 'AD', 'VV', 'AS', 'JJ', 'NN'], ['NN', 'VV', 'CC', 'NN', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VA', 'NN'], ['NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'LC', 'VE', 'NN', 'DEC', 'NN', 'VV'], ['NN', 'VV', 'NN', 'JJ', 'NN', 'VV', 'NR', 'JJ', 'NN', 'MSP', 'VA'], ['NN', 'VV', 'NR', 'VV', 'NN', 'VV', 'VV', 'NN', 'LC'], ['SP', 'VV', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN', 'VV'], ['VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'VV'], ['NN', 'NN', 'VV', 'DT', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'DEC', 'JJ', 'NN', 'VC', 'NN', 'NN', 'NN'], ['NR', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['NR', 'NN', 'NN', 'CD', 'NT', 'NT', 'P', 'NR', 'NN', 'VV'], ['VE', 'AD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'VV', 'P', 'PN', 'VV'], ['AD', 'NN', 'LC', 'AD', 'NN'], ['P', 'NR', 'NR', 'VV', 'PN', 'VC', 'SB', 'VV', 'DEC', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VA', 'AD', 'AD', 'VV', 'VV'], ['NR', 'P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NR', 'AD', 'VC', 'NR', 'NN', 'DEG', 'NN', 'NN', 'AD', 'P', 'NN', 'MSP', 'VV', 'NN'], ['PN', 'AD', 'P', 'PN', 'NN', 'LC', 'VV'], ['AD', 'P', 'NN', 'NN', 'PN', 'P', 'PN', 'PN', 'VV', 'VV', 'NN', 'SP'], ['NN', 'VV', 'VC', 'AD', 'VA', 'DEC'], ['NN', 'DEG', 'NN', 'AD', 'VA', 'AD', 'NN', 'NN', 'VA'], ['NR', 'VV', 'JJ', 'NN', 'JJ', 'NN', 'NN', 'VV', 'VV'], ['VV', 'NR', 'NR', 'NN', 'NN', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'P', 'NN', 'VV', 'AS'], ['VV', 'AD', 'VV', 'VV'], ['PN', 'VV', 'NR', 'P', 'NR', 'VV', 'P', 'PN', 'VA', 'VV', 'NN', 'NN'], ['AD', 'NR', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN'], ['PN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'VV', 'VV', 'PN', 'VV', 'JJ', 'NR', 'DEG', 'NN', 'NN', 'NN'], ['VV', 'P', 'NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN'], ['CD', 'NT', 'NT', 'VV', 'CD', 'NT', 'NN', 'NR', 'VV'], ['NT', 'VE', 'VA', 'DEC', 'NN', 'NN', 'JJ'], ['NN', 'VV', 'PN', 'VV', 'VE', 'NN', 'SP', 'VV'], ['NN', 'AD', 'VV', 'NR', 'NN'], ['VV', 'DEC', 'NN', 'P', 'NN', 'AD', 'VA'], ['NN', 'NN', 'NN', 'NN', 'VV', 'VV', 'JJ', 'NN', 'NN'], ['JJ', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NN', 'AD', 'NN'], ['NR', 'NN', 'VV', 'SP'], ['VV', 'NR', 'VC', 'NN', 'VV', 'NR', 'AD', 'VV', 'DT', 'NR', 'DEG', 'NN', 'NN'], ['NN', 'P', 'NN', 'NR', 'NR', 'NN', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'AS', 'DT', 'M', 'NN', 'NN', 'SP'], ['NN', 'VV', 'VV', 'VV', 'NN', 'LC', 'VV', 'NN', 'NN'], ['NT', 'NR', 'NN', 'NN', 'VV', 'LC', 'AD', 'VV', 'NR', 'VV', 'NN'], ['NR', 'NN', 'AD', 'NN', 'AD', 'P', 'VV', 'DEC', 'NN', 'LC', 'AD', 'VV'], ['NN', 'AD', 'JJ', 'NN', 'NN', 'M'], ['NN', 'NR', 'NN', 'VV', 'NR', 'NR', 'VV', 'CC', 'VV', 'DT', 'JJ', 'NN'], ['NN', 'VV', 'PN', 'VV', 'PN', 'VV', 'JJ', 'NN', 'SP'], ['NT', 'NR', 'AD', 'VV', 'AS', 'NN', 'P', 'NR', 'VV', 'NN'], ['VV', 'AD', 'NN', 'LC', 'PN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'AS', 'VV', 'DEC', 'NN'], ['CD', 'NN', 'AD', 'VV', 'AS', 'VA', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'LB', 'NN'], ['NN', 'NR', 'NN', 'NN', 'NN'], ['P', 'NR', 'DEG', 'NN', 'NR', 'VV', 'NR', 'NN', 'NR', 'VV', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN'], ['DT', 'NN', 'VV', 'VV', 'NN', 'NN', 'CC', 'AD', 'VV', 'NN'], ['VV', 'CD', 'NN', 'AD', 'VV', 'AD', 'VA'], ['PN', 'VV', 'SP', 'PN', 'NN', 'VV', 'VV', 'DEC', 'NN', 'VV', 'NN', 'DT', 'NN', 'NN', 'NN', 'SP'], ['NN', 'NN', 'P', 'VV', 'NR', 'NN', 'VV'], ['NR', 'AD', 'P', 'NN', 'NN', 'VV'], ['NR', 'VV', 'AD', 'NN', 'AD', 'VV', 'VV', 'NN', 'PN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VC', 'M', 'NN', 'AD', 'CS', 'P', 'AD', 'AD', 'JJ', 'NN', 'DEG', 'JJ', 'CC', 'VA'], ['NR', 'AD', 'VV', 'VA', 'DEC', 'NN', 'NN', 'SP'], ['VV', 'VV', 'JJ', 'NN', 'VV', 'NR', 'AD', 'VV', 'PN', 'AD', 'VA'], ['P', 'NN', 'DT', 'NN', 'VV', 'P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'VV'], ['PN', 'VV', 'VV', 'NR', 'DEG', 'NN', 'CC', 'NN'], ['NN', 'NR', 'NN', 'CD', 'CD', 'CD', 'CD'], ['NN', 'NN', 'VV', 'NN', 'LC', 'VV', 'AS', 'NN', 'CC', 'NN'], ['VV', 'NN', 'VA', 'DEC', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV'], ['NN', 'NN', 'AD', 'P', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'VV'], ['NR', 'VV', 'AD', 'VV', 'VV', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'VA'], ['DT', 'NN', 'VV', 'VV', 'NN'], ['NR', 'NR', 'NR', 'NN'], ['DT', 'VV', 'PN', 'VV', 'DEC', 'PN', 'VV', 'VV'], ['VE', 'AD', 'PN', 'P', 'NN', 'DEG', 'NN', 'VV', 'PN', 'VV', 'AD', 'VV', 'NN', 'VV'], ['PN', 'VV', 'NN', 'P', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'NR', 'VV', 'NN'], ['PN', 'VA', 'DEC', 'NN', 'NN', 'VV'], ['DT', 'NR', 'NN', 'VV', 'AD', 'VA'], ['PN', 'VV', 'CD', 'CD', 'VV'], ['AD', 'NN', 'NN', 'NT', 'AD', 'AD', 'VA'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'P', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'AD', 'AD', 'VV', 'NN'], ['DT', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'VV', 'VV', 'DER', 'VE', 'NN', 'CD'], ['VV', 'DEC', 'NN', 'VV', 'VV'], ['AD', 'JJ', 'NN', 'NN'], ['AD', 'AD', 'AD', 'AD', 'AD', 'SP'], ['VV', 'NN', 'NN', 'NN'], ['PN', 'VV', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'SP'], ['NR', 'AD', 'VA', 'DEC', 'NN', 'NN', 'NN', 'VV', 'NN', 'VV'], ['VA', 'DEC', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'NN', 'NN', 'AD', 'VA'], ['VV', 'NN', 'NN', 'VE', 'NN'], ['VV', 'NN', 'VV', 'AD', 'VV', 'VV'], ['JJ', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV'], ['NR', 'NN', 'P', 'NN', 'VA', 'AD', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'NR', 'AD', 'VV', 'NN', 'SP', 'AD', 'AD', 'VV', 'VV', 'AS', 'CD', 'NN'], ['AD', 'P', 'VA', 'NN', 'VV', 'NN', 'NN'], ['P', 'PN', 'AD', 'VV', 'NN', 'VV', 'VV', 'AD', 'AD'], ['AD', 'NR', 'VV', 'AS', 'NN', 'NR', 'AD', 'VV', 'PN', 'P', 'NR', 'VV', 'VV'], ['NN', 'VV', 'VV', 'AS', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NN', 'NN', 'LC', 'DT', 'AD', 'VV'], ['NN', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'LC', 'NN', 'AD', 'BA', 'CD', 'CD', 'CD', 'CD', 'VV', 'VV', 'JJ', 'NN', 'NN'], ['DT', 'NN', 'NN', 'NN', 'JJ'], ['P', 'NN', 'VV', 'LC', 'NR', 'AD', 'AD', 'BA', 'NN', 'VV', 'AD', 'VV'], ['AD', 'VV', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'PN', 'NN'], ['NN', 'NN', 'NN', 'AD', 'VV', 'CC', 'VV', 'VV', 'NN'], ['NN', 'VV', 'PN', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VE', 'NN', 'NN'], ['PN', 'VC', 'AD', 'NN', 'AD', 'NN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'ETC', 'VV', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'VV', 'NN'], ['P', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NT', 'NN', 'VV', 'NN'], ['AD', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'NR', 'NR'], ['DT', 'NN', 'NN', 'AD', 'AD', 'VV', 'JJ', 'NN', 'CC', 'NN'], ['AD', 'NR', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'NN'], ['AD', 'AD', 'VV', 'M', 'NN', 'NN', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN', 'AD', 'NN', 'VV', 'AD', 'VA'], ['PN', 'VV', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV', 'JJ', 'NN', 'VV', 'NT', 'DEG', 'NN', 'VV', 'NN', 'NN'], ['AD', 'NR', 'P', 'NN', 'LC', 'JJ', 'P', 'NN', 'AD', 'AD', 'JJ'], ['AD', 'VC', 'DT', 'NN', 'VV', 'DEC', 'VV', 'DER', 'PN', 'NN', 'AD', 'VV'], ['NR', 'P', 'NN', 'NN', 'NN', 'VV', 'NN'], ['VV', 'NN', 'AD', 'VV', 'NR', 'VV', 'NR'], ['JJ', 'M', 'NN', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN', 'VV', 'NR', 'NN'], ['NT', 'VA', 'AS', 'AD', 'VV', 'VV', 'P', 'DT', 'AD', 'DEV', 'VV', 'SP'], ['NR', 'VV', 'NN', 'NN', 'AD', 'NR', 'AD', 'VV', 'PN'], ['VV', 'NN', 'DEC', 'NR', 'AD', 'AD', 'AD', 'VV', 'NR', 'P', 'NN', 'VV', 'NR'], ['AD', 'NN', 'CS', 'AD', 'CC', 'VE', 'NN', 'NN', 'AD', 'SB', 'VV', 'VV'], ['PN', 'VV', 'AS', 'NR', 'NN', 'VV', 'NT', 'NN', 'NN'], ['NN', 'NN', 'VC', 'VV', 'NN', 'LC', 'DEG', 'NN'], ['JJ', 'NN', 'VV', 'P', 'CD', 'CD', 'NT', 'LC', 'VV'], ['NR', 'NN', 'P', 'NN', 'AD', 'VV', 'AS', 'NR', 'NN'], ['VV', 'P', 'NN', 'VV', 'VV', 'DT', 'NN'], ['NN', 'AD', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV', 'M', 'VA'], ['AD', 'VV', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['JJ', 'NN', 'VV', 'DT', 'NN', 'P', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NR', 'NN'], ['NR', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN'], ['VA', 'DEC', 'NR', 'CC', 'NR', 'VV'], ['VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'P', 'NN', 'VV'], ['NN', 'VV', 'AD', 'VV'], ['NN', 'NN', 'VE', 'JJ', 'NN'], ['NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'NR', 'NN', 'VV', 'NN', 'AD', 'VC', 'NN', 'NN'], ['AD', 'NR', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['P', 'NR', 'VV', 'DEC', 'NN', 'LC', 'PN', 'DT', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NT', 'NN', 'NR', 'VV', 'VE', 'CD', 'NN'], ['NN', 'LC', 'AD', 'VV', 'AS', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'NR', 'VV', 'PN', 'VV', 'AS', 'NN'], ['PN', 'VV', 'VV', 'JJ', 'NN', 'CC', 'VV'], ['VV', 'NT', 'VV', 'NR', 'VV', 'VV', 'DEC', 'VC', 'PN', 'NN', 'NR'], ['DT', 'NN', 'VE', 'CD', 'VV', 'AD'], ['JJ', 'NN', 'NN', 'NN'], ['PN', 'VV', 'P', 'NN', 'DEG', 'NN', 'NN', 'MSP', 'VV', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'AS', 'NR', 'BA', 'VV', 'DEC', 'NR', 'VV', 'P', 'NN', 'VA', 'VV'], ['NR', 'ETC', 'NR', 'NN', 'NN', 'NR', 'NN', 'NN', 'VV', 'AS', 'NN'], ['NN', 'AD', 'CD', 'M', 'NN', 'AD', 'AD', 'VA', 'DEV', 'VV', 'NN', 'SP'], ['NR', 'JJ', 'NN', 'NN', 'P', 'VV'], ['CD', 'NN', 'PN', 'VV', 'NN', 'AD', 'CD', 'CD', 'CD', 'CD', 'M'], ['PN', 'AD', 'VV', 'P', 'PN', 'VV', 'CD', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'VV', 'AS', 'VV', 'VV', 'NN', 'VV', 'SP'], ['NR', 'JJ', 'NN', 'NN', 'VE', 'VV', 'DEC', 'NN', 'VV'], ['VV', 'NR', 'NN', 'NN', 'AD', 'VV', 'VV'], ['PN', 'VV', 'VV', 'NN', 'NN', 'VV', 'AS'], ['NN', 'JJ', 'NN', 'LC', 'NN', 'VV'], ['NN', 'NN', 'VV', 'NN', 'CC', 'JJ', 'CD', 'VV', 'NT', 'NN', 'LC', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VV', 'VV', 'AD', 'PN', 'VV', 'NR', 'DEG', 'NN', 'CC', 'NN', 'VV'], ['AD', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NN', 'NN', 'VV', 'AD', 'DT', 'NN', 'LB', 'NN', 'VV', 'VV', 'NN', 'VV', 'AD'], ['PN', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN'], ['NN', 'AD', 'AD', 'AD', 'AD', 'VV', 'VV', 'P', 'NN', 'LC', 'VV'], ['AD', 'P', 'NN', 'DEG', 'NN', 'CC', 'NN', 'LC', 'AD', 'VV', 'VV'], ['NN', 'VV', 'CD', 'CD', 'CD', 'CD', 'NN', 'NN', 'VC', 'NT', 'CD'], ['NT', 'NN', 'AD', 'VE', 'VV', 'NR', 'NN', 'VE', 'JJ', 'NN'], ['VV', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NT'], ['NT', 'DEG', 'NR', 'NN', 'NR', 'NN', 'VV', 'M', 'LC', 'VV'], ['NN', 'NN', 'AD', 'AD', 'P', 'NT', 'VV'], ['PN', 'NT', 'AD', 'P', 'NN', 'VA', 'SP', 'DT', 'NN', 'VV', 'DEC', 'PN', 'VV'], ['VE', 'NN', 'NN', 'DEG', 'NN', 'PN', 'VC', 'NN', 'VV', 'VV', 'MSP', 'NN', 'AD', 'VV', 'NN'], ['VV', 'AD', 'VV', 'VV', 'VA', 'PN'], ['NN', 'JJ', 'NN', 'VV', 'VV', 'NN'], ['NR', 'NR', 'CC', 'JJ', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NR'], ['PN', 'VV', 'VV', 'MSP', 'VV'], ['NN', 'NN', 'CC', 'JJ', 'NN', 'AD', 'P', 'NR', 'NN', 'NN', 'VV'], ['JJ', 'NN', 'VC', 'NN', 'NN', 'VA', 'NN', 'NN', 'NN', 'NN'], ['P', 'NN', 'VV', 'NT', 'VE', 'VV', 'DEC', 'NN', 'VV', 'VV', 'NN', 'VV', 'SP'], ['AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'VV', 'NN', 'CC', 'VV', 'NN', 'NN'], ['AD', 'AD', 'VV', 'JJ', 'NN'], ['VV', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VE', 'AD', 'VA'], ['AD', 'VE', 'NN', 'P', 'PN', 'AD', 'VV', 'JJ', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'AD', 'VV', 'P', 'SB', 'NR', 'VV', 'LC', 'DEG', 'NR', 'LC'], ['JJ', 'NN', 'NN', 'NN', 'NR', 'VV', 'NN'], ['NN', 'VV', 'AD', 'VV', 'NN', 'CC', 'NN', 'NN'], ['NN', 'CC', 'NR', 'NR', 'DEG', 'NN'], ['AD', 'NN', 'VA', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'LC', 'VV', 'NN', 'AD', 'VV', 'AS', 'VV', 'JJ', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['NT', 'VV', 'NR', 'AD', 'VV', 'NN'], ['AD', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV'], ['VV', 'CD', 'NN', 'VV', 'VV', 'NN'], ['P', 'NN', 'VV', 'LC', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'AD', 'NN', 'CD'], ['AD', 'NN', 'AD', 'VV'], ['NR', 'NN', 'VV', 'OD', 'NN', 'JJ', 'NN', 'NN', 'CD', 'CD', 'M'], ['PN', 'VV', 'VV', 'SP'], ['VV', 'DT', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'CC', 'NN', 'LC', 'VV', 'NN'], ['NT', 'NN', 'NN', 'NR', 'VV', 'NN', 'NN', 'NN', 'P', 'NN', 'VV'], ['NN', 'AD', 'VV', 'AD', 'BA', 'DT', 'NN', 'NN', 'VA'], ['NR', 'NN', 'P', 'NN', 'VV', 'AS', 'NN'], ['NT', 'VV', 'P', 'NN', 'LC', 'VV', 'DEC', 'VV', 'NN', 'VV'], ['PN', 'P', 'NR', 'NR', 'VV', 'NR', 'NN', 'VV', 'NN'], ['AD', 'NR', 'AD', 'P', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NR', 'NT', 'VV', 'CD', 'M', 'VV', 'VV', 'VV'], ['AD', 'NR', 'NN', 'AD', 'VA', 'AS', 'CD'], ['NR', 'NN', 'AD', 'VV', 'NR'], ['AD', 'NN', 'AD', 'VV', 'AD', 'AD', 'VA'], ['PN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['VV', 'LC', 'DEG', 'NR', 'JJ', 'NN', 'AD', 'AD', 'VA'], ['NR', 'AD', 'AD', 'CD', 'NR', 'NR'], ['NR', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NR', 'VV', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NR', 'P', 'NN', 'M', 'NN', 'VA', 'PN', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'NN', 'NN'], ['VV', 'VV', 'VV', 'DEC', 'VE', 'NN', 'VV'], ['AD', 'NN', 'AD', 'SB', 'VV', 'NN'], ['PN', 'VA', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN', 'LC'], ['NN', 'AD', 'VV', 'VV', 'AD', 'SB', 'VV', 'VV', 'SP'], ['PN', 'VV', 'P', 'NN', 'LC', 'VV', 'NN', 'LC', 'DEG', 'NN'], ['NR', 'NN', 'VV', 'AS', 'VV', 'NR', 'NN', 'CC', 'VV', 'NN'], ['NN', 'AD', 'AD', 'SB', 'VV', 'NR', 'NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['AD', 'NR', 'AD', 'AD', 'VV', 'VV', 'P', 'NN', 'VC', 'P', 'NN', 'VV'], ['NN', 'VV', 'NT', 'VV', 'DER', 'VA', 'SP'], ['NN', 'NR', 'NN', 'NN', 'NN'], ['VV', 'NT', 'VA', 'DEC', 'NN', 'AD', 'VV', 'MSP', 'NT', 'DEG', 'VA', 'VA', 'DEC', 'NN'], ['NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'AS', 'CD', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'VA', 'VA'], ['NR', 'CC', 'NR', 'DEG', 'NN', 'VA', 'AD', 'CD', 'NN', 'CD', 'NN'], ['NT', 'NR', 'NN', 'NN', 'NN', 'NN', 'NR', 'VV', 'PN', 'NN', 'NR', 'SB', 'VV', 'VV'], ['P', 'CD', 'NN', 'NN', 'VV', 'NN', 'VA', 'LC', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN', 'NN'], ['P', 'P', 'NN', 'VV', 'VV', 'LC', 'NR', 'AD', 'VV', 'AS', 'NN', 'NN'], ['P', 'DT', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'DEC', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'CD', 'NN', 'DEG', 'NN'], ['NN', 'NR', 'CC', 'NR', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'NR'], ['NR', 'NN', 'NR'], ['AD', 'VV', 'NN', 'VV', 'NN', 'LC', 'BA', 'NN', 'VV', 'AS', 'NN'], ['NN', 'VV', 'DEC', 'NR', 'NN', 'NN', 'NN', 'NN'], ['PN', 'VV', 'NN', 'SP', 'AD', 'VV', 'VV', 'NN'], ['AD', 'NN', 'AD', 'P', 'NN', 'VV', 'AD', 'NN'], ['AD', 'CS', 'VA', 'DEV', 'P', 'NN', 'VV', 'AD', 'VV', 'NN'], ['AD', 'VV', 'PN', 'PN', 'CD', 'VV', 'PN', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV'], ['NN', 'VV', 'PN', 'VV', 'NN', 'VV', 'NN', 'VV', 'P', 'NN', 'VV', 'VV'], ['PN', 'DEG', 'NN', 'VV'], ['PN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'DEC', 'CC', 'NR'], ['AD', 'VV', 'CD', 'VA', 'NN', 'LC', 'VV', 'VV', 'AS', 'NR', 'DEG', 'NN'], ['NN', 'VV', 'CD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NN', 'LC', 'NR', 'NR', 'NN', 'CC', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'NR', 'AD', 'P', 'NT', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['NN', 'NN', 'NN', 'JJ', 'JJ', 'NN', 'VV', 'NN', 'NN', 'NR', 'DEG', 'NN'], ['PN', 'P', 'DT', 'NN', 'AD', 'VV', 'AD', 'VV'], ['NR', 'NN', 'VV', 'PN', 'P', 'JJ', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'VA', 'AS', 'CD'], ['NT', 'PN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'AD', 'LB', 'NN', 'VV', 'AS'], ['VV', 'NN', 'NN', 'CC', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'P', 'VV', 'NN', 'P', 'NN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['NT', 'NT', 'VV', 'AS', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'SP'], ['P', 'NT', 'NN', 'DEG', 'NN', 'NN', 'LC', 'DT', 'JJ', 'NT', 'AD', 'AD', 'AD', 'VV'], ['AD', 'VV', 'AD', 'VV', 'SP'], ['NN', 'NN', 'AD', 'VV'], ['NR', 'VV', 'DT', 'NN', 'VV', 'JJ', 'NR', 'NN', 'NN'], ['NT', 'NT', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'PN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NN', 'VV', 'NN', 'NN', 'VV'], ['NN', 'NN', 'DT', 'NN', 'AD', 'VE', 'NN', 'SP'], ['NN', 'VV', 'AD', 'VC', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'VV', 'PN'], ['JJ', 'NN', 'CC', 'NN', 'AD', 'VC', 'VA'], ['NR', 'ETC', 'NN', 'AD', 'VV', 'NN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'P', 'NT', 'P', 'NR', 'NN', 'VV', 'NN', 'CC', 'NN'], ['AD', 'AD', 'VV', 'NN', 'NN', 'LC', 'VV', 'AD', 'VV', 'VV', 'AS', 'VV', 'PN', 'AD', 'VV', 'PN', 'DEG', 'AD', 'AD', 'SP'], ['JJ', 'NN', 'NR', 'P', 'NR', 'VV', 'NR', 'P', 'NN', 'AD', 'VV', 'NN'], ['NN', 'LC', 'NR', 'NN', 'VA', 'LC', 'AD', 'VV', 'BA', 'NN', 'VV', 'VV', 'AS'], ['AD', 'CS', 'P', 'NN', 'AD', 'NN', 'AD', 'VV', 'VV'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV', 'VV', 'VV', 'CC', 'AD', 'VA'], ['AD', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV'], ['DT', 'NN', 'AD', 'NR', 'VV'], ['NR', 'NN', 'NN', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['VV', 'NN', 'AD', 'VV', 'PN'], ['PN', 'VV', 'AD', 'NN', 'VV', 'NN', 'NN'], ['AD', 'AD', 'AD', 'VV', 'VV', 'LC', 'VV', 'AS'], ['VV', 'VV', 'AD', 'VV', 'CD', 'NN'], ['AD', 'PN', 'BA', 'NN', 'NN', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV', 'NN'], ['PN', 'NT', 'AD', 'VV', 'VV', 'PN', 'NN'], ['AD', 'VV', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['PN', 'NN', 'VV', 'VA', 'VV', 'BA', 'NN', 'VV', 'VV', 'AD', 'AD', 'VV'], ['P', 'NN', 'NN', 'NR', 'DEG', 'NN', 'NN', 'NN', 'VC', 'VV', 'AD', 'AD', 'VV', 'AS', 'NN'], ['VV', 'DEC', 'NN', 'ETC', 'JJ', 'NN'], ['NN', 'VC', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'NT', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NR', 'CC', 'NN', 'NN', 'NR', 'P', 'NN', 'VV', 'PN'], ['PN', 'AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'VE', 'NN', 'SP'], ['NR', 'NT', 'P', 'NR', 'VV', 'NN', 'NN'], ['NR', 'DEG', 'NN', 'VC', 'VV', 'NR', 'NN', 'VV', 'NN'], ['AD', 'NN', 'AD', 'VV', 'P', 'NN', 'NN', 'VV', 'NN'], ['NR', 'VV', 'VV', 'NR', 'NN', 'PN', 'VV', 'SP', 'AD', 'NN', 'SP'], ['NN', 'NN', 'NN', 'VC', 'NR', 'CC', 'NR', 'AD', 'LC', 'P', 'VV', 'DEC', 'AD', 'NN', 'NN'], ['VV', 'JJ', 'NN', 'NN', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'AD', 'VV'], ['AD', 'VC', 'NN', 'SP'], ['NN', 'AD', 'VV', 'SP'], ['AD', 'AD', 'VV', 'SP'], ['VV', 'VA', 'VA', 'SP'], ['NR', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AS', 'NN'], ['CD', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'LC', 'CD', 'NN'], ['VV', 'AD', 'VV', 'VV', 'DEC', 'NN', 'CC', 'VV', 'DEC', 'NN'], ['NN', 'NR', 'AD', 'VV', 'VV', 'NR', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'AD', 'VE', 'CD', 'VV', 'NN'], ['NN', 'CD', 'VV', 'NN', 'AD', 'VV', 'PN'], ['AD', 'NN', 'VV', 'NN', 'VV', 'AS', 'JJ', 'NN', 'VA'], ['NN', 'NR', 'AD', 'P', 'NN', 'VV'], ['NR', 'CC', 'NR', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VA', 'NN', 'AD', 'VV', 'VA'], ['NN', 'NN', 'AD', 'NR', 'VV', 'AD', 'NN'], ['AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'AD', 'VA', 'DEC', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VA', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VV', 'DER', 'VA'], ['PN', 'VV', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'CC', 'NN', 'VV', 'NN', 'NN', 'VV', 'AD', 'VV', 'AS', 'NN'], ['NN', 'VV', 'AD', 'AD', 'VA', 'VA'], ['AD', 'AD', 'JJ', 'NN', 'VV'], ['CS', 'PN', 'VC', 'VV', 'NN', 'SP'], ['PN', 'VV', 'VV', 'PN', 'NN', 'DEG', 'NN', 'SP'], ['PN', 'DEG', 'NN', 'VV', 'VV', 'NN'], ['AD', 'AD', 'VV'], ['P', 'JJ', 'NN', 'LC', 'VV', 'P', 'NR', 'DEG', 'NN', 'VV', 'VV'], ['AD', 'AD', 'LB', 'AD', 'VV'], ['AD', 'NR', 'NN', 'VV', 'LC', 'JJ', 'NN', 'DEG', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'MSP', 'VV', 'VV', 'NN', 'SP'], ['NR', 'AD', 'VA', 'AD', 'VV', 'VV'], ['NT', 'NN', 'NN', 'VV', 'NN', 'OD', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['DT', 'NN', 'P', 'NN', 'VV', 'AD', 'VA'], ['NN', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV'], ['VV', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'VC', 'VA', 'SP'], ['AD', 'NR', 'AD', 'AD', 'AD', 'VV', 'NN'], ['NN', 'NN', 'DT', 'DEG', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'DT', 'NN', 'VV'], ['VV', 'NR', 'NN', 'NR', 'VV', 'AD', 'VV', 'NR', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'P', 'NN'], ['PN', 'VC', 'PN', 'DEG', 'JJ', 'SP', 'NN'], ['PN', 'AD', 'AD', 'P', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['NN', 'AD', 'VV', 'DT', 'NN', 'DEG', 'JJ', 'NN', 'NN'], ['AD', 'DT', 'NN', 'AD', 'VC', 'JJ', 'NN'], ['NR', 'VV', 'NN', 'AD', 'VA'], ['AD', 'NR', 'VV', 'AS', 'VV', 'VA'], ['NN', 'VV', 'BA', 'NN', 'VV'], ['AD', 'NN', 'NN', 'NN', 'VV', 'NR', 'AD', 'VV', 'VV'], ['PN', 'VV', 'M', 'NN', 'NN', 'VV', 'NR', 'PN', 'VV', 'NR'], ['PN', 'DEG', 'NN', 'VC', 'OD', 'NN', 'NN', 'VV'], ['AD', 'VV', 'NN', 'AD', 'VA', 'DEC', 'JJ', 'NN'], ['P', 'NN', 'VV', 'P', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'VV', 'NR', 'NN', 'LC'], ['NN', 'VV', 'NN', 'NN', 'NN', 'DEC', 'NN', 'VA', 'AD', 'VV', 'VA'], ['VV', 'VV', 'PN', 'DEC', 'NN', 'NR', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['NR', 'VC', 'NR', 'JJ', 'NN', 'AD', 'VV', 'NN', 'NR', 'NN', 'LC'], ['AD', 'VV', 'BA', 'NN', 'VV', 'NN', 'NN'], ['P', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'VV', 'DT', 'NN', 'LC', 'DEG', 'NN', 'SP'], ['NN', 'AD', 'VV', 'NN', 'LC', 'VV', 'AS'], ['NN', 'VV', 'VV', 'DEC', 'NR', 'NN', 'VV', 'NN', 'VA', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'AS', 'CD', 'M', 'VV', 'VV', 'SP'], ['AD', 'AD', 'PN', 'VV', 'VV', 'NN', 'NN', 'CC', 'CD', 'NN'], ['CS', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'JJ', 'DEG', 'NR', 'SP'], ['DT', 'NN', 'P', 'JJ', 'NN', 'AD', 'VV', 'NR', 'P', 'NR', 'VV', 'NN'], ['NR', 'AD', 'VV', 'NR', 'VV', 'DEC', 'NN', 'NN'], ['NT', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'JJ', 'NN', 'VV', 'NN'], ['AD', 'AD', 'PN', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NN', 'VV', 'JJ', 'NN'], ['PN', 'VV', 'CD', 'NN', 'PN', 'AD', 'VV', 'CD', 'NN'], ['NN', 'VV', 'NN', 'VV', 'VV'], ['DT', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'VV', 'VV', 'VV', 'NR', 'NR', 'NR', 'ETC'], ['PN', 'VV', 'CD', 'NN', 'VV', 'VV', 'PN', 'VV', 'SP'], ['P', 'P', 'NN', 'VV', 'NN', 'NN', 'AD', 'P', 'NN', 'NN'], ['AD', 'NN', 'VV', 'VV', 'NN', 'JJ', 'NN', 'LC', 'NN', 'AD', 'P', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'NN', 'NN', 'VA'], ['AD', 'NR', 'VV', 'VV', 'NN', 'AD', 'NR', 'NN', 'NR', 'DEG', 'NN', 'SP'], ['NN', 'CC', 'NN', 'VE', 'DT', 'JJ', 'DEG', 'NN', 'CC', 'NN', 'SP'], ['NN', 'LC', 'DEG', 'NN', 'VV', 'AD', 'VV', 'NN', 'P', 'NN', 'VV'], ['PN', 'VV', 'BA', 'PN', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NR', 'AD', 'AD', 'VV', 'VV', 'NN', 'VC', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'AS', 'AD'], ['AD', 'NT', 'AD', 'VV', 'NN', 'NN', 'NT', 'JJ', 'NN', 'VC', 'VV', 'NR'], ['AD', 'PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'SP'], ['NR', 'NN', 'NN', 'AD', 'VV', 'NR', 'DEG', 'NN', 'VV', 'DEC', 'NN'], ['DT', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['NN', 'AD', 'VV', 'NN', 'VV'], ['NT', 'LC', 'NR', 'NN', 'NN', 'NN', 'NR', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'NR', 'VC', 'VV', 'SP'], ['NN', 'NR', 'NN', 'NN', 'NN'], ['AD', 'NN', 'VV', 'NN', 'JJ', 'NN', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'P', 'JJ', 'NN', 'DT', 'NN', 'AD', 'VV'], ['P', 'PN', 'P', 'NN', 'DEG', 'NN', 'VV', 'NN', 'MSP', 'VV', 'PN', 'VV', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VE', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'VA', 'DEC', 'NN', 'LC', 'VV', 'NN', 'NN'], ['NN', 'NN', 'CC', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'VV', 'AS', 'CD'], ['PN', 'VV', 'NR', 'DT', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'AS', 'NN', 'AD', 'AD', 'VV', 'VV', 'AS'], ['NN', 'CC', 'PN', 'AD', 'VV', 'NN', 'DT', 'NN', 'VA', 'AD', 'VV', 'SP'], ['NN', 'NR', 'P', 'NN', 'AD', 'VV', 'PN', 'NN', 'NR'], ['VV', 'VV', 'NR', 'VV', 'P', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VC', 'VV', 'CC', 'VV', 'NN', 'DEC', 'JJ', 'NN'], ['NN', 'VV', 'AD', 'P', 'JJ', 'DEG', 'NN', 'CC', 'NN', 'MSP', 'VV'], ['JJ', 'NN', 'AD', 'VV', 'NN', 'VV', 'AS'], ['AD', 'VV', 'NN', 'VV', 'VV', 'VV'], ['NR', 'P', 'VV', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'LC', 'P', 'NN', 'NN', 'VV', 'NR', 'NN'], ['NR', 'VA', 'NT', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['P', 'NN', 'VV', 'VV', 'NR', 'NN', 'NT', 'AD', 'VV', 'VV'], ['AD', 'NR', 'CC', 'NN', 'ETC', 'NN', 'P', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'LC'], ['NN', 'VV', 'PN', 'DEG', 'NN', 'VV', 'AS'], ['PN', 'VV', 'VV', 'NN', 'P', 'NN', 'VV'], ['AD', 'VV', 'PN', 'VV', 'PN', 'AD', 'VV', 'NN', 'VA', 'NN'], ['VV', 'DT', 'VV', 'NN', 'DEC', 'NN', 'VE', 'DT', 'NN'], ['NR', 'NN', 'NN', 'NN', 'LC', 'AD', 'VV', 'NN'], ['NN', 'LC', 'NR', 'P', 'CD', 'CD', 'CD', 'NT', 'VV', 'NR'], ['NN', 'VV', 'VV', 'LC', 'VV', 'NN', 'LC', 'VE', 'NN', 'CC', 'NN'], ['DT', 'M', 'NN', 'VV', 'VA', 'AS', 'NN', 'VV', 'AS', 'NN'], ['AD', 'NR', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NR', 'VV', 'AD', 'VV', 'NN'], ['NT', 'JJ', 'NN', 'VV', 'VV', 'VE', 'CD', 'VA'], ['NN', 'NR', 'NR', 'VV', 'NR', 'VA', 'VA', 'VC', 'PN', 'NN', 'LC', 'DEG', 'NN'], ['NN', 'NN', 'NN', 'VV'], ['AD', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'JJ', 'NR', 'NN', 'VV', 'NR', 'VV', 'NN', 'NN'], ['AD', 'VV', 'NN', 'NN'], ['AD', 'VA', 'DEC', 'NR', 'NN', 'AD', 'AD', 'AD', 'VV', 'P', 'NR', 'NN', 'VV', 'PN', 'NN'], ['AD', 'JJ', 'NN', 'DEG', 'NN', 'LB', 'NN', 'VV', 'VV', 'AS', 'AD', 'AD', 'VV', 'AD', 'AD', 'AD', 'VV'], ['NR', 'NN', 'VV', 'AS', 'VV'], ['NR', 'NN', 'VA', 'VV', 'NN', 'AD', 'VA'], ['NR', 'AD', 'VV', 'NR'], ['PN', 'AD', 'P', 'NR', 'NN', 'VV', 'AS'], ['AD', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NR', 'NR', 'NN', 'NR', 'NN', 'NN'], ['NN', 'CC', 'NN', 'P', 'NN', 'VV', 'NR'], ['VV', 'DEC', 'NN', 'AD', 'AD', 'AD', 'CD'], ['NN', 'CD', 'VV', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'SP'], ['VV', 'VV', 'AD', 'VV', 'DEC', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'LC', 'AD', 'VE', 'CD', 'DT', 'NN', 'AD', 'VV', 'CD', 'AD', 'AD', 'VV', 'VV'], ['AD', 'AD', 'NR', 'VV', 'AS', 'NN', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN'], ['P', 'NN', 'NN', 'DEG', 'NN', 'LC', 'DT', 'NN', 'VV', 'NN', 'NN'], ['JJ', 'CD', 'NT', 'VV', 'NR', 'VV', 'LC', 'NN', 'PN', 'P', 'NN', 'VV'], ['VV', 'NN', 'NN', 'NN', 'AD', 'VE', 'NN', 'VV', 'NN'], ['P', 'NN', 'MSP', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NT', 'NR', 'NN', 'VV', 'NN', 'AD', 'P', 'NT', 'VV', 'NN', 'JJ', 'NN'], ['NN', 'PN', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN', 'SP', 'P', 'VV', 'VV', 'VV', 'NN', 'NN', 'SP'], ['NN', 'NN', 'NN', 'P', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'VV'], ['PN', 'VV', 'VV', 'AS', 'CD', 'VV', 'DEC', 'NN', 'AD', 'DEV', 'VV', 'AS'], ['NR', 'VA', 'DEV', 'VV', 'NN', 'AD', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'CC', 'VV', 'NN', 'NN'], ['DT', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'NN', 'CC', 'CD', 'JJ', 'DEG', 'NN'], ['AD', 'NR', 'NR', 'NN', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'P', 'NR', 'CD', 'JJ', 'NN', 'DEG', 'NR'], ['NT', 'NR', 'NN', 'NN', 'NN', 'AD', 'CC', 'NN', 'NN', 'SP'], ['NT', 'VE', 'NN', 'NN', 'NT', 'AD', 'VE', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VE', 'NN', 'VV', 'AS', 'P', 'PN', 'VV', 'NR', 'NN', 'VV', 'NN', 'AD', 'PN'], ['NR', 'NN', 'VV', 'VV', 'PN', 'VV'], ['NN', 'NN', 'AD', 'P', 'PN', 'VV', 'AS', 'CD', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'AD', 'VA', 'VA', 'VA', 'AD', 'P', 'VV', 'NR', 'DEG', 'NN'], ['VV', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'P', 'NN', 'VV', 'VV', 'VV', 'NN'], ['JJ', 'NN', 'NR', 'AD', 'AD', 'P', 'NT', 'NT', 'VV', 'NR'], ['CS', 'VE', 'NN', 'NN', 'AD', 'AD', 'VA', 'SP'], ['NT', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VE', 'NN', 'VV', 'VV', 'DT', 'NN', 'DEG', 'NN', 'LB', 'NN', 'VV', 'VV'], ['PN', 'AD', 'VC', 'VV', 'VA', 'AD', 'DEC', 'AD', 'JJ', 'AD', 'VA', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VE', 'PN'], ['NR', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'CS', 'AD', 'VV', 'NR', 'AD', 'VV', 'NR', 'NN', 'NN'], ['JJ', 'NN', 'NN', 'NN'], ['P', 'NN', 'NN', 'NR', 'DEG', 'JJ', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'CC', 'NN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VV', 'CC', 'VV', 'VV'], ['NN', 'NR', 'VV', 'DT', 'AD', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['PN', 'VV', 'VV', 'NN', 'VV', 'AS', 'NR'], ['AD', 'VV', 'JJ', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'AS', 'NN', 'LC', 'PN', 'AD', 'P', 'NN', 'NN', 'VV', 'P', 'PN', 'AD', 'NN', 'AD', 'VE'], ['AD', 'DT', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'AD', 'P', 'VA', 'AD', 'VA', 'VA', 'VA'], ['AD', 'PN', 'AD', 'VV', 'P', 'NN', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['JJ', 'NN', 'NN', 'NN', 'LC', 'CD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'LC'], ['VV', 'VV', 'SP', 'PN', 'VC', 'M', 'NN', 'SP'], ['NR', 'NN', 'CC', 'NN', 'AD', 'VV', 'NN'], ['AD', 'PN', 'SB', 'VV', 'VV', 'VV', 'NR', 'DEC', 'JJ', 'NN', 'NN', 'MSP'], ['VV', 'VV', 'VV', 'VV', 'PN', 'VA', 'PN', 'AD', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'P', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VV', 'AD', 'VC', 'P', 'VV', 'NN', 'AD', 'P', 'NN', 'LC', 'DEG', 'NN'], ['NR', 'NR', 'VV', 'NR', 'CD', 'NN'], ['AD', 'VV', 'PN', 'NN', 'AD', 'SP'], ['PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NR', 'DEG', 'JJ', 'NN'], ['NN', 'P', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VA'], ['NR', 'NN', 'AD', 'VE', 'CD', 'VV'], ['AD', 'AD', 'VV', 'DEC', 'NR', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'P', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'JJ', 'NN', 'AD', 'VA'], ['AD', 'AD', 'VV', 'VV', 'VV', 'VV', 'BA', 'NR', 'CC', 'NR', 'DEG', 'NN', 'VV', 'PN', 'NN'], ['NT', 'NT', 'NR', 'NN', 'NN', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VV', 'AS', 'SP'], ['NN', 'VE', 'NN', 'NN', 'VV'], ['CD', 'NN', 'VV', 'LC', 'AD', 'AD', 'AD', 'VV'], ['NR', 'NR', 'NN', 'VV', 'VV', 'NR', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'AD', 'P', 'NN', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VV', 'VV'], ['VV', 'NN', 'NN', 'VV', 'AS', 'CD'], ['NN', 'VV', 'P', 'NN', 'LC', 'VV', 'AD', 'VV', 'AS'], ['PN', 'VV', 'NR', 'AD', 'VV'], ['NR', 'VV', 'NN', 'PN', 'VV', 'VV', 'AS', 'NN'], ['NR', 'NN', 'NR', 'VV', 'P', 'NN', 'LC', 'VV', 'NN', 'VV', 'NR', 'VV'], ['PN', 'VV', 'NR', 'NN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN'], ['AD', 'VV', 'NN', 'CC', 'JJ', 'NN', 'NN', 'VV', 'CC', 'VV', 'VV'], ['VC', 'PN', 'DEG', 'NN', 'PN', 'AD', 'VV'], ['AD', 'NR', 'NN', 'NN', 'AD', 'AD', 'VV', 'P', 'NR', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'AS', 'PN', 'VV', 'AS', 'VV'], ['VE', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN', 'DEG', 'NR', 'NR', 'NR', 'VC', 'NN', 'NN'], ['VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'NN', 'DEG', 'NN', 'VV', 'NN', 'NN'], ['P', 'VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN'], ['AD', 'NN', 'AD', 'CC', 'AD', 'VV'], ['PN', 'AD', 'NR', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'VV', 'VA', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV', 'AD'], ['NN', 'NR', 'NR', 'AD', 'VV', 'AD', 'VV', 'VV', 'VE', 'AD', 'VA', 'DEC', 'NN'], ['PN', 'VV', 'PN', 'VV', 'NN', 'PN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VV', 'AS', 'JJ', 'NN', 'VV', 'CC', 'VV'], ['AD', 'VV', 'NR', 'DEG', 'NN', 'VV', 'VV', 'P', 'NR', 'VV', 'VV'], ['AD', 'NT', 'NN', 'AD', 'P', 'NN', 'VV', 'AD', 'P', 'NN'], ['AD', 'VV', 'P', 'NN', 'LC', 'VV', 'NN'], ['JJ', 'CD', 'CD', 'CD', 'NT', 'NR', 'AD', 'VV', 'NR', 'VV', 'NN'], ['NT', 'NN', 'NN', 'AD', 'P', 'NN', 'VV'], ['AD', 'P', 'NN', 'VV', 'NN'], ['VV', 'AD', 'VV', 'NN', 'SP'], ['PN', 'VV', 'NT', 'VC', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NN', 'DT', 'NN', 'NN', 'P', 'NN', 'VV', 'AS', 'NN', 'NN'], ['AD', 'VC', 'NN', 'AD', 'VV', 'AS', 'JJ', 'AD', 'VV', 'AS', 'NN', 'AD', 'AD', 'VA', 'SP'], ['AD', 'PN', 'VV', 'AS', 'NR', 'NN', 'NN', 'LC', 'NR', 'JJ', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'VV', 'NN', 'VV', 'NN'], ['AD', 'NR', 'OD', 'NN', 'CD', 'VV', 'DEC', 'NN', 'VC', 'VV', 'VV', 'PN', 'AD', 'VV'], ['AD', 'NN', 'VV', 'AD', 'AD', 'VV', 'CD', 'CD', 'CC', 'CD', 'AD', 'CD', 'CD', 'M'], ['CD', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VA', 'AD', 'VA', 'NN'], ['NN', 'AD', 'VV', 'OD', 'CC', 'PN', 'VV', 'DEC', 'NN', 'VV', 'NR', 'AD', 'VV'], ['NR', 'VV', 'LC', 'NN', 'VE', 'NN', 'NN', 'LC', 'AD', 'VV', 'VV', 'VV', 'PN', 'NN'], ['NN', 'LC', 'AD', 'VE', 'CD', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'JJ', 'NN', 'VA'], ['AD', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'JJ', 'NN', 'VV'], ['P', 'PN', 'CD', 'NN', 'SP'], ['VV', 'VV', 'AD', 'JJ', 'DEG', 'NN', 'VV', 'VV', 'VV', 'AD', 'VA', 'DEC', 'NN', 'JJ', 'NN', 'NN'], ['NR', 'NN', 'VC', 'NR', 'NN', 'P', 'NN', 'NN', 'NN', 'NN', 'CC', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'PN', 'DEG', 'NN', 'JJ', 'JJ', 'NN', 'SP'], ['AD', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'AD', 'VA'], ['NN', 'VV', 'PN', 'VV', 'SP', 'AD', 'VV', 'PN', 'AD', 'VV', 'NN', 'VV', 'PN', 'PN', 'AD', 'AD', 'VV', 'VV'], ['NN', 'LC', 'NN', 'AD', 'AD', 'VV', 'SB', 'NN', 'VV', 'NN'], ['P', 'NN', 'VV', 'NN', 'VE', 'NN', 'NN', 'CC', 'NN', 'NN'], ['NN', 'NN', 'NN', 'NN', 'AD', 'BA', 'PN', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'NN', 'VV', 'PN', 'VA'], ['NN', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NN', 'DEG', 'NN', 'VV', 'NR', 'VV', 'NR', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NN', 'CC', 'NN', 'CD', 'CD', 'NT', 'NN', 'NN', 'NN', 'VV'], ['PN', 'VV', 'VV', 'PN', 'AD', 'VV', 'SP'], ['NN', 'AD', 'VV', 'VV', 'AD', 'PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'DEC', 'SP'], ['P', 'NN', 'VV', 'JJ', 'NN', 'LC', 'VV', 'NN', 'NN', 'NN'], ['AD', 'P', 'NN', 'VV', 'NN'], ['JJ', 'NT', 'DEG', 'NR', 'JJ', 'NR', 'AD', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NN', 'CD', 'NT', 'CD', 'M', 'CD', 'VV', 'VV'], ['NT', 'NR', 'NN', 'VV', 'NN'], ['PN', 'VV', 'NN', 'VV', 'AD', 'VV', 'DT', 'VV', 'DEC', 'NN', 'BA', 'NN', 'VV', 'P', 'NN'], ['PN', 'AD', 'AD', 'VV', 'AS', 'NN', 'VV', 'AD', 'JJ', 'NN', 'SP'], ['NN', 'NR', 'AD', 'VV', 'AD', 'JJ', 'NN', 'DEG', 'JJ', 'NN'], ['DT', 'JJ', 'NN', 'VV'], ['NR', 'P', 'AD', 'VV', 'AD', 'AD', 'VA'], ['NN', 'VV', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'NN', 'NN', 'VA', 'DEV', 'VV', 'PN', 'VV'], ['PN', 'AD', 'JJ', 'AD', 'JJ', 'AD', 'JJ', 'SP'], ['NN', 'NN', 'VV', 'VV', 'NR', 'NN', 'NR', 'VV', 'NR', 'NN', 'VV'], ['VV', 'NN', 'SP', 'VV', 'P', 'NN', 'VV'], ['CD', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VV', 'PN', 'AD', 'VV', 'NN', 'AD', 'P', 'NN', 'VV', 'VV'], ['NT', 'NR', 'AD', 'VV', 'NN'], ['NN', 'AD', 'VV'], ['NR', 'NN', 'DEG', 'NN'], ['AD', 'NR', 'P', 'NN', 'AD', 'CC', 'VV', 'AS'], ['VV', 'NR', 'LC', 'CC', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'VV'], ['AD', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'NT'], ['JJ', 'NN', 'AD', 'BA', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV'], ['NR', 'NN', 'AD', 'VC', 'PN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'PN', 'SP', 'VA', 'SP'], ['P', 'NR', 'VV', 'DEC', 'CD', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NT', 'NT', 'DT', 'CD', 'NN', 'AD', 'P', 'NN', 'DEG', 'JJ', 'NN', 'VV'], ['PN', 'DEG', 'NN', 'AD', 'VA'], ['NR', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV', 'AS', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'P', 'VA', 'DEC', 'NN', 'AD', 'P', 'NN', 'VV'], ['NN', 'NN', 'AD', 'P', 'NN', 'LC', 'P', 'PN', 'NN', 'VV'], ['AD', 'P', 'PN', 'NN', 'NN', 'NR', 'BA', 'PN', 'VV'], ['DT', 'NN', 'AD', 'VV', 'VV', 'PN', 'NN', 'SP'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'P', 'VV', 'NN', 'MSP', 'VV'], ['P', 'NR', 'VA', 'DEC', 'NN', 'NR', 'AD', 'VV', 'NN'], ['VV', 'NR', 'NN', 'NN', 'VV', 'DEC', 'NR', 'NN', 'NN', 'NN'], ['VV', 'DEC', 'NN', 'NN', 'AD', 'AD', 'SP'], ['JJ', 'DEG', 'NN', 'VV', 'NN'], ['AD', 'NR', 'AD', 'VC', 'NN', 'NR', 'DEG', 'CD', 'NN'], ['NN', 'NN', 'VA', 'NN', 'NN', 'VV'], ['VV', 'VV', 'VV', 'SP', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NT', 'NN', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN'], ['AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'NN', 'SP'], ['NR', 'NN', 'NN', 'VV', 'VV', 'NN', 'NR', 'VV'], ['VV', 'NN', 'VV', 'AD', 'NT'], ['NT', 'NN', 'DEG', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV'], ['VV', 'NR', 'DEC', 'NN', 'P', 'VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NN', 'CC', 'NT', 'VV', 'DEC', 'NN', 'AD', 'VV', 'SP'], ['VV', 'DEC', 'NN', 'SB', 'VV', 'DER', 'VV'], ['AD', 'NN', 'NN', 'CS', 'AD', 'VV', 'JJ', 'NN', 'VA', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV', 'AS'], ['NN', 'LC', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VE', 'DT'], ['NN', 'NN', 'AD', 'NN', 'AD', 'VA'], ['P', 'AD', 'VV', 'NR', 'P', 'CD', 'NN', 'LC', 'VV'], ['VV', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'JJ', 'NN', 'ETC', 'VV', 'CS'], ['AD', 'NN', 'AD', 'VV', 'P', 'PN', 'SP'], ['PN', 'AD', 'VV', 'PN', 'DT', 'PN', 'VC', 'NN', 'SP'], ['NN', 'NN', 'VC', 'AD', 'VV', 'SP'], ['NN', 'NN', 'AD', 'VV', 'P', 'NN', 'LC', 'P', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'VA', 'VA'], ['NN', 'LC', 'NN', 'VA', 'AD', 'VV', 'M', 'NN'], ['PN', 'AD', 'VC', 'AD', 'VV', 'NR', 'SP'], ['PN', 'VV', 'VV', 'PN', 'DEG', 'NN', 'NN', 'DEG', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN'], ['NT', 'PN', 'VV', 'VV', 'AD', 'VV'], ['PN', 'AD', 'VV', 'PN', 'VC', 'PN', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['P', 'VV', 'NN', 'NN', 'NN', 'NN', 'VA'], ['NN', 'NN', 'CC', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'VV', 'AD', 'VA', 'VV', 'VV', 'DEC', 'PN', 'VC', 'VV'], ['NN', 'NR', 'VV', 'PN', 'NN', 'VV'], ['NN', 'NT', 'LC', 'VV', 'AD', 'DEV', 'AD', 'NR', 'NN', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NR', 'P', 'NN', 'VV', 'AD'], ['VV', 'VV', 'M', 'NN', 'SP'], ['PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VC', 'PN'], ['NN', 'DEG', 'NN', 'VA', 'VA', 'VV', 'VV'], ['AD', 'VE', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'AD', 'VV'], ['AD', 'AD', 'VV', 'AD', 'NN', 'DEG', 'NN'], ['NN', 'BA', 'NR', 'VV', 'VV', 'DER', 'VA'], ['PN', 'AD', 'VV', 'VV', 'PN', 'DT', 'NN', 'VC', 'NN', 'NN'], ['NR', 'VV', 'JJ', 'NN', 'LC', 'VV', 'VV', 'NN', 'AD', 'VA'], ['NT', 'JJ', 'NN', 'CD', 'CC', 'CD'], ['JJ', 'NT', 'NN', 'AD', 'VV'], ['NR', 'NR', 'P', 'NR', 'NN'], ['NN', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV'], ['NR', 'DEG', 'NN', 'AD', 'BA', 'PN', 'VV'], ['AD', 'VV', 'JJ', 'NN', 'P', 'NN', 'AD', 'VV', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'PN', 'VV', 'NR', 'NN', 'VV', 'VV', 'PN'], ['AD', 'AD', 'AD', 'AD', 'CC', 'NT', 'LC', 'SP'], ['NN', 'NN', 'NN', 'LC', 'NN', 'VA', 'VA', 'DEC', 'NN', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NR', 'VE', 'CD', 'NN', 'AD', 'VA', 'VE', 'CD', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'P', 'PN', 'NN'], ['AD', 'JJ', 'NN', 'AD', 'VA'], ['VA', 'DEV', 'VV', 'CC', 'VV', 'AD', 'VV', 'MSP', 'VV', 'SP'], ['AD', 'NN', 'P', 'NN', 'CC', 'NN', 'VV'], ['NN', 'VV', 'AS', 'AD', 'VV', 'VV', 'PN', 'VV', 'NN'], ['PN', 'AD', 'VC', 'VV', 'NR', 'NN', 'NN', 'VV', 'VA', 'DEC', 'NN', 'NN', 'SP'], ['NN', 'NN', 'AD', 'VC', 'NN', 'NN'], ['NN', 'VV', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NR', 'VC', 'DT', 'NN', 'DEG', 'JJ', 'AD', 'P', 'NN', 'NN', 'VV', 'NN'], ['LB', 'NN', 'VV', 'VV', 'NN', 'SB', 'VV', 'DEC', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV'], ['NN', 'NR', 'AD', 'VV', 'PN', 'VV'], ['VV', 'DEC', 'NN', 'NN', 'LC', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['JJ', 'NN', 'AD', 'VA', 'VE', 'DT', 'NN'], ['NR', 'VV', 'VV', 'NN', 'VV', 'AD', 'VA'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VE', 'AS', 'NN', 'CC', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NN', 'VV', 'BA', 'NN', 'P', 'NN', 'VV'], ['AD', 'PN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'VV'], ['P', 'NN', 'VV', 'NN', 'LC', 'PN', 'AD', 'P', 'NN', 'NN', 'VV', 'VV', 'AS', 'PN', 'DEG', 'NN', 'NN'], ['VV', 'P', 'NN', 'VV', 'ETC', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['NN', 'AD', 'VC', 'DT', 'NN', 'LC', 'DEG', 'NN'], ['P', 'DT', 'AD', 'NN', 'VV', 'NN', 'NR', 'P', 'PN', 'VV'], ['P', 'NN', 'LC', 'NR', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NR', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'CC', 'VV', 'NN', 'CC', 'NN', 'NN'], ['VV', 'AD', 'VV', 'VV', 'VA'], ['PN', 'AD', 'AD', 'VV'], ['PN', 'DEG', 'NN', 'VC', 'P', 'AD', 'VV', 'VV', 'NN'], ['AD', 'AD', 'NR', 'VV', 'AS', 'JJ', 'NN', 'DEG', 'JJ', 'NN', 'P', 'NR', 'CD', 'NN', 'VV'], ['NR', 'VV', 'AS', 'NN', 'NN'], ['AD', 'P', 'NN', 'NN', 'VV'], ['AD', 'VV', 'PN', 'VV', 'AS', 'NN', 'VV'], ['PN', 'AD', 'AD', 'VV', 'PN', 'NN'], ['NR', 'SB', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['P', 'PN', 'VV', 'AD', 'VV', 'VV', 'SP'], ['NR', 'VV', 'VA', 'DEV', 'VA', 'VV', 'SP'], ['NN', 'PN', 'AD', 'AD', 'VV', 'SP'], ['NR', 'DEG', 'NN', 'AD', 'VA'], ['NT', 'NR', 'NN', 'NN', 'NN', 'VV', 'AD', 'NN'], ['NN', 'NN', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'AD', 'AD', 'BA', 'NN', 'VV'], ['NT', 'NR', 'NN', 'VV', 'AS', 'DEC', 'NN', 'NN', 'NR', 'VV', 'AD', 'VV'], ['NN', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN', 'P', 'NN'], ['AD', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'AD', 'NN', 'NN', 'AD', 'VV', 'VA', 'AD', 'PN', 'NN'], ['NN', 'VV', 'NT', 'LB', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NN', 'VV', 'DT', 'NN', 'VE', 'JJ', 'NN'], ['NN', 'P', 'VV', 'NN', 'AD', 'NN', 'VV', 'NN', 'AD', 'AD', 'VA', 'SP', 'PN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'AD', 'VV', 'NN', 'VV', 'CC', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NR', 'DEG', 'NN', 'CC', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'PN', 'AD', 'AD', 'VA'], ['AD', 'NR', 'NN', 'VV'], ['PN', 'VV', 'VV', 'AD', 'VV', 'NR', 'VV', 'AS', 'PN', 'NN', 'LC', 'NN', 'LC', 'DEG', 'M'], ['NN', 'NN', 'MSP', 'VV', 'P', 'NN', 'VV', 'P', 'PN', 'P', 'VV', 'NN', 'CD', 'NN'], ['AD', 'PN', 'NN', 'LC', 'DT', 'M', 'NN', 'NN', 'AD', 'VC', 'NN'], ['NR', 'DEG', 'NN', 'NR', 'AD', 'VV', 'NN', 'VV', 'VV', 'NR', 'VV'], ['AD', 'JJ', 'PN', 'P', 'VE', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV'], ['VA', 'DEC', 'PN', 'AD', 'LB', 'NN', 'DEG', 'NN', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['NR', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'LC', 'VV', 'VV', 'NN', 'IJ'], ['NR', 'NR', 'VV', 'VA', 'VV'], ['P', 'NR', 'NR', 'NN', 'ETC', 'JJ', 'NN', 'LC', 'NR', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'NR', 'NR', 'NR', 'NR', 'ETC', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NN', 'AD', 'NN', 'VV', 'AS', 'NN', 'AD', 'JJ', 'NN', 'VV', 'AS', 'CD', 'NN'], ['JJ', 'JJ', 'NN', 'P', 'NT', 'AD', 'VA'], ['NR', 'AD', 'VV', 'NN', 'VA'], ['AD', 'VV', 'SP', 'AD', 'AD', 'VA', 'VA', 'SP'], ['CD', 'NN', 'P', 'NN', 'DEC', 'NN', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'VV', 'AD', 'VA', 'DEC', 'JJ', 'NN', 'NN'], ['VV', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN', 'DEC', 'NR', 'NN'], ['NN', 'NN', 'VA', 'DEC', 'NR', 'AD', 'VV', 'NN', 'VV', 'NR', 'NN'], ['AD', 'AD', 'VA', 'SP', 'PN', 'VV', 'PN', 'CD', 'NN', 'PN', 'VV', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN', 'SP'], ['AD', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'AD', 'VC', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'BA', 'PN', 'VV', 'AS', 'AD', 'VV', 'VV', 'AD', 'VV'], ['NT', 'DEG', 'DT', 'CD', 'NN', 'P', 'NT', 'NT', 'CD', 'CD', 'CD', 'M'], ['DT', 'NN', 'NN', 'VV', 'VV'], ['VV', 'VV', 'NR', 'NR', 'LC', 'DEG', 'NN'], ['P', 'NN', 'DEG', 'NN', 'LC', 'NR', 'VV', 'AS', 'NN', 'VV', 'NR', 'DEG', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NR', 'AD', 'VV', 'P', 'NR', 'NR', 'NN', 'VV', 'VV', 'AS', 'JJ', 'NN'], ['VV', 'NN', 'VV', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'CC', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VA', 'DEC', 'NN'], ['CD', 'NT', 'NR', 'VV', 'VV', 'AD', 'VV', 'VV'], ['VV', 'AS', 'AD', 'VV', 'VV', 'NN'], ['CS', 'NR', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'NT', 'AD', 'LB', 'NN', 'NN', 'VV', 'VV'], ['NT', 'NT', 'PN', 'AD', 'BA', 'NN', 'VV', 'VV', 'AS', 'AD', 'NN', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN', 'SP'], ['PN', 'VV', 'M', 'AD', 'VA', 'VV', 'VV', 'NN'], ['PN', 'VC', 'NN', 'NN', 'PN', 'VV', 'NN', 'NN'], ['AD', 'NR', 'NN', 'NR', 'VV', 'AS'], ['NN', 'VA', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VV', 'PN', 'LB', 'NN', 'VV', 'AS'], ['PN', 'CC', 'NN', 'DT', 'NN', 'SP'], ['NN', 'NN', 'AD', 'VA', 'DEC', 'VV', 'AD', 'VV', 'VA'], ['NN', 'VV', 'LC', 'NR', 'AD', 'VV', 'DER', 'AD', 'VV', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['VV', 'NN', 'VV', 'DEC', 'NN'], ['JJ', 'NN', 'DEG', 'NN', 'VV', 'VV', 'AS', 'CD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AS'], ['AD', 'VV', 'AD', 'NN', 'VV', 'NN'], ['NR', 'DEG', 'CD', 'NN', 'NN', 'VV', 'VV'], ['NN', 'VV', 'AS', 'NN', 'LC', 'NN', 'NR', 'AD', 'AD', 'AD', 'VV', 'NR', 'VV', 'NR'], ['AD', 'DT', 'VV', 'NR', 'VV', 'AS', 'NN'], ['NR', 'NR', 'NN', 'JJ', 'NN', 'VV'], ['PN', 'VC', 'PN', 'NN', 'NN', 'VV', 'NN', 'SP'], ['P', 'NN', 'JJ', 'NN', 'AD', 'VV'], ['NR', 'DEG', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NN', 'CD', 'NN'], ['VV', 'AS', 'NN', 'NN', 'PN', 'NN', 'VV', 'AS', 'AD', 'VV', 'NN'], ['NN', 'VV', 'PN', 'P', 'NN', 'VE', 'VA'], ['AD', 'PN', 'NN', 'VV', 'NN', 'NN', 'AD', 'CD', 'M', 'SP'], ['AD', 'P', 'NR', 'NN', 'AD', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['NR', 'NR', 'P', 'NR', 'NR', 'DEG', 'NN'], ['NN', 'CC', 'NR', 'NN', 'DEG', 'NN', 'VV', 'DEC', 'AD', 'VC', 'VV', 'NN'], ['AD', 'AD', 'P', 'PN', 'VV', 'SP'], ['NR', 'NR', 'NR', 'NN', 'VV', 'NN', 'JJ', 'VV'], ['AD', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NT', 'AD', 'VV', 'AS', 'NR', 'DEG', 'NN', 'CD', 'NN', 'CD', 'NN', 'NN'], ['AD', 'NT', 'LC', 'NN', 'NR', 'AD', 'VV', 'VV', 'NN', 'AD', 'AD', 'P', 'PN', 'NN', 'AD', 'VV'], ['NR', 'SP', 'PN', 'VV', 'DER', 'VV', 'VA', 'SP'], ['AD', 'P', 'NR', 'AD', 'VV', 'NN', 'DEC', 'NR', 'SB', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN'], ['NN', 'VV', 'NT', 'NN', 'VV', 'JJ', 'NN', 'AD', 'AD', 'NN', 'VV'], ['AD', 'VV', 'VV', 'NN', 'VV', 'DEC', 'VV', 'NR', 'VV'], ['NN', 'VV', 'NR', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN'], ['VV', 'DEC', 'NN', 'AD', 'VC', 'NN', 'NN', 'ETC', 'NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'PN', 'AD', 'AD', 'P', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VV', 'NN', 'AD', 'NN', 'AD', 'AD', 'VE', 'VV', 'DEC', 'NN'], ['AD', 'VA', 'DEC', 'VC', 'AD', 'VV', 'AD', 'AD', 'VC', 'NN', 'VV'], ['VE', 'NR', 'NN', 'VV', 'VV', 'NR', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'VA', 'DEV', 'VV', 'AS'], ['NR', 'JJ', 'NN', 'VV', 'NN', 'NN', 'VV', 'NR', 'NN'], ['P', 'NN', 'DEG', 'NN', 'VV', 'VA', 'CC', 'NR', 'NN', 'AD', 'VV', 'NN'], ['PN', 'VC', 'VA', 'VA', 'NN', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'NN', 'NN', 'ETC', 'NN', 'SB', 'VV', 'VV'], ['VV', 'AS', 'AD', 'VV', 'AD', 'VV'], ['DT', 'NN', 'AD', 'VA', 'VV', 'AS'], ['PN', 'AD', 'VV'], ['DT', 'NN', 'VV', 'NN'], ['NN', 'AD', 'AD', 'AD', 'NN', 'AD', 'VV', 'AD', 'VV'], ['VV', 'NR', 'NN'], ['PN', 'AD', 'AD', 'VA', 'SP', 'IJ'], ['NR', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'LC', 'DEG', 'PN', 'CC', 'CD', 'NN', 'AD', 'AD', 'VE', 'CD', 'NN', 'NN'], ['NR', 'VV', 'NN', 'NN'], ['AD', 'AD', 'VA', 'DEC', 'SP'], ['NN', 'VA', 'VV', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'VA', 'AD', 'VV', 'AD', 'VA'], ['NR', 'NN', 'NR', 'VV', 'P', 'NN', 'NN', 'NN'], ['NN', 'NN', 'VV', 'JJ', 'NN', 'SB', 'VV', 'VV', 'JJ', 'DEG', 'NN'], ['NN', 'VV', 'AS', 'P', 'NN', 'NN', 'VV', 'AS', 'VV', 'NN'], ['VV', 'AS', 'NN', 'AD', 'VV', 'AD', 'VA'], ['NR', 'NN', 'AD', 'VV'], ['JJ', 'NR', 'AD', 'P', 'NN', 'VV', 'VV', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'LC', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'P'], ['NN', 'LC', 'NN', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'SP'], ['AD', 'NR', 'NN', 'VV', 'NN', 'CC', 'NN'], ['NR', 'AD', 'VV', 'DEC', 'NN', 'VC', 'VV', 'NR', 'CC', 'NR', 'NN', 'DEG', 'NN'], ['VV', 'DEC', 'P', 'NN', 'VV', 'VV', 'P', 'VV', 'LC', 'DEG', 'NN'], ['PN', 'VV', 'PN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'VC', 'JJ', 'NN', 'VV', 'P', 'JJ', 'JJ'], ['NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'P', 'NN', 'NN', 'VV'], ['VV', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NR', 'NN', 'NR', 'NR', 'AD', 'VV', 'VA'], ['NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VC', 'CD', 'NN'], ['VV', 'NN', 'VV', 'DEC', 'NR', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV'], ['NN', 'NN', 'CC', 'VV', 'NN', 'SP'], ['AD', 'AD', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'PN', 'VV', 'NR', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'NR', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS', 'CD', 'M'], ['AD', 'NR', 'VC', 'NR', 'NN', 'DEG', 'JJ', 'NN'], ['PN', 'VC', 'VV', 'VV', 'MSP', 'AD', 'VV', 'PN', 'DEG', 'SP'], ['NN', 'VA', 'DER', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'VV', 'AD', 'VV', 'VV', 'NR', 'NR'], ['VE', 'NN', 'VV', 'VV', 'NN', 'NT', 'VE', 'NR', 'NN', 'NR', 'NR', 'NR'], ['PN', 'AD', 'VC', 'BA', 'PN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'P', 'NN', 'NN'], ['JJ', 'NN', 'AD', 'VC', 'JJ', 'NN', 'AD', 'VV', 'VV'], ['CS', 'NN', 'VV', 'VV', 'CC', 'VV', 'P', 'NN', 'NN', 'VV', 'VV', 'P', 'NN', 'NT', 'NN', 'VV', 'VA'], ['NN', 'AD', 'VV', 'NN', 'VV', 'AS', 'AD', 'VV'], ['AD', 'DT', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'NR', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'P', 'VV', 'VV', 'VV', 'NN'], ['AD', 'NN', 'SB', 'VV', 'VV', 'AD', 'NN'], ['SB', 'VV', 'DEC', 'NR', 'VV', 'VV', 'JJ', 'NN', 'VV', 'VV'], ['VV', 'DT', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'NR'], ['P', 'NN', 'NN', 'VV', 'CC', 'NR', 'NN', 'CC', 'NN', 'NN', 'VV'], ['NT', 'DEG', 'NR', 'NN', 'NN', 'NN', 'VA'], ['AD', 'VV', 'P', 'NR', 'VV', 'VV', 'NN'], ['NN', 'AD', 'VV', 'VV', 'VV'], ['VV', 'VV', 'NN', 'NN', 'VV', 'PN', 'VV', 'VV', 'NN', 'SP', 'PN', 'AD', 'AD', 'VV', 'SP'], ['PN', 'DEG', 'NN', 'AD', 'VE', 'NN', 'DEC', 'NN'], ['NR', 'CC', 'NR', 'AD', 'VV', 'AS', 'NN'], ['NN', 'NN', 'AD', 'P', 'NR', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV'], ['NR', 'NR', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN', 'NN'], ['PN', 'P', 'NN', 'LC', 'M', 'VA', 'P', 'NN', 'P', 'NN', 'LC', 'AD', 'VV', 'P', 'NN', 'LC'], ['NR', 'AD', 'VV', 'NN', 'NN', 'VC', 'NR', 'DEG', 'JJ', 'NN'], ['AD', 'CD', 'VV', 'VV', 'DEC', 'NN', 'VV', 'AS', 'VV', 'P', 'NN', 'CD', 'NN', 'VV'], ['P', 'NR', 'DEG', 'NN', 'PN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VV'], ['AD', 'AD', 'NN', 'NN', 'VV', 'VV', 'NN', 'P', 'NN', 'AD', 'VV'], ['NN', 'BA', 'NN', 'VV', 'P', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'VV', 'NN'], ['NN', 'VV', 'SP', 'VV', 'NR', 'NN', 'AD', 'VA'], ['VV', 'AS', 'CD', 'NN', 'AD', 'VA'], ['PN', 'DEG', 'NN', 'VC', 'VV'], ['NR', 'NN', 'VC', 'NN', 'VV', 'NR', 'DEC', 'OD', 'NN', 'NN'], ['CD', 'NN', 'AD', 'VV', 'P', 'NT', 'AD', 'P', 'NN', 'LC', 'VV', 'VV'], ['NN', 'NN', 'DEG', 'OD', 'NN', 'CD', 'NN', 'NN', 'VV'], ['P', 'NN', 'LC', 'VV', 'NN', 'NN', 'VV'], ['AD', 'NR', 'AD', 'VV'], ['NR', 'NR', 'CC', 'NR', 'NN', 'VV'], ['NR', 'NN', 'VV', 'VV', 'VV', 'DEV', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NN', 'VV', 'NR', 'NN'], ['PN', 'DEG', 'NR', 'NN', 'NN', 'NT', 'VV', 'AS', 'NN', 'P', 'PN', 'VV', 'NN'], ['AD', 'AD', 'NN', 'CD', 'NN', 'AD', 'VV', 'VV', 'NR', 'NR', 'DEG', 'NN', 'VV', 'NN', 'VV'], ['NT', 'NR', 'NN', 'CD', 'NN', 'AD', 'VV', 'AS', 'VV', 'NN', 'NN'], ['AD', 'VV', 'VV', 'M', 'NN'], ['CD', 'NN', 'NN', 'VA', 'VV', 'VV'], ['VV', 'PN', 'AD', 'P', 'VV', 'NN'], ['PN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'NN', 'AD', 'VA'], ['NN', 'NN', 'CC', 'NN', 'JJ', 'NN', 'VV', 'P', 'VV', 'NN', 'AD', 'VA'], ['NN', 'AD', 'AD', 'AD', 'NN', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['P', 'NN', 'NR', 'NN', 'P', 'NN', 'VV', 'AS', 'NN', 'NN'], ['NR', 'VV', 'VV', 'DEC', 'NN', 'CD'], ['AD', 'NR', 'AD', 'AD', 'AD', 'VV', 'AD', 'VA'], ['NR', 'AD', 'VV', 'NN'], ['P', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VV'], ['AD', 'AD', 'VV', 'NN', 'NN', 'NN'], ['AD', 'AD', 'VV', 'NN', 'DEG', 'NN', 'NN', 'VE', 'AD', 'JJ', 'NN', 'VV', 'SP'], ['VV', 'NN', 'LC', 'VV', 'VA', 'AS', 'NN', 'DEG', 'NN'], ['AD', 'P', 'NR', 'AD', 'VE', 'NN'], ['DT', 'NN', 'VC', 'AD', 'VA', 'SP'], ['AD', 'P', 'NT', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'MSP', 'VV'], ['NN', 'VV', 'PN', 'DEC', 'NN', 'PN', 'AD', 'NR', 'VV', 'PN', 'NN', 'AD', 'VV', 'PN'], ['JJ', 'NN', 'VC', 'NN', 'VV', 'VV', 'DEC', 'NN', 'PN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NT', 'NR', 'VV', 'AS', 'NR', 'OD', 'NN', 'NN', 'OD', 'NN'], ['NR', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'P', 'NN', 'NR', 'LC', 'VV', 'NN'], ['NN', 'VV', 'AS', 'VV', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'P', 'JJ', 'VV', 'VV'], ['VV', 'DEC', 'DT', 'JJ', 'NN', 'SP', 'NR', 'NN', 'PN', 'AD', 'VV', 'VV', 'NN'], ['VV', 'LC', 'VV', 'NN', 'VV', 'AS', 'VE', 'NN', 'AD', 'VV', 'VV', 'PN'], ['NR', 'NN', 'P', 'NN', 'VA', 'VV', 'CD', 'NN'], ['NN', 'DEG', 'NN', 'P', 'NN', 'VV', 'NN'], ['P', 'VV', 'NN', 'NR', 'VV', 'VV'], ['PN', 'VV', 'AD', 'VV', 'AD', 'VV', 'P', 'PN', 'VV', 'VV', 'AD', 'VV', 'VV', 'DEC', 'NN', 'CC', 'NN'], ['VV', 'NN', 'NN', 'NR', 'VV'], ['CD', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'AS', 'NN', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VA', 'DEC', 'VA', 'VC', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'LC', 'VV', 'DEC'], ['NT', 'DT', 'AD', 'VV', 'NN', 'CD', 'SP', 'CD', 'VV', 'JJ', 'JJ', 'DEG', 'NN'], ['AD', 'JJ', 'JJ', 'NN', 'CC', 'VA', 'AD', 'VV', 'AD', 'VV'], ['PN', 'AD', 'VV', 'PN', 'AD'], ['NR', 'NN', 'NN', 'NN', 'NN'], ['AD', 'NR', 'VV', 'AS', 'PN', 'DEG', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV'], ['NR', 'NN', 'VV', 'NR', 'VV', 'AS', 'VV', 'VV', 'NR', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['PN', 'NN', 'VE', 'AD', 'VA', 'AS', 'IJ', 'NT', 'PN', 'AD', 'VE', 'NN', 'P', 'NN', 'VV'], ['NN', 'VV', 'AS', 'NN', 'SP', 'VV', 'PN', 'VV', 'DEC', 'VC', 'NN', 'NN', 'VV', 'VV', 'SP'], ['VV', 'PN', 'AD', 'BA', 'NN', 'VV', 'PN', 'VV', 'SP'], ['P', 'JJ', 'NN', 'NN', 'LC', 'BA', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'CC', 'AD', 'VV'], ['NN', 'NR', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NN', 'VA', 'VE', 'NN', 'CC', 'NN'], ['NN', 'VV', 'VV', 'DT', 'NN', 'AD', 'VV'], ['P', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'AD', 'AD', 'VC', 'NN', 'VV', 'SP'], ['VE', 'NN', 'VV', 'NN', 'VV', 'CD', 'CD', 'CD', 'NN', 'CD', 'CD', 'CD', 'CD'], ['P', 'PN', 'DEG', 'NN', 'VV', 'PN', 'NN', 'PN', 'VV', 'PN', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NR', 'NR', 'VV', 'NN', 'CC', 'NN', 'NR', 'DEG', 'NN', 'NN'], ['NN', 'VV', 'NN', 'AD', 'BA', 'NN', 'VV'], ['P', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VC', 'NR', 'NR', 'NR', 'NN'], ['VV', 'AD', 'VA', 'DEC', 'NN', 'VV'], ['VV', 'AS', 'JJ', 'NN', 'DT', 'NN', 'VV', 'AS', 'AD', 'VV'], ['AD', 'VV', 'AS', 'VV', 'NN', 'NN', 'VE', 'NN', 'NN', 'NN', 'VV', 'VV', 'PN', 'VV'], ['AD', 'VV', 'NN', 'SP'], ['NN', 'VV', 'NN'], ['NR', 'CC', 'NR', 'NR', 'NN', 'NN', 'NN', 'VV', 'PN', 'VV', 'VV', 'NR'], ['AD', 'NT', 'NR', 'NN', 'NR', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['AD', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'PN', 'AD', 'VV', 'VA', 'DEC', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'DEV', 'VV', 'AS', 'PN', 'AD'], ['NR', 'NR', 'VV', 'NN', 'VV', 'AD', 'CD', 'NN'], ['P', 'JJ', 'NN', 'NR', 'VV', 'VV', 'NN', 'VV'], ['PN', 'P', 'PN', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV'], ['NN', 'NR', 'VV', 'NR', 'NR', 'NN', 'VV', 'NN', 'NR', 'NN', 'NN', 'NN'], ['PN', 'VE', 'CD', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'PN', 'DEG', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'NR'], ['NR', 'DEG', 'CD', 'NN', 'VV', 'VV'], ['P', 'VV', 'NN', 'JJ', 'NN', 'CC', 'NN', 'VV', 'DT', 'JJ', 'NN', 'DEC', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'AD', 'VV'], ['AD', 'VV', 'SP', 'PN', 'PN', 'AD', 'VC', 'VV'], ['AD', 'NR', 'P', 'P', 'NN', 'DT', 'NN', 'AD', 'VV', 'AS', 'VV'], ['NR', 'P', 'AD', 'VV', 'MSP', 'VV', 'NN', 'DEC', 'NN'], ['DT', 'NN', 'AD', 'VV', 'AS', 'NR', 'DEG', 'NN', 'NN'], ['AD', 'VC', 'CD', 'VV', 'DEC', 'NN', 'NN'], ['LB', 'PN', 'VV', 'VV', 'AS', 'AD', 'VV', 'PN', 'SP'], ['JJ', 'NN', 'LC', 'AD', 'VA', 'VV', 'VV', 'NN', 'NN', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'P', 'VV', 'NN', 'LC', 'P', 'NN', 'LC', 'NN', 'MSP', 'AD', 'VV', 'VV', 'NN', 'VV'], ['AD', 'NN', 'LC', 'VV', 'VE', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'LC', 'NN', 'VA'], ['PN', 'VV', 'AS', 'NN', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'NR', 'DEG', 'NN', 'JJ', 'NN'], ['NR', 'VV', 'NR', 'NN', 'NT', 'AD', 'VV', 'AS', 'AD'], ['AD', 'NN', 'NN', 'CD', 'NN', 'VV', 'NR', 'VC', 'DT', 'VV', 'DEC', 'NN'], ['NR', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'P', 'NR', 'VV', 'NN'], ['NN', 'PN', 'AD', 'AD', 'VA', 'SP'], ['PN', 'VV', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV'], ['NN', 'AD', 'VV'], ['VC', 'NN', 'P', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'VV', 'AS', 'NR'], ['VA', 'DEC', 'VC', 'PN', 'LB', 'NN', 'VV', 'DEC', 'NR', 'VV'], ['AD', 'VV', 'NN', 'NN', 'VV', 'DEC', 'VV', 'VV', 'VV', 'VA'], ['AD', 'NN', 'NN', 'VV', 'VV', 'PN', 'NN', 'VV', 'NN', 'VV', 'NR'], ['P', 'NR', 'NN', 'NN', 'LC', 'IJ', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NN', 'VV', 'VV', 'LC', 'NN', 'SB', 'AD', 'DEV', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'VV'], ['CS', 'NN', 'AD', 'VA', 'VA', 'AD', 'AD', 'VV', 'VV', 'SP'], ['DT', 'M', 'NN', 'AD', 'VE', 'NR', 'NN'], ['NN', 'CD', 'NN', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VA', 'VV', 'VV', 'VV', 'LC', 'AD', 'NN', 'NN'], ['PN', 'DT', 'NN', 'VV', 'VV', 'PN', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'AD'], ['NR', 'AD', 'NN', 'P', 'VV', 'NN', 'NN', 'LC', 'AD', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'PN', 'NN', 'VV', 'AD', 'VA'], ['NR', 'VV', 'M', 'NR', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN', 'VV', 'VV'], ['P', 'VV', 'JJ', 'NN', 'VV', 'AS', 'BA', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'DT', 'NN', 'VC', 'CD', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'NN', 'AD', 'NN', 'AD', 'NN', 'LC', 'PN', 'VV', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV'], ['NR', 'DEG', 'NN', 'VV', 'AD', 'VV', 'SP'], ['NN', 'NN', 'VV', 'NT', 'BA', 'NN', 'VV', 'VV', 'NN', 'AD', 'BA', 'PN', 'VV', 'VV', 'AS', 'NN'], ['VV', 'JJ', 'NN', 'NN', 'NN', 'VV', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VC', 'AD', 'VV', 'SP'], ['VV', 'PN', 'AD', 'VV', 'VV', 'SP'], ['VA', 'VA', 'VA', 'SP'], ['DT', 'NN', 'AD', 'VV', 'SP'], ['VA', 'AD', 'AD', 'PN'], ['NT', 'NT', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NR', 'NR', 'NT', 'NT', 'NN', 'CD', 'CD', 'CD', 'AD', 'VV'], ['AD', 'PN', 'BA', 'NN', 'NN', 'VV', 'DT', 'VV', 'AD', 'VV', 'VV', 'AS', 'CD', 'NN'], ['NR', 'AD', 'P', 'NN', 'LC', 'VV', 'NR', 'DEG', 'NN'], ['AD', 'NN', 'AD', 'P', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'NR', 'NN', 'NR', 'VV'], ['VE', 'NN', 'PN', 'VV', 'AD', 'VA', 'SP'], ['AD', 'NN', 'VV', 'PN', 'VV', 'NR', 'NN', 'NN', 'CC', 'NR', 'VV', 'NN', 'VV'], ['VV', 'NN', 'DEC', 'JJ', 'NN', 'AD', 'SB', 'VV'], ['P', 'NN', 'NN', 'VV', 'LC', 'AD', 'JJ', 'NN', 'NN', 'VV', 'VV', 'AS', 'JJ', 'NN'], ['NR', 'NN', 'AD', 'NN', 'VA'], ['NR', 'VV', 'VV', 'PN', 'NN', 'VV', 'P', 'NR', 'NN', 'NN'], ['P', 'NN', 'VV', 'LC', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV'], ['NN', 'AD', 'PN', 'AD', 'VV', 'SP', 'AD', 'VV', 'AS', 'VA', 'SP'], ['AD', 'VV', 'VV', 'PN', 'VV', 'AS'], ['PN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NR', 'VV', 'DER', 'AD', 'VA'], ['NN', 'NN', 'NN', 'VV', 'NN', 'VV'], ['AD', 'NN', 'NN', 'LC', 'P', 'NN', 'NN', 'DEG', 'NN', 'LB', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'JJ', 'NN', 'VV', 'CD', 'NT', 'CD', 'CD', 'NN', 'VV', 'CD', 'NT', 'NT'], ['VV', 'VV', 'DEC', 'PN', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV'], ['AD', 'AD', 'VV', 'AD', 'VV', 'DER', 'VV', 'AD', 'VV', 'VV'], ['PN', 'NT', 'VV', 'P', 'NN', 'CD', 'JJ', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'P', 'NN', 'VV', 'AS'], ['P', 'NN', 'NN', 'VV', 'NR', 'LC', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV'], ['PN', 'AD', 'VV', 'AS', 'VV', 'CD', 'NN', 'NN', 'VV'], ['JJ', 'JJ', 'NN', 'VE', 'VV'], ['AD', 'P', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AD', 'VV', 'VV'], ['AD', 'NR', 'VV', 'AS', 'NN', 'CC', 'NN', 'NN', 'NN', 'VV', 'VA', 'VA'], ['AD', 'NN', 'VV', 'AD', 'JJ', 'NN', 'NN', 'CC', 'NN', 'VV'], ['PN', 'VV', 'NR', 'CD', 'CD', 'NT', 'DEG', 'NN', 'NN', 'VC', 'CD'], ['PN', 'VV', 'DEC', 'NN', 'VV', 'NN', 'JJ', 'NN', 'P', 'PN', 'VV', 'VV'], ['P', 'VV', 'VV', 'NN', 'AD', 'BA', 'NN', 'NN', 'NR', 'VV'], ['NN', 'VV', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'NN', 'NN', 'NN'], ['VV', 'NN', 'VV', 'AS', 'AD', 'VV', 'BA', 'NN', 'VV', 'AD'], ['AD', 'AD', 'DEV', 'VV', 'NT', 'AD', 'VV', 'AS', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VE', 'NN', 'VV', 'VV', 'VV'], ['VV', 'AD', 'VV', 'MSP', 'VV', 'AD', 'VC', 'AD', 'VV'], ['PN', 'NN', 'AD', 'AD', 'VV', 'M', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV'], ['AD', 'NR', 'VV', 'NR', 'LB', 'NN', 'VV', 'NN'], ['NR', 'VV', 'CD', 'NN', 'CD', 'CD', 'NN', 'NN', 'AD', 'SB', 'VV', 'NN'], ['NN', 'VE', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NT', 'AD', 'VV', 'NN', 'VV'], ['AD', 'NR', 'AD', 'CC', 'VA', 'SP'], ['NN', 'VV', 'NN', 'AD', 'VA', 'VA'], ['NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'LC'], ['NT', 'P', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN', 'NN', 'DT', 'NN', 'DEG', 'NN', 'AD', 'VA'], ['VV', 'NT', 'LC', 'NN', 'AD', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV'], ['VV', 'PN', 'P', 'NN', 'LC', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'VV', 'AD', 'AD', 'VV', 'NN'], ['AD', 'VE', 'NN', 'P', 'NN', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['NT', 'PN', 'AD', 'NT', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV'], ['PN', 'VA', 'NN', 'VV', 'VV', 'AD', 'VV', 'PN'], ['NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'NN', 'AD', 'VV'], ['CS', 'VV', 'AD', 'NN', 'VV', 'VV'], ['AD', 'VA', 'NN', 'AD', 'VC', 'JJ', 'NR', 'DEG', 'NN', 'VV'], ['NR', 'DEG', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VC', 'NN', 'LC', 'VV', 'SP'], ['NT', 'PN', 'VV', 'AS', 'NR', 'NN', 'AD', 'VV', 'LC', 'VV', 'AD', 'VV', 'CD'], ['AD', 'NN', 'AD', 'VV'], ['AD', 'VV', 'VV', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'P', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'AD', 'VV', 'AD', 'VA', 'NN', 'P', 'NN', 'VA', 'VV'], ['AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'NN', 'AD', 'VE', 'CD', 'NN'], ['NN', 'NN', 'CC', 'JJ', 'NN', 'NR', 'AD', 'VV', 'NN', 'NN'], ['CS', 'VV', 'AS', 'AD', 'VV', 'VV'], ['AD', 'PN', 'BA', 'NN', 'VV', 'VV', 'P', 'PN', 'VV', 'AS', 'CD', 'NN'], ['VE', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV'], ['JJ', 'P', 'NR', 'VV', 'AS', 'CD', 'JJ', 'JJ', 'DEG', 'NN'], ['VV', 'VV', 'LC', 'PN', 'SP'], ['NN', 'DEG', 'CD', 'NN', 'VC', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NT', 'PN', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NT'], ['M', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'SP'], ['NN', 'NN', 'NN', 'VA', 'NN', 'VA'], ['NR', 'NN', 'AD', 'DEV', 'VV', 'NR'], ['NN', 'LC', 'NN', 'CD', 'NN', 'NN'], ['P', 'NR', 'LC', 'VV', 'VE', 'NN', 'NN', 'NN'], ['PN', 'AD', 'AD', 'P', 'NN', 'VV'], ['VV', 'AS', 'NR', 'JJ', 'DEG', 'NN', 'VV', 'AD', 'VV', 'VA', 'VV', 'NN', 'VV', 'NN', 'VV'], ['VV', 'LC', 'NT', 'NR', 'NN', 'AD', 'VV', 'VV', 'AD', 'VV', 'SP'], ['AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV'], ['NR', 'AD', 'VV', 'AD', 'P', 'NR', 'VV'], ['NN', 'VV', 'NR', 'LB', 'NR', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['AD', 'NN', 'VV', 'VV', 'AD', 'AD', 'VV'], ['NT', 'NN', 'CC', 'NR', 'VV', 'VV', 'AS', 'NN'], ['AD', 'VA', 'AD', 'VA', 'NN', 'SP'], ['NN', 'NN', 'NN', 'AD', 'VV', 'M'], ['VV', 'DEC', 'PN', 'BA', 'PN', 'VV', 'AS', 'VA', 'DER', 'AD', 'DEV', 'VV', 'NR', 'VV'], ['VV', 'DEC', 'NN', 'NN', 'LC', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'NT', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'NN', 'P', 'AD', 'VV', 'NR', 'VV', 'NN', 'AD', 'VV'], ['AD', 'NT', 'LC', 'AD', 'VV', 'DT', 'NN', 'NN', 'NN'], ['NN', 'VV', 'AS', 'AD', 'JJ', 'NN', 'VV', 'NN', 'LC'], ['NN', 'NN', 'NN', 'AD', 'VV', 'P', 'VV', 'NN', 'VV', 'PN', 'VV', 'AD', 'AD', 'VV', 'PN'], ['VV', 'NN', 'NN', 'VV', 'VV', 'NR'], ['NN', 'DEG', 'JJ', 'NN', 'VV', 'VV', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VV', 'NN', 'PN', 'VV', 'PN', 'AD', 'VV', 'AD', 'VV'], ['VV', 'NN', 'CC', 'NN', 'AD', 'AD', 'VV', 'NR', 'AD', 'VV', 'NN', 'LC', 'VV'], ['DT', 'NR', 'NN', 'AD', 'NR', 'DEG', 'CD', 'NN'], ['PN', 'VV', 'VV', 'PN', 'VV', 'NN', 'NN'], ['VV', 'AS', 'VV', 'AS', 'PN', 'AD', 'NN', 'NR', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['VV', 'DT', 'NN', 'VV', 'NN', 'PN', 'NN', 'AD', 'VV', 'NN', 'NR', 'NN', 'NN'], ['P', 'NN', 'LC', 'NN', 'DEC', 'VV', 'VV', 'P', 'NN'], ['CS', 'NR', 'VV', 'NN', 'NN', 'NN', 'PN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VA'], ['CD', 'NN', 'VV', 'VV', 'VV', 'DEC', 'NN', 'VV', 'NN', 'VA'], ['SB', 'VV', 'NN', 'NR', 'DEG', 'NN', 'NN'], ['P', 'NN', 'LC', 'LC', 'NR', 'NN', 'VV', 'NN', 'NN', 'AD', 'VC', 'NR', 'VA'], ['P', 'NT', 'DEG', 'NN', 'NN', 'VV', 'NT', 'NN', 'NN', 'AD', 'VE', 'VV', 'VV'], ['NN', 'VV', 'PN', 'VV', 'LC', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'SP', 'AD', 'VC', 'M', 'NN'], ['NN', 'VV', 'AD', 'VV', 'LC', 'NR', 'VV', 'PN', 'NN'], ['NR', 'CC', 'NN'], ['PN', 'NN', 'VV', 'NN', 'VA', 'SP'], ['P', 'NR', 'VV', 'NN', 'LC', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV'], ['NN', 'CS', 'AD', 'VV', 'AD', 'DT', 'NN', 'DEG', 'NN', 'NN', 'VV', 'NN', 'VV'], ['VV', 'NN', 'CC', 'JJ', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'VA', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'CD', 'NN', 'CD', 'NN', 'CD', 'CD', 'CC', 'CD', 'CD', 'OD', 'M', 'NN', 'AD', 'VV', 'VV'], ['VC', 'NR', 'NN', 'NN', 'SP'], ['NR', 'AD', 'VC', 'CD', 'NN', 'VV', 'NN'], ['P', 'PN', 'VA', 'AS', 'AD', 'VV', 'VV', 'VV', 'SP'], ['NN', 'NN', 'NN', 'NN', 'VV', 'AS', 'VV', 'BA', 'NN', 'VV', 'DT', 'VV'], ['PN', 'NT', 'VV', 'NN', 'NN', 'VV', 'AD', 'NN', 'AD', 'VA'], ['NR', 'NN', 'JJ', 'NN', 'NR', 'VV', 'CD', 'NN'], ['PN', 'VV', 'VV', 'PN', 'VV'], ['AD', 'NR', 'NR', 'VV', 'AS', 'VA', 'VA', 'JJ', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['CD', 'NN', 'NN', 'VV', 'NN', 'NN', 'MSP', 'JJ'], ['NR', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['NR', 'VV', 'NR', 'NN', 'NN', 'LC', 'VE', 'JJ', 'NN', 'CC', 'NN', 'NN', 'NN'], ['NR', 'VC', 'NN', 'CC', 'VV', 'NN'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'P', 'P', 'NN', 'VV', 'NN', 'P', 'VV', 'NN', 'NN', 'VE', 'CD', 'NN', 'VE', 'NN'], ['NR', 'CC', 'NR', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'VA'], ['NT', 'NR', 'NN', 'CD', 'NN', 'NN', 'NN', 'LC', 'JJ', 'NN', 'SB', 'VV', 'NN', 'VV', 'AD'], ['NT', 'CD', 'NN', 'NN', 'VC', 'NN', 'NN', 'NN', 'NN'], ['AD', 'PN', 'JJ', 'NN', 'AD', 'VC', 'VE', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'PN', 'VV', 'JJ', 'NN', 'DEG', 'VV', 'AS', 'VV', 'JJ', 'NN', 'NN', 'NR', 'NN'], ['VV', 'NR', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'VV', 'PN', 'VV', 'PN', 'DEG', 'NN', 'NN', 'P', 'NN', 'VV', 'CC', 'VV', 'VV'], ['NR', 'AD', 'AD', 'LB', 'NN', 'VV', 'VC', 'NN', 'DEC', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'AS', 'NR', 'NN', 'AD', 'VV', 'VV', 'AS', 'VV', 'NN', 'VV', 'VV', 'VV'], ['NR', 'AD', 'VV', 'VV', 'AS', 'NN', 'VA', 'DEV', 'VV', 'P', 'NN'], ['AD', 'AD', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VE', 'NN', 'VV', 'PN', 'VV'], ['PN', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VE', 'NN', 'SP'], ['VV', 'DT', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'BA', 'NN', 'VV', 'M'], ['AD', 'AD', 'VV', 'LC', 'DEG', 'NN', 'NN', 'VV'], ['IJ', 'AD', 'PN', 'VV', 'AD', 'VV', 'PN', 'SP'], ['PN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'VV'], ['P', 'PN', 'AD', 'VV', 'NN', 'NN', 'LC', 'VV', 'VV', 'AD', 'VA', 'VA'], ['VV', 'NN', 'NN', 'VV', 'AS', 'M', 'VV', 'M'], ['AD', 'NR', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV'], ['VV', 'PN', 'DEG', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'AD', 'NR', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEC', 'DT', 'M', 'NN'], ['NR', 'AD', 'P', 'JJ', 'NN', 'LC', 'VV', 'VV'], ['VV', 'DEC', 'NN', 'VC', 'NR', 'NR', 'NN'], ['PN', 'VV', 'VV', 'JJ', 'NN', 'VV', 'NN', 'NN', 'VV', 'PN', 'VV', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'P', 'NN', 'PN', 'VV', 'AD', 'VV', 'AD', 'VV', 'VV', 'NR', 'VV', 'PN', 'NN'], ['AD', 'NN', 'NN', 'P', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'AD', 'NN'], ['PN', 'AD', 'CC', 'NR', 'AD', 'VV', 'JJ', 'NN'], ['P', 'JJ', 'NN', 'NR', 'NN', 'VV', 'NR', 'NR', 'JJ', 'NN'], ['P', 'NR', 'NN', 'DEG', 'NN', 'NT', 'AD', 'P', 'NR', 'NR', 'NN', 'VV'], ['SB', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'NN', 'NN'], ['VV', 'VV', 'NN', 'VA', 'DEC', 'NN', 'NN', 'NN', 'VV', 'JJ'], ['NN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'SB', 'NR', 'NN', 'NR', 'VV', 'DER'], ['NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'P', 'NN', 'VV', 'NN'], ['PN', 'VV', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VE', 'NN', 'SP', 'SP'], ['P', 'NN', 'NN', 'LC', 'NN', 'VV', 'CD', 'NR', 'NN', 'NN', 'VE', 'NN', 'VV', 'NN'], ['NR', 'CC', 'NR', 'NR', 'NN', 'VA'], ['NN', 'NR', 'VA', 'DEV', 'VV', 'PN', 'VC', 'VV', 'VV', 'AD', 'P', 'PN', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'P', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'CD', 'CD', 'M', 'AD', 'SB', 'VV', 'VC', 'NR', 'NN'], ['P', 'VV', 'NR', 'NR', 'P', 'NN', 'VV', 'AD', 'VV', 'VA', 'VA', 'SP'], ['P', 'NT', 'NT', 'VV', 'NN', 'CD', 'CD', 'CD'], ['VV', 'VV', 'LC', 'NN', 'NN', 'AD', 'P', 'NN', 'NN', 'VV', 'NN', 'P', 'VV', 'NN'], ['VV', 'VV', 'NN', 'NR', 'P', 'NN', 'NN', 'NN', 'NN'], ['AD', 'AD', 'VA', 'AD', 'AD', 'VV', 'PN', 'NN', 'VA', 'DEC', 'NN'], ['NT', 'NN', 'PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'SP'], ['NR', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'VV', 'PN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'PN', 'SP'], ['CS', 'NN', 'NN', 'VV', 'VA'], ['AD', 'NR', 'VV', 'JJ', 'NN'], ['AD', 'P', 'NR', 'NN', 'VA', 'DEC', 'NN', 'AD', 'AD', 'SB', 'VV', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN'], ['PN', 'NN', 'VA', 'SP'], ['NN', 'VV', 'VV', 'NN'], ['NR', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'AD', 'AD', 'VV', 'NR', 'VV'], ['P', 'VV', 'NN', 'VV', 'VV', 'DT', 'P', 'NN', 'VV', 'NN', 'VV'], ['NN', 'AD', 'VE', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV'], ['NR', 'AD', 'VV', 'VV', 'NN', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NR', 'P', 'NN', 'P', 'NN', 'JJ', 'NN', 'LC', 'VV', 'NR', 'DEG', 'NN'], ['PN', 'P', 'NN', 'NN', 'VE', 'NN'], ['NN', 'VV', 'NN', 'VV', 'AS', 'NN', 'VV', 'VA', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'VV', 'CD', 'NN', 'VV'], ['NN', 'LC', 'VV', 'NN', 'CC', 'NN', 'VV', 'VV', 'NN'], ['VV', 'VV', 'NN', 'VV', 'LC', 'VA'], ['NR', 'NN', 'NN', 'NN'], ['NN', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['PN', 'DEG', 'AD', 'VV', 'VV', 'NN', 'SP'], ['P', 'JJ', 'NN', 'NN', 'VV', 'JJ', 'NN', 'NR', 'VV'], ['PN', 'NN', 'AD', 'VV', 'VV', 'NT', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN'], ['AD', 'NN', 'NN', 'VV', 'AD', 'VV', 'PN', 'AD', 'VA'], ['VV', 'AD', 'VV', 'VC', 'NN', 'AD', 'VV', 'VV', 'SP'], ['NR', 'AD', 'AD', 'VV', 'NR', 'NN'], ['NN', 'VV', 'PN', 'P', 'NN', 'NN', 'VV', 'PN'], ['VV', 'VE', 'NN', 'VV', 'AD', 'VE', 'NN'], ['NR', 'AD', 'P', 'JJ', 'NN', 'VV', 'NN'], ['NR', 'VV', 'VV', 'NR', 'NR', 'NR', 'NR', 'JJ', 'NN', 'VA'], ['VV', 'NN', 'PN', 'NN', 'AD', 'SB', 'PN', 'VV', 'JJ', 'DEG'], ['NR', 'VV', 'VV', 'NR', 'NN', 'NN', 'VV', 'VV', 'AD', 'JJ', 'NN', 'AD', 'VA'], ['NR', 'NN', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['VE', 'CD', 'NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'NR', 'NN', 'NN'], ['NN', 'AD', 'VV', 'P', 'NN', 'VV', 'AD', 'NN', 'VC', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'VV', 'AS', 'VV'], ['NT', 'AD', 'VV', 'NN', 'CC', 'NN', 'NN'], ['PN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'P', 'NN', 'NN', 'P', 'NN', 'NN', 'DEG', 'NN'], ['NT', 'NR', 'VV', 'CS', 'CD', 'M', 'NT', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'CD', 'NN', 'VV', 'PN', 'VV', 'AS', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'P', 'NR', 'VV', 'NR', 'AD', 'NR', 'NR', 'VV', 'AS', 'NN'], ['VV', 'NN', 'LC', 'NN', 'VV', 'NR', 'VV', 'NN'], ['NN', 'NN', 'DEG', 'NR', 'NN', 'M', 'JJ', 'NN', 'NN', 'VV', 'NR'], ['NN', 'VV', 'DEC', 'NN', 'LC', 'VV'], ['P', 'VV', 'DT', 'NN', 'VV', 'NR', 'VV', 'NR', 'NN', 'JJ'], ['NT', 'NT', 'VV', 'NN', 'VV'], ['PN', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'VV'], ['NR', 'VV', 'SP', 'PN', 'AD', 'AD', 'SP'], ['NT', 'CD', 'M', 'VV', 'VV', 'CD', 'CD', 'CD', 'NN', 'DEG', 'NN'], ['P', 'NN', 'LC', 'VV', 'CC', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'AD', 'CD', 'NN', 'AD', 'P', 'NN', 'VA', 'DEC', 'NN', 'NN', 'LC', 'NN'], ['NN', 'VA', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'VV', 'P', 'PN', 'ON', 'VV'], ['NR', 'NN', 'M', 'VV', 'AD', 'VV', 'NN', 'VV', 'DER', 'AD', 'AD', 'VV'], ['NR', 'AD', 'VE', 'AD', 'VA', 'VV'], ['PN', 'AD', 'AD', 'P', 'JJ', 'VV', 'SP'], ['NN', 'VV', 'NR', 'NR', 'NN', 'NN', 'VV', 'NN', 'VV'], ['NR', 'NN', 'LC', 'VV', 'DEC', 'NN', 'AD', 'PN'], ['VV', 'PN', 'AD', 'VV', 'VA'], ['NN', 'P', 'NR', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'NR', 'NN', 'VV', 'VV', 'BA', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['PN', 'AD', 'AD', 'VV', 'AD', 'VE', 'VV', 'MSP', 'VV'], ['NN', 'VV', 'VC', 'NN', 'AD', 'VV', 'NN'], ['VV', 'AD', 'VV', 'VV', 'CD', 'NN', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NN', 'SP'], ['AD', 'VV', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'P', 'NN', 'LC', 'VV', 'CD', 'M', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'AD', 'AD', 'P', 'JJ', 'NR', 'SP'], ['P', 'NR', 'NR', 'NR', 'CD', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['PN', 'VV', 'AS', 'NN', 'NN', 'NN', 'NR'], ['AD', 'AD', 'NN', 'VV', 'NN'], ['VV', 'NN', 'VA', 'AS', 'VV', 'P', 'CD', 'NN', 'VV', 'VE', 'NN'], ['P', 'NR', 'NN', 'AD', 'SP'], ['NR', 'NN', 'VV', 'NN', 'VV', 'NR', 'CD', 'NN', 'VV', 'NN'], ['IJ', 'SP', 'AD', 'VC', 'JJ', 'NN'], ['AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV'], ['NN', 'NN', 'NN'], ['VV', 'NR', 'SP', 'JJ', 'NN'], ['P', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'DEC', 'AD', 'VV', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'NR', 'VV', 'NR', 'NN', 'AD', 'AD', 'AD', 'AD', 'VV'], ['NR', 'NN', 'NR', 'NT', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'AD', 'VV', 'NN'], ['VV', 'DER', 'AD', 'AD', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NN', 'AD', 'VA'], ['AD', 'VE', 'NN', 'VV', 'JJ', 'NN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'AD', 'VA'], ['PN', 'NN', 'DEG', 'NN', 'VV', 'DEC', 'VC', 'NN'], ['NR', 'NN', 'VV', 'NN', 'NR', 'VC', 'CD', 'NN', 'NN', 'NR', 'VV', 'AD', 'VV'], ['NR', 'VV', 'NN', 'VC', 'PN', 'VV', 'SP'], ['NR', 'NN', 'NT', 'P', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NN', 'NN', 'NN', 'AD', 'NR', 'NR', 'NN', 'NN'], ['NR', 'CC', 'NN', 'NN', 'VV', 'NR'], ['NN', 'AD', 'VV', 'AD', 'VV', 'AD', 'VA', 'DEV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'PN', 'DEG', 'JJ', 'NN'], ['VV', 'DEC', 'VC', 'NR', 'NN', 'NR', 'NR', 'CD', 'NN', 'SP'], ['NR', 'DEG', 'NN', 'NR', 'AD', 'VC', 'AD', 'VV', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'JJ'], ['VE', 'AS', 'PN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'BA', 'AD', 'AD', 'DEV', 'NN', 'VV', 'VV'], ['NR', 'NR', 'NT', 'LC', 'VV', 'JJ', 'NN'], ['NR', 'DEG', 'NN', 'VV', 'NN', 'AD', 'P', 'NN', 'VV', 'LC', 'CD'], ['NN', 'CD', 'M', 'NN', 'CD', 'NN', 'AD', 'VV', 'VV', 'OD', 'NN'], ['VV', 'CD', 'VV', 'NN', 'DEC', 'NN', 'P', 'PN', 'VV', 'VV', 'AD', 'VV', 'AD', 'VV', 'PN', 'PN', 'VV', 'NR', 'VV', 'NN'], ['NR', 'VV', 'NN', 'VV', 'DER', 'VV', 'NN'], ['PN', 'VV', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VV', 'NN', 'SB', 'VV'], ['NR', 'NN', 'VV', 'NN', 'NN'], ['AD', 'AD', 'PN', 'AD', 'VV', 'VV', 'VA'], ['AD', 'AD', 'P', 'NN', 'P', 'NN', 'CC', 'NN', 'VV', 'NN', 'DEC', 'JJ', 'JJ', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'ETC', 'NN', 'AD', 'NN', 'NN', 'AD', 'VV', 'VA'], ['NT', 'NR', 'CC', 'NN', 'P', 'NR'], ['NT', 'NT', 'NR', 'VV', 'VV', 'AD', 'DEV', 'VV'], ['NR', 'JJ', 'NN', 'AD', 'SB', 'VV', 'VV', 'VV'], ['NR', 'CC', 'NN', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV'], ['CS', 'DT', 'NN', 'VV', 'AD', 'DT', 'NN', 'NN', 'AD', 'AD', 'VV', 'JJ', 'NN'], ['AD', 'PN', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['NR', 'VV', 'AS', 'NR', 'VV', 'VV', 'NR', 'VV', 'P', 'PN', 'VV', 'AD'], ['PN', 'AD', 'VV', 'VV', 'AD', 'VV', 'PN', 'VV', 'VV', 'SP'], ['NN', 'VV', 'NN', 'VV'], ['PN', 'VC', 'VV', 'P', 'PN', 'VV', 'SP'], ['NN', 'JJ', 'AD', 'VV'], ['NN', 'AD', 'VV'], ['AD', 'VV', 'JJ', 'NN', 'NN'], ['VE', 'NN', 'NN', 'VV', 'AD', 'VC', 'P', 'NN', 'VV', 'NN', 'NN'], ['P', 'AD', 'NN', 'LC', 'NN', 'VV', 'AS', 'LC', 'NN', 'AD', 'CC', 'VA'], ['JJ', 'NN', 'VV', 'NN', 'VV', 'P', 'NN', 'LC', 'DEG', 'NN', 'LC', 'AD', 'VA'], ['NN', 'VV', 'NN', 'VV', 'AD', 'VC', 'VV', 'NN', 'NN', 'AD', 'VC', 'NN'], ['NT', 'PN', 'VV', 'AD', 'VV', 'AD'], ['NN', 'VV', 'P', 'JJ', 'NN', 'CC', 'VV', 'DEC', 'NN', 'CC', 'NN', 'VV', 'NN', 'VV'], ['AD', 'NR', 'VV', 'NN', 'VV', 'PN', 'AD', 'VV', 'NN', 'VV', 'SP'], ['NR', 'VV', 'JJ', 'NN', 'NN', 'VV', 'P', 'NR', 'VV', 'VC', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'LB', 'NN', 'VV', 'VV', 'P', 'NN'], ['NN', 'VV', 'M', 'DT', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV'], ['VE', 'JJ', 'NN', 'NN', 'P', 'AD', 'VV', 'P', 'NN', 'VE', 'VE', 'VV', 'DEC', 'NN'], ['AD', 'VV', 'AD', 'VV', 'P', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'AD', 'DEV', 'VV', 'AS', 'NN', 'DEC', 'NN', 'NN', 'VV'], ['JJ', 'NN', 'NN', 'P', 'CD', 'M', 'CD', 'CD', 'CD'], ['PN', 'AD', 'VV', 'VV', 'DEC', 'NR', 'NN', 'SP'], ['P', 'VV', 'NN', 'CC', 'NN', 'DEC', 'VV', 'PN', 'VV', 'AS', 'VV', 'VV', 'VV'], ['DT', 'NN', 'VV', 'OD', 'DEG', 'P', 'NR', 'NN', 'VV', 'VV', 'CD'], ['NN', 'NN', 'NN', 'VA'], ['NR', 'VV', 'NN', 'NN', 'NN', 'VA'], ['NN', 'VV', 'NN', 'NN', 'NR', 'NN', 'VV', 'DEC', 'NN'], ['CS', 'VV', 'VV', 'PN', 'BA', 'NN', 'VV', 'AS', 'NN', 'LC', 'VV', 'VV', 'PN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'SP'], ['VV', 'CD', 'CD', 'CD', 'AD', 'VV'], ['NR', 'VV', 'VV', 'AD', 'P', 'NN', 'VV', 'VV', 'NR', 'CC', 'NN', 'VV', 'VV'], ['P', 'NN', 'AD', 'VC', 'PN', 'VV'], ['NT', 'VV', 'AD', 'BA', 'NN', 'VV', 'VV', 'VV', 'AD', 'VV', 'NR', 'NN'], ['NR', 'NN', 'NR', 'CC', 'NR', 'NN', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'VV', 'AS', 'AD', 'VV', 'DEC', 'PN', 'VV', 'VV', 'AD', 'VV'], ['P', 'NR', 'NR', 'LC', 'NR', 'CC', 'NR', 'VC', 'NR', 'NN', 'LC', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'AD', 'JJ', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'VA'], ['AD', 'VV', 'NR', 'CD', 'M', 'NN', 'NN'], ['NN', 'AD', 'VA', 'VA', 'DEV', 'VV', 'PN', 'VV', 'PN', 'VV', 'SP'], ['NT', 'DEG', 'NR', 'JJ', 'AD', 'AD', 'AD', 'VA', 'SP', 'AD', 'AD', 'VV', 'AS'], ['NN', 'NR', 'AD', 'NR', 'NN', 'DEG', 'NN', 'P', 'VV', 'VV', 'VV', 'VV'], ['VV', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'VV', 'VV', 'NN', 'LB', 'PN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'PN', 'AD', 'VV', 'SP'], ['JJ', 'NN', 'NN', 'NN'], ['AD', 'VV', 'VV', 'CC', 'VV', 'P', 'NN', 'LC'], ['AD', 'NN', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'NN', 'VE', 'DT', 'CS', 'VV', 'VV', 'NN', 'NN'], ['NN', 'P', 'NN', 'NN', 'LC', 'DEG', 'NN'], ['P', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN'], ['NR', 'NR', 'NN', 'P', 'CD', 'NN', 'VV', 'PN', 'P', 'NR', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'AD', 'VV', 'AD'], ['NN', 'VC', 'NR', 'NN', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV'], ['PN', 'AD', 'P', 'JJ', 'NN', 'AD', 'VA'], ['AD', 'VV', 'P', 'JJ', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'NN', 'VV', 'P', 'NN', 'AD', 'VA'], ['NT', 'NN', 'AD', 'SB', 'VV'], ['VV', 'NN', 'VA', 'NN', 'NN', 'NR', 'NN', 'P', 'NR', 'NR', 'AD', 'VV'], ['NN', 'MSP', 'VV', 'NN', 'AD', 'AD', 'P', 'NR', 'NR', 'VC', 'NN'], ['NN', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'PN', 'AD', 'P', 'PN', 'VA', 'SP'], ['VA', 'DEC', 'NN', 'VV', 'AS', 'NN', 'ETC', 'NN', 'P', 'NN', 'VV', 'AD', 'VV'], ['AD', 'NR', 'NN', 'NN', 'NN', 'LC', 'NN', 'VV', 'PN'], ['OD', 'NN', 'NN', 'NN', 'NN'], ['NT', 'PN', 'SB', 'VV', 'AS', 'AD', 'VV', 'NR', 'AD', 'VV', 'VA'], ['VV', 'NN', 'NN', 'NT', 'NT', 'LC', 'AD', 'AD', 'VV'], ['NT', 'PN', 'VV', 'VV', 'AS', 'AD', 'JJ', 'NN', 'PN', 'AD', 'LB', 'PN', 'VV', 'SP'], ['NT', 'NN', 'P', 'NN', 'VV', 'NN', 'NN', 'VV', 'LC', 'P', 'NR', 'NN', 'VV'], ['P', 'NR', 'NN', 'VV', 'AD', 'NN', 'VV', 'NN', 'VV', 'PN', 'CC', 'PN', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VA'], ['NR', 'DEG', 'NN', 'NN', 'NR', 'VV', 'AS', 'NN'], ['AD', 'NN', 'NN', 'NN', 'AD', 'LC', 'VV'], ['NR', 'VE', 'CD', 'CC', 'PN', 'VV', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'NN'], ['NN', 'NN', 'VE', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NR', 'SB', 'PN', 'DEG', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'P', 'NR', 'VV', 'NN'], ['AD', 'AD', 'CD', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'DEC', 'IJ'], ['NN', 'VA', 'AS', 'NN', 'VA', 'SP'], ['NR', 'NN', 'NN', 'NN'], ['NN', 'LC', 'CC', 'NN'], ['AD', 'AD', 'VA'], ['P', 'NN', 'VV', 'PN', 'VV', 'VV'], ['NN', 'VV', 'PN', 'VC', 'NN', 'PN', 'AD', 'VV', 'PN', 'AD', 'LC', 'VV', 'MSP', 'VV', 'P', 'NN', 'VV', 'AS', 'NN'], ['NN', 'NN', 'M', 'VV', 'NN', 'AD', 'VA'], ['CD', 'JJ', 'NN', 'VV', 'PN', 'VC', 'NN', 'VV', 'NN', 'CC', 'NN', 'NN', 'VV', 'NN'], ['NN', 'P', 'NR', 'NN', 'VV'], ['NT', 'NT', 'VV', 'NN', 'NN'], ['NN', 'NN', 'AD', 'NR', 'NN', 'CC', 'NN', 'NN', 'VV', 'AS', 'VV', 'VV', 'PN', 'VV'], ['AD', 'NR', 'AD', 'JJ', 'NN', 'AD', 'AD', 'VV', 'CD', 'JJ', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'VV', 'PN', 'VV', 'NN', 'LC', 'AD', 'AD', 'VE', 'SP', 'SP'], ['AD', 'NR', 'VV', 'NN', 'AD', 'AD', 'NN', 'VV', 'P', 'NR', 'NN', 'NN'], ['NN', 'NR', 'VV', 'NN', 'NN'], ['VA', 'DEC', 'NN', 'LC', 'VV', 'NN'], ['NR', 'VV', 'NR'], ['AD', 'AD', 'AD', 'NR', 'CC', 'CC', 'NR', 'NN', 'P', 'NR', 'DEG', 'NN'], ['AD', 'PN', 'VV', 'P', 'JJ', 'NN', 'NN', 'SP'], ['AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'SB', 'VV', 'CC', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'NN', 'AD', 'JJ', 'PN', 'AD', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VC', 'NN', 'VV', 'NN', 'VV', 'JJ', 'ETC', 'NN'], ['NN', 'JJ', 'NN', 'NR', 'VV', 'NN', 'NN', 'JJ', 'NN', 'NR', 'VV', 'NN'], ['PN', 'VC', 'JJ', 'NN', 'CC', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['NN', 'LC', 'DEG', 'NN', 'NN', 'NN', 'NN', 'NN', 'VA', 'NR', 'AD', 'VV'], ['NN', 'DEG', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN', 'OD'], ['AD', 'VV', 'NN', 'VV', 'NN', 'AD', 'AD', 'LC', 'VE', 'NN', 'VV', 'NN'], ['DT', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'CC', 'NN'], ['NN', 'AD', 'P', 'CD', 'CC', 'CD', 'VV', 'VV'], ['AD', 'VV', 'NN', 'NN', 'VV', 'AS', 'VV', 'AD', 'AD', 'VC', 'VV'], ['AD', 'AD', 'AD', 'P', 'PN', 'VV', 'AS', 'SP'], ['VV', 'NR', 'NR', 'NN', 'AD', 'VA', 'PN', 'AD', 'VV', 'VV', 'VV', 'SP'], ['P', 'NN', 'VV', 'NN', 'LC', 'NR', 'AD', 'P', 'NN', 'VV', 'DEC', 'NN', 'VV', 'SP'], ['CD', 'NN', 'VV', 'VV', 'NN', 'VV', 'PN', 'AD', 'VV', 'VA'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'VV', 'CD', 'NN'], ['NR', 'VV', 'VV', 'PN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'NT', 'DEG', 'NN', 'AD', 'VC', 'JJ'], ['JJ', 'NN', 'VV', 'VV', 'VV', 'VV', 'VV', 'AD', 'VA'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'AS', 'DT', 'NN', 'NN', 'AD', 'VA'], ['NN', 'DEG', 'NN', 'AD', 'VV'], ['OD', 'PN', 'VV', 'AD', 'NN', 'NN', 'VA', 'NN', 'AD', 'VA', 'SP'], ['PN', 'VV', 'NN', 'VA'], ['NN', 'NN', 'LB', 'NN', 'NN', 'VV', 'VV'], ['NN', 'P', 'VV', 'NN', 'AD', 'AD', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['NR', 'NN', 'P', 'P', 'NR', 'DEG', 'NN', 'NN', 'AD', 'NN', 'VV', 'NT', 'NN', 'CD'], ['PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'SP'], ['NR', 'NN', 'NN', 'NN', 'LC', 'VV', 'NN', 'NN', 'AD', 'P', 'NR', 'VV', 'VV'], ['VA', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['P', 'NN', 'DEG', 'PN', 'NN', 'AD', 'VV', 'NN', 'SP'], ['NN', 'P', 'JJ', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VA'], ['NN', 'VV', 'NN', 'CC', 'NN', 'NN', 'P', 'NN', 'AD', 'VV'], ['P', 'NN', 'VV', 'CC', 'P', 'NN', 'NN', 'NN', 'AD', 'VV', 'P', 'JJ', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN'], ['NR', 'NN', 'VV', 'P', 'NN', 'VV'], ['AD', 'DEV', 'NN', 'VV', 'PN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['PN', 'VV', 'VC', 'PN', 'VV', 'NN', 'AD', 'SP'], ['NN', 'VV', 'VV', 'AD', 'P', 'DT', 'VV', 'NN', 'VV', 'NN'], ['VV', 'VV', 'CD', 'NN', 'PN', 'VV', 'VV', 'NN', 'NN'], ['AD', 'NT', 'VV', 'CD', 'CD', 'CD', 'M', 'VV', 'NN', 'AD', 'CD', 'CD', 'M'], ['NT', 'NR', 'AD', 'VV', 'VV', 'NR', 'AD', 'AD', 'VV'], ['VV', 'VV', 'NN', 'AD', 'VA', 'AS', 'VV', 'VV', 'NN'], ['NR', 'AD', 'AD', 'VV', 'AD', 'AD', 'VV', 'VV'], ['AD', 'VV', 'NN', 'SB', 'VV', 'NR', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['PN', 'NN', 'VV', 'NN', 'VE', 'NN', 'AD', 'VA', 'DEC', 'VV'], ['NN', 'VV', 'AS', 'JJ', 'NN', 'VV'], ['NN', 'NN', 'VV', 'AS', 'VV', 'VV', 'AD', 'AD', 'VV', 'AD', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NN', 'VV', 'VV', 'DEC', 'PN', 'VV', 'AS', 'AD', 'VV', 'NN', 'VV', 'NN'], ['VV', 'NT', 'NT', 'CD', 'NT', 'VV', 'VV'], ['NN', 'VV', 'CD', 'JJ', 'DEG', 'NN', 'VV', 'P', 'NN', 'VA', 'NN', 'VV'], ['VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'DEC', 'PN', 'VV', 'NN', 'VV', 'VV', 'AS', 'AD', 'VV', 'NN'], ['AD', 'CD', 'NN', 'AD', 'P', 'NN', 'LC', 'VV', 'NN', 'VA', 'DEC', 'NN'], ['NN', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VE', 'NN'], ['VV', 'VV', 'LC', 'NN', 'AD', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NN', 'AD', 'VV', 'VV', 'PN', 'VV', 'SP'], ['NN', 'AD', 'VA', 'DEV', 'VV'], ['NT', 'NN', 'NN', 'AD', 'NN', 'VV'], ['NT', 'VV', 'VV', 'AS', 'AD', 'AD', 'AD', 'AD', 'VV', 'PN', 'VV', 'AD', 'NN'], ['AD', 'PN', 'AD', 'P', 'NN', 'VV', 'P', 'NN', 'JJ', 'NN'], ['AD', 'CS', 'VV', 'NN', 'VE', 'M', 'NN', 'DEG', 'AD', 'VV', 'P', 'NN', 'VV'], ['VV', 'NN', 'P', 'NR', 'VV', 'LC', 'VV', 'PN', 'NN', 'VV', 'CD', 'NN'], ['P', 'NN', 'VV', 'AD', 'PN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'NN'], ['NR', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NN', 'NN', 'ETC', 'NN', 'NN', 'VV', 'AD', 'VV', 'ETC'], ['AD', 'JJ', 'NN', 'CC', 'NN', 'SP', 'NN', 'NN', 'DT', 'NN', 'NN', 'SP'], ['PN', 'VV', 'DEC', 'NN', 'AD', 'VA', 'NN', 'AD', 'VV', 'VV', 'PN', 'AD', 'VC', 'NN', 'NN'], ['NT', 'P', 'NN', 'VV', 'VV', 'VV', 'AD', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['VV', 'VV', 'VV', 'AS', 'DT', 'NN', 'NN', 'VV', 'NN', 'VV', 'AS', 'VV'], ['VV', 'AS', 'AD', 'CD', 'M', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['AD', 'JJ', 'DEG', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'SP', 'SP', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NT', 'NN', 'NN', 'NN', 'NN', 'VC', 'CD', 'NN', 'NN', 'VV'], ['AD', 'NN', 'DEG', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'DT', 'NN', 'NN', 'NN', 'NN', 'DEG', 'AD', 'VA', 'NN'], ['PN', 'NN', 'AD', 'VV', 'AD', 'VV', 'P', 'DT', 'NN', 'AD', 'VV', 'NN', 'AD', 'VA', 'SP'], ['AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['AD', 'PN', 'AD', 'VV', 'SP'], ['NR', 'NN', 'AD', 'AD', 'VV', 'VV', 'PN', 'AD', 'AD', 'SB', 'VV'], ['PN', 'NN', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'DEG', 'NN', 'SP'], ['CD', 'NN', 'VV', 'VV', 'NR', 'NN', 'AD', 'CD'], ['NR', 'ETC', 'NN', 'P', 'NR', 'SB', 'VV', 'NR'], ['NR', 'NN', 'VV', 'CD', 'AD', 'NN', 'LB', 'NR', 'NN', 'VV', 'NT', 'JJ', 'NN'], ['AD', 'NN', 'VV', 'SP'], ['NN', 'SP', 'PN', 'AD', 'VV', 'VV', 'DEV', 'VV', 'PN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'VA'], ['NN', 'NN', 'NN', 'P', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'P', 'PN', 'VA', 'DEC', 'NN', 'AD', 'PN', 'CS', 'VV', 'VV', 'AD', 'VV', 'VV', 'AS', 'PN'], ['VV', 'NN', 'LC', 'AD', 'AD', 'VV', 'NN', 'CC', 'NN', 'VV'], ['PN', 'AD', 'VV', 'NR', 'NR', 'DEG', 'NN', 'VV', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'NR'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'VV', 'NN', 'JJ', 'NN', 'DEG', 'NN'], ['NT', 'NN', 'LC', 'DEG', 'NN', 'VV', 'CD', 'NT', 'DEG', 'NR'], ['NR', 'NN', 'VV', 'MSP', 'VV', 'SP'], ['NR', 'VV', 'AS', 'NR', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'CD', 'NN', 'CC', 'NN'], ['P', 'NR', 'VV', 'NR', 'AD', 'VV', 'M', 'NN', 'VV', 'DEC', 'NN', 'PN', 'AD', 'AD', 'VV', 'AS', 'DT', 'M', 'NN'], ['JJ', 'NT', 'NN', 'P', 'AD', 'VV', 'JJ', 'NN', 'AD', 'VA'], ['NR', 'AD', 'VV', 'DT', 'JJ', 'NN', 'VV', 'AS', 'JJ', 'NN', 'VV', 'P', 'NN', 'LC', 'CD', 'VV', 'VV'], ['AD', 'AD', 'PN', 'VV', 'PN', 'PN', 'PN', 'AD', 'VV'], ['PN', 'NT', 'PN', 'VV', 'AS'], ['VV', 'SB', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'LC', 'VV', 'NN', 'MSP', 'VV', 'CC', 'SB', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV'], ['NT', 'NR', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'VA'], ['NR', 'NN', 'CC', 'NN', 'VV', 'VV', 'NR', 'NN', 'VV', 'VV', 'JJ', 'DEG', 'NN'], ['NT', 'PN', 'VV', 'VV', 'NN', 'AD', 'VA'], ['PN', 'P', 'NR', 'NN', 'NN', 'NR', 'AD', 'VV', 'VA', 'VA', 'DEC', 'NN'], ['VV', 'NN', 'DEC', 'P', 'DT', 'VV', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV', 'CD', 'NN', 'VV'], ['NN', 'VV', 'AD', 'P', 'VA', 'MSP', 'VV'], ['AD', 'VV', 'JJ', 'NN', 'NN'], ['CS', 'P', 'NR', 'NN', 'VV', 'AD', 'VV', 'NR', 'NN', 'NN', 'VV'], ['VV', 'NN', 'LC', 'NR', 'VV', 'BA', 'VV', 'JJ', 'NN', 'VV', 'NR'], ['P', 'NT', 'VV', 'DT', 'NN', 'NN', 'VC', 'NT', 'CD', 'CD', 'CD', 'CD', 'M'], ['NN', 'NN', 'VV', 'VV', 'AD', 'AD', 'VV', 'JJ', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'PN', 'VA', 'SP'], ['NN', 'LC', 'VE', 'CD', 'NN', 'CC', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'DEG', 'NN', 'LB', 'NN', 'VV', 'VV'], ['OD', 'NN', 'NR', 'VV'], ['AD', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NR', 'NN', 'NN', 'NR', 'AD', 'VV', 'VV', 'VV', 'JJ', 'NN'], ['NR', 'VV', 'NN', 'NR', 'AD', 'VV', 'NN', 'NN', 'AD', 'AD', 'VA'], ['NR', 'PN', 'VA', 'DEV', 'VV', 'AS', 'NN'], ['NN', 'CC', 'NN', 'LB', 'NR', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'VV'], ['P', 'VV', 'NR', 'NN', 'DEG', 'NN', 'LC', 'NR', 'VV', 'VV', 'VV'], ['PN', 'NN', 'CD', 'AD', 'VV'], ['AD', 'NR', 'SB', 'AD', 'NN', 'AD', 'VV', 'VV', 'P', 'NN'], ['NN', 'AD', 'VV', 'NN', 'VE', 'NN', 'VV', 'DT', 'NN', 'AD', 'VA'], ['NR', 'NN', 'NN', 'NN', 'NN', 'VV', 'DT', 'NN', 'AD', 'VA', 'VV'], ['VV', 'NN', 'P', 'CD', 'M', 'CD', 'CD', 'NN'], ['NN', 'NT', 'VV', 'NR', 'CD', 'NN'], ['NN', 'NR', 'NN', 'CC', 'NR', 'NN', 'NN', 'LC', 'DEG', 'NN'], ['VA', 'DEC', 'NN', 'AD', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NR', 'AD', 'VV', 'VV', 'VV', 'NN', 'AD', 'AD'], ['PN', 'VV', 'AS', 'JJ', 'DEG', 'NN', 'P', 'VV', 'CD', 'NN', 'VV', 'VV', 'PN', 'VV'], ['P', 'PN', 'P', 'NN', 'AD', 'VA', 'NN', 'SB', 'PN', 'VV', 'DER', 'VA'], ['NR', 'NR', 'NR', 'AD', 'P', 'NN', 'VV'], ['NN', 'VV', 'VV', 'NN', 'VV', 'DEC', 'VE', 'NR', 'NR', 'NR', 'NR', 'VV', 'NN', 'NN', 'ETC'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'JJ', 'AD', 'VE', 'DT', 'NN', 'SP'], ['NN', 'VV', 'VV', 'PN', 'DEG', 'NN', 'VV', 'DEC', 'CD', 'NN'], ['DT', 'JJ', 'AD', 'VV', 'DT', 'CD', 'NN', 'NR', 'AD', 'LB', 'NN', 'VV'], ['AD', 'VV', 'DT', 'JJ', 'DEG', 'AD', 'VV', 'PN', 'NN', 'VV', 'SP', 'SP'], ['VV', 'NN', 'LC', 'DEG', 'NR', 'NN', 'NN', 'NR', 'VA'], ['NN', 'VV', 'VV', 'PN', 'DEG', 'NN', 'VV', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'AD', 'VA', 'DT', 'AD', 'VV', 'NN'], ['CD', 'JJ', 'NN', 'NR'], ['AD', 'NR', 'NR', 'AD', 'AD', 'VV', 'PN', 'VV', 'AD', 'AD', 'VV', 'AD', 'VV', 'SP'], ['AD', 'VV', 'AD', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV'], ['P', 'NN', 'NN', 'VV', 'VV'], ['AD', 'NN', 'NN', 'DT', 'NN', 'DEG', 'NN'], ['NN', 'AD', 'AD', 'VV', 'VV', 'DT', 'DEG', 'NN'], ['NR', 'NN', 'AD', 'VC', 'NN', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NR', 'NN', 'VV', 'SP'], ['AD', 'NN', 'VV', 'AS', 'M', 'NN'], ['VV', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NR', 'AD', 'NN', 'LC', 'NN', 'VV'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'VV', 'NR', 'NN', 'VV'], ['PN', 'VV', 'JJ', 'PN', 'AD', 'VA', 'SP'], ['NN', 'VV', 'NN', 'AD', 'AD', 'VA'], ['NN', 'NN', 'NN', 'VV', 'PN', 'VV', 'AD', 'AD', 'VV', 'AS'], ['AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'P', 'PN', 'DEG', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'DEC', 'VV', 'VV', 'AD', 'CD'], ['NN', 'VV', 'VC', 'VV', 'DEC', 'NR', 'NN'], ['NN', 'NN', 'VV', 'NN', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV'], ['VV', 'VV', 'VV', 'P', 'PN', 'VA', 'VA', 'SP'], ['NR', 'NN', 'VV', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['AD', 'PN', 'VV', 'AD', 'VV', 'NN'], ['AD', 'PN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NR', 'VV', 'VV', 'NR', 'VV', 'AD'], ['DT', 'NN', 'AD', 'VV', 'NN', 'NN', 'LC'], ['AD', 'NN', 'NN', 'P', 'NN', 'LC', 'AD', 'VC', 'AD', 'VV', 'SP'], ['P', 'NN', 'VV', 'VC', 'NN', 'NN'], ['NR', 'VV', 'PN', 'P', 'NR', 'NN', 'VC', 'JJ', 'NN'], ['PN', 'AD', 'VV', 'VV', 'VV', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'PN', 'AD', 'VV', 'SP'], ['VV', 'PN', 'DEG', 'NN', 'VV', 'VV', 'NN', 'AD', 'VA', 'AD'], ['AD', 'VV', 'NN', 'VA', 'VV', 'NN', 'VA'], ['NN', 'AD', 'P', 'NR', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN', 'CC', 'JJ', 'NN'], ['NR', 'NR', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VC', 'P', 'DT', 'VV', 'DEC'], ['PN', 'VV', 'AS', 'AD', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'NR'], ['NR', 'CC', 'NR', 'NT', 'LC', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NN', 'VV', 'NR', 'CD', 'NN', 'VA', 'DEC', 'NN', 'VV', 'P', 'VV', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'AD', 'P', 'PN', 'AD', 'AD', 'AD', 'PN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'AD', 'CD', 'NN', 'CC', 'NN', 'NN', 'DEG', 'NN', 'VV', 'JJ', 'NN'], ['CS', 'PN', 'VV', 'VA', 'AS', 'AD', 'VV', 'AS'], ['NN', 'AD', 'VV', 'NN', 'NR', 'AD', 'AD', 'AD', 'VA'], ['NN', 'NN', 'VE', 'NN', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'NN'], ['NN', 'AD', 'VC', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'NT', 'NT', 'NT', 'NR'], ['NN', 'VV', 'AD', 'BA', 'NN', 'VV', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'PN', 'NR', 'DEG', 'P', 'NN', 'VV', 'VV', 'SP'], ['NN', 'VV', 'VV', 'P', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VC', 'NN'], ['NN', 'AD', 'VA', 'DEC', 'AD', 'VA', 'DEC', 'NN', 'AD', 'P', 'NN', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'NN', 'VA', 'DEC', 'NR'], ['AD', 'VV', 'PN', 'NN', 'DEG', 'NN', 'NN'], ['AD', 'JJ', 'NR', 'AD', 'VV', 'VE', 'CD', 'NN', 'P', 'VV', 'VV', 'NN', 'MSP', 'VV', 'VV'], ['NT', 'DEG', 'NR', 'NN', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'SP', 'DEC', 'AD', 'VV', 'DER', 'PN', 'VV', 'AD', 'VV'], ['NT', 'AD', 'NN', 'AD', 'VV', 'VV', 'DT', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NR', 'NN', 'LC', 'SB', 'VV', 'VV', 'DEC', 'NN', 'JJ', 'NN'], ['PN', 'AD', 'P', 'PN', 'VV', 'CD', 'SP'], ['NR', 'JJ', 'NN', 'NR', 'VV', 'PN', 'VV', 'VV', 'DT', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'NR', 'NN', 'VV', 'NN'], ['NT', 'DT', 'NN', 'AD', 'AD', 'SB', 'VV'], ['NR', 'VV', 'DT', 'NN', 'AD', 'VE', 'NN'], ['VV', 'VV', 'NN', 'JJ', 'NN', 'NN', 'VV', 'AS'], ['NN', 'JJ', 'AD', 'VV', 'AS', 'AD', 'VV', 'DEC', 'NN'], ['NR', 'P', 'NN', 'NN', 'AD', 'AD', 'VA'], ['NR', 'VV', 'NN', 'VC', 'JJ', 'VA'], ['AD', 'NN', 'PN', 'P', 'NN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['P', 'NR', 'LC', 'NR', 'AD', 'VV', 'VV', 'PN', 'VV', 'VC', 'VV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'AD', 'VV', 'NR', 'NN'], ['AD', 'DT', 'NN', 'AD', 'PN', 'VV', 'SP'], ['JJ', 'NN', 'LC', 'NN', 'VC', 'NN', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VV', 'VV', 'AS', 'NN', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'VV', 'AS', 'DT', 'NN'], ['NT', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'VV', 'AD', 'AD'], ['NN', 'P', 'VV', 'NN', 'NN', 'NN', 'VA', 'DEV', 'P', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NR', 'NN', 'P', 'NN', 'MSP', 'VV'], ['NR', 'VV', 'VV', 'NN'], ['NR', 'VV', 'AS', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VV', 'AD', 'VV'], ['NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'VV'], ['VV', 'NR', 'DEG', 'NN', 'NR', 'AD', 'CC', 'VE', 'NN'], ['NT', 'NT', 'NN', 'P', 'NR', 'NR', 'NN', 'VV', 'VV', 'AD', 'JJ', 'NN'], ['NT', 'NN', 'NR', 'NN', 'VV'], ['PN', 'DEG', 'OD', 'NN', 'NN', 'CC', 'CC', 'AD', 'VA'], ['PN', 'CD', 'NN', 'PN', 'VV', 'AD', 'VV', 'AD', 'VV', 'SP'], ['AD', 'AD', 'CD', 'DEC'], ['PN', 'VE', 'NN', 'NN', 'NN'], ['NR', 'NN', 'DEG', 'NN'], ['PN', 'NT', 'AD', 'AD', 'VV'], ['NN', 'CC', 'NR', 'NN'], ['AD', 'P', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VE', 'DEC', 'VV', 'VE', 'DEC', 'VV', 'VA', 'VV', 'CC', 'PN', 'VV', 'PN', 'PN', 'VE', 'PN', 'AD', 'VA', 'SP', 'SP'], ['VC', 'PN', 'VV', 'AS', 'NN', 'SP'], ['VV', 'NN', 'SB', 'VV', 'NN', 'NN', 'AD', 'SB', 'VV', 'NN'], ['NN', 'VV', 'AS', 'NR', 'NR', 'NN', 'NN', 'DEG', 'CD', 'NN'], ['AD', 'P', 'VV', 'NR', 'NN', 'NR', 'AD', 'VV', 'AS', 'P', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'CS', 'VA', 'CC', 'P', 'NN', 'AD', 'VV', 'AD', 'JJ', 'NN', 'VV'], ['VV', 'NN', 'NN', 'NN'], ['NR', 'NT', 'P', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NR', 'P', 'NN', 'LC', 'AD', 'NN', 'AD', 'VC', 'VV', 'NN', 'VV'], ['AD', 'NN', 'NR', 'VV', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['AD', 'VE', 'NN', 'VC', 'VV', 'DEC', 'NN', 'NN', 'NN'], ['NR', 'VV', 'NR', 'NR', 'NN', 'AD', 'VV', 'NN', 'NN', 'SB', 'VV'], ['NR', 'NN', 'NN', 'VV', 'NN', 'P', 'NN', 'CC', 'NN', 'DEG', 'NN', 'VV'], ['VV', 'CC', 'VV', 'DT', 'NN', 'DEG', 'NN', 'AD', 'P', 'NN', 'VV'], ['NR', 'VV', 'VV', 'VV', 'NN', 'VV'], ['NR', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'NN', 'NN', 'IJ', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['NN', 'P', 'NN', 'VV', 'NN'], ['NR', 'VV', 'AD', 'LC', 'P', 'NR', 'VV', 'VV', 'PN', 'NN', 'CC', 'PN', 'VV'], ['NN', 'VV', 'NN', 'VC', 'AD', 'VV', 'NN', 'NN'], ['AD', 'SP', 'AD', 'CD', 'NN', 'PN', 'AD', 'VV', 'VV', 'SP', 'AD', 'AD', 'VV', 'VA'], ['NR', 'NR', 'AD', 'VV', 'AS', 'NN'], ['NR', 'AD', 'VV', 'NR', 'AD', 'VA'], ['PN', 'P', 'VE', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VA'], ['NN', 'AD', 'P', 'NN', 'VV', 'VV', 'NR', 'VV', 'VV', 'VV', 'VV'], ['AD', 'NR', 'AD', 'NN', 'VV', 'AS', 'CD', 'NN'], ['NR', 'AD', 'VV', 'VV', 'DT'], ['VV', 'VV', 'DEC', 'NR', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'NR', 'AD', 'VV', 'P', 'NN', 'LC'], ['VV', 'DEC', 'NR', 'AD', 'VV', 'AD', 'VV', 'VA'], ['PN', 'VV', 'DT', 'NN', 'VV', 'NR', 'NR', 'AD', 'VV'], ['NT', 'VV', 'DEC', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'NR', 'AD', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'DEG', 'NN', 'VV'], ['NN', 'VC', 'VV', 'DEC', 'AD', 'AD', 'VC', 'VA', 'SP'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV'], ['AD', 'PN', 'PN', 'VV', 'NN', 'CC', 'VV', 'PN', 'AD', 'VV', 'VV'], ['NR', 'NR', 'VV', 'LC', 'CD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'P', 'NN', 'LC'], ['JJ', 'NN', 'VV', 'LC', 'VV', 'VA'], ['PN', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'AD', 'SP'], ['AD', 'AD', 'VV', 'P', 'NN', 'LC', 'DEG', 'NN', 'VV'], ['PN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'PN', 'VV'], ['NN', 'NN', 'VV', 'VV', 'CC', 'VV', 'AS', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['P', 'NN', 'NN', 'LC', 'NN', 'VV', 'AS', 'NN', 'VV', 'AD', 'AD', 'VV', 'VA'], ['AD', 'VE', 'CD', 'NN', 'P', 'NN', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['PN', 'VV', 'VV', 'VV', 'NN', 'BA', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NT', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'DEC', 'VV', 'SP'], ['NN', 'NN', 'VC', 'NR', 'NN', 'DEG', 'NR', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'AD', 'VV', 'AD'], ['NT', 'DEG', 'NR', 'AD', 'VV', 'NN'], ['AD', 'PN', 'P', 'NN', 'P', 'NN', 'VV', 'NN', 'NN', 'AD', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VA', 'VV', 'NN'], ['NT', 'NT', 'P', 'NR', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['P', 'NT', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NR', 'NN', 'LC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'NN', 'VV', 'PN', 'NN', 'VV', 'VV', 'VV', 'NN'], ['NR', 'VV', 'NN', 'AD', 'P', 'NR', 'NN', 'VV', 'NN'], ['CD', 'M', 'LC', 'LB', 'NR', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'VV'], ['NR', 'NN', 'AD', 'VA'], ['AD', 'NR', 'DEG', 'NN', 'NR', 'VV', 'PN', 'DEG', 'NN', 'VC', 'NR', 'LC', 'NN', 'NR'], ['NN', 'NN', 'NR', 'AD', 'NN', 'NN', 'LC', 'VV', 'NN'], ['VV', 'NN', 'VV', 'NN', 'SP'], ['CS', 'P', 'PN', 'VV', 'NN', 'VV'], ['NR', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NN', 'CD', 'NN', 'NN', 'NN', 'AD', 'NN', 'NN'], ['VV', 'VV', 'VV', 'DEC', 'NN', 'NN', 'NN', 'NR', 'NN', 'M', 'NN', 'DEV', 'VV', 'AS'], ['AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'DEC', 'JJ', 'DEG', 'NN', 'BA', 'PN', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'VV', 'PN', 'VC', 'AD', 'VV', 'SP'], ['P', 'DT', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'VV', 'AD'], ['NN', 'VV', 'VV', 'VV', 'AD', 'VV', 'DEC', 'NN', 'VV', 'VA'], ['NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['P', 'NN', 'VV', 'NT', 'VV', 'NR', 'DEG', 'NN', 'NN', 'VV'], ['VV', 'P', 'NN', 'VV', 'NN', 'VV', 'VV', 'AD', 'VV'], ['NR', 'NR', 'DEG', 'NN', 'P', 'NN', 'LC', 'VV', 'VV', 'AD', 'NR', 'NR', 'VA'], ['NN', 'AD', 'VV', 'PN', 'AS', 'AD', 'VV', 'SP'], ['NN', 'P', 'JJ', 'NN', 'VV', 'SP'], ['NN', 'AD', 'VV', 'VV', 'NR', 'NN'], ['NN', 'VV', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NR', 'NR', 'NN', 'AD', 'VV', 'NN', 'VV', 'NR', 'CC', 'NR'], ['VV', 'PN', 'VV', 'PN', 'AD', 'NN', 'VV', 'PN', 'NN'], ['AD', 'VV', 'NR', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['NR', 'AD', 'AD', 'VV', 'DT', 'NN', 'NN', 'AD', 'VE', 'NN', 'VV'], ['NN', 'LC', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV'], ['P', 'NR', 'CC', 'NR', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VC', 'NR', 'NR', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'VV', 'AS', 'VV', 'VV', 'NN'], ['NR', 'VV', 'VV', 'NN', 'VA', 'AS', 'AD'], ['NN', 'P', 'NN', 'AD', 'AD', 'BA', 'DT', 'NN', 'AD', 'VV', 'AS'], ['JJ', 'NN', 'NN', 'VV', 'AD', 'VC', 'CD', 'VV'], ['NN', 'LC', 'NR', 'VV', 'NN', 'CC', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV'], ['NR', 'DEG', 'NN', 'AD', 'AD', 'VV', 'P', 'VV', 'NN', 'LC', 'SP'], ['NN', 'NN', 'P', 'VV', 'NR', 'NN'], ['NN', 'P', 'NN', 'VV', 'AD', 'VV', 'P', 'NN', 'DEG', 'VV'], ['NR', 'VV', 'PN', 'AD', 'AD', 'VV', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN'], ['DT', 'NN', 'AD', 'VV'], ['NR', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'DEC', 'JJ', 'VV', 'JJ', 'AD', 'AD', 'VE', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'AD', 'VC', 'NN', 'JJ', 'DEG', 'NN', 'NN', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'VE', 'VV'], ['NN', 'NN', 'NN', 'P', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'AD', 'VV', 'NR', 'VV', 'AD', 'VV', 'AD', 'AD', 'AD', 'VA'], ['VV', 'P', 'NN', 'VV', 'P', 'NN', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV'], ['NT', 'NN', 'P', 'NN', 'VV', 'AS', 'NR', 'NN', 'DEG', 'NN', 'NN'], ['VV', 'NN', 'AD', 'VV', 'CS', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'CD'], ['AD', 'NN', 'NN', 'NN', 'NR', 'DEG', 'NN', 'AD', 'SB', 'VV'], ['NR', 'AD', 'JJ', 'DEG', 'VV', 'NN', 'NN'], ['AD', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NR', 'VV', 'AS', 'NN', 'VV', 'AD', 'NR', 'AD', 'AD', 'VV'], ['AD', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'VV', 'AS'], ['CS', 'NN', 'LC', 'VV', 'VV', 'VV', 'VV', 'NN', 'AD', 'AD', 'AD', 'AD', 'VA'], ['NR', 'CD', 'JJ', 'NN', 'SB', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['VV', 'DEC', 'NN', 'AD', 'VC', 'PN', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'NN', 'AD', 'LB', 'NN', 'VV', 'VV', 'NN', 'NN', 'LC', 'VV', 'VA', 'AS', 'NN'], ['NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'VE', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VE', 'PN', 'AD', 'VV', 'SP'], ['P', 'NN', 'VV', 'NN', 'DEG', 'NN', 'SP'], ['P', 'NN', 'NN', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'VV', 'CD'], ['NT', 'VE', 'NN', 'VV', 'PN', 'P', 'P', 'OD', 'NN', 'VV', 'NN'], ['AD', 'VV', 'VV', 'VV', 'VV', 'VV', 'NN', 'PN', 'VV', 'VV'], ['VV', 'P', 'JJ', 'NN', 'CC', 'NN', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'VV'], ['NT', 'VV', 'DEC', 'NN', 'NR', 'NN', 'AD', 'VA'], ['VV', 'NN', 'JJ', 'NN', 'NN', 'VV'], ['PN', 'VV', 'NN', 'VV', 'VA', 'DEC', 'NN'], ['AD', 'NR', 'VV', 'AS', 'M', 'NN', 'NN'], ['AD', 'PN', 'AD', 'VV', 'NN', 'AD', 'NN', 'VV', 'P', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV'], ['P', 'PN', 'LC', 'NN', 'NN', 'NN', 'AD', 'CD', 'CD', 'CD', 'CD', 'M'], ['AD', 'AD', 'NR', 'AD', 'P', 'NN', 'NN', 'NN', 'JJ', 'NN'], ['NR', 'CC', 'AD', 'VV', 'AD', 'VV', 'VV', 'NR', 'P', 'NR', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['CD', 'NN', 'VV', 'VC', 'NN'], ['PN', 'DEG', 'NN', 'VV', 'PN', 'VV', 'SP'], ['NN', 'NN', 'NN', 'VV', 'VV', 'NR', 'PN', 'DEG', 'NN', 'NN', 'NN'], ['VV', 'NN', 'DEC', 'NR', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV', 'AD', 'DEC', 'NN', 'NN'], ['NT', 'NN', 'AD', 'VV', 'LC'], ['DT', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'NR', 'VV', 'NN'], ['AD', 'AD', 'VV', 'NN', 'AD'], ['AD', 'VV', 'PN', 'AD', 'VV', 'NR'], ['CS', 'NN', 'JJ', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VA', 'CC', 'VV'], ['NN', 'NN', 'PN', 'VV', 'VE', 'NN'], ['NN', 'NN', 'AD', 'VC', 'VV', 'VV', 'VV', 'AD', 'VA'], ['NN', 'VV', 'NN', 'VV'], ['NR', 'NR', 'NN', 'VV'], ['VV', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'VV', 'LC', 'VV', 'VV', 'M', 'PN'], ['NN', 'NN', 'NN', 'VV', 'VV', 'NR', 'VV', 'NN', 'P', 'PN', 'VV', 'AD', 'VV'], ['PN', 'VV', 'NR', 'NR', 'P', 'NN', 'VV', 'AS', 'NN', 'NN', 'VV', 'AS', 'NN'], ['AD', 'NR', 'VV', 'PN', 'DEG', 'NN', 'VV', 'NN', 'CC', 'NR', 'VV'], ['AD', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NR', 'DEG', 'NN', 'VA'], ['PN', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'AS', 'NN', 'P', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['NR', 'VV', 'NR', 'VV', 'JJ', 'NN', 'NN', 'VC', 'VV', 'NR', 'NN', 'NN'], ['DT', 'NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VC', 'NN', 'NN', 'AD', 'VA', 'SP', 'AD'], ['VV', 'AD', 'NN', 'VV', 'VV', 'P', 'NN', 'LC', 'AD', 'VA'], ['AD', 'NR', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'DEC', 'JJ', 'NN'], ['NN', 'VV', 'DEC', 'JJ', 'NN', 'VC', 'AD', 'VV', 'BA', 'NN', 'AD', 'VV', 'NN', 'VV'], ['PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'VV'], ['PN', 'VV', 'P', 'NN', 'SP'], ['VV', 'SP', 'VA', 'DEC', 'NN', 'VV', 'SP', 'CD', 'M', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NR', 'VE', 'JJ', 'NN'], ['NT', 'DEG', 'NR', 'NR', 'ETC', 'NN', 'NN', 'NN', 'AD', 'VA', 'NN', 'VA'], ['NT', 'VV', 'AD', 'PN', 'DEG', 'NN', 'P', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV'], ['AD', 'NR', 'NN', 'P', 'NN', 'P', 'NN', 'NN', 'CC', 'NR', 'VV', 'NN', 'NN'], ['CD', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'CD', 'NN', 'CC', 'NN', 'NN', 'NN'], ['AD', 'NR', 'CC', 'NN', 'NN', 'NN', 'NN'], ['VV', 'NN', 'AD', 'VA', 'NN', 'NN', 'VV'], ['AD', 'NN', 'VA', 'AS', 'NR', 'VV', 'LC', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NN', 'PN', 'AD', 'VV', 'VV', 'PN', 'VV', 'NN', 'VV', 'SP', 'VV', 'NN', 'NN', 'AD', 'VA', 'SP'], ['AD', 'NN', 'AD', 'VV', 'PN', 'VV', 'NN', 'AD', 'VV', 'VV', 'PN', 'M', 'VV', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['AD', 'VV', 'NN', 'BA', 'PN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VC', 'NN', 'VA'], ['AD', 'VV', 'AD', 'VC', 'NN', 'CS', 'NN', 'AD', 'AD', 'VV', 'VV'], ['AD', 'VV', 'AD', 'VV', 'NR', 'NN', 'DEG', 'NN', 'NN', 'SP'], ['PN', 'CC', 'NN', 'NN', 'VV', 'VV', 'NR', 'NR', 'NN', 'VV'], ['AD', 'VV', 'VV', 'AD', 'VV', 'NN', 'JJ', 'NN', 'NN', 'NN'], ['CD', 'NN', 'AD', 'AD', 'AD', 'VV', 'AD', 'AD', 'AD', 'VV', 'AS', 'NN'], ['NN', 'NN', 'AD', 'AD', 'P', 'NN', 'NN', 'P', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NN', 'NR'], ['AD', 'VV', 'NN', 'PN', 'PN', 'VV', 'DEC', 'NN'], ['NR', 'AD', 'P', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN'], ['AD', 'VV', 'PN', 'NN', 'AD', 'VV', 'NR', 'PN'], ['PN', 'VV', 'NN', 'NN', 'NN'], ['NT', 'NR', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VV', 'AD', 'AD', 'AD', 'VV'], ['P', 'NR', 'CC', 'JJ', 'NN', 'ETC', 'NN', 'DEC', 'VV', 'LC', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN'], ['P', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'LC', 'NN', 'AD', 'P', 'NN', 'VV'], ['VE', 'AD', 'CD', 'M', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VV', 'NR', 'LC', 'PN'], ['VV', 'AD', 'AD', 'BA', 'CD', 'VV', 'CD', 'SP'], ['AD', 'DT', 'VV', 'DT', 'M', 'NN', 'SP'], ['VV', 'NN', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'NN', 'NN', 'NN', 'P', 'NN', 'NN'], ['NR', 'NR', 'AD', 'NN', 'AD', 'VV', 'VV'], ['P', 'NR', 'NN', 'LC', 'NN', 'VC', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NN', 'VV', 'AS', 'AS', 'NN', 'AD', 'VV', 'VV', 'VV', 'NT', 'CD', 'JJ', 'NN', 'NN', 'DEG', 'JJ'], ['JJ', 'CC', 'NN', 'NN', 'VV', 'AD', 'VC', 'P', 'NN', 'VV', 'VV'], ['NT', 'NR', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'AD', 'CD', 'CC', 'VA', 'CC', 'VA', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VV', 'CD', 'CD', 'CD', 'AD'], ['NN', 'VV', 'NN', 'VV', 'PN'], ['NR', 'AD', 'VV', 'JJ', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NN', 'VV', 'NT', 'CD', 'CD', 'CD', 'VV', 'NN', 'VV', 'LC', 'NN'], ['NR', 'VV', 'NR', 'NN', 'VV', 'CC', 'VV', 'JJ', 'NN'], ['NR', 'NN', 'AD', 'VA', 'VC', 'NR', 'NN', 'NN', 'JJ', 'VV', 'VA', 'DEC', 'NN', 'JJ', 'NN'], ['NN', 'LC', 'NN', 'JJ', 'NN', 'VA'], ['AD', 'PN', 'VV', 'VV', 'VV', 'VA', 'DEC', 'NN'], ['VV', 'VV', 'VV', 'NN'], ['NR', 'P', 'NR', 'VV', 'VV'], ['AD', 'NN', 'AD', 'VV', 'AS', 'CD', 'M'], ['AD', 'VV', 'VV', 'AD', 'VA'], ['AD', 'P', 'NR', 'DEG', 'NN', 'NN', 'NR', 'VV', 'VE', 'NN', 'DEC', 'NN'], ['NR', 'NN', 'NN', 'AD', 'AD', 'VV', 'DEC', 'VV', 'VV', 'NN', 'DEG', 'NN', 'SP'], ['PN', 'NN', 'CC', 'NN', 'NN', 'AD', 'P', 'NR', 'VV', 'NN', 'NR', 'NN', 'VV', 'VV'], ['VV', 'P', 'NN', 'CD', 'M', 'DEG', 'VV', 'NR', 'NN', 'VV', 'NN', 'VV', 'NN'], ['VV', 'DEC', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'AD', 'VV'], ['CS', 'NR', 'NN', 'VV', 'AS', 'VV', 'NN', 'NN', 'ETC', 'NN', 'AD', 'NN', 'AD', 'VA'], ['AD', 'AD', 'VV', 'AD', 'NN', 'NN', 'CC', 'JJ', 'DEG', 'JJ', 'NN'], ['NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS', 'VV', 'NN', 'NN', 'NN', 'AD', 'NN', 'VA'], ['DT', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'VC', 'NR', 'JJ', 'NN'], ['VV', 'AS', 'NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'AS', 'NN', 'VV', 'NN'], ['AD', 'CD', 'DEG', 'NN', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'AD', 'VE', 'NN', 'AD', 'VV'], ['NN', 'VA', 'DEV', 'VV', 'P', 'PN', 'VA', 'DEC', 'NN', 'SP'], ['P', 'AD', 'VV', 'AD', 'VV', 'NN', 'NN'], ['PN', 'DEG', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV', 'DER', 'NN', 'VA'], ['PN', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'NR', 'VV', 'NN'], ['VV', 'AS', 'AD', 'VV', 'AS', 'NN', 'NN'], ['VV', 'DEC', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VA'], ['DT', 'NN', 'NN', 'VC', 'VV', 'VV', 'NN', 'SP'], ['NR', 'AD', 'VV', 'VV', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN'], ['PN', 'VV', 'VV', 'NN', 'LC', 'VV', 'PN', 'NN', 'VV', 'VV', 'AS', 'NR', 'NN', 'AD', 'VV'], ['AD', 'JJ', 'PN', 'VV', 'NR', 'VV', 'NN'], ['PN', 'AD', 'DEV', 'NN', 'NN', 'VV', 'VV', 'SP'], ['NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'LC'], ['PN', 'VV', 'VV', 'VV', 'PN', 'AD', 'VV'], ['NN', 'VV', 'PN', 'VE', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN'], ['DT', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'DT', 'AD', 'NR', 'DEG', 'NN'], ['NN', 'NN', 'NR', 'NN', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NR', 'NR', 'VV', 'VV', 'AS', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'NR', 'NN', 'P', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'LC', 'PN', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['AD', 'VV', 'NN', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'VA'], ['PN', 'DEG', 'NN', 'CD', 'M'], ['NR', 'DEG', 'NN', 'NR', 'AD', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NT', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'LC', 'NN', 'VV', 'PN', 'AD', 'P', 'NN', 'NN'], ['AD', 'VC', 'P', 'NN', 'AD', 'VV', 'VV', 'SP'], ['PN', 'VV', 'JJ', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'NR', 'VV', 'AD'], ['NR', 'VV', 'DEC', 'NN', 'VV', 'P', 'NN', 'VV', 'NR', 'DEG', 'NN'], ['NN', 'VV', 'VV', 'NR', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VV', 'VV'], ['VV', 'PN', 'VV', 'PN', 'VV', 'AD'], ['NR', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'NR', 'VV', 'AD', 'PN', 'VV', 'AS'], ['AD', 'AD', 'VV', 'NN', 'P', 'NN', 'VV'], ['PN', 'CD', 'DEG', 'NN', 'P', 'PN', 'VV', 'DEC', 'VV', 'PN', 'VV', 'NN'], ['P', 'NN', 'DEG', 'NN', 'VA', 'NR', 'P', 'NN', 'NN', 'VV', 'AS', 'NR', 'NN'], ['PN', 'VV', 'AD', 'CD', 'NN', 'AD', 'P', 'NR', 'NN', 'JJ', 'NN', 'VV', 'VV'], ['NN', 'VA', 'DEC', 'NN', 'AD', 'AD', 'AD', 'VV', 'BA', 'PN', 'VV', 'M', 'NN'], ['NR', 'NR', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['NT', 'DEG', 'NR', 'NN', 'AD', 'VA'], ['NN', 'VV', 'VC', 'PN', 'NN', 'VV', 'PN', 'SP'], ['PN', 'AD', 'PN', 'AD', 'PN', 'VV', 'VV', 'VV'], ['VV', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'CC', 'NN'], ['NN', 'DEG', 'CD', 'NN', 'VE', 'NN', 'AD', 'AD', 'VA'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NT', 'NR', 'NN', 'BA', 'VV', 'DEC', 'NR', 'VV'], ['PN', 'P', 'NN', 'VV', 'VV', 'NR'], ['NN', 'NN', 'VV', 'SP'], ['P', 'VV', 'VV', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'P', 'NN', 'MSP', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VA', 'AD', 'DT', 'NN', 'NN', 'AD', 'VC', 'NN', 'AD', 'NN', 'VV'], ['AD', 'NR', 'AD', 'VV', 'NN', 'VV'], ['NR', 'DT', 'VV', 'NN', 'AD', 'VV', 'AS', 'NN'], ['NN', 'AD', 'VV', 'VV', 'VV', 'NR', 'VV', 'VV', 'SP'], ['AD', 'VV', 'NR', 'VV', 'NR', 'VV', 'AD', 'AD', 'VA', 'VA', 'SP'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NN', 'VV', 'DEC', 'NN', 'NN', 'JJ', 'JJ', 'NN', 'VA'], ['NN', 'AD', 'VV', 'AS', 'PN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NR', 'VV', 'NR'], ['NR', 'VV', 'NN', 'VV', 'VV', 'VV', 'SP', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'NR'], ['NN', 'AD', 'VV', 'VV', 'NN', 'NR'], ['NT', 'DEG', 'NN', 'NN', 'CC', 'NN', 'AD', 'AD', 'AD', 'VV', 'AS'], ['NN', 'VV', 'VV', 'NN', 'VE', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'AD', 'VV', 'CD'], ['NN', 'AD', 'VA', 'SP'], ['NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'VA'], ['NN', 'NN', 'NR', 'VE', 'JJ', 'NN', 'NN'], ['NN', 'AD', 'VV', 'VV', 'VV', 'NR', 'DEC', 'NN', 'NN'], ['NR', 'VV', 'VV', 'CD', 'NN', 'VV', 'VV', 'NN', 'AD', 'VA', 'SP'], ['P', 'PN', 'SP', 'VV', 'P', 'PN', 'JJ', 'DEG', 'NN'], ['NT', 'VE', 'NN', 'VV', 'AD', 'VV'], ['NN', 'AD', 'VV', 'DER', 'VV', 'VV', 'VV'], ['NN', 'AD', 'VE', 'CD', 'JJ', 'NN', 'VV', 'NN', 'LC', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'SP', 'AD', 'AD', 'VE', 'CD', 'NN', 'AD', 'VA'], ['AD', 'P', 'NN', 'LC', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NR', 'VC', 'P', 'NR', 'VV', 'NN', 'NN', 'NN', 'DEG', 'OD', 'CD', 'NN'], ['AD', 'AD', 'JJ', 'NN', 'CC', 'NN', 'NR', 'AD', 'VV', 'VV', 'AS', 'AD', 'NN'], ['PN', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NR', 'NN', 'VV', 'AD'], ['VV', 'NN', 'AD', 'AD', 'VV', 'CD', 'NN'], ['NN', 'PN', 'VC', 'VV', 'VV', 'PN', 'DEG', 'NN', 'SP'], ['VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN'], ['NN', 'NN', 'NR', 'NN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'NN', 'VV', 'NN', 'AD', 'VE', 'JJ', 'AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'AD', 'VA'], ['P', 'OD', 'VV', 'NN', 'NN', 'AD', 'VV', 'AS', 'NR', 'VV', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'VE', 'VV', 'NR', 'P', 'VV', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'VV'], ['VV', 'AS', 'NR', 'PN', 'AD', 'VV', 'VV', 'AS'], ['NT', 'VV', 'NN', 'VC', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'VV', 'VV', 'AD', 'AD', 'VV'], ['NN', 'CC', 'NN', 'AD', 'P', 'NN', 'VV', 'NN', 'LC', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'JJ', 'NN', 'P', 'NN', 'VV'], ['AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'NN', 'NN', 'VA', 'VA'], ['PN', 'AD', 'AD', 'VV', 'NN', 'VV', 'SP', 'PN', 'AD', 'P', 'PN', 'VV'], ['NN', 'LC', 'DEG', 'NN', 'VV', 'AS', 'AD', 'VV', 'CC', 'VV', 'NN', 'VV', 'PN', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NR', 'VV', 'AD', 'NN', 'VA', 'NN', 'VA'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV'], ['PN', 'VV', 'VV', 'PN', 'NN', 'NN', 'AD', 'VV', 'VV'], ['CS', 'PN', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'NN', 'VC', 'M', 'JJ', 'DEG', 'NN'], ['PN', 'VV', 'NN', 'VA', 'DEC', 'NN'], ['VV', 'LC', 'VV', 'VV', 'VV', 'DEC', 'VV', 'AS', 'NN', 'CD', 'M', 'NN'], ['NR', 'NR', 'VV', 'NR', 'NR'], ['P', 'DT', 'NN', 'LC', 'NR', 'PN', 'AD', 'VV', 'AD', 'VV'], ['NN', 'AD', 'VV', 'VV', 'PN', 'VV', 'NR', 'AD', 'VV'], ['PN', 'AD', 'P', 'P', 'NN', 'VV', 'P', 'NR', 'ETC', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'VE', 'VV', 'DEC', 'NN', 'NN', 'VA', 'AD', 'AD', 'VV'], ['NN', 'VA', 'DEC', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'AS', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'AD', 'SP'], ['PN', 'AD', 'AD', 'P', 'NN', 'NN', 'VV', 'AS'], ['PN', 'P', 'PN', 'NN', 'VV', 'AD', 'VV'], ['NN', 'AD', 'P', 'NN', 'VV', 'VV', 'NN', 'ETC', 'CC', 'NN', 'VV', 'AD', 'NN'], ['P', 'NR', 'P', 'NN', 'DEG', 'PN', 'AD', 'SB', 'VV', 'VV', 'NN'], ['AD', 'VV', 'NN', 'VV', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['VC', 'NN', 'VV', 'PN', 'VA', 'SP'], ['NN', 'NN', 'VE', 'NN', 'NR', 'VC', 'PN', 'DEG', 'NN', 'NN'], ['VV', 'VV', 'LC', 'NN', 'NN', 'AD', 'SB', 'VV', 'VA'], ['AD', 'VV', 'NN', 'SP', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'PN', 'SP'], ['NR', 'AD', 'DT', 'P', 'VV', 'NN', 'DEC', 'NN'], ['JJ', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN'], ['P', 'NN', 'CD', 'NN', 'AD', 'VV', 'M', 'NN', 'NN'], ['DT', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VV', 'LC', 'CD', 'NN'], ['VV', 'NR', 'NR', 'VV', 'NN', 'NN', 'VV', 'NR'], ['NN', 'NN', 'NN', 'ETC', 'NN', 'VV'], ['NN', 'DEG', 'DT', 'NN', 'BA', 'NN', 'VV', 'DER', 'AD', 'VA', 'SP'], ['NN', 'P', 'DT', 'NN', 'AD', 'VV', 'AS', 'PN', 'AD', 'AD', 'VC', 'VA', 'SP'], ['VV', 'NR', 'NN', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN'], ['PN', 'VV', 'VV', 'AD', 'VV', 'VV', 'NN', 'JJ', 'NN', 'DEG', 'NR', 'NN'], ['DT', 'NN', 'VV', 'VV', 'VA', 'DEC', 'NR', 'NN', 'SP'], ['PN', 'VV', 'NN', 'P', 'NN', 'P', 'NN', 'VV', 'VV', 'NN'], ['DT', 'NN', 'AD', 'VA', 'DEC', 'AD', 'VA'], ['PN', 'NT', 'LB', 'PN', 'VV', 'AS', 'PN', 'VV', 'VV', 'VA', 'AD', 'VV', 'VA', 'VV'], ['NR', 'VV', 'AD', 'VA', 'NN', 'VV', 'VV'], ['NT', 'AD', 'VA', 'AD', 'DEC', 'NN', 'NN', 'VV', 'PN', 'AD', 'VV', 'NN', 'PN', 'VV'], ['NR', 'VV', 'NR', 'NN', 'DT', 'NN', 'AD', 'AD', 'VV', 'VA'], ['NN', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'VC', 'VV', 'NN', 'VV'], ['NR', 'NN', 'NN', 'SP', 'AD', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'AS', 'AD', 'AD', 'AD'], ['NN', 'AD', 'VA', 'NN', 'DEG', 'NN', 'PN', 'VV', 'P', 'CD', 'JJ', 'NN', 'P', 'NN', 'VV', 'AS'], ['P', 'NN', 'DEG', 'CD', 'NN', 'ETC', 'NN', 'VV', 'PN', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'CS', 'VC', 'VV', 'NN', 'VV', 'VV', 'PN', 'VV', 'NN'], ['VV', 'VC', 'JJ', 'NN', 'NN', 'AD', 'VE', 'P', 'PN', 'NN', 'DEG', 'JJ', 'JJ', 'NN', 'NN'], ['PN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'PN', 'AD', 'NN', 'VV', 'DEV', 'VV', 'PN', 'AD', 'VV', 'SP'], ['P', 'NN', 'LC', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'VA', 'VA', 'AD', 'VV', 'AD', 'VE', 'NN'], ['NN', 'VV', 'NN', 'VV', 'JJ', 'NN'], ['VV', 'DEC', 'NN', 'P', 'NN', 'BA', 'NN', 'VV', 'VV', 'VV', 'NN', 'LC', 'PN', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VC', 'P', 'NR', 'NR', 'NN', 'VV', 'LC', 'DEG', 'AD', 'CD', 'NN', 'NN'], ['P', 'PN', 'VV', 'CD', 'NN'], ['VV', 'NN', 'VV', 'DER', 'AD', 'SP'], ['NR', 'VV', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'VE', 'NN', 'NN', 'DEC', 'NN'], ['NN', 'LC', 'VV', 'VV', 'P', 'NN'], ['NT', 'NN', 'VV', 'NN', 'NN', 'VV'], ['P', 'VV', 'NN', 'NN', 'AD', 'CC'], ['NN', 'LC', 'AD', 'VV', 'NN', 'NN'], ['NN', 'LC', 'AD', 'VV', 'NN', 'NN', 'PN', 'VV', 'VV', 'VV', 'NN', 'SP'], ['NN', 'VV', 'VV', 'DEC', 'NR', 'CC', 'NR', 'NR', 'AD', 'AD', 'VA'], ['AD', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'PN', 'VV'], ['NT', 'NN', 'NR', 'VV', 'PN'], ['P', 'AD', 'NN', 'PN', 'VV', 'P', 'NR', 'VV', 'NN'], ['VV', 'VV', 'LC', 'NN', 'AD', 'VC', 'AD', 'CD', 'VV'], ['NN', 'VV', 'NR', 'AD', 'AD', 'VV', 'VA', 'CC', 'VA'], ['NN', 'NN', 'AD', 'VV', 'PN', 'VE', 'DT'], ['AD', 'VV', 'VV', 'PN', 'AD', 'AD', 'VV', 'VV', 'NN', 'SP', 'VV', 'NN'], ['VV', 'NN', 'ETC', 'PN', 'VV', 'VV', 'VV', 'PN', 'VV', 'IJ'], ['PN', 'AD', 'VV', 'PN', 'NN', 'VC', 'VA', 'DEC', 'NN', 'VC', 'LB', 'NN', 'VV', 'SP'], ['PN', 'VV', 'NR', 'LC', 'DEG', 'NN', 'NN', 'VV', 'NN', 'PN', 'VC', 'PN', 'DEG', 'NN'], ['AD', 'NT', 'NT', 'NN', 'VV', 'SP'], ['VE', 'VA', 'DEC', 'NN', 'PN', 'AD', 'AD', 'VV', 'VV'], ['P', 'NT', 'VV', 'VV', 'NN', 'VV', 'AD', 'AD', 'VA', 'SP'], ['AD', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'PN', 'AD', 'JJ', 'CC', 'JJ', 'NN'], ['NR', 'AD', 'P', 'NN', 'NN', 'VV', 'NN', 'CC', 'NN'], ['LB', 'NN', 'VV', 'AS', 'NN', 'VV', 'AD', 'VV', 'CC', 'PN', 'VV', 'NN'], ['NN', 'DEG', 'PN', 'BA', 'PN', 'NN', 'VV', 'SP', 'SP', 'SP'], ['NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['VV', 'AS', 'VV', 'AS', 'NR', 'DEG', 'NN', 'P', 'VV', 'AD', 'P', 'NR', 'AD', 'VV'], ['NN', 'BA', 'JJ', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'NR', 'AD', 'AD', 'NN', 'SP'], ['NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN', 'JJ', 'DEG', 'NN'], ['PN', 'VV', 'VC', 'NR', 'NN', 'NR', 'NR', 'VV', 'DEC', 'NN'], ['NR', 'NR', 'VV', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'SP'], ['NR', 'VV', 'VV', 'NN', 'DEG', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VC', 'VA'], ['PN', 'VV', 'VV', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV'], ['VV', 'VV', 'LC', 'PN', 'VV', 'VV', 'VV'], ['DT', 'VA', 'VA', 'DEC', 'NN', 'NN', 'VC', 'JJ', 'NR', 'AD', 'VV', 'VV', 'DEC', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'PN', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['NR', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV'], ['NT', 'LC', 'NR', 'CC', 'NR', 'VV', 'AS', 'NN', 'NN'], ['IJ', 'AD', 'BA', 'PN', 'VV', 'AS', 'PN', 'VV', 'PN', 'VV', 'VV', 'NN', 'SP'], ['NN', 'NT', 'VV', 'VV', 'VV', 'AS', 'VV', 'DT', 'VV', 'DEC'], ['NR', 'AD', 'VV', 'JJ', 'NN', 'AD', 'VV', 'VV', 'AD', 'JJ'], ['NN', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VV', 'PN'], ['JJ', 'NR', 'VA', 'DEV', 'VV', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'AS', 'VV'], ['PN', 'VV', 'VV', 'AD', 'VV', 'P', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VA'], ['NR', 'NN', 'LC', 'DEG', 'NN'], ['AD', 'NR', 'NR', 'NN', 'VV', 'NT', 'VV', 'NR', 'NN', 'NN'], ['P', 'NR', 'VV', 'PN', 'AD', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'AS', 'CD', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VC', 'NN', 'NN', 'NN', 'NR'], ['NR', 'P', 'NR', 'VV', 'NR'], ['AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'ETC'], ['NR', 'AD', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['DT', 'NN', 'LC', 'VV', 'VV', 'NR', 'NN', 'NN', 'PN'], ['AD', 'PN', 'AD', 'AD', 'VV', 'VV', 'PN'], ['AD', 'NN', 'AD', 'AD', 'AD', 'DEV', 'VV', 'VV', 'VV', 'VV'], ['NN', 'JJ', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'AS', 'NN'], ['VV', 'NN', 'VV', 'NN', 'LC', 'AD', 'VV', 'VV', 'NN', 'NN'], ['NR', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['NN', 'NR', 'VV', 'VV', 'NN'], ['NN', 'NR', 'DEG', 'NN', 'VA'], ['AD', 'JJ', 'CD', 'VV', 'AD', 'VV', 'VA'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'NN', 'VV'], ['VV', 'AS', 'VV', 'CD', 'NR', 'NN', 'DEG', 'NN', 'VV', 'PN', 'MSP', 'VV', 'SP'], ['AD', 'NN', 'NN', 'SP'], ['NN', 'VE', 'AS', 'NN', 'NR', 'NN', 'VV', 'P', 'NN', 'VV'], ['NN', 'LC', 'NR', 'NR', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NN', 'NR', 'NN'], ['P', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'NN', 'DEC', 'VC', 'M', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'AD', 'VA'], ['NT', 'NR', 'NN', 'AD', 'VV'], ['AD', 'P', 'NR', 'NN', 'LC', 'NN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'VV', 'AS', 'NN', 'NN', 'NN', 'AD', 'PN', 'VV', 'VV', 'PN', 'PN', 'VC', 'AD', 'VV'], ['NR', 'AD', 'VV', 'VV', 'VV', 'CC', 'VV', 'NN', 'AD', 'VV', 'DT', 'NN', 'AD', 'VV', 'VV'], ['VV', 'P', 'PN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN', 'VV', 'NN'], ['VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'DEC', 'NN', 'NN', 'NN', 'NN'], ['PN', 'VV', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['CD', 'NN', 'VV', 'NR', 'VV', 'AD', 'P', 'NT', 'VV', 'AD', 'VV', 'VV'], ['NR', 'AD', 'VV', 'VV', 'P', 'JJ', 'NN', 'VV'], ['NT', 'NR', 'VV', 'NN', 'VV', 'NN'], ['VV', 'NN', 'P', 'PN', 'LC', 'AD', 'VA', 'AS', 'VV', 'BA', 'NN', 'DEG', 'NN', 'VV', 'VV', 'AS', 'SP'], ['AD', 'VV', 'LC', 'LC', 'NN', 'NN', 'JJ'], ['AD', 'NN', 'NN', 'NN', 'ETC', 'VE', 'NN', 'NN', 'SB', 'VV'], ['NN', 'VV', 'NN', 'VE', 'NN', 'NN', 'AD', 'VA', 'DEV', 'VV', 'PN'], ['AD', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'NN', 'VA', 'AD', 'VV'], ['AD', 'VV', 'AD', 'NN', 'VA', 'NT', 'PN', 'AD', 'AD', 'VV', 'VV', 'P', 'NN', 'VV'], ['NR', 'VV', 'AD', 'JJ', 'PN', 'VV', 'AD', 'VV', 'P', 'PN', 'VV'], ['AD', 'CS', 'AD', 'VV', 'P', 'NR', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN'], ['AD', 'AD', 'AD', 'VV', 'NR'], ['AD', 'VV', 'VV', 'NN', 'VV', 'PN', 'VV'], ['P', 'VV', 'NN', 'NR', 'NN', 'VV', 'NR', 'AD', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'DT', 'CD', 'VV', 'NN', 'NN'], ['NT', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'CD', 'CD', 'CD', 'CD', 'NN'], ['NN', 'NN', 'NN', 'VA', 'AS', 'NT', 'DEG', 'NN', 'VA'], ['P', 'NR', 'VV', 'LC', 'PN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'NN', 'VV', 'JJ', 'NN'], ['DT', 'NN', 'VV', 'SP'], ['NN', 'SP', 'PN', 'AD', 'VV', 'PN'], ['P', 'NN', 'NN', 'NN', 'VV', 'NN', 'AD', 'AD', 'VA', 'CC', 'VA'], ['NN', 'AD', 'VV', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'CC', 'CD', 'NN', 'P', 'VV', 'VE', 'NN', 'LC', 'PN', 'AD', 'VA', 'SP'], ['NR', 'P', 'NR', 'CD', 'JJ', 'NN', 'AD', 'VE', 'NN'], ['VV', 'NR', 'NR', 'JJ', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VA', 'AD', 'AD', 'VV', 'NN', 'DEG', 'NN', 'NN'], ['VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'DEV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'AD', 'SB', 'VV', 'AS', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'P', 'VV', 'NN', 'NN', 'DEG', 'NN', 'VV', 'PN', 'NN'], ['AD', 'NR', 'VV', 'NR', 'AD', 'BA', 'NN', 'VV', 'AD', 'AD', 'VV', 'VV'], ['NN', 'NN', 'JJ', 'NN', 'VV', 'AD', 'VV'], ['VV', 'NN', 'AD', 'SB', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VA'], ['NN', 'NN', 'NN', 'NN', 'LB', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NR', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NT', 'NN', 'NN', 'NT', 'VV', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'AS'], ['NN', 'VV', 'NR', 'CC', 'NR', 'DEG', 'NN', 'NN', 'NN', 'NN', 'VA'], ['NT', 'NN', 'AD', 'VV', 'AS'], ['AD', 'SB', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NR', 'NN', 'VV', 'NN'], ['PN', 'NT', 'VV', 'CD', 'NN', 'CD', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NN', 'VV', 'VV', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'PN', 'VE', 'NR', 'CC', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'NR', 'VV', 'AS', 'VV', 'JJ', 'NN', 'AD', 'VV'], ['VV', 'NN', 'DEC', 'CC', 'VE', 'NN', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VC', 'VV', 'NN'], ['AD', 'PN', 'SB', 'VV', 'NN', 'DT', 'NN', 'DEG', 'NN'], ['VV', 'AS', 'VV', 'PN', 'AD', 'NN', 'VA'], ['AD', 'AD', 'VV', 'AD', 'AD', 'PN', 'SP'], ['P', 'VV', 'NN', 'CC', 'NN', 'CD', 'NN', 'AD', 'VV', 'NN'], ['CS', 'VV', 'NN', 'SP', 'CD', 'M', 'AD', 'BA', 'NN', 'VV', 'AS'], ['NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'JJ', 'NN', 'NN'], ['DT', 'NN', 'VV', 'PN', 'VV', 'AD', 'VV'], ['NR', 'NR', 'VV', 'NN', 'NN', 'AD', 'VE', 'AS', 'JJ', 'DEG', 'NN'], ['P', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'CD', 'DEG', 'NN', 'VV', 'PN', 'DEG', 'SP'], ['NR', 'VA', 'VV', 'AD', 'VV', 'VV', 'AD', 'NT', 'VV', 'VV', 'VV'], ['P', 'CD', 'M', 'NR', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['VV', 'AD', 'AD', 'VV', 'AD', 'VV', 'VV', 'SP'], ['VV', 'NN', 'AD', 'P', 'VV', 'NN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'VV', 'VC', 'NN', 'AD', 'AD', 'VA', 'AD', 'VA'], ['AD', 'NN', 'AD', 'VV', 'VV', 'BA', 'NN', 'NN', 'AD', 'VV', 'NN'], ['P', 'NN', 'AD', 'AD', 'DEV', 'NN', 'NN', 'VV', 'DT', 'DT', 'NN', 'VV', 'DEC', 'NR'], ['VV', 'NN', 'NN', 'NN', 'NN'], ['P', 'P', 'JJ', 'JJ', 'NN', 'DEC', 'NN', 'LC', 'NR', 'NN', 'NN', 'NN'], ['AD', 'NR', 'P', 'PN', 'DEG', 'NN', 'VV', 'AS', 'NN', 'AD', 'PN', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN', 'AD', 'P', 'PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NR', 'LC', 'VE', 'PN', 'VV', 'AS', 'NR', 'DEG', 'NN'], ['P', 'NR', 'CD', 'NN', 'P', 'NN', 'VA', 'MSP', 'VV'], ['VV', 'NN', 'NR', 'NR', 'ETC', 'VV', 'NN'], ['VV', 'NN', 'NN', 'NN', 'VV', 'CC', 'NR', 'SP'], ['NN', 'AD', 'VV', 'SP'], ['NT', 'NT', 'NN', 'LC', 'NR', 'NN', 'NN', 'AD', 'VV', 'CD'], ['AD', 'NN', 'AD', 'VV', 'PN', 'AD', 'SB', 'VA', 'DEV', 'NN', 'NN', 'VV'], ['NN', 'NN', 'VV', 'VE', 'NN', 'VV', 'AD', 'DT', 'NT', 'VV', 'AD'], ['NN', 'NN', 'NN', 'VC', 'NN', 'NN', 'NN', 'LC', 'AD', 'CD', 'JJ', 'NN'], ['PN', 'NT', 'DEG', 'NN', 'NN', 'VV', 'VV', 'NN'], ['VV', 'AD', 'IJ', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'AS', 'PN', 'NN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'VV', 'VA', 'SP'], ['VV', 'NN', 'VV', 'CD', 'CD', 'CD', 'VV', 'NN', 'NN'], ['JJ', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VV', 'P', 'JJ', 'NN', 'AD', 'VC', 'AD', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'NR', 'AD', 'VV', 'LC', 'VV', 'DER', 'VV'], ['NN', 'VV', 'AD', 'VV', 'PN'], ['DT', 'NN', 'NN', 'AD', 'VC', 'NN'], ['NN', 'VE', 'VV', 'NN'], ['AD', 'DT', 'NN', 'NN', 'AD', 'VA'], ['VV', 'NN', 'AD', 'P', 'PN', 'VV'], ['NN', 'VV', 'PN', 'SP'], ['NN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'DEC', 'JJ', 'NN'], ['AD', 'NN', 'AD', 'VE'], ['NN', 'NR', 'VV'], ['NN', 'AD', 'P', 'PN', 'NT'], ['AD', 'AD', 'AD', 'VV', 'PN'], ['NN', 'NN', 'AD', 'VV'], ['VV', 'VV', 'PN', 'SP'], ['NN', 'VA', 'NN', 'AD', 'AD', 'VA'], ['NN', 'NN', 'VV', 'VV'], ['AD', 'AD', 'NR', 'NN'], ['AD', 'P', 'VV', 'VV'], ['NN', 'VV', 'NN'], ['AD', 'VV', 'SP'], ['NN', 'VV', 'AS', 'AD', 'VV', 'AD', 'VV'], ['AD', 'VV', 'VV', 'AD'], ['NN', 'VC', 'DT', 'NN', 'SP'], ['AD', 'VV'], ['PN', 'NN', 'VV', 'NN', 'AD', 'VE', 'NN'], ['NN', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['VV', 'JJ', 'NN', 'CC', 'NN', 'NN', 'NN'], ['AD', 'PN', 'AD', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'P', 'VV', 'AD', 'VV', 'AS', 'NN'], ['NN', 'VV', 'NR', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'NN', 'P', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'AD', 'AD', 'VV', 'VV', 'VV'], ['NN', 'NN', 'NN', 'VV', 'NN', 'AD', 'VC', 'JJ', 'DEG'], ['NR', 'AD', 'VV', 'VA', 'DER', 'AD', 'VV', 'AS', 'VV', 'VV', 'VV', 'AD'], ['NT', 'AD', 'VV', 'AD', 'VV', 'VA', 'AD'], ['NN', 'VV', 'NN', 'NN', 'VV', 'VV'], ['AD', 'VA', 'CC', 'VA', 'DEC', 'JJ', 'NN', 'VV', 'P', 'NN', 'NN'], ['PN', 'VV', 'VV', 'VV', 'VV', 'NN', 'VV', 'P', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VC', 'NN', 'NR', 'NR', 'DEG', 'CD', 'VV', 'NN'], ['VA', 'VA', 'DEC', 'NN', 'NN', 'P', 'NN', 'LC'], ['NN', 'SP', 'DT', 'AD', 'NN', 'AD', 'VC', 'JJ', 'DEG'], ['VV', 'AD', 'VV', 'DT', 'AD', 'VV', 'NN', 'AD', 'VE', 'NN'], ['AD', 'AD', 'VV', 'NN', 'LC', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'JJ', 'NN', 'NN', 'AD', 'AD', 'CD', 'NN', 'P', 'PN', 'VV'], ['NN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'VV', 'AD', 'VV', 'VV', 'AS', 'VV', 'AS', 'NN'], ['NR', 'AD', 'VV', 'NN'], ['NR', 'VV', 'NR', 'DEG', 'NN', 'NN', 'NR', 'NR', 'VV', 'VV', 'NN', 'VV', 'PN', 'VA'], ['VA', 'DEC', 'VC', 'DT', 'NN', 'NR', 'SB', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NR', 'VV', 'VV'], ['AD', 'NN', 'VV', 'NN', 'VV', 'CD', 'NN', 'LC', 'VV', 'AS', 'NN', 'DEG', 'NN', 'NN'], ['NT', 'NT', 'NT', 'P', 'VV', 'NN', 'NN', 'AD', 'VV'], ['JJ', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'VV', 'AS', 'NN', 'SP'], ['NN', 'NN', 'VV', 'AS', 'NN', 'DT', 'SP', 'VV', 'VV', 'P', 'NN', 'VV'], ['NN', 'VV', 'AS', 'VV', 'AD', 'AD', 'VV', 'JJ', 'DEG', 'NN', 'AD', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'VC', 'CD', 'NT', 'NN', 'NT'], ['PN', 'VV', 'PN', 'PN', 'AD', 'PN', 'VV', 'AD', 'VV', 'SP'], ['NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'JJ', 'NN'], ['PN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'PN', 'VC', 'NN', 'NN', 'VV', 'AS'], ['NR', 'NN', 'DEG', 'NN', 'AD', 'P', 'NN', 'NR', 'AD', 'AD', 'VV', 'JJ'], ['AD', 'VV', 'P', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['AD', 'NN', 'NN', 'LC', 'AD', 'VA', 'DEC', 'VC', 'NN', 'NN', 'VA'], ['AD', 'VV', 'NN', 'AD', 'VV', 'AD', 'VA', 'VV'], ['VV', 'DER', 'PN', 'VV', 'AD', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV'], ['VV', 'DEC', 'NN', 'NN', 'VV', 'PN', 'NN', 'AD', 'VV', 'VV', 'VV'], ['OD', 'DT', 'PN', 'VV', 'PN', 'AD', 'VV', 'DT', 'NN', 'PN', 'AD', 'AD', 'VV'], ['AD', 'VA', 'PN', 'AD', 'VV', 'SP'], ['PN', 'P', 'NN', 'VV', 'CD', 'CD', 'CD', 'CD', 'CD', 'M', 'NN', 'AD', 'VV', 'LC', 'VV'], ['NN', 'NN', 'VV', 'NN', 'VA', 'VC', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'NR', 'NN', 'CC', 'NN', 'JJ', 'DEG', 'NR', 'JJ', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NR', 'DEG', 'VV', 'NN', 'AD', 'VV'], ['NN', 'AD', 'P', 'NR', 'ETC', 'NN', 'AD', 'P', 'PN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'NN'], ['PN', 'VV', 'CS', 'NN', 'VV', 'PN', 'AD', 'VV', 'VV', 'NR', 'NN'], ['AD', 'NR', 'NR', 'VV', 'VV', 'PN', 'SB', 'VV', 'AS', 'CD', 'M'], ['JJ', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['NN', 'VV', 'VV', 'NN', 'AD', 'AD', 'P', 'NR', 'VV', 'LC', 'NN', 'DEG', 'NN', 'VV', 'AS'], ['VV', 'NN', 'VV', 'AD', 'NN', 'NN', 'SP'], ['AD', 'NN', 'LC', 'VV', 'VV', 'NN'], ['NR', 'NN', 'LC', 'DEG', 'NN', 'NN', 'NR', 'NN', 'VV', 'DER', 'VV'], ['AD', 'NN', 'VV', 'CD', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NT', 'DT', 'M', 'VV', 'P', 'NN', 'LC', 'VV', 'AS', 'CD', 'NN'], ['P', 'NN', 'NR', 'CC', 'VA', 'LC', 'NR', 'NR', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VC', 'P', 'NN', 'DEG', 'NN', 'NN', 'SP', 'VV', 'AD', 'VV', 'PN', 'VV', 'NN'], ['NT', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'VV', 'AD', 'VE', 'NN'], ['AD', 'P', 'NN', 'VV', 'NN', 'VA', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'LC', 'DEG', 'AD', 'VV', 'NN'], ['VV', 'NN', 'NN', 'NR', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN', 'NN', 'VV'], ['NT', 'PN', 'VV', 'NN', 'VV', 'PN', 'VV', 'NN'], ['AD', 'NN', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'AS'], ['NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'JJ', 'NN', 'VV', 'JJ', 'NN'], ['P', 'NN', 'VA', 'VA', 'LC', 'PN', 'VC', 'M', 'NN', 'CS', 'NN', 'AD', 'AD', 'VV', 'NN'], ['P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['AD', 'NN', 'VV', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'PN', 'VV', 'VA'], ['AD', 'VV', 'PN', 'NN', 'VA', 'NT', 'LC', 'VV', 'DEC', 'NN', 'VV', 'JJ', 'NN', 'AD', 'VV', 'VV', 'VA'], ['P', 'AD', 'NN', 'CC', 'JJ', 'NN', 'DT', 'NN', 'VV', 'AD', 'VV'], ['NN', 'P', 'NN', 'VV', 'NN'], ['P', 'JJ', 'NN', 'NT', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'NR', 'VV', 'NN'], ['VV', 'NR', 'NN', 'VV', 'NN', 'VV', 'NN', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'NN'], ['AD', 'VV', 'NN', 'P', 'DT', 'CD', 'NN', 'VV', 'NN'], ['NN', 'CD', 'NN', 'AD', 'AD', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'VC', 'VA', 'NR'], ['NR', 'P', 'NN', 'NN', 'LC', 'VV', 'NN', 'NR', 'VV', 'NN', 'VV', 'NN'], ['P', 'AD', 'VV', 'NN', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['NR', 'NN', 'NN', 'VA', 'DEC', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'VC', 'AD', 'VV', 'SP'], ['IJ', 'PN', 'VV', 'PN', 'VC', 'AD', 'VV', 'AD', 'CD', 'SP'], ['AD', 'VV', 'VV', 'P', 'P', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'AD'], ['PN', 'VV', 'VV', 'NN', 'VV', 'NR', 'DEG', 'JJ', 'NN', 'SP'], ['PN', 'VV', 'VV', 'NN', 'NN', 'VC', 'JJ', 'NN', 'NN'], ['NR', 'P', 'NN', 'LC', 'VV'], ['AD', 'VV', 'P', 'VV', 'VV', 'NN', 'NN', 'VC', 'CD', 'CD', 'CD', 'CD'], ['AD', 'LC', 'P', 'NN', 'LC', 'PN', 'VE', 'NN'], ['NR', 'SB', 'VV', 'DER', 'VV', 'VV', 'NN', 'VV', 'DEV', 'VV', 'VV'], ['NT', 'LC', 'AD', 'JJ', 'NN', 'SB', 'NR', 'VV', 'CD', 'CD', 'CD', 'VV', 'LC'], ['AD', 'VE', 'NR', 'DEG', 'SB', 'VV', 'CC', 'VV', 'NN', 'VA', 'SP'], ['PN', 'DEG', 'JJ', 'DEG', 'NN', 'VV', 'P', 'JJ', 'NN', 'P', 'NN', 'VV', 'AD', 'VV', 'DT', 'P', 'DT'], ['NR', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'NR', 'NR', 'VV', 'NN'], ['NR', 'P', 'NR', 'NN', 'NN', 'VV'], ['NN', 'AD', 'BA', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV'], ['VV', 'NN', 'CC', 'NR', 'NT', 'NT', 'VV'], ['NT', 'NR', 'AD', 'LB', 'NR', 'NN', 'NN', 'VV', 'NN'], ['AD', 'P', 'NR', 'DEG', 'NN', 'AD', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NN', 'LC', 'DEG', 'NN', 'NN', 'NN', 'NT', 'AD', 'VV', 'AS', 'NN', 'ETC', 'NN', 'DEG', 'NN'], ['PN', 'VC', 'P', 'NR', 'NN', 'NN', 'NN', 'NR', 'VV', 'DEC', 'NN', 'NN', 'NN'], ['NR', 'DEG', 'NN', 'VV', 'NN', 'SP', 'NN', 'VA'], ['VV', 'PN', 'VV', 'AD', 'VV', 'VV', 'AS', 'DT', 'VV', 'SP'], ['AD', 'JJ', 'NR', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'ETC', 'NN'], ['CD', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['VV', 'NR', 'NR', 'VV', 'P', 'NN', 'LC', 'PN', 'DEG', 'NN', 'P', 'NN', 'VV'], ['NN', 'NR', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VA'], ['VV', 'SP', 'VE', 'NN', 'AD', 'PN', 'AD', 'VV', 'DEC', 'NT', 'NN', 'AD', 'AD', 'VV', 'VV', 'DER', 'VV', 'CD'], ['PN', 'VV', 'VV', 'NN', 'VV'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NN', 'P', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV', 'NT'], ['P', 'VV', 'DEC', 'NN', 'VV', 'NN', 'SP'], ['NR', 'P', 'NN', 'VV', 'PN', 'VV', 'NN', 'NN', 'NN', 'AD', 'NN', 'VV', 'PN', 'VV'], ['VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'LC', 'NR', 'AD', 'VV'], ['DT', 'NN', 'AD', 'NN', 'NN', 'NN', 'NN', 'NN'], ['JJ', 'JJ', 'DEG', 'NN', 'VV', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['NR', 'NN', 'VV', 'NN', 'NN'], ['NT', 'VV', 'DEC', 'NN', 'AD', 'VV'], ['PN', 'P', 'NN', 'LC', 'DEG', 'NR', 'VV'], ['NN', 'AD', 'NN', 'NN', 'DEV', 'VV', 'NR', 'NN'], ['NN', 'VV', 'NR', 'CC', 'NR', 'NR', 'VC', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'NN', 'VV', 'PN', 'NN', 'AD', 'VA'], ['P', 'NR', 'VV', 'PN', 'AD', 'VV', 'NN', 'NN', 'VE', 'NN', 'AD', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'P', 'NN', 'VV', 'NN', 'VV', 'VV', 'SP'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'VV', 'NN'], ['NT', 'NR', 'VC', 'NN', 'P', 'NR', 'AD', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['NR', 'VV', 'VV', 'NN', 'VV', 'AD', 'DEC', 'NN'], ['NN', 'NR', 'VV', 'NN', 'PN', 'AD', 'P', 'NN', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NN'], ['NT', 'NR', 'NN', 'VE', 'CD', 'VV', 'AD', 'VE', 'NN'], ['P', 'PN', 'VV', 'PN', 'DEC', 'NN', 'PN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'P', 'NN', 'VV'], ['NN', 'NR', 'PN', 'VE', 'VV', 'NR', 'NN', 'NR', 'NR', 'VV', 'P', 'NN', 'NN'], ['NN', 'AD', 'VE', 'NN', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'ETC'], ['AD', 'NN', 'NN', 'AD', 'AD', 'VA'], ['VV', 'DEV', 'VV', 'PN', 'VV', 'NN', 'VV', 'PN', 'SP'], ['NN', 'VV', 'CD', 'NN', 'DEG', 'NN'], ['P', 'PN', 'VV', 'AD', 'AD', 'AD', 'P', 'PN', 'CC', 'JJ', 'CC', 'JJ', 'DEG'], ['PN', 'NN', 'VV', 'NR', 'VV', 'LC', 'DEG', 'NN', 'NN'], ['NR', 'BA', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN', 'NN'], ['NR', 'AD', 'VV', 'VV', 'JJ', 'NN', 'DEG', 'NR', 'NN'], ['NN', 'NN', 'VC', 'VV', 'DEC', 'JJ', 'NN'], ['AD', 'VE', 'CD', 'NN', 'PN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['DT', 'NN', 'VC', 'NN', 'NN', 'NN', 'NN', 'LC', 'DEG', 'JJ', 'NN'], ['AD', 'VV', 'AS', 'NR'], ['AD', 'AD', 'VC', 'P', 'NN', 'LC', 'P', 'NN', 'LC', 'VV', 'NN', 'NN', 'NN', 'AD', 'VA'], ['NN', 'VV', 'DEC', 'JJ', 'NN', 'VV', 'DEC', 'NN', 'VA', 'CC', 'VA', 'VA'], ['OD', 'VC', 'VV', 'VV', 'VV', 'LC', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'CD', 'JJ', 'DEG', 'NN', 'NN'], ['NR', 'NN', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV', 'AS', 'NN', 'VV'], ['P', 'CD', 'CD', 'CD', 'CD', 'NN', 'VE', 'NN', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NN', 'CC', 'NN', 'CC', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AS'], ['P', 'VV', 'NR', 'LC', 'NR', 'NN', 'LB', 'NN', 'VV'], ['CS', 'AD', 'VV', 'LC', 'PN', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'P', 'NN', 'VV', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN', 'VV', 'AD', 'AD', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['PN', 'AD', 'BA', 'DT', 'JJ', 'NN', 'VV', 'VV', 'PN', 'AS', 'VA', 'SP', 'VV', 'VV', 'NN', 'SP', 'PN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VC', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['PN', 'VE', 'CD', 'VV', 'NN', 'DEC', 'NN'], ['CD', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV'], ['NR', 'NN', 'VV', 'NN', 'P', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD'], ['AD', 'NN', 'P', 'JJ', 'NN', 'NN', 'AD', 'VA', 'AD', 'AD', 'VV', 'NN', 'VV'], ['NN', 'AD', 'VV', 'AS', 'VV', 'PN', 'VV', 'NR'], ['NN', 'LC', 'AD', 'AD', 'AD', 'VV', 'NN'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VC', 'M', 'JJ', 'NN'], ['DT', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['NN', 'VA', 'P', 'NR', 'VV', 'NN'], ['NT', 'LC', 'PN', 'DEG', 'NN', 'NN', 'VV', 'AS'], ['NR', 'AD', 'AD', 'NR', 'VV', 'NN', 'NN', 'VV', 'PN', 'VV', 'SP'], ['P', 'VV', 'NN', 'NR', 'NN', 'NN', 'PN', 'VV', 'AS', 'NR', 'DEG', 'NN'], ['NN', 'VV', 'AD', 'VV', 'VA', 'VA'], ['NN', 'CD', 'M', 'VV', 'VV', 'AS', 'AD', 'NN', 'AD', 'VV', 'AS', 'NR'], ['JJ', 'NN', 'NN', 'LC', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NR', 'AD', 'VV', 'NN', 'CS', 'NR', 'P', 'PN', 'VV', 'NN', 'AD', 'P', 'NR', 'NN', 'VV'], ['NT', 'NN', 'AD', 'VV', 'AS', 'VA', 'AD', 'AD', 'VV', 'VV'], ['CS', 'NN', 'VA', 'NN', 'VA', 'AD', 'VV', 'AD', 'CD'], ['NR', 'JJ', 'CC', 'VV', 'NN', 'SP', 'SP'], ['AD', 'P', 'AD', 'VA', 'DEC', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'PN', 'VV'], ['NR', 'VV', 'VV', 'M', 'VV', 'PN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'AD', 'VV', 'VV', 'BA', 'NN', 'NR', 'NR', 'VV', 'VV'], ['AD', 'P', 'VV', 'LC', 'NN', 'AD', 'AD', 'VC', 'NR', 'JJ', 'DEG', 'NN'], ['AD', 'PN', 'AD', 'VV', 'VV', 'VA'], ['VV', 'NN', 'VC', 'NR', 'LC', 'NN', 'NN', 'NR', 'CC', 'NR', 'NR', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'AS', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NR', 'NN', 'NR', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'VV', 'NT', 'NT', 'LC', 'AD', 'VV', 'VE', 'CD', 'CD', 'DEG', 'NN', 'JJ', 'NN', 'VV'], ['AD', 'VV', 'NN', 'VV'], ['NN', 'VV', 'NN', 'AD', 'VV', 'P', 'NR', 'VV', 'NN', 'NN', 'NN'], ['NT', 'NN', 'NN', 'P', 'JJ', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'VV', 'NN', 'P', 'AD', 'JJ', 'NN', 'VV', 'NN'], ['VV', 'PN', 'VV', 'DT', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NR', 'NR', 'CC', 'NR', 'VV', 'VV', 'AD', 'VV', 'NN'], ['AD', 'PN', 'VV', 'AS', 'AD', 'AD', 'VV', 'VV', 'PN'], ['NR', 'VV', 'NN', 'VE', 'VV', 'VV', 'NN', 'AD', 'NN', 'LC', 'AD', 'VE', 'NN'], ['NN', 'VA', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VV', 'VA', 'DEC', 'NN'], ['NN', 'P', 'NR', 'NR', 'VV', 'NN', 'VV', 'NR', 'DEG', 'NN'], ['NN', 'NN', 'AD', 'SB', 'VV', 'VV', 'AS', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'VV'], ['P', 'NT', 'NT', 'AD', 'VV', 'CD', 'NT', 'NT', 'CD', 'CD', 'NN'], ['PN', 'DEG', 'NN', 'VV', 'PN', 'VC', 'VV', 'AS', 'NN', 'VV', 'AS', 'NN', 'DEC', 'VA', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['VV', 'VV', 'NN', 'LC', 'VV', 'VV', 'VV', 'NN', 'LB', 'NN', 'VV', 'VV', 'CD', 'NN'], ['NN', 'VV', 'NT', 'NT', 'AD', 'VV', 'CD', 'CD', 'M', 'NN', 'NN'], ['PN', 'AD', 'VA', 'CD', 'AD', 'VV', 'DEC', 'NN', 'NR', 'AD', 'VV', 'NN'], ['NN', 'NT', 'NN', 'AD', 'P', 'NN', 'VA', 'AD', 'PN', 'NT', 'VV', 'VV', 'VV'], ['JJ', 'NN', 'AD', 'AD', 'NN', 'NN'], ['P', 'VV', 'NN', 'NN', 'NN'], ['NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'AD', 'AD', 'VV'], ['P', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'NN', 'DEC', 'AD', 'CD'], ['NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VV', 'JJ', 'JJ', 'NN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'AD'], ['NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['VV', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['AD', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'JJ', 'NN', 'NN'], ['NR', 'PN', 'NT', 'LC', 'VV', 'DT', 'M', 'NN', 'VV', 'PN', 'VV', 'PN', 'NN', 'VV', 'NN', 'VV'], ['AD', 'AD', 'VV', 'PN'], ['PN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['P', 'NN', 'NN', 'NT', 'VV', 'NR', 'AD', 'LB', 'DT', 'JJ', 'NN', 'VV', 'VV', 'SP'], ['AD', 'NR', 'VV', 'SB', 'AD', 'VV', 'NN', 'LC', 'VV', 'VV', 'AS', 'CD', 'NN'], ['NR', 'NN', 'NN', 'P', 'NR', 'NR', 'DEG', 'NN', 'NR'], ['PN', 'VV', 'NN', 'DEC', 'NN', 'CC', 'NN', 'VV', 'AS', 'AD', 'NN'], ['PN', 'VV', 'PN', 'NN', 'P', 'NR', 'NR', 'CC', 'NR', 'PN', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VC', 'NN', 'LC', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NN', 'VV', 'VC', 'M', 'JJ', 'DEG', 'JJ', 'JJ', 'JJ', 'NN'], ['VV', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN'], ['AD', 'AD', 'VE', 'NN', 'AD', 'VV', 'SP'], ['NT', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NN', 'LC', 'VV', 'NN', 'NN', 'VV', 'VV', 'CD', 'CD', 'CD', 'NN'], ['NT', 'NR', 'DEG', 'NN', 'VV', 'NR', 'CC', 'NR'], ['NR', 'NR', 'NR', 'AD', 'VV', 'P', 'OD', 'CD', 'NN'], ['NR', 'AD', 'VA', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'SP'], ['VV', 'DEC', 'NR', 'AD', 'VV', 'AD', 'AD', 'VV', 'NR', 'CC', 'VV', 'AS'], ['AD', 'PN', 'VE', 'NN', 'VV', 'BA', 'DT', 'NN', 'NN', 'VA'], ['NN', 'AD', 'VV', 'AS', 'NN', 'SP', 'VA', 'SP'], ['PN', 'VC', 'AD', 'VV', 'PN', 'NN', 'VV', 'PN', 'AD', 'PN'], ['CD', 'NN', 'VV', 'NN', 'DEV', 'VV', 'VV'], ['NN', 'VV', 'VV', 'VV', 'NR', 'NN', 'PN'], ['AD', 'VA', 'DEC', 'NN', 'AD', 'VV'], ['AD', 'NR', 'AD', 'VA', 'SP'], ['NN', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'DEG', 'JJ', 'NN'], ['VV', 'NN', 'AD', 'VE', 'NN', 'VV', 'VV', 'AD', 'AD', 'VA'], ['PN', 'DEG', 'NN', 'NN', 'VV', 'VV', 'SP'], ['NT', 'NN', 'NN', 'NN', 'VV'], ['PN', 'AD', 'BA', 'NN', 'VV', 'NN', 'VV'], ['AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN'], ['NR', 'VV', 'AD', 'VV'], ['AD', 'VC', 'CD', 'NN', 'VV', 'VV', 'NN', 'NN', 'P', 'NN', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['NT', 'NR', 'NR', 'NN', 'NR', 'CC', 'NR', 'DEG', 'NR', 'AD', 'VV', 'NN'], ['NT', 'NR', 'AD', 'VV', 'NN', 'NR', 'CC', 'NN', 'NN', 'ETC', 'NN'], ['NR', 'NN', 'VV', 'NN', 'AD', 'NN', 'VV', 'CD', 'JJ', 'NN', 'VV', 'NN', 'VV'], ['NR', 'P', 'NR', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'NR', 'NN', 'VV', 'AS', 'JJ', 'NN', 'AD', 'NR', 'AD', 'VV', 'AS', 'NN'], ['NR', 'AD', 'NN', 'VV', 'AS', 'NR'], ['NN', 'VV', 'AD', 'JJ', 'NN', 'AD', 'VA', 'NN'], ['NR', 'VV', 'CS', 'NN', 'VV', 'DEC', 'AD', 'AD', 'AD', 'VE', 'NR', 'NN', 'VV', 'NN'], ['NN', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'VV'], ['AD', 'VE', 'DEC', 'NN', 'P', 'OD', 'NN', 'VV', 'NN', 'OD', 'NN', 'AD', 'VV', 'NN'], ['NT', 'NR', 'NR', 'AD', 'AD', 'VE', 'NN', 'DEC', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VA'], ['NR', 'NR', 'DEG', 'NN'], ['AD', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'AD', 'VV', 'SP'], ['NN', 'CD', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV'], ['NR', 'VV', 'NR', 'NN'], ['VV', 'NN', 'AD', 'VA'], ['AD', 'VV', 'VV', 'NN', 'NN', 'VC', 'AD', 'VV', 'VV', 'SP'], ['PN', 'NT', 'VV', 'PN', 'VV', 'AD', 'VA'], ['AD', 'AD', 'VA', 'SP', 'AD', 'VV', 'PN', 'PN', 'SP'], ['VV', 'PN', 'NN', 'NN', 'VV', 'DEC', 'VC', 'DT', 'AD', 'VV', 'VV'], ['NR', 'NN', 'NN', 'VV', 'NR', 'BA', 'NN', 'AD', 'VV', 'VV'], ['AD', 'LC', 'NR', 'NN', 'BA', 'NN', 'NN', 'NN', 'P', 'NR', 'NN', 'NN', 'NN', 'VV'], ['CD', 'NN', 'VV', 'VV', 'NN', 'NN', 'P', 'VA', 'NN', 'VV', 'NN'], ['VV', 'LC', 'NR', 'NN', 'AD', 'VV', 'VV', 'PN', 'M', 'NN', 'PN', 'AD', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN', 'NT', 'VV', 'NN'], ['PN', 'P', 'VV', 'NN', 'NN', 'LC', 'VV', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'VC', 'PN', 'VV', 'AS', 'PN', 'CC', 'PN', 'VV', 'PN'], ['VV', 'VV', 'VV', 'VA', 'DEC', 'NN', 'NN', 'NN', 'PN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'P', 'NN', 'DEG', 'NN', 'LC', 'VV', 'AS', 'DEC', 'NN', 'P', 'NN', 'VV', 'NN', 'LC'], ['NN', 'VV', 'VA', 'NN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VA'], ['AD', 'VV', 'VV', 'NN', 'NN'], ['P', 'NN', 'NN', 'NN', 'AD', 'P', 'NN', 'NN', 'CC', 'NN', 'VA', 'AD', 'AD', 'VE', 'NN'], ['CD', 'VV', 'NN', 'DEC', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV'], ['AD', 'VV', 'VV', 'PN', 'VC', 'JJ', 'DEG', 'NN', 'NR', 'NR', 'VV', 'CC', 'NR', 'NN'], ['VV', 'PN', 'NN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'AS', 'PN', 'VA', 'VA', 'VA', 'VA', 'DT', 'NN', 'SP'], ['P', 'NN', 'LC', 'NR', 'AD', 'VV', 'NN'], ['NN', 'LC', 'VV', 'DEC', 'NN', 'NN'], ['DT', 'DEG', 'NN', 'AD', 'P', 'JJ', 'NN', 'NN', 'VE', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'CC', 'NR', 'AD', 'VE', 'VA', 'DEC', 'NN', 'CC', 'NN'], ['AD', 'VV', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'DT', 'M', 'VV', 'AS', 'SP'], ['NT', 'VV', 'NN', 'JJ', 'CD', 'M'], ['PN', 'AD', 'VV', 'AD', 'VA', 'SP'], ['NR', 'AD', 'AD', 'VV', 'VV', 'VV', 'DEC', 'JJ', 'NN', 'CC', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'CD', 'NN', 'AD', 'VV', 'CC', 'NR', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'VV', 'NN', 'DEC', 'VV', 'NR'], ['VV', 'VV', 'VV', 'DEC', 'NR', 'DT', 'VV', 'NR', 'NN', 'NR', 'NN', 'NN', 'AD'], ['NR', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'CD', 'JJ', 'NN', 'NN'], ['NN', 'NN', 'NN', 'VV', 'VV', 'NR', 'CC', 'NR', 'NN', 'VV', 'JJ', 'NN', 'P', 'NN', 'VV'], ['AD', 'P', 'P', 'NN', 'NN', 'AD', 'NN', 'NN', 'ETC'], ['NN', 'NN', 'AD', 'VC', 'NN', 'NN'], ['NN', 'NN', 'NN', 'NR', 'JJ', 'VV', 'NN', 'AD', 'VV', 'NN'], ['CD', 'NN', 'CC', 'NN', 'NN', 'VV', 'VA', 'VA', 'VA'], ['NR', 'VV', 'P', 'VA', 'DEC', 'NN', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD'], ['PN', 'AD', 'P', 'VV', 'CS', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV'], ['NN', 'VV', 'LC', 'NR', 'NR', 'NN'], ['NN', 'P', 'NR', 'NN', 'VV', 'VV', 'NR'], ['AD', 'VC', 'NT', 'VV', 'MSP', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VC', 'VA', 'NR'], ['NR', 'VV', 'AS', 'NN', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV'], ['AD', 'VC', 'NT', 'CD', 'VV', 'AD', 'NN', 'P', 'NN', 'AD', 'NN', 'AD', 'VV', 'AS'], ['PN', 'VV', 'AS', 'AD', 'AD', 'VV', 'VV', 'SP'], ['NN', 'AD', 'VV', 'PN', 'VE', 'NN', 'AD', 'P', 'NN', 'VV'], ['NR', 'NN', 'AD', 'VV', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'PN', 'AD', 'P', 'NN', 'VV', 'SP'], ['NN', 'SB', 'VV', 'AS', 'NR', 'VA'], ['VV', 'P', 'NR', 'NN', 'VV'], ['NT', 'NR', 'NN', 'VV'], ['NR', 'NN', 'VE', 'DT', 'NN', 'AD', 'NN', 'SB', 'VV', 'AS'], ['CD', 'VV', 'PN', 'VV', 'NN', 'AD', 'AD', 'VV'], ['P', 'NT', 'VV', 'DEC', 'NR', 'JJ', 'NN', 'NN', 'VA', 'VA'], ['AD', 'P', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'NR'], ['NN', 'NR', 'NR', 'NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['AD', 'NT', 'P', 'PN', 'VV', 'VV', 'NR'], ['NN', 'LC', 'VE', 'CD', 'M', 'NN', 'PN', 'AD', 'VV', 'PN', 'DEG', 'NR', 'NR'], ['P', 'AD', 'AD', 'AD', 'VV', 'DEC', 'NN', 'VV', 'NN', 'SP', 'VV', 'VE', 'JJ', 'NN', 'SP'], ['NR', 'VV', 'AS', 'NN', 'P', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN', 'NR', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'NR', 'NR', 'NN', 'NN', 'VV', 'NN', 'JJ', 'NN', 'VV'], ['AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['PN', 'P', 'PN', 'VV', 'PN', 'BA', 'NN', 'VV', 'VA', 'AD'], ['AD', 'VV', 'NN', 'VV', 'VV', 'SP'], ['NR', 'NN', 'NN', 'NN', 'NN'], ['PN', 'NN', 'AD', 'VV', 'AS', 'NN', 'AD', 'AD', 'VC', 'NN'], ['NT', 'NT', 'NN', 'VV', 'VV', 'NR'], ['CD', 'AD', 'VV', 'AS', 'DT', 'NN', 'NN', 'SP'], ['NR', 'NN', 'AD', 'AD', 'AD', 'VV', 'JJ', 'NR', 'DEG', 'NN', 'NN', 'VC', 'AD', 'VV'], ['AD', 'VV', 'VV', 'NN', 'NN', 'VV'], ['NT', 'VE', 'NN', 'AD', 'VV', 'NN', 'VV', 'P', 'NN', 'LC'], ['NR', 'NN', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'SB', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'AD', 'VV', 'PN', 'NN'], ['NR', 'NR', 'DEG', 'NR', 'CC', 'NR', 'DEG', 'NR', 'VV', 'JJ', 'JJ', 'NN', 'NN'], ['PN', 'NT', 'VV', 'VV', 'VV', 'NT', 'NN', 'ETC', 'CD', 'NN', 'AD', 'VV', 'P', 'PN'], ['NN', 'VV', 'VV', 'NR', 'VV', 'P', 'NR', 'NN'], ['NR', 'AD', 'P', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VA', 'AD', 'VC', 'NN', 'NN', 'NN', 'CC', 'NN', 'ETC'], ['NN', 'VV', 'AD', 'NN', 'P', 'NN', 'VV', 'DEC', 'NR', 'NR', 'VV', 'NN'], ['VV', 'AS', 'NN', 'VV', 'AD', 'VV', 'PN', 'SP'], ['AD', 'VV', 'NN', 'NN', 'P', 'NT', 'CD', 'CD', 'CD', 'LC'], ['JJ', 'NN', 'P', 'NR', 'VV', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VC', 'NN', 'NN', 'VC', 'VA', 'CC', 'VE', 'JJ', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'PN', 'VV', 'NR', 'VV'], ['NT', 'NN', 'CD', 'M', 'NN', 'LC', 'NR', 'NN', 'NN', 'VA'], ['NR', 'ETC', 'NN', 'VV', 'NN'], ['AD', 'PN', 'VV', 'NR', 'VV', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN', 'AD', 'VV', 'AS', 'CD', 'NN', 'NN'], ['AD', 'NN', 'VV', 'VV', 'VV', 'LC', 'NN', 'NN'], ['NN', 'LC', 'CD', 'NN', 'P', 'NN', 'NN', 'VV'], ['NN', 'NN', 'VV', 'AS', 'NN', 'NN'], ['NN', 'VA', 'AD', 'VV', 'DEC', 'NN', 'VC', 'NR'], ['CS', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['VV', 'LC', 'AD', 'AD', 'P', 'NN', 'VV', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VA'], ['CD', 'NN', 'AD', 'AD', 'VV', 'BA', 'NR', 'VV', 'NN', 'NN', 'VV', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NR', 'VV', 'VV', 'NN', 'ETC', 'NN'], ['PN', 'VV', 'VV', 'PN', 'AD', 'VV', 'DT', 'NN', 'SP'], ['NR', 'NN', 'LC', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NR', 'DEG', 'NN', 'NN', 'VV', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV'], ['VV', 'NN', 'NN', 'NR', 'NN', 'VV', 'VV', 'NN', 'P', 'PN', 'VV'], ['AD', 'VA', 'VA', 'AD', 'VV', 'NN', 'VV'], ['NT', 'NN', 'AD', 'AD', 'VV', 'PN', 'AD', 'VA'], ['NR', 'JJ', 'VV', 'NR', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'CC', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'CD'], ['PN', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['DT', 'NN', 'LC', 'NR', 'NN', 'VV', 'NN', 'JJ', 'NN', 'NN'], ['PN', 'VV', 'AS', 'AD', 'VC', 'NR', 'VV', 'NN'], ['NT', 'VV', 'NN', 'NN', 'VV', 'NN', 'PN', 'VV', 'NN', 'VV', 'PN'], ['AD', 'VV', 'DEC', 'NN', 'VV', 'VV', 'VV', 'VV', 'DEC', 'NN', 'VV', 'AD', 'VV', 'NN'], ['AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'NR', 'NN', 'NT', 'VV', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['CS', 'VC', 'NN', 'NN', 'NR', 'AD', 'AD', 'VE', 'VV'], ['NN', 'CD', 'NN', 'AD', 'VV', 'VV', 'NR', 'DEC', 'NN', 'SP'], ['NN', 'NN', 'VV', 'AS', 'JJ', 'NN', 'AD', 'PN', 'VV', 'VV'], ['VV', 'PN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NT', 'PN', 'AD', 'AD', 'AD', 'M'], ['NR', 'VV', 'VV', 'VV', 'DEC', 'NN', 'VV', 'PN', 'VV', 'DEC', 'AD', 'VE', 'NN'], ['NN', 'VV', 'NN', 'VE', 'NN', 'CC', 'NN', 'DEG', 'VV', 'VV', 'VV', 'CC', 'VV'], ['P', 'JJ', 'NN', 'VV', 'NN', 'DEC', 'NR', 'AD', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NR', 'AD', 'AD', 'AD', 'VV', 'AD', 'VV', 'VA', 'DEC', 'NN'], ['NN', 'NR', 'VV', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN'], ['P', 'PN', 'NN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'VV', 'CD'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'DEC', 'VC', 'VV', 'AS', 'NN', 'OD', 'M', 'M', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'PN', 'P', 'NN', 'AD', 'VV', 'VV'], ['AD', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'DEG', 'NR', 'NN', 'AD', 'VV', 'NN'], ['NN', 'CD', 'NN', 'P', 'NN', 'VV', 'NN', 'NN', 'VV', 'NR', 'DT', 'VV'], ['VV', 'SP', 'AD', 'NN', 'AD', 'P', 'NN', 'VV', 'CD'], ['PN', 'SB', 'VV', 'P', 'NR', 'NN', 'NR', 'NN', 'DEG', 'NN', 'NN'], ['IJ', 'IJ', 'JJ', 'PN', 'P', 'NR', 'NR'], ['AD', 'PN', 'AD', 'AD', 'P', 'PN', 'NT', 'VV', 'NN', 'NN', 'DEG', 'NN', 'PN', 'VV', 'AS'], ['NR', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'VA', 'VC', 'NN'], ['NR', 'VV', 'NN', 'VA', 'AD', 'SB', 'NN', 'NN', 'VV', 'PN'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'P', 'JJ', 'NN'], ['P', 'NN', 'VA', 'NN', 'NN', 'VV', 'DEC', 'AD', 'VC', 'NN'], ['AD', 'NN', 'NN', 'PN', 'AD', 'BA', 'NN', 'AD', 'VV', 'NN', 'LC'], ['NR', 'CC', 'NN', 'AD', 'VV', 'NR', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'NT', 'VV', 'NN', 'JJ', 'NN', 'AD', 'VA', 'AS', 'VV'], ['PN', 'AD', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'CD', 'IJ'], ['NN', 'NN', 'NN', 'VV', 'VV', 'AS', 'NN', 'LC'], ['NN', 'LC', 'DEG', 'NN', 'VV', 'BA', 'VV', 'DEC', 'NN', 'LC', 'VE', 'NN', 'NN'], ['NR', 'P', 'P', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['JJ', 'NN', 'JJ', 'NN', 'NN', 'AD', 'NT', 'CD', 'CD', 'NN'], ['NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'SP', 'VV', 'VE', 'NN'], ['JJ', 'DEG', 'NN', 'NN', 'VV', 'NR', 'CC', 'NR', 'AD', 'P', 'NR', 'NN', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VE', 'PN', 'DEG', 'NN', 'CC', 'NN', 'VV', 'VV'], ['NN', 'NN', 'VV', 'VV', 'DEC', 'AD', 'AD', 'AD', 'VV', 'NN', 'NN'], ['P', 'NN', 'NN', 'LC', 'NN', 'NN', 'AD', 'LB', 'NN', 'VV', 'NN', 'MSP', 'VV', 'NN'], ['NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VA', 'PN', 'VE', 'DEC', 'NN', 'P', 'P', 'PN', 'VV', 'AD', 'VV', 'AS'], ['P', 'JJ', 'NN', 'AD', 'VV', 'NN'], ['VV', 'DT', 'P', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'NN'], ['NR', 'AD', 'P', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'CC', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'NN', 'CD', 'M', 'NT', 'VV'], ['NN', 'DEG', 'P', 'NN', 'VV', 'NN', 'P', 'NN', 'NN', 'VV', 'AD', 'VV', 'VV'], ['P', 'NN', 'DEG', 'NN', 'DT', 'VV', 'NN', 'AD', 'VE', 'NN', 'P', 'DT', 'NN', 'P', 'NN', 'VV'], ['NR', 'NN', 'VV', 'NR', 'VV', 'NN', 'NN'], ['NR', 'VV', 'P', 'NT', 'LC', 'NR', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'NN', 'VC', 'JJ', 'NN', 'NN'], ['VA', 'DEC', 'SP', 'VV', 'AD', 'SP'], ['NN', 'AD', 'VV', 'NN', 'VE', 'NN'], ['NN', 'NN', 'VV', 'VV', 'AD', 'JJ', 'NN'], ['AD', 'P', 'NN', 'SB', 'VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'AD', 'VV'], ['AD', 'PN', 'DEG', 'NN', 'NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'LC', 'VV', 'VV'], ['NR', 'AD', 'AD', 'VV', 'AS', 'VV', 'VE', 'JJ', 'NN'], ['NN', 'VV', 'NN', 'NN', 'NN', 'VA'], ['NN', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'AD', 'VA', 'AD', 'P', 'PN', 'P', 'NN', 'VV', 'NN', 'NN', 'VA'], ['NN', 'NN', 'VV', 'NN', 'NN', 'VV'], ['P', 'NN', 'NN', 'VV', 'NN', 'NN', 'VC', 'NT', 'CD', 'NN', 'NN'], ['AD', 'NR', 'NN', 'VV', 'NN', 'NN'], ['AD', 'LB', 'NN', 'NN', 'AD', 'VV'], ['VV', 'NN', 'DEC', 'AD', 'NN'], ['NR', 'VV', 'NR', 'VV', 'DT', 'NN'], ['AD', 'VV', 'VV', 'IJ', 'NN', 'VV', 'PN', 'VV', 'NN', 'SP'], ['NR', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'PN', 'VV', 'NN', 'VV', 'AD', 'VV', 'AD'], ['NN', 'VV', 'AS', 'CD', 'NN'], ['VV', 'NN', 'NN', 'NN'], ['NR', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'P', 'NN', 'SP'], ['NR', 'NN', 'NN', 'VA'], ['NR', 'VV', 'AS', 'NN', 'NN', 'VV'], ['PN', 'AD', 'AD', 'AD', 'VV', 'ON'], ['OD', 'NN', 'AD', 'VV', 'NN', 'VC', 'CD', 'CD', 'CD', 'CD', 'M'], ['VV', 'AD', 'VV', 'NR', 'DEG', 'NR', 'NN', 'AD', 'NR'], ['AD', 'PN', 'VV', 'CD', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['IJ', 'IJ', 'PN', 'AD', 'AD', 'VV', 'SP'], ['NR', 'NN', 'NR', 'NN', 'NR', 'NN', 'JJ', 'NN'], ['VV', 'NN', 'AD', 'VV'], ['NT', 'NT', 'NR', 'NN', 'NN', 'AD', 'AD', 'NN', 'VV'], ['NR', 'NN', 'CC', 'NN', 'AD', 'VV', 'SP'], ['NN', 'VV', 'VV', 'VV', 'LC', 'NN', 'AD', 'P', 'NN', 'VV', 'AS', 'NN'], ['VV', 'VA', 'AS', 'NN', 'LC', 'AD', 'AD', 'VC', 'VV', 'P', 'VV', 'CC', 'VV'], ['NN', 'VV', 'AD', 'VV', 'NN', 'NN'], ['NN', 'DT', 'VA', 'DEC', 'NN', 'P', 'NN', 'NN', 'VA', 'VA', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['VV', 'AS', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN'], ['VV', 'LC', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV'], ['NN', 'VV', 'P', 'NT', 'CD', 'CD', 'M', 'DEC', 'NN'], ['NT', 'M', 'NN', 'VV', 'NN', 'AD', 'VV', 'P', 'CD', 'NT', 'NT', 'NT'], ['VV', 'PN', 'P', 'PN', 'VV', 'NN', 'NN', 'AD', 'PN', 'AD', 'VV', 'NN'], ['NR', 'VA', 'DEV', 'VV', 'VV', 'AS', 'NN', 'LC', 'DEG', 'NN'], ['P', 'PN', 'LC', 'VV', 'NN', 'AD', 'CD', 'VC', 'VV', 'NN', 'NN'], ['NN', 'BA', 'CD', 'NN', 'VV', 'SP'], ['AD', 'VV', 'DEC', 'NN', 'VC', 'VA', 'DEC', 'MSP', 'VV', 'DEC', 'NN', 'VC', 'VA'], ['VV', 'LC', 'NR', 'CC', 'CD', 'NN', 'VV', 'NN'], ['NN', 'AD', 'NN', 'VV', 'VV', 'PN', 'SP', 'IJ'], ['NR', 'AD', 'NN', 'NN', 'NR', 'P', 'NN', 'VC', 'CD', 'NN'], ['NR', 'AD', 'VV', 'PN', 'PN', 'PN', 'VV', 'DT', 'AD', 'VA', 'SP'], ['AD', 'P', 'NN', 'VV', 'NN', 'LC', 'AD', 'NN', 'VV', 'AD', 'VV', 'LC', 'AD', 'VV', 'VV', 'SP'], ['AD', 'VV', 'DEC', 'PN', 'P', 'NT', 'AD', 'VV', 'SP'], ['NN', 'VV', 'AS', 'NR', 'AD', 'AD', 'VV', 'VV'], ['NR', 'P', 'NN', 'BA', 'NR', 'VV', 'VV', 'P', 'NN', 'CC', 'BA', 'NR', 'CC', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NR', 'NN', 'DEG', 'NR', 'NN', 'P', 'OD', 'NN', 'CD', 'NN', 'NN'], ['NR', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'PN', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV'], ['NN', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'LC', 'NN', 'VV', 'NR', 'VV', 'NN', 'CC', 'VV', 'NN', 'NN'], ['P', 'NN', 'AD', 'VA', 'DEC', 'NN', 'NN', 'NR', 'VV', 'PN', 'NN', 'AD', 'VV', 'NN', 'SP'], ['PN', 'VV', 'VV', 'NN', 'VE', 'CD', 'AD', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VV', 'AD', 'NN', 'SP'], ['PN', 'AD', 'AD', 'VV', 'VV', 'DEV', 'VV', 'AS'], ['NN', 'CC', 'NN', 'VV', 'P', 'NN'], ['PN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['AD', 'PN', 'VV', 'NN', 'NN'], ['VE', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['AD', 'AD', 'P', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'VV', 'VV', 'NR', 'NN', 'NR', 'DT', 'JJ', 'DEG'], ['NN', 'NN', 'IJ', 'VV', 'VA', 'DEC', 'NN', 'SB', 'VV', 'DER', 'VV', 'NN'], ['NR', 'VV', 'NR', 'NN', 'LC', 'VV', 'AD', 'VV', 'NN', 'NN', 'BA', 'NR', 'VV', 'P', 'M', 'NN'], ['NT', 'NN', 'AD', 'VA', 'NN', 'VV', 'VV', 'AD', 'AD', 'VV', 'VV', 'PN'], ['NT', 'NR', 'NR', 'NR', 'NN', 'NN', 'NN', 'NN'], ['NT', 'NN', 'AD', 'AD', 'VV'], ['DT', 'NN', 'AD', 'VV', 'PN', 'CD', 'NN', 'SP', 'VV', 'AD', 'VV', 'VV', 'NN', 'CD', 'NN'], ['VV', 'JJ', 'NN', 'DEC', 'VC', 'NN', 'NN', 'CC', 'JJ', 'NN'], ['NR', 'NN', 'AD', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'AS', 'NR', 'VV', 'NN', 'CD', 'NR', 'DEG', 'VA', 'VV'], ['VE', 'DEC', 'NN', 'VV', 'NR', 'VC', 'P', 'VV', 'LC', 'AD', 'VV', 'SP'], ['P', 'NN', 'AD', 'JJ', 'NN', 'P', 'NN', 'CC', 'NN', 'VV'], ['VV', 'NN', 'AD', 'NN', 'CD', 'NN', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN', 'CC', 'NN', 'NN'], ['PN', 'NT', 'VV', 'VV', 'SP', 'NN', 'VV', 'SP'], ['NN', 'VV', 'VV', 'NN', 'AD', 'VC', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'PN', 'VA', 'VA', 'SP'], ['PN', 'VV', 'VC', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'AS', 'VV', 'NN'], ['NR', 'DEG', 'CD', 'NN', 'CC', 'NN', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'AS'], ['PN', 'DEG', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'NR', 'P', 'VV', 'NN', 'DEG', 'NN'], ['P', 'NN', 'NN', 'VA', 'NR', 'VC', 'AD', 'VA', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'VV', 'DEC', 'NN', 'JJ', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'P', 'NN', 'VV', 'VV', 'DEC', 'NN', 'CC', 'NN', 'P', 'NN', 'VV', 'PN', 'DEG', 'CD', 'NN'], ['NR', 'VV', 'VV', 'NN', 'NN', 'LC', 'VV', 'PN', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'PN', 'VV', 'SP'], ['NN', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN'], ['DT', 'NN', 'AD', 'NN', 'NN', 'DT', 'NN', 'SP'], ['VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VE', 'DT', 'PN', 'VE', 'DT'], ['AD', 'AD', 'VV', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'AD', 'VV', 'DEC', 'NN', 'NN', 'MSP', 'VV'], ['AD', 'NN', 'NN', 'P', 'NR', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'VV', 'DT', 'M', 'NN', 'CC', 'NN', 'NN', 'NN', 'AD', 'PN', 'VC', 'PN', 'PN', 'VC', 'PN', 'VV', 'PN', 'PN'], ['VV', 'NT', 'PN', 'VV', 'AS', 'PN', 'NN', 'LC', 'VV', 'AD', 'AD', 'VV'], ['DT', 'CD', 'NN', 'AD', 'VC', 'NT', 'CD', 'CD', 'CD', 'CD', 'NN'], ['NT', 'LC', 'P', 'NR', 'NR', 'DEG', 'NN', 'LC', 'NR', 'AD', 'VV', 'NN'], ['AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'CS', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NR', 'CC', 'NN', 'NN', 'AD', 'VV', 'VV', 'NR'], ['JJ', 'NN', 'JJ', 'DEG', 'NN', 'NN', 'ETC', 'NN', 'AD', 'AD', 'VV', 'NR'], ['AD', 'VV', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['VV', 'DEC', 'NN', 'NN', 'VV', 'CD'], ['VV', 'NN', 'LC', 'NN', 'DEG', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VE', 'NN'], ['NN', 'VC', 'VA', 'VA', 'DEC', 'JJ', 'NN', 'CC', 'VV', 'P', 'NR', 'DEG', 'NR'], ['PN', 'P', 'NR', 'NT', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['VV', 'NN', 'VV', 'AS', 'AD', 'BA', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'VV', 'CD', 'NN', 'ETC'], ['VE', 'CD', 'VV', 'VV', 'VV', 'DEC', 'NN', 'P', 'NN', 'LC', 'VV'], ['AD', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VA'], ['NN', 'CD', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'LC', 'DEG', 'JJ', 'NN', 'VV', 'NN', 'VV', 'M', 'VA', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NR', 'VV', 'NN', 'NN', 'VE', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['VV', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VC', 'NN', 'LC', 'DEG', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'LC', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'MSP', 'VV', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VV', 'LC', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'JJ', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'VV'], ['P', 'NN', 'NN', 'VV', 'NN', 'PN', 'P', 'PN', 'VV'], ['PN', 'AD', 'NR', 'OD', 'M'], ['PN', 'NN', 'VV', 'NR', 'NN', 'VV', 'NN', 'NT', 'AD', 'AD', 'VV', 'AS', 'NN', 'SP'], ['NN', 'VA', 'DEC', 'NN', 'VV', 'PN'], ['NN', 'NR', 'VV', 'VV', 'NN'], ['VV', 'VV', 'VC', 'DEC', 'NN', 'VV', 'AS', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'PN', 'VV', 'VV', 'NR', 'NN', 'NN', 'VV', 'VV'], ['PN', 'VV', 'VA', 'LC', 'PN', 'CC', 'NN', 'AD', 'VV', 'NN', 'VV'], ['AD', 'CD', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV'], ['AD', 'PN', 'VV', 'VV', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NR', 'CC', 'NN', 'CD', 'JJ', 'NN', 'NT', 'VV', 'VV'], ['AD', 'NT', 'NN', 'VV', 'DT', 'NN', 'DEG', 'NN', 'NN'], ['VV', 'NR', 'NR', 'VV', 'VV', 'VV', 'NR', 'LC', 'AD', 'AD', 'SB', 'VV', 'VV', 'NR'], ['P', 'JJ', 'DEG', 'NN', 'DEG', 'NN', 'NN', 'NR', 'VV', 'AS', 'JJ', 'CD', 'NN'], ['P', 'VV', 'PN', 'P', 'AD', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'NR', 'AD', 'VV', 'AS', 'CD', 'NN'], ['NN', 'NN', 'NR', 'NN', 'P', 'VV', 'NN'], ['VV', 'NN', 'NN', 'DEG', 'NN'], ['VC', 'LB', 'NN', 'MSP', 'VV', 'DEC', 'SP'], ['VV', 'NN', 'NN', 'VC', 'NN'], ['NN', 'AD', 'VV', 'VV', 'DEC', 'AD', 'CC', 'NR', 'DEG', 'NN', 'NN'], ['VV', 'NN', 'VV', 'NN', 'NR', 'NR', 'NN', 'NN', 'VA', 'VV', 'VV'], ['AD', 'VE', 'DT', 'AD', 'DT', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN'], ['AD', 'VV', 'VV', 'DEC', 'NN', 'VA', 'DEC', 'NN'], ['AD', 'P', 'NN', 'P', 'PN', 'NN', 'VV', 'VV', 'VV', 'VA', 'AD', 'SP'], ['AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['AD', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'CD', 'NN'], ['P', 'CD', 'CD', 'NN', 'NR', 'NN', 'LC', 'AD', 'NN', 'LB', 'NN', 'VV'], ['PN', 'AD', 'VA', 'P', 'NN', 'P', 'NN', 'VV', 'AS', 'PN'], ['AD', 'PN', 'AD', 'VV', 'NR', 'NR', 'LC', 'AD', 'VV', 'DT', 'NN'], ['AD', 'PN', 'VV', 'JJ', 'NN', 'PN', 'VV', 'NR', 'AD', 'BA', 'PN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'P', 'VV', 'NN', 'OD', 'NN', 'MSP', 'VV', 'NN', 'NN'], ['NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'P', 'NN', 'NN', 'NN', 'LC', 'VV', 'BA', 'PN', 'VV'], ['NN', 'AD', 'VE', 'CD', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['SB', 'VV', 'DEC', 'NN', 'VV', 'AS', 'SP'], ['AD', 'VV', 'VV', 'VV', 'VV'], ['VA', 'NN', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NN', 'AD', 'VV', 'PN', 'NN', 'VA', 'AD', 'NR', 'NN', 'VV'], ['AD', 'NR', 'AD', 'VV', 'VV', 'NN'], ['AD', 'DT', 'P', 'NN', 'DEG', 'NN', 'VV'], ['NN', 'AD', 'VV', 'AS', 'NN', 'JJ', 'NN', 'AD', 'JJ', 'NN', 'AD', 'VA'], ['PN', 'AD', 'AD', 'VV', 'VV', 'AS', 'SP'], ['CC', 'VV', 'AD', 'VV', 'AD', 'VV', 'SP'], ['VV', 'AD', 'VV', 'AS', 'NN', 'DEG', 'VV'], ['AD', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'P', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'AS', 'VV', 'P', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'P', 'NN', 'DEG', 'NN', 'VV', 'JJ', 'NN'], ['NR', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'AD', 'VE', 'CD', 'NN'], ['OD', 'NN', 'NN', 'VC', 'NR', 'NN', 'VV', 'NR', 'DEG', 'NR', 'ETC'], ['VV', 'AS', 'P', 'PN', 'NN', 'VV', 'AS'], ['AD', 'VA', 'DEC', 'NN', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'VV', 'AD'], ['NT', 'NR', 'AD', 'VV', 'AS', 'NN'], ['PN', 'NT', 'VV', 'VV', 'VV', 'VV', 'PN', 'AD', 'AD', 'AD', 'VV', 'VV', 'SP'], ['PN', 'AD', 'VV', 'LC', 'VV', 'VV', 'PN', 'NN'], ['AD', 'NN', 'VV', 'P', 'NN', 'VV', 'AS', 'NN', 'NN', 'VA'], ['CD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV'], ['AD', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AD', 'NN', 'DT', 'VV', 'NN'], ['VV', 'PN', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN', 'AD', 'P', 'NR', 'NN', 'NN', 'VV'], ['AD', 'NN', 'NR', 'DT', 'DEG', 'AD', 'P', 'CD', 'AD', 'AD', 'VV', 'SP'], ['AD', 'VV', 'NN', 'MSP', 'VV', 'AD', 'VV', 'NR', 'DEG', 'NN', 'VA'], ['NR', 'CC', 'NR', 'NN', 'NN', 'DEG', 'NN', 'VE', 'DT', 'NN'], ['CD', 'NN', 'NR', 'NT', 'NN', 'NN', 'VV'], ['CD', 'NN', 'P', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'VV', 'VV'], ['NN', 'AD', 'P', 'NN', 'NN', 'LC', 'VV'], ['NN', 'DEG', 'NN', 'VV', 'JJ', 'NN', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['P', 'NN', 'VV', 'LC', 'NR', 'NN', 'JJ', 'NN', 'VV', 'VA', 'VA'], ['DT', 'NN', 'P', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['P', 'NN', 'VA', 'CD', 'NN', 'VV', 'NN', 'VV'], ['AD', 'NN', 'VV', 'AS', 'NN', 'VA', 'DEV', 'VV', 'NN', 'VV'], ['AD', 'JJ', 'JJ', 'NN', 'AD', 'VV', 'AD', 'VA'], ['AD', 'NN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV', 'P', 'PN'], ['PN', 'VV', 'P', 'JJ', 'NN', 'VV', 'DEC', 'NN', 'P', 'NR', 'VV', 'NN'], ['JJ', 'DEG', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN', 'MSP', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['NT', 'AD', 'BA', 'DT', 'NN', 'VV', 'VV', 'AS', 'NN', 'VA', 'DEC', 'VA'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'NN', 'VV', 'NN', 'VV', 'VV', 'LC', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['JJ', 'DEG', 'NN', 'NN', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'LB', 'VV'], ['NN', 'VE', 'NN', 'VV', 'AS', 'NN'], ['NN', 'VV', 'NN', 'VV', 'PN', 'VV', 'VV', 'PN', 'VV', 'SP'], ['AD', 'VV', 'PN', 'AD', 'AS', 'AD', 'VV', 'PN', 'VV', 'PN', 'AD', 'VV', 'PN', 'NN', 'VV'], ['AD', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NR', 'NN', 'VV', 'NN'], ['NN', 'NN', 'VV', 'AD', 'VA', 'DEC', 'NN', 'VV', 'P', 'NN', 'VV'], ['NR', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN'], ['NN', 'NN', 'LC', 'VV', 'AD', 'P', 'VA', 'DEC', 'NN', 'VV', 'VV', 'VV', 'NT'], ['NN', 'NN', 'NN', 'CC', 'JJ', 'NN', 'AD', 'P', 'NN', 'NN', 'LC', 'VV', 'VV'], ['VV', 'DEC', 'NN', 'NN', 'NN', 'NN', 'NN', 'CC', 'NR', 'NR', 'ETC', 'NN', 'AD', 'AD', 'VV'], ['NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'JJ', 'NN'], ['NR', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'SB', 'NN', 'VV'], ['AD', 'VV', 'VV', 'JJ', 'NN', 'AD', 'VE', 'NN', 'VE', 'NN', 'AD', 'VE', 'NN'], ['AD', 'NN', 'NR', 'NN', 'DEG', 'NN', 'NN', 'VC', 'VV', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'NN', 'JJ', 'P', 'NR', 'VE', 'NN'], ['NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'NN', 'VE', 'NN', 'SB', 'VV', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN'], ['NR', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'VA'], ['PN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'AD', 'VA'], ['VV', 'NN', 'PN', 'VV', 'AD', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NN', 'ETC', 'NN'], ['NN', 'VA', 'DEV', 'VV', 'AD', 'VV', 'VV', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['NR', 'NR', 'VC', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'LC', 'NR', 'NR', 'NN', 'DEG', 'NN', 'VA', 'VA'], ['CD', 'NN', 'VV', 'DT', 'NN', 'P', 'JJ', 'NN', 'NN', 'LC', 'AD', 'VV', 'AD'], ['NT', 'VV', 'AD', 'VV'], ['AD', 'NN', 'AD', 'NN', 'VV', 'NN', 'VV'], ['NN', 'NN', 'VV', 'VA', 'VA', 'AS', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'VC', 'NN', 'JJ', 'DEG', 'NN', 'VV', 'P', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['VV', 'NN', 'DEC', 'JJ', 'NN', 'SP'], ['AD', 'VV', 'JJ', 'NN', 'DEG', 'NR', 'AD', 'AD', 'VV', 'NN', 'NN', 'DT', 'NN'], ['VV', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'SP'], ['NR', 'VV', 'DEC', 'NR', 'NN', 'NR', 'AD', 'VV', 'NN', 'SP'], ['AD', 'P', 'NN', 'NN', 'DEG', 'NN', 'VA', 'VA'], ['AD', 'VV', 'LC', 'VV', 'NN', 'AD', 'NN', 'VV'], ['AD', 'NN', 'VV', 'VV', 'VV', 'VV'], ['PN', 'AD', 'VE', 'VV', 'NN', 'NN', 'NN', 'VV', 'PN'], ['NT', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'VV', 'PN', 'AD', 'VV', 'NN', 'VV', 'AS'], ['AD', 'NN', 'AD', 'VV', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN'], ['NT', 'VC', 'NN', 'NN', 'VV', 'NR'], ['VV', 'NN', 'NN', 'LC', 'NN', 'NN', 'VE', 'CD'], ['DT', 'NN', 'PN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'VV'], ['PN', 'VV', 'NN', 'NN', 'VV', 'AS', 'NN'], ['VV', 'DEC', 'VV', 'VV', 'NN', 'AD', 'VE'], ['PN', 'AD', 'JJ', 'AD', 'VV', 'NN'], ['VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NR', 'P', 'JJ', 'NN', 'NN', 'VC', 'NN', 'VV', 'VV', 'NN'], ['VA', 'DEC', 'NN', 'CC', 'VA', 'DEC', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'PN', 'AD', 'VC', 'NN', 'NN', 'NT', 'DEG', 'NN'], ['PN', 'P', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'NN', 'VV', 'AS', 'CD', 'M', 'AD', 'VV', 'NN', 'DEC', 'NR', 'VV', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VC', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'NN'], ['AD', 'VV', 'NN', 'VV', 'VV', 'DEC', 'VC', 'NN', 'LC', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'SP'], ['NN', 'VV', 'PN', 'NN', 'VV', 'AS', 'M', 'NN', 'NN', 'VV', 'VV', 'PN', 'VV', 'PN', 'PN', 'VV', 'AD', 'AD', 'VV', 'AS', 'NN'], ['CS', 'VV', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'PN', 'NN', 'DEG', 'NN'], ['NR', 'NR', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'VE', 'NN', 'VV', 'NN', 'NN', 'SP'], ['P', 'NN', 'VV', 'P', 'NR', 'NN', 'LC', 'DEG', 'NN', 'AD', 'VV', 'VV', 'JJ'], ['PN', 'VV', 'JJ', 'NN', 'PN', 'NN', 'NN'], ['NR', 'NR', 'NR', 'VV', 'NN', 'NN', 'VC', 'JJ', 'DEG', 'NN'], ['AD', 'NR', 'AD', 'VV', 'P', 'VV', 'DEC', 'NN', 'AD', 'VV'], ['PN', 'VC', 'NR', 'OD', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'P', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN'], ['NT', 'LC', 'NR', 'CC', 'NN', 'VV'], ['CC', 'NN', 'NN', 'AD', 'VV', 'AD', 'PN', 'VV', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'P', 'NN', 'VV', 'JJ', 'NN', 'LC', 'VV', 'NN', 'LC'], ['NN', 'NN', 'VV', 'AD', 'AD', 'DEV', 'NN', 'NN', 'P', 'NN', 'PN', 'VV', 'NN', 'VV', 'VV'], ['NN', 'VV', 'VV', 'NR', 'NN', 'NN', 'VV', 'VV', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV', 'AD', 'VV'], ['VV', 'AD', 'VV', 'NN', 'VE', 'DT', 'NN', 'CC', 'NN', 'AD', 'VV', 'VV', 'NR', 'VV', 'VV'], ['P', 'NN', 'NN', 'AD', 'AD', 'VV', 'DEC', 'NN', 'LC', 'NN', 'AD', 'AD', 'LB', 'NN'], ['AD', 'VV', 'PN', 'VV', 'BA', 'JJ', 'NN', 'VV', 'VV', 'PN', 'VV', 'AD', 'NN', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'LC', 'VV', 'NN', 'AD', 'VV', 'AS', 'NN', 'P', 'NN', 'VV'], ['VV', 'NR', 'VV', 'LC', 'AD', 'NN', 'AD', 'AD', 'VV', 'DT', 'NN', 'VC', 'PN'], ['P', 'PN', 'VV', 'VV', 'AS', 'PN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'AS'], ['NN', 'LC', 'JJ', 'DEG', 'NN', 'VV', 'NN', 'AD', 'NN', 'VV', 'NN'], ['CS', 'VV', 'CD', 'NN', 'AD', 'NN', 'VV', 'VV', 'NN', 'VV', 'VA'], ['NR', 'VV', 'NN', 'AD', 'VV', 'NR', 'AD', 'P', 'NN', 'VV'], ['VV', 'NN', 'P', 'NR', 'NN', 'AD', 'VV', 'NN'], ['CD', 'NN', 'NN', 'CD', 'NN', 'CC', 'CD', 'NN', 'VV', 'AS', 'AD'], ['AD', 'PN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NN', 'VV', 'VC', 'VV', 'NN', 'NN', 'AD', 'VE', 'VV', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'VV', 'PN', 'VV', 'NR', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'PN', 'VV', 'DEC', 'NN'], ['AD', 'NN', 'NN', 'VV', 'NR', 'DEG', 'NN', 'P', 'NR', 'NR', 'ETC', 'NN', 'VV', 'CD', 'NN'], ['AD', 'AD', 'VV', 'DEC', 'VA', 'PN', 'AD', 'VV', 'PN', 'VA', 'DEC', 'NN', 'VV', 'AS', 'M', 'JJ', 'NN', 'NN'], ['NT', 'P', 'NN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'VV'], ['AD', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'AD', 'NN', 'AD', 'VV', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NN', 'NR', 'VV', 'VV', 'AD', 'PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['VE', 'NN', 'PN', 'AD', 'VV', 'VV', 'SP'], ['VV', 'NR', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'AD', 'NT', 'CD', 'AD', 'VV', 'VV'], ['NR', 'DT', 'VV', 'LC', 'NN', 'AD', 'VA', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'AD', 'VV', 'VV', 'VV', 'NR'], ['VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'NR', 'VA', 'DEC', 'NN', 'VV'], ['CS', 'NN', 'AD', 'M', 'NN', 'VV', 'PN', 'AD', 'VV', 'VV', 'M', 'NN', 'NN', 'NN'], ['AD', 'VV', 'PN', 'AD', 'VA', 'AD', 'PN', 'NN', 'SP'], ['AD', 'P', 'NN', 'VV', 'AS', 'NN', 'LC'], ['CS', 'PN', 'DEG', 'NN', 'NN', 'P', 'NN', 'SP'], ['JJ', 'NN', 'AD', 'P', 'NN', 'VA', 'VA', 'CD', 'NN', 'VV', 'PN', 'DEG', 'NN', 'CC', 'NN'], ['NN', 'AD', 'VV', 'VE', 'AD', 'VV', 'AS', 'SP'], ['IJ', 'AD', 'AD', 'VE', 'NN', 'SP'], ['NR', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VC', 'VA', 'DEC', 'NN', 'NN', 'PN', 'VV', 'NN'], ['CD', 'NN', 'VV', 'VV', 'NN', 'SB', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'NN'], ['AD', 'PN', 'VV', 'VV'], ['AD', 'AD', 'VV', 'VV', 'SP'], ['PN', 'VV', 'NN', 'NN', 'VV', 'JJ', 'NN', 'VV', 'JJ', 'NN'], ['AD', 'VV', 'AS', 'AD', 'NN', 'AD', 'VE', 'NR', 'NN', 'LC', 'AD', 'AD', 'VE', 'NN', 'VV', 'VV'], ['AD', 'VC', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'AD', 'VA'], ['AD', 'AD', 'VV', 'PN', 'VV', 'VA', 'NN', 'NN', 'ETC'], ['NN', 'P', 'NT', 'VA', 'CD', 'SP'], ['VV', 'NN', 'LC', 'DEG', 'NR'], ['AD', 'VV', 'NN', 'P', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'PN', 'VA', 'SP', 'NN', 'DEG', 'NN', 'AD', 'VA', 'VV', 'VV', 'DEC', 'NN', 'AD', 'BA', 'NN', 'VV', 'AS'], ['AD', 'VV', 'NR', 'NR', 'DEG', 'NR', 'NN', 'LC', 'VE', 'NR', 'NN', 'DEG', 'JJ', 'NN'], ['NR', 'AD', 'VV', 'M', 'AD', 'VV', 'PN'], ['NN', 'NN', 'AD', 'NN', 'NN', 'NN', 'NN', 'VV', 'AS', 'AD', 'NN'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'AS', 'VE', 'DT', 'NN', 'SP'], ['AD', 'NR', 'SB', 'NN', 'VV', 'JJ', 'NN'], ['AD', 'AD', 'VV', 'DT', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['JJ', 'NN', 'AD', 'VA', 'DEC', 'NN', 'VV', 'AD', 'VV', 'AD', 'VE', 'VV', 'CC', 'VE', 'VV', 'NN', 'AD', 'VV', 'VA'], ['NR', 'DT', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'NN', 'NN', 'VC', 'NN', 'VV', 'NN'], ['PN', 'VC', 'P', 'NT', 'NN', 'LC', 'NN', 'AD', 'VV', 'DT', 'NN'], ['AD', 'AD', 'NR', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV', 'NR', 'NN'], ['NN', 'AD', 'P', 'VV', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NR', 'NN', 'VV', 'VV', 'NN', 'LC', 'VV', 'DEC', 'NN', 'NN'], ['NR', 'VV', 'PN', 'VV', 'PN', 'JJ', 'JJ', 'NN', 'AD', 'VV', 'VV', 'PN', 'VV', 'PN'], ['NN', 'VV', 'PN', 'VV', 'NT', 'P', 'VV', 'PN', 'NN', 'DEG', 'NN', 'BA', 'NN', 'NN', 'DEG', 'NN', 'VV', 'VV', 'AS'], ['NR', 'VV', 'AD', 'AD', 'VV', 'VE', 'NN', 'AD', 'AD', 'VV', 'VV'], ['VV', 'NN', 'VV', 'VV', 'DEC', 'NN', 'MSP', 'CC', 'VV', 'NN', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['P', 'P', 'NR', 'VV', 'AS', 'NT', 'PN', 'AD', 'VV', 'AD', 'AD', 'VV', 'NR', 'DEG', 'NN'], ['CD', 'NN', 'NN', 'VE', 'NN', 'VV', 'VV', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'AD', 'VV', 'PN', 'VV', 'VC', 'P', 'NR', 'CC', 'NN', 'DEC', 'VV'], ['NT', 'PN', 'VV', 'NR', 'NN', 'VV', 'AD', 'VV', 'LC', 'CD', 'NN', 'VV'], ['NR', 'NN', 'P', 'NR', 'VV', 'DEC', 'NR', 'NN', 'NN', 'LC', 'VV', 'NN', 'NN'], ['VV', 'DT', 'CD', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'VV'], ['BA', 'JJ', 'NN', 'VV', 'NN', 'DEG', 'NN', 'LC', 'P', 'NR', 'AD', 'AD'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV'], ['NR', 'AD', 'VV', 'M', 'JJ', 'DEG', 'NN', 'PN', 'VE', 'CD', 'NN'], ['NT', 'NT', 'NN', 'NN', 'NR', 'JJ', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VA', 'DEC', 'NN'], ['PN', 'VV', 'NT', 'NN', 'NN', 'NN', 'VC', 'NR', 'AD', 'VA'], ['PN', 'AD', 'VV', 'AD', 'VV', 'SP', 'SP'], ['NR', 'VV', 'NR', 'NR', 'NN', 'NN', 'NR', 'NN'], ['P', 'NN', 'VV', 'NN', 'LC', 'NN', 'VV', 'PN', 'VV', 'AD', 'VV'], ['P', 'NR', 'VV', 'VV', 'LC', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'NN', 'LC', 'VV', 'AS', 'PN', 'CC', 'NN', 'NN', 'VV', 'AD', 'PN', 'VV', 'VV', 'AS', 'PN'], ['NN', 'NR', 'VC', 'NN', 'VV', 'PN', 'DEC', 'JJ', 'NN'], ['NN', 'NT', 'VV', 'NN', 'NN', 'CC', 'VV', 'JJ', 'NN'], ['NN', 'DEG', 'NN', 'SB', 'VV', 'AS', 'NN', 'VV', 'PN', 'VV', 'VV'], ['AD', 'NN', 'AD', 'VV', 'NN', 'NN', 'ETC', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'VA'], ['NR', 'NN', 'NN', 'LC', 'VV', 'VV', 'VV', 'VV', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'DT', 'NR', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NN', 'SP'], ['VV', 'NN', 'AD', 'AD', 'VC', 'JJ', 'NN', 'AD', 'VV'], ['NT', 'AD', 'JJ', 'NN', 'VV', 'NN', 'CC', 'NR', 'NN', 'VE', 'NN'], ['NR', 'VV', 'NN', 'CD', 'M', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'NN', 'AS', 'VV', 'NN', 'VV', 'VV', 'PN', 'SP', 'PN', 'DEG', 'VV', 'AD', 'AD', 'VA', 'SP'], ['NN', 'LC', 'VV', 'NN', 'DEC', 'NR', 'NN', 'LC', 'AD', 'VE', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'VV', 'PN', 'VV', 'AS', 'NN', 'VV', 'PN', 'VV', 'AD', 'VV', 'NN', 'DEG', 'NN', 'VV', 'PN'], ['JJ', 'NN', 'NN', 'NN', 'AD', 'VA', 'NN', 'VA'], ['NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'AD', 'VA'], ['VV', 'JJ', 'NN', 'NN', 'VV', 'AD', 'P', 'VV', 'JJ', 'NN'], ['NR', 'VV', 'NR', 'NT', 'NN', 'AD', 'VA', 'CC', 'AD', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'AD', 'AD', 'NR', 'VV', 'NN', 'AD', 'VC', 'AD', 'VV', 'NN'], ['P', 'CD', 'NN', 'VV', 'AS', 'DEC', 'NN', 'PN', 'VV', 'VV', 'AS', 'PN', 'VC', 'VA', 'SP'], ['NN', 'AD', 'AD', 'BA', 'P', 'NN', 'DEG', 'NN', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AS'], ['NN', 'AD', 'VC', 'AD', 'VE', 'VV', 'DEC', 'IJ'], ['AD', 'PN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'P', 'NR', 'LC', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VA', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'PN', 'AD', 'AD', 'VA'], ['NR', 'AD', 'VV', 'AS', 'AS', 'NN', 'AD', 'AD', 'VV', 'VV'], ['VV', 'NN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'SP'], ['DT', 'AD', 'AD', 'VV', 'SP'], ['NR', 'AD', 'VV', 'VV', 'NR', 'NN', 'NN', 'NN'], ['NN', 'NN', 'VV'], ['VV', 'SP', 'NN', 'NN', 'SP'], ['NN', 'VV', 'CD', 'NN', 'SP'], ['PN', 'VV', 'DT', 'AD', 'SP'], ['PN', 'VV', 'PN', 'VV', 'SP'], ['PN', 'AD', 'SP', 'NN'], ['VV', 'DEC', 'VV', 'AD', 'VV', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NN', 'VV', 'DEC', 'NN'], ['CS', 'VV', 'NN', 'VV', 'NN', 'VE', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'VV'], ['VV', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['P', 'DT', 'NN', 'NN', 'NR', 'NN', 'NN', 'NN', 'NR', 'NR', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'P', 'DT', 'CC', 'NR', 'DT', 'AD', 'VA', 'VV'], ['NR', 'NN', 'PN', 'AD', 'VV', 'AS', 'JJ', 'DEG', 'NN', 'SP'], ['VV', 'JJ', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'VV', 'PN', 'NN', 'VV', 'PN', 'AD', 'AD', 'PN', 'BA', 'AD', 'VV', 'VV', 'SP'], ['AD', 'CD', 'VV', 'NN', 'AD', 'VV', 'VV', 'PN', 'VV', 'VV', 'NN', 'AD', 'SB', 'VV', 'AD', 'VA'], ['NN', 'NN', 'VA', 'AS', 'NN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NT', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['AD', 'AD', 'AD', 'NN', 'SP', 'AD', 'NN', 'VV', 'AS'], ['PN', 'VC', 'VA', 'VA', 'DEC', 'NN', 'AD', 'AD', 'VV', 'AD', 'VA', 'CC', 'VA'], ['NN', 'AD', 'VV', 'NN', 'DT', 'M', 'NN', 'NN', 'AD', 'P', 'DT', 'NN', 'SP'], ['NR', 'NR', 'P', 'NN', 'NN', 'AD', 'VV', 'DT', 'NN'], ['JJ', 'AD', 'AD', 'VV', 'AD', 'VV', 'VA'], ['VV', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'NN', 'NN', 'NR', 'VV', 'NN', 'NR', 'NN', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'VV', 'PN', 'VA'], ['NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['CS', 'DT', 'VV', 'NN', 'VV', 'DEC', 'AD', 'AD', 'VA'], ['VV', 'DEC', 'NN', 'AD', 'P', 'PN', 'NN', 'VV', 'PN'], ['NN', 'AD', 'VV', 'NR', 'NN', 'AD', 'AD', 'VV', 'VV'], ['P', 'CD', 'NR', 'NN', 'NN', 'VV', 'P', 'NR', 'VV', 'JJ', 'DEG', 'NN'], ['VV', 'AD', 'VV', 'NN', 'AD', 'VV'], ['PN', 'VV', 'PN', 'VV', 'AD', 'VV', 'DT'], ['CD', 'NN', 'VV', 'VV', 'NN', 'NN'], ['DT', 'NN', 'CC', 'NT', 'AD', 'VV', 'NN', 'AD', 'VC', 'CD', 'CC', 'NT', 'CD'], ['DT', 'NN', 'AD', 'VV', 'VV', 'DT', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'NN', 'NN', 'CD', 'NN'], ['NR', 'PN', 'AD', 'P', 'NN', 'NN', 'LC', 'P', 'DT', 'JJ', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NR', 'NR', 'DT', 'NN', 'VV', 'VV'], ['NR', 'VC', 'NR', 'NR', 'NR', 'NR', 'NN', 'NN', 'NN'], ['NR', 'NN', 'AD', 'P', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'VA'], ['AD', 'VV', 'JJ', 'NN', 'VC', 'NR', 'NN', 'NN', 'VV', 'VA', 'NN', 'VV', 'NN'], ['VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD'], ['AD', 'NN', 'NR', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV'], ['PN', 'CC', 'NR', 'VV', 'CD', 'NN'], ['P', 'NR', 'P', 'VV', 'NN', 'VE', 'DEC', 'NN', 'VV', 'AS', 'NR', 'VV', 'DEC', 'NN'], ['VV', 'VV', 'DEC', 'NN', 'AD', 'NN', 'VV'], ['AD', 'VV', 'AD', 'VA', 'AD', 'AD', 'VV', 'PN', 'VV'], ['NN', 'VV', 'NR', 'VE', 'DT', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['VE', 'AD', 'NN', 'VV', 'NN', 'BA', 'PN', 'DEG', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['OD', 'M', 'NN', 'DEG', 'OD', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'CD', 'M', 'NN', 'SP', 'AD', 'NR', 'NN', 'AD', 'AD', 'AD', 'VA'], ['NN', 'DT', 'NN', 'AD', 'AD', 'VV', 'PN', 'IJ'], ['VV', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['NN', 'LC', 'AD', 'AD', 'VV', 'NN', 'SP', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'VC', 'AD', 'VV', 'NR', 'VV', 'AS', 'NN', 'SP'], ['NT', 'VV', 'AD', 'SB', 'VV', 'AS', 'NR', 'NN', 'NN', 'NN'], ['DT', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'NN'], ['CD', 'M', 'NN', 'VV', 'NN', 'NN', 'CC', 'JJ', 'NN'], ['NR', 'NN', 'NN', 'VV', 'AS', 'PN', 'NN', 'LC', 'NN', 'M', 'NN', 'AD', 'BA', 'PN', 'VV', 'VV'], ['NN', 'P', 'NN', 'VV', 'AS', 'VV', 'DEC', 'NN', 'VV', 'AS', 'NR', 'NN', 'VV', 'NN'], ['P', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VC', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'CS', 'VV', 'AS', 'NN', 'AD', 'NN', 'LC', 'AD', 'VE', 'NN'], ['NR', 'NN', 'NR', 'AD', 'VC', 'NN', 'NR', 'NN', 'NN'], ['AD', 'VV', 'VV', 'DT', 'NN', 'VE', 'CD', 'JJ', 'NN'], ['NN', 'NN', 'P', 'NT', 'NN', 'AD', 'CD', 'CD'], ['NN', 'NN', 'VC', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'DEC', 'NN'], ['PN', 'VV', 'VV', 'CD', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'AD', 'P', 'NN', 'VV'], ['CS', 'NN', 'VV', 'PN', 'AD', 'SB', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VA'], ['NT', 'AD', 'VV', 'AD', 'VV', 'NN'], ['PN', 'JJ', 'NN', 'VV', 'VV', 'VV', 'NN'], ['PN', 'AD', 'AD', 'VC', 'NR', 'MSP', 'VV', 'VV', 'SP'], ['NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VC', 'PN', 'DEG', 'VV', 'NN'], ['NN', 'NN', 'AD', 'CS', 'VV', 'VV', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'VV', 'SP'], ['AD', 'VV', 'AD', 'VE', 'JJ', 'DEG', 'NN', 'SP'], ['AD', 'VV', 'AD', 'VV', 'AD', 'VA', 'SP'], ['AD', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV', 'VV', 'AD', 'NN', 'DEG', 'NN', 'PN', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'NR', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'P', 'NN', 'NN', 'AD', 'VV', 'AS', 'VA'], ['PN', 'AD', 'P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'ETC', 'VV', 'JJ', 'NN'], ['NN', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'AD', 'VV', 'AD', 'VV', 'NT', 'NT', 'AD', 'VV', 'VV', 'DER', 'VV'], ['NN', 'NN', 'NR', 'NN', 'NR', 'NR'], ['NR', 'NR', 'VV', 'NR', 'LC', 'VV', 'VV', 'DEC', 'NN', 'VV'], ['VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEC', 'VV', 'NN', 'AD', 'VV'], ['NR', 'P', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'VA', 'DER', 'VV', 'NN'], ['P', 'VV', 'VA', 'DEC', 'NN', 'NN'], ['P', 'BA', 'NN', 'VV', 'NR', 'LC', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'NN', 'NR', 'AD', 'P', 'CD', 'M', 'CD', 'M', 'VV', 'OD'], ['AD', 'NN', 'VV', 'AS', 'NN', 'NN', 'SB', 'VV', 'AS', 'AD', 'AD', 'VV', 'NN'], ['VV', 'VV', 'VV', 'DEC', 'DT', 'NN', 'SP'], ['JJ', 'NN', 'PN', 'CS', 'VV', 'P', 'NN', 'VV', 'PN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NT', 'NR', 'AD', 'NN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['NN', 'P', 'NR', 'NN', 'VV', 'VV', 'VV', 'VV', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'VV', 'DEC', 'NN', 'VC', 'NN', 'AD', 'VV', 'LC', 'VV', 'DEC', 'JJ', 'NN'], ['PN', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'P', 'VA', 'NN', 'VV'], ['NN', 'SB', 'VV', 'LC', 'LB', 'NN', 'VV', 'VV', 'AS', 'NN', 'VV', 'NN', 'VV', 'LC', 'NN', 'VV'], ['NR', 'NR', 'PN', 'AD', 'VV'], ['NR', 'NN', 'DEG', 'NN', 'VV', 'NN', 'NT', 'AD', 'VE', 'AS', 'AD', 'VA', 'SP'], ['NR', 'DT', 'AD', 'VV', 'PN', 'VV', 'PN', 'PN', 'VV', 'AS', 'NR', 'JJ', 'DEG', 'NN'], ['NN', 'NN', 'AD', 'VV', 'P', 'VV', 'DEC', 'NN', 'VV', 'NN', 'NN'], ['NR', 'AD', 'AD', 'VA'], ['AD', 'NN', 'VE', 'CD', 'NN'], ['NN', 'VV', 'AD', 'NR', 'SP'], ['DT', 'VV', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'CD', 'M', 'VV', 'NN'], ['NN', 'P', 'NN', 'NN', 'VV'], ['NN', 'P', 'NN', 'LC', 'DEG', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['PN', 'VV', 'NN', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NR', 'AD', 'NN', 'VA'], ['NR', 'NN', 'NN', 'AD', 'VA', 'NN'], ['NN', 'NN', 'VV', 'AS', 'PN', 'CD', 'NN', 'P', 'DT', 'BA', 'VA', 'DEC', 'AD', 'VV', 'PN'], ['NN', 'NN', 'VC', 'NR', 'VV', 'NN', 'NN', 'VC', 'CD', 'M', 'NN', 'CD'], ['NR', 'NN', 'NT', 'VV', 'AD', 'BA', 'PN', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['VE', 'NN', 'VV', 'NR', 'VV', 'AS', 'CD', 'NN'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'NN', 'LC', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'SP'], ['NN', 'DEG', 'JJ', 'DT', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV'], ['CS', 'VV', 'VV', 'AD', 'P', 'NN', 'DEG', 'NR', 'NN', 'VV', 'NN'], ['NT', 'LC', 'NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV', 'SP'], ['AD', 'VC', 'NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV'], ['AD', 'CD', 'NN', 'AD', 'VV', 'VC', 'PN', 'VV', 'VV', 'SP'], ['NN', 'NT', 'P', 'NN', 'CD', 'NT', 'NN', 'AD', 'VV'], ['VV', 'NN', 'LC', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NR', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NR'], ['PN', 'DEG', 'NN', 'AD', 'VA', 'SP'], ['VV', 'VV', 'M', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'AS', 'CD', 'NN', 'VV', 'CD', 'VV'], ['PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'SP'], ['NR', 'AD', 'VV', 'VV', 'VV', 'VV', 'AS', 'PN', 'AD', 'VV', 'NN', 'SP', 'VV', 'PN', 'CC', 'DT', 'NN', 'DEG', 'VV', 'SP'], ['AD', 'NR', 'AD', 'VV', 'NR', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VE', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VV', 'LB', 'NN', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'PN', 'NT', 'VV', 'PN', 'SP'], ['VA', 'DEC', 'NR', 'NN', 'LC', 'VV', 'NN', 'NN', 'VV', 'NN', 'VA'], ['NN', 'AD', 'NN', 'NN', 'VV'], ['NN', 'NN', 'AD', 'NN'], ['DT', 'NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['NR', 'VV', 'AD', 'VV', 'CC', 'NN', 'VV', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'AD', 'PN', 'VA', 'DEC', 'NN'], ['P', 'NN', 'VV', 'VV', 'LC', 'PN', 'NN', 'JJ', 'NN', 'AD', 'AD', 'VV', 'DEV', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NN'], ['NR', 'DEG', 'NN', 'NR', 'P', 'VV', 'P', 'JJ', 'NN', 'LC', 'NN', 'AD', 'VV', 'VA'], ['NT', 'NR', 'VV', 'DEC', 'NN', 'NN'], ['P', 'NN', 'PN', 'VV', 'NN', 'NN', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN', 'LC'], ['AD', 'AD', 'VV', 'VV', 'NN', 'JJ', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'PN', 'VV', 'PN', 'AD', 'AD', 'VV', 'NN', 'SP'], ['VV', 'VV', 'NN', 'VV', 'PN'], ['AD', 'CD', 'JJ', 'DEG', 'NN', 'NN', 'VV'], ['AD', 'VV', 'NR', 'VV', 'VV'], ['VV', 'NN', 'VV', 'VV', 'VC', 'NN', 'VA', 'SP'], ['NN', 'NN', 'AD', 'SB', 'VV', 'NN', 'NN'], ['AD', 'NR', 'VV', 'AD', 'VV', 'AS', 'NR'], ['NN', 'VV', 'NN', 'VV', 'VC', 'VA', 'SP'], ['PN', 'VV', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'LC', 'AD', 'VA'], ['PN', 'DEG', 'NN', 'AD', 'VC', 'AD', 'VV', 'AD', 'VA', 'NN', 'AD', 'AD', 'VV'], ['PN', 'VC', 'M', 'JJ', 'DEG', 'NN', 'NN', 'VE', 'CD'], ['NN', 'NN', 'AD', 'VV', 'AS', 'AD', 'P', 'NN', 'VV', 'VV', 'NN', 'P', 'NN', 'VV'], ['CD', 'NN', 'VV', 'AD', 'VE', 'NN', 'VV', 'VA'], ['CS', 'VV', 'AD', 'VV', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VE', 'NN', 'SP', 'PN', 'AD', 'VV', 'AS', 'AD', 'AD', 'NN', 'AD', 'VV', 'PN'], ['AD', 'VA', 'DEC', 'VC', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['VV', 'NN', 'DT', 'NN', 'JJ', 'DEG', 'NN', 'VV', 'PN', 'DEG', 'NN', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VC', 'NN', 'NN', 'AD', 'VV', 'AD', 'VA', 'DEC', 'VV', 'NN'], ['DT', 'NN', 'NN', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'AD', 'VA', 'DEC', 'JJ', 'NN', 'NN', 'NN', 'NN', 'VV'], ['AD', 'AD', 'NN', 'LC', 'PN', 'AD', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN'], ['PN', 'NN', 'AD', 'VV', 'VE', 'NN', 'VE', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['VV', 'AD', 'P', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['PN', 'AD', 'CD', 'NR', 'NN', 'NN'], ['PN', 'P', 'JJ', 'NN', 'VE', 'NN', 'NN'], ['NN', 'JJ', 'VA', 'VV', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'AD', 'NN', 'VV', 'AS', 'PN', 'P', 'NN', 'VV'], ['VV', 'JJ', 'NN', 'PN', 'CD', 'AD', 'VV', 'VV', 'SP'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV'], ['NN', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'PN', 'VV', 'NN', 'PN', 'NN', 'AD', 'VE', 'NN', 'SP'], ['VV', 'AS', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['VV', 'SP', 'PN', 'VV', 'P', 'PN', 'AD', 'SP'], ['P', 'NN', 'AD', 'P', 'AD', 'VV', 'AD', 'AD', 'AD', 'VV', 'NN'], ['NN', 'LC', 'CD', 'NN', 'VV', 'NN'], ['P', 'VV', 'AD', 'NN', 'NR', 'VV', 'PN', 'VV'], ['VV', 'AD', 'VV', 'VV', 'JJ', 'NN', 'VA', 'SP', 'IJ'], ['PN', 'AD', 'VA', 'PN', 'AD', 'VA', 'PN', 'PN', 'AD', 'P', 'PN', 'VV'], ['PN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VC', 'P', 'NR', 'VV', 'P', 'NN', 'SP'], ['NT', 'NR', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN'], ['VV', 'AD', 'VV', 'NN', 'DEC', 'PN', 'NN', 'VA', 'NN', 'NN', 'VA'], ['VV', 'DER', 'AD', 'AD', 'AD', 'P', 'NN', 'VV', 'NN', 'BA', 'NN', 'VV'], ['VV', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN', 'VA', 'NN', 'VA', 'DEV', 'VV', 'P', 'NN', 'NN', 'LC'], ['AD', 'NN', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NR', 'AD', 'VV', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['VV', 'VV', 'PN', 'VV', 'NN', 'NN', 'VV', 'NN', 'SP'], ['NN', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'AS', 'AD', 'PN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['AD', 'AD', 'BA', 'DT', 'NN', 'P', 'NN', 'VV'], ['NN', 'NN', 'NN', 'VA', 'NN', 'CC', 'NN'], ['JJ', 'NN', 'LC', 'CD', 'NN', 'SB', 'VV', 'NN'], ['NR', 'NR', 'NN', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'VV', 'VV', 'NT'], ['P', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'NN'], ['PN', 'P', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'MSP', 'DT', 'NN', 'AD', 'AD'], ['DT', 'NN', 'PN', 'AD', 'VV', 'AS'], ['CS', 'NN', 'VV', 'AD', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VA', 'SP'], ['PN', 'NT', 'AD', 'NN', 'AS', 'NT', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NN', 'CD', 'PN', 'VC', 'NN'], ['NT', 'DEG', 'NN', 'NN', 'NR', 'AD', 'CD', 'NN', 'AD', 'VV', 'OD'], ['VA', 'VV', 'NN', 'AD', 'NN'], ['NR', 'NN', 'VV', 'NN'], ['AD', 'NR', 'NN', 'P', 'NN', 'NN', 'VV', 'NN', 'VV', 'P', 'NN', 'NN', 'VV', 'AS', 'NN'], ['NT', 'VV', 'VV', 'NN', 'PN', 'NN', 'P', 'NT', 'AD', 'AD', 'SB', 'VV', 'SP'], ['AD', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NT', 'NR', 'SB', 'NN', 'NN', 'VV'], ['AD', 'NN', 'VV', 'NN', 'AD', 'VA'], ['NN', 'VV', 'DEC', 'JJ', 'NN', 'VV', 'NN'], ['AD', 'P', 'NN', 'P', 'VV', 'NN', 'NN', 'VA', 'SP'], ['AD', 'P', 'NN', 'NN', 'AD', 'AD', 'NN', 'SP'], ['NR', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'CD', 'IJ', 'AD', 'VV', 'AD', 'P', 'PN', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VV', 'NN', 'VV', 'P', 'NR', 'NN', 'VV'], ['NR', 'VV', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NT', 'NR', 'AD', 'P', 'PN', 'VV', 'NN', 'VV', 'PN', 'PN', 'VV', 'DEC', 'NN'], ['NN', 'NR', 'VV', 'AD', 'PN', 'CD', 'VV', 'PN', 'VV', 'VV', 'PN', 'VV', 'NN', 'SP'], ['PN', 'VV', 'LC', 'PN', 'AD', 'AD', 'VV', 'NN', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'BA', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV'], ['NT', 'VV', 'VV', 'NN', 'BA', 'NN', 'LC', 'DEG', 'NN', 'VV'], ['AD', 'CC', 'DT', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV'], ['AD', 'VE', 'NR', 'CD', 'M', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'VV', 'AS', 'SP', 'PN', 'VV', 'VV', 'VV', 'SP', 'AD', 'VV', 'AD', 'VV', 'VV', 'SP'], ['P', 'NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV', 'NN'], ['VV', 'VV', 'NN', 'NN', 'NN'], ['NT', 'DT', 'NN', 'VV', 'P', 'NR', 'NR', 'NN', 'NN', 'NN', 'LC'], ['AD', 'NN', 'AD', 'VV', 'VV', 'CD', 'NN', 'CD', 'M', 'AD', 'VV', 'CD'], ['AD', 'VV', 'NN', 'AD', 'P', 'NN', 'NN', 'NN', 'NR', 'AD', 'VV'], ['CD', 'NN', 'AD', 'VE', 'VA', 'VA', 'DEC', 'NN', 'VV', 'VV', 'NN', 'VV'], ['P', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'AS', 'PN', 'AD', 'VV'], ['NR', 'VV', 'PN', 'AD', 'AD', 'P', 'NN', 'AD', 'VV'], ['NR', 'VC', 'JJ', 'NN', 'NN', 'AD', 'VE', 'NN'], ['PN', 'VV', 'DT', 'NN', 'VV', 'PN', 'AD', 'AD', 'AD'], ['AD', 'VC', 'P', 'JJ', 'NN', 'CC', 'NN', 'VV'], ['NN', 'DEG', 'NR', 'NN', 'VA', 'AS', 'CD', 'VV', 'NN', 'VV', 'VV'], ['NT', 'NR', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'PN', 'VV', 'AS'], ['VV', 'DT', 'NR', 'NN', 'SP', 'VV', 'VV', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN'], ['AD', 'P', 'NT', 'NN', 'VV', 'VV', 'DEC', 'NN', 'LC', 'VV', 'NN', 'VV', 'NN'], ['P', 'NN', 'LC', 'P', 'NR', 'VC', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN'], ['PN', 'P', 'PN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['AD', 'PN', 'VC', 'VV', 'PN', 'NN', 'AD', 'P', 'NR', 'NN'], ['VA', 'NN', 'VV', 'AS', 'PN', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'NN', 'SB', 'P', 'VA', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'NN', 'NN', 'P', 'NN', 'NN', 'NN', 'VA'], ['VV', 'AD', 'AD', 'VV', 'NN', 'NN', 'AD', 'AD', 'AD', 'VA'], ['VV', 'VV', 'PN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV'], ['PN', 'VV', 'VV', 'NN', 'VV', 'NN', 'PN', 'VV', 'AS', 'PN', 'NT', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'NN', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN'], ['P', 'NN', 'NN', 'AD', 'NN', 'NN', 'NN', 'VC', 'NN'], ['VV', 'VV', 'VA', 'DEC', 'NN'], ['VV', 'AD', 'NN', 'AD', 'VV', 'AD', 'M'], ['AD', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'AD', 'AD', 'VV', 'NN', 'VV'], ['NN', 'LC', 'NR', 'AD', 'AD', 'VV', 'NN', 'NN'], ['P', 'JJ', 'NN', 'NN', 'PN', 'P', 'NN', 'VV', 'AD', 'VV'], ['DT', 'NR', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['VV', 'AD', 'VV', 'AD', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV', 'SP'], ['AD', 'NT', 'NT', 'AD', 'VV', 'NR', 'AD', 'VA', 'DEC', 'VV', 'DT', 'NR', 'AD', 'VV', 'SP'], ['VV', 'VV', 'NR', 'CC', 'VV', 'VV', 'NN', 'VV', 'CD'], ['NR', 'AD', 'VV', 'AD', 'AD', 'JJ', 'NN', 'AD', 'VV', 'AS'], ['VV', 'NN', 'DEC', 'VV', 'NN', 'VV', 'P', 'NR', 'VV', 'NR', 'VV', 'NR', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'NN', 'JJ', 'NN', 'P', 'NN', 'NN', 'NR', 'NN'], ['NN', 'VV', 'DEC', 'VC', 'JJ', 'NN', 'NN'], ['NN', 'LC', 'NR', 'AD', 'VV', 'NN'], ['PN', 'VV', 'AD', 'VV', 'NN', 'NT', 'CD', 'CD', 'CD', 'CD', 'M'], ['NN', 'LC', 'DEG', 'NN', 'P', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['AD', 'NN', 'NN', 'AD', 'P', 'VV', 'VV', 'JJ', 'NN', 'NN'], ['AD', 'NN', 'VV', 'NN', 'LC', 'AD', 'P', 'NN', 'VV'], ['NN', 'VV', 'DEC', 'NN', 'VC', 'AD', 'VA'], ['DT', 'NR', 'NN', 'AD', 'AD', 'VA', 'NN', 'AD', 'VA'], ['NR', 'AD', 'VV', 'PN', 'VA', 'DEC', 'NN'], ['NN', 'P', 'NN', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['VV', 'P', 'DT', 'M', 'NN', 'NR', 'VV', 'NN', 'JJ'], ['DT', 'NN', 'VV', 'AD', 'AD', 'VV', 'NN', 'NN', 'SP'], ['PN', 'NT', 'NN', 'LC', 'AD', 'AD', 'VV', 'VV', 'AS', 'DT', 'NN'], ['PN', 'AD', 'AD', 'VV', 'NR', 'AD', 'P', 'NR', 'VE', 'NN'], ['AD', 'VV', 'NN', 'LC', 'AD', 'VV', 'NN', 'NN', 'DT', 'NN', 'AD', 'VV'], ['CS', 'P', 'NN', 'CC', 'NN', 'NN', 'CC', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN'], ['CS', 'NN', 'AD', 'VA', 'NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'P', 'NN', 'VV'], ['NR', 'NN', 'NR', 'VV', 'NN', 'NN', 'VC', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'VV', 'AD', 'VV', 'VA', 'DEV', 'VV'], ['NT', 'NN', 'DT', 'P', 'VV', 'VV', 'NN', 'NN', 'SB', 'AD', 'AD', 'VV'], ['NR', 'AD', 'VV', 'VV', 'NR', 'DEG', 'NN', 'NN', 'SP'], ['AD', 'AD', 'AD', 'VV', 'AD', 'NN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NR', 'NR', 'AD', 'P', 'VV', 'NN', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'PN'], ['VV', 'VV', 'LC', 'VV', 'LB', 'NN', 'VV', 'AD', 'P', 'NN', 'BA', 'PN', 'VV'], ['NN', 'VV', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NT', 'PN', 'PN', 'AD', 'AD', 'VA', 'AS', 'AD', 'AD', 'VV', 'AS', 'PN'], ['JJ', 'NN', 'DEG', 'AD', 'VV', 'VV', 'VV', 'AD', 'VA', 'SP'], ['PN', 'VC', 'VV', 'PN', 'VV', 'AS', 'DEC', 'NN', 'SP'], ['PN', 'VV', 'VV', 'AS', 'JJ', 'DEG', 'NN', 'VV', 'NN', 'CC', 'NN'], ['P', 'NT', 'JJ', 'NN', 'P', 'NR', 'VA', 'AD', 'AD', 'NN', 'AD', 'VV', 'VA', 'NN'], ['JJ', 'NN', 'NN', 'VV', 'VV', 'NR', 'NR', 'MSP', 'NN', 'NN', 'AD', 'VE', 'NN'], ['AD', 'AD', 'VA', 'DEC', 'NN', 'NN', 'VV', 'AD', 'AD', 'VV', 'PN'], ['AD', 'VV', 'LB', 'NR', 'MSP', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['CS', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'AD', 'NN'], ['NR', 'CD', 'NN', 'VV', 'P', 'NN', 'NN', 'DEG', 'NN'], ['DT', 'NN', 'AD', 'VC', 'JJ', 'NR', 'NN', 'VV'], ['NN', 'LC', 'NN', 'CC', 'NN', 'VV', 'VV', 'VV', 'AS', 'NN', 'CC', 'NR', 'AD', 'VV'], ['VV', 'JJ', 'NN', 'NN', 'AD', 'VE', 'NN'], ['AD', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['VV', 'M', 'DT', 'NN', 'AD', 'JJ', 'NN', 'PN', 'AD', 'VV', 'SP'], ['NN', 'NN', 'AD', 'VV', 'VV', 'VA'], ['PN', 'CC', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'AS', 'NN'], ['AD', 'VV', 'AD', 'VC', 'VV', 'NN', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['PN', 'VC', 'AD', 'P', 'DT', 'NN', 'VV', 'DEC', 'NN', 'VV', 'SP', 'SP'], ['VV', 'NN', 'NN', 'CS', 'NR', 'VV', 'AD', 'JJ', 'NN', 'AD', 'VV', 'NN'], ['NR', 'P', 'JJ', 'NN', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'NN', 'VV'], ['NR', 'AD', 'VV', 'PN', 'JJ', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'SP', 'NN', 'NN', 'PN', 'AD', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'NN', 'ETC', 'NN', 'NN', 'AD', 'VA'], ['NN', 'NN', 'LC', 'PN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['NT', 'PN', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VA'], ['VV', 'LC', 'PN', 'VA', 'LC', 'VV', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV'], ['P', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'VV', 'VV', 'ETC', 'NN', 'NN'], ['NN', 'NN', 'NN', 'AD', 'CD', 'CD', 'CD', 'CD', 'CD', 'CD', 'NN'], ['PN', 'VV', 'AS', 'NT', 'SP'], ['DT', 'CD', 'AD', 'VV', 'VV', 'PN', 'P', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'AD', 'CC', 'VV', 'AD', 'CC'], ['PN', 'VV', 'NN', 'AD', 'VV', 'AD', 'AD', 'AD', 'VE'], ['JJ', 'NN', 'LC', 'DEG', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'JJ', 'CC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'JJ', 'NN'], ['NN', 'SB', 'VV', 'P', 'NN', 'NN', 'VV', 'VV'], ['VV', 'AS', 'NR', 'NN', 'AD', 'VV', 'DEC', 'NN', 'AD', 'NT', 'DEG', 'NN', 'NN', 'VA'], ['NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'VA', 'DEC', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'P', 'NN', 'LC', 'VV', 'AS', 'NN', 'NN'], ['VV', 'NN', 'VV', 'NR', 'P', 'JJ', 'DEG', 'NN', 'P', 'NR', 'NN', 'VV', 'NN'], ['AD', 'PN', 'AD', 'VC', 'NN', 'NN'], ['JJ', 'NN', 'NR', 'DEG', 'NN', 'NN', 'NN', 'JJ', 'VV', 'AS', 'NN'], ['NN', 'AD', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['PN', 'AD', 'CC', 'NR', 'NN', 'VV', 'AS', 'AD', 'VE', 'NN'], ['AD', 'CD', 'NN', 'SB', 'VV', 'NN'], ['NR', 'VV', 'CC', 'VV', 'NN', 'AD', 'AD', 'P', 'NN', 'VV', 'NN'], ['PN', 'P', 'PN', 'NN', 'VV'], ['PN', 'VV', 'P', 'CD', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'AS'], ['PN', 'P', 'PN', 'VV', 'NN', 'SP'], ['NN', 'JJ', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NN'], ['NR', 'CC', 'NN', 'AD', 'VC', 'NN'], ['DT', 'NN', 'JJ', 'NN'], ['NT', 'NN', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'AD', 'NN', 'VV'], ['AD', 'AD', 'AD', 'VV', 'PN', 'VV'], ['AD', 'P', 'VV', 'AD', 'VA', 'DEC', 'NN', 'SP'], ['AD', 'PN', 'VV', 'VV', 'AD', 'JJ', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['PN', 'VV', 'AS', 'NN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['NT', 'NR', 'NN', 'NR', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'NR', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['AD', 'VV', 'LC', 'VV', 'SB', 'VV', 'VV', 'P', 'NN'], ['AD', 'NN', 'AD', 'VA', 'AD', 'VA', 'DEV', 'VV', 'AD', 'NN', 'VV', 'VV'], ['PN', 'VV', 'NR', 'VC', 'DT', 'NN', 'JJ', 'NN', 'NR', 'DEG', 'NN', 'VV', 'CD', 'NN'], ['VV', 'VV', 'NN', 'VC', 'NN', 'NN', 'NN', 'LC', 'AD', 'VA', 'DEC', 'CD', 'NN'], ['NN', 'LC', 'DT', 'AD', 'VV', 'NR', 'AD', 'VV', 'DEC', 'NN', 'VV', 'AS'], ['NR', 'NR', 'NR', 'NN', 'VC', 'NR', 'VV', 'NN', 'NN', 'JJ', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'CC', 'NN', 'NN', 'LC', 'DEG', 'NN', 'AD', 'VA', 'CC', 'VA'], ['P', 'NN', 'LC', 'DEG', 'NR', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'AD', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'P', 'NN', 'AD', 'VC', 'VE', 'NN', 'SP'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'VV', 'CD', 'CD', 'CD', 'CD', 'CD'], ['PN', 'VV', 'NN', 'NN', 'VV', 'JJ', 'VV', 'PN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'NN', 'LC', 'AD', 'VC', 'VV', 'NN', 'VV', 'P', 'NN', 'LC', 'VV', 'VV'], ['CD', 'VV', 'DEC', 'NN', 'AD', 'VC', 'NN', 'AD', 'P', 'NN', 'VV'], ['NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'AS', 'AD', 'NT', 'PN', 'VC', 'VV', 'PN', 'NN', 'SP'], ['NN', 'NN', 'NT', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'P', 'NN', 'VV'], ['CS', 'NN', 'NN', 'NN', 'VV', 'AD', 'DT', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV'], ['CS', 'PN', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'AD', 'NN', 'VV', 'P', 'PN', 'NN', 'VV'], ['NN', 'NN', 'VC', 'CD', 'VV', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'CD', 'NN', 'CD', 'NN'], ['PN', 'SB', 'AD', 'VV', 'LC', 'P', 'NN', 'NN', 'NN', 'NN', 'VA'], ['NN', 'AD', 'AD', 'P', 'NN', 'VV', 'NR', 'PN', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV'], ['PN', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'VV', 'PN', 'DEG', 'NN', 'VV'], ['AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['NT', 'NN', 'AD', 'NR', 'DT', 'VV', 'NN'], ['AD', 'AD', 'AD', 'VA', 'AD', 'PN', 'VA', 'SP'], ['NR', 'NN', 'NN', 'VC', 'VA', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'VV', 'P', 'NR', 'AD', 'VV', 'NN', 'VA', 'DEC', 'NN'], ['NR', 'VV', 'P', 'NN', 'DEG', 'NN', 'VV', 'VV', 'AD', 'VV'], ['PN', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS'], ['AD', 'NN', 'VV', 'P', 'VV', 'VV', 'DEC', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'SB', 'VV', 'AS', 'AD', 'NN', 'AD', 'VE', 'SB', 'VV', 'DEC', 'NN', 'NN', 'VE', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NN', 'SB', 'VV', 'DEC', 'VA', 'SP'], ['JJ', 'NN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN'], ['P', 'PN', 'VV', 'AD', 'P', 'NN', 'LC', 'P', 'NN', 'VV', 'CD', 'NN', 'NN'], ['JJ', 'DEG', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'VV', 'PN', 'VV', 'AS', 'CD', 'NN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VV', 'VV', 'SP'], ['NR', 'VV', 'PN', 'AD', 'VV', 'NN', 'NN', 'ETC', 'VV', 'VA'], ['AD', 'NN', 'VV', 'P', 'NR', 'CC', 'NN', 'VV', 'NN', 'LC', 'VV'], ['VV', 'PN', 'AD', 'BA', 'NN', 'VV', 'PN', 'SP'], ['NT', 'AD', 'P', 'CD', 'CD', 'CD', 'NN', 'VV', 'VV', 'M'], ['PN', 'AD', 'AD', 'AD', 'VV', 'PN', 'AD', 'VV'], ['AD', 'DT', 'NN', 'NT', 'P', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN'], ['P', 'DT', 'NR', 'NN', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'NN', 'VV', 'PN', 'P', 'NN', 'LC', 'VV', 'NR', 'NN', 'JJ', 'AD', 'VV'], ['NN', 'NN', 'VC', 'NR', 'NN', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NR', 'VV', 'VV', 'NN', 'VV'], ['AD', 'AD', 'VV', 'AS', 'NN', 'LC', 'AD', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NT', 'AD', 'VV'], ['AD', 'NN', 'VV', 'DEC', 'VC', 'AD', 'NN', 'NN', 'CD', 'M', 'CD'], ['AD', 'VV', 'PN', 'AD', 'VV', 'P', 'PN', 'VV', 'VV', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'AD', 'VV', 'DEC', 'NR', 'AD', 'CD', 'JJ', 'NN', 'JJ', 'NN', 'AD', 'VV'], ['PN', 'VC', 'NN', 'SP', 'PN', 'NN', 'CC', 'CD', 'NN', 'SP'], ['AD', 'NT', 'AD', 'VV', 'PN', 'BA', 'NN', 'VV', 'NN', 'LC', 'VV', 'PN', 'DT', 'VE', 'NN', 'DEC', 'NN', 'SP'], ['VV', 'P', 'NN', 'AD', 'VV', 'P', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['VV', 'VV', 'NN', 'NN', 'VV', 'NN'], ['P', 'NT', 'LC', 'VV', 'AD', 'NN', 'AD', 'VV', 'VV', 'NR', 'NR'], ['NR', 'NR', 'NN', 'NN', 'VC', 'NR', 'NR', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'VV', 'NN', 'VV', 'P', 'NN', 'NN', 'VV', 'AD', 'VV'], ['NR', 'VC', 'NR', 'NR', 'JJ', 'NN', 'JJ', 'NN', 'JJ', 'NN', 'NN', 'NN'], ['NR', 'NN', 'NR', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['PN', 'AD', 'BA', 'P', 'NN', 'DEG', 'NN', 'VV', 'VV', 'VV'], ['NR', 'JJ', 'JJ', 'NN'], ['AD', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'P', 'NN', 'VV'], ['NT', 'JJ', 'NN', 'AD', 'P', 'PN', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'VC', 'NR', 'NN', 'AD', 'VC', 'NN', 'JJ', 'NN', 'NN', 'SP'], ['PN', 'VV', 'NN', 'CC', 'VV', 'VV', 'NN', 'P', 'PN', 'VV'], ['NN', 'VV', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'NN', 'SP'], ['NT', 'VE', 'NN', 'VV', 'AD', 'VV', 'DEC', 'CD', 'IJ', 'NN', 'VV', 'NN', 'NN'], ['VC', 'NR', 'NN', 'CC', 'NN', 'NN'], ['NN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'LC', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'NR', 'P', 'NR', 'CD', 'DEG', 'JJ', 'NN', 'VV', 'NN'], ['NT', 'CD', 'NN', 'NN', 'NN', 'DEC', 'NN', 'AD', 'VA'], ['PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VA'], ['NN', 'AD', 'VV', 'NR', 'VV', 'VV', 'AS', 'CD', 'NN', 'MSP', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NR', 'VC', 'NN', 'VV', 'AS', 'NN', 'VV', 'PN', 'VV', 'AD', 'VV', 'AS'], ['NR', 'AD', 'VV', 'VV', 'CD', 'NN', 'P', 'IJ', 'VV', 'NN'], ['NN', 'NN', 'VA', 'DER', 'VV', 'NN', 'NN', 'AD', 'VV', 'SP'], ['NN', 'P', 'P', 'NR', 'VV', 'LC', 'VV', 'PN', 'AD', 'VA'], ['NN', 'VV', 'CD', 'CD', 'NN', 'AD', 'JJ', 'NN', 'NN'], ['PN', 'AD', 'CC', 'VV', 'NR', 'NN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'LB', 'NN'], ['NR', 'NR', 'NN', 'NN', 'NR', 'AD', 'PN', 'DEG', 'JJ', 'NN'], ['AD', 'AD', 'NN', 'AD', 'VV', 'JJ', 'DEG', 'NN', 'AD', 'P', 'NT', 'AD', 'AD', 'VV', 'VV'], ['NN', 'AD', 'VV', 'CD', 'NN', 'DEG', 'NN', 'AD', 'NN', 'VE', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'NN', 'NN', 'VV'], ['NR', 'NN', 'NN', 'NN', 'LC', 'AD', 'VE', 'CD', 'NN', 'P', 'AD', 'VV'], ['NN', 'NR', 'DEG', 'NN', 'VC', 'NR', 'DEG', 'JJ', 'NN', 'CC', 'NR', 'NN'], ['NR', 'P', 'CD', 'VV', 'VE', 'NN', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'LC', 'VV'], ['NN', 'NN', 'P', 'JJ', 'NN', 'NN'], ['NN', 'VV', 'AD', 'AD', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['NR', 'BA', 'NR', 'VV', 'DER', 'NR', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'VA', 'DEV', 'VV', 'AS', 'VV', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'LC', 'AD', 'VV'], ['NR', 'NN', 'NR', 'AD', 'AD', 'VV'], ['NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'VA', 'SP'], ['AD', 'VV', 'NR', 'AD', 'VC', 'VA', 'DEC', 'NN'], ['VV', 'NN', 'DEG', 'NN', 'NN', 'VE', 'PN', 'VV'], ['NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NR', 'DEG', 'NN', 'VV', 'AS', 'VV', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV'], ['NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'NR', 'DEC', 'NN', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'P', 'NR', 'VV', 'VA'], ['NN', 'AD', 'VV', 'NN', 'P', 'CD', 'M', 'LC', 'DEG', 'NN', 'DT', 'NN', 'LC'], ['NN', 'AD', 'P', 'NT', 'VV', 'AD', 'VV'], ['AD', 'VV', 'VV', 'NN', 'BA', 'NN', 'DEG', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['P', 'PN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'PN', 'NN', 'NN', 'LC', 'NN', 'SP'], ['P', 'NN', 'JJ', 'DEG', 'NN', 'LC', 'PN', 'AD', 'VV', 'AS', 'AD', 'VV', 'AD', 'VV', 'CC', 'AD', 'VV'], ['VV', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN'], ['DT', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VC', 'VV', 'AS', 'CC', 'PN', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'NN', 'VA', 'CC', 'NN', 'NR'], ['DT', 'NN', 'SB', 'VV', 'NR', 'NN', 'NN', 'NR', 'VV'], ['AD', 'NN', 'AD', 'VV', 'NN', 'VV', 'DER', 'VV', 'SP'], ['VV', 'NR', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'AD', 'VA'], ['NN', 'VV', 'CD', 'NN', 'CS', 'VV', 'AS', 'NN', 'AD', 'AD', 'AD', 'VV', 'AD', 'AD', 'VV', 'AS'], ['AD', 'JJ', 'NN', 'AD', 'JJ', 'NN', 'VA', 'VA'], ['CD', 'M', 'NN', 'VV', 'DT', 'NN', 'AD', 'VC', 'P', 'PN', 'DEG', 'NN', 'VV'], ['AD', 'NR', 'NN', 'AD', 'VC', 'NN', 'NR', 'NN', 'NN', 'VV', 'P', 'NT'], ['JJ', 'DEG', 'NN', 'NN', 'VV', 'AS', 'PN', 'VV', 'NN', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'NN', 'AD', 'SB', 'VV', 'VC', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['NT', 'VV', 'AS', 'NN', 'NN', 'SP'], ['NN', 'AD', 'AD', 'VA'], ['NN', 'VV', 'AD', 'VV', 'VV', 'NR', 'DEG', 'NN', 'NN'], ['VV', 'NN', 'NN', 'DEG', 'NN'], ['CC', 'VV', 'NN', 'VV'], ['AD', 'VV', 'PN', 'VC', 'VV', 'SP'], ['NR', 'NN', 'NN', 'VA'], ['CS', 'VV', 'AS', 'NN', 'AD', 'VC', 'VV', 'VV', 'DEC', 'P', 'P', 'NN', 'VV', 'VE', 'NN'], ['AD', 'AD', 'PN', 'AD', 'VV', 'VV', 'SP'], ['AD', 'PN', 'AD', 'VV', 'AS', 'NR', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'AD', 'VV', 'VV', 'NN'], ['PN', 'VV', 'VC', 'P', 'NN', 'VV', 'SP'], ['VV', 'NN', 'NR', 'AD', 'VV'], ['NN', 'AD', 'VV', 'VV', 'LC', 'NN', 'BA', 'NR', 'NN', 'VV', 'CD', 'NN'], ['NR', 'VV', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'CD', 'NN', 'VV', 'VV'], ['VV', 'CD', 'VV', 'DEC', 'JJ', 'AD', 'BA', 'NN', 'VV', 'AD', 'VV', 'PN', 'AD', 'VV'], ['VV', 'NR', 'DEG', 'NN', 'VC', 'NR', 'NN', 'NN'], ['P', 'PN', 'LC', 'VV', 'JJ', 'NN', 'VV', 'AD', 'VA'], ['NN', 'NN', 'VV', 'DER', 'AD', 'VV', 'AS', 'AD', 'VV', 'NR', 'NN', 'NN'], ['NN', 'NN', 'P', 'VV', 'NN', 'NN', 'NN'], ['NR', 'NN', 'VA', 'AS', 'AD', 'PN', 'SP'], ['NT', 'VE', 'DT', 'JJ', 'DEG', 'NN', 'NN'], ['P', 'PN', 'VE', 'AS', 'NN', 'AD', 'VV', 'PN', 'VV', 'NN'], ['VV', 'NN', 'NN', 'VV', 'P', 'NN', 'VV'], ['NN', 'VV', 'P', 'NN', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'LC', 'VV', 'DEC', 'NN', 'AD', 'VA', 'AS', 'CD'], ['NN', 'DEG', 'NN', 'VV', 'VV', 'NN'], ['VE', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'VV'], ['CD', 'NN', 'NN', 'VV', 'NN', 'NN'], ['JJ', 'NT', 'NN', 'VV', 'SP'], ['AD', 'NN', 'LC', 'CC', 'NN', 'DEG', 'NN'], ['NN', 'P', 'NN', 'NN', 'VV', 'AS', 'AD', 'PN', 'P', 'VV', 'NN', 'VE', 'DT', 'NN', 'SP'], ['NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'VA', 'NN'], ['AD', 'NN', 'VV', 'AD', 'VA', 'VA', 'DEC', 'NN', 'NN'], ['NR', 'AD', 'AD', 'AD', 'VV', 'AD', 'VV', 'VA', 'NN', 'VV', 'NN', 'P', 'NN', 'VV'], ['VV', 'CC', 'VV', 'NN', 'P', 'NT', 'CD', 'CD', 'CD', 'CD', 'M'], ['BA', 'NN', 'VV', 'VA', 'AS', 'NN', 'AD', 'VE', 'NN', 'SP'], ['VV', 'AS', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NT', 'DEG', 'NN', 'NN', 'VV', 'NN', 'VA'], ['VV', 'JJ', 'DEG', 'NN', 'LC', 'DT', 'AD', 'NR', 'P', 'NR', 'VV', 'AS', 'M', 'VV'], ['P', 'VV', 'NN', 'LC', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'AD', 'NN', 'VV'], ['NR', 'NR', 'VV', 'AD', 'VV', 'NN', 'VV', 'PN', 'VC', 'M', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'SP', 'SP'], ['NN', 'PN', 'AD', 'VV', 'NR', 'NN', 'DEG', 'NN', 'SP'], ['PN', 'VV', 'NN', 'P', 'NN', 'VC', 'NR', 'VV', 'NN'], ['AD', 'NR', 'VV', 'VV', 'VV', 'NN', 'AD', 'AD', 'VA'], ['DT', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['P', 'NN', 'NN', 'VV', 'AS', 'CD', 'M', 'NN'], ['AD', 'NR', 'P', 'NN', 'NN', 'AD', 'VV', 'NN'], ['P', 'PN', 'VV', 'AD', 'VV', 'VV', 'DT', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'VV'], ['VV', 'NR', 'OD', 'DEC', 'NR', 'NN', 'AD', 'VV', 'VV'], ['NR', 'DEG', 'NN', 'VV', 'AD', 'VV', 'AS', 'DT', 'NN', 'VV', 'AD', 'VV'], ['AD', 'VV', 'PN', 'NN', 'DEG', 'MSP', 'VV', 'PN', 'VV', 'PN', 'PN', 'VV', 'PN'], ['P', 'NN', 'NN', 'NN', 'VV', 'AS', 'DEC', 'NN', 'NN', 'VC', 'PN', 'DEG'], ['CD', 'P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV'], ['AD', 'NT', 'NN', 'AD', 'VV', 'NR', 'LC', 'AD', 'VV', 'AS', 'NR'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'NR', 'NN', 'VV', 'CD', 'NN', 'LC', 'VV'], ['VV', 'VV', 'DEC', 'NN', 'AD', 'VC', 'P', 'NN', 'CC', 'NN', 'VV', 'SP'], ['NN', 'LC', 'P', 'NR', 'NN', 'VV', 'AS', 'NR', 'VV', 'CC', 'VV', 'VV'], ['P', 'AD', 'NN', 'LC', 'NR', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN'], ['VC', 'VV', 'DEC', 'NN', 'NN', 'VV', 'NN', 'JJ', 'NN', 'CC', 'NN', 'SP'], ['PN', 'P', 'VV', 'SP'], ['NT', 'NR', 'SB', 'NR', 'NR', 'NN', 'AD', 'VV', 'VV'], ['NN', 'LC', 'NR', 'NN', 'NN', 'AD', 'VA', 'VA', 'VA'], ['NR', 'CC', 'NR', 'JJ', 'NN', 'VC', 'NR', 'NN', 'VV', 'NR', 'DEG', 'CD', 'M', 'NN', 'NN'], ['AD', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN', 'NR', 'DEG', 'NN', 'NR', 'VE', 'NN'], ['AD', 'VV', 'DEC', 'PN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['AD', 'PN', 'AD', 'VV', 'BA', 'AD', 'VV', 'VV', 'NR', 'SP'], ['NN', 'NN', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['P', 'NN', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN', 'VV'], ['VV', 'VV', 'AD', 'CD', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VC', 'NR', 'JJ', 'NN', 'LC', 'DEG', 'NN', 'SB', 'VV'], ['NR', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'P', 'AD', 'VV', 'VV', 'CD'], ['NR', 'NN', 'DEG', 'NN', 'NR', 'PN'], ['OD', 'AD', 'VA', 'DEC', 'NN', 'VA'], ['NN', 'VV', 'VV', 'PN', 'VV', 'VV', 'SP'], ['PN', 'NN', 'LC', 'AD', 'VV', 'NN'], ['P', 'VC', 'OD', 'VV', 'VA', 'DEC', 'NN'], ['PN', 'VC', 'NN', 'NN'], ['VV', 'SP', 'NT', 'AD', 'VE', 'NN'], ['VV', 'VV', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'AD', 'AD', 'AD', 'VV', 'AS'], ['PN', 'AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'PN', 'VV', 'NR', 'AD', 'VV'], ['CD', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'NN', 'VA'], ['VE', 'PN', 'AD', 'VV', 'SP'], ['NR', 'VV', 'NR', 'VV', 'P', 'NN', 'VV'], ['PN', 'VV', 'AD', 'VV', 'VV', 'NN'], ['P', 'NN', 'VV', 'VV', 'AS', 'DT', 'NN', 'NR', 'AD', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VA', 'VA', 'VV', 'NN', 'NN', 'SP', 'CD'], ['NN', 'AD', 'AS', 'CD', 'NN', 'AD', 'VV', 'AD', 'VV', 'MSP', 'VV', 'NN', 'VC', 'PN', 'NN'], ['VA', 'VV', 'AD', 'SP'], ['AD', 'NN', 'AD', 'NN', 'NN', 'VV', 'AS', 'NR'], ['VV', 'PN', 'VV', 'NN', 'NR', 'NR', 'AD', 'VE', 'NN', 'SP'], ['AD', 'NR', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'AD', 'VV', 'VV', 'AD', 'NN', 'AD'], ['NT', 'AD', 'NN', 'NN', 'P', 'NR', 'AD', 'VV'], ['NN', 'VV', 'VV', 'VC', 'DT'], ['PN', 'AD', 'VV', 'NN', 'NN', 'SP', 'PN', 'AD', 'AD', 'AD', 'AD', 'P', 'VV', 'AS'], ['VV', 'NT', 'NR', 'VV', 'AS', 'CD', 'NN', 'VV', 'VV'], ['NR', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'NN', 'AD', 'VV', 'NN', 'DEG', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['CD', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'JJ', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'JJ', 'NN', 'DEC', 'P', 'NR', 'VC', 'NN', 'DEC', 'NN', 'NN', 'NN'], ['AD', 'NT', 'NN', 'NN', 'NN', 'AD', 'VV', 'DT', 'NN', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VV', 'VV', 'VV', 'NR', 'JJ', 'DEG', 'NN'], ['NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'P', 'NN', 'SP'], ['P', 'JJ', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV'], ['BA', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VE', 'CD', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'VV', 'M', 'NN'], ['VE', 'NN', 'NN', 'VV', 'DEC', 'NN', 'M'], ['CS', 'NN', 'NN', 'NN', 'NT', 'CD', 'NT', 'AD', 'VV'], ['PN', 'VV', 'NN', 'PN', 'P', 'VV', 'VV', 'NN', 'P', 'NN', 'DEG', 'AD', 'VC', 'VA', 'SP'], ['NN', 'NN', 'VV', 'VV', 'AS', 'JJ', 'NN', 'VV', 'VV', 'SP'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'NN', 'JJ', 'NN'], ['VV', 'NR', 'PN', 'VC', 'VV', 'VV', 'PN', 'AD', 'VV', 'PN', 'SP'], ['VV', 'NN', 'NN', 'NR', 'AD', 'VV', 'NN', 'ETC', 'NN', 'DEG', 'NN'], ['AD', 'PN', 'P', 'NN', 'NN', 'VV', 'VV', 'AD', 'DEV', 'VV', 'AD', 'VV'], ['NR', 'VV', 'AS', 'NR', 'NN', 'AD', 'NN', 'P', 'NN', 'VE', 'AD', 'CD', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'NR', 'CC', 'DT', 'CD', 'NN', 'NN', 'VC', 'NN'], ['NT', 'NN', 'LC', 'NN', 'P', 'PN', 'VV', 'NN', 'VV', 'AD', 'VV', 'SP', 'VV', 'AS', 'NN', 'VV', 'AS', 'NN'], ['AD', 'DT', 'CD', 'NN', 'VV', 'NR', 'VV', 'NN', 'VV', 'VV'], ['VV', 'NR', 'NN', 'AD', 'VV', 'DEC', 'NN', 'NR', 'AD', 'VV'], ['AD', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN', 'VV', 'VV', 'DER', 'VA', 'AD'], ['CS', 'NN', 'JJ', 'NN', 'AD', 'AD', 'VV', 'NN'], ['VV', 'AS', 'VV', 'NN', 'VV', 'DEC', 'AD', 'VV', 'CD', 'NN', 'VV', 'DEC'], ['NR', 'NN', 'JJ', 'NN', 'DEG', 'NR', 'AD', 'VV', 'AS', 'NN', 'VV', 'SP', 'NR', 'AD', 'VE', 'NN'], ['NR', 'VV', 'LC', 'AD', 'AD', 'VV', 'P', 'NR', 'VC', 'JJ', 'NN', 'DEC', 'NR', 'NN', 'NN'], ['NN', 'NN', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'NN', 'VV', 'LC', 'NR', 'VC', 'NR', 'NN', 'JJ', 'NN', 'NN', 'NN'], ['AD', 'PN', 'NR', 'NR', 'DEG', 'NN', 'AD', 'LB', 'NR', 'NN', 'AD', 'VV'], ['P', 'NN', 'JJ', 'NN', 'NN', 'VV', 'VV'], ['AD', 'JJ', 'JJ', 'DEG', 'NN', 'AD', 'P', 'PN', 'VV', 'VV', 'NN', 'NN'], ['P', 'NN', 'LC', 'NN', 'VC', 'NN', 'DEG', 'NN'], ['PN', 'DEG', 'NN', 'VV', 'SP', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AS'], ['NN', 'SB', 'VV', 'AD', 'P', 'NN', 'AD', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'CD', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'PN', 'VA', 'DEV', 'BA', 'DT', 'NN', 'AD', 'P', 'NN', 'AD', 'AD', 'VA', 'DEC', 'NN'], ['VV', 'AS', 'VV', 'PN', 'VA', 'VA', 'AS', 'VV', 'PN', 'VV', 'VV'], ['PN', 'VA', 'DEC', 'NN', 'PN', 'AD', 'VV', 'CD', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'VV', 'AD', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NN', 'DEC', 'PN', 'AD', 'VV', 'VV'], ['AD', 'P', 'DT', 'AD', 'VV', 'AS', 'NN', 'CD', 'CD', 'CD', 'CD', 'VV', 'NN'], ['NN', 'VV', 'NN', 'PN', 'BA', 'PN', 'VV', 'VV', 'SP'], ['AD', 'NN', 'NN', 'NN', 'VA'], ['VV', 'VC', 'CD', 'CD', 'CD', 'CD'], ['PN', 'PN', 'VC', 'VV', 'AS', 'VV', 'AD', 'VV'], ['NN', 'VA', 'VA', 'DEV', 'VV', 'AS', 'NN'], ['AD', 'PN', 'DT', 'JJ', 'DEG', 'NN', 'VV', 'VV'], ['NN', 'NN', 'NR', 'NN', 'VV', 'AS', 'VV', 'CC', 'JJ', 'NN', 'AD', 'BA', 'NN', 'VV', 'AD', 'VV', 'DEV'], ['NN', 'VV', 'PN', 'VV', 'AS', 'NN', 'AD', 'NN', 'VV', 'DEC', 'NN', 'P', 'NN', 'VV'], ['NN', 'AD', 'VV', 'PN', 'CS', 'VV', 'AS', 'NN', 'AD', 'NT', 'NN', 'AD', 'VV', 'PN'], ['JJ', 'NN', 'CD', 'CD', 'M', 'CD', 'CD', 'CD', 'CD', 'NN'], ['NN', 'AD', 'NR', 'VV', 'VV', 'NN', 'VV', 'AS', 'NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'NN', 'VC', 'NN', 'VV'], ['NR', 'NN', 'NN', 'LC', 'VV', 'NN', 'DEC', 'NN', 'P', 'NN', 'VV'], ['JJ', 'NN', 'AD', 'VV', 'PN', 'VV', 'VC', 'P', 'NN', 'VV', 'VV'], ['AD', 'NR', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN'], ['NN', 'VV', 'NN', 'DEC', 'VA', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['VV', 'AD', 'VV', 'VV', 'SP', 'AD', 'NN', 'AD', 'VV', 'AS', 'VV', 'PN', 'SP'], ['P', 'NN', 'NN', 'NR', 'AD', 'VA', 'AD', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'NN', 'NN', 'NN', 'LC', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VA'], ['NN', 'VV', 'BA', 'PN', 'DEG', 'NN', 'VV', 'NR', 'VA', 'DEC', 'NN'], ['P', 'NR', 'NN', 'NN', 'NN', 'VV', 'CD', 'NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'AD', 'VC', 'NN', 'JJ', 'PN', 'VV', 'NT', 'AD', 'AD', 'AD', 'VV', 'PN', 'VV', 'AS'], ['NR', 'NN', 'NR', 'LC', 'NN', 'VE', 'CD', 'M', 'VV', 'DEC', 'NN'], ['VV', 'PN', 'P', 'VV', 'LC', 'PN', 'NN', 'AD', 'VV', 'VV', 'VV', 'PN', 'SP'], ['AD', 'P', 'DT', 'AD', 'NR', 'CD', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'LC'], ['VV', 'NN', 'NN', 'AD', 'VV', 'NN'], ['VV', 'PN', 'SP', 'VV', 'VC', 'VV', 'NN', 'AD', 'AD', 'VV'], ['NR', 'NN', 'VV', 'DEC', 'VC', 'NN', 'NN', 'CC', 'NN', 'VV', 'P', 'PN', 'AD', 'VA'], ['PN', 'VV', 'PN', 'VC', 'NN', 'VV', 'VV', 'SP'], ['NR', 'AD', 'VV', 'AS', 'NR', 'NN'], ['NR', 'AD', 'VV', 'VV', 'CC', 'CD', 'NN', 'AD', 'PN', 'AD', 'AD', 'VV', 'NN'], ['AD', 'AD', 'VV', 'AD', 'P', 'NR', 'VV', 'AS', 'NN', 'NN', 'NT', 'AD', 'NN', 'NN'], ['CD', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['PN', 'AD', 'NN', 'NN', 'AD', 'VC', 'NN', 'NN', 'AD', 'VV', 'NR', 'NR'], ['NT', 'NN', 'P', 'NR', 'NN', 'VV', 'VV', 'NN'], ['NT', 'VV', 'AS', 'NN', 'PN', 'CD', 'NN', 'NN', 'VV', 'IJ'], ['VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'NN', 'NN', 'VV'], ['P', 'NN', 'PN', 'VV', 'VV', 'VV', 'CD', 'NN'], ['NR', 'NR', 'CC', 'NR', 'NN', 'AD', 'AD', 'VV', 'NR', 'DEG', 'NN', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'AD', 'VE', 'CD', 'JJ', 'JJ', 'DEG', 'NN'], ['DT', 'NN', 'AD', 'VV', 'VV', 'NR', 'NR', 'NN', 'NN', 'NN'], ['AD', 'NN', 'AD', 'JJ', 'NN'], ['NN', 'LC', 'PN', 'AD', 'P', 'NN', 'VV', 'AS', 'NN'], ['AD', 'VA', 'AD', 'PN', 'VV', 'VV', 'NN', 'VV', 'NN', 'VV', 'AS', 'LC', 'VV', 'VV', 'VV', 'CD', 'DEG', 'NN'], ['VE', 'NN', 'VV', 'DT', 'CC', 'P', 'VV', 'VV', 'DT', 'JJ', 'NN', 'VV', 'AS', 'NN'], ['NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'AS', 'DT', 'NN', 'NN', 'VV'], ['AD', 'AD', 'VV', 'VV', 'P', 'NN', 'VA', 'AD', 'VV', 'AD', 'VV', 'VV', 'SP'], ['CC', 'NT', 'CD', 'VV', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VC', 'P', 'NN', 'NN', 'VV', 'VV', 'SP'], ['NN', 'VV', 'NR', 'NR', 'NN', 'NN', 'NN', 'NN', 'AD', 'VC', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'PN', 'VC', 'VV', 'NN', 'NN'], ['AD', 'NR', 'NN', 'LC', 'NN', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NR', 'NN'], ['P', 'PN', 'AD', 'NN', 'VA', 'AD', 'SP'], ['AD', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['P', 'NR', 'DEG', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['NR', 'VV', 'AS', 'BA', 'NN', 'VV', 'AS', 'NN', 'VV', 'NN', 'LC'], ['PN', 'VV', 'AS', 'DT', 'VA', 'DEC', 'NN', 'NN', 'DEC', 'NN', 'DEV', 'VV', 'NN', 'NN', 'DEG', 'NN', 'SP', 'SP'], ['NR', 'SB', 'VV', 'CD', 'NT', 'NR', 'NN', 'NN', 'CD', 'M', 'NN', 'NN'], ['AD', 'AD', 'VV', 'AD', 'CD', 'NN', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'SP'], ['PN', 'AD', 'NR', 'VV', 'VA', 'CC', 'AD', 'VA', 'VA'], ['AD', 'NN', 'VV', 'CS', 'VV', 'CD', 'NN', 'AD', 'AD', 'NN'], ['PN', 'VV', 'NN', 'ETC', 'NN', 'NN', 'VV', 'DT', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'AS', 'AD'], ['NN', 'P', 'PN', 'VV', 'SP'], ['PN', 'VV', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'PN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NR', 'VE', 'VV'], ['NN', 'VV', 'AD', 'VV', 'NN'], ['P', 'NR', 'NN', 'ETC', 'NN', 'AD', 'VE', 'NN', 'VV', 'NN', 'VV', 'PN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'VV'], ['AD', 'VV', 'AD', 'AD', 'JJ', 'AD', 'BA', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS'], ['NR', 'AD', 'VV', 'AD', 'VV', 'VV', 'AS', 'NN', 'LC'], ['AD', 'NN', 'AD', 'NN', 'VV', 'AS', 'VV', 'JJ', 'JJ', 'NN', 'DEG', 'NN'], ['NR', 'P', 'NN', 'VV', 'NN', 'P', 'VV', 'NN', 'VV', 'CC', 'VV', 'P', 'PN', 'VV', 'VV'], ['PN', 'P', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NT', 'NR', 'AD', 'VV', 'DEC', 'NN', 'VC', 'NN', 'NT', 'DEG', 'NN', 'NR'], ['DT', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'VV', 'CC', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'DEC', 'M', 'NN', 'P', 'NN', 'AD', 'VV', 'AD', 'VV'], ['NT', 'PN', 'P', 'NN', 'NN', 'NN', 'ETC', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN'], ['AD', 'VV', 'AD', 'PN', 'DEG', 'NN', 'VV'], ['NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS', 'NN'], ['PN', 'VV', 'NN', 'LC', 'AD', 'AD', 'JJ', 'NN'], ['NN', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'AD', 'VA', 'NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'SP'], ['NR', 'VV', 'NR', 'NT', 'AD', 'VC', 'PN', 'DEG', 'VV', 'NN'], ['VV', 'NR', 'NN', 'DEC', 'VC', 'PN', 'DEG', 'NN'], ['AD', 'AD', 'AD', 'VV', 'VE', 'NN', 'DEC', 'NN', 'NN', 'NN', 'VC', 'NR', 'VV', 'VV'], ['AD', 'VV', 'NN', 'NN', 'P', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV'], ['VV', 'VV', 'JJ', 'NN', 'VV', 'NN', 'AD'], ['PN', 'DT', 'NN', 'AD', 'VV', 'NN', 'CC', 'JJ', 'NN', 'DEC', 'NN', 'AD', 'NN'], ['VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'CD', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NN'], ['VV', 'NR', 'CC', 'NT', 'NN', 'LC', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'M', 'NN'], ['VV', 'DEC', 'NN', 'VV', 'NN', 'NR'], ['NR', 'DEG', 'NN', 'AD', 'VC', 'VA'], ['DT', 'NN', 'AD', 'VC', 'NN', 'VV', 'NN'], ['PN', 'VC', 'AD', 'VV', 'PN', 'DEC', 'JJ', 'SP', 'IJ', 'DT', 'AD', 'M', 'NN'], ['PN', 'AD', 'P', 'PN', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'NR', 'NR', 'NN'], ['NR', 'NN', 'VC', 'NR', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['NT', 'NN', 'DEG', 'NN', 'AD', 'VA', 'DEC', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV'], ['VV', 'NR', 'SP', 'VV', 'VV', 'AS', 'AD', 'VV', 'NN', 'LC', 'VV', 'SP'], ['AD', 'P', 'NN', 'NN', 'ETC', 'NN', 'NN', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'AD', 'AD', 'AD', 'VA', 'NN', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV', 'CD', 'VV'], ['PN', 'AD', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'NR', 'CC', 'NR', 'AD', 'VV'], ['NN', 'P', 'NR', 'NN', 'NN', 'NN', 'VV', 'NN', 'CC', 'VV', 'NN'], ['P', 'NN', 'AD', 'VV', 'P', 'NN', 'LC', 'DEG', 'NR', 'NN', 'P', 'M', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'AD', 'VA'], ['VV', 'NN', 'AD', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['NR', 'P', 'NN', 'AD', 'VA'], ['VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN'], ['AD', 'AD', 'VV', 'SP', 'NN', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV', 'NN', 'SP'], ['JJ', 'VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'LC'], ['VA', 'DEC', 'NN', 'VV', 'AD', 'BA', 'NN', 'VV', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV'], ['NN', 'LC', 'VV', 'DEC', 'NR', 'AD', 'NN'], ['AD', 'NN', 'VA', 'AS', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'JJ', 'NN', 'AD', 'VA'], ['AD', 'VV', 'DEC', 'VC', 'DT', 'NN', 'DEG', 'NN', 'NR'], ['PN', 'VC', 'P', 'NN', 'LC', 'DEG', 'NN', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['NR', 'P', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VE', 'AD', 'VV'], ['JJ', 'NN', 'P', 'DT', 'NN', 'DEG', 'JJ', 'NN', 'SP'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'LC', 'VV', 'AD'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'VA'], ['NN', 'VV', 'PN', 'NN', 'CC', 'NN', 'VV', 'NN'], ['NT', 'NT', 'NN', 'VV', 'VV', 'LC', 'NR', 'NR', 'AD', 'VV', 'NN', 'VV', 'NN'], ['P', 'NN', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NR', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['NN', 'CD', 'VV', 'AD', 'VV', 'AD', 'VC', 'NT', 'DEG', 'NN', 'NN'], ['NN', 'NR', 'LC', 'VV', 'NN', 'LC', 'VV', 'AS', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'PN', 'AD', 'VC', 'VV', 'AD', 'VC', 'VV', 'NN'], ['AD', 'NR', 'DEG', 'NN', 'CC', 'NN', 'VV', 'AS', 'NN', 'VV', 'DEC', 'NN'], ['JJ', 'DEG', 'NN', 'VV', 'NR', 'NN', 'JJ', 'NN', 'CC', 'NR', 'NN', 'NN', 'VV', 'NN'], ['AD', 'NN', 'AD', 'AD', 'VV', 'VV'], ['JJ', 'NN', 'DEG', 'DT', 'NN', 'AD', 'VA', 'VA', 'DEC', 'NN', 'AD', 'NN', 'VV', 'VV'], ['AD', 'VA', 'NT', 'AD', 'VV', 'SP'], ['PN', 'CC', 'VV', 'PN', 'AD', 'VV', 'NN', 'SP'], ['VV', 'NR', 'NN', 'VV', 'VV', 'NN'], ['NN', 'AD', 'AD', 'P', 'NR', 'NR', 'CC', 'JJ', 'NN', 'NN', 'VC', 'M', 'JJ', 'JJ', 'NN', 'NN'], ['AD', 'NN', 'VV', 'DEC', 'CD', 'NN', 'NT', 'NT', 'NN', 'VC', 'NR', 'JJ', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NR', 'VV', 'DEC', 'NN'], ['AD', 'PN', 'AD', 'VV', 'AD', 'AD', 'VV', 'DT', 'VA', 'CC', 'VA', 'DEC', 'NN'], ['P', 'NT', 'DEG', 'CD', 'NN', 'LC', 'DT', 'NN', 'VE', 'VV', 'NN'], ['NN', 'VV', 'AS', 'JJ', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'CD', 'NN', 'SP'], ['AD', 'VV', 'DEC', 'NN', 'VV', 'PN', 'VV', 'DEC', 'VC', 'NN'], ['NN', 'VV', 'NR', 'DEG', 'NN', 'VV', 'AS'], ['VV', 'NR', 'DT', 'NN', 'DEG', 'NN', 'AD'], ['AD', 'NN', 'VV', 'NN', 'LC', 'AD', 'VE'], ['AD', 'NT', 'NT', 'DEG', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['NN', 'JJ', 'NN', 'VV', 'VV', 'JJ', 'AD', 'AD', 'VV', 'AD', 'VV'], ['NT', 'LC', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['AD', 'VC', 'P', 'NR', 'P', 'PN', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VV', 'AD', 'NN'], ['NR', 'NN', 'LC', 'AD', 'VV'], ['P', 'NN', 'VV', 'NN', 'NR', 'CC', 'NR', 'DEG', 'NN', 'VV'], ['NN', 'NR', 'NR', 'NN', 'CC', 'NN', 'NN', 'VV', 'NR', 'NN', 'NN', 'NR', 'NR', 'NN'], ['P', 'NR', 'VV', 'NN', 'NN', 'VV', 'NN', 'P', 'VA', 'DEC', 'VV', 'NN'], ['PN', 'VE', 'NN', 'VE', 'NN', 'AD', 'NN', 'AD'], ['NR', 'NN', 'NN', 'NR', 'CC', 'NR', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'LC', 'VE', 'CD', 'P', 'NN', 'DEG', 'NN'], ['P', 'NN', 'VV', 'VV', 'VV', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'VV', 'VV', 'NT', 'NT', 'CD', 'CD', 'CD'], ['P', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV'], ['AD', 'CD', 'VV', 'VV', 'NT', 'NR', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NR', 'VV', 'LC', 'AD', 'VV', 'VV'], ['VV', 'NN', 'AD', 'VV', 'NN'], ['DT', 'NN', 'NN', 'VV', 'AS', 'VE', 'NN', 'DEC', 'NN', 'CC', 'NN', 'NN'], ['AD', 'NR', 'SB', 'VV', 'NR', 'OD', 'NN', 'NN'], ['P', 'NN', 'NT', 'AD', 'VV', 'AS', 'AD', 'CD', 'M'], ['AD', 'VA', 'SP', 'AD', 'VV', 'AD', 'VV', 'DER', 'AD', 'DEV'], ['DT', 'NN', 'VV', 'AS', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'NR', 'VV', 'JJ', 'NN', 'VV', 'VA', 'VA'], ['AD', 'NN', 'P', 'NN', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'PN', 'AD', 'VV', 'CD', 'VV', 'AS', 'NR', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'SP', 'AD', 'AD', 'P', 'PN', 'VV', 'NN', 'SP'], ['NT', 'NN', 'AD', 'BA', 'NN', 'VV', 'VV', 'AD', 'VV', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'AD', 'SB', 'VV', 'NR', 'JJ', 'NN', 'NN'], ['AD', 'VV', 'VV', 'NN', 'VA', 'VA', 'SP'], ['NN', 'NN', 'VV', 'NR', 'VV', 'NN', 'NN', 'LC', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['NN', 'VV', 'NN', 'AD', 'VV', 'PN', 'P', 'NN', 'NN', 'NN', 'NR', 'NR', 'DEG', 'NN'], ['AD', 'VV', 'NR', 'AD', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['NR', 'NR', 'NN', 'AD', 'VV', 'NN', 'JJ', 'NN', 'NN'], ['P', 'NN', 'NN', 'VV', 'VV', 'NN', 'AS', 'LC', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'NN', 'NN', 'LC', 'NN', 'LC', 'VV', 'CD', 'NN'], ['NN', 'NN', 'P', 'NN', 'NN', 'LC', 'NN', 'VA', 'VV', 'JJ', 'NN', 'NN', 'DEG', 'AD', 'VV'], ['NN', 'VV', 'AD', 'VV', 'AS', 'VV', 'VV', 'VV', 'AS', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV'], ['AD', 'VC', 'CD', 'NN', 'AD', 'VV', 'NN', 'MSP', 'VV', 'NN', 'NN', 'NN'], ['NR', 'NN', 'AD', 'VV', 'JJ', 'NN', 'AD', 'VV', 'DEC', 'VV', 'VV'], ['NR', 'VV', 'DER', 'VV', 'VV', 'VV', 'LB', 'NN', 'VV', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV'], ['PN', 'VV', 'NN', 'VV', 'NN', 'NN', 'MSP', 'VV', 'VV', 'NN', 'AD', 'AD', 'VV', 'PN', 'VV'], ['NR', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'P', 'NN', 'VV', 'NN'], ['P', 'NN', 'NN', 'NT', 'DEG', 'NN', 'LC', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'AD', 'VA'], ['JJ', 'NN', 'CD', 'NT', 'NR', 'VV', 'AS', 'NN', 'NN'], ['NR', 'AD', 'VV', 'VA', 'VC', 'AD', 'VV', 'DEC'], ['DT', 'NN', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NN', 'PN', 'AD', 'VV', 'NR', 'NN'], ['NR', 'VV', 'NN', 'VV', 'NN'], ['DT', 'NN', 'P', 'VV', 'NN', 'NN', 'LC', 'VV', 'NN'], ['CD', 'CD', 'M', 'AD', 'VV', 'DEC', 'NN', 'NR', 'AD', 'P', 'NN', 'VV'], ['VV', 'LC', 'VE', 'NN', 'LC', 'VE', 'NN', 'VV'], ['PN', 'VV', 'PN', 'VV', 'SP', 'NN', 'AD', 'VA', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NR', 'NN', 'DEG', 'AD', 'VV'], ['VE', 'NN', 'VV', 'DT', 'NN', 'VC', 'NR', 'JJ', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'SP'], ['PN', 'AD', 'VV', 'NN', 'LC', 'AD', 'VA', 'DEC', 'NN', 'SP'], ['VC', 'M', 'NN', 'SP'], ['NN', 'AD', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'NN'], ['PN', 'CC', 'AD', 'VV', 'NN'], ['PN', 'NN', 'AD', 'NN', 'AD', 'NN', 'AD', 'P', 'NN', 'VA', 'AD'], ['NT', 'VV', 'NN', 'VV', 'NR', 'VC', 'NN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'AD', 'VC', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'NN', 'P', 'NN', 'VV', 'VC', 'NN', 'NN', 'AD', 'VV'], ['PN', 'VV', 'DT', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'VA'], ['PN', 'VV', 'AD', 'VA', 'DEV', 'VV', 'AS', 'AD', 'VV', 'VV', 'VV', 'NN'], ['JJ', 'NN', 'DEG', 'NN', 'VV', 'PN', 'VA'], ['AD', 'AD', 'VV', 'DT', 'NN', 'NN'], ['NN', 'VA', 'AD', 'VV', 'NN', 'ETC'], ['PN', 'VV', 'NR', 'VV', 'DER', 'AD', 'VA'], ['AD', 'NN', 'NN', 'NN', 'ETC', 'NN'], ['P', 'NR', 'DT', 'NN', 'AD', 'VA'], ['NN', 'LC', 'DEG', 'NN', 'NN', 'AD', 'AD', 'AD', 'P', 'NR', 'VV', 'VV', 'NN'], ['NT', 'DEG', 'NN', 'CC', 'SB', 'NN', 'AD', 'VV', 'AD', 'SB', 'VV'], ['AD', 'CS', 'NN', 'NR', 'VV', 'AS', 'NN', 'VV', 'VV', 'P', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'JJ', 'NN', 'DEG', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['NN', 'AD', 'P', 'NN', 'VV', 'NN'], ['AD', 'P', 'NT', 'NT', 'VV', 'DEC', 'NR', 'NN', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'NN'], ['NR', 'VV', 'AS', 'NR', 'DEG', 'NN', 'NN'], ['AD', 'NR', 'CC', 'NR', 'AD', 'VV', 'NN'], ['VV', 'LC', 'NN', 'VV', 'AD', 'LB', 'NN', 'VV', 'VV'], ['NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['P', 'NN', 'NN', 'NN', 'AD', 'VE', 'CD', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'VV', 'NN'], ['AD', 'NR', 'NR', 'DEG', 'NN', 'NN', 'VC', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['VA', 'DEC', 'NN', 'NN', 'AD', 'VA', 'CD', 'SP'], ['NN', 'VV', 'NR', 'CC', 'NN', 'VV', 'NN'], ['NN', 'AD', 'NN', 'VV', 'NN', 'CD', 'VV', 'NR', 'VV', 'NN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'IJ', 'BA', 'PN', 'DEG', 'NN', 'VV', 'AD'], ['VA', 'DEC', 'VC', 'NN', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'JJ', 'P', 'PN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['VE', 'DT', 'VA', 'DEC', 'VV', 'NN', 'NN', 'SP'], ['AD', 'VV', 'NN', 'LC', 'VV', 'DER', 'AD', 'VE', 'NN', 'DT', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN'], ['DT', 'NN', 'P', 'NR', 'VV', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'NN', 'AD', 'VA'], ['AD', 'NR', 'DEG', 'NN', 'VV', 'NN'], ['NT', 'NN', 'AD', 'VE', 'NN', 'NN', 'VV'], ['VV', 'NR', 'NN', 'DEC', 'VV', 'VV', 'SB', 'VV', 'VA', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VV'], ['P', 'NN', 'NN', 'VV', 'LC', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'CC', 'VV', 'NN'], ['P', 'DT', 'NN', 'VV', 'NN', 'PN', 'AD', 'AD', 'VA'], ['NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['VV', 'NN', 'DEC', 'JJ', 'NN', 'P', 'NN', 'LC', 'AD', 'VV'], ['NN', 'VV', 'DER', 'VV', 'JJ', 'NN', 'NN', 'VV', 'VV'], ['VA', 'VA', 'VA', 'VV', 'NN', 'NN', 'VV'], ['P', 'NR', 'VV', 'DEC', 'NN', 'DT', 'NR', 'NN', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['NR', 'DEG', 'NN', 'NN', 'NN', 'P', 'VV', 'NN', 'VV'], ['PN', 'VV', 'JJ', 'PN', 'VV', 'JJ', 'NR', 'VV', 'NN', 'NN', 'P', 'NR', 'NN', 'NN'], ['AD', 'NT', 'NT', 'LC', 'DEG', 'NN', 'NT', 'AD', 'VV', 'NN'], ['PN', 'NT', 'VV', 'AD', 'VV', 'VV', 'PN', 'VV', 'AD', 'PN', 'VV', 'AD', 'NN', 'VV', 'AD', 'VA'], ['NN', 'VA', 'AD', 'AD', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VC', 'VV', 'VV', 'NN', 'NN', 'SP'], ['PN', 'VE', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VA'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VV', 'SP', 'AD', 'VC', 'NN'], ['P', 'JJ', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'P', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['NT', 'LC', 'NN', 'DEG', 'NN', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'P', 'CD', 'JJ', 'NN', 'NN', 'NN', 'VV'], ['NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'AD'], ['AD', 'PN', 'AD', 'VV', 'AD', 'VV', 'VV', 'PN', 'SP'], ['CS', 'PN', 'VV', 'PN', 'VV', 'AS', 'NN', 'AD', 'VV', 'PN'], ['NT', 'PN', 'VV', 'CD', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'NN'], ['AD', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'P', 'NR', 'VA', 'CD'], ['AD', 'VV', 'PN', 'VV', 'SP'], ['PN', 'VV', 'VA', 'VA', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VV', 'CD'], ['AD', 'AD', 'PN', 'VV', 'NN', 'AD', 'P', 'DT', 'M', 'JJ', 'NN', 'VV'], ['VV', 'NN', 'NN', 'NN', 'NN'], ['NR', 'NR', 'VV', 'PN', 'AD', 'VC', 'NN'], ['AD', 'AD', 'NN', 'VV', 'AS', 'CD', 'NN', 'NN', 'VV', 'VV', 'AS', 'NN'], ['NR', 'VV', 'NR', 'CC', 'M', 'P', 'NN', 'NN', 'VE', 'AD', 'VV', 'DEC', 'NN'], ['P', 'NN', 'NN', 'LC', 'VV', 'NN', 'VV', 'NN', 'SB', 'NR', 'VV'], ['AD', 'NN', 'VA', 'DEV', 'VV', 'AS', 'VV', 'M', 'NN', 'AD', 'VV', 'VV', 'NR', 'NN'], ['VV', 'NT', 'NN', 'VV', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'CC', 'NN'], ['VV', 'LC', 'DEG', 'NN', 'AD', 'P', 'NR', 'NN', 'CC', 'NR', 'DEG', 'NN', 'NN', 'VV'], ['PN', 'VV', 'NR', 'NN', 'NN', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VC', 'PN', 'NN', 'SP'], ['NN', 'LC', 'AD', 'VV', 'DEC', 'NR', 'NR', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'VE', 'NN', 'SP'], ['NT', 'NN', 'AD', 'VV', 'SP', 'PN', 'DEG', 'NN'], ['NN', 'NN', 'AD', 'VV', 'JJ', 'JJ', 'NN', 'AD', 'NT', 'AD', 'VV'], ['PN', 'VV', 'AS', 'AD', 'VV', 'P', 'NN', 'PN', 'DEG', 'NN', 'PN', 'AD', 'VV'], ['DT', 'NN', 'NN', 'NN', 'VV', 'DER', 'P', 'M', 'NN', 'NN'], ['NN', 'AD', 'VA', 'VV', 'NN', 'DEC', 'NN', 'AD', 'NN', 'VA'], ['AD', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VC', 'NR', 'JJ', 'NN', 'NN', 'NN'], ['AD', 'AD', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['PN', 'P', 'VE', 'NN', 'NN', 'CD', 'NN', 'CD', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['AD', 'JJ', 'NN', 'VV', 'JJ', 'DEG', 'NN'], ['NR', 'CC', 'NN', 'AD', 'VV', 'NR', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'VC', 'NN', 'VV', 'CD'], ['NR', 'JJ', 'M', 'AD', 'VA', 'VA'], ['NN', 'CC', 'NN', 'NN', 'VA', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN', 'CC', 'DT', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'VC', 'AD', 'VV', 'DEC', 'NN', 'SP'], ['NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['DT', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'AS', 'VV', 'NR', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'VA', 'NN', 'VV', 'NN'], ['NR', 'BA', 'NN', 'VV', 'VV', 'VV', 'NN', 'DT', 'M', 'NN', 'AD', 'VA', 'IJ', 'AD', 'VV', 'VV'], ['VV', 'VV', 'P', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['P', 'DT', 'NN', 'JJ', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'NN'], ['AD', 'AD', 'DEV', 'NR', 'NN', 'NR', 'AD', 'VV', 'SP'], ['NN', 'VV', 'VV', 'AS', 'VV', 'VV', 'DEC', 'NN', 'VV', 'SP'], ['AD', 'VC', 'PN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'CD', 'NN', 'AD', 'NN', 'AD', 'VV', 'VV', 'SP'], ['NN', 'VA', 'DEC', 'PN', 'AD', 'VE', 'NN', 'NN', 'NN', 'VV', 'NN'], ['PN', 'AD', 'AD', 'VV', 'AS', 'AD', 'VV'], ['LB', 'PN', 'MSP', 'AD', 'VV', 'DEC', 'VV', 'AS', 'NN', 'AD', 'LB', 'PN', 'MSP', 'AD', 'VV', 'CC', 'VV', 'DEC', 'NN', 'VV', 'AS', 'NN'], ['AD', 'NN', 'NR', 'NN', 'NN', 'NN', 'P', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['AD', 'VE', 'NN', 'PN', 'NN'], ['AD', 'AD', 'VV', 'VV', 'VV', 'SP'], ['P', 'NN', 'DEG', 'JJ', 'PN', 'AD', 'VV', 'NN', 'VV'], ['NT', 'NN', 'P', 'VV', 'NN', 'SB', 'VV', 'NN', 'VV'], ['DT', 'NN', 'AD', 'P', 'PN', 'AD', 'VV', 'DEC', 'NN', 'SP'], ['AD', 'P', 'PN', 'VV', 'DEC', 'NN', 'PN', 'NN', 'AD', 'DEV', 'VV', 'PN', 'PN', 'AD', 'VV', 'VV', 'PN', 'SP'], ['CS', 'NN', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN', 'AD', 'NN', 'VV'], ['NN', 'LC', 'VV', 'VV', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'NR', 'VV', 'DT', 'CD', 'NN'], ['AD', 'AD', 'AD', 'NR', 'AD', 'AD', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['NR', 'VV', 'VV', 'CD', 'M', 'VV', 'VV', 'VE', 'DT', 'JJ', 'NN', 'VV'], ['NR', 'NR', 'P', 'NR', 'NN', 'NN'], ['PN', 'VV', 'PN', 'VV', 'DT', 'JJ', 'NN', 'CS', 'VV', 'MSP', 'AD', 'VV', 'VE', 'NN', 'VV', 'VE', 'NN', 'VV'], ['NR', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'VE', 'NN', 'NN'], ['NR', 'AD', 'VV', 'PN', 'AD', 'VA'], ['NN', 'VE', 'VV', 'CD', 'NN', 'CD', 'NN'], ['NR', 'PN', 'AD', 'VV', 'NR'], ['NT', 'AD', 'VV', 'NN', 'NN'], ['NN', 'VV', 'AD', 'AD', 'DT', 'NN', 'AD', 'VC', 'JJ', 'NN', 'AD', 'VV'], ['AD', 'VA', 'VV', 'PN', 'NN', 'AD', 'VV', 'AS'], ['PN', 'NT', 'P', 'DT', 'NN', 'LC', 'VV'], ['NN', 'NN', 'VC', 'VV', 'VV', 'NR', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'PN', 'VV', 'P', 'NN', 'VV'], ['AD', 'DT', 'NN', 'AD', 'VV'], ['VV', 'P', 'NN', 'NN', 'ETC', 'JJ'], ['PN', 'VV', 'DEC', 'JJ', 'NN', 'CC', 'NN', 'NN'], ['PN', 'VC', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['NR', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'VV', 'JJ', 'DEG', 'NN'], ['PN', 'VV', 'VV', 'AS', 'AD', 'NN', 'VV', 'AD', 'VA', 'AD', 'NN'], ['NN', 'LC', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NR', 'NR'], ['NN', 'CC', 'NN', 'P', 'PN', 'PN', 'DEG', 'NN', 'NN', 'CC', 'NN', 'VV', 'VV', 'PN', 'AD', 'VV', 'VV'], ['CC', 'DT', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'CD', 'M', 'CD', 'M', 'DEG', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['P', 'NN', 'DEG', 'JJ', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NN', 'VV', 'PN'], ['NT', 'NN', 'AD', 'VV', 'VV'], ['PN', 'PN', 'VV', 'PN', 'VV', 'SP', 'IJ'], ['NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'P', 'CD', 'NN', 'LC'], ['VV', 'NN', 'VV', 'NN', 'NN'], ['P', 'AD', 'NR', 'NN', 'VV', 'AD', 'CD', 'NN', 'VV', 'CD', 'M'], ['LB', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN'], ['VV', 'VV', 'AS', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN'], ['CD', 'NN', 'VV', 'NN', 'VC', 'VV', 'NN', 'DEG', 'JJ', 'NN'], ['AD', 'NN', 'AD', 'VV', 'NN', 'AD', 'VE', 'NN', 'DEC', 'NN'], ['NN', 'P', 'NN', 'VV', 'NN'], ['NR', 'NR', 'NN'], ['NN', 'NN', 'VV', 'PN', 'VC', 'P', 'VV', 'VV', 'NN'], ['PN', 'VV', 'PN', 'AD', 'VA', 'SP'], ['PN', 'VV', 'NN'], ['VV', 'PN', 'VV', 'PN', 'SP'], ['NT', 'AD', 'AD', 'VV'], ['NN', 'NN', 'NN', 'VV', 'P', 'PN'], ['AD', 'AD', 'VV'], ['PN', 'NT', 'VV', 'NN'], ['P', 'NN', 'NN', 'LC', 'PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'SP'], ['AD', 'PN', 'VV', 'VV', 'NR', 'NN', 'AD', 'VV', 'DEC', 'NN', 'NN'], ['VV', 'AD', 'LC', 'NR', 'VV', 'VV', 'NN', 'P', 'DT', 'NN', 'NR', 'AD', 'VV', 'VV'], ['NN', 'LC', 'JJ', 'NN', 'NN', 'NN', 'AD', 'NN', 'P', 'AD', 'VA', 'DEC', 'NN', 'LC', 'VV'], ['AD', 'AD', 'VE', 'NN', 'NN', 'VA', 'DEV', 'P', 'NN', 'NN', 'VV', 'VV', 'NN'], ['NT', 'PN', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'AS'], ['NN', 'NN', 'VV', 'VV', 'AD', 'VC', 'VV', 'NN', 'SB', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'VE', 'AD', 'PN', 'VV', 'VV', 'NN', 'VV', 'PN', 'VV', 'VC', 'JJ', 'DEG', 'SP'], ['AD', 'VV', 'AS', 'AD', 'VV', 'VC', 'JJ', 'NN', 'MSP', 'VV', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'VV', 'AS', 'CD', 'M', 'NN', 'NN', 'NN', 'P', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'VE', 'SP', 'VV', 'PN', 'DEC', 'NN', 'VV', 'AS', 'PN', 'P', 'VV', 'SP'], ['AD', 'PN', 'AD', 'AD', 'VV', 'DEV', 'VV', 'PN', 'SP'], ['NT', 'VV', 'AS', 'NN', 'NN', 'NN', 'LC', 'NN', 'AD', 'VV', 'VV'], ['VV', 'AS', 'CD', 'NN', 'VV', 'DER', 'AD', 'CD', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'AS', 'AD', 'VV'], ['JJ', 'JJ', 'NN', 'AD', 'NN', 'P', 'NR', 'DEG', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NT', 'NN', 'NN', 'VA'], ['AD', 'VV', 'NN', 'NN'], ['AD', 'NR', 'P', 'NR', 'NN', 'NN', 'NT', 'LC', 'DEG', 'VV', 'CC', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['VE', 'NN', 'VV', 'VV', 'VV'], ['NR', 'NN', 'CD', 'M', 'VV', 'VV', 'AS', 'NN', 'PN', 'NN'], ['VA', 'DEC', 'NN', 'AD', 'NN', 'AD', 'VV', 'AS', 'NN'], ['P', 'NN', 'CC', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['NR', 'VV', 'PN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VV'], ['P', 'AD', 'VV', 'LC', 'NN', 'NN', 'LC', 'DEG', 'NN', 'CC', 'NN', 'AD', 'P', 'NN', 'VV', 'AS', 'M', 'NN'], ['AD', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'LC', 'PN', 'VV', 'P', 'NN', 'VV'], ['NN', 'DEG', 'NN', 'VC', 'AD', 'VA', 'SP'], ['VV', 'NN', 'VV', 'VV', 'VV', 'CC', 'VV', 'NN', 'DEG', 'NN', 'NN'], ['NT', 'P', 'NR', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['PN', 'NT', 'NN', 'AD', 'VV', 'PN', 'CS', 'VE', 'DT', 'NN', 'VV', 'PN', 'VV'], ['CS', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'VV', 'PN', 'NN'], ['NR', 'VV', 'NN', 'NN', 'AD', 'VA', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV'], ['NR', 'NN', 'VV', 'NT', 'DT', 'NN', 'AD', 'VV', 'JJ', 'JJ', 'NN', 'CC', 'VA'], ['AD', 'VA', 'VC', 'PN', 'AD', 'AD', 'AD', 'VV', 'DEC', 'JJ', 'NT', 'VV', 'AD', 'VA', 'AD', 'VV', 'VV'], ['PN', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'BA', 'NN', 'VV', 'VV'], ['VA', 'AD', 'VV', 'VV', 'NN', 'AD', 'VC', 'AD', 'VA', 'DEC'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'VV', 'VV', 'DEC', 'VV', 'VV', 'VV', 'DEC', 'VV', 'VA'], ['NR', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN'], ['NT', 'NR', 'AD', 'VV', 'NN'], ['NN', 'AD', 'AD', 'AD', 'VV', 'PN', 'VV', 'NN', 'NN', 'DEG', 'VV', 'AS', 'NN', 'VV', 'NN', 'VA'], ['PN', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NR', 'NN', 'NN', 'NN', 'VV'], ['CS', 'NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'CD'], ['AD', 'NN', 'VV', 'NR', 'DEG', 'NN', 'CC', 'VV', 'NN'], ['P', 'NT', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VE', 'CD', 'NN'], ['NN', 'CC', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VA', 'VA', 'NN', 'AD', 'VA'], ['NR', 'CC', 'NN', 'VV', 'VV', 'BA', 'NN', 'VV'], ['AD', 'AD', 'VV', 'P', 'PN', 'VV', 'M', 'PN', 'VV', 'IJ', 'VE', 'NN', 'PN', 'AD', 'VV'], ['NN', 'NN', 'VV', 'P', 'NN', 'DEG', 'NN', 'CC', 'NN', 'NN', 'VV', 'NN'], ['AD', 'AD', 'NN', 'NN', 'NN', 'AD', 'VA', 'NN', 'AD', 'VA'], ['AD', 'NN', 'VE', 'AS', 'NN', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['CS', 'AD', 'VV', 'VV', 'M', 'JJ', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN', 'AD', 'VC', 'NR'], ['PN', 'AD', 'VV', 'PN', 'AD', 'VA', 'SP', 'AD', 'PN', 'VC', 'AD', 'VA', 'AD', 'VA', 'DEC', 'NN', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'VV', 'JJ', 'NN', 'AD', 'VA', 'AD', 'VA'], ['NN', 'AD', 'VV', 'NR', 'DEG', 'NR', 'NN', 'AD', 'VA'], ['NR', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'NN', 'NR', 'VA', 'VV', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'VV', 'CD', 'CD'], ['NN', 'AD', 'AD', 'VV', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN', 'M', 'NN', 'NN'], ['AD', 'BA', 'VV', 'NN', 'DEC', 'NN', 'VV', 'P', 'VV', 'NN', 'NN'], ['AD', 'SP', 'PN', 'VC', 'PN', 'DEG', 'NN', 'SP'], ['NN', 'VE', 'NN', 'SP'], ['NN', 'AD', 'VA', 'SP'], ['NN', 'NN', 'NN', 'NN', 'ETC', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NR', 'AD', 'VV', 'VV', 'NR', 'P', 'NR', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['CD', 'NN', 'LC', 'DT', 'VE', 'NT', 'NN', 'DEG', 'NN', 'VV', 'AS'], ['P', 'NR', 'CS', 'NN', 'PN', 'VE', 'NN', 'DT', 'NN', 'AD', 'VV', 'VA'], ['NN', 'DEG', 'NN', 'VV', 'NN', 'DEC', 'NN', 'SB', 'VV', 'CD'], ['VV', 'NN', 'PN', 'AD', 'VV', 'NN', 'VV', 'AD', 'NN', 'VV', 'DER', 'AD', 'VV'], ['VV', 'NN', 'NN', 'NN', 'AD', 'VE', 'P', 'NN', 'VV', 'DEC', 'NN', 'CC', 'VA'], ['NN', 'VV', 'AS', 'AD', 'BA', 'NN', 'NN', 'VV'], ['AD', 'NN', 'NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'NR', 'NN'], ['AD', 'NN', 'NN', 'JJ', 'NN', 'VV', 'VC', 'VV', 'JJ', 'IJ'], ['AD', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV', 'CC', 'JJ', 'VV', 'NN'], ['NR', 'NN', 'NT', 'VV', 'VV', 'NN'], ['VE', 'DEC', 'NN', 'AD', 'VV', 'VE', 'DEC', 'NN', 'CC', 'NN', 'AD', 'AD', 'VA'], ['NR', 'AD', 'VA', 'AD', 'P', 'NN', 'NN', 'CC', 'BA', 'NN', 'VV', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'CD', 'CD', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['NR', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN', 'LC', 'P', 'NN', 'NN', 'NN', 'VV', 'VV'], ['NR', 'NN', 'AD', 'JJ', 'NN', 'VV', 'VV', 'DEC', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'AD'], ['NN', 'NN', 'NN', 'VV', 'DEC', 'AD', 'JJ', 'NN'], ['P', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VV'], ['NT', 'NN', 'VV', 'PN', 'NN', 'VV', 'AS', 'VE', 'CD', 'NN', 'PN', 'VV', 'AD', 'VV', 'CD', 'SP'], ['NN', 'P', 'NN', 'LC', 'VC', 'AD', 'SB', 'VV', 'DEC', 'AD', 'VC', 'NN', 'NN', 'NN', 'DEC', 'AD', 'NN', 'VA', 'SP'], ['NT', 'VV', 'VA', 'AD'], ['NN', 'VV', 'NN', 'VA', 'PN', 'AD', 'P', 'AD', 'VV', 'VA', 'DEC', 'NN', 'VV', 'AS'], ['AD', 'DT', 'NN', 'AD', 'VC', 'NN', 'SP'], ['NN', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'CD', 'CD', 'CD', 'NN', 'JJ'], ['PN', 'AD', 'AD', 'VA', 'DEV', 'VV', 'PN', 'VV', 'AD', 'VV', 'DT', 'VV', 'DT', 'JJ', 'NN'], ['P', 'NN', 'NR', 'VV', 'NN', 'VV', 'JJ', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NN', 'VV', 'LC', 'VV', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NT', 'NN', 'AD', 'NN', 'AD', 'AD', 'AD', 'CD', 'CD'], ['VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'VV'], ['AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VC', 'VV', 'VV', 'DEC', 'NN', 'JJ', 'NN'], ['NR', 'VV', 'NR', 'NN', 'VC', 'NR', 'JJ', 'DEG', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'NN'], ['NR', 'P', 'NR', 'NN', 'LC', 'DEG', 'NN', 'VE', 'NN', 'AD', 'VV'], ['NR', 'VV', 'CD', 'NN', 'VV', 'AS', 'DT', 'NN', 'DEG', 'NN', 'LC', 'VV', 'AS', 'AD', 'VV', 'PN'], ['CD', 'NN', 'VV', 'BA', 'NN', 'VV', 'NN', 'VV', 'NT', 'VV', 'NT'], ['PN', 'VV', 'VV', 'NN', 'NN', 'JJ', 'DEG', 'NN', 'LB', 'JJ', 'NN', 'VV', 'NN'], ['NR', 'NR', 'JJ', 'DEG', 'NN', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VV', 'NR', 'AD', 'VV', 'P', 'PN', 'NN', 'VV', 'PN', 'VV', 'PN', 'VV'], ['AD', 'NN', 'CD', 'NN', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['NN', 'LC', 'VV', 'DEC', 'VC', 'NN', 'NN', 'CC', 'JJ', 'NN'], ['NR', 'NN', 'NN', 'LC', 'NN', 'VV', 'NN'], ['AD', 'VE', 'CD', 'NN', 'VV', 'VV', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VV', 'PN', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NN', 'NR', 'AD', 'VV', 'NN', 'VV', 'NR', 'VV', 'AS', 'NN', 'VV', 'AS'], ['AD', 'VV', 'PN', 'NN', 'AD', 'VV', 'VA', 'DEC'], ['AD', 'P', 'NN', 'VV', 'AS', 'NT', 'CD', 'CD', 'CD', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN'], ['PN', 'AD', 'AD', 'AD', 'VV', 'DEC', 'NN', 'VV', 'VV', 'VA', 'SP'], ['P', 'NN', 'NN', 'VA', 'DEC', 'NN', 'AD', 'NT', 'NN', 'LC', 'AD', 'AD', 'VA'], ['NN', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VA', 'AS', 'NN', 'CC', 'JJ', 'DEG', 'AD', 'CD', 'VA', 'SP'], ['VV', 'AS', 'NN', 'LC', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'LC', 'DEG', 'NN', 'VV', 'VV', 'NN', 'NN'], ['PN', 'AD', 'AD', 'PN', 'AD', 'P', 'NN', 'NN', 'VV', 'NN', 'SP'], ['NN', 'VV', 'VV', 'VV', 'VV', 'VC', 'VA', 'DEC', 'NN'], ['NR', 'NN', 'VV', 'AS', 'JJ', 'NN', 'VV', 'AD', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'SB', 'VV', 'AS', 'CD', 'NN'], ['NT', 'NN', 'DT', 'NN', 'NN', 'AD', 'VV', 'SP', 'PN', 'VV', 'DER', 'VA'], ['NR', 'NN', 'NR', 'DT', 'NN', 'AD', 'VA'], ['AD', 'PN', 'AD', 'VV', 'DEC', 'NN', 'AD', 'AD', 'AD', 'VV', 'PN', 'AD'], ['P', 'JJ', 'NN', 'VV', 'NN', 'VV', 'NR'], ['VV', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VV', 'JJ', 'NN', 'VV', 'CD', 'NN', 'NN'], ['NN', 'NR', 'AD', 'P', 'NN', 'NN', 'P', 'NR', 'AD', 'VA', 'DEC', 'NN'], ['NR', 'VV', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'AS'], ['VA', 'DEV', 'VV', 'VV', 'NN', 'LC', 'PN', 'AD', 'VV', 'NN', 'LC', 'VV'], ['VV', 'PN', 'VV', 'VV', 'PN', 'DEG', 'AD', 'VC', 'AD', 'VV', 'DEC'], ['NR', 'AD', 'P', 'NN', 'NN', 'P', 'NR', 'NN', 'VV'], ['P', 'VV', 'VA', 'VA', 'AD', 'AD', 'VV'], ['PN', 'AD', 'NN', 'PN', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'PN', 'PN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NN', 'VV', 'AS', 'VV', 'NN', 'AD', 'VV'], ['AD', 'CS', 'AD', 'VV', 'AD', 'AD', 'VV', 'AD', 'NN', 'CC', 'NN', 'SP'], ['NN', 'AD', 'P', 'NR', 'NN', 'LC', 'NN', 'NN', 'CD', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'AD', 'AD', 'VV', 'PN', 'AD', 'AD', 'VA', 'DEC', 'AD', 'VE', 'CD', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'NR', 'CC', 'NR', 'P', 'NR', 'LC', 'VV'], ['AD', 'OD', 'VV', 'NN', 'NN', 'VV', 'PN', 'VV', 'NN', 'DEC', 'NR', 'NN'], ['NR', 'NN', 'VV', 'NN', 'JJ', 'NN', 'VA'], ['PN', 'NN', 'VE', 'NN', 'PN', 'AD', 'P', 'PN', 'VV'], ['NR', 'NN', 'NR', 'DEG', 'NN', 'AD', 'VV', 'NR', 'VC', 'NN', 'VV', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'JJ', 'NN', 'AD', 'P', 'NN', 'NN', 'VV', 'VA', 'NN'], ['PN', 'VV', 'VV', 'AS', 'AD', 'VV', 'M', 'NN', 'VV', 'NN', 'NN', 'VV', 'AD', 'VV', 'AS'], ['VV', 'PN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VE', 'CD', 'NN', 'VV', 'NN'], ['NR', 'NN', 'VV', 'VV', 'P', 'PN', 'DEG', 'AD', 'JJ', 'NN', 'NN'], ['P', 'NN', 'VV', 'PN', 'VV', 'LC', 'VV', 'NN', 'VV', 'NN', 'CC', 'NN'], ['NR', 'DEG', 'NN', 'NR', 'AD', 'VV', 'NR', 'VV', 'NN', 'VA', 'AS', 'AD', 'VA', 'DEC', 'NN'], ['DT', 'NN', 'AD', 'AD', 'VA', 'SP'], ['AD', 'NN', 'LC', 'JJ', 'JJ', 'DEG', 'NN', 'P', 'NN', 'AD', 'VA', 'VA', 'DEC', 'NN'], ['CD', 'VV', 'NR', 'NN', 'AD', 'VV', 'P', 'NR', 'VV'], ['VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN'], ['P', 'NN', 'NN', 'NR', 'VV', 'NN', 'AD', 'VA'], ['NN', 'VV', 'NN', 'P', 'NN', 'LC', 'VV', 'CD', 'AD'], ['P', 'NN', 'VA', 'DEC', 'DT', 'CD', 'NN', 'AD', 'VV'], ['P', 'PN', 'DEG', 'AD', 'VV', 'LC', 'AD', 'VA', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV'], ['P', 'NR', 'NN', 'NN', 'LC', 'VV', 'CC', 'NR', 'VA', 'CC', 'VV', 'DEC', 'NN'], ['PN', 'DEG', 'JJ', 'NN', 'AD', 'VV', 'P', 'PN', 'DT', 'JJ', 'NN', 'DEC', 'NN', 'VV', 'CD', 'SP'], ['NR', 'PN', 'VV', 'VV', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'NN', 'VV'], ['NR', 'VV', 'AD', 'P', 'NR', 'NN', 'AD', 'VV', 'AD', 'NN', 'NN'], ['NR', 'VV', 'NN', 'LC', 'VV', 'PN', 'DEG', 'NN', 'VV', 'AS', 'VV'], ['DT', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NT', 'AD', 'VV', 'VV'], ['P', 'NN', 'DEG', 'NN', 'LC', 'NN', 'NN', 'JJ', 'NN', 'BA', 'PN', 'NN', 'VV'], ['AD', 'NR', 'P', 'VV', 'NN', 'VV', 'P', 'NN', 'LC'], ['NT', 'NN', 'VA', 'DEC', 'NR', 'NN', 'VV'], ['NT', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'P', 'NN', 'VV', 'AS', 'JJ', 'NN'], ['NN', 'VV', 'LC', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['NT', 'NR', 'VV', 'AS', 'CD', 'NR', 'NN'], ['CD', 'VV', 'PN', 'VV', 'NN', 'VE', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'VV', 'DEC', 'NN', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VA'], ['NT', 'JJ', 'NR', 'NN', 'NR', 'NR', 'AD', 'VV', 'JJ', 'NN'], ['AD', 'AD', 'CD', 'P', 'NN', 'NN', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['P', 'NR', 'DEG', 'NN', 'NR', 'P', 'NN', 'LC', 'AD', 'VC', 'M', 'JJ', 'NN'], ['VV', 'NT', 'NN', 'NN', 'LC', 'NR', 'NR', 'NR', 'ETC', 'NN', 'NN', 'NN', 'VA'], ['NN', 'DEG', 'NN', 'NN', 'SP'], ['PN', 'VV', 'VV', 'NN', 'VV', 'LC', 'AD', 'AD', 'VA'], ['AD', 'NR', 'NN', 'DEG', 'CD', 'NN', 'AD', 'VV', 'NR'], ['VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'AD', 'NN', 'JJ'], ['NN', 'NR', 'VV', 'NN', 'LC', 'VV', 'DEC', 'NN', 'NN', 'AD', 'NN'], ['NN', 'P', 'NN', 'LC', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['P', 'NN', 'VV', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN', 'NR'], ['AD', 'VE', 'NN', 'AD', 'AD', 'AD', 'VA'], ['NN', 'LC', 'NN', 'AD', 'LB', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NR', 'VV', 'NR', 'CC', 'NR', 'VV', 'DEC', 'NN'], ['AD', 'NR', 'NR', 'AD', 'AD', 'VV'], ['VV', 'NR', 'NN', 'AD', 'VV', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['PN', 'NT', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NN', 'NN', 'VA', 'VA'], ['AD', 'DEV', 'NN', 'NN', 'VV', 'NR', 'NR'], ['NN', 'AD', 'AD', 'VV', 'AD', 'AD', 'VV', 'AD', 'AD', 'VA'], ['NN', 'VV', 'NN', 'NN', 'JJ', 'NN', 'VC', 'NN', 'DEG', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'P', 'NN', 'NN', 'LC', 'VV', 'NN', 'CC', 'NN'], ['VV', 'VV', 'NN', 'NN', 'NN', 'LC', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['DT', 'NN', 'VV', 'P', 'CD', 'CD', 'NT', 'VV', 'NR', 'DEG', 'NR'], ['LB', 'NN', 'VV', 'DEC', 'NR', 'NN', 'NN', 'VV', 'NN', 'CC', 'NN', 'NN'], ['NR', 'VV', 'AS', 'AD', 'VV', 'VV', 'AD', 'VV', 'VE', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'NR', 'VV', 'PN', 'VC', 'VV', 'DEC', 'NR'], ['VV', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['AD', 'NN', 'AD', 'VV', 'AD', 'VV', 'P', 'PN', 'DEG', 'NN', 'NN', 'JJ'], ['NN', 'AD', 'VA', 'VA', 'DEC', 'NN', 'NN', 'NN', 'VV', 'NR', 'NN', 'VA'], ['AD', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN', 'LC', 'VV', 'SP'], ['P', 'NN', 'LC', 'NR', 'AD', 'VV'], ['NT', 'DT', 'PN', 'NN', 'AD', 'VV'], ['NT', 'PN', 'AD', 'AD', 'VV', 'VV'], ['CS', 'CC', 'NN', 'VV', 'AD', 'VV', 'P', 'PN', 'VE', 'NN', 'NN', 'AD', 'VA'], ['NT', 'NN', 'NN', 'AD', 'BA', 'NN', 'VV', 'VV'], ['CD', 'NT', 'VV', 'DEC', 'NR', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VA'], ['NR', 'NR', 'DEG', 'NN', 'AD', 'VV', 'P', 'NR', 'NN', 'AD', 'VV', 'JJ', 'NN', 'VV', 'AS', 'NT'], ['VV', 'PN', 'VA', 'AD', 'VV', 'AS', 'NN', 'LC', 'NR', 'P', 'NN', 'CC', 'NN', 'LC', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'AS', 'AD', 'AD', 'NN', 'LC', 'AD', 'VV', 'VV', 'JJ', 'NN', 'PN', 'NN'], ['PN', 'AD', 'VV', 'AS', 'NR', 'AD', 'P', 'NN', 'VV', 'VV', 'VA', 'VV', 'DEC', 'NN'], ['VV', 'VV', 'AD', 'VV', 'NN', 'AD', 'AD', 'NN', 'DT', 'VV', 'VV', 'DEC', 'NN'], ['P', 'NN', 'NN', 'NR', 'NR', 'ETC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'AD', 'AD', 'VV', 'AS', 'NR', 'DEG', 'NN', 'VV', 'P', 'NN', 'NN'], ['P', 'NN', 'CC', 'NN', 'LC', 'NR', 'CC', 'NR', 'CC', 'CD', 'JJ', 'NN'], ['JJ', 'NN', 'SB', 'VV', 'CC', 'NR', 'NN', 'VV', 'JJ'], ['NR', 'AD', 'AD', 'VV', 'VV', 'CD', 'NN', 'AD', 'AD', 'VV', 'VA'], ['AD', 'VV', 'PN', 'VV', 'VV', 'AD', 'LB', 'VV', 'NN', 'VV', 'VV'], ['VV', 'VV', 'PN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV'], ['NR', 'NN', 'CD', 'CD', 'CD', 'CD', 'NN', 'NN', 'NN', 'NT', 'CD', 'CD'], ['JJ', 'CD', 'CD', 'NT', 'NN', 'P', 'AD', 'VV', 'NN'], ['NT', 'NN', 'VV', 'LC', 'AD', 'VV', 'NN', 'VV', 'VV', 'NN'], ['AD', 'AD', 'NR', 'NN', 'NT', 'AD', 'VE', 'NN', 'VV', 'NR'], ['DT', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN', 'VC', 'VV', 'JJ', 'DEG', 'NN'], ['CS', 'P', 'PN', 'CD', 'NN', 'PN', 'AD', 'VV', 'PN', 'AD', 'NR'], ['NN', 'VV', 'NN', 'NN', 'VE', 'NN', 'NN', 'LC', 'VV', 'DEC', 'NN', 'PN', 'CD', 'NN', 'VV', 'VV'], ['AD', 'JJ', 'NN', 'VV', 'VV'], ['JJ', 'NN', 'NN', 'NN', 'VV', 'NN'], ['AD', 'PN', 'AD', 'SB', 'VV', 'AS', 'CD', 'NN', 'AD', 'LB', 'NN', 'NN', 'VV', 'AS', 'NN'], ['AD', 'NR', 'NR', 'NR', 'PN', 'NT', 'AD', 'P', 'NR', 'VV', 'VV', 'VV', 'NN'], ['NN', 'VC', 'VV', 'VE', 'PN', 'DEG', 'NN', 'PN', 'PN', 'VC', 'NN', 'AD', 'VV', 'NN', 'SP'], ['AD', 'VV', 'NN', 'VV', 'VE', 'NN', 'VV', 'NN', 'NN', 'AD', 'VA'], ['NT', 'PN', 'VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['VE', 'JJ', 'NN', 'NN', 'AD', 'VV', 'VV', 'AD', 'NN', 'NN', 'VV'], ['NT', 'PN', 'VV', 'VV', 'AS', 'NN', 'VV', 'AS', 'AD', 'VV', 'DEC', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'AD', 'PN', 'AD', 'VV', 'SB', 'VV', 'DEC', 'NN', 'MSP', 'VV'], ['NN', 'NN', 'VV', 'AD', 'VV', 'DEC', 'NR', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'AD', 'VV', 'NN', 'VA', 'AD', 'VA', 'P', 'NN', 'VC', 'VV', 'VV', 'DEC', 'MSP', 'VV', 'AD', 'VV', 'AD', 'VV'], ['PN', 'AD', 'NN', 'NN', 'P', 'NR', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'NN', 'VV', 'PN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'PN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN', 'VA'], ['NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'AD', 'VA'], ['VV', 'CD', 'NN', 'VV', 'AD', 'NN'], ['AD', 'NN', 'AD', 'LB', 'NN', 'VV'], ['NR', 'AD', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NT', 'NN', 'AD', 'VV', 'AS', 'VA', 'VA', 'VA'], ['NN', 'NN', 'AD', 'VV', 'AS', 'VV', 'NN', 'AD', 'VV', 'PN', 'VV', 'PN', 'VV', 'NN', 'SP'], ['NN', 'NR', 'NN', 'NN', 'AD', 'P', 'NR', 'VV'], ['DT', 'SB', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VC', 'NN', 'NT', 'DEG', 'NN', 'AD', 'VV', 'CD'], ['NN', 'NR', 'NN', 'PN', 'AD', 'VV', 'AD', 'VV', 'PN'], ['CD', 'JJ', 'NN', 'NN', 'VV', 'NN', 'PN', 'AD', 'SB', 'VV', 'NN', 'NN'], ['NT', 'NR', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV'], ['VV', 'AD', 'VA', 'DEC', 'NN', 'P', 'NN', 'NN', 'LC', 'VV', 'VA', 'DEC', 'NN', 'NN'], ['NR', 'DT', 'PN', 'VV', 'NN', 'PN', 'VV', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'SP'], ['PN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'JJ', 'NN', 'CC', 'NN', 'AD', 'PN'], ['NN', 'P', 'VA', 'DEC', 'NN', 'NN', 'LC', 'VV', 'NN', 'VV', 'VV', 'VV'], ['VV', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'NR', 'CC', 'NR', 'DEG', 'NN'], ['NR', 'NN', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'JJ', 'DEG', 'AD', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'VV', 'AD', 'AD', 'VV'], ['AD', 'NR', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'AS', 'NN', 'NN'], ['NR', 'CC', 'PN', 'VV', 'P', 'CD', 'NN', 'DEG', 'CC', 'NN', 'LC', 'DEG', 'NN', 'NN'], ['P', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV'], ['VV', 'PN', 'VV', 'NN', 'PN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['P', 'NR', 'NN', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['VV', 'VV', 'VV', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'PN', 'AD', 'VV', 'VV', 'VV', 'NN'], ['PN', 'VV', 'VV', 'VV', 'PN', 'VV', 'NR', 'NR', 'NN'], ['NT', 'NT', 'NN', 'NN', 'P', 'NR', 'VV', 'VV', 'AS', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['PN', 'VE', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'VV', 'LC', 'NN', 'AD', 'VV'], ['NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['P', 'PN', 'VV', 'NT', 'JJ', 'NN', 'VC', 'AD', 'VV', 'NN'], ['AD', 'NR', 'AD', 'AD', 'P', 'NN', 'VV', 'AS', 'NN'], ['NR', 'VV', 'VV', 'CC', 'P', 'NN', 'VV', 'AS', 'JJ', 'NN'], ['PN', 'AD', 'AD', 'AD', 'VE', 'VV', 'NT', 'VV', 'DEC', 'NN', 'SP'], ['PN', 'VV', 'P', 'NR', 'VV', 'NR', 'CD', 'VV', 'P', 'NT', 'NN', 'VV', 'BA', 'PN', 'VV', 'VV', 'PN'], ['NN', 'AD', 'CD', 'JJ', 'DEG', 'NN', 'VV', 'DER', 'VA', 'AD', 'VA'], ['NR', 'AD', 'P', 'NN', 'VV', 'CD', 'M', 'DEG', 'NN', 'PN', 'NT', 'AD', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'NR', 'LC', 'NN', 'VV', 'LC', 'AD', 'VV', 'VV', 'SP'], ['NT', 'NT', 'NN', 'NN', 'AD', 'NN', 'VV', 'AS', 'AD'], ['VV', 'NN', 'VV', 'NR', 'P', 'NR', 'VE', 'CD', 'VV', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'AD', 'VV', 'VV', 'DEC', 'NN', 'PN', 'VV', 'NN', 'DEC', 'AD', 'VV'], ['NN', 'NN', 'VA', 'AD', 'VV', 'LC', 'DEG', 'JJ', 'CC', 'NN'], ['VV', 'VV', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NR', 'VA', 'VV', 'NN', 'AD', 'VV', 'NN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'JJ', 'NN', 'PN', 'AD', 'VC', 'VA', 'VV', 'DEC', 'NN'], ['NT', 'PN', 'VV', 'AS', 'AD', 'NN', 'VV', 'NN'], ['AD', 'CD', 'M', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VA', 'SP'], ['AD', 'PN', 'AD', 'AD', 'VV', 'VE', 'JJ', 'DEG', 'NN', 'NN'], ['DT', 'CD', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'VV'], ['JJ', 'DEG', 'NN', 'NN', 'VV', 'NN', 'LC', 'DEG', 'NN', 'NN'], ['P', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NR', 'VV', 'CD', 'VV', 'LC', 'DEG', 'AD', 'NN', 'NR'], ['VV', 'DEC', 'NN', 'NN', 'VC', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VE', 'CD', 'NN', 'NN', 'VV', 'DEC', 'NN', 'VV', 'NN', 'SP', 'AD', 'AD', 'VV', 'VV'], ['NN', 'VV', 'AS', 'VV', 'NN', 'AD', 'VC', 'NN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'NN', 'VA'], ['NR', 'NN', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['PN', 'VA', 'DEV', 'VV', 'P', 'NN', 'LC'], ['AD', 'NR', 'DEG', 'NN', 'VV'], ['NN', 'VV', 'DEC', 'NN', 'VV', 'NN', 'DEC', 'VV', 'NN', 'AD', 'VV'], ['NR', 'VV', 'DEC', 'NN', 'VV', 'NN', 'CD', 'NN', 'CD', 'CD', 'NN'], ['NN', 'CC', 'NN', 'NN', 'SB', 'VV', 'VV', 'CD', 'NN', 'JJ', 'DEG', 'NN'], ['PN', 'AD', 'AD', 'VV', 'P', 'VV', 'VV', 'NN', 'CC', 'VV', 'NN', 'NN', 'MSP', 'VV', 'DEC', 'NN'], ['P', 'NR', 'NR', 'AD', 'VV', 'DEC', 'NN', 'NN', 'NN', 'LC', 'VV', 'NN', 'AD', 'VV'], ['PN', 'VV', 'P', 'NN', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AD', 'P', 'NN', 'NN', 'SP'], ['P', 'NN', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'VV', 'CD', 'SB', 'VV', 'P', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'DEG', 'NN', 'AD', 'NN'], ['AD', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'DEC', 'VV', 'AD', 'AD', 'VV', 'NN'], ['CD', 'NN', 'AD', 'NN', 'CD', 'M', 'VV', 'P', 'NN', 'LC', 'AD', 'AD', 'P', 'NN', 'MSP', 'VV'], ['JJ', 'NN', 'NR', 'NN', 'LC', 'VV', 'NN'], ['P', 'VV', 'DEC', 'NN', 'VE', 'NN', 'VV', 'VV', 'DEC', 'NN', 'VE', 'NN', 'VV'], ['NR', 'NN', 'AD', 'VV', 'NN', 'PN', 'AD', 'AD', 'VE', 'PN', 'SP'], ['P', 'NT', 'VV', 'NR', 'NT', 'LC', 'NN', 'VA', 'AS', 'CD'], ['NR', 'VV', 'PN', 'DEG', 'NN', 'VV', 'NN', 'AD', 'VV', 'SP'], ['CD', 'JJ', 'NR', 'NN', 'AD', 'VV'], ['NR', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'NN', 'SB', 'VV'], ['NR', 'CD', 'NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'NN', 'NN', 'VV'], ['VV', 'NN', 'NN', 'VV', 'AD', 'VA', 'DEC', 'VC', 'NN', 'NN'], ['NR', 'VV', 'P', 'NN', 'NN', 'VV', 'NN'], ['DT', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NN', 'PN', 'AD', 'AD', 'P', 'NN', 'DEG', 'NN', 'NN', 'LC', 'P', 'NR', 'NN', 'VV', 'AS', 'NN'], ['DT', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'DT', 'NN', 'AD', 'NN', 'VV'], ['NN', 'DEG', 'NN', 'M', 'NN', 'LC', 'DEG', 'NN', 'NN', 'ETC', 'NN', 'NN', 'AD', 'VV'], ['AD', 'PN', 'AD', 'VC', 'NN', 'SP', 'PN', 'AD', 'VV', 'PN', 'VC', 'NN'], ['AD', 'PN', 'P', 'NN', 'LC', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['VV', 'NR', 'JJ', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['NN', 'VV', 'AD', 'VA', 'PN', 'VV', 'VV', 'SP'], ['NT', 'VV', 'AD', 'VV', 'LB', 'PN', 'VV', 'SP'], ['NN', 'NN', 'AD', 'P', 'NN', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'NN'], ['AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['NN', 'NN', 'NN', 'AD', 'AD', 'AD'], ['P', 'NN', 'NN', 'NN', 'LC', 'VV', 'PN'], ['PN', 'AD', 'VV', 'P', 'PN', 'LC', 'VV'], ['AD', 'VV', 'VV', 'DEC', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'NR', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'DEC', 'AD', 'VA', 'NN', 'NN', 'PN'], ['AD', 'NR', 'SB', 'VV', 'NN', 'VV'], ['AD', 'PN', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV'], ['AD', 'DT', 'NN', 'AD', 'SB', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'NN'], ['VV', 'PN', 'PN', 'AD', 'VC', 'JJ', 'DEG', 'JJ', 'NN'], ['AD', 'NN', 'VV', 'NN', 'VA'], ['AD', 'VC', 'CD', 'NN', 'VV', 'CD', 'NN'], ['AD', 'VV', 'AD', 'VE', 'PN', 'DEG', 'NN'], ['VV', 'AD', 'VA', 'VV', 'NR', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['NN', 'LC', 'PN', 'VV', 'NN', 'VV'], ['NR', 'JJ', 'NN', 'VV', 'NN', 'AD', 'VV', 'NR', 'VV', 'NN', 'CC', 'NN'], ['AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'PN', 'VV', 'AS', 'NN'], ['NT', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN'], ['NN', 'VV', 'DEC', 'NR', 'AD', 'VV', 'VV', 'VV', 'VV', 'NN'], ['NN', 'AD', 'LC', 'NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VA'], ['P', 'PN', 'VV', 'DEC', 'JJ', 'NN', 'LC', 'CC', 'VV', 'VV', 'NN'], ['NR', 'AD', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV'], ['P', 'AD', 'VV', 'VV', 'AD', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV'], ['PN', 'DEG', 'NN', 'AD', 'AD', 'JJ', 'NN', 'VV', 'P', 'NR', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['AD', 'PN', 'VV', 'VV', 'AD', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['VV', 'NN', 'VA', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['NN', 'AD', 'NN', 'VV', 'DT', 'NN', 'VV', 'VV', 'VE', 'PN', 'AD', 'VV', 'NN', 'SP', 'SP'], ['PN', 'CS', 'VV', 'AS', 'CD', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['VC', 'PN', 'NN', 'VE', 'NN'], ['NN', 'VV', 'JJ', 'NN'], ['NR', 'NN', 'AD', 'VV'], ['PN', 'AD', 'P', 'PN', 'VV', 'VV'], ['P', 'PN', 'AD', 'VV', 'VV', 'NN'], ['VV', 'NN', 'VV', 'VV', 'VV'], ['NR', 'VV', 'LC', 'AD', 'VV'], ['CD', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'LC', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'CD', 'NN', 'VV', 'AS', 'VV'], ['NN', 'AD', 'AD', 'VV', 'DT', 'JJ', 'NN', 'AD', 'BA', 'PN', 'VV', 'P', 'NN', 'LC', 'VV'], ['P', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV'], ['P', 'NN', 'NR', 'NR', 'NN', 'SB', 'PN', 'VV', 'NN', 'VV', 'NN', 'VV', 'JJ', 'NN', 'VV'], ['AD', 'CD', 'CD', 'AD', 'VV', 'VC', 'CD', 'CD', 'CD', 'CD', 'NN', 'VV', 'CD', 'NT', 'CD'], ['PN', 'DEG', 'NN', 'PN', 'AD', 'AD', 'VV'], ['AD', 'NR', 'AD', 'VV', 'AS', 'NR'], ['NN', 'LC', 'NN', 'VV', 'NN', 'VA', 'DEC', 'NN', 'NR', 'NN', 'VV', 'VV'], ['NN', 'VA', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['DT', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'PN', 'NN', 'DEG', 'NN'], ['NR', 'SB', 'VV', 'NN', 'VC', 'NN', 'NN'], ['NN', 'VV', 'P', 'NN', 'LC', 'VV', 'VV', 'PN', 'VA', 'AD', 'JJ'], ['AD', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'VC', 'JJ', 'DEG', 'NN', 'NN'], ['NR', 'CD', 'NN', 'VV', 'VV', 'NN', 'AD', 'NT', 'NT', 'LC', 'NN', 'AD', 'VV', 'AS'], ['AD', 'DT', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NT', 'NR', 'CC', 'NR', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN'], ['AD', 'P', 'CD', 'NN', 'NT', 'NT', 'VV', 'VV'], ['PN', 'AD', 'P', 'NR', 'AD', 'VV', 'VV', 'DEC', 'NT', 'NN', 'AD', 'VV', 'AS', 'CD', 'NN'], ['AD', 'PN', 'VV', 'NR', 'NT', 'AD', 'AD', 'VV', 'NN', 'NN', 'VV'], ['P', 'DT', 'NN', 'LC', 'NN', 'VA', 'VV', 'VV', 'NN', 'NN', 'VV', 'CD', 'NN'], ['AD', 'NR', 'NT', 'AD', 'VV', 'PN', 'AD', 'VV', 'LC', 'VV', 'AD', 'JJ', 'DEG', 'NN'], ['IJ', 'PN', 'NT', 'AD', 'VV', 'VV', 'VA', 'SP'], ['NN', 'LC', 'NN', 'VA', 'AS', 'M', 'NN', 'AD', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'AD', 'VC', 'NN'], ['NT', 'AD', 'NN', 'AD', 'VE', 'VV', 'AS'], ['PN', 'VV', 'SP', 'PN', 'VV', 'SP', 'PN', 'VV', 'SP', 'PN', 'VV', 'VA', 'AS', 'PN', 'AD', 'VV', 'PN'], ['DT', 'NN', 'AD', 'VV', 'P', 'NR', 'NR', 'NN', 'NR', 'NN'], ['NR', 'NT', 'DEG', 'NN', 'NN', 'VC', 'NN', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'P', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['P', 'NN', 'NN', 'VV', 'JJ', 'NN', 'NN', 'P', 'NN', 'NN', 'LC', 'VV', 'NN'], ['PN', 'P', 'NN', 'AD', 'VV', 'AS'], ['VV', 'VV', 'NN', 'VA', 'DEC', 'NN', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['NR', 'P', 'NT', 'P', 'NR', 'VV', 'AS', 'NN', 'VV', 'DEC', 'NN'], ['P', 'VV', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'NR', 'AD', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VV', 'NN', 'NN'], ['PN', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'LC', 'VV', 'VV', 'VV'], ['NN', 'VV', 'VV', 'P', 'NN', 'NN'], ['NN', 'SP', 'SP', 'VV', 'VV', 'NN', 'SP'], ['AD', 'NN', 'VV', 'VV'], ['AD', 'NT', 'VV', 'DER', 'VV', 'NN'], ['AD', 'VV', 'VV', 'JJ', 'M', 'NN'], ['PN', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN'], ['CS', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['CD', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'SP', 'AD', 'AD', 'SB', 'AD', 'VV', 'SP'], ['AD', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['PN', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VV', 'P', 'NN'], ['AD', 'AD', 'NN', 'BA', 'DT', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['NR', 'AD', 'VV', 'AS', 'NR', 'NN', 'LC', 'NN', 'NR', 'NN'], ['DT', 'AD', 'NN', 'AD', 'P', 'JJ', 'JJ', 'NN', 'VV', 'JJ', 'NN', 'NN'], ['NN', 'NN', 'NN', 'CD', 'NN', 'CD', 'NN', 'DEV', 'P', 'PN', 'VV', 'NN'], ['VV', 'LC', 'NN', 'NN', 'NN', 'AD', 'VV', 'NR', 'VV', 'VV', 'P', 'VV', 'NN', 'VV'], ['NR', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN', 'VV', 'AD', 'VV'], ['AD', 'PN', 'DEG', 'NN', 'NR', 'NN', 'AD', 'VE', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['VV', 'LC', 'VV', 'NN', 'CC', 'DT', 'CD', 'NN', 'VV', 'VV', 'PN', 'VV', 'AD', 'VA'], ['NN', 'VV', 'VV', 'VV', 'VV', 'VC', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'NT', 'P', 'NR', 'NR', 'NN', 'NN', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NN', 'LC', 'P', 'NN', 'AD', 'VV', 'VV', 'AS', 'NR', 'NN'], ['PN', 'AD', 'VV', 'VA', 'AS', 'AD', 'AD', 'AD', 'VV', 'SP'], ['VV', 'PN', 'VV', 'DEC', 'DT', 'NN', 'NN', 'NN', 'VV', 'SP'], ['VE', 'CD', 'CD', 'NN', 'VV', 'VV', 'PN', 'PN', 'CD', 'VV', 'VV', 'NN', 'PN', 'AD', 'AD', 'VV', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'PN', 'AD', 'AD', 'AD', 'VA', 'VV', 'PN', 'SP'], ['VV', 'VV', 'NN', 'NR', 'AD', 'VV'], ['NN', 'VV', 'AS', 'VA', 'NR', 'NN'], ['NN', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'NN', 'DEG', 'NN', 'VV'], ['NR', 'VV', 'NN', 'VV', 'AD', 'AD', 'VV'], ['NN', 'DT', 'VV', 'NN', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NR', 'VV', 'NN', 'LC', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'P', 'PN'], ['PN', 'VV', 'PN', 'M', 'NN', 'VV', 'AS', 'NR', 'VV', 'NR', 'JJ', 'NN', 'DEC', 'NN'], ['AD', 'P', 'AD', 'NN', 'VV', 'JJ', 'NN', 'P', 'NR', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'VV', 'PN', 'VC', 'CD', 'NN', 'VA', 'DEC', 'NR', 'NN'], ['VV', 'LC', 'CC', 'VV', 'NR', 'JJ', 'NN', 'SB', 'VV', 'P', 'NR', 'DEG', 'NN', 'LC'], ['NN', 'VV', 'AD', 'AD', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['AD', 'DT', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VE', 'CD', 'NN'], ['AD', 'VV', 'NR', 'NN', 'NN', 'VV', 'VV', 'NN'], ['PN', 'AD', 'VV', 'P', 'NN', 'AD', 'VA', 'NN', 'CC', 'P', 'NN', 'LC', 'VV', 'NN', 'VV'], ['VV', 'NR', 'AD', 'VV', 'AD', 'VV', 'NR', 'P', 'DT', 'NN', 'CD', 'OD', 'M', 'NN'], ['AD', 'VV', 'VV', 'DEC', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'M', 'DEG', 'NN'], ['P', 'NN', 'VV', 'PN', 'AD', 'VV', 'NN', 'CC', 'VV', 'VV'], ['AD', 'PN', 'VC', 'M', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'P', 'NN', 'LC', 'VA'], ['AD', 'DT', 'NN', 'NN', 'AD', 'VC', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['P', 'VV', 'DEC', 'NN', 'VV', 'VE', 'NN', 'NN', 'VV', 'VV', 'CD', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN'], ['PN', 'VV', 'VV', 'NN', 'PN', 'AD', 'AD', 'VV', 'AS', 'NN'], ['NN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'NR', 'NR', 'NN', 'NT', 'DEG', 'NN', 'VV'], ['PN', 'AD', 'VV', 'PN', 'VC', 'VV', 'AD', 'VV', 'NN', 'NN', 'VA'], ['JJ', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['NR', 'VV', 'AD', 'SP', 'PN', 'VV', 'M', 'NN', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'SP', 'SP'], ['VV', 'VV', 'AD', 'AD', 'AD', 'VV', 'PN', 'VV', 'NN', 'VV', 'PN', 'VV', 'SP', 'AD', 'AD', 'P', 'PN', 'VV', 'NN', 'SP'], ['AD', 'VV', 'DER', 'PN', 'NN', 'VV', 'VV', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'CD', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'AS', 'NN', 'VE'], ['PN', 'AD', 'VV', 'PN', 'BA', 'NN', 'VV', 'AD', 'JJ', 'AD', 'VC', 'PN', 'VV', 'AS', 'NN'], ['DT', 'P', 'NN', 'CC', 'NN', 'VV', 'AS', 'NN', 'VV', 'NR', 'LC'], ['AD', 'CD', 'M', 'AS', 'NN', 'AD', 'VV', 'VV', 'SP', 'SP'], ['NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'VV', 'JJ'], ['NR', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'SP'], ['VV', 'AD', 'VV', 'AD', 'P', 'NN', 'LC', 'P', 'NN', 'VV', 'NN', 'VV'], ['PN', 'P', 'NR', 'VV', 'AS', 'AD', 'CD'], ['PN', 'BA', 'NN', 'VV', 'P', 'PN', 'SP'], ['CS', 'AD', 'AD', 'VV', 'AS', 'SP'], ['NN', 'VV', 'NT', 'VA', 'SP'], ['CS', 'VV', 'AD', 'P', 'NN', 'VV', 'NN'], ['NN', 'VV', 'LC', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV', 'PN', 'VV', 'DER', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'P', 'PN', 'VV', 'VV', 'AS', 'JJ', 'NN', 'NN', 'P', 'NN', 'NN', 'VV', 'LC', 'VV'], ['PN', 'CD', 'NT', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NR', 'AD', 'NN', 'NN', 'VV', 'AS', 'PN', 'NN', 'NN', 'VV', 'CC', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VA', 'DEC', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VA', 'DEV', 'PN', 'AD', 'P', 'PN', 'VV'], ['NN', 'SP', 'SP', 'AD', 'VV', 'DT', 'NN', 'VV', 'DEC', 'NN', 'CC', 'NN', 'NN', 'VV', 'DEC', 'NN', 'VA'], ['PN', 'AD', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'SP'], ['P', 'NN', 'VV', 'NR', 'JJ', 'VV', 'AS'], ['AD', 'PN', 'AD', 'AD', 'VA', 'SP'], ['AD', 'P', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NR', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'CC', 'VV', 'VV', 'CD'], ['NN', 'VV', 'NN', 'NN', 'NR', 'VV', 'NN', 'VV', 'NN', 'AD', 'VA'], ['PN', 'NN', 'VV', 'NN', 'VV', 'VV', 'AS', 'NR', 'NN', 'DEG', 'NN'], ['CS', 'PN', 'AD', 'VC', 'NN', 'AD', 'NR', 'NR', 'AD', 'VE', 'NN', 'VV', 'VV', 'NN'], ['AD', 'VV', 'AS', 'AD', 'JJ', 'NN', 'AD', 'P', 'NN', 'NN', 'VV', 'VV', 'CC', 'AD', 'VV', 'NN', 'NN'], ['NT', 'VV', 'NN', 'NN', 'P', 'NN', 'VV', 'CD', 'NN', 'VV', 'PN', 'NN', 'PN', 'VV', 'P', 'PN', 'SP'], ['P', 'NN', 'NN', 'NN', 'LC', 'NR', 'NN', 'AD', 'P', 'NN', 'NR', 'VV'], ['PN', 'NN', 'AD', 'VV', 'VC', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV', 'PN', 'NN', 'DEG', 'PN', 'AD', 'VC', 'SP'], ['AD', 'AD', 'CD', 'NN', 'AD', 'VV', 'SP'], ['NN', 'VV', 'AD', 'PN', 'PN', 'VE', 'NN', 'SP'], ['VV', 'PN', 'NN', 'AD', 'VA', 'NN', 'VV', 'CD'], ['NR', 'NR', 'VV', 'NN', 'NN', 'AD', 'VV'], ['PN', 'VC', 'AD', 'NN', 'DEG', 'NN', 'NN', 'VA'], ['NR', 'PN', 'P', 'NN', 'LC', 'VV', 'NN', 'SP', 'SP'], ['AD', 'P', 'NN', 'VV', 'AS', 'NR', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'AS', 'NN', 'AD', 'P', 'NR', 'NR', 'VV'], ['VV', 'VV', 'VV', 'VC', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'AD', 'VV', 'CC', 'PN', 'NN', 'VV', 'NN', 'VE', 'NN'], ['NR', 'DEG', 'NN', 'P', 'NN', 'CD', 'CD', 'CD', 'CD', 'CD'], ['VV', 'VV', 'AD', 'VV', 'PN', 'SP'], ['NN', 'AD', 'AD', 'AD', 'VV'], ['JJ', 'NN', 'P', 'NR', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VA'], ['VV', 'VV', 'NN', 'NN', 'NN'], ['P', 'NN', 'NN', 'NN', 'NN', 'LC', 'VV', 'VE', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NT', 'CD', 'JJ', 'NN', 'VV', 'P', 'NN', 'NN', 'VV', 'PN', 'VV', 'PN', 'PN', 'VC', 'NN', 'NN', 'SP'], ['PN', 'VV', 'VV', 'NR', 'NN', 'P', 'NN', 'AD', 'VV', 'NN'], ['VV', 'JJ', 'NN', 'NN', 'CD', 'AD', 'JJ', 'NN', 'NN', 'DEC', 'AD', 'AD', 'VA'], ['VV', 'LC', 'NR', 'VV', 'AD', 'VV', 'VV', 'AD', 'VV', 'VV'], ['AD', 'CD', 'VV', 'NN', 'VV', 'DEC', 'VC', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'CC', 'NN', 'NN', 'P', 'NR', 'NR', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'PN', 'AD', 'VV'], ['P', 'LB', 'NN', 'VV', 'NR', 'VV', 'NN', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'JJ', 'P', 'BA', 'DT', 'NN', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN', 'LC'], ['PN', 'VV', 'AS', 'PN', 'VC', 'VV', 'PN', 'P', 'PN', 'VA', 'VA', 'DEC', 'NN', 'NN', 'DT', 'CD', 'NN', 'SP'], ['NT', 'NT', 'NN', 'DEG', 'NN', 'AD', 'SB', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'P', 'CD', 'CD', 'CD', 'CD', 'CD'], ['NN', 'VA', 'NN', 'NR', 'NR', 'NN'], ['AD', 'AD', 'AD', 'NN', 'DEG', 'NN', 'JJ', 'NN', 'SP'], ['AD', 'PN', 'AD', 'AD', 'VA', 'DEC', 'NN', 'NR', 'NN'], ['P', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'VC', 'AD', 'VE', 'NN'], ['AD', 'NT', 'PN', 'AD', 'P', 'NR', 'BA', 'VV', 'NN', 'CC', 'NN', 'AD', 'VV', 'AS'], ['VV', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'P', 'NR', 'DEG', 'NR', 'NN', 'NR', 'VV'], ['CD', 'NN', 'AD', 'VV', 'AS', 'VV', 'NN', 'ETC', 'NN'], ['AD', 'PN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['VC', 'BA', 'NN', 'DEG', 'DT', 'NN', 'AD', 'VV', 'NN', 'SP'], ['NN', 'NN', 'VC', 'NN', 'VV', 'AS', 'AD', 'CD', 'M', 'NN'], ['VV', 'DEV', 'VV', 'VV', 'NN', 'NR', 'CD', 'OD', 'NN'], ['NN', 'VV', 'NN', 'AD', 'VV', 'SP'], ['NR', 'P', 'NN', 'NN', 'CC', 'NN', 'VV'], ['JJ', 'NN', 'NR', 'NN', 'AD', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'NN', 'AD', 'VA', 'VV'], ['NN', 'NN', 'VV', 'NR', 'NR', 'VV', 'VV'], ['NN', 'NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'VA', 'DEC', 'NN', 'CC', 'NN', 'AD', 'VV', 'VV', 'VV', 'AD', 'DEC', 'NN'], ['VV', 'VV', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV'], ['P', 'NN', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VV'], ['NT', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['PN', 'AD', 'AD', 'VA', 'SP', 'VV', 'AS', 'NN', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'NN', 'JJ', 'DEG', 'NN', 'SP', 'PN', 'VV', 'AD', 'PN', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'VV', 'NN', 'SP', 'VA'], ['AD', 'VV', 'JJ', 'NN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AS', 'DT', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VE', 'AS', 'M', 'NN', 'VV', 'VA', 'DEV', 'VV', 'AS', 'NN'], ['NN', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN'], ['DT', 'NN', 'AD', 'AD', 'VA', 'SP', 'AD', 'VV', 'SP'], ['NR', 'AD', 'LC', 'AD', 'VV', 'AS', 'VV', 'CC', 'VV', 'NN', 'NN', 'DEC', 'NN'], ['NR', 'DEG', 'NN', 'PN', 'AD', 'VV'], ['DT', 'NN', 'NN', 'VC', 'CD', 'CD', 'M'], ['PN', 'AD', 'VV', 'P', 'PN', 'VV', 'VV'], ['AD', 'VV', 'NR', 'NN', 'DEG', 'NN', 'SP'], ['BA', 'NN', 'VV', 'NN', 'AD', 'VC', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV'], ['PN', 'AD', 'AD', 'P', 'NN', 'VV', 'DEC', 'NN', 'LB', 'NN', 'VV'], ['NN', 'AD', 'CC', 'NN', 'AD', 'AD', 'VA', 'DEC', 'NN'], ['NR', 'VV', 'AS', 'NN', 'LC', 'NN', 'NN', 'NN', 'NN', 'LC', 'VV', 'NN', 'NR'], ['VV', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['P', 'NR', 'VV', 'NR', 'VC', 'JJ', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'LC', 'NR', 'VV', 'AS', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'P', 'NN', 'VV', 'AS', 'NN', 'NN', 'SB', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'NR', 'AD', 'AD', 'VV'], ['DT', 'M', 'VV', 'AD', 'VA', 'DEC', 'NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VA'], ['AD', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'LC', 'PN', 'VV', 'NT', 'NN'], ['NR', 'AD', 'VV', 'NR', 'AD', 'VV', 'AS', 'PN', 'P', 'NR', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN'], ['NT', 'NN', 'VV', 'VV', 'PN', 'AD', 'P', 'PN', 'M', 'VA', 'SP', 'PN', 'VV', 'SP'], ['VC', 'SP', 'VC', 'SP', 'NR', 'VV', 'PN', 'VV', 'DER', 'VA', 'SP'], ['DT', 'NN', 'DT', 'AD', 'SB', 'VV', 'NN', 'NN', 'VV'], ['AD', 'JJ', 'DEG', 'AD', 'NN', 'VC', 'NN', 'NN', 'NN'], ['PN', 'VV', 'DER', 'AD', 'VV', 'VV', 'NN', 'LC', 'AD', 'VV'], ['PN', 'VV', 'PN', 'VV', 'VV'], ['VV', 'CD', 'VV', 'NT', 'PN', 'VV', 'AS', 'AD', 'VA', 'DEC', 'NN'], ['NR', 'VV', 'NN'], ['NN', 'NN', 'CD', 'CD', 'CD', 'M', 'CD', 'CD', 'NN', 'CC', 'NN', 'CD', 'CD'], ['NR', 'CC', 'NR'], ['NN', 'DEG', 'NN', 'NN', 'VV', 'PN', 'AD', 'VV', 'AD', 'PN', 'AD', 'VV', 'NN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['AD', 'NR', 'DEG', 'NN', 'NN', 'AD', 'NR', 'DEG', 'JJ', 'NN', 'NN', 'CC', 'JJ', 'NN'], ['NN', 'AD', 'AD', 'VV', 'DEC', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'VC', 'NN', 'JJ', 'NN', 'DEG', 'JJ', 'NN'], ['NR', 'NR', 'NN', 'NN', 'AD', 'VA', 'NT', 'VV', 'NN', 'P', 'PN', 'NN', 'NN'], ['NR', 'VV', 'NR', 'VV', 'DEC', 'OD', 'VC', 'DT', 'NN', 'PN', 'AD', 'VV', 'AS', 'DEC'], ['AD', 'VV', 'NN', 'VV', 'VA', 'DEC', 'NN', 'VV', 'NN', 'VV', 'NN', 'VA'], ['P', 'JJ', 'VV', 'VV', 'VV', 'VV', 'NN', 'AD', 'VV', 'P', 'P', 'NN', 'VV', 'VV', 'AS'], ['NN', 'P', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'AD', 'CD', 'JJ', 'NN'], ['NT', 'VV', 'NN', 'AD', 'VA', 'DEC', 'NN', 'AD', 'P', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VA', 'DEC', 'NN', 'VV', 'NN', 'VV', 'M', 'NN'], ['CS', 'NN', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'JJ', 'NN', 'NN', 'NN', 'AD', 'SB', 'VV'], ['NN', 'P', 'NN', 'LC', 'VV', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'AD', 'VV'], ['AD', 'P', 'NR', 'LC', 'JJ', 'NN', 'NN', 'AD', 'VC', 'VA', 'SP', 'SP'], ['AD', 'AD', 'VV', 'AS', 'NN', 'VV', 'DEC', 'NN', 'AD', 'NN', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'AD', 'VE', 'NN', 'NN', 'P', 'PN', 'VV', 'NN'], ['VV', 'NN', 'VV', 'AS', 'PN', 'AD', 'AD', 'VV', 'AD', 'AD', 'NN', 'VV', 'AS', 'VV'], ['AD', 'NT', 'NT', 'LC', 'NN', 'VV', 'P', 'CD', 'M', 'VV'], ['NN', 'VV', 'PN', 'P', 'NN', 'NN', 'VV', 'VC', 'JJ', 'NN'], ['NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'CC', 'AD', 'VV', 'NN', 'ETC', 'JJ', 'NN'], ['NN', 'VA', 'DEC', 'PN', 'AD', 'SB', 'NR', 'AD', 'VV', 'DER', 'VV', 'NN', 'NN', 'AD', 'VV', 'NR', 'VV'], ['NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'CC', 'VV', 'NN', 'NN', 'VV'], ['AD', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'ETC', 'NN', 'AD', 'AD', 'VV'], ['CS', 'VV', 'NR', 'AD', 'VV', 'P', 'NN', 'NN', 'VA', 'MSP', 'VV', 'VA'], ['PN', 'VV', 'PN', 'AD', 'AD', 'VA', 'AD', 'NN', 'AD', 'AD', 'VV', 'PN', 'DEG', 'JJ'], ['CD', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'NN', 'AD', 'VA'], ['NN', 'VV', 'NR', 'NN', 'AD', 'P', 'VV', 'NN', 'NN', 'AD', 'VC', 'AD', 'VV', 'SP'], ['NN', 'P', 'DT', 'NR', 'VV', 'NN', 'VV'], ['DT', 'M', 'NR', 'NN', 'AD', 'AD', 'VA'], ['NN', 'CC', 'NR', 'AD', 'VV', 'AS', 'VV', 'NN', 'NN', 'VV', 'IJ'], ['AD', 'P', 'NN', 'NR', 'VV', 'DEC', 'NN', 'LC', 'NR', 'VV', 'VV', 'AS', 'NN'], ['NT', 'VV', 'NN', 'DEC', 'NR', 'VV', 'NN', 'SP', 'NN', 'AD', 'VV', 'NN', 'SP'], ['NN', 'VV', 'VV', 'NR', 'VV', 'PN', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'VC', 'NN'], ['DT', 'NN', 'NN', 'NN', 'VA'], ['NN', 'AD', 'CD', 'NN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['VV', 'VV', 'P', 'NN', 'NN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'VA', 'DEV', 'VV', 'VV'], ['NT', 'NN', 'AD', 'VV', 'JJ', 'VV', 'DEC', 'NN'], ['VE', 'NN', 'VV', 'NR', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VC', 'NN', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['NT', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'P', 'NN', 'NN', 'NN', 'VV', 'SB', 'VV', 'VV', 'AS', 'JJ', 'NR', 'DEG', 'NN', 'NN'], ['NN', 'VV', 'VV', 'AS', 'LC', 'NN', 'AD', 'VV', 'VV', 'NN', 'BA', 'NN', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'VV', 'NN', 'P', 'NN', 'NN', 'NN', 'AD', 'NN', 'VV', 'NN'], ['P', 'NR', 'AD', 'AD', 'VV', 'AD', 'VV', 'VV', 'DER', 'PN', 'AD', 'VV', 'VV'], ['NT', 'P', 'NN', 'NN', 'LC', 'AD', 'AD', 'VE', 'AS', 'JJ', 'NN'], ['AD', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NN', 'NN'], ['NN', 'LC', 'VV', 'AS', 'NN', 'PN', 'AD', 'VV', 'AS', 'NN'], ['NN', 'P', 'NR', 'NR', 'ETC', 'NR', 'NN', 'LC', 'VV', 'AD', 'VA'], ['AD', 'VV', 'AD', 'AD', 'VV', 'AD', 'AD', 'VE', 'NN'], ['AD', 'NN', 'NN', 'P', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'VV', 'LC', 'BA', 'PN', 'VV', 'VV', 'NN', 'NN', 'LC', 'VV'], ['VA', 'NN', 'CC', 'NR', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN'], ['PN', 'P', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA', 'CD'], ['PN', 'P', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VA'], ['P', 'PN', 'VV', 'DEC', 'VC', 'DT', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'AS', 'JJ', 'NR', 'VV', 'VV', 'NN'], ['NR', 'VV', 'NN', 'NN', 'DT', 'NN', 'VV', 'DEC', 'NN', 'CD', 'SP'], ['NN', 'VV', 'AS', 'JJ', 'NN', 'NR', 'JJ', 'NN', 'VV', 'P', 'AD', 'NN', 'LC'], ['NN', 'NN', 'VV', 'NR', 'NN', 'NR', 'NN', 'VV'], ['PN', 'AD', 'VV', 'NN', 'DEC', 'PN', 'VV', 'IJ', 'AD', 'AD', 'VE', 'PN', 'DEG', 'NN'], ['NR', 'NN', 'NN', 'AD', 'VV', 'JJ', 'VV', 'NN', 'DEC', 'VV'], ['P', 'NN', 'NN', 'NN', 'LC', 'NR', 'NN', 'VC', 'NN', 'DEG', 'NN'], ['AD', 'NR', 'VV', 'AS', 'NN', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['NR', 'CC', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'CD'], ['P', 'NN', 'AD', 'NN', 'DEC', 'VC', 'VV', 'CD', 'CD', 'M', 'NN', 'DEG', 'NR', 'NR'], ['NR', 'VV', 'NN', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV', 'NR', 'VV'], ['PN', 'AD', 'VC', 'NN', 'NN', 'VV', 'AD', 'VV', 'DEC', 'JJ', 'NN'], ['NN', 'VV', 'NR', 'VV', 'NN', 'CC', 'VV', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'VV', 'VV', 'CD', 'NN'], ['NR', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'JJ'], ['PN', 'VV', 'VV', 'AD', 'VV', 'PN', 'AD', 'P', 'VV', 'SP'], ['AD', 'VV', 'PN', 'VV', 'AD', 'AD', 'P', 'NN', 'VV', 'VV'], ['AD', 'NN', 'VV', 'NN', 'PN', 'VV', 'JJ', 'CC', 'NN', 'VA', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'VV', 'PN', 'VV', 'VV', 'SP', 'VV', 'PN', 'AD', 'VV', 'VA', 'VV', 'DEC'], ['AD', 'NR', 'AD', 'VV', 'VV', 'PN', 'VV', 'NN', 'NR', 'VV', 'PN', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'VV', 'NR', 'NR', 'AD', 'P', 'NN', 'VV', 'AS', 'NN'], ['AD', 'PN', 'VC', 'AD', 'NN', 'NN', 'SP', 'PN', 'VA', 'SP'], ['NN', 'AD', 'VV', 'AS', 'VV', 'VV', 'JJ', 'NN', 'BA', 'NN', 'VV', 'VV'], ['PN', 'VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'JJ', 'NN', 'AD', 'VV', 'VV'], ['AD', 'NN', 'AD', 'VV', 'NR', 'VV', 'NR', 'DEG', 'NN', 'P', 'PN', 'VV', 'VV'], ['VV', 'AD', 'VV', 'CD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'P', 'DT', 'AD', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['NN', 'NN', 'VV', 'VV'], ['PN', 'VV', 'LC', 'AD', 'VA', 'AD', 'VA', 'AD', 'AD', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'P', 'NN', 'NN', 'LC', 'NN', 'AD', 'VV', 'VV', 'VV', 'VV', 'PN', 'VV'], ['NR', 'SB', 'VV', 'LC', 'VV', 'AD', 'NT', 'LC', 'PN', 'AD', 'VV', 'NN'], ['PN', 'NN', 'AD', 'VA', 'AD', 'P', 'CD', 'NN', 'VV', 'NN', 'AD', 'VV', 'CD', 'AD', 'JJ', 'DEG', 'NN'], ['NR', 'AD', 'VA', 'AD', 'AD', 'VV', 'DT', 'PN', 'VV', 'PN'], ['NN', 'NR', 'NN', 'VV', 'NR', 'AD', 'VV'], ['VV', 'NN', 'NN', 'P', 'NR', 'LC', 'DEG', 'NN', 'NN', 'VV', 'VV', 'VV', 'NN'], ['DT', 'M', 'NN', 'AD', 'SB', 'VV', 'VV', 'NR', 'JJ', 'JJ', 'NN', 'NN', 'AD', 'VV'], ['CD', 'NN', 'LC', 'SB', 'VV', 'VV', 'DEC', 'NR', 'NN', 'AD', 'SB', 'VV', 'VV', 'VV'], ['AD', 'VV', 'VV', 'BA', 'NN', 'VV', 'VV', 'NN'], ['VV', 'AS', 'P', 'NT', 'LC', 'AD', 'VV', 'VV', 'CD', 'CD', 'M', 'NR', 'NN'], ['NN', 'VV', 'PN', 'VV', 'PN', 'VV', 'VA', 'NN', 'AD', 'VV', 'NN', 'VV'], ['NN', 'PN', 'P', 'PN', 'NN', 'VE', 'AD', 'VA', 'SP'], ['NR', 'AD', 'VV', 'AD', 'VV', 'VV'], ['NT', 'DEG', 'NN', 'AD', 'VE', 'NT', 'VA'], ['AD', 'NR', 'AD', 'VV', 'PN', 'VV', 'NN'], ['VV', 'NN', 'VV', 'SP', 'AD', 'VA'], ['NT', 'AD', 'VC', 'NR', 'VV', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'SB', 'VV', 'P', 'NN'], ['BA', 'NN', 'CC', 'JJ', 'NN', 'VV', 'AS'], ['JJ', 'NN', 'P', 'NN', 'VV', 'AS', 'AD', 'VA'], ['NT', 'NR', 'P', 'VV', 'NN', 'NN'], ['AD', 'VE', 'CD', 'VC', 'NN', 'NN', 'VV', 'SP'], ['NR', 'NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'SP'], ['NR', 'NN', 'P', 'AS', 'NN', 'VV', 'VV', 'AD', 'VV', 'VV', 'DT', 'NN', 'AD', 'VA'], ['P', 'VV', 'NN', 'LC', 'NT', 'AD', 'AD', 'AD', 'VV'], ['NR', 'PN', 'NN', 'PN', 'VV', 'VV', 'PN', 'NN', 'NN'], ['AD', 'AD', 'PN', 'AD', 'VV', 'BA', 'NN', 'VV', 'BA', 'NN', 'VV', 'VA'], ['AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NT', 'NT', 'NR', 'NN', 'VV', 'NN', 'NN'], ['DT', 'NN', 'VV', 'VV', 'CD', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'SP'], ['P', 'NN', 'CC', 'NN', 'PN', 'DEG', 'NN', 'VV', 'VA'], ['PN', 'VV', 'NN', 'NN', 'CC', 'NN', 'AD', 'VA', 'AS', 'CD'], ['NN', 'VV', 'AS', 'PN', 'P', 'VV', 'NN', 'VV', 'AS', 'VV'], ['NN', 'LC', 'NR', 'AD', 'DEV', 'VV', 'AS', 'NN', 'AD', 'AD', 'VV', 'VV', 'AS', 'NN'], ['P', 'NR', 'VV', 'NN', 'NN', 'NN', 'VA'], ['AD', 'VC', 'AD', 'VV', 'AD', 'AD', 'VV', 'AD', 'VA'], ['PN', 'AD', 'BA', 'NN', 'VV', 'P', 'NN', 'NN', 'AD', 'NN', 'LC'], ['AD', 'VV', 'JJ', 'NN', 'JJ', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'AD', 'VV', 'AS', 'AD', 'JJ', 'DEG', 'NN', 'DEG', 'NN'], ['AD', 'AD', 'P', 'NN', 'LB', 'NN', 'VV', 'SB', 'NN', 'VV', 'NR', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NT', 'NN', 'VC', 'AD', 'VV', 'SP'], ['PN', 'AD', 'VV', 'DT', 'AD', 'JJ', 'NN', 'NN', 'NN', 'LC', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'PN', 'P', 'NR', 'VV', 'AS', 'JJ', 'NN'], ['NN', 'DEG', 'JJ', 'NN', 'VV', 'NN', 'DEG', 'NN', 'LC'], ['NR', 'NR', 'AD', 'VV', 'PN'], ['CS', 'VV', 'P', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'AD', 'AD', 'AD', 'VA'], ['VV', 'PN', 'M', 'JJ', 'NN', 'VV', 'AD', 'VV', 'M', 'NN', 'LC', 'AD', 'VV'], ['PN', 'DEG', 'NN', 'AD', 'P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'VV', 'VE', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'AD', 'NR', 'VV', 'P', 'NN', 'VV', 'PN', 'VV', 'VV'], ['NN', 'NN', 'NN', 'VC', 'NT', 'NT', 'NT', 'CC', 'NT', 'NT', 'NT'], ['AD', 'NR', 'NN', 'AD', 'P', 'PN', 'VV', 'NN'], ['NR', 'NN', 'NR'], ['VV', 'NR', 'NN', 'NN', 'NN', 'NR', 'NN', 'ETC', 'JJ', 'NN', 'NN', 'NN'], ['JJ', 'NN', 'AD', 'P', 'PN', 'PN', 'PN', 'VV', 'P', 'NN', 'NN', 'VV', 'AD', 'NN'], ['NN', 'JJ', 'BA', 'NR', 'VV', 'LC', 'NN', 'NN', 'AD', 'SB', 'VV', 'NN', 'NN'], ['NR', 'NR', 'NN', 'VV', 'NR', 'DEG', 'NN', 'VV', 'AD', 'DEC', 'NN'], ['NN', 'VV', 'P', 'VV', 'NN', 'NN', 'NN', 'LC', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['VV', 'LC', 'NR', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'AS'], ['CD', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'AS', 'JJ', 'NN'], ['VV', 'NN', 'CC', 'JJ', 'NN', 'AD', 'LC', 'NR', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['AD', 'JJ', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'PN'], ['NR', 'AD', 'VV', 'NN', 'CC', 'NR', 'NN', 'NN', 'NN'], ['NR', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN'], ['AD', 'P', 'NN', 'LC', 'VV', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN'], ['P', 'NN', 'VV', 'LC', 'NR', 'VV', 'VV', 'AS', 'CD', 'NN', 'NR', 'NN'], ['P', 'NN', 'LC', 'NN', 'NN', 'VV', 'AS', 'VV', 'NR', 'NN'], ['NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'JJ', 'DEG'], ['AD', 'VV', 'CC', 'VV', 'NN'], ['AD', 'PN', 'AD', 'VV', 'PN', 'AD', 'SP'], ['PN', 'P', 'NN', 'VV', 'AD', 'VV'], ['PN', 'VV', 'AS', 'NR', 'JJ', 'NN'], ['AD', 'VV', 'DT', 'JJ', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'AD', 'VE', 'NN'], ['VV', 'NN', 'AD', 'VA'], ['PN', 'NN', 'VV', 'P', 'PN', 'VV', 'AD'], ['VV', 'AD', 'VA', 'DEC', 'NN', 'PN', 'AD', 'AD', 'VV', 'PN'], ['NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'CD', 'AD', 'VV', 'PN', 'VV', 'SP'], ['NR', 'NN', 'VV', 'NN', 'CC', 'NN', 'VA', 'AD', 'VA'], ['VV', 'LC', 'PN', 'NN', 'VV', 'VV', 'VV', 'LC', 'AD', 'VV', 'VV', 'SP'], ['NN', 'P', 'P', 'NR', 'VV', 'NN'], ['NN', 'VV', 'NN', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'VV', 'AS', 'NN', 'VV'], ['NR', 'P', 'NR', 'NN', 'NN', 'NN', 'VV', 'NR', 'JJ', 'NN', 'VV', 'NN'], ['NR', 'VV', 'AD', 'VV', 'P', 'NR', 'NN', 'AD', 'VV', 'DT', 'NN'], ['AD', 'VC', 'VV', 'AS', 'NN', 'NR', 'VV', 'DEV', 'AD', 'VV', 'VV', 'AS', 'NN', 'P', 'NN', 'VV', 'VV'], ['PN', 'AD', 'P', 'DT', 'NN', 'AD', 'VA', 'DEC', 'NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'VV'], ['VV', 'P', 'NN', 'NN', 'NN', 'VV', 'VV', 'DEC', 'VA', 'CC', 'VV', 'CD', 'NN', 'AD', 'AD', 'VA'], ['NR', 'NN', 'JJ', 'NN', 'NN', 'AD', 'VA'], ['AD', 'NR', 'NN', 'NN', 'VV', 'NN', 'AD', 'NN', 'VV', 'VV'], ['AD', 'VV', 'NN', 'VV', 'DEC', 'NN', 'AD', 'CC', 'JJ', 'DEG', 'NN', 'CC', 'NN'], ['AD', 'NN', 'VV', 'CD', 'CD', 'CD', 'VV', 'NT', 'CD', 'NN', 'NN'], ['AD', 'DT', 'NN', 'AD', 'VV', 'P', 'P', 'NN', 'NN', 'AD', 'VV'], ['PN', 'AD', 'P', 'PN', 'NN', 'P', 'PN', 'VV', 'VV', 'AD', 'NN', 'AD', 'VV'], ['P', 'NN', 'NN', 'NN', 'VA', 'AD', 'CD', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV'], ['VV', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'DER', 'VV'], ['NR', 'P', 'NN', 'VV', 'DER', 'VE', 'NN', 'NN', 'NT', 'AD', 'VV', 'NR', 'VV', 'NN'], ['NN', 'NN', 'VV', 'VV', 'PN', 'VV', 'JJ', 'NN', 'NT', 'PN', 'VV', 'VV', 'PN'], ['NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'PN', 'VV', 'VV', 'PN', 'AD', 'VV'], ['NR', 'CC', 'NR', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['JJ', 'NN'], ['NN', 'NN', 'SB', 'VV', 'VV'], ['NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'AD', 'VV'], ['PN', 'VV', 'PN', 'VV', 'SP'], ['VV', 'P', 'NR', 'NN', 'NR', 'VV'], ['NN', 'VE', 'NN', 'P', 'NR', 'NN', 'VV', 'NR', 'CC', 'NR', 'VV', 'NN', 'NN', 'DEC', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'SB', 'NN', 'NN', 'VV', 'NR', 'P', 'NN', 'NN'], ['NR', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NT', 'VV', 'NN', 'AD', 'P', 'PN', 'VV', 'AD', 'AD', 'VA'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'VV', 'AD', 'NN', 'VA', 'AS', 'AD', 'AD', 'AD', 'VA'], ['CD', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'ETC', 'VV', 'AS', 'NT'], ['JJ', 'DEG', 'NN', 'AD', 'VC', 'NR', 'NN', 'NT', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'DEG', 'CD', 'NN', 'SB', 'VV', 'VV', 'NN', 'AD', 'SB', 'VV', 'VV'], ['PN', 'AD', 'P', 'NN', 'NR', 'VV', 'NN', 'VV', 'AS', 'NR', 'NN'], ['AD', 'VV', 'NN', 'VV', 'VV', 'LC', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN'], ['VV', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV', 'VV', 'CD', 'CD', 'CD', 'CD', 'CD', 'CD', 'CD', 'CD'], ['AD', 'DT', 'CD', 'NN', 'AD', 'VV', 'DT', 'NN', 'AD', 'VV', 'NR', 'NN'], ['NR', 'AD', 'VA', 'AD', 'AD', 'AD', 'BA', 'NN', 'VV', 'AD', 'VV', 'DEV'], ['NR', 'AD', 'VA', 'DEC', 'NN', 'NN', 'NR', 'NR', 'AD', 'AD', 'VV'], ['VV', 'NN', 'VV', 'AS', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV'], ['PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN', 'JJ', 'NN'], ['P', 'P', 'NN', 'VV', 'DEC', 'AD', 'VV', 'NN', 'DEC', 'NN', 'SB', 'AD', 'VV', 'AD', 'VA'], ['PN', 'AD', 'AD', 'VV', 'NN', 'DEC', 'AD', 'VE', 'VV'], ['NN', 'ETC', 'NN', 'NT', 'AD', 'VV', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'CD', 'VV', 'VV'], ['NT', 'VV', 'VV', 'NN', 'P', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['AD', 'SP', 'PN', 'AD', 'AD', 'VV', 'AD', 'VA', 'SP'], ['NR', 'NT', 'AD', 'VV', 'P', 'VA', 'DEC', 'NN', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV'], ['NN', 'AD', 'VV', 'NR', 'LC', 'VV', 'NN'], ['PN', 'VV', 'NR', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['VV', 'VV', 'AD', 'VV', 'CD', 'NN', 'NN', 'NN', 'VA'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'NR', 'DEG', 'VA', 'VV'], ['NT', 'NT', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'NR'], ['NR', 'NN', 'JJ', 'DEG', 'NN', 'NN', 'VC', 'VV', 'NN', 'VV'], ['NR', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'AS', 'NR', 'NN', 'VV', 'NN', 'VV', 'NN', 'MSP', 'VV', 'NR', 'DEG', 'NN'], ['NN', 'VE', 'NN', 'NN', 'VE', 'NN', 'VV', 'ETC', 'DT', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'VV', 'VV', 'AD', 'VV', 'NN'], ['NR', 'VV', 'P', 'AD', 'VA', 'DEC', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'CD', 'NN', 'VV', 'VV', 'AD'], ['AD', 'LC', 'VV', 'NN', 'NN', 'ETC', 'NN', 'NN', 'AD', 'VA'], ['NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'AS', 'JJ', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'NR', 'NN', 'AD', 'P', 'NR', 'NR', 'VV', 'AD', 'JJ', 'AD', 'AD', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VV', 'DER', 'AD', 'VA', 'VV', 'PN', 'SB', 'VV', 'AS', 'NN', 'VV'], ['AD', 'AD', 'PN', 'AD', 'VV', 'VV', 'NN', 'SP'], ['NR', 'NR', 'AD', 'VV', 'NN', 'NN'], ['NR', 'NT', 'AD', 'P', 'PN', 'DEG', 'NN', 'VV', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VA', 'VV', 'P', 'NR', 'DT'], ['NR', 'NN', 'VV', 'NN', 'VA'], ['CD', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NN', 'VC', 'NR', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VV', 'NN', 'AD', 'VV'], ['AD', 'VC', 'VV', 'VV', 'VV', 'NR', 'VV', 'AD', 'AD', 'AD', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['CD', 'JJ', 'NN', 'VC', 'NN', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['P', 'VV', 'DEC', 'NN', 'P', 'NN', 'VV', 'PN', 'AD', 'AD', 'VV', 'NN', 'NN', 'VV'], ['AD', 'NN', 'AD', 'AD', 'VV', 'DER', 'VA'], ['AD', 'AD', 'AD', 'PN', 'AD', 'VV'], ['NR', 'NR', 'CC', 'NN', 'NN'], ['AD', 'AD', 'AD', 'VV', 'NN', 'SP'], ['AD', 'AD', 'P', 'NN', 'VV', 'NN', 'VE', 'NN', 'AD', 'VE', 'NN', 'VV'], ['NN', 'CD', 'NN', 'AD', 'VV'], ['CD', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'CC', 'NR', 'AD', 'AD', 'VV', 'PN', 'PN', 'VE', 'NN', 'VV', 'VV', 'SP'], ['CS', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'AD', 'AD', 'AD', 'VV', 'AD', 'DEV', 'VV', 'AD'], ['PN', 'AD', 'VV', 'DER', 'VV', 'NN', 'NN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'AS', 'AD', 'AD', 'VV'], ['AD', 'VC', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'VV', 'SP'], ['CS', 'NN', 'AD', 'VV', 'AD', 'VV'], ['NR', 'CD', 'CD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'VV', 'VV', 'VV', 'AD', 'VV', 'NR', 'DEG', 'NN'], ['OD', 'NN', 'AD', 'VC', 'NN', 'VV'], ['AD', 'NR', 'VA', 'DER', 'AD', 'AD', 'VV'], ['NN', 'NN', 'AD', 'VV', 'NT', 'AD', 'VE', 'NN', 'DT', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'LC', 'PN', 'AD', 'AD', 'AD', 'VV', 'NN', 'AD', 'NN', 'AD', 'VV', 'AS'], ['NR', 'NN', 'AD', 'VA', 'DEV', 'VV', 'CD', 'NN', 'NN', 'VC', 'PN', 'VV', 'DEC', 'PN', 'PN', 'VV', 'SP'], ['AD', 'VV', 'DEC', 'NN', 'VC', 'VV', 'NN', 'LC', 'VV', 'CD', 'NN', 'AD', 'VA', 'CC', 'AD', 'AD', 'VV'], ['NN', 'NN', 'NN', 'VV'], ['NR', 'VV', 'AD', 'VV', 'VV', 'NT', 'NN', 'VV', 'AS', 'NN', 'NN'], ['VV', 'NR', 'NN', 'DEG', 'NN', 'PN', 'AD', 'AD', 'VV', 'NR', 'DEG', 'NN'], ['VV', 'VV', 'NN', 'VV', 'NN', 'VV', 'PN', 'DEG', 'NN', 'SP', 'VV', 'AS', 'JJ', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'AD', 'AD', 'VV', 'AS', 'VV', 'AD', 'NN', 'VA'], ['NN', 'DEG', 'NN', 'AD', 'VA', 'AD', 'VV', 'VV', 'DEC', 'AD', 'VA'], ['NN', 'VV', 'AD', 'JJ', 'NN', 'VV', 'VV', 'NN'], ['NN', 'VV', 'P', 'NR', 'NN', 'VV', 'AD', 'VV', 'VA'], ['AD', 'VA', 'DEC', 'NN', 'NN', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'NN', 'VV', 'AS', 'PN', 'CC', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VV', 'VV', 'SP', 'SP'], ['NN', 'P', 'NR', 'VV', 'NR', 'P', 'NR', 'VV', 'NR'], ['AD', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'VC', 'AD', 'VV', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['P', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'NR', 'AD', 'VV'], ['P', 'NN', 'AD', 'NR', 'NR', 'ETC', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VC', 'VV', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['NT', 'VE', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV', 'PN', 'MSP', 'VV', 'DEC', 'NN'], ['AD', 'VV', 'NR', 'NR', 'NN', 'DEG', 'NN', 'DEG', 'NN', 'NN'], ['NT', 'NT', 'AD', 'VV', 'NN', 'NN', 'NR', 'NN', 'NN'], ['PN', 'VC', 'AD', 'VV', 'NN', 'NN', 'SP'], ['NR', 'NN', 'AD', 'NN'], ['JJ', 'NN', 'LC', 'CD', 'AD', 'NN', 'VV', 'DEC', 'NN', 'VC', 'NN', 'VV', 'VV', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'VV'], ['AD', 'SB', 'NR', 'NN', 'VV', 'DEC', 'NR', 'NN', 'NN', 'AD', 'VV', 'VC', 'AD', 'VV'], ['P', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'NN'], ['DT', 'CD', 'NN', 'VV', 'NN', 'P', 'NN', 'LC', 'VV'], ['NT', 'NR', 'VV', 'P', 'NN', 'AD', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN'], ['P', 'P', 'NR', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VC', 'AD', 'VA', 'SP'], ['P', 'VE', 'NN', 'NN', 'CC', 'VE', 'NN', 'NN', 'NN', 'PN', 'P', 'VV', 'VV', 'AS', 'VA'], ['AD', 'NR', 'VV', 'DT', 'NN', 'NN', 'NR', 'NN', 'NN'], ['AD', 'AD', 'P', 'NN', 'CC', 'NN', 'VV', 'DEC', 'NN', 'VA', 'VA'], ['VV', 'PN', 'DEG', 'NN', 'NN', 'VV', 'VV', 'M'], ['NN', 'NN', 'AD', 'VV', 'NN', 'CC', 'AD', 'VV'], ['AD', 'NN', 'VV', 'AD', 'VV', 'NR', 'DEG', 'NN'], ['VE', 'NN', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'CD'], ['AD', 'NN', 'VV', 'NN', 'DEC', 'PN', 'AD', 'VV'], ['NN', 'AD', 'SB', 'VV', 'AS', 'NN', 'VV'], ['NT', 'P', 'NR', 'NN', 'NN', 'LB', 'NN', 'NN', 'VA', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'AD', 'DT', 'NN', 'CC', 'NR', 'CC', 'NR', 'AD', 'AD', 'VV'], ['NR', 'NT', 'VV', 'NR', 'NN', 'NN', 'NR', 'AD', 'P', 'NN'], ['AD', 'PN', 'AD', 'AD', 'VV', 'NN', 'BA', 'PN', 'DEG', 'NN', 'NN'], ['NR', 'P', 'NR', 'P', 'JJ', 'NN', 'NN', 'NN', 'VV', 'VA'], ['VV', 'NN', 'VC', 'NR', 'VA', 'DEC', 'NN'], ['NR', 'NN', 'NT', 'AD', 'AD', 'VV'], ['NT', 'NT', 'NN', 'VV', 'AD', 'VA', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['AD', 'VV', 'NN', 'NR', 'VV', 'DEC', 'AD', 'VV', 'NN'], ['P', 'NN', 'BA', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'AS', 'PN', 'AD', 'AD', 'DEC', 'NN'], ['P', 'NN', 'NT', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV'], ['DT', 'NN', 'NN', 'VV', 'AD', 'VC', 'NT', 'VV', 'DEC', 'NN'], ['AD', 'NR', 'AD', 'VV', 'VV', 'P', 'NN', 'VA', 'VV', 'NR', 'DEG', 'NN'], ['AD', 'VV', 'VV', 'NR', 'AD', 'AD', 'VV', 'NN', 'VA', 'DEC', 'NN', 'PN', 'NN', 'AD', 'AD', 'VV', 'NN', 'LC'], ['NN', 'VC', 'NT', 'AD', 'VV', 'DEC', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV'], ['PN', 'NN', 'P', 'PN', 'NN', 'AD', 'AD', 'AD', 'AD', 'NN'], ['PN', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VA', 'VA'], ['NT', 'NN', 'VV', 'PN', 'DEG', 'NN', 'VV', 'AS', 'NN', 'PN', 'AD', 'VV', 'PN', 'PN', 'VV', 'PN', 'AD', 'SP'], ['NN', 'VV', 'M', 'NN', 'NN', 'LC', 'NN', 'NN', 'AD', 'VV'], ['PN', 'VV', 'PN', 'VE', 'DT', 'NN', 'SP', 'PN', 'AD', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'NR', 'VV', 'AD', 'P', 'VV', 'NR', 'VV', 'AD', 'VV'], ['AD', 'NR', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN'], ['AD', 'NR', 'NR', 'NN', 'VV', 'DEC', 'VC', 'NR', 'NN', 'NN', 'NR', 'CC', 'NR', 'VV'], ['AD', 'NR', 'NN', 'NN', 'LC', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'AD', 'VE', 'P', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['VE', 'NN', 'VV', 'NN', 'NN', 'VV', 'NT', 'JJ', 'DEG', 'NN', 'NN'], ['NR', 'AD', 'AD', 'AD', 'VV', 'AS', 'PN', 'P', 'NN', 'VC', 'AD', 'VA', 'SP'], ['NN', 'VV', 'AD', 'VV', 'ETC', 'NN'], ['PN', 'VV', 'VV', 'NN', 'CC', 'VE', 'NN'], ['P', 'NR', 'NN', 'NN', 'VV', 'P', 'NN', 'NN', 'VV', 'VV', 'DEV', 'VV', 'AD', 'VA'], ['VV', 'AD', 'PN', 'AD', 'VV', 'P', 'NN', 'AD', 'VV', 'NN', 'NN', 'SP'], ['AD', 'VC', 'NN', 'NN', 'NN', 'CC', 'DT', 'NN', 'PN', 'AD', 'VV', 'AD', 'VA'], ['NN', 'AD', 'VV', 'VV', 'VV', 'LB', 'NN', 'NN', 'VV', 'VV', 'AS', 'NN'], ['NT', 'NN', 'AD', 'VC', 'NN', 'NN', 'VV', 'CD', 'DEG', 'NN', 'NN'], ['AD', 'PN', 'NT', 'AD', 'VV', 'VV', 'DEC', 'AD', 'VV', 'AD', 'VV'], ['AD', 'VV', 'VV', 'VV', 'SB', 'PN', 'NN', 'VV', 'AD', 'VV', 'PN', 'AD', 'BA', 'PN', 'NN', 'VV', 'AD', 'VV', 'DEV'], ['PN', 'VC', 'NN', 'VV', 'VV', 'PN', 'CD', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'SB', 'VV', 'AS', 'VV'], ['NR', 'NT', 'AD', 'VV', 'NR', 'NN'], ['P', 'NT', 'NR', 'AD', 'VV', 'NN', 'CD', 'M', 'CD', 'CD', 'NN'], ['NN', 'VV', 'NT', 'PN', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'AD', 'VA', 'NN', 'VV', 'AS'], ['AD', 'NR', 'NN', 'NN', 'VV', 'AD', 'NN', 'VV', 'DEC', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'PN', 'DEG', 'NN', 'CC', 'VV', 'PN', 'NN'], ['PN', 'AD', 'VV', 'VV'], ['JJ', 'NN', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'VV', 'SP'], ['P', 'NR', 'NN', 'NR', 'VV', 'VV', 'NN', 'LC', 'VV', 'NN', 'LC'], ['VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'LC', 'NR', 'AD', 'VV', 'AS', 'NN'], ['NR', 'NN', 'NR', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'AS', 'JJ', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'NN', 'VV', 'AD', 'VV'], ['NN', 'VV', 'VV', 'P', 'JJ', 'CD', 'DT', 'NN', 'AD', 'VV', 'VV', 'VA', 'DEC', 'NN', 'NN', 'NN'], ['NN', 'VV', 'NN', 'P', 'PN', 'DT', 'NN', 'NN', 'PN', 'PN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'SP'], ['NN', 'AD', 'VV', 'DEC', 'NN', 'LC', 'AD', 'DT', 'NN', 'VV', 'AS', 'M', 'AD', 'CD', 'NN', 'AD', 'VV'], ['LB', 'NN', 'VV', 'DER', 'AD', 'VA', 'DEC', 'NN'], ['P', 'NR', 'NN', 'DT', 'NN', 'VV', 'NR', 'NR', 'ETC', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'AD', 'VV', 'SP'], ['NN', 'AD', 'PN', 'AD', 'VV', 'AS'], ['NR', 'AD', 'AD', 'VV', 'NN'], ['NT', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'AD', 'JJ', 'NN', 'VV', 'NN', 'LC', 'CD', 'NN', 'NN'], ['PN', 'AD', 'PN', 'AD', 'VV', 'PN'], ['NN', 'NR', 'CC', 'NR', 'NN', 'VV', 'P', 'NT', 'VV', 'VV'], ['AD', 'AD', 'VV', 'AD', 'VV', 'VV', 'NR', 'NN', 'NN', 'NN'], ['AD', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'VV', 'NN', 'VV'], ['AD', 'DT', 'NN', 'VC', 'NN'], ['NN', 'CC', 'NN', 'LC', 'VV', 'NN', 'JJ', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NN', 'AD', 'VC', 'CD', 'JJ', 'DEG', 'NN', 'CC', 'NR', 'NN', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['CS', 'NN', 'SP', 'AD', 'VV', 'PN', 'VC', 'AD', 'P', 'NR', 'PN', 'VV', 'SP'], ['NN', 'P', 'NN', 'NN', 'VA', 'AD', 'NN', 'NN', 'AD', 'VC', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'LC', 'VV', 'PN', 'DEG', 'NN', 'LC', 'VV', 'PN', 'VV'], ['P', 'NN', 'NN', 'VV', 'NR', 'NN', 'NN', 'VV', 'NN', 'VA'], ['NR', 'NN', 'CC', 'VV', 'NN', 'DEC', 'DT', 'NN', 'AD', 'P', 'JJ', 'NN', 'VV'], ['P', 'NN', 'VA', 'SP', 'NT', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'M', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'NN', 'SP', 'NN', 'VV', 'PN', 'DEG', 'NN', 'VA', 'SP'], ['NT', 'OD', 'CD', 'CD', 'CD', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'AD', 'VV'], ['NT', 'NR', 'VV', 'AS', 'NN', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'DT', 'NN', 'NN', 'DT', 'NN', 'AD', 'AD'], ['VV', 'NN', 'NN', 'DEC', 'VC', 'NR', 'NR', 'NR', 'NN', 'NN'], ['VA', 'LC', 'VV', 'VV', 'JJ', 'NN', 'CC', 'JJ', 'NN', 'ETC'], ['NN', 'NR', 'VV', 'VV', 'PN', 'VV', 'NN', 'SP', 'AD', 'VV', 'VV', 'VV', 'NN'], ['NN', 'AD', 'VV', 'DEC', 'NN', 'VV', 'LC', 'NN', 'AD', 'AD', 'VA'], ['NR', 'JJ', 'NN', 'NR', 'NN', 'NN', 'NN', 'P', 'VV', 'NN', 'VV'], ['NN', 'AD', 'VV', 'AS', 'NR', 'NN', 'NR', 'CC', 'NR', 'ETC', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'VE', 'PN', 'VV', 'DT', 'AD', 'VV', 'SP', 'PN', 'AD', 'VV', 'AS', 'CD', 'NN', 'AD', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'NN', 'VV'], ['NR', 'AD', 'VV', 'VV', 'NT', 'VE', 'NN', 'NN', 'VV', 'AD', 'NN'], ['VE', 'NN', 'AD', 'AD', 'NN', 'AD', 'VV'], ['PN', 'DEG', 'NN', 'VV', 'AS', 'NN', 'NN'], ['P', 'VV', 'NR', 'DEC', 'NN', 'SB', 'VV', 'DEC', 'NN', 'VV', 'NN', 'SB', 'VV', 'VV', 'VV'], ['NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'VV', 'PN', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'NR', 'NR', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN'], ['NN', 'NN', 'VV', 'VV', 'LC', 'PN', 'VV', 'VV', 'NN', 'VV'], ['VV', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VE', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'VV', 'VV', 'AS', 'NN', 'NN', 'NN', 'LC'], ['NT', 'VV', 'NR', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['PN', 'VV', 'CD', 'M', 'AD', 'AD', 'VV', 'VV', 'M', 'NR', 'SP', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NN', 'CC', 'NR', 'P', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'NR', 'NR', 'VV'], ['AD', 'P', 'NN', 'NN', 'P', 'PN', 'AD', 'JJ', 'CD', 'NN', 'AD', 'VV', 'AS', 'PN', 'CD', 'NN'], ['NN', 'NT', 'AD', 'VV', 'VV', 'AS', 'NN', 'VV'], ['NT', 'NN', 'AD', 'VV', 'AS', 'JJ', 'NN', 'VV', 'VV', 'VV'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VC', 'NN'], ['NN', 'LC', 'NN', 'DEC', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'AD', 'VV', 'NN', 'SP', 'VA', 'CC', 'PN', 'VA', 'VV', 'AD'], ['VV', 'VV', 'NT', 'CD', 'NN', 'VV', 'VV', 'SP', 'AD', 'VV', 'VV', 'AS', 'AD', 'VV', 'VV'], ['DT', 'NN', 'NN', 'VV', 'NR', 'DEG', 'NR', 'NT', 'AD', 'NN', 'AD', 'VE'], ['PN', 'DEG', 'NN', 'AD', 'VA', 'NR', 'CC', 'PN', 'AD', 'VA', 'PN', 'AD', 'VA'], ['PN', 'AD', 'AD', 'VV', 'DT', 'NN', 'VE', 'CD', 'VV', 'VC', 'DT', 'NN', 'VA', 'SP'], ['PN', 'VV', 'VV', 'PN', 'DEG', 'NN', 'AD', 'PN', 'AD', 'VV', 'VV', 'VV', 'DT', 'NN'], ['NN', 'NN', 'CD', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEG', 'NN', 'NN'], ['DT', 'DEG', 'NT', 'CD', 'CD', 'CD', 'NN', 'NN', 'NN', 'AD', 'SB', 'NN', 'VV'], ['NN', 'NN', 'AD', 'CC', 'VE', 'NN'], ['P', 'NT', 'NN', 'LC', 'NN', 'AD', 'VA', 'NN', 'NN', 'LB', 'NN', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VV', 'SP'], ['NT', 'NN', 'AD', 'VV', 'CD', 'M', 'VV', 'NN'], ['PN', 'VV', 'NN', 'PN', 'AD', 'VV', 'AD', 'VV', 'PN', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['NN', 'LC', 'VE', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VE', 'NN'], ['PN', 'VV', 'DT', 'CD', 'NN', 'VV', 'DER', 'VV', 'NR', 'NN'], ['PN', 'NN', 'DEG', 'NN', 'VC', 'NN', 'NN', 'NN', 'NN'], ['PN', 'VV', 'BA', 'NN', 'NN', 'VV', 'VV', 'JJ', 'NN', 'NN'], ['PN', 'VV', 'P', 'JJ', 'P', 'JJ', 'NN', 'VC', 'VV', 'VV', 'PN', 'DEG', 'NN', 'SP'], ['AD', 'AD', 'VV', 'LC', 'PN', 'AD', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'DEC', 'AD', 'JJ', 'JJ', 'NN'], ['VV', 'PN', 'VV', 'PN', 'NN', 'AD', 'VA', 'AD', 'AD', 'VV'], ['NR', 'VV', 'VC', 'P', 'P', 'JJ', 'DEG', 'NN', 'VV'], ['NN', 'NN', 'NN', 'AD', 'VC', 'NN', 'LC', 'DEG', 'NN', 'NN', 'AD', 'VE', 'NN'], ['DT', 'NN', 'VV', 'NN', 'CD', 'CD', 'CD', 'VV', 'NN', 'CD'], ['NR', 'NR', 'AD', 'VV', 'NR', 'P', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'NN', 'LC', 'VV', 'VE', 'NR', 'NN', 'VV', 'DEC', 'NN'], ['NR', 'CD', 'CD', 'NT', 'NT', 'NT', 'VV', 'VV', 'NR'], ['AD', 'AD', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN', 'AD', 'AD', 'VV'], ['VV', 'NN', 'DEC', 'VV', 'CC', 'JJ', 'AD', 'PN', 'DEG', 'NN'], ['AD', 'NR', 'AD', 'VV', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'SP', 'AD'], ['PN', 'VA', 'PN', 'DEG', 'NN', 'PN', 'VV', 'PN', 'AD', 'LB', 'NN', 'VV', 'VV', 'SP'], ['NN', 'P', 'PN', 'VV', 'AS', 'CD', 'M', 'NN', 'DT', 'NN', 'VV'], ['AD', 'PN', 'P', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'DEC', 'CD', 'NN', 'NN'], ['PN', 'VV', 'MSP', 'VV', 'NN', 'VV', 'LC', 'VV', 'VV', 'P', 'NN', 'VV', 'NN', 'VV'], ['PN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'PN', 'DT', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VA'], ['VV', 'PN', 'AD', 'AD', 'DT', 'NN', 'DT', 'NN', 'AD', 'VV', 'LC', 'VV', 'VV'], ['NT', 'VV', 'VV', 'DT', 'JJ', 'NN', 'VV', 'AS', 'PN', 'AD', 'VV', 'VV', 'AS', 'NN'], ['VV', 'P', 'NN', 'NN', 'NN', 'CC', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'AD', 'VA'], ['VV', 'VV', 'NR', 'VV', 'AS', 'NT', 'PN', 'AD', 'VC', 'NN'], ['VV', 'NN', 'NN', 'CD', 'CD', 'CD', 'M', 'CD', 'CD', 'NN'], ['NR', 'AD', 'AD', 'VV', 'DEV', 'VV', 'AS', 'CC', 'VV', 'P', 'NN', 'NN', 'LC'], ['PN', 'VV', 'NN', 'AD', 'VC', 'NN'], ['VE', 'DT', 'NN', 'VV', 'VV', 'NR', 'VE', 'JJ', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'CD', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'PN', 'VV', 'VV', 'VV', 'AD', 'NN', 'VV', 'JJ', 'NN', 'VC', 'JJ', 'DEG', 'NN'], ['AD', 'PN', 'P', 'AD', 'VV', 'VV', 'NR', 'NN', 'JJ', 'NN', 'ETC', 'NN', 'P', 'NN', 'VV'], ['PN', 'VV', 'NN', 'P', 'PN', 'SP'], ['NT', 'DEG', 'NR', 'NN', 'NN', 'NR', 'AD', 'NR', 'DEG', 'NN', 'VV', 'NR'], ['PN', 'VV', 'NN', 'VV', 'VV', 'NN', 'VC', 'P', 'NN', 'NN', 'VV', 'NN', 'BA', 'NN', 'VV', 'AS'], ['NT', 'NR', 'NN', 'AD', 'VV', 'AS', 'JJ', 'NN'], ['AD', 'NN', 'SP', 'VV', 'P', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'NR', 'VC', 'NR', 'NN', 'NN', 'NN'], ['DT', 'CC', 'VA', 'DEV', 'AD', 'VV', 'NN', 'NN', 'SP'], ['P', 'NN', 'VV', 'NN', 'VC', 'NR', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'VV', 'VV', 'VV', 'DT', 'NN', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'NN', 'P', 'NN', 'DEG', 'NN', 'AD', 'JJ'], ['NR', 'DEG', 'NN', 'AD', 'VV', 'NR', 'AD', 'VV', 'CD', 'NN'], ['PN', 'DEG', 'NN', 'P', 'NN', 'LC', 'SB', 'VV', 'AS', 'CD', 'NN'], ['CD', 'VV', 'NN', 'DEC', 'NN', 'P', 'NN', 'VV', 'VV', 'AD', 'P', 'NN', 'VV'], ['AD', 'P', 'DT', 'NN', 'VV', 'NN', 'AD', 'P', 'SB', 'NN', 'VV', 'AS', 'NN'], ['AD', 'NN', 'VV', 'AD', 'SB', 'VV', 'AD', 'VE', 'NN', 'VV', 'NR', 'AD', 'NN', 'VV', 'DER', 'VV'], ['AD', 'NN', 'NN', 'VA', 'NN', 'DEG', 'NN', 'VV', 'P', 'JJ', 'NN', 'AD', 'VA', 'VA'], ['NR', 'VV', 'CD', 'NN', 'AD', 'P', 'NN', 'LC', 'AD', 'P', 'NN', 'LB', 'NN', 'VV'], ['NR', 'VV', 'BA', 'PN', 'VV', 'AD', 'AD', 'AD', 'SB', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NN', 'DEG', 'NN', 'VA'], ['AD', 'NR', 'AD', 'VV', 'AS', 'P', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'LC', 'NN', 'NN', 'VV', 'CD', 'NN', 'JJ', 'NN', 'VV'], ['AD', 'NR', 'VV', 'NN', 'LB', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'VV'], ['NR', 'VV', 'NR', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'VV', 'AD', 'DT', 'NN', 'VV', 'DEC', 'VA'], ['AD', 'CD', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NR'], ['AD', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VE', 'NN', 'P'], ['NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'PN'], ['NN', 'NN', 'VA', 'DER', 'CD', 'NN', 'P', 'PN', 'NN', 'AD', 'VA'], ['P', 'NN', 'DEG', 'NN', 'NR', 'VV', 'NN', 'VV'], ['PN', 'NN', 'VE', 'NN', 'AD', 'AD', 'VV'], ['NN', 'NR', 'VV', 'VV', 'PN', 'AD', 'AD', 'NN'], ['AD', 'AD', 'VV', 'P', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'LC', 'DT', 'NN', 'AD', 'VV', 'SP'], ['NN', 'VV', 'NN', 'AD', 'AD', 'P', 'VV', 'M', 'VV', 'NN', 'CC', 'NN'], ['NR', 'NR', 'P', 'NN', 'BA', 'PN', 'VV', 'VV', 'VV', 'NR', 'NN'], ['P', 'P', 'NN', 'AD', 'VA', 'LC', 'NR', 'P', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VA'], ['CD', 'VV', 'DEC', 'NN', 'LC', 'CD', 'VC', 'JJ', 'NN', 'NN', 'CD', 'VC', 'AD', 'NN', 'VV', 'VV', 'AS'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'VC', 'CD', 'NN', 'P', 'VV', 'VV', 'AD', 'VV', 'NR', 'DEC', 'NN'], ['AD', 'NN', 'VA', 'NN', 'VA'], ['AD', 'NR', 'CC', 'NR', 'NN', 'NN', 'VA'], ['NT', 'VV', 'VV', 'VV', 'NN', 'VV'], ['NN', 'NN', 'VV', 'NN', 'VV'], ['AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'VA', 'NN', 'NN'], ['AD', 'PN', 'VV', 'AD', 'DEV', 'VV', 'VA'], ['P', 'NR', 'AD', 'P', 'PN', 'P', 'PN', 'DEG', 'NN'], ['NN', 'VV', 'JJ', 'NN', 'DEC', 'NN'], ['VV', 'NN', 'VV', 'SB', 'NN', 'VV'], ['AD', 'VE', 'CD', 'NN', 'PN', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'VA'], ['NN', 'AD', 'AD', 'LB', 'PN', 'VV', 'SP'], ['NN', 'VV', 'NR', 'NN', 'NN', 'NR'], ['JJ', 'NN', 'NN', 'LC', 'P', 'JJ', 'NN'], ['NN', 'LC', 'DEG', 'NN', 'NN', 'AD', 'NN', 'AD', 'VC', 'VA', 'SP'], ['VE', 'JJ', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'AD', 'AD', 'VV', 'VV'], ['PN', 'AD', 'NN', 'NN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'P', 'VV', 'P', 'NT', 'NT', 'LC', 'VV', 'AD', 'CD'], ['VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'NN'], ['NT', 'NR', 'AD', 'AD', 'VV', 'VV', 'VC', 'NR', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['PN', 'AD', 'VV', 'NR', 'NN', 'VV', 'P', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['AD', 'AD', 'PN', 'VV', 'PN', 'VV', 'PN', 'VV', 'AS', 'AD', 'CD', 'NN', 'AD', 'VV', 'DEC', 'VC', 'PN'], ['JJ', 'DEG', 'NN', 'NN', 'NN', 'NN', 'CC', 'JJ', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'VE', 'DT', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'P', 'NN', 'LC', 'VV', 'NN', 'VV', 'NN', 'MSP', 'VV', 'NN'], ['VV', 'PN', 'DEG', 'NN', 'NN', 'PN', 'VV', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NT', 'VV', 'AD', 'AD', 'VV', 'SP'], ['P', 'DT', 'NN', 'MSP', 'VV', 'DT', 'NR', 'AD', 'VC', 'VA', 'SP'], ['NR', 'VV', 'JJ', 'NR', 'AD', 'VV', 'AS', 'NN', 'LC', 'AD', 'VV', 'PN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['NR', 'JJ', 'DEV', 'VV', 'AS'], ['DT', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NR', 'NN'], ['PN', 'VV', 'AS', 'DT', 'NN', 'AD', 'VV', 'AD', 'VV', 'PN', 'NN'], ['NT', 'NN', 'NN', 'SB', 'VV', 'NN'], ['VV', 'NN', 'VA', 'AS', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'AD', 'VV'], ['P', 'VV', 'NR', 'NN', 'DEG', 'NN', 'AD', 'VA', 'VV', 'NN', 'NN', 'CC', 'NN'], ['AD', 'VV', 'VV', 'NN', 'VA', 'DEC', 'NN', 'VC', 'VV', 'NR', 'NR', 'NN', 'DEG', 'NN', 'NN'], ['P', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['PN', 'VV', 'PN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'DT', 'VV', 'VV', 'NN'], ['AD', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'P', 'CD', 'NN', 'VV', 'CC', 'AD', 'VV', 'VV', 'NN', 'NN'], ['VV', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'NR', 'NN', 'VA'], ['JJ', 'NN', 'NN', 'VC', 'NN', 'VV', 'AS', 'AD', 'VV', 'DEC', 'NN'], ['NR', 'AD', 'VV', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV'], ['P', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'NN', 'VV'], ['P', 'VV', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'VV', 'SP'], ['NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['VV', 'NN', 'VV', 'NN', 'CC', 'NN', 'VE', 'NN', 'VV', 'VV'], ['VV', 'NR', 'VV', 'DT', 'NN', 'P', 'NN', 'VV', 'NN', 'LC', 'AD', 'AD', 'VV'], ['AD', 'NR', 'VV', 'DEC', 'VC', 'NN', 'CC', 'CC', 'NN'], ['NN', 'VC', 'NN', 'CC', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'VV', 'P', 'NN', 'LC', 'P', 'NN', 'DEG', 'NN', 'DEC', 'NN', 'VA', 'DEV', 'VV', 'AS', 'NN'], ['VV', 'NN', 'AD', 'VA', 'NR', 'VV', 'NN', 'NN', 'VV', 'AD', 'BA', 'NN', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'VV', 'LC', 'NR', 'NN', 'AD', 'VV', 'AS', 'CD', 'M', 'NN'], ['AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'ETC', 'JJ', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NT', 'NT', 'NT', 'NN', 'NN'], ['AD', 'NN', 'AD', 'JJ', 'CC', 'JJ', 'CS', 'VA', 'DEC', 'NN', 'AD', 'VC', 'AD', 'VV', 'PN', 'VV', 'DEC'], ['VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'NR', 'DEG', 'NN', 'AD', 'CC', 'AD', 'VA'], ['NN', 'LC', 'DEG', 'NN', 'NN', 'VV', 'NR', 'AD', 'DEV', 'NR', 'NN'], ['AD', 'VV', 'NN', 'VV', 'PN', 'PN', 'VC', 'NN', 'SP'], ['NR', 'VV', 'NN', 'AD', 'NN', 'VV', 'AS'], ['AD', 'VV', 'NR', 'AD', 'VV', 'VV', 'NR', 'NR', 'DEG', 'NN'], ['NR', 'VV', 'AD', 'P', 'DT', 'NN', 'NN', 'LC', 'DEG', 'NN', 'MSP', 'VV', 'PN', 'JJ', 'NN'], ['VC', 'NN', 'NN', 'PN', 'BA', 'NN', 'VV', 'AS', 'NN', 'PN', 'AD', 'VA'], ['OD', 'NN', 'NN', 'NN', 'NN', 'DT', 'NN', 'VV', 'AS', 'NN'], ['NT', 'AD', 'VV', 'NR', 'NN', 'NN', 'NN', 'DEG', 'AD', 'VV'], ['NR', 'AD', 'SB', 'VV', 'AS', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN', 'SP'], ['AD', 'VA', 'DEC', 'VC', 'AD', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'VA', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VC', 'PN', 'AD', 'VA', 'DEC', 'NR', 'NR', 'NN'], ['NR', 'VV', 'NN', 'NR', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'DEG', 'NN', 'NN', 'AD', 'VA', 'NR', 'NN', 'OD', 'M', 'CC', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AS', 'PN', 'DEG'], ['NR', 'NN', 'P', 'NN', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VV', 'VV', 'PN', 'VC', 'VV', 'NN'], ['NN', 'AD', 'VC', 'VA', 'CC', 'VA', 'DEC', 'NN'], ['AD', 'JJ', 'DEG', 'NN', 'VC', 'VA'], ['VA', 'AD', 'VV', 'VV', 'NN'], ['P', 'NN', 'NN', 'NN', 'NN'], ['AD', 'PN', 'AD', 'AD', 'VV', 'PN', 'SP', 'IJ'], ['VV', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NR', 'VV', 'NN', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'VE', 'M', 'NN'], ['DT', 'NN', 'DEG', 'NN', 'LB', 'VV', 'JJ', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['AD', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'VV'], ['NR', 'BA', 'DT', 'M', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'NN', 'CC', 'AD', 'JJ', 'DEG', 'NN'], ['NN', 'NN', 'CC', 'NR', 'NN', 'NN', 'NN', 'NN', 'VV'], ['AD', 'NN', 'AD', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN'], ['NR', 'NN', 'NR', 'AD', 'LB', 'VV', 'NN', 'CC', 'SB', 'VV'], ['P', 'VV', 'AS', 'NN', 'NN', 'VA', 'DEC', 'NN', 'NR', 'AD', 'AD', 'VV', 'AS', 'AD', 'NN'], ['AD', 'VV', 'DEC', 'NN'], ['CD', 'AD', 'AD', 'VV', 'AS', 'DEC', 'NN', 'CD', 'VA', 'CC', 'VA', 'VA', 'DEC', 'NN'], ['NN', 'AD', 'JJ', 'NN', 'BA', 'PN', 'VV', 'VV', 'SP', 'PN'], ['NN', 'LC', 'VV', 'NR', 'VV', 'DEC', 'NN'], ['DT', 'NN', 'PN', 'P', 'NR', 'VE', 'NN', 'VV'], ['VV', 'AS', 'NN', 'NN', 'P', 'NR', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'AS', 'AD', 'P', 'PN', 'VV'], ['VA', 'DEC', 'NN', 'P', 'NN', 'AD', 'VV'], ['NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'VE', 'NN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'LC', 'VE', 'JJ', 'NN', 'VA', 'DEV', 'VV', 'NN', 'AD', 'AD', 'VV', 'AS'], ['NN', 'NN', 'VV', 'NN', 'LC', 'P', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'PN', 'P', 'PN', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['AD', 'VV', 'LC', 'NN', 'AD', 'AD', 'VV'], ['AD', 'AD', 'VV', 'VV', 'NN', 'ETC', 'NN'], ['AD', 'NT', 'VE', 'NN', 'P', 'DT', 'NN', 'AD', 'VV', 'NN', 'VV'], ['VV', 'AS', 'NN', 'DEG', 'NN', 'NN', 'VV', 'AD', 'AD', 'CD', 'NN', 'AD', 'VA'], ['AD', 'P', 'P', 'NN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VA'], ['NR', 'NN', 'LC', 'VE', 'JJ', 'NN', 'DEC', 'AD', 'AD', 'AD', 'AD', 'VV', 'NN', 'VA'], ['CS', 'VE', 'NN', 'VV', 'P', 'NN', 'CC', 'JJ', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV'], ['AD', 'NR', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN'], ['PN', 'VV', 'AD', 'P', 'NT', 'VV', 'NN', 'VV', 'CD', 'NN', 'VV', 'NN'], ['CD', 'NN', 'AD', 'VV', 'NR', 'NN', 'VC', 'NN', 'VV', 'AS', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'P', 'AD', 'NN', 'VV', 'AD', 'NN', 'SP', 'PN', 'AD', 'P', 'AD', 'VV'], ['AD', 'PN', 'AD', 'CD', 'NN', 'AD', 'VA', 'SP'], ['NR', 'CC', 'NR', 'VC', 'NN', 'DEG', 'NN', 'P', 'NR', 'NR', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV'], ['NR', 'M', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'AS', 'NN', 'VV', 'AS', 'NN'], ['NN', 'DEG', 'NN', 'VC', 'JJ', 'NN', 'VA', 'VA', 'AD', 'VA', 'VV'], ['AD', 'NN', 'LC', 'DEG', 'NR', 'AD', 'P', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VA'], ['NN', 'VV', 'VV', 'NN', 'VV'], ['AD', 'DT', 'AD', 'VV', 'NN'], ['P', 'NN', 'NN', 'CC', 'NN', 'VA', 'NN', 'AD', 'SB', 'VV', 'NN', 'NN'], ['NT', 'NR', 'NN', 'P', 'VV', 'NN', 'VV', 'NN', 'VV', 'NR'], ['AD', 'CD', 'NN', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VV', 'PN', 'PN', 'AD', 'VV', 'PN', 'VV'], ['NN', 'AD', 'VV', 'AS', 'NN', 'CC', 'NN'], ['VV', 'DEC', 'NN', 'VV', 'NN'], ['NN', 'NN', 'AD', 'AD', 'MSP', 'PN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VC', 'M', 'NN'], ['VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['AD', 'NN', 'AD', 'VV', 'PN'], ['AD', 'NN', 'NN', 'NT', 'AD', 'CD', 'NN', 'VV', 'VV', 'AS', 'NN', 'LC'], ['JJ', 'CD', 'NN', 'VC', 'NN', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['NR', 'PN', 'DEG', 'NN', 'AD', 'VA', 'IJ'], ['NN', 'NN', 'VV', 'LC', 'AD', 'VA'], ['PN', 'VV', 'AD', 'AD', 'VV'], ['VV', 'NR', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['PN', 'NN', 'AD', 'AD', 'AD', 'VA', 'SP'], ['AD', 'VV', 'VV', 'P', 'NN', 'NN'], ['AD', 'VV', 'JJ', 'NN', 'NN'], ['AD', 'NR', 'AD', 'VV', 'NN', 'VV', 'VV', 'AS', 'PN', 'DEG', 'NN', 'NN'], ['AD', 'VE', 'VE', 'VV', 'DEC', 'DT', 'NN', 'PN', 'VV', 'NN', 'AD', 'VA'], ['AD', 'DT', 'NN', 'NN', 'VV', 'DEC', 'AD', 'CD', 'NN', 'VV'], ['AD', 'AD', 'VV', 'DEC', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NR', 'AD', 'AD', 'VV', 'AD', 'AD', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'VV', 'AS', 'NN'], ['VV', 'NN', 'VV', 'NN', 'JJ', 'NN', 'VA', 'DEC', 'NN', 'NN'], ['AD', 'NN', 'AD', 'VV', 'VV', 'CD', 'NR', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'NN', 'LC', 'DEG', 'NN', 'AD', 'VA', 'VA'], ['NN', 'DEG', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NR', 'DT', 'AD', 'VA'], ['NR', 'NT', 'VV', 'NR', 'NN', 'VA', 'DEC', 'PN', 'VV', 'NN'], ['AD', 'VV', 'VV', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'NN'], ['AD', 'NR', 'P', 'NR', 'NN', 'VV', 'VV', 'NT', 'AD', 'VV', 'PN'], ['NN', 'VV', 'NN', 'LC', 'CD', 'NN', 'VV', 'NN', 'VV', 'CD', 'JJ', 'DEG'], ['NT', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VA'], ['NN', 'VE', 'NN', 'AD', 'P', 'NN', 'VE', 'NN', 'NN', 'VV'], ['AD', 'NN', 'AD', 'AD', 'BA', 'VA', 'DEC', 'NN', 'VV', 'NR'], ['P', 'VV', 'AD', 'VV', 'VV', 'DEC', 'NN', 'LC', 'VV', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VV', 'NN', 'PN', 'AD', 'AD', 'VA', 'DEV', 'VV', 'JJ', 'DEG', 'NN'], ['VV', 'AS', 'NN', 'DEG', 'NN', 'CD', 'LB', 'NN', 'VV', 'NN', 'VV', 'VV'], ['AD', 'NN', 'AD', 'P', 'NN', 'NN', 'LC', 'VV', 'NN', 'NN'], ['AD', 'PN', 'VV', 'NN', 'VA', 'AD', 'VA', 'NN'], ['NR', 'VV', 'NN', 'VV', 'LC', 'AD', 'VV', 'AD', 'VV', 'NR', 'VV', 'AS', 'NN', 'LC', 'PN'], ['AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'VV', 'AS', 'VV', 'PN', 'VV', 'VV'], ['PN', 'DEG', 'NN', 'P', 'NN', 'VV', 'NN', 'LC', 'VV', 'AS', 'AD', 'AD', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'SP'], ['PN', 'VE', 'JJ', 'NN', 'PN', 'VE', 'NN', 'DEC', 'NN', 'AD', 'VV'], ['VV', 'AS', 'AD', 'AD', 'VA', 'VV', 'AD', 'AD', 'VE', 'AD', 'VV', 'NR'], ['AD', 'VV', 'PN', 'DEC', 'NN', 'AD', 'VV', 'JJ', 'DEG', 'NN'], ['NN', 'VV', 'AD', 'P', 'VV', 'NN', 'AD', 'NR', 'NN', 'VV', 'NN', 'AD', 'VV'], ['P', 'NN', 'NN', 'VV', 'NN', 'P', 'NN', 'NR', 'NN', 'NN', 'NR', 'NN'], ['CD', 'OD', 'NN', 'VV', 'NR', 'VV', 'SP'], ['DT', 'M', 'JJ', 'NN', 'VV', 'VV', 'AD', 'VA'], ['NR', 'VC', 'M', 'JJ', 'NN'], ['NN', 'VA', 'DEV', 'VV'], ['VC', 'P', 'PN', 'AD', 'AD', 'AD', 'VV', 'VE', 'AD', 'CD', 'NN', 'SP'], ['AD', 'VV', 'VV'], ['NN', 'NN', 'VV', 'P', 'NT', 'NT', 'LC', 'VV', 'NN'], ['NN', 'SP'], ['AD', 'IJ', 'VV', 'NN', 'SP'], ['NN', 'NN', 'AD', 'AD', 'LB', 'PN', 'NN', 'VV', 'AS', 'NR'], ['P', 'PN', 'NT', 'VV', 'AD', 'SP'], ['VV', 'NN', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'AS', 'VV'], ['AD', 'AD', 'VV', 'NN'], ['VA', 'VA', 'DEC', 'AD', 'SP'], ['AD', 'VV', 'PN', 'DEG', 'NN', 'MSP'], ['NN', 'NN', 'VV', 'CD', 'M'], ['AD', 'VV', 'VV', 'PN', 'VV'], ['NR', 'SP', 'SP', 'AD'], ['NN', 'DEG', 'NN', 'NN', 'NN'], ['PN', 'AD', 'VC', 'NN', 'NN', 'P', 'PN', 'VV', 'DEC', 'NN'], ['DT', 'AD', 'VV', 'PN', 'SP'], ['NN', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'AD', 'CD', 'NN'], ['AD', 'VC', 'AD', 'VV', 'PN', 'SP'], ['PN', 'VV', 'PN', 'AD', 'VV', 'AS'], ['NR', 'P', 'NR', 'VV'], ['VV', 'VC', 'VV', 'NR', 'NN', 'SP'], ['NN', 'NN', 'NN'], ['AD', 'AD', 'VV', 'VV', 'SP'], ['NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VE', 'CD', 'VA', 'SP'], ['VV', 'MSP', 'AD', 'VC', 'JJ', 'NN'], ['NT', 'NN', 'AD', 'VV'], ['VV', 'VV', 'PN', 'PN', 'VV', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'SP'], ['NN', 'LC', 'DEG', 'NN', 'NN'], ['NR', 'AD', 'VV', 'AD'], ['VV', 'P', 'PN', 'VV', 'SP'], ['PN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NN'], ['AD', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'NN'], ['PN', 'AD', 'VV', 'DEC', 'NN'], ['VV', 'NR', 'PN', 'AD', 'VV', 'AS'], ['NR', 'NN', 'VC', 'NN'], ['NR', 'NN', 'NN', 'P', 'PN', 'DEG', 'NN', 'VV', 'AS', 'AD'], ['NN', 'VV', 'VV', 'PN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'SP'], ['AD', 'NN', 'AD', 'VV'], ['NN', 'NN', 'NR'], ['PN', 'AD', 'VV', 'VA', 'SP'], ['PN', 'VC', 'PN', 'DEG', 'CD', 'NN'], ['NR', 'AD', 'AD', 'VV', 'AS'], ['VV', 'NN', 'NR', 'VV'], ['NR', 'AD', 'VV', 'AS'], ['VV', 'PN', 'AD', 'VV', 'PN', 'SP'], ['PN', 'AD', 'VE', 'NN', 'VV', 'PN'], ['PN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'PN', 'SP'], ['PN', 'AD', 'VV', 'P', 'NN'], ['NR', 'NN', 'VV', 'P', 'OD', 'P', 'NT', 'NT', 'CD', 'CD', 'M'], ['VV', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NR', 'VV', 'VV'], ['AD', 'VV', 'AD', 'VE', 'NN'], ['PN', 'NN', 'VA', 'SP'], ['NN', 'VV', 'VV', 'AD'], ['VV', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VA', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NN', 'SP'], ['NT', 'VA', 'VA'], ['VV', 'VV', 'AD', 'VV', 'IJ'], ['AD', 'VV', 'AD', 'VV'], ['IJ', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'NN', 'VV', 'AS'], ['PN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['VV', 'NN', 'DEC', 'JJ', 'NN', 'VV', 'VV', 'VV', 'VV'], ['P', 'NN', 'LC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NR', 'NR', 'NN'], ['AD', 'VV', 'SP', 'NN'], ['P', 'PN', 'DEG', 'NN', 'CC', 'NN', 'SP'], ['AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'VV', 'VV', 'AS', 'VV'], ['NN', 'AD', 'VE', 'AD', 'JJ', 'SP'], ['PN', 'AD', 'AD', 'VV'], ['NN', 'NN', 'NN', 'NN'], ['AD', 'NN', 'NN', 'VA'], ['PN', 'VV', 'AD', 'VV'], ['VV', 'AD', 'VV', 'SP'], ['VV', 'CD', 'AD', 'VC', 'VV', 'NN'], ['VV', 'PN', 'VV', 'NR'], ['NN', 'LC', 'VV', 'NN'], ['PN', 'VV', 'VV', 'NN', 'SP'], ['NR', 'NN', 'VV', 'VV', 'VA'], ['VV', 'PN', 'VV', 'NR', 'NN'], ['VV', 'AS', 'VV', 'PN', 'VV', 'SP'], ['PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV', 'VV', 'PN', 'VC', 'NN', 'DT', 'NN'], ['P', 'NN', 'NN', 'DEG', 'NN', 'VE', 'CD', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'P', 'VV'], ['NT', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV'], ['VV', 'NN', 'LC', 'NR', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'LC'], ['NN', 'AD', 'AD', 'AS', 'NN', 'LC', 'VV', 'VV', 'AD', 'VA', 'DEC', 'NN', 'VV', 'AD', 'AD', 'VA'], ['AD', 'NN', 'CC', 'NN', 'NN', 'AD', 'VC', 'NN'], ['NN', 'BA', 'PN', 'DEG', 'NN', 'NN', 'VV', 'AS', 'VV', 'NN', 'VV', 'P', 'NN', 'AD', 'VV'], ['NR', 'VV', 'NN', 'NN', 'NN', 'JJ', 'NN', 'VV', 'VV'], ['AD', 'P', 'JJ', 'PN', 'AD', 'NR', 'AD', 'AD', 'VV'], ['AD', 'NR', 'DEG', 'VA', 'VV', 'VA', 'VV'], ['AD', 'P', 'NR', 'NN', 'LC', 'CD', 'NN', 'VV', 'AS', 'CD', 'AD', 'VV', 'DEC', 'NN', 'NN'], ['NN', 'P', 'VV', 'DEC', 'NN', 'LC', 'VV'], ['AD', 'NT', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VC', 'NN', 'JJ', 'NN', 'VV'], ['AD', 'NR', 'NN', 'AD', 'VV', 'AD', 'P', 'DT', 'NN', 'VV'], ['VV', 'PN', 'VV', 'VV', 'AS', 'CD', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'VE', 'NN', 'SP'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['AD', 'NN', 'P', 'NN', 'CD', 'CD', 'SB', 'VV'], ['NR', 'DEG', 'NN', 'AD', 'NN', 'VV', 'NN', 'VV', 'DEC', 'CD', 'NN'], ['PN', 'P', 'NN', 'VV', 'AS', 'NN'], ['P', 'NN', 'VV', 'DEC', 'NN', 'VC', 'VA', 'SP'], ['PN', 'AD', 'VV', 'P', 'VV', 'NN', 'DEG', 'NN', 'LC', 'VV', 'VC', 'NN', 'CC', 'NN', 'MSP', 'VV'], ['CC', 'PN', 'VV', 'DEC', 'NN', 'AD', 'PN', 'AD', 'VV', 'PN', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['PN', 'VV', 'PN', 'VC', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN'], ['PN', 'NT', 'AD', 'AD', 'AD', 'VV', 'NN', 'NN'], ['P', 'NN', 'NN', 'NN', 'NR', 'VV', 'AS', 'NN', 'NN', 'CC', 'JJ', 'NN', 'NN'], ['PN', 'P', 'NN', 'VV', 'VV', 'VV', 'PN', 'DEG', 'NN', 'NN', 'MSP', 'VV', 'AS', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV', 'DEC', 'NR'], ['AD', 'VV', 'VV', 'AD', 'VV', 'AD', 'VV', 'VA', 'DEC', 'NN', 'NN', 'SP'], ['NT', 'NT', 'CC', 'NT', 'NR', 'NN', 'JJ', 'NN', 'VV'], ['NT', 'DEG', 'NR', 'NN', 'NR', 'AD', 'AD', 'P', 'DT', 'CD', 'NN', 'DEG', 'NN', 'LC', 'VV', 'VV'], ['AD', 'VV', 'SP', 'PN', 'VV', 'VV', 'PN', 'VV', 'VE', 'NN', 'VV', 'VV', 'VV', 'VV', 'VV'], ['CD', 'NN', 'AD', 'AD', 'VV', 'NR', 'AD', 'VV', 'CD', 'NN', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VV', 'NN', 'VV', 'NN', 'NN'], ['NR', 'VV', 'AS', 'NR', 'DEG', 'VV'], ['NT', 'LB', 'NN', 'NN', 'VV', 'DER', 'AD', 'NT', 'DT', 'NN', 'AD', 'VV', 'PN', 'PN', 'VV', 'VV'], ['AD', 'VV', 'NR', 'VV', 'NN', 'CS', 'VV', 'VV', 'AD', 'AD', 'VC', 'JJ', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NR', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'P', 'NT'], ['P', 'NR', 'VV', 'NN', 'NN', 'VV', 'AS', 'NR', 'CC', 'NN', 'VA', 'DEC', 'NN', 'NN'], ['NN', 'LC', 'VV', 'AS', 'NN', 'DEG', 'NN', 'VV', 'DER', 'VV', 'NN', 'VV', 'NT', 'VV'], ['VV', 'CD', 'NN', 'NN', 'AD', 'VA', 'DEC', 'NN', 'NN', 'NN'], ['AD', 'PN', 'AD', 'VV', 'AS', 'NN', 'VV', 'VV', 'AD', 'VV'], ['P', 'NN', 'NN', 'DEG', 'NN', 'VV', 'P', 'NN', 'NN', 'NN'], ['PN', 'VV', 'VV', 'VV', 'CD', 'DEG', 'NN', 'CC', 'NN', 'NN', 'NN'], ['AD', 'VV', 'NN', 'P', 'NN', 'NN', 'LC', 'AD', 'AD', 'AD', 'VV', 'AS', 'NN', 'NN'], ['AD', 'CS', 'NR', 'VV', 'VV', 'AD', 'VV', 'SP', 'PN', 'AD', 'VV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'AS', 'CD', 'IJ', 'DEG', 'NN', 'NN'], ['NR', 'DEG', 'NN', 'NT', 'AD', 'VV'], ['VV', 'NN', 'NN', 'VA', 'AD', 'VA', 'VC', 'VV', 'NN', 'NN'], ['AD', 'P', 'NN', 'LC', 'VV', 'NR', 'NN', 'JJ', 'AD', 'VA', 'AD'], ['NN', 'NN', 'VV', 'AS', 'AD', 'AD', 'VV', 'VV', 'AS', 'PN'], ['AD', 'VC', 'VV', 'NR', 'NN', 'VV', 'NN', 'VA', 'DEC', 'NN', 'VV'], ['P', 'NN', 'CC', 'NN', 'DEG', 'JJ', 'NN', 'LC', 'NN', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'LB', 'NN', 'VV', 'AS'], ['AD', 'BA', 'JJ', 'NN', 'VV'], ['VV', 'AD', 'VV', 'VV', 'SP'], ['AD', 'AD', 'VE', 'PN', 'VA', 'SP'], ['PN', 'P', 'NN', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NR', 'NN', 'VV'], ['P', 'VE', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'VV'], ['JJ', 'NN', 'P', 'NN', 'AD', 'VV', 'NN', 'LC', 'VV', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'DEG', 'JJ', 'NN', 'CD', 'M', 'LC', 'AD', 'AD', 'VV', 'NR', 'NN', 'NN'], ['VA', 'SP', 'PN', 'VV', 'VV', 'NN', 'JJ', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'VV', 'PN'], ['JJ', 'NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'AD', 'AD', 'VA', 'DEC', 'VV', 'NN'], ['VV', 'AD', 'NN', 'LC', 'NR', 'AD', 'P', 'NR', 'VV', 'NR', 'VV', 'NN', 'JJ', 'NN'], ['VV', 'PN', 'VV', 'AD', 'AD', 'VV', 'SP', 'PN', 'VV', 'AS'], ['NR', 'VV', 'NR', 'NR', 'NN', 'VV', 'NN', 'NN'], ['AD', 'P', 'NN', 'NN', 'VV', 'JJ', 'NN', 'VV'], ['AD', 'AD', 'VV', 'AD', 'AD', 'AD', 'VV'], ['AD', 'VV', 'VV', 'DT', 'M', 'NN', 'DEG', 'NN', 'SP'], ['NN', 'LC', 'AD', 'VV', 'NN', 'JJ', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'AS', 'AD', 'AD', 'BA', 'PN', 'NN', 'VV', 'AS', 'SP'], ['NN', 'PN', 'VV', 'PN', 'VV', 'SP'], ['PN', 'AD', 'PN', 'AD', 'VV', 'DEC', 'NR', 'JJ', 'NN', 'SP'], ['NN', 'VV', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'P', 'NN'], ['AD', 'NR', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'JJ', 'NN', 'VV', 'VV', 'JJ', 'NN', 'DEG', 'JJ', 'NN'], ['JJ', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'PN', 'VV', 'DEC', 'VC', 'NN', 'PN', 'VV', 'VV', 'PN', 'AS', 'NN'], ['NR', 'VV', 'DEC', 'NN', 'AD', 'AD', 'P', 'NT', 'NN', 'NN'], ['NN', 'NT', 'P', 'NR', 'VV', 'NN', 'VA', 'VV', 'AS', 'NN', 'AD', 'VV', 'AD', 'VV'], ['VA', 'DEC', 'NN', 'P', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'PN', 'VV', 'VV', 'VV', 'NN', 'SP'], ['AD', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'LC', 'AD', 'VV', 'AS', 'DT', 'NN'], ['NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'AD', 'CD', 'CD'], ['AD', 'NR', 'NN', 'DEG', 'DT', 'NN', 'NN', 'AD', 'SB', 'VV', 'VV'], ['VV', 'DT', 'NN', 'SP', 'VV', 'VV', 'LC', 'AD', 'AD', 'VV', 'VV', 'AD'], ['AD', 'VV', 'AS', 'NT', 'LC', 'PN', 'AD', 'VV', 'VV', 'AD', 'AD', 'AD', 'VV', 'VV'], ['DT', 'NN', 'AD', 'VV', 'AS', 'AS', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AD', 'VE', 'VV', 'DEC'], ['NN', 'VV', 'PN', 'VA', 'DEC', 'NN', 'MSP', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'NR', 'AD', 'VV', 'NN', 'NN', 'NR'], ['P', 'NN', 'LC', 'VV', 'AD', 'NR', 'VV', 'NN'], ['AD', 'AD', 'BA', 'JJ', 'NN', 'VV', 'P', 'NR', 'NN'], ['VV', 'NN', 'VV', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN'], ['NN', 'AD', 'SB', 'NN', 'VV', 'AS', 'JJ', 'NN'], ['AD', 'NR', 'P', 'JJ', 'DEG', 'NN', 'LC', 'VV'], ['AD', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NT', 'NN', 'DEG', 'NN', 'VC', 'NN', 'VV', 'P', 'NN'], ['PN', 'AD', 'VC', 'NN', 'AD', 'BA', 'PN', 'NN', 'LC', 'DEG', 'NN', 'VV', 'VV', 'AS', 'PN', 'DEG', 'NN', 'LC'], ['VV', 'NN', 'VV', 'NN', 'CC', 'NN', 'NN', 'VC', 'AD', 'VV', 'VV', 'DEC', 'VV', 'NN'], ['NN', 'AD', 'VC', 'VA', 'DEC', 'NN', 'SP'], ['PN', 'VV', 'AD', 'VV', 'VV', 'VV'], ['NN', 'NN', 'DEG', 'NN', 'JJ', 'VV', 'P', 'NN', 'VV', 'JJ', 'NN'], ['NN', 'AD', 'VV', 'NN', 'LC', 'VV', 'AS', 'NN'], ['VV', 'AS', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'NT', 'VV'], ['PN', 'DT', 'NN', 'VV', 'AS', 'LC', 'AD', 'VV', 'VA'], ['AD', 'VC', 'AD', 'VV', 'DEC', 'NN'], ['JJ', 'NN', 'VV', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'PN', 'VC', 'NN'], ['NR', 'DEG', 'NN', 'NN', 'VV'], ['AD', 'NN', 'VV', 'P', 'OD', 'NN', 'NN'], ['NT', 'NT', 'NN', 'NN', 'VV', 'P', 'NN'], ['PN', 'AD', 'VC', 'VE', 'NN', 'VV', 'VV', 'NN', 'DEC', 'AD', 'NN', 'AD', 'VV', 'NN', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['AD', 'PN', 'NN', 'DEG', 'NN', 'AD', 'VA', 'VA', 'CC', 'PN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NR', 'NR', 'VV', 'NN', 'VV', 'P', 'PN', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'VV', 'DT', 'NN', 'VC', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN', 'SP'], ['NN', 'NN', 'VV', 'VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['PN', 'AD', 'NR', 'SB', 'VV', 'VC', 'NR', 'NN', 'NN'], ['AD', 'NR', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV'], ['NN', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['DT', 'NN', 'VV', 'NR', 'NR', 'NR', 'CD', 'NN', 'LC'], ['P', 'CD', 'CD', 'CD', 'NN', 'DEG', 'NN', 'NN', 'VV', 'VA', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'AD', 'VE', 'CD', 'VV', 'VV', 'SP'], ['CD', 'NN', 'VV', 'NN', 'AD', 'VC', 'CD', 'CD', 'CD', 'NN'], ['AD', 'PN', 'DEG', 'NN', 'VV', 'P', 'DT', 'CD', 'NN', 'LC', 'VV', 'NR', 'NN'], ['VV', 'NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV'], ['NN', 'NN', 'VV', 'PN', 'AD', 'PN', 'AD', 'VV', 'AD', 'PN', 'AD', 'AD', 'AD', 'VV', 'DT', 'NN'], ['NT', 'NR', 'NN', 'NN', 'NR', 'VV', 'AD', 'SB', 'VV', 'VV', 'NN'], ['NR', 'NN', 'VA', 'DEV', 'P', 'NN', 'DEC', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV'], ['VV', 'NT', 'PN', 'VV', 'SP', 'AD', 'P', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV'], ['VV', 'VV', 'VV', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'P', 'P', 'NR', 'NN', 'NN'], ['VV', 'JJ', 'NN', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'VV', 'VV', 'NR', 'AD', 'AD', 'VV', 'VV', 'AS', 'NR', 'NN'], ['NR', 'AD', 'DEV', 'VA', 'AD', 'VV', 'P', 'NN', 'LC'], ['PN', 'NN', 'VV', 'NN', 'AD', 'VC', 'VV', 'P', 'NR', 'NN', 'VV'], ['NR', 'NN', 'CC', 'NN', 'NN', 'AD', 'VC', 'NN', 'DT', 'NN', 'NT', 'NN'], ['AD', 'PN', 'DEG', 'NN', 'SP', 'PN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'CC', 'NN'], ['NT', 'NT', 'NN', 'P', 'NR', 'VV', 'VV', 'NR'], ['DT', 'CD', 'NN', 'AD', 'VC', 'AD', 'VV', 'SP'], ['PN', 'AD', 'VV', 'PN', 'NN', 'DEG', 'NN'], ['DT', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'AD', 'VC', 'P', 'VA', 'DEC', 'NN', 'NN', 'NN'], ['VV', 'VV', 'NN', 'VV', 'DEC', 'VC', 'PN', 'VV', 'SP'], ['AD', 'VV', 'NN', 'PN', 'P', 'PN', 'VV', 'NN'], ['NR', 'VV', 'DER', 'AD', 'AD', 'VV', 'PN'], ['AD', 'NR', 'NN', 'NN', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VC', 'NN', 'VV'], ['NN', 'NN', 'VV', 'AS', 'NN', 'DEG', 'JJ', 'NN'], ['JJ', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'DT', 'JJ', 'DEG', 'NN', 'VV', 'AS'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NN', 'CC', 'VV', 'NN'], ['JJ', 'NN', 'NN', 'NN', 'NN', 'ETC'], ['AD', 'VC', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'PN', 'VV', 'VV', 'VV', 'NN', 'VE', 'NN', 'VV', 'PN', 'VC', 'VV', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['NR', 'VV', 'NN', 'CC', 'NR', 'AD', 'P', 'DT', 'NN', 'AD', 'VA', 'AD', 'PN', 'DEG', 'NN', 'VV'], ['VV', 'NR', 'NN', 'VV', 'NR', 'VC', 'CD', 'VE', 'JJ', 'NN', 'JJ', 'DEG', 'NN'], ['P', 'NN', 'SB', 'VV', 'DEC', 'NN', 'VC', 'VV', 'NR', 'NN', 'CD', 'DEG', 'NN', 'NN'], ['NN', 'AD', 'VV', 'DEC', 'CD', 'NN', 'LC', 'NR', 'NN', 'AD', 'P', 'NN', 'VE', 'AS', 'NN'], ['AD', 'VV', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'AD', 'AD', 'DEV', 'VV'], ['AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'P', 'PN', 'VV', 'PN', 'CS', 'VV', 'AD', 'NN'], ['PN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'AS', 'VV', 'PN', 'VV', 'SP'], ['DT', 'NN', 'VV', 'DEC', 'VC', 'P', 'NN', 'CC', 'NN', 'DEC', 'VV', 'CC', 'VV'], ['NT', 'DEG', 'NN', 'AD', 'VC', 'NN', 'NN', 'VV', 'VV', 'NN', 'DEC', 'PN', 'VV', 'VV'], ['NR', 'AD', 'AD', 'PN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN'], ['VV', 'VV', 'PN', 'DT', 'NN', 'AD', 'VV', 'VV', 'PN'], ['NR', 'VV', 'NR', 'DEG', 'NN'], ['PN', 'VV', 'VV', 'SB', 'VV', 'P', 'NN', 'NN', 'LC', 'DEG', 'NN', 'NN'], ['AD', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'P', 'NN', 'VV', 'NN', 'AD', 'VV', 'P', 'NN'], ['NN', 'VV', 'NR', 'NN', 'LC', 'VV', 'LC', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV'], ['VV', 'AS', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'CD', 'NT', 'NN', 'P', 'NN', 'VV', 'NN'], ['AD', 'VV', 'AD', 'LB', 'NN', 'AD', 'P', 'NN', 'DT', 'NN', 'LC', 'NR', 'AD', 'VV'], ['NR', 'VC', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'PN', 'VV', 'VV', 'VV'], ['NR', 'VV', 'NT', 'LC', 'DEG', 'NN', 'NN', 'VV', 'VV', 'AD', 'NN'], ['AD', 'NN', 'P', 'AD', 'VV', 'LC', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['NR', 'VV', 'VV', 'PN', 'VV', 'LC', 'VV', 'NN', 'AD', 'AD', 'VE', 'NN'], ['AD', 'PN', 'VV', 'NN', 'VV', 'NR', 'NN', 'NR', 'DEG', 'NN'], ['NT', 'PN', 'NN', 'NN', 'NN', 'DEC', 'NN', 'AD', 'VV', 'CD', 'M'], ['AD', 'PN', 'AD', 'AD', 'VA', 'DEV', 'VV', 'PN', 'AD', 'VV', 'DER', 'AD', 'VV', 'SP'], ['AD', 'P', 'CD', 'NN', 'LC', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV', 'AS'], ['NR', 'NR', 'NN', 'AD', 'VV', 'DT', 'NN', 'CC', 'NN', 'VA'], ['NN', 'VV', 'AS', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NN', 'JJ', 'NN', 'VV', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VA'], ['CS', 'VC', 'NN', 'AD', 'JJ', 'AD', 'AD', 'VE', 'NN', 'VV', 'NN', 'VV'], ['JJ', 'JJ', 'NN', 'AD', 'CD', 'JJ', 'NN', 'DEG', 'NN'], ['CD', 'NN', 'VV', 'PN', 'VV', 'DT', 'DEG', 'AD', 'AD', 'AD', 'NN', 'VV'], ['NN', 'VV', 'NN', 'DEC', 'JJ', 'NN', 'VC', 'CD', 'CD', 'NN', 'CD', 'CD', 'CD'], ['PN', 'AD', 'P', 'PN', 'VV', 'AS', 'CD', 'NN', 'MSP', 'CS', 'AD', 'BA', 'PN', 'VV', 'VV', 'AD', 'PN', 'AD', 'VV', 'PN', 'VV', 'AS', 'PN', 'VV'], ['AD', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS', 'CD'], ['AD', 'NN', 'VV', 'DEC', 'NR', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN'], ['CS', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'VV', 'PN', 'P', 'PN', 'NN', 'LC'], ['P', 'NN', 'VV', 'NN', 'PN', 'VV', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['NR', 'VE', 'CD', 'VV', 'VV', 'JJ', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['AD', 'AD', 'AD', 'AD', 'P', 'NR', 'NR', 'VV', 'LC', 'VV', 'DT', 'M', 'NN', 'VV', 'NN', 'VA'], ['NR', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'NN', 'NN', 'CD', 'NN', 'VV', 'VV', 'PN', 'CD', 'M', 'NN', 'VV', 'DEC', 'NN', 'AD', 'AD', 'AD', 'NN'], ['NT', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VA'], ['AD', 'NR', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['VV', 'AD', 'VV', 'AS', 'NN', 'AD', 'VA', 'CC', 'DT', 'VV', 'DEC'], ['AD', 'VE', 'NN', 'VV', 'AD', 'NN', 'VV', 'VV', 'AD', 'VE', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'VV', 'AD', 'PN', 'AD', 'VV'], ['CD', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN', 'CC', 'NN', 'VV', 'NN', 'NN', 'VV', 'VC', 'VV', 'VV', 'DEC', 'NN'], ['P', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'VE', 'CD'], ['AD', 'VV', 'NN', 'VV', 'PN', 'P', 'PN', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'NR', 'AD', 'BA', 'NN', 'CC', 'NN', 'VV', 'NN'], ['NT', 'VV', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NT', 'DEG', 'NN'], ['P', 'VV', 'NN', 'LC', 'VV', 'P', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN'], ['NN', 'VA', 'VV', 'NN', 'NN', 'ETC', 'PN', 'AD', 'AD', 'AD', 'AD', 'VV', 'VV', 'PN'], ['NR', 'VV', 'NN', 'NN', 'VV', 'CD', 'NN', 'P', 'NN', 'LC', 'VV'], ['PN', 'AD', 'VV', 'PN', 'NN', 'DEG', 'NN', 'NR', 'NN'], ['AD', 'VA', 'VV', 'DT', 'AD', 'AD', 'AD', 'VV'], ['JJ', 'DEG', 'NN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['AD', 'NR', 'VV', 'VV', 'PN', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NN', 'NN', 'NR', 'NN', 'NN', 'NR', 'VV', 'NN', 'VV', 'AS', 'NN'], ['PN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'BA', 'CD', 'NN', 'VV', 'NN'], ['JJ', 'NN', 'P', 'NN', 'VV', 'VV', 'NN', 'NN'], ['PN', 'AD', 'AD', 'VV', 'VV', 'PN', 'VC', 'JJ', 'NN', 'SP'], ['VA', 'DEC', 'JJ', 'NN', 'LC', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'VV', 'P', 'NN', 'NN', 'LC', 'AD', 'VA'], ['AD', 'NR', 'AD', 'P', 'VV', 'NN', 'VV'], ['PN', 'AD', 'NR', 'NN', 'NN', 'DEG', 'NR'], ['NR', 'NN', 'VV', 'VV', 'AD', 'AD', 'VA'], ['NR', 'VV', 'NN', 'VV', 'NN', 'NR', 'VV', 'NN'], ['NR', 'VV', 'NN', 'VE', 'NN', 'NN', 'AD', 'NN', 'VC', 'NN', 'NN'], ['P', 'VV', 'PN', 'SB', 'VV', 'LC', 'CD', 'M', 'NN', 'DEG', 'NN', 'NN', 'VV', 'AS'], ['PN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN', 'AD', 'VE', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['CD', 'NN', 'NN', 'NN', 'VV'], ['NR', 'NN', 'NR', 'NR', 'NN', 'NN', 'VV', 'NN'], ['AD', 'NN', 'AD', 'NR', 'AD', 'NN', 'VV', 'SP', 'PN', 'AD', 'NR', 'SP'], ['NR', 'NN', 'AD', 'VV', 'AS', 'VV', 'VV', 'NN', 'NN', 'NN', 'LC', 'DEG', 'VV', 'NN'], ['NT', 'VV', 'NN', 'VV', 'VE', 'NN', 'VV', 'CC', 'M', 'NN'], ['AD', 'AD', 'VA', 'VV', 'NN', 'P', 'DT', 'NN', 'VV', 'VV', 'AS'], ['AD', 'VV', 'NN', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VA'], ['JJ', 'NN', 'NN', 'LC', 'VV', 'LC', 'DEG', 'NN', 'NN', 'VA', 'CD', 'M', 'JJ', 'NN', 'VV'], ['NN', 'AD', 'VV', 'DEC', 'VC', 'NN', 'NN', 'NN', 'VV', 'NN'], ['NT', 'NR', 'NN', 'NN', 'AD', 'VV', 'AD', 'NR', 'NN', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'NT', 'NN', 'NN', 'P', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AD', 'VA'], ['P', 'NN', 'JJ', 'NN', 'CC', 'NN', 'NN', 'VV', 'PN', 'NN', 'VA'], ['CS', 'CD', 'NN', 'VV', 'VV', 'DER', 'VA', 'AD', 'VA', 'PN', 'PN', 'VC', 'VV', 'NN', 'VA', 'SP'], ['AD', 'VC', 'DT', 'NN', 'VV', 'NR', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['NN', 'VA', 'DEC', 'P', 'P', 'NN', 'NN', 'VV', 'PN', 'VV', 'VA'], ['CD', 'NN', 'DEG', 'NN', 'VV', 'AD', 'AD', 'VV', 'PN'], ['P', 'PN', 'DEG', 'NN', 'LC', 'AD', 'VV', 'CD', 'VV'], ['CS', 'NR', 'VV', 'AD', 'NN', 'SP', 'AD', 'VV', 'P', 'PN', 'VE', 'NN', 'SP'], ['NT', 'NR', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV'], ['NR', 'VE', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN'], ['VA', 'DEC', 'NN', 'VV', 'AS', 'NN', 'VV', 'VV', 'NN'], ['PN', 'PN', 'P', 'NN', 'AD', 'VV', 'P', 'PN', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'LC', 'PN', 'AD', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN'], ['NT', 'VV', 'VV', 'VA', 'AD', 'DEV', 'NN', 'SP', 'AD', 'VV', 'M', 'NN', 'CC', 'NN', 'VA'], ['PN', 'VE', 'CD', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'CD', 'M', 'DEG', 'NN'], ['AD', 'P', 'NR', 'VV', 'DEC', 'NN', 'NN', 'LC', 'NN', 'NN', 'AD', 'VV', 'NN', 'JJ', 'NN'], ['JJ', 'NN', 'NT', 'AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'LC', 'VV', 'AS', 'JJ', 'NN', 'DEG', 'NN'], ['NR', 'VV', 'VV', 'AS', 'NN', 'NN', 'LC', 'VV'], ['VV', 'AD', 'PN', 'VV', 'AS', 'PN', 'VV', 'PN', 'NN', 'VV', 'PN', 'VV', 'VV', 'VV', 'VV', 'VV', 'AD', 'AD', 'VV', 'SP'], ['PN', 'AD', 'VV', 'PN', 'SP', 'AD', 'VV', 'P', 'PN', 'VV', 'VV', 'CC', 'SP'], ['NR', 'NN', 'AD', 'BA', 'JJ', 'NN', 'VV', 'NN', 'NN'], ['DT', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN', 'NN', 'P', 'PN', 'VV', 'DEC', 'NN', 'VV', 'PN'], ['AD', 'AD', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD', 'P', 'NN', 'LC', 'VV', 'NN', 'NN'], ['P', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'NR', 'AD', 'VV', 'NN', 'VV', 'JJ', 'NN'], ['PN', 'VV', 'NN', 'SP'], ['NR', 'NN', 'OD', 'NN', 'NN', 'VV', 'NN', 'NN'], ['P', 'NN', 'DEG', 'JJ', 'DT', 'VV', 'DEC', 'VC', 'AD', 'NN', 'NN', 'CC', 'NN'], ['NN', 'NN', 'VV', 'P', 'NN', 'NN', 'LC', 'VV', 'NN', 'VA'], ['AD', 'DEV', 'NN', 'AD', 'VV', 'AS', 'VV', 'DEC', 'NN', 'NR'], ['NN', 'AD', 'VV', 'AD', 'AD', 'VC', 'P', 'NN', 'NN', 'NN', 'VV'], ['PN', 'P', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'NR', 'DEG', 'NN'], ['NT', 'NT', 'NN', 'NR', 'VV', 'NR', 'NR', 'NR', 'CD', 'NN', 'VV', 'NN'], ['PN', 'NN', 'SP', 'VV', 'AD', 'VA', 'DEC', 'NN', 'MSP', 'VV', 'NN'], ['PN', 'NN', 'VA', 'VV', 'VV'], ['NR', 'NR', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VE', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VV', 'AS', 'PN', 'IJ', 'VV'], ['NR', 'VV', 'LC', 'VV', 'AS', 'AD', 'NN'], ['DT', 'NN', 'VV', 'VV', 'NR', 'VV'], ['NN', 'DEG', 'JJ', 'NN', 'VV', 'AS', 'NN', 'VV', 'P', 'PN', 'AD', 'VV'], ['VV', 'VV', 'VV', 'NN', 'JJ', 'NN'], ['NR', 'AD', 'VV', 'JJ', 'NN'], ['CS', 'PN', 'VV', 'PN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'VV', 'VV', 'DEC', 'NN', 'P', 'NR'], ['PN', 'VV', 'DEC', 'PN', 'AD', 'VV', 'AS', 'PN'], ['NN', 'DEV', 'VV', 'AD', 'VV', 'NN'], ['AD', 'P', 'NT', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'AS', 'NN', 'CD', 'NN'], ['VV', 'NN', 'LC', 'NN', 'AD', 'VV', 'NN', 'NN', 'ETC', 'NN', 'VV', 'VA'], ['CS', 'NN', 'AD', 'P', 'NN', 'NN', 'MSP', 'VV', 'AD', 'VV', 'SB', 'AD', 'VV'], ['NT', 'NT', 'NR', 'NN', 'CC', 'NR', 'NR', 'DEG', 'NN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'NN', 'VA', 'DEC', 'NN', 'VV', 'P', 'NN', 'NN', 'VV', 'AS'], ['NN', 'NN', 'P', 'NN', 'VA', 'DEC', 'NN', 'AD', 'VV'], ['AD', 'P', 'NN', 'NN', 'NN', 'MSP', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'AD', 'AD'], ['AD', 'VC', 'NR', 'NN'], ['NR', 'DEG', 'NN', 'VV', 'AS', 'NR', 'DEG', 'NN', 'NN', 'NN', 'VV', 'PN', 'VV'], ['NR', 'VV', 'NT', 'VV', 'VV', 'NN', 'DEC', 'CC', 'NR', 'NN'], ['VV', 'PN', 'VE', 'VV', 'AS', 'NN', 'VC', 'VA', 'DEC', 'SP'], ['NR', 'PN', 'AD', 'VV', 'PN', 'VV', 'NN', 'SP'], ['DT', 'NN', 'VV', 'NN', 'NN'], ['VV', 'VV', 'AD', 'VV', 'SP'], ['NN', 'LC', 'NN', 'DEG', 'NN', 'VA', 'SP'], ['PN', 'AD', 'NN', 'AD', 'VV', 'VV', 'NN', 'VE', 'CD', 'NN', 'JJ', 'NN', 'VV', 'SP'], ['NN', 'NN', 'P', 'CD', 'CD', 'CD', 'CD', 'CD', 'VV', 'CD'], ['NN', 'VV', 'AS', 'CD', 'JJ', 'CD', 'JJ', 'JJ', 'NN'], ['NN', 'NN', 'LC', 'VV', 'VV', 'NN', 'VV', 'NN'], ['AD', 'VV', 'AS', 'AD', 'NN', 'VV'], ['AD', 'PN', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'P', 'NN', 'VV', 'NN', 'AD', 'VA'], ['AD', 'NR', 'VV', 'DEC', 'CD', 'NN', 'VV'], ['AD', 'VV', 'DEC', 'JJ', 'NN', 'VA'], ['PN', 'AD', 'VV', 'PN', 'VA', 'DEC', 'NN'], ['NR', 'NN', 'PN', 'VV', 'PN', 'SP', 'PN', 'PN', 'AD', 'VE', 'AD', 'NN', 'AD', 'VV', 'VV'], ['NN', 'NN', 'P', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'SP'], ['NN', 'JJ', 'DEG', 'NR', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'VC', 'CD', 'CD', 'CD', 'CD', 'M'], ['AD', 'VV', 'NR', 'NN', 'NR', 'AD'], ['AD', 'NR', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VV', 'AS', 'DT', 'VV', 'NN'], ['VV', 'NN', 'AD', 'SB', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VA'], ['NN', 'JJ', 'NN', 'NN', 'VA'], ['NR', 'NR', 'VV', 'AS', 'NN', 'NN'], ['AD', 'NN', 'AD', 'AD', 'AD', 'NN'], ['VV', 'NN', 'VV', 'NR', 'JJ', 'NN'], ['AD', 'CD', 'VC', 'NN', 'NN', 'VV', 'VA'], ['NN', 'P', 'NN', 'NN', 'NN', 'AD', 'VA'], ['NR', 'NN', 'VV', 'VV', 'AD', 'NN', 'VV', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VV'], ['VV', 'VV', 'NN', 'AD', 'AD', 'VV', 'PN', 'SP', 'CD', 'NN', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NN'], ['AD', 'P', 'CD', 'NN', 'LC', 'DT', 'NN', 'AD', 'P', 'JJ', 'NN', 'AD', 'VV', 'AD', 'VV'], ['DT', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VV', 'LC', 'AD', 'AD', 'P', 'NR', 'VV', 'CD'], ['AD', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN', 'P', 'NN', 'VV', 'VV', 'VV'], ['NR', 'VV', 'NN', 'VV', 'NR', 'NN', 'PN', 'NN', 'DEG', 'NN', 'AD', 'VC', 'VV', 'DEC', 'NN'], ['NT', 'P', 'PN', 'NN', 'VV', 'DEC', 'DT', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'SP'], ['NN', 'LC', 'NN', 'VV', 'NN', 'VA', 'DEC', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'NR', 'NN', 'P', 'NN', 'NN', 'LC', 'VV', 'JJ', 'DEG', 'NN'], ['AD', 'NN', 'AD', 'P', 'CD', 'CD', 'NN', 'VV', 'AS', 'AD', 'CD', 'NN'], ['NN', 'VV', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'NN', 'DEG', 'VV', 'VV', 'NN', 'NN'], ['AD', 'PN', 'VV', 'NN', 'AD', 'VV'], ['VV', 'DEC', 'VC', 'AD', 'AD', 'VV', 'VA', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN'], ['PN', 'AD', 'AD', 'VV', 'SP', 'PN', 'VV', 'VV', 'SP'], ['VV', 'NR', 'AD', 'VV', 'NN', 'VC', 'AD', 'VA', 'VC', 'PN', 'NN', 'LC', 'DEG', 'NN', 'NN'], ['NR', 'NN', 'AD', 'BA', 'AD', 'VV', 'VA', 'DEC', 'NN', 'P', 'PN', 'VV', 'VA', 'VV', 'NN', 'NN'], ['NT', 'NR', 'AD', 'AD', 'P', 'NN', 'ETC', 'NN', 'VV', 'CD', 'NN', 'NN', 'P', 'NN', 'VV'], ['VE', 'NN', 'VC', 'VV', 'VV', 'DT', 'JJ', 'NN', 'SP'], ['VV', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'AS', 'NN'], ['P', 'JJ', 'NN', 'VV', 'LC', 'PN', 'AD', 'VV', 'PN', 'VV'], ['AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VA', 'SP'], ['PN', 'AD', 'VV', 'P', 'NN', 'NN', 'AD', 'VV'], ['NN', 'VV', 'NN', 'NR', 'NN', 'VV', 'NN'], ['P', 'NN', 'DEG', 'PN', 'VV', 'VV', 'AD', 'VA'], ['P', 'CD', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VV', 'AD', 'VA', 'DEC', 'NN'], ['PN', 'NN', 'NN', 'NN', 'NN', 'AD', 'BA', 'PN', 'VV', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['P', 'NN', 'VE', 'CD', 'NN', 'VV', 'VV', 'AS', 'AD', 'VV', 'VV', 'DT', 'NN', 'NN', 'JJ', 'NN', 'VA'], ['PN', 'AD', 'VV', 'DEC', 'AD', 'VV', 'VV', 'VV', 'SP'], ['NR', 'VV', 'DER', 'VA', 'VV', 'VV', 'AS', 'DT', 'NN', 'VV', 'DEC', 'NN'], ['P', 'NN', 'DEG', 'NN', 'VV', 'NN', 'P', 'NT', 'AD', 'DEV', 'VV', 'VV'], ['NN', 'LC', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'VV', 'AS', 'AD', 'VV'], ['OD', 'AD', 'BA', 'NN', 'NN', 'AD', 'M', 'VV', 'CC', 'VV', 'NN'], ['PN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'VV', 'PN', 'IJ', 'PN', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NT', 'P', 'NN', 'NN', 'LC', 'VV'], ['NT', 'LC', 'NR', 'DEG', 'NN', 'NR', 'NN', 'SP'], ['AD', 'NN', 'NT', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'AD', 'VC', 'NN', 'VV', 'VV', 'NN', 'VV'], ['AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN', 'PN', 'AD', 'AD', 'VV'], ['VV', 'NR', 'AD', 'VV', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'NR', 'VV', 'CD', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'AD', 'VV'], ['PN', 'VV', 'AD', 'VV', 'NN', 'P', 'NN', 'NN', 'VV', 'CD', 'NN'], ['NN', 'VV', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN'], ['VV', 'NN', 'AD', 'BA', 'NN', 'NN', 'AD', 'VE', 'NN'], ['P', 'NN', 'NN', 'AD', 'VC', 'NN', 'VA', 'DEC', 'NN'], ['P', 'NN', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['AD', 'VE', 'CD', 'NN', 'VV', 'NN', 'VE', 'CD', 'M'], ['VV', 'NR', 'DEG', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'NN', 'PN', 'AD', 'VV', 'VV'], ['PN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'NN', 'AD', 'VE'], ['PN', 'DEG', 'NN', 'VC', 'NN', 'VV', 'PN', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'PN', 'VV', 'NN'], ['DT', 'NN', 'SP', 'VV', 'AS', 'NN', 'AD', 'BA', 'NN', 'VV', 'P', 'NN', 'LC', 'NN', 'AD', 'VA'], ['AD', 'NR', 'CC', 'NR', 'DT', 'AD', 'AD', 'VV', 'NR', 'VV', 'VV', 'VV', 'PN', 'VV', 'NN'], ['VV', 'VV', 'AD', 'VV', 'VE', 'NN', 'DEC', 'NN', 'VV', 'VV', 'P', 'JJ', 'NN', 'VV', 'NN'], ['AD', 'P', 'NN', 'VA', 'VA', 'DEC', 'NN', 'SP'], ['NN', 'DEG', 'NN', 'SB', 'VV', 'DER', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV', 'AD'], ['PN', 'VA', 'LC', 'AD', 'VV', 'NR', 'DT', 'NN', 'NN', 'NN'], ['PN', 'AD', 'VV', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'NN', 'VV'], ['AD', 'P', 'NN', 'NN', 'NN', 'VV', 'NR', 'DEG', 'NN', 'VV', 'VV'], ['VV', 'LC', 'PN', 'AD', 'AD', 'VV', 'PN', 'NN', 'VV', 'NN', 'VV', 'VV'], ['P', 'VV', 'NN', 'NN', 'VV', 'LC', 'AD', 'VV', 'VV'], ['NN', 'LC', 'NN', 'VV', 'NN', 'P', 'NN', 'VA', 'AD'], ['AD', 'NN', 'AD', 'P', 'PN', 'P', 'NN', 'VV', 'AS', 'NN'], ['IJ', 'AD', 'VV', 'VV', 'AS', 'AD', 'VE', 'M', 'NN', 'NN', 'SP'], ['CC', 'NT', 'NR', 'DEG', 'NN', 'LC', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV'], ['NR', 'NN', 'NN', 'NN', 'VV'], ['AD', 'AD', 'VV', 'DEC', 'NR', 'AD', 'BA', 'NR', 'VV', 'VV', 'VV', 'VV'], ['NR', 'DEG', 'NN', 'AD', 'P', 'NT', 'CD', 'CD', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'NR', 'VV', 'NN'], ['NN', 'P', 'NN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VA'], ['NR', 'NR', 'VC', 'NR', 'NR', 'NN', 'JJ', 'NN', 'DEG', 'NN', 'NN'], ['DT', 'NN', 'VV', 'NN', 'VV', 'DEC', 'AD', 'NR', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'LC', 'VV', 'VV', 'NN', 'DEC', 'NN', 'CC', 'NN'], ['NR', 'AD', 'VV', 'AS', 'NN', 'VV', 'AD', 'AD', 'VV', 'VV', 'NN'], ['AD', 'NN', 'CC', 'CD', 'NN', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'JJ', 'NN', 'NN', 'CC', 'NN', 'NN'], ['NN', 'VC', 'NR', 'NN', 'PN', 'VV', 'NN', 'NN', 'VV', 'JJ', 'NR', 'DEG', 'NN'], ['CD', 'NN', 'CD', 'NN', 'VC', 'NN', 'PN', 'NN', 'VC', 'VV', 'DEC', 'PN', 'AD', 'VV'], ['NT', 'NR', 'VV', 'VV', 'P', 'NN', 'VV', 'NN', 'CC', 'BA', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['AD', 'VV', 'VV', 'VV', 'PN', 'NN', 'VV', 'P', 'PN', 'VV', 'VV'], ['IJ', 'PN', 'DEG', 'NN', 'VV', 'SP'], ['VV', 'AS', 'AD', 'VC', 'AD', 'VA', 'SP'], ['NT', 'NN', 'AD', 'VC', 'VV', 'NN', 'CC', 'NN', 'AD', 'AD', 'VA'], ['VE', 'NN', 'VV', 'NR', 'JJ', 'NN', 'NN', 'AD', 'VV', 'NN'], ['PN', 'VV', 'PN', 'VV', 'AS', 'NN', 'VV', 'NN', 'AD', 'AD', 'VA'], ['NR', 'VV', 'PN', 'NN', 'VV', 'VV', 'VV', 'NR', 'VV', 'VA', 'AD'], ['NN', 'CS', 'AD', 'VV', 'NN', 'AD', 'AD', 'P', 'NR', 'VV', 'NN', 'VV', 'PN', 'VV', 'AD', 'VV', 'NN'], ['AD', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN', 'PN', 'VV', 'PN', 'NN', 'VV', 'AS', 'PN', 'AD', 'VV'], ['NR', 'NT', 'NN', 'VA', 'VV', 'VV', 'NN', 'CC', 'NN'], ['NN', 'LC', 'NR', 'DT', 'NN', 'AD', 'VV', 'BA', 'NN', 'CC', 'NN', 'P', 'PN', 'VV', 'NT'], ['NN', 'NN', 'NN', 'NN', 'VC', 'NR', 'NN', 'NN', 'NR', 'NN', 'DEG', 'CD'], ['VV', 'DT', 'VV', 'VV', 'PN', 'VV', 'PN', 'DEG', 'NN', 'VC', 'AD', 'VA', 'VA', 'DEC', 'NN'], ['PN', 'VV', 'AS', 'NN', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NR', 'NN', 'NN', 'DEG', 'NN'], ['NT', 'NR', 'NN', 'DEG', 'NN', 'NN', 'NN', 'P', 'NR', 'NN', 'VV'], ['AD', 'PN', 'AD', 'VV', 'NR', 'AD', 'VV', 'P', 'NN', 'ETC', 'JJ', 'JJ', 'NN'], ['PN', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'NR', 'VV', 'DEC', 'NN', 'VV', 'DT', 'NN'], ['VV', 'NR', 'NN', 'NN', 'DEG', 'PN', 'AD', 'VV', 'AD', 'VA', 'CC', 'VA'], ['NR', 'JJ', 'AD', 'VV', 'VV', 'CD', 'NN', 'NN', 'DEG', 'NN', 'LB', 'NN', 'VV', 'AS', 'DEC', 'NN', 'AD', 'VV', 'VV', 'SP'], ['AD', 'VV', 'P', 'NN', 'NN', 'DEG', 'NR', 'NN', 'VV', 'CC', 'VV', 'NR'], ['AD', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'P', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'AD', 'VV', 'NN', 'PN', 'AD', 'VV', 'AD', 'AD', 'VV', 'NN', 'DEC', 'VV', 'NN', 'P', 'NN', 'VV'], ['VV', 'LC', 'AD', 'VA', 'AD', 'P', 'NN', 'VV', 'PN', 'SP', 'VV', 'VV', 'PN', 'NN', 'AD', 'VV', 'VV', 'CD'], ['NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'NR', 'CC', 'NR', 'LC', 'AD', 'VC', 'P', 'NR', 'VV', 'NN', 'SB', 'VV', 'VV', 'AD', 'VV', 'AS', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV'], ['VV', 'PN', 'PN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'NN', 'AD', 'AD', 'AD', 'NN', 'PN', 'VV', 'AD', 'VV', 'PN', 'AD', 'VV', 'PN'], ['VA', 'VV', 'DEC', 'NR', 'VV', 'NN', 'NN', 'NN', 'VV'], ['AD', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'SP'], ['OD', 'CD', 'NN', 'NN', 'VC', 'VV', 'NN', 'NN', 'CC', 'NN'], ['VC', 'NN', 'JJ', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'NT', 'AD', 'NT', 'AD', 'AD', 'VA', 'SP'], ['NN', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'AS', 'CD', 'JJ', 'NN'], ['VE', 'NN', 'VV', 'VE', 'NN', 'VV', 'AD', 'VE', 'AS', 'NN', 'VV'], ['P', 'DT', 'NN', 'NN', 'NN', 'AD', 'SB', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'VC', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'JJ', 'NN', 'LC', 'NN', 'VV', 'NN', 'NN'], ['NR', 'VV', 'NN', 'VV'], ['CD', 'NN', 'LC', 'VV', 'AD', 'VV', 'AS', 'NN', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'P', 'VV', 'JJ', 'NN'], ['VV', 'PN', 'AD', 'VV', 'DT', 'NN', 'AD', 'VV', 'AS', 'PN', 'VV', 'AS', 'NN'], ['NN', 'NN', 'DEC', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'VV'], ['P', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'ETC', 'NN', 'AD', 'AD'], ['P', 'NN', 'NN', 'VV', 'VV', 'LC', 'VV', 'NN', 'DEC', 'NN', 'AD', 'BA', 'NN', 'VV'], ['AD', 'AD', 'VV', 'VV', 'DT', 'NN', 'MSP', 'VV', 'NN'], ['AD', 'VC', 'CD', 'CD', 'CC', 'CD', 'M', 'CD', 'JJ', 'DEG', 'NN'], ['CC', 'P', 'P', 'NN', 'VV', 'SP'], ['NR', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV'], ['NN', 'AD', 'P', 'NT', 'VA', 'CD'], ['VV', 'AD', 'NN', 'NR', 'AD', 'VV', 'VV'], ['DT', 'NN', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NT'], ['NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'P', 'PN', 'VV', 'PN', 'DEG', 'NN'], ['NT', 'NN', 'NN', 'AD', 'VV', 'NR'], ['AD', 'NN', 'AD', 'LB', 'NN', 'VV', 'VV', 'AS', 'NN'], ['AD', 'VV', 'AS', 'NN', 'DEG', 'LC', 'NN', 'VV'], ['NR', 'VV', 'VV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'AD', 'VV', 'VV', 'VV'], ['NT', 'NR', 'NN', 'NN', 'NN', 'NN'], ['VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'NR', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'CC', 'NN', 'NN', 'ETC', 'NN'], ['PN', 'DEG', 'NN', 'CC', 'VV', 'PN', 'VV', 'AD', 'VV'], ['AD', 'NN', 'NR', 'AD', 'NN', 'AD', 'NN', 'VV'], ['AD', 'NN', 'VV', 'NN', 'VV', 'NN'], ['VV', 'NN', 'VV', 'CC', 'NN', 'NN', 'NN'], ['DT', 'VV', 'DEC', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VV', 'NN', 'SP'], ['NT', 'NR', 'NN', 'JJ', 'AD', 'AD', 'VV', 'VV', 'SP'], ['AD', 'VV', 'DT', 'NN', 'VV', 'AS', 'LC', 'PN', 'AD', 'VA', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NR'], ['PN', 'AD', 'AD', 'VE', 'PN', 'AD', 'M', 'JJ', 'NN', 'AD', 'VV', 'AD', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'CD', 'NN', 'NN', 'VV', 'AD', 'VV', 'VV'], ['NR', 'NR', 'VV', 'NT', 'NR', 'NR', 'VC', 'NN'], ['NR', 'NN', 'CD', 'NN'], ['NR', 'CD', 'NN', 'P', 'PN', 'DEG', 'NN', 'VV', 'NN', 'VV'], ['PN', 'VC', 'NN', 'VV', 'NN', 'SP'], ['NN', 'VA', 'VA', 'VA'], ['VV', 'NN', 'VV', 'AD', 'VV'], ['AD', 'PN', 'VV', 'VC', 'NN', 'DEG', 'NN', 'AD', 'VV', 'JJ', 'DEG', 'NN'], ['NR', 'NT', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'CD', 'NN', 'NN', 'AD', 'P', 'NT', 'VV', 'P', 'NN', 'NN', 'LC'], ['NT', 'PN', 'NN', 'VV', 'AS'], ['PN', 'AD', 'NN', 'AD', 'VV', 'VV'], ['AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'ETC', 'VV'], ['VA', 'DEC', 'NN', 'VV', 'VV', 'PN', 'IJ'], ['NN', 'VC', 'P', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'VV', 'DEC', 'VV', 'NN', 'NN', 'NN', 'NN', 'ETC'], ['IJ', 'NN', 'SP', 'PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'VV', 'PN'], ['AD', 'NN', 'NN', 'NN'], ['NT', 'NT', 'NR', 'VV', 'VV', 'AS', 'VV', 'CD', 'M', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'OD', 'CC', 'NN', 'CS', 'OD', 'AD', 'PN', 'AD', 'OD', 'SP'], ['NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VA', 'JJ', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'JJ', 'NN', 'NN', 'P', 'JJ', 'NN', 'VV', 'NN'], ['NN', 'AD', 'P', 'NR', 'DT', 'NN', 'LC', 'VV', 'SB', 'VV', 'DEC', 'NN', 'CC', 'NN'], ['AD', 'VV', 'NN', 'VV'], ['NT', 'NN', 'VV', 'VA'], ['AD', 'BA', 'NR', 'VV', 'VV', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['VE', 'PN', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'AD', 'VV', 'AD', 'VV', 'AD', 'VE', 'AS', 'NN'], ['VV', 'NN', 'AD', 'DEV', 'NN', 'NR', 'SP', 'AD', 'VV', 'SP'], ['P', 'VV', 'PN', 'PN', 'AD', 'VV', 'NN', 'VA', 'DEC', 'AD', 'VV', 'VV', 'SP'], ['VV', 'NN', 'LC', 'JJ', 'NN', 'VV', 'M', 'DEC', 'NN', 'AD', 'BA', 'NN', 'VV'], ['P', 'DT', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'DEG', 'NN', 'NN', 'VV'], ['PN', 'AD', 'PN', 'NN', 'M', 'NN'], ['PN', 'AD', 'VA', 'AD', 'VV'], ['AD', 'VA', 'SP', 'PN', 'AD', 'VV', 'AD', 'VA'], ['P', 'NN', 'NN', 'NN'], ['AD', 'VE', 'DT', 'CD', 'NN', 'AD', 'P', 'DT', 'NN', 'SP'], ['NN', 'VV', 'NN', 'VV', 'NN', 'PN', 'SP'], ['PN', 'VV', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'AS', 'SP'], ['PN', 'VV', 'NN', 'CC', 'NN', 'VV', 'VV', 'NN'], ['VV', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'AS', 'NN'], ['JJ', 'NN', 'VV', 'AS', 'NN', 'VV', 'AS', 'NN'], ['JJ', 'DEG', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VV', 'PN', 'VV', 'VV'], ['NN', 'VA', 'VV', 'VV'], ['AD', 'NN', 'VC', 'NN', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'VV'], ['NN', 'NN', 'P', 'NN', 'VE', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'VC', 'NN', 'NN', 'SP'], ['NN', 'AD', 'VV', 'NN', 'VV', 'NR', 'JJ', 'NN', 'VV', 'NN', 'CC', 'NN'], ['NR', 'DEG', 'NN', 'P', 'NR', 'AD', 'JJ', 'NN', 'NR', 'NN', 'NN', 'NR', 'NN', 'NN'], ['PN', 'VC', 'PN', 'NN', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'DEC', 'NN', 'NN'], ['NR', 'CC', 'NR', 'AD', 'VV', 'NN', 'NN', 'VA', 'DEV', 'VV', 'AS', 'NN', 'VV'], ['AD', 'VV', 'AD', 'VC', 'VE', 'NN', 'DEC', 'NN', 'VV', 'AD', 'CC', 'VE', 'NN', 'DEC', 'NN', 'NN'], ['AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV', 'DT', 'NN'], ['JJ', 'NN', 'VV', 'AD', 'AD', 'SB', 'VV', 'IJ', 'VE', 'NN', 'NN', 'SP', 'NN', 'DEC'], ['AD', 'VE', 'VV', 'AS', 'DEC', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN', 'VV'], ['PN', 'NN', 'P', 'PN', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV', 'PN', 'VV', 'AS', 'PN', 'PN', 'DEG', 'NN'], ['AD', 'NR', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'NN', 'VV', 'VV', 'VV', 'LC', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'DT', 'CD', 'VA', 'CC', 'VV', 'VV', 'VV', 'NR'], ['P', 'VV', 'AD', 'VV', 'VV', 'DEC', 'NN', 'PN', 'AD', 'VV'], ['VV', 'NN', 'PN', 'VV', 'PN', 'DEG', 'NN', 'LB', 'NN', 'VV', 'AS'], ['P', 'VV', 'NN', 'VV', 'VV', 'JJ', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'NN', 'LC', 'DEG', 'NN', 'VV', 'NN', 'NN', 'NN'], ['AD', 'P', 'NN', 'NN', 'VV', 'NN', 'ETC', 'NN'], ['VV', 'VV', 'PN', 'VV', 'SP', 'AD', 'VV', 'NN'], ['NN', 'JJ', 'DEG', 'VC', 'NR', 'CC', 'NR'], ['NN', 'DEG', 'NN', 'P', 'NN', 'VA', 'VA'], ['NN', 'VV', 'NT', 'NN', 'NN', 'VV', 'NN', 'AD', 'CD', 'CD', 'CD', 'NN'], ['AD', 'BA', 'NN', 'VV', 'VV', 'AS', 'M', 'NN'], ['NN', 'VV', 'DEC', 'VV', 'PN', 'VC', 'PN', 'NN'], ['VV', 'PN', 'AD', 'VA', 'DEC', 'NN'], ['VV', 'LC', 'PN', 'AD', 'AD', 'VV'], ['AD', 'NR', 'AD', 'VV'], ['NN', 'NN', 'AD', 'VV'], ['AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'NR', 'AD', 'VV', 'NR', 'AD', 'NN', 'VV', 'NN', 'DEG', 'NN', 'VV', 'AS', 'PN', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'NR', 'AD', 'P', 'NR', 'NR', 'DEG', 'NN', 'NN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'AD', 'NN', 'VV', 'P', 'AD', 'VV', 'VV', 'VV', 'AS'], ['NR', 'NN', 'VV', 'NN', 'NR', 'DEG', 'CC', 'VV', 'VA', 'DEC', 'NN', 'VV', 'PN', 'AD', 'VV', 'VV'], ['P', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VC', 'PN', 'VV', 'VV', 'LC', 'VV', 'SP'], ['NR', 'VV', 'DEC', 'VV', 'MSP', 'LC', 'VV', 'AS', 'AD', 'JJ', 'NN', 'AD', 'NN', 'VA'], ['AD', 'DT', 'NN', 'NN', 'VA', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['PN', 'LB', 'CD', 'NN', 'VV', 'NR', 'MSP', 'AD', 'VV', 'DEC', 'JJ', 'DEG', 'NN'], ['VV', 'VA', 'LC', 'VV', 'BA', 'NN', 'CC', 'NN', 'AD', 'VV', 'NN'], ['PN', 'VV', 'PN', 'VV', 'AS', 'AD', 'NT', 'NT', 'PN', 'AD', 'NN', 'AD', 'VA', 'SP'], ['VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV'], ['AD', 'PN', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV', 'VV', 'PN', 'VA'], ['DT', 'JJ', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'VE', 'NN'], ['NN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'NN', 'CC', 'VV', 'NN'], ['NN', 'AD', 'CD', 'NN', 'CD', 'DEG', 'NN', 'NN', 'VV', 'AS', 'PN', 'CD'], ['JJ', 'NN', 'DEG', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VV', 'VA'], ['VE', 'DEC', 'NN', 'AD', 'VV', 'AD', 'P', 'NN', 'NN', 'VV', 'NN', 'P', 'VV', 'NN'], ['JJ', 'VV', 'DEC', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN'], ['P', 'PN', 'DEG', 'NN', 'NR', 'NR', 'NN', 'VV', 'NN'], ['AD', 'NN', 'VV', 'DEC', 'VC', 'NN', 'NN', 'NR', 'NN', 'ETC'], ['NN', 'P', 'NN', 'VV', 'JJ', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VA', 'SP'], ['NN', 'LC', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'NT', 'DEG', 'NN', 'AD', 'VC', 'NN', 'VV', 'VV'], ['CD', 'M', 'NN', 'VV', 'AS', 'NN', 'NN', 'CD', 'IJ'], ['AD', 'VV', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV'], ['VE', 'NN', 'AD', 'VV', 'PN', 'VV', 'AD', 'VV', 'VV', 'PN'], ['AD', 'NN', 'VV', 'P', 'VV', 'NN', 'AD', 'PN', 'AD', 'VV', 'AS'], ['NN', 'NN', 'JJ', 'NN'], ['NR', 'AD', 'P', 'PN', 'VV', 'AS', 'VV'], ['NT', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['VV', 'VV', 'CC', 'NN', 'IJ'], ['P', 'LC', 'AD', 'VA', 'VV', 'LC', 'AD', 'VA', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'AD', 'VA'], ['CS', 'AD', 'AD', 'VV', 'SP'], ['AD', 'VV', 'VV', 'PN'], ['AD', 'VV', 'NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN'], ['DT', 'NN', 'DEG', 'NN', 'AD', 'AD', 'P', 'VV', 'AS', 'DEC', 'NN', 'NR', 'NR', 'NN', 'NN', 'NN'], ['VV', 'NR', 'P', 'NN', 'DEG', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'JJ', 'NN', 'AD', 'VV'], ['P', 'NR', 'JJ', 'NR', 'NN', 'AD', 'SB', 'VV', 'AD', 'VV', 'NN', 'NN'], ['PN', 'VA', 'DEC', 'NN', 'VA', 'DEC', 'NN', 'NR', 'NN', 'LC', 'NN', 'NN', 'VV', 'VA'], ['AD', 'VV', 'P', 'PN', 'VV', 'AD', 'VV', 'PN', 'NT', 'NN', 'DEG', 'NN', 'NN'], ['P', 'VV', 'NT', 'DEG', 'JJ', 'NN', 'NR', 'VV', 'AS', 'AD', 'JJ', 'NN'], ['VV', 'NT', 'DEG', 'VA', 'VA', 'AD', 'VV', 'P', 'NN', 'NN', 'LC', 'VA', 'VV'], ['PN', 'AD', 'VV', 'AS', 'VV', 'DT', 'NN', 'DT', 'NN', 'VV', 'VV', 'AS', 'DT', 'NN', 'SP'], ['NN', 'VV', 'NN', 'NN', 'PN', 'VV', 'VV', 'DEV', 'VV', 'AS', 'CD', 'JJ', 'DEG', 'NN', 'VV', 'VV', 'SP'], ['PN', 'VV', 'PN', 'AD', 'VC', 'VV', 'NR', 'NN', 'DEG', 'DT', 'NN', 'AD', 'VE', 'VA', 'DEC', 'NN', 'NN'], ['NT', 'NR', 'CD', 'CD', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NR', 'VV', 'AD', 'VA', 'DEC', 'VC', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NT', 'AD', 'AD', 'VA', 'AD', 'AD', 'AD', 'VV'], ['CS', 'VV', 'VV', 'NN', 'NN', 'NN', 'VE', 'NN', 'BA', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'AS', 'PN', 'AD', 'VV', 'DEC', 'NN'], ['P', 'NR', 'NN', 'VV', 'NR', 'NN', 'VV', 'NR', 'NR', 'NN', 'NN', 'NN'], ['JJ', 'VV', 'VV', 'PN', 'NN', 'DEG', 'NN', 'VV', 'AS', 'LC', 'DEG', 'NN', 'PN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['PN', 'P', 'NN', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['NN', 'VE', 'PN', 'AD', 'VC', 'AD', 'VA', 'DEC', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'AD', 'VV', 'VV', 'AS', 'AD', 'VV', 'LC', 'VV', 'PN', 'VE', 'PN', 'VV', 'AS'], ['NN', 'ON', 'PN', 'VA', 'DEC', 'NN', 'VV', 'NN', 'VV', 'PN', 'NT', 'VV', 'VV', 'DEC', 'NN'], ['CS', 'NN', 'VE', 'NN', 'VV', 'VE', 'VV', 'P', 'NN', 'NN', 'VV', 'VV', 'P', 'NN', 'NN', 'VA'], ['NN', 'NN', 'AD', 'VV', 'AD', 'AD', 'AD', 'LB', 'NR', 'P', 'NN', 'NN', 'NN', 'VV', 'NT'], ['NT', 'NT', 'DT', 'NN', 'VV', 'AD', 'NR', 'DEG', 'NN', 'NN', 'DEC', 'NN', 'AD', 'VV'], ['CS', 'NN', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'LC', 'PN', 'DEG', 'NN', 'P', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'SP'], ['VA', 'PN', 'VC', 'NN', 'AD', 'PN', 'CC', 'AD', 'JJ', 'NN', 'SP'], ['NN', 'VV', 'NN', 'CC', 'NN', 'ETC', 'VV', 'JJ', 'NN', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['P', 'NN', 'VV', 'DEC', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VC', 'VV', 'NN'], ['P', 'NR', 'NN', 'NR', 'NR', 'NR', 'VV', 'VV', 'NN', 'AD', 'VC', 'VA'], ['AD', 'VV', 'NN', 'P', 'VV', 'NN', 'DEC', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'JJ', 'NN', 'NN'], ['DT', 'NN', 'NN', 'AD', 'NR', 'LC', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'VE', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'AD', 'VA'], ['VV', 'VV', 'BA', 'NR', 'VV', 'VV', 'DT', 'NN', 'DT', 'CC', 'VV', 'NN', 'SP'], ['PN', 'VV', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN', 'NN', 'NN', 'NN', 'AD', 'NN'], ['JJ', 'NN', 'DEG', 'PN', 'NN', 'VV', 'NN', 'P', 'NN', 'P', 'VA', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'AS', 'NR', 'VC', 'NN', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['VC', 'AD', 'VV', 'DEC', 'NN', 'VV', 'NN', 'NN', 'NN', 'CC', 'NN'], ['CD', 'NN', 'CD', 'NN', 'NR', 'NN', 'NN', 'VV', 'NN', 'SB', 'VV'], ['P', 'NN', 'NN', 'VV', 'NN', 'JJ', 'NN', 'NN', 'JJ'], ['PN', 'VV', 'PN', 'VV', 'AD', 'AD', 'SP'], ['NN', 'VV', 'NN', 'NN', 'NN', 'NT', 'NT', 'AD', 'VV', 'NT', 'CD', 'CD'], ['NR', 'NN', 'NN', 'CC', 'AD', 'VV', 'P', 'NN', 'VV', 'NN'], ['NR', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA', 'NN', 'CC', 'NN', 'VV', 'DER', 'AD', 'VA'], ['AD', 'NN', 'DEG', 'NN', 'AD', 'BA', 'PN', 'DEG', 'NN', 'VV', 'VA', 'NN'], ['NT', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'JJ'], ['AD', 'NN', 'VV', 'VV', 'PN', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'NR', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VC', 'NN', 'NN', 'P', 'NN', 'VV', 'LC', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'P', 'NR', 'VV', 'NN', 'VV', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'CD', 'NN', 'VV', 'NN', 'NN', 'NN'], ['VV', 'NN', 'NN', 'NN', 'ETC', 'NN', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN'], ['VV', 'PN', 'VV', 'NN', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['NN', 'AD', 'AD', 'VV', 'NN', 'VA', 'DEC', 'NN', 'DT', 'NN'], ['CD', 'NN', 'AD', 'VV', 'VV', 'AD', 'CD', 'JJ', 'AD', 'VV', 'VV'], ['P', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'LC', 'VV', 'NR', 'VV', 'CC', 'VV', 'NN'], ['NN', 'NR', 'NR', 'AD', 'VV', 'VV', 'AD', 'VV', 'NN', 'VV'], ['PN', 'NN', 'VV', 'MSP', 'VV', 'AD', 'VV', 'PN', 'VV'], ['NT', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['VV', 'NN', 'VV', 'DER', 'AD', 'VV', 'NR', 'NN', 'AD', 'VV', 'NN'], ['VV', 'AS', 'AD', 'NN', 'NN', 'NR', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'AD', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NT', 'VV', 'VV', 'VC', 'AD', 'VV'], ['AD', 'AD', 'VV', 'VV', 'VV', 'BA', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VV', 'LC', 'AD', 'VV', 'VV', 'NN'], ['PN', 'VV', 'AS', 'JJ', 'NR', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['NT', 'AD', 'VV', 'VV', 'NN', 'JJ', 'DEG', 'NN', 'VE', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'AD', 'AD', 'VA', 'SP'], ['AD', 'PN', 'NN', 'VV', 'NN', 'VV', 'PN', 'AD', 'VV'], ['NR', 'NN', 'VV', 'NN', 'LC', 'ETC', 'VV', 'AS', 'CD', 'NN'], ['CD', 'NN', 'AD', 'DEV', 'VV', 'NN', 'NN', 'VA'], ['VV', 'BA', 'NR', 'DEG', 'NN', 'VV', 'P', 'NN', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'DEC', 'VC', 'NR', 'DEG', 'NN', 'NN', 'NN'], ['AD', 'NR', 'NR', 'AD', 'AD', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['AD', 'VV', 'DT', 'NN'], ['AD', 'CD', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN'], ['CD', 'NN', 'DEC', 'NN', 'NN', 'NN', 'VV', 'VC', 'PN', 'DEG'], ['AD', 'P', 'NN', 'NN', 'VV', 'NN', 'JJ', 'NN'], ['VE', 'CD', 'NN', 'VV', 'NR', 'NN', 'NN', 'VC', 'NN', 'LC', 'DEG', 'CD', 'NN'], ['AD', 'NN', 'LC', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VC', 'AD', 'VA', 'DEV', 'VV', 'NN', 'SP'], ['AD', 'VV', 'CD', 'CD', 'NT', 'DT', 'NN', 'AD', 'BA', 'VV', 'CD', 'CD'], ['NN', 'VV', 'VV', 'PN', 'VV', 'DEC', 'VV', 'VV', 'PN', 'VV'], ['NR', 'VV', 'VV', 'PN', 'NN', 'DEC', 'NN', 'NR', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV', 'AD', 'VV', 'AS', 'VV', 'LC', 'BA', 'PN', 'VV'], ['AD', 'DT', 'NN', 'AD', 'AD', 'VV', 'NR', 'NN', 'VV', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'NN', 'NN', 'AD', 'P', 'NN', 'NN', 'P', 'NN', 'NN', 'DEG', 'NN', 'VV', 'JJ', 'NN'], ['NT', 'NN', 'AD', 'VV', 'P', 'NN', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['NN', 'AD', 'VV', 'AS', 'CD', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AS', 'AD', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'P', 'NN', 'VV'], ['PN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['AD', 'VA', 'DEC', 'NN', 'VV', 'NN', 'VV'], ['PN', 'VV', 'NN', 'VV', 'PN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VA', 'AS', 'AD', 'JJ', 'VV', 'NT'], ['AD', 'VV', 'P', 'NN', 'VE', 'PN', 'JJ', 'DEG', 'NN'], ['NT', 'NT', 'VV', 'LC', 'PN', 'AD', 'SB', 'VV', 'VV', 'NN', 'PN', 'VV', 'NN'], ['NR', 'VV', 'PN', 'VV', 'NR', 'NN', 'AD', 'VA', 'AD', 'VV', 'DT', 'NN', 'SP'], ['NN', 'NN', 'AD', 'VV', 'NN', 'VC', 'P', 'NN', 'NR', 'NN', 'NN', 'NN', 'VV', 'SP'], ['NT', 'PN', 'AD', 'VE', 'AS', 'AD', 'SB', 'NR', 'NN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VC', 'NN'], ['NR', 'NR', 'NR', 'NN', 'NN', 'NN', 'ETC', 'AD', 'JJ', 'AD', 'VV', 'JJ', 'NN'], ['P', 'NR', 'NR', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'P', 'NN', 'VV', 'JJ', 'NN'], ['NR', 'NR', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['NT', 'PN', 'AD', 'VV', 'NN'], ['DT', 'NN', 'NN', 'PN', 'VC', 'PN', 'VV'], ['NN', 'AD', 'VV', 'AD', 'VV', 'VV'], ['NN', 'NN', 'NT', 'CD', 'CD', 'CD', 'M', 'CD'], ['PN', 'VV', 'DEC', 'PN', 'AD', 'VV', 'VV', 'PN'], ['PN', 'VC', 'NN', 'NN', 'DEG', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'AD', 'VV'], ['AD', 'SP', 'VV', 'VV', 'AD', 'VA'], ['PN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['PN', 'AD', 'VV', 'BA', 'NN', 'VV'], ['CS', 'NN', 'AD', 'NN', 'VA', 'AS', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'CC', 'NN'], ['AD', 'JJ', 'DEG', 'NN', 'VC', 'NN', 'VV', 'NN', 'NR', 'NR', 'NN', 'DEG'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'LC', 'AD', 'VV', 'NN', 'NN', 'VV'], ['PN', 'VV', 'DT', 'VV', 'VC', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'NN', 'P', 'NR', 'VV', 'CD', 'JJ', 'NN'], ['NN', 'NR', 'NR', 'VV', 'NN', 'P', 'PN', 'PN', 'CD', 'VV', 'CD', 'M', 'AD', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'LC', 'NN', 'NN', 'AD', 'AD', 'VV', 'AS', 'VV', 'NN', 'NN'], ['AD', 'AD', 'AD', 'VV', 'VV', 'VV', 'CD', 'NN', 'DEG', 'NN', 'P', 'NN', 'VV', 'NN'], ['P', 'NT', 'NN', 'AD', 'P', 'NR', 'NN', 'NN', 'VV', 'AS', 'CD', 'NN', 'DEG', 'NN'], ['VV', 'DEC', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'AS', 'M', 'VA', 'NN', 'VV', 'NN'], ['VV', 'VV', 'DEC', 'NN', 'NN', 'AD', 'CD', 'NN', 'LC', 'NN', 'LC', 'DEG', 'NN', 'VV', 'AD', 'VA'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'JJ', 'JJ', 'AD', 'VV', 'DER', 'VV', 'AD', 'VE', 'NN'], ['AD', 'P', 'DT', 'NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'PN', 'AD', 'VV', 'NN'], ['DT', 'JJ', 'NN', 'AD', 'VC', 'NR', 'AD', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'P', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'SB', 'VV', 'DEC', 'NN'], ['AD', 'P', 'DT', 'LC', 'VE', 'CD', 'VV', 'AS'], ['AD', 'VV', 'LC', 'NN', 'AD', 'VV'], ['NR', 'VV', 'VV', 'VV', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'AD', 'VV', 'VV', 'P', 'VV', 'NN'], ['NN', 'P', 'PN', 'NN', 'VC', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'AD', 'VV', 'VV'], ['AD', 'NN', 'VV', 'JJ', 'NN'], ['AD', 'VV', 'VV', 'PN', 'CD', 'NN', 'AD'], ['NN', 'VV', 'P', 'NN', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'P', 'NN', 'NR'], ['PN', 'DEG', 'NN', 'AD', 'SB', 'NR', 'DEG', 'VV'], ['PN', 'AD', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'VV', 'NR', 'VV', 'VV', 'PN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NR', 'NN'], ['AD', 'NN', 'DEV', 'VV', 'AS', 'P', 'NN', 'LC', 'VV', 'CD', 'NR', 'DEG', 'NN', 'P', 'PN', 'VV'], ['NN', 'NN', 'P', 'NN', 'NN', 'VV', 'VV', 'CD', 'NN', 'CC', 'NN'], ['PN', 'AD', 'VV', 'AS', 'PN', 'NN', 'NN'], ['AD', 'NN', 'BA', 'NN', 'VV', 'VV', 'AS', 'NN', 'LC'], ['AD', 'AD', 'AD', 'VV', 'DEC', 'NN', 'NN', 'AD'], ['NN', 'NN', 'VV', 'VV', 'NN'], ['DT', 'NN', 'VV', 'PN', 'AD', 'VV', 'AS', 'SP'], ['NN', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['AD', 'VV', 'PN', 'VV', 'SP', 'AD', 'VV', 'AD', 'VV'], ['P', 'NR', 'NN', 'LC', 'VV', 'NR', 'JJ', 'NN', 'NN'], ['AD', 'VV', 'NN', 'VV', 'SP'], ['NR', 'VA', 'P', 'NN', 'LC', 'AD', 'VV', 'PN', 'PN', 'AD', 'AD', 'VV', 'NN'], ['P', 'VV', 'AD', 'VV', 'NN', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['P', 'VV', 'JJ', 'NN', 'NR', 'NT', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN', 'NN', 'MSP'], ['P', 'AD', 'VV', 'AD', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN'], ['NN', 'DEG', 'NN', 'P', 'NR', 'CC', 'NR', 'NN', 'NN', 'DEG', 'NR'], ['VV', 'DEC', 'NN', 'VV', 'NN', 'VV', 'LC', 'LB', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'BA', 'NR', 'NN', 'LC', 'DEG', 'VV', 'NN', 'VV', 'AS', 'NT'], ['AD', 'NN', 'VV', 'PN', 'VA', 'VA', 'AD', 'VV', 'NR', 'AD', 'VV', 'PN', 'VV', 'DEC', 'NN'], ['JJ', 'NN', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VC', 'NN', 'LC', 'NR', 'NN', 'JJ', 'DEG', 'AD', 'NN'], ['NR', 'VV', 'AD', 'NT', 'NT', 'VV', 'DEC', 'AD', 'VV', 'AS', 'CD', 'CD', 'M', 'VA', 'DEC', 'NN'], ['DT', 'AD', 'VV', 'NN', 'SP', 'SP'], ['AD', 'CD', 'VV', 'NN', 'AD', 'SB', 'VA', 'DEC', 'NN', 'VV'], ['AD', 'NN', 'NT', 'AD', 'VE', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'AD', 'AD', 'DEV', 'VV'], ['NT', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'DEC', 'DT', 'CD', 'NN', 'MSP', 'VV', 'NN', 'SP'], ['NT', 'NR', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN'], ['VE', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'VV'], ['NN', 'NN', 'VV', 'NT', 'NT', 'M', 'NN', 'VV', 'CD', 'CD', 'CD', 'CD', 'M', 'CD'], ['NN', 'LC', 'NR', 'NN', 'P', 'NN', 'VV', 'AS', 'NN', 'LC', 'NN', 'NN', 'NN', 'NN'], ['JJ', 'NN', 'AD', 'VV', 'NN', 'AD', 'NN', 'VV', 'VV', 'NN', 'LC'], ['NN', 'NN', 'VV', 'NN', 'NN', 'LC', 'AD', 'VC', 'VV', 'P', 'NN', 'NR'], ['NR', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NR', 'DEG', 'NN'], ['P', 'NN', 'NN', 'NN', 'VV', 'VV', 'LC', 'AD', 'VV', 'NN'], ['AD', 'AD', 'VV', 'P', 'NN', 'DEG', 'VA', 'VV', 'NN', 'VV'], ['VV', 'NR', 'NN', 'AD', 'AD', 'NN', 'AS', 'NN', 'DT', 'JJ', 'DEG', 'NN'], ['PN', 'VV', 'PN', 'P', 'NN', 'AD', 'VA', 'PN', 'VV', 'DEC', 'AD', 'VE', 'NN'], ['PN', 'NT', 'VV', 'PN', 'CC', 'NN', 'VE', 'AS', 'CD', 'NN', 'NN'], ['NN', 'VV', 'NN', 'VV', 'DEC', 'JJ', 'NN', 'VV', 'CD', 'VA'], ['P', 'NN', 'VV', 'NN', 'DEG', 'NN', 'NR', 'VV', 'P', 'NR', 'VV'], ['PN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'PN', 'AD', 'PN', 'DEG', 'SP'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'CD', 'M', 'VV', 'SP'], ['NN', 'VV', 'CD', 'VE', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['P', 'JJ', 'NN', 'LC', 'NN', 'NN', 'VV', 'NN', 'AD', 'AD', 'VA'], ['PN', 'P', 'PN', 'AD', 'VV', 'AS', 'NR', 'VV', 'DEC', 'NN'], ['AD', 'NR', 'VV', 'AS', 'NN', 'CC', 'NN', 'NN', 'NN'], ['AD', 'JJ', 'NN', 'NN', 'NN', 'AD', 'P', 'NN', 'CD', 'CD', 'CD'], ['VV', 'VV', 'VV', 'CS', 'VE', 'NN', 'SP', 'PN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'NR', 'NN', 'CC', 'NR', 'NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'P', 'VE', 'NN', 'VV', 'NR', 'AD', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['NR', 'AD', 'P', 'DT', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['CD', 'NN', 'P', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NT', 'NN', 'NN', 'P', 'PN', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['NN', 'AD', 'VV', 'PN', 'VV', 'NN', 'CC', 'VV', 'NN'], ['NT', 'PN', 'MSP', 'VV', 'DEC', 'NR', 'AD', 'VA', 'AD', 'VC', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'JJ'], ['P', 'NN', 'LC', 'PN', 'AD', 'VV', 'NN', 'DEC', 'VA', 'NN', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'VA', 'VA', 'DEV', 'VV', 'IJ', 'NT', 'VE', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VA', 'SP'], ['AD', 'PN', 'VV', 'VV', 'NR', 'NN', 'P', 'NR', 'NN', 'P', 'NN', 'P', 'NN', 'LC', 'DEG', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'VV', 'PN', 'SP', 'VC', 'PN', 'VV', 'PN', 'SP', 'SP'], ['AD', 'PN', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'VV', 'PN', 'VV', 'VV', 'AS', 'SP'], ['NT', 'NN', 'NN', 'AD', 'VV', 'VV', 'AD', 'VV', 'AS', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'VV', 'CC', 'VV', 'VV', 'AD', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'P', 'VV', 'NN', 'VV', 'NN', 'LC', 'NN', 'VV', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'DEC', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['AD', 'NN', 'VV', 'AS', 'AD', 'VV', 'AD', 'NR', 'SB', 'VV', 'NN', 'LC', 'VV', 'P', 'NN'], ['PN', 'VV', 'LC', 'AD', 'VV', 'VV', 'PN', 'VV', 'PN', 'VV', 'CD', 'JJ', 'NN', 'VV', 'VV', 'VV'], ['PN', 'VV', 'NN', 'NR', 'NN', 'VV', 'NN', 'JJ', 'NN', 'PN', 'VV', 'CD', 'VV', 'VV', 'VV', 'PN'], ['PN', 'AD', 'NN', 'AD', 'PN', 'VE', 'NN', 'VV', 'PN', 'DEC', 'NN', 'JJ', 'NN'], ['PN', 'VC', 'DT', 'NR', 'NN', 'VV', 'DEC', 'NN', 'NR', 'NN'], ['AD', 'VV', 'AD', 'VV', 'VV', 'AD', 'VV', 'PN', 'AD', 'VV'], ['NN', 'NN', 'NN', 'VV'], ['CS', 'PN', 'VV', 'AS', 'NN', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'PN', 'AD', 'VV', 'NN', 'SP', 'SP'], ['CS', 'NN', 'AD', 'AD', 'AD', 'VA', 'AD', 'VV', 'VV', 'VV'], ['JJ', 'NN', 'LC', 'AD', 'NR', 'VV', 'AD', 'VA'], ['PN', 'VV', 'PN', 'CC', 'NR', 'DEG', 'NN', 'CD', 'AD', 'VV'], ['NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'SB', 'VV', 'DEC', 'NN'], ['PN', 'AD', 'AD', 'VA', 'NN', 'VA', 'VA', 'SP', 'AD', 'VV', 'AD', 'VA'], ['AD', 'VE', 'CD', 'JJ', 'DEG', 'VV', 'PN', 'DEG', 'NN', 'VV', 'JJ', 'VA', 'DEC', 'NN', 'VV', 'VV', 'PN', 'VC', 'PN'], ['AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'CD', 'VV', 'NN'], ['VV', 'AD', 'VV', 'NN', 'NN'], ['NR', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'JJ', 'NN', 'NN'], ['CS', 'NN', 'AD', 'VV', 'VV', 'DT', 'NN', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['P', 'NN', 'NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'LC', 'NN', 'NN', 'CC', 'NN'], ['NN', 'LC', 'LC', 'VV', 'AS', 'VA', 'DEC', 'NN', 'VE', 'NN', 'VC', 'AD', 'VV', 'SP'], ['NR', 'NN', 'NR', 'NN', 'AD', 'NT', 'NN', 'AD', 'VV'], ['NN', 'NN', 'CD', 'M', 'VV', 'DEV', 'VV', 'PN', 'NN', 'NN', 'VV', 'DER', 'AD', 'SP'], ['NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV'], ['VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'P', 'PN', 'NN', 'VV', 'VV'], ['AD', 'PN', 'DEG', 'NN', 'AD', 'AD', 'JJ', 'NN', 'NN', 'VV', 'VV', 'DER', 'VA', 'AD'], ['NT', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV', 'SP'], ['AD', 'VC', 'P', 'PN', 'VV', 'VV', 'PN', 'VV', 'NN', 'SP'], ['AD', 'NN', 'AD', 'VV', 'AD', 'P', 'VV', 'NN', 'SP'], ['NN', 'NN', 'VV', 'AS', 'P', 'PN', 'JJ', 'NN', 'LC', 'VV', 'NN'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['AD', 'VV', 'NN', 'JJ', 'NN', 'AD', 'VV', 'VV', 'CD', 'NN'], ['VV', 'SP', 'PN', 'VC', 'PN', 'DEG', 'NN', 'VC', 'CD', 'NN', 'VA', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NN', 'VV', 'JJ', 'NN', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV'], ['PN', 'AD', 'AD', 'VA', 'DT', 'CC', 'JJ', 'NN', 'SP', 'PN', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NN', 'LC', 'VV', 'NN', 'PN', 'VV', 'VV', 'AD', 'PN'], ['NN', 'VV', 'AS', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'VE', 'AD', 'JJ', 'DEG', 'NN', 'VV', 'P', 'VV', 'LC', 'VV', 'CD', 'NN'], ['AD', 'PN', 'AD', 'AD', 'P', 'NN', 'LC', 'VV', 'VV'], ['P', 'VE', 'CD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AS', 'AD', 'NN', 'AD', 'VV', 'AD', 'NN', 'PN', 'VC', 'PN', 'VV', 'DEC'], ['CD', 'NN', 'AD', 'NT', 'AD', 'VV', 'VV', 'VV', 'CD', 'NN'], ['NN', 'CC', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN', 'VV'], ['DT', 'NN', 'AD', 'VV', 'AS', 'AD', 'NN', 'DEG', 'NN', 'AD', 'VV', 'NR'], ['NR', 'VV', 'VV', 'PN', 'AD', 'VV', 'CD', 'NN', 'PN', 'VV', 'VV', 'AD', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'VV', 'AD', 'VV'], ['AD', 'P', 'CD', 'NN', 'LC', 'PN', 'AD', 'VC', 'AD', 'VA', 'DEC', 'NN'], ['NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'P', 'PN', 'VV', 'NN'], ['AD', 'VV', 'VV', 'NR', 'CC', 'NR', 'NN', 'VV', 'AS', 'JJ', 'NN'], ['AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'AD', 'P', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'LC', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'PN', 'VV', 'CD', 'NN'], ['PN', 'P', 'VV', 'NN', 'VV', 'NN', 'NN', 'ETC', 'AD', 'VC', 'VE', 'NN', 'SP'], ['NT', 'AD', 'VV', 'PN', 'NN', 'CC', 'NN'], ['PN', 'AD', 'NR', 'NR', 'VV', 'JJ', 'DEG', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'VE', 'VV', 'VE', 'AD', 'VE', 'NN'], ['NT', 'NN', 'AD', 'VV', 'VV', 'AS', 'VV'], ['PN', 'VV', 'VV', 'NN', 'VV', 'NN', 'VA', 'DEC', 'NN'], ['NT', 'AD', 'NN', 'LB', 'JJ', 'VV', 'AS'], ['PN', 'VV', 'P', 'JJ', 'NN', 'DEC', 'VV', 'NN'], ['AD', 'VA', 'VC', 'PN', 'VV', 'PN', 'NN', 'IJ'], ['PN', 'VV', 'AS', 'PN', 'AD', 'NN', 'VV', 'VV', 'NN', 'SP'], ['PN', 'SP', 'AD', 'VC', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV'], ['P', 'NN', 'NT', 'VV', 'DEC', 'NN', 'AD', 'VV'], ['PN', 'VV', 'NR', 'CC', 'PN', 'NN', 'AD', 'VA'], ['NN', 'AD', 'VV', 'AD', 'NN', 'NN', 'SP'], ['VV', 'NR', 'DEG', 'NN', 'AD', 'CC', 'VA'], ['AD', 'NR', 'NR', 'AD', 'VV', 'VV', 'DER', 'AD', 'VA'], ['AD', 'NR', 'NN', 'VV', 'VC', 'NN'], ['NN', 'AD', 'VV', 'DEC', 'NN', 'NN', 'VA'], ['AD', 'NN', 'LC', 'CC', 'CD', 'NN', 'AD', 'VV'], ['VV', 'NN', 'AD', 'P', 'AD', 'VA', 'DEC', 'NN', 'VV', 'VV', 'NN', 'VA'], ['VV', 'P', 'NN', 'NN', 'LC', 'DEG', 'NN', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'VC', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['AD', 'PN', 'AD', 'AD', 'DEV', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN', 'NT', 'DEG', 'PN'], ['PN', 'AD', 'VC', 'JJ', 'DEG', 'NN', 'NN'], ['PN', 'VV', 'AD', 'VV', 'VV', 'NT', 'LC', 'VV', 'VV', 'VV'], ['AD', 'VC', 'NT', 'DEG', 'NN', 'NR', 'AD', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'DEC', 'NN', 'NN', 'NN'], ['JJ', 'NN', 'VV', 'VV', 'LC', 'VV', 'NN', 'VV', 'AD', 'VV', 'VV', 'P', 'NN', 'NN', 'NN'], ['P', 'NN', 'JJ', 'NN', 'VV', 'NN', 'LC', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['NR', 'NN', 'NN', 'VV', 'BA', 'NN', 'P', 'NN', 'LC', 'VV', 'VV'], ['AD', 'CS', 'P', 'NN', 'NN', 'CD', 'IJ', 'CD', 'NN', 'DEG', 'JJ', 'JJ', 'NN', 'JJ'], ['NN', 'AD', 'AD', 'VV', 'AS', 'CD', 'NR', 'DEG', 'NN', 'AD', 'P', 'NR', 'VV'], ['PN', 'VV', 'NN', 'P', 'NN', 'AD', 'VV', 'LC', 'NN', 'AD', 'VV', 'NN', 'VV', 'JJ', 'NN'], ['CD', 'NN', 'VV', 'NN', 'NN', 'VA'], ['VV', 'NN', 'NN', 'AD', 'VV', 'PN', 'VV', 'AD', 'VV', 'VV', 'VV', 'NN', 'CC', 'VV', 'PN', 'VV', 'VV', 'AD', 'AD'], ['DT', 'CD', 'NN', 'NN', 'VV', 'NN', 'NN', 'VC', 'NT', 'CD', 'CD', 'M'], ['NT', 'VV', 'DT', 'DEG', 'NN', 'AD', 'VV', 'AD', 'AS', 'PN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'DEV', 'VV', 'PN', 'AD', 'VV', 'NN', 'VV', 'VV', 'VV'], ['DT', 'CD', 'NN', 'VC', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'NN', 'IJ', 'NR', 'NN', 'NT', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['PN', 'BA', 'NN', 'VV', 'AD', 'VV', 'NN', 'VV', 'AS', 'AD', 'NN', 'VA', 'DEC', 'NN'], ['VV', 'NR', 'VC', 'NR', 'DEG', 'NN', 'P', 'NR', 'VE', 'NR', 'AD', 'AD', 'AD', 'VV', 'VV'], ['NN', 'NN', 'AD', 'P', 'DT', 'CD', 'NN', 'VV', 'NN'], ['NR', 'NN', 'JJ', 'NN', 'JJ', 'DEG', 'NN', 'AD', 'P', 'NR'], ['CD', 'NN', 'P', 'NR', 'NN', 'VA', 'DEC', 'NN', 'VV', 'NN'], ['NR', 'NN', 'NN', 'NN', 'NN', 'CD', 'M', 'NT', 'VV'], ['P', 'NN', 'VV', 'VV', 'NR', 'VV', 'AD', 'VV', 'NR'], ['PN', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN', 'PN', 'DEG', 'NN', 'VV'], ['NR', 'VV', 'LC', 'AD', 'AD', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['AD', 'PN', 'P', 'NR', 'NN', 'VV', 'NN', 'VV', 'AD', 'VC', 'VA'], ['AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'P', 'NN', 'NN', 'VV'], ['PN', 'VV', 'NN', 'BA', 'VV', 'DEC', 'NN', 'VV', 'VV', 'VV'], ['NR', 'NN', 'AD', 'AD', 'VV', 'NN'], ['CS', 'PN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN', 'PN', 'DEG', 'NN', 'VV', 'AD', 'VA'], ['AD', 'VV', 'PN', 'DEG', 'CC', 'NN', 'CC', 'NN', 'AD', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['NN', 'LC', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'PN', 'VV', 'NN', 'DEC', 'NN', 'DT', 'NN', 'AD', 'VV'], ['NN', 'AD', 'VV', 'VV', 'PN', 'VV', 'AD', 'VV', 'AS', 'VV', 'DEC', 'VA', 'AS', 'PN', 'VV', 'PN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VV', 'NR', 'NN', 'DEG', 'JJ', 'NN', 'NN'], ['AD', 'P', 'NN', 'DEG', 'NN', 'CC', 'AD', 'VV', 'AS', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'AD', 'P', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN', 'JJ', 'CC', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['NR', 'NR', 'VV', 'NN', 'LC', 'AD', 'VA', 'AS', 'M', 'NN', 'AD', 'AD', 'VV', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'PN', 'VV', 'PN', 'DEG', 'NN', 'P', 'PN', 'VV'], ['AD', 'P', 'NR', 'NN', 'VV', 'LC', 'AD', 'AD', 'VV', 'VV'], ['CD', 'NN', 'PN', 'VV', 'VV', 'VV', 'NN', 'VV', 'SP'], ['PN', 'VV', 'AD', 'NN', 'NN', 'NN', 'PN', 'VV', 'AS'], ['CD', 'NN', 'AD', 'VV', 'NN', 'VV', 'DT', 'NN'], ['PN', 'AD', 'VV', 'VV', 'DT', 'VV', 'VV', 'DEC', 'NN', 'LC'], ['NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'VA'], ['NN', 'AD', 'AD', 'VV', 'AS', 'VV', 'NT', 'DEG', 'NN', 'NN'], ['AD', 'AD', 'NR', 'VE', 'NN', 'NN', 'DEG', 'NN', 'VC', 'VA', 'SP'], ['VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['VV', 'NN', 'VV', 'AD', 'VV', 'AD', 'SB', 'VV', 'MSP', 'VV', 'AS', 'NN'], ['PN', 'VV', 'PN', 'AD', 'PN', 'VC', 'NN', 'DEG', 'VV', 'NR'], ['PN', 'AD', 'VV', 'AD', 'NN', 'NN', 'AD', 'VA', 'AD', 'VV'], ['P', 'NN', 'NN', 'NN', 'VV', 'NR', 'NR', 'CC', 'NN'], ['AD', 'CD', 'NN', 'VV', 'NN', 'P', 'NN', 'NN', 'VV'], ['VE', 'JJ', 'NN', 'DEG', 'NN', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VA', 'SP'], ['NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VA'], ['VV', 'VV', 'AS', 'VV', 'DEC', 'CD', 'AD', 'NN', 'VV', 'AS'], ['NR', 'AD', 'VV', 'NR', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NT', 'NR', 'VV', 'P', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'VV'], ['NN', 'JJ', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'LC', 'VV', 'DER', 'VV'], ['NN', 'NN', 'NN', 'NT', 'CD', 'NT', 'VV', 'CD', 'NN', 'VV'], ['AD', 'PN', 'NN', 'VV', 'NN', 'MSP', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NT', 'NR', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV'], ['NT', 'LB', 'VV', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'SB', 'VV', 'NN', 'NN', 'DEG', 'NN', 'VV'], ['P', 'NN', 'NN', 'NR', 'JJ', 'NN', 'AD', 'VV', 'JJ', 'NN', 'VV'], ['NT', 'CD', 'NN', 'LC', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV', 'P', 'NN', 'LC', 'NN'], ['VV', 'VV', 'AD', 'VV', 'LC', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'BA', 'JJ', 'NN', 'VV', 'VV'], ['P', 'PN', 'VV', 'LC', 'NN', 'AD', 'AD', 'AD', 'AD', 'JJ', 'NN', 'NN', 'VE', 'VV', 'VV'], ['P', 'PN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'AD', 'VV', 'NR', 'NN', 'JJ', 'NR', 'NN'], ['PN', 'VC', 'PN', 'DEG', 'NN', 'NN', 'VV', 'LC', 'VV', 'NN', 'VV', 'VV', 'AD', 'PN', 'VV', 'VV'], ['NN', 'NN', 'NN', 'ETC', 'VV', 'NN', 'AD', 'VC', 'PN', 'AD', 'VV', 'SP'], ['NN', 'PN', 'AD', 'VV', 'VE', 'NN'], ['VV', 'LC', 'NN', 'VV', 'VV', 'NN'], ['NN', 'AD', 'PN', 'AD', 'VV', 'NN', 'LC', 'VV', 'AD', 'VV', 'CD'], ['VV', 'LC', 'VV', 'NN', 'P', 'NN', 'VV', 'VV', 'AS', 'NN', 'NN'], ['VV', 'VV', 'NN', 'AD', 'VV', 'AS', 'VA'], ['DT', 'NN', 'VV', 'CD', 'NN', 'DEG', 'NN'], ['DT', 'JJ', 'DEG', 'NN', 'VV', 'NR'], ['AD', 'AD', 'VV', 'BA', 'PN', 'VV', 'SP'], ['PN', 'AD', 'VV', 'NN', 'P', 'NT', 'NN', 'LC', 'VV', 'AS', 'PN', 'CD', 'NN'], ['PN', 'AD', 'VV', 'VV', 'DEV', 'AD', 'VV', 'AD', 'AD', 'NT', 'AD', 'VA', 'SP'], ['NN', 'DEG', 'NN', 'NN', 'NN', 'VA'], ['PN', 'NT', 'NN', 'NN', 'AD', 'AD', 'VA'], ['NR', 'NN', 'NN', 'AD', 'VA'], ['AD', 'NR', 'NR', 'NN', 'VV', 'LC', 'CC', 'VV', 'LC', 'DEG', 'NN'], ['NN', 'NN', 'VV', 'PN', 'VV', 'VV', 'SP'], ['AD', 'BA', 'NN', 'VV', 'NN', 'NN'], ['NT', 'VV', 'NN', 'VV', 'NR'], ['AD', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'P', 'NR', 'VC', 'NN', 'VV', 'DEC', 'NN'], ['PN', 'NN', 'VV', 'NN', 'VV', 'AS', 'NR', 'NN', 'VV', 'VA', 'VV', 'DEC', 'JJ', 'NN'], ['AD', 'P', 'NN', 'NN', 'VV', 'DEC', 'VV', 'AD', 'VE', 'NN', 'VV', 'DEC', 'VA'], ['VV', 'PN', 'AD', 'DEV', 'NN', 'AD', 'AD', 'VV', 'AS'], ['NN', 'VV', 'NN', 'LC', 'VV', 'NN', 'NN', 'NN', 'AD', 'P', 'CD', 'NN', 'NN'], ['AD', 'AD', 'AD', 'VV', 'VV', 'PN', 'VV', 'VV', 'DEC', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'SP'], ['NN', 'VV', 'PN', 'DEG', 'NN', 'VV', 'VV', 'CD', 'AS', 'NT', 'AD', 'VV', 'AS', 'NN', 'NN'], ['CS', 'VV', 'AS', 'PN', 'AD', 'VV', 'NN', 'CC', 'CD', 'CD', 'NN', 'NN', 'NN'], ['AD', 'NN', 'VC', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'SP'], ['PN', 'VV', 'NN', 'NN', 'AD', 'SB', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VA'], ['AD', 'NN', 'P', 'VV', 'NN', 'LC', 'NN', 'NN', 'NN', 'VV', 'VV'], ['PN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'JJ', 'JJ', 'NN', 'NN', 'JJ', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'JJ', 'NN', 'NT', 'VV', 'AD', 'VV'], ['CD', 'VV', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'LC'], ['NN', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'VV'], ['CD', 'NN', 'VV', 'P', 'NN', 'BA', 'NN', 'VV', 'LC', 'AD', 'VV', 'NN', 'NN', 'LC'], ['VV', 'VV', 'DT', 'NN', 'JJ', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'VV'], ['AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'VC', 'PN', 'DEG', 'NN', 'NN', 'SP', 'AD', 'NN', 'NN', 'PN'], ['NN', 'VV', 'AS', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'VV', 'DEC', 'JJ', 'AD', 'AD', 'VV', 'AS', 'NN'], ['AD', 'PN', 'VV', 'AS', 'JJ', 'NN', 'VV', 'DEC', 'CD', 'NN'], ['AD', 'AD', 'VV', 'VV'], ['AD', 'VV', 'NN'], ['VV', 'DEC', 'NR', 'NN', 'P', 'NN', 'LC', 'SB', 'VV', 'VV'], ['AD', 'NN', 'NN', 'DEC', 'NN', 'AD', 'VE', 'P', 'NN', 'VV', 'VV'], ['DT', 'NN', 'VV', 'P', 'CD', 'CD', 'CD', 'NT', 'NN', 'NN', 'VV'], ['AD', 'NN', 'AD', 'PN', 'NN', 'VV', 'DEC', 'PN'], ['VC', 'AD', 'VA', 'DEC', 'NN', 'SP'], ['NR', 'NN', 'VE', 'CD', 'NN'], ['NN', 'VV', 'NN', 'VV', 'AD', 'DEC', 'NN', 'VV', 'VV', 'VV', 'SP'], ['NR', 'VV', 'AD', 'P', 'CD', 'CD', 'NT', 'LC', 'VV', 'CC', 'VV'], ['PN', 'DT', 'NN', 'NN', 'AD', 'VA', 'DEC', 'AD', 'AD', 'AD', 'VA', 'SP', 'SP'], ['AD', 'PN', 'VV', 'VV', 'DEC', 'VC', 'NN'], ['AD', 'VA', 'VV', 'NN', 'AD', 'VA'], ['VV', 'VV', 'PN', 'NN'], ['NR', 'PN', 'VC', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'P', 'NN', 'NN', 'NN', 'LC', 'NN', 'NN', 'VV'], ['NN', 'VV', 'LC', 'NN', 'NN', 'NN', 'VA', 'AD', 'VV'], ['AD', 'NN', 'AD', 'VA', 'SP'], ['AD', 'VV', 'AD', 'JJ', 'NN'], ['NN', 'LC', 'AD', 'P', 'PN', 'VA'], ['PN', 'AD', 'AD', 'VV', 'SP'], ['NR', 'NN', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['AD', 'SP', 'PN', 'AD', 'AD', 'VA'], ['PN', 'VV', 'PN', 'AD', 'VV', 'VA', 'SP'], ['AD', 'P', 'PN', 'VV', 'PN', 'PN', 'AD', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN', 'NN'], ['PN', 'BA', 'NN', 'NN', 'VA', 'VA', 'SP'], ['DT', 'NN', 'VV', 'NR', 'NN', 'NN', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'SP', 'PN', 'AD', 'VA'], ['PN', 'AD', 'SB', 'VV', 'NR', 'NR', 'DEG', 'NN', 'NN', 'VV'], ['NN', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'VV'], ['AD', 'P', 'NN', 'LC', 'AD', 'VV'], ['AD', 'VV', 'AD', 'VV', 'PN'], ['P', 'CD', 'NN', 'CD', 'NN', 'VV', 'VV', 'NN', 'VV', 'AD', 'CD', 'M'], ['NR', 'NN', 'NR', 'NR', 'NR'], ['NN', 'VE', 'JJ', 'NN', 'SP'], ['AD', 'VV', 'P', 'NN', 'VA'], ['P', 'PN', 'AD', 'VV', 'NN', 'VA'], ['NR', 'DT', 'NN', 'NN', 'NR'], ['NN', 'VC', 'NN', 'JJ', 'NN', 'JJ'], ['AD', 'NN', 'LC', 'VE', 'NN', 'VV'], ['AD', 'P', 'PN', 'VV', 'AS', 'NR'], ['NN', 'VV', 'NR', 'VV', 'CD', 'NN', 'NR', 'NN', 'NN', 'NN'], ['AD', 'NN', 'PN', 'AD', 'AD', 'VV'], ['PN', 'VV', 'P', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VA', 'DEV', 'VV'], ['AD', 'VV', 'VV', 'VV', 'NN', 'MSP', 'NN', 'AD', 'VE', 'NN'], ['PN', 'P', 'NN', 'NN', 'AD', 'VA'], ['PN', 'AD', 'VV', 'NN', 'P', 'NR', 'NN', 'VV', 'NR'], ['DT', 'NN', 'AD', 'VV', 'AS', 'VV'], ['JJ', 'DEG', 'NN', 'AD', 'VV', 'P', 'NN'], ['JJ', 'NN', 'AD', 'VV', 'VV', 'VV', 'VE', 'NN', 'DEC', 'NN', 'VA', 'DEV', 'VV', 'NN', 'AD', 'VE', 'DEC', 'PN', 'VV', 'PN', 'SP'], ['NN', 'CC', 'NN', 'AD', 'NN', 'DEG', 'VV', 'NN', 'AD', 'VV', 'AS', 'PN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'VV', 'VV', 'NR', 'LC', 'NR', 'NN', 'BA', 'NN', 'VV', 'VV', 'VV', 'AD', 'AS', 'VV', 'VV', 'VV'], ['AD', 'AD', 'AD', 'AD', 'VV', 'PN', 'AD', 'VV', 'NT', 'VV', 'AS'], ['PN', 'VV', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'P', 'NN', 'LC', 'VV', 'PN', 'CC', 'NR', 'VV', 'PN', 'AD', 'VV', 'PN', 'NN'], ['AD', 'VV', 'VE', 'M', 'JJ', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'AD', 'VV', 'AD', 'AD', 'JJ', 'NN', 'AD', 'VV', 'DT', 'AD', 'PN', 'P', 'PN', 'NN'], ['PN', 'NT', 'AD', 'NN', 'VA', 'DEV', 'VV', 'AD', 'VC', 'JJ', 'DEG'], ['NN', 'VV', 'AS', 'NN', 'VV', 'VV', 'AS', 'VV', 'SP'], ['NN', 'NN', 'P', 'NN', 'SB', 'VV', 'AS', 'NN'], ['AD', 'AD', 'NN', 'AD', 'CD', 'JJ', 'NN'], ['AD', 'VE', 'NN', 'VV', 'NN', 'VV', 'JJ', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['P', 'NN', 'LC', 'NN', 'VV', 'VV', 'DEC', 'AD', 'NN', 'CC', 'NN'], ['NN', 'AD', 'VV', 'VV', 'VA', 'DEC', 'NN', 'VV', 'NN', 'AS', 'PN', 'VV', 'M'], ['AD', 'NN', 'P', 'VV', 'AD', 'VV', 'NN', 'LC', 'VV', 'AS', 'NN', 'NN'], ['NN', 'NN', 'VV', 'NR', 'NN', 'NN', 'DEG', 'NN', 'VV', 'DEC', 'NN', 'VA'], ['NT', 'NN', 'P', 'NN', 'VV', 'AS', 'NN', 'VV', 'NN', 'BA', 'NN', 'VV', 'NN', 'SP'], ['NR', 'CC', 'CD', 'NN', 'VC', 'AD', 'AD', 'VV', 'VV', 'DER', 'VV'], ['AD', 'NT', 'NN', 'AD', 'AD', 'VA', 'AS', 'AD', 'AD', 'VA', 'DEC', 'NN', 'NN', 'VV', 'VV'], ['AD', 'PN', 'AD', 'P', 'PN', 'NN', 'NN', 'SP'], ['AD', 'CD', 'NN', 'AD', 'SB', 'VV', 'SP'], ['VV', 'P', 'NN', 'AD', 'AD', 'AD', 'VA'], ['NR', 'AD', 'MSP', 'VV', 'NR', 'NN', 'DEG', 'NN', 'CC', 'NN', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'AD', 'JJ', 'NN', 'NN', 'CC', 'VV', 'NN', 'NN', 'VV', 'VV', 'SP'], ['AD', 'NN', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'NT', 'NT', 'PN', 'VV', 'NN'], ['VV', 'NR', 'NR', 'NR', 'NN', 'VV', 'NN', 'VV', 'VA', 'AD', 'VA'], ['NN', 'NR', 'VV', 'BA', 'NN', 'DEG', 'NN', 'VV', 'VV', 'NN', 'AD', 'AD', 'VV', 'AD', 'VA'], ['VV', 'VV', 'PN', 'AD', 'AD', 'AD', 'AD', 'VV', 'NN', 'VV', 'JJ', 'NN'], ['NN', 'VV', 'MSP', 'VV', 'P', 'JJ', 'NN', 'NN', 'VV', 'PN', 'AD', 'PN', 'VC', 'NR', 'SP'], ['PN', 'VV', 'NN', 'VV', 'P', 'JJ', 'NN', 'LC', 'VV', 'CD', 'NN', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'AS', 'PN', 'DEG', 'NN', 'NT', 'PN', 'AD', 'VV', 'PN', 'VE', 'NN', 'AD', 'VV', 'PN', 'VC', 'M', 'NN'], ['P', 'NN', 'VV', 'NN', 'AD', 'VV', 'P', 'NN', 'VV', 'VV', 'JJ', 'NN', 'VV', 'VV'], ['PN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'VV', 'LC', 'NR', 'AD', 'VV', 'AD', 'VV', 'NN', 'CC', 'NN', 'VV', 'NR'], ['PN', 'CD', 'VV', 'VV', 'DT', 'NN', 'VV', 'NN', 'NR', 'AD', 'NN', 'NN', 'SP'], ['AD', 'NN', 'VV', 'DT', 'M', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'LC', 'NN', 'DEG', 'NN'], ['AD', 'NR', 'AD', 'VV', 'P', 'JJ', 'NN', 'VE', 'JJ', 'DEG', 'NN'], ['AD', 'AD', 'CD', 'CD', 'M', 'VC', 'NN', 'NN', 'NN'], ['AD', 'VV', 'DT', 'NN', 'PN', 'AD', 'AD', 'VV', 'VV', 'PN'], ['NR', 'NR', 'AD', 'VE', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['P', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV'], ['AD', 'AD', 'VV', 'DEC', 'NN', 'VV', 'AS'], ['VV', 'NN', 'NN', 'AD', 'NN', 'NN', 'NN'], ['NN', 'P', 'NT', 'AD', 'VV', 'AS', 'NN'], ['VV', 'NN', 'VV', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['AD', 'AD', 'NN', 'AD', 'AD', 'VV', 'SP'], ['PN', 'VV', 'PN', 'VA', 'AD', 'VV', 'JJ', 'NN', 'CD'], ['P', 'NN', 'NR', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'NN', 'DEG', 'NN', 'JJ'], ['PN', 'SB', 'VV', 'VC', 'JJ', 'NN', 'CC', 'NR', 'VV', 'NR', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['PN', 'VV', 'VV', 'CD', 'AD', 'VV', 'AS', 'VC', 'VV', 'NN'], ['VV', 'AS', 'DEC', 'NR', 'SB', 'VV', 'VV', 'LC', 'VV', 'VV', 'VV', 'P', 'JJ', 'NN', 'VV', 'VV'], ['NR', 'VV', 'P', 'JJ', 'NN', 'NN', 'VV', 'AD', 'P', 'VV', 'VV', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'JJ', 'NN', 'P', 'VV', 'NN'], ['AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'PN', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV', 'P', 'PN', 'VV'], ['NR', 'NR', 'CC', 'NR', 'NN', 'DEG', 'NN', 'VC', 'NR', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'LC', 'NN', 'SB', 'VV', 'VV', 'VV'], ['AD', 'AD', 'NT', 'PN', 'AD', 'P', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV', 'VV', 'DEC', 'NN', 'NN'], ['PN', 'VC', 'AD', 'AD', 'VE', 'PN', 'NN'], ['PN', 'VV', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'PN', 'NN', 'VE', 'NN'], ['DT', 'NN', 'VV', 'NR', 'VV', 'VA'], ['AD', 'NT', 'DEG', 'NN', 'NR', 'AD', 'VV'], ['NT', 'NN', 'DEC', 'NN', 'NN', 'VC', 'PN'], ['VV', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['AD', 'DT', 'NN', 'VC', 'VV', 'VV', 'DEC', 'NN', 'VC', 'NN', 'NN', 'VC', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'AD', 'NR', 'VV', 'NN', 'VV', 'CD', 'M', 'AD', 'AD', 'AD', 'VA'], ['AD', 'AD', 'VV', 'NN', 'DEC', 'PN', 'AD', 'AD', 'AD', 'VV', 'P', 'VV', 'NR', 'SP'], ['NN', 'DEG', 'NN', 'VA', 'VV', 'CD', 'NN', 'AD', 'VA', 'VE', 'DEC', 'AD', 'VV', 'NN'], ['NR', 'VV', 'JJ', 'NN'], ['VC', 'M', 'AD', 'VA', 'DEC', 'NN', 'SP'], ['NN', 'VV', 'NN', 'LC', 'DEG', 'NR', 'NN'], ['DT', 'NN', 'AD', 'VC', 'AD', 'VA', 'SP'], ['NR', 'P', 'NN', 'VV', 'JJ', 'NN', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'DER', 'AD', 'VA', 'SP'], ['NN', 'VV', 'P', 'NN', 'VV', 'JJ', 'NN'], ['PN', 'VV', 'NN', 'SP', 'AD', 'VV', 'VV'], ['VV', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN'], ['NR', 'VE', 'AD', 'VA', 'DEC', 'NR'], ['AD', 'VA', 'DEC', 'VC', 'P', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VV', 'PN', 'PN', 'AD', 'VV', 'PN', 'SP'], ['NR', 'NN', 'CD', 'NN', 'NN', 'NN', 'PN', 'AD', 'VV'], ['NN', 'VV', 'AD', 'SP'], ['NT', 'AD', 'VV', 'NN', 'AD', 'VV', 'NR', 'VC', 'CD', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'NN', 'VV', 'AS', 'CD', 'NN', 'NN', 'DT', 'NN', 'AD', 'VC', 'VV', 'DEC', 'NN'], ['NR', 'DEG', 'NN', 'NN', 'AD', 'AD', 'AD', 'VV', 'SP'], ['NN', 'P', 'NN', 'NN', 'VV', 'JJ', 'NN', 'VV', 'VV', 'VA', 'ETC', 'NN', 'VE', 'JJ', 'NN'], ['AD', 'VC', 'JJ', 'NN', 'PN', 'AD', 'VV', 'VV', 'PN', 'DT', 'NN', 'AD', 'AD', 'VV', 'SP'], ['DT', 'NN', 'DEG', 'CD', 'NN', 'VV', 'VV', 'AD', 'VA', 'NN', 'NN', 'AD', 'VA'], ['AD', 'VC', 'NN', 'NR', 'CC', 'NR', 'DEG', 'NN', 'P', 'NN', 'LC', 'VV', 'NN'], ['DT', 'NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'VC', 'NN', 'NN', 'AD', 'AD', 'VV', 'AS'], ['AD', 'NR', 'NN', 'AD', 'VV', 'DEC', 'JJ', 'AD', 'VV', 'NN', 'NN', 'NN'], ['VV', 'NN', 'AD', 'VV', 'VV', 'NT', 'DEG', 'NN', 'NN', 'AD', 'VV', 'AS'], ['AD', 'VV', 'VV', 'NN', 'NN', 'VC', 'P', 'VV', 'PN', 'DEG', 'NN', 'NR'], ['NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN'], ['AD', 'AD', 'VV', 'VV', 'VV', 'AD', 'P', 'VV', 'NR', 'VV', 'AS', 'NN'], ['P', 'NN', 'VV', 'DEC', 'NN', 'PN', 'VV', 'VV', 'VV', 'NR', 'NN', 'VC', 'PN', 'AD', 'VV'], ['CD', 'NN', 'VV', 'JJ', 'NN', 'VV', 'P', 'NN', 'NN', 'LC', 'NN', 'VV'], ['NT', 'NR', 'NN', 'NN', 'DT', 'NR', 'NR', 'SB', 'AD', 'AD', 'VV'], ['NR', 'AD', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN', 'VA', 'NN', 'DEG', 'NN', 'NN', 'VV', 'PN'], ['NT', 'NR', 'VV', 'VV', 'JJ', 'NN', 'VC', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'AD', 'VC', 'VV', 'NN', 'NR', 'VV'], ['NN', 'AD', 'VV', 'AS', 'M', 'PN', 'AD', 'VV', 'VV', 'VV', 'NN', 'DEG', 'NN', 'SP'], ['VV', 'DEC', 'NR', 'VV', 'P', 'NN', 'LC', 'NN', 'VV', 'NN', 'VV'], ['NN', 'NR', 'JJ', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV'], ['AD', 'NR', 'JJ', 'VV', 'NN', 'P', 'NN', 'VV', 'NN', 'LC', 'AD', 'VV', 'DEC', 'JJ', 'NN'], ['P', 'NR', 'DT', 'NN', 'NN', 'DEC', 'VV', 'NN', 'LC', 'NN', 'LC', 'AD', 'VV', 'VV'], ['VV', 'AD', 'SB', 'AD', 'VV', 'DEC', 'NN', 'NR', 'AD', 'VC', 'VV'], ['JJ', 'NN', 'VV', 'LC', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['P', 'NN', 'VV', 'VV', 'AS', 'VV', 'NN', 'LC', 'AD', 'VV', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'LC', 'VV', 'DEC', 'NN', 'AD', 'VV', 'CD'], ['NR', 'P', 'NN', 'NN', 'DEG', 'NN', 'LC', 'AD', 'NN'], ['AD', 'VE', 'NN', 'AD', 'VV', 'PN', 'DT', 'NN', 'CD', 'NN', 'AD', 'VE', 'CD', 'NN', 'VV', 'VV', 'PN', 'DEC', 'NN'], ['NN', 'LC', 'DEG', 'NN', 'NN', 'VV', 'NN', 'LC', 'AD', 'AD', 'VC', 'LB', 'NN', 'PN', 'VV'], ['NT', 'P', 'NN', 'NN', 'NN', 'VV', 'NN', 'VV', 'PN', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'JJ', 'NN', 'AD', 'P', 'NR', 'DEG', 'NN', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['NR', 'VV', 'LC', 'NR', 'DEG', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'NR', 'NR', 'CD', 'NN'], ['NR', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['PN', 'DEG', 'NN', 'VE', 'CD', 'NN', 'AD', 'AD', 'NN'], ['P', 'PN', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'SP'], ['VV', 'NN', 'JJ', 'NN', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'AD', 'VV', 'NR'], ['NN', 'NN', 'VV', 'NT', 'NN', 'NN', 'P', 'AD', 'NN', 'VV'], ['NR', 'NN', 'NT', 'VV', 'NN', 'AD', 'VV', 'AS', 'PN', 'CC', 'NN', 'DEG', 'NN'], ['P', 'NN', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'VV', 'BA', 'NN', 'VV', 'VV'], ['P', 'JJ', 'NN', 'LC', 'AD', 'VV', 'CD', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['NT', 'AD', 'NN', 'AD', 'VA', 'DEC', 'AD', 'CD', 'JJ', 'NN', 'DEV'], ['NT', 'NT', 'VV', 'NR', 'NR', 'NN', 'DEG', 'NN'], ['AD', 'VC', 'NN', 'VV', 'AS', 'NR', 'AD', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NN', 'SP', 'PN', 'VC', 'PN', 'CC', 'NN'], ['NN', 'VV', 'AD', 'VA', 'AD', 'VC', 'VV', 'DEC', 'NN', 'VV', 'AS', 'SP'], ['PN', 'CS', 'VA', 'SP', 'AD', 'VV', 'AD', 'PN', 'AD', 'VV', 'AS'], ['VV', 'NN', 'VV', 'VC', 'NR', 'VV', 'VV', 'VV', 'SP'], ['VV', 'NT', 'JJ', 'NN', 'P', 'VV', 'NN'], ['AD', 'VA', 'DEC', 'JJ', 'NN', 'VV', 'VV', 'AD', 'VA'], ['AD', 'CD', 'NN', 'AD', 'NN', 'VE', 'CD', 'VV'], ['AD', 'BA', 'PN', 'VV', 'VA', 'DEC', 'NN', 'NT', 'VV', 'AS'], ['NR', 'VV', 'VV', 'PN', 'AD', 'VV', 'DT', 'NN'], ['NR', 'CC', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['AD', 'AD', 'PN', 'VA', 'DEV', 'VV', 'VV', 'NN', 'VV'], ['VC', 'VV', 'AD', 'VA', 'AD', 'VV', 'VC', 'PN', 'NN'], ['NN', 'NN', 'P', 'NN', 'LC', 'AD', 'VV', 'VV', 'VV'], ['PN', 'NT', 'AD', 'P', 'PN', 'AD', 'VV', 'ON', 'ON'], ['NN', 'PN', 'NT', 'AD', 'VV', 'VV', 'PN', 'PN', 'VV', 'SP'], ['NN', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'CC', 'NN'], ['P', 'JJ', 'NN', 'LC', 'AD', 'PN', 'VC', 'VV'], ['AD', 'LC', 'NN', 'P', 'NN', 'VV', 'VV'], ['P', 'VV', 'NR', 'NN', 'VV', 'AS', 'CD', 'NN'], ['AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NR'], ['PN', 'VV', 'DEC', 'VC', 'DT', 'NN', 'DEG', 'NN', 'SP'], ['NN', 'LC', 'DEG', 'NN', 'VV', 'AS', 'VV', 'DEC', 'NN', 'AD', 'PN', 'AD', 'AD', 'VV', 'PN', 'P', 'NN', 'AD', 'VE', 'NN'], ['VV', 'NN', 'CC', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'VA', 'CC', 'VA', 'DEC', 'NN', 'VV', 'DEC', 'VC', 'PN', 'SP'], ['AD', 'NR', 'DEG', 'NN', 'AD', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'PN', 'AD'], ['VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'CD', 'NN', 'NN', 'NN', 'AD', 'VA'], ['P', 'NN', 'VV', 'NR', 'JJ', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['NN', 'VV', 'NR', 'NT', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'AD', 'VA', 'VV'], ['NN', 'VV', 'PN', 'AD', 'AD', 'AD', 'AD', 'PN', 'VC', 'P', 'NN', 'SP'], ['P', 'NN', 'VV', 'VV', 'VC', 'JJ', 'NN', 'VV', 'DER', 'VE', 'NN'], ['NR', 'NN', 'LC', 'VE', 'CD', 'NR', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN', 'CC', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'AD', 'VC', 'VA', 'DEC', 'VV', 'NN', 'SP'], ['VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['VA', 'DEC', 'NN', 'AD', 'DEV', 'VV', 'NN', 'NN'], ['NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'AD', 'NN', 'NR'], ['P', 'VV', 'NN', 'LC', 'NR', 'AD', 'VV', 'AS', 'NR', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'AS', 'NR', 'NN', 'NN', 'AD', 'AD', 'VV'], ['AD', 'NR', 'VV', 'NR', 'VV', 'VV', 'CD', 'NN', 'P', 'PN', 'VV'], ['VV', 'VV', 'AD', 'VV', 'VV', 'VV', 'CD', 'NN', 'VV', 'PN', 'VV', 'SP'], ['DT', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV'], ['AD', 'VV', 'NR', 'NN', 'AD', 'DEV', 'VV', 'VV', 'AS', 'NN'], ['VE', 'NN', 'VV', 'AD', 'NN', 'P', 'NN', 'VV', 'NT', 'NN'], ['VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV'], ['AD', 'NN', 'P', 'NN', 'NN', 'DEG', 'VV', 'VV', 'AD', 'VA'], ['NN', 'NN', 'DEC', 'AD', 'VV', 'P', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['DT', 'VV', 'VV', 'NN', 'VC', 'VV', 'NN', 'DEC', 'DT', 'NN', 'AD', 'AD', 'VV'], ['NN', 'NN', 'VV', 'NN', 'NR', 'JJ', 'DEG', 'NN', 'LB', 'NN', 'VV'], ['NN', 'VV', 'NR', 'AD', 'VV', 'NN', 'NN', 'NT', 'CD', 'M', 'NN'], ['NN', 'VC', 'NN', 'CS', 'NN', 'AD', 'VE', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['NN', 'SB', 'AD', 'VV', 'NN', 'VA', 'VV', 'NN', 'NN'], ['AD', 'AD', 'BA', 'NN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'AD'], ['PN', 'CC', 'CD', 'SP', 'AD', 'AD', 'VV', 'PN', 'NN', 'AD', 'VV', 'PN', 'SP'], ['NT', 'DEG', 'NN', 'NN', 'AD', 'VA', 'AD', 'NN', 'AD', 'VV', 'VA', 'DEV', 'VV'], ['AD', 'PN', 'AD', 'VV', 'VV', 'M', 'NN', 'VV', 'M', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'VA'], ['DT', 'NN', 'VV', 'JJ', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN', 'VV', 'P', 'NN', 'NN'], ['VV', 'VE', 'NN', 'AD', 'CC', 'NN', 'SP'], ['NR', 'NN', 'VV', 'AD', 'NR', 'NN', 'NN', 'VV', 'VV', 'VC', 'SP'], ['P', 'AD', 'VV', 'DEC', 'NN', 'LC', 'AD', 'JJ', 'NN', 'VV', 'P', 'NN', 'NN'], ['NT', 'DEG', 'NR', 'NN', 'NN', 'P', 'NN', 'AD', 'VV', 'DEC', 'NN', 'MSP', 'AD', 'VA', 'VA'], ['NR', 'NN', 'LC', 'AD', 'CD', 'NN', 'NN', 'NN', 'VA', 'NN', 'VA', 'DEC', 'NN'], ['NR', 'NN', 'P', 'NN', 'VV', 'AS', 'AD', 'PN', 'CC', 'NR', 'NN', 'NR', 'DEG', 'NN', 'NN'], ['NR', 'NN', 'P', 'VV', 'NR', 'NN', 'AD', 'VV', 'AS', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'PN', 'VV', 'VV', 'NN', 'VV', 'NR'], ['NR', 'NN', 'P', 'VV', 'DEC', 'NR', 'NN', 'VV'], ['AD', 'NN', 'P', 'AD', 'VV', 'VV', 'VV', 'VV'], ['AD', 'NN', 'VC', 'NR', 'VV', 'AS', 'NR', 'VV', 'NN', 'SB', 'VV'], ['PN', 'NT', 'P', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NR', 'VV'], ['NT', 'NT', 'AD', 'BA', 'DT', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['NN', 'AD', 'VV', 'VV', 'AD', 'VV', 'NN', 'NN', 'DEG', 'DT', 'P', 'M', 'NN'], ['AD', 'NR', 'NR', 'NN', 'BA', 'NR', 'NN', 'NN', 'VV', 'VV', 'NN'], ['P', 'NN', 'NN', 'VV', 'JJ', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['PN', 'AD', 'AD', 'VV', 'DEC', 'NR', 'NN', 'NR', 'SP'], ['NR', 'NN', 'VC', 'NR', 'NN', 'NN', 'VV', 'P', 'NN', 'NN'], ['AD', 'P', 'NN', 'AD', 'VA', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['NR', 'DEG', 'NN', 'NR', 'P', 'NN', 'LC', 'VV', 'AS', 'JJ', 'NN'], ['NN', 'JJ', 'NN', 'NR', 'VV', 'NN', 'NN', 'VV', 'NN'], ['AD', 'CD', 'M', 'PN', 'AD', 'VV', 'NN', 'LC', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'VV', 'AD', 'VV', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NR', 'VV', 'NR', 'NN', 'DEC', 'NN', 'VV', 'VV', 'PN', 'CD'], ['PN', 'AD', 'VV', 'AD', 'CD', 'NN', 'DT', 'AD', 'VA', 'DEC', 'NN', 'NN', 'VV'], ['NR', 'VV', 'PN', 'VV', 'NR', 'NN', 'AD', 'VA', 'AD', 'VA'], ['AD', 'NR', 'AD', 'VV', 'JJ', 'DEG', 'NN', 'BA', 'PN', 'P', 'P', 'NN'], ['NR', 'NN', 'LC', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['NT', 'VV', 'AD', 'P', 'PN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'VV', 'NR', 'VV', 'VV', 'NN', 'VV'], ['NR', 'VV', 'NT', 'PN', 'VV', 'NN', 'NN', 'AD', 'VE', 'AS', 'NN'], ['PN', 'VA', 'DEC', 'NN', 'VV', 'VV', 'AD', 'AD', 'VA', 'AD', 'VV', 'AS', 'NN'], ['NN', 'DEG', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN', 'VA'], ['NR', 'NN', 'AD', 'VV', 'VC', 'VV', 'CC', 'VV', 'NN', 'VV', 'CC', 'NN', 'NN'], ['NN', 'AD', 'NN', 'P', 'NN', 'LC', 'AD', 'VE', 'NN', 'CC', 'NN', 'VA'], ['NR', 'VV', 'NR', 'NR', 'NN', 'DEG', 'NR', 'NN', 'VV', 'NN', 'VA'], ['VV', 'AS', 'VV', 'AS', 'NN', 'SP', 'AD', 'AD', 'VV', 'VA', 'CD'], ['NN', 'NN', 'VV', 'BA', 'DT', 'NN', 'NN', 'VV', 'VV', 'CD', 'JJ', 'DEG', 'NN'], ['NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN', 'ETC', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'VV', 'AD', 'VC', 'NN', 'JJ', 'DEG', 'NN', 'NN'], ['P', 'NN', 'PN', 'SB', 'VV', 'AS', 'NN', 'VV', 'NR', 'NN', 'NN', 'SP'], ['NT', 'NT', 'NR', 'NR', 'LB', 'CD', 'M', 'NN', 'VV', 'VV'], ['NR', 'DEG', 'NN', 'NT', 'P', 'NR', 'NR', 'VV', 'NN', 'NN'], ['NN', 'NN', 'NT', 'DEG', 'NN', 'VC', 'NT', 'CD', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'AD', 'LC', 'AD', 'AD', 'VV'], ['AD', 'NT', 'VC', 'JJ', 'NN', 'NN', 'VV', 'NT', 'JJ', 'CC', 'M', 'CC', 'PN', 'SP'], ['P', 'NT', 'NN', 'VV', 'NR', 'OD', 'NN', 'DEG', 'NN', 'NN', 'VC', 'AD', 'VA', 'SP'], ['NR', 'VV', 'VV', 'DEC', 'NN', 'NN', 'CD', 'JJ', 'NN', 'IJ', 'DT', 'M', 'NN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'VV', 'VV'], ['P', 'NN', 'NN', 'ETC', 'NN', 'VV', 'NN', 'NN', 'P', 'NT', 'CC', 'NT', 'VV', 'VC', 'CD', 'SP'], ['P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'NN', 'NR', 'VV', 'DT', 'AD', 'VV'], ['NN', 'AD', 'P', 'JJ', 'DEG', 'NN', 'MSP', 'VV', 'AD', 'P', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'AD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AS', 'VV', 'SP', 'VV', 'SP'], ['NN', 'CS', 'VE', 'VV', 'NN', 'NN', 'VA', 'AD', 'NN', 'AD', 'AD', 'VV', 'CC', 'VV'], ['NN', 'NN', 'VV', 'NN', 'NN', 'NR', 'VV', 'AD', 'VV', 'AS', 'VV', 'NN', 'NN', 'VV'], ['AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'VV', 'VV', 'NN', 'VV', 'AS', 'NR', 'NN', 'P', 'NR', 'DEG', 'NN', 'NN'], ['NT', 'VV', 'DEC', 'NN', 'AD', 'VA', 'DEV', 'VV', 'VC', 'NN', 'NN'], ['NN', 'VV', 'VC', 'IJ', 'JJ', 'JJ', 'NN', 'AD', 'VV'], ['IJ', 'AD', 'PN', 'AD', 'VV', 'SP', 'AD', 'PN', 'P', 'PN', 'AD', 'VV', 'VV', 'SP'], ['P', 'NN', 'LC', 'DEG', 'NN', 'NN', 'NN', 'VV', 'NR', 'DEG', 'JJ', 'NN'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN'], ['VV', 'PN', 'DEG', 'PN', 'VV', 'P', 'PN', 'VV', 'SP'], ['VE', 'NN', 'NN', 'DEG', 'NN', 'VV', 'AS'], ['VV', 'AS', 'PN', 'DEG', 'NN', 'PN', 'VV', 'VV', 'DER', 'AD', 'VA'], ['PN', 'VV', 'AS', 'NN', 'NN', 'VV', 'DEC', 'NN', 'SP'], ['NN', 'CS', 'P', 'NN', 'AD', 'VE', 'VA', 'DEC', 'NN', 'AD', 'AD', 'AD', 'VV'], ['NR', 'NR', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV', 'NN'], ['P', 'PN', 'VV', 'JJ', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VE', 'NN', 'AD', 'SB', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'PN', 'DT', 'VV', 'DEC', 'NN', 'CC', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VA'], ['AD', 'NR', 'NN', 'VV', 'AS', 'NN'], ['VV', 'PN', 'DEG', 'NN', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'BA', 'NN', 'VV'], ['AD', 'NN', 'NR', 'CD', 'NN', 'VV', 'PN', 'AD', 'VC', 'M', 'NN', 'NN'], ['NN', 'VV', 'LC', 'NN', 'VV', 'AS', 'NR', 'NR', 'NN'], ['PN', 'CS', 'VV', 'PN', 'VC', 'VA', 'PN', 'CS', 'NN', 'PN', 'VC', 'NN'], ['IJ', 'VA', 'PN', 'AD', 'AD', 'BA', 'CS', 'NN', 'VV', 'SP'], ['NN', 'NN', 'NN', 'NN'], ['P', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'DEC', 'NN', 'P', 'NN', 'LC', 'NN', 'VV', 'VV', 'NN'], ['AD', 'VV', 'M', 'NN', 'PN', 'AD', 'VV', 'M'], ['NN', 'VV', 'AS', 'PN', 'NN', 'VV', 'CD', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['PN', 'P', 'PN', 'NN', 'VV', 'VV', 'PN', 'VV', 'CD', 'NN', 'AD', 'P', 'PN', 'VV'], ['AD', 'P', 'NN', 'NN', 'PN', 'SB', 'VV', 'NN', 'VV', 'AS', 'NN', 'SP'], ['NR', 'PN', 'VC', 'CD', 'NN', 'NN', 'NN'], ['PN', 'VV', 'AS', 'NN', 'AD', 'AD', 'P', 'PN', 'VV', 'AS', 'AD', 'NN'], ['PN', 'VV', 'AS', 'NN', 'NN', 'VV', 'P', 'NT', 'NT', 'CC', 'NT', 'VC', 'NN', 'SP'], ['NT', 'NT', 'JJ', 'NN', 'VC', 'NT', 'CD', 'CD', 'CD', 'CD', 'M'], ['NN', 'DEG', 'NN', 'P', 'NR', 'VV', 'VA', 'NN'], ['NT', 'NN', 'VC', 'VA', 'VA', 'DEC', 'NN', 'NN'], ['NR', 'VV', 'AD', 'AD', 'VV', 'VV'], ['VV', 'PN', 'AD', 'VV', 'PN'], ['PN', 'VV', 'AS', 'AD', 'PN', 'DEG', 'NN', 'AD', 'DT', 'NN', 'VV', 'VE', 'DEC', 'AD', 'VA', 'DEC', 'NN'], ['NN', 'NN', 'VV', 'DEV', 'VV', 'AS', 'NN', 'NN', 'CC', 'NN', 'DEG', 'NN', 'NN'], ['PN', 'NN', 'JJ', 'DEG', 'NN', 'P', 'VV', 'NN', 'VV', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'DER', 'VV', 'PN', 'VV', 'M'], ['NT', 'P', 'NN', 'AD', 'AD', 'VA'], ['AD', 'AD', 'NN', 'VA', 'DEV', 'VV', 'VV', 'NN'], ['NR', 'NN', 'AD', 'NN', 'P', 'NR'], ['AD', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV', 'SP'], ['NT', 'AD', 'PN', 'P', 'NR', 'VV', 'NR', 'MSP', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['JJ', 'NN', 'VV', 'VV', 'NR', 'DEG', 'NN', 'LC', 'VV', 'CD', 'NN', 'P', 'PN', 'VV', 'LC', 'NN'], ['VV', 'NN', 'P', 'PN', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'CD', 'CD'], ['PN', 'VC', 'NN', 'NN', 'NN', 'AD', 'VV', 'CD', 'NN', 'AD', 'VV', 'AS', 'PN'], ['P', 'NN', 'CD', 'NN', 'P', 'NR', 'NR', 'NN', 'ETC', 'NN', 'AD', 'AD', 'VA'], ['NN', 'NN', 'NN', 'NN', 'VV', 'AS', 'VV', 'PN', 'VV', 'NN'], ['NN', 'AD', 'NN', 'VV', 'NN', 'LC', 'VV', 'VV', 'VV', 'LC', 'VV', 'NN', 'AD', 'VV'], ['NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'CC', 'NN', 'NN', 'CC', 'NN', 'VV', 'PN'], ['NR', 'VC', 'NN', 'NN', 'CC', 'NN', 'JJ', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'P', 'NN', 'VV'], ['NR', 'NN', 'NN', 'NN', 'NN'], ['NT', 'VC', 'AD', 'PN', 'PN', 'AD', 'VV', 'AS'], ['VV', 'VV', 'PN', 'NN', 'PN', 'NN', 'VA', 'NN'], ['NR', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'NN', 'NN', 'DEC', 'NN'], ['NT', 'NT', 'NR', 'NT', 'VV', 'NN', 'NN', 'VV', 'NN'], ['AD', 'PN', 'AD', 'VV', 'AD', 'VV', 'NN', 'PN', 'VE', 'PN', 'DEG', 'NN'], ['P', 'DT', 'NN', 'LC', 'VV', 'DEC', 'PN', 'AD', 'VV', 'P', 'JJ', 'VV', 'DEC', 'NN', 'VV'], ['NN', 'VV', 'DEC', 'NR', 'NN', 'NN', 'NN', 'NT', 'P', 'NR', 'VV', 'NN', 'VV'], ['P', 'NN', 'NN', 'NN', 'NN', 'LC', 'AD', 'VV', 'NN'], ['NN', 'NR', 'NN'], ['NN', 'VV', 'DEC', 'AD', 'VA', 'DEC', 'VC', 'NN', 'AD', 'VV', 'VA', 'VA'], ['AD', 'NR', 'DEG', 'NN'], ['PN', 'NT', 'VV', 'SP'], ['NN', 'AD', 'VV', 'SP'], ['PN', 'AD', 'VV', 'PN', 'CC', 'NN'], ['NT', 'AD', 'VA', 'DEC', 'NN', 'NN', 'SP', 'VV', 'AD', 'AD', 'VV'], ['IJ', 'VV', 'SP', 'AD'], ['VV', 'VV', 'NN', 'NN', 'SP'], ['PN', 'AD', 'AD', 'VV', 'VV', 'SP'], ['DT', 'NN', 'VV', 'VA'], ['NN', 'LC', 'AD', 'NN', 'VV'], ['PN', 'AD', 'AD', 'VA', 'SP'], ['PN', 'AD', 'AD', 'VV', 'NN', 'SP'], ['PN', 'AD', 'AD', 'JJ', 'NN', 'PN'], ['VV', 'DT', 'NN', 'DEC', 'NN', 'AD'], ['NR', 'VV', 'LC', 'VV', 'SP'], ['PN', 'DT', 'NN', 'SP'], ['VV', 'VV', 'ON'], ['VV', 'PN', 'AD', 'AD', 'VV', 'VV', 'SP'], ['PN', 'AD', 'AD', 'VV', 'NN', 'VV', 'SP'], ['PN', 'AD', 'VV', 'SP'], ['PN', 'AD', 'AD', 'PN', 'VA'], ['PN', 'VV', 'AD', 'VV', 'PN'], ['P', 'VV', 'VV', 'NN', 'SP'], ['PN', 'PN', 'VC', 'AD', 'VV', 'PN', 'SP'], ['VV', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'NN', 'VA'], ['BA', 'NN', 'VV', 'VV'], ['PN', 'VV', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'VV'], ['VV', 'VV', 'P', 'PN', 'VV', 'VV'], ['NR', 'AD', 'VV'], ['NT', 'NR', 'AD', 'LB', 'VV', 'NN', 'NN', 'AD', 'VV'], ['VV', 'AS', 'VV', 'AS', 'PN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'SP'], ['VV', 'VA', 'AS', 'AD', 'VA'], ['NN', 'CD', 'NN', 'AD', 'AD', 'VV', 'PN', 'VC', 'CD', 'NN'], ['VA', 'DEC', 'NN', 'VV', 'LC'], ['AD', 'VV', 'AD', 'JJ', 'PN'], ['DT', 'NN', 'AD', 'SB', 'VV'], ['PN', 'AD', 'VV', 'NN', 'NN'], ['NR', 'AD', 'VC', 'PN'], ['VV', 'AD', 'VE', 'NN', 'SP'], ['VV', 'NN', 'SP'], ['VV', 'PN', 'NN', 'PN', 'VV'], ['NR', 'P', 'NR', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV'], ['VA', 'DEC', 'NN', 'VV', 'PN'], ['PN', 'VV', 'DEV', 'AD', 'VV'], ['VA', 'CC', 'VA', 'DEC', 'NN', 'AD', 'VV', 'VV', 'NN'], ['CS', 'NN', 'VV', 'VE', 'SB', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'NN', 'VV', 'SP'], ['DT', 'NN', 'AD', 'AD', 'VA'], ['NN', 'VV', 'NR', 'NN'], ['NR', 'VV', 'VV'], ['PN', 'AD', 'AD', 'VV', 'VV'], ['P', 'NN', 'NN', 'JJ', 'NN', 'AD', 'DEV', 'VV', 'AS'], ['DT', 'NN', 'PN', 'VV', 'VC', 'AD', 'VV', 'JJ', 'NN'], ['NT', 'NR', 'NN', 'NN', 'VA'], ['VV', 'NN', 'VV', 'VV', 'NN', 'SP'], ['VV', 'NT', 'NR', 'NN', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VA'], ['VC', 'AD', 'VV', 'NN', 'DEG', 'SP', 'PN', 'VV', 'AD', 'VV', 'SP'], ['NR', 'NN', 'VV', 'VV', 'AS', 'NR', 'NR', 'DEG', 'NN'], ['PN', 'VV', 'VV', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV'], ['PN', 'LB', 'NN', 'VV', 'P', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'AD', 'VV', 'NN', 'VV', 'NN', 'BA', 'NN', 'VV', 'AS', 'NN', 'NN'], ['PN', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'SP'], ['P', 'VV', 'NR', 'VV', 'NN', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'AD', 'VV', 'AD', 'AS', 'CD', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN'], ['NR', 'VV', 'SB', 'VV', 'DER', 'AD', 'VV', 'AD', 'VV', 'NN'], ['AD', 'DT', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'DEV', 'VV', 'NN'], ['NN', 'LC', 'NN', 'VV', 'AD', 'VV'], ['NT', 'NN', 'BA', 'PN', 'AD', 'VV', 'P', 'NN'], ['P', 'AD', 'VV', 'PN', 'VV', 'VV', 'NN'], ['AD', 'VV', 'NN', 'DEG', 'NN', 'P', 'NN', 'VV'], ['AD', 'AD', 'DEC', 'VC', 'NR', 'AD', 'VC', 'NR', 'NN'], ['NT', 'PN', 'CC', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VA', 'DEC', 'NN', 'NN', 'NN', 'VV'], ['NN', 'VV', 'NN', 'VV', 'AS', 'CD', 'NN'], ['VV', 'NN', 'LC', 'JJ', 'DT', 'VV', 'AS', 'NN'], ['P', 'NN', 'VV', 'NR', 'VC', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'VV', 'CD', 'NN'], ['PN', 'CC', 'NN', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'P', 'PN', 'VV', 'VV'], ['NN', 'LC', 'AD', 'VV', 'VV', 'NN'], ['AD', 'P', 'NT', 'PN', 'SB', 'NN', 'VV', 'AS', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'NN', 'JJ', 'NN'], ['NN', 'NN', 'P', 'NN', 'NN', 'NN'], ['NN', 'CC', 'NN', 'NN', 'VC', 'NN', 'NN', 'NN', 'VV', 'LC', 'AD', 'VV'], ['NN', 'NN', 'P', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['NR', 'VV', 'VV', 'NN', 'AD', 'PN', 'CC', 'NR', 'VV', 'VV', 'VV'], ['NR', 'NR', 'AD', 'NN', 'VE', 'NN', 'VV'], ['NR', 'NR', 'NN', 'P', 'NN', 'VV', 'VV'], ['AD', 'DT', 'NN', 'P', 'NN', 'AD', 'VA'], ['VV', 'DEC', 'NN', 'LC', 'NR', 'P', 'NR', 'VV', 'NN', 'VV', 'VV', 'AD'], ['CD', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'LC', 'AD', 'AD', 'VV', 'VV', 'NN'], ['NN', 'VV', 'NR', 'VV', 'P', 'VE', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['AD', 'NN', 'NN', 'NN', 'CC', 'VV', 'NN', 'AD', 'P', 'NN', 'NN', 'LC', 'AD', 'VV', 'AS', 'NN'], ['PN', 'VV', 'AS', 'NN', 'AD', 'VV', 'VV', 'AS', 'AD', 'AD', 'VV', 'NN', 'LC', 'DEG', 'NR'], ['NN', 'NN', 'NN', 'AD', 'BA', 'PN', 'DEG', 'NN', 'VV', 'VV', 'NN'], ['AD', 'CD', 'CD', 'NN', 'DEG', 'NN', 'AD', 'VC', 'P', 'NT', 'NT', 'VV'], ['VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VC', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN'], ['VV', 'NN', 'AD', 'VV', 'NN', 'MSP', 'VV', 'VV', 'NN'], ['NN', 'NN', 'P', 'P', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['P', 'NN', 'NN', 'NT', 'VV', 'NN', 'AD', 'VV', 'CD', 'NN'], ['AD', 'NN', 'NT', 'P', 'NN', 'NN', 'NN', 'VV', 'DEC'], ['NR', 'AD', 'VV', 'NR', 'DEG', 'CD', 'JJ', 'DEG', 'NN', 'NN'], ['AD', 'NR', 'AD', 'P', 'NN', 'VV', 'VV', 'VV', 'VV'], ['VV', 'P', 'PN', 'VV', 'DEC', 'PN', 'VV', 'VC', 'CD', 'NN'], ['AD', 'NN', 'VV', 'NR', 'DEC', 'NN', 'AD', 'VV', 'NR', 'NN'], ['NN', 'DEG', 'VA', 'LC', 'NR', 'VV', 'VE', 'CD', 'VV', 'DEC', 'NR'], ['PN', 'AD', 'VV', 'NR', 'DEC', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['VV', 'NN', 'AD', 'VV', 'VA', 'DEC', 'NN', 'VV', 'AD', 'VV'], ['NN', 'CS', 'VV', 'AS', 'VV', 'VV', 'VV'], ['PN', 'CS', 'PN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'PN', 'VV', 'AS'], ['VV', 'PN', 'PN', 'P', 'NN', 'VV', 'DER', 'VA', 'AD'], ['PN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV'], ['NN', 'AD', 'VV', 'P', 'NR', 'DEG', 'NN'], ['PN', 'VV', 'AS', 'VV', 'LC', 'AD', 'VV', 'AS', 'AD'], ['P', 'NN', 'VV', 'VV', 'NN', 'VE', 'SP'], ['NR', 'VV', 'NN', 'AD', 'VV', 'AD', 'P', 'NN', 'NN', 'NN', 'VV'], ['NR', 'OD', 'NN', 'NN', 'VC', 'VV', 'AD', 'VV', 'NN', 'NN', 'DEC', 'NN', 'NN'], ['P', 'NR', 'DEG', 'NN', 'AD', 'VA', 'AD', 'VV', 'VV', 'PN', 'DEG', 'JJ', 'NN', 'DEG', 'NN'], ['P', 'DT', 'NN', 'LC', 'NR', 'CC', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN'], ['CS', 'AD', 'VV', 'VV', 'JJ', 'NN', 'AD', 'AD', 'AD', 'VV', 'BA', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['NN', 'VV', 'MSP', 'AD', 'VV', 'PN', 'PN', 'P', 'NN', 'DEG', 'NN', 'VE', 'DT', 'NN'], ['DT', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'DT', 'NR', 'NN'], ['AD', 'AD', 'VV', 'NN', 'VV', 'LC', 'AD', 'VV', 'AS', 'NN', 'DEC', 'VV'], ['PN', 'VV', 'PN', 'VV', 'PN', 'AD', 'VV', 'AD', 'P', 'PN', 'CD', 'NN', 'NN', 'VV'], ['NN', 'NN', 'NN', 'DEG', 'NN', 'NR', 'NN', 'VV', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['NR', 'NR', 'VV', 'PN', 'VC', 'M', 'VV', 'NN', 'DEG', 'NN'], ['VV', 'AD', 'AD', 'AD', 'CD', 'NN', 'AD', 'LB', 'PN', 'VV', 'AS'], ['NN', 'LC', 'PN', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'VV', 'NR', 'NN', 'NN', 'NR', 'NN', 'NN'], ['AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NR', 'NN', 'NN', 'VC', 'NT', 'NT', 'CD', 'NN', 'VV', 'NN', 'DEG', 'NT', 'CD'], ['NR', 'VV', 'NR', 'NN', 'AD', 'VV', 'NR', 'NN', 'VV', 'NN'], ['NT', 'NT', 'VV', 'NN', 'VE', 'CD', 'NN', 'P', 'NN', 'NN', 'VV', 'PN', 'AD', 'VV', 'PN', 'SP', 'SP'], ['NR', 'NR', 'VV', 'NR', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'P', 'NN', 'NN', 'LC', 'VV'], ['NN', 'NN', 'CC', 'JJ', 'NR', 'NN', 'NN', 'NN', 'NN', 'VV', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['VV', 'VV', 'P', 'PN', 'VV', 'SP'], ['PN', 'VV', 'VV', 'AD', 'NN', 'VV', 'NN'], ['AD', 'PN', 'VV', 'DEC', 'NN', 'CC', 'NN', 'AD', 'VA'], ['AD', 'DEV', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN'], ['NN', 'P', 'AD', 'VV', 'DEC', 'NN', 'VV'], ['VV', 'AD', 'VV', 'PN', 'VV', 'AD', 'VV', 'NN'], ['PN', 'AD', 'AD', 'CC', 'NR', 'MSP', 'VV', 'VV', 'DEC'], ['VV', 'AD', 'VV', 'AS', 'NN', 'DEG', 'VV', 'LC', 'AD', 'VV'], ['PN', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV', 'VV'], ['VV', 'NN', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN'], ['AD', 'NN', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'NN', 'NR', 'AD', 'VV', 'VE', 'NN'], ['PN', 'VC', 'AD', 'CD', 'NN', 'SP', 'AD', 'P', 'NN', 'VA'], ['VV', 'LC', 'AD', 'VV', 'NN', 'AD', 'VV', 'AS', 'CD'], ['CD', 'VC', 'NT', 'CD', 'CD'], ['AD', 'VV', 'NN', 'DEC', 'NN', 'VC', 'VA', 'DEC', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NN', 'JJ', 'NN', 'VV', 'VV', 'VV'], ['VV', 'VV', 'NR', 'NT', 'NN', 'CD', 'CD', 'NN', 'DEG', 'CD', 'JJ', 'NN'], ['AD', 'NN', 'CC', 'NN', 'VV', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'NN', 'P', 'NN', 'LC', 'SB', 'VV', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'NT', 'NN'], ['AD', 'AD', 'LB', 'NN', 'NN', 'SP'], ['VV', 'PN', 'VV', 'DT', 'NN'], ['VV', 'AD', 'VV', 'AD', 'VV'], ['AD', 'AD', 'VV', 'AS', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'SB', 'VV', 'VV', 'AS', 'CD', 'NR', 'NN', 'AD', 'SB', 'VV', 'AS'], ['VV', 'PN', 'AD', 'VV', 'VV'], ['VV', 'NN', 'CC', 'NN', 'NN', 'CD', 'M', 'CD', 'CD', 'CD', 'CD', 'VV'], ['PN', 'VV', 'DEC', 'NN', 'VC', 'PN'], ['AD', 'NR', 'AD', 'VA', 'SP'], ['AD', 'VV', 'NN', 'PN', 'AD', 'VV', 'AS', 'NN'], ['PN', 'AD', 'AD', 'LB', 'NN', 'VV', 'SP'], ['NN', 'VV', 'AS', 'PN', 'VV', 'AD', 'VV'], ['PN', 'VC', 'PN', 'AD', 'AD', 'VV', 'JJ', 'DEG', 'SP'], ['NN', 'NN', 'NN', 'LC', 'VV', 'NN', 'NN', 'AD', 'AD', 'VV'], ['VV', 'VV', 'NR', 'NN', 'DEG', 'NN', 'P', 'NN', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VA'], ['PN', 'VV', 'AS', 'PN', 'DEG', 'NN', 'PN', 'AD', 'VV'], ['AD', 'CS', 'P', 'NN', 'NN', 'PN', 'LC', 'P', 'NN', 'AD', 'VC', 'AD', 'VA', 'SP'], ['NN', 'P', 'PN', 'DEG', 'NN', 'VV', 'AS', 'M', 'NN', 'NN', 'PN', 'VV', 'PN', 'VV'], ['AD', 'VV', 'JJ', 'DEG', 'VV', 'AS', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'AS', 'NN', 'VV', 'VV'], ['NR', 'NN', 'DEG', 'NN', 'VV', 'PN', 'AD', 'VV', 'SP'], ['AD', 'P', 'NN', 'AD', 'VV', 'MSP', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN'], ['VV', 'AD', 'VV', 'VV', 'VV', 'PN', 'DEG'], ['VV', 'AS', 'CD', 'NN', 'DEG', 'NN', 'SP', 'AD', 'VV', 'NN', 'SP', 'AD', 'VV', 'AD'], ['AD', 'VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'NR', 'VV', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'VV', 'VV'], ['AD', 'SB', 'PN', 'VV', 'DEC', 'DT', 'NN', 'VV', 'AS', 'AD', 'AD', 'AD', 'VV', 'PN', 'AD', 'VV', 'SP', 'VV', 'VV'], ['NN', 'DEG', 'NN', 'VV', 'PN', 'VV', 'DEC', 'NN', 'VC', 'NN', 'NN'], ['AD', 'AD', 'VV', 'JJ', 'NN', 'SP'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'PN'], ['AD', 'AD', 'AD', 'AD', 'VV', 'SP'], ['PN', 'VV', 'VA', 'PN', 'AD', 'AD', 'VV'], ['DT', 'NN', 'AD', 'VA'], ['NN', 'NN', 'AD', 'VV', 'NN'], ['P', 'NN', 'AD', 'NN', 'NN'], ['NN', 'VV', 'VV', 'JJ', 'NN', 'VV', 'VV', 'NN'], ['JJ', 'NN', 'NN', 'VV', 'P', 'JJ', 'NN', 'NN', 'NN', 'LC'], ['PN', 'AD', 'AD', 'VV', 'AD', 'VV', 'AD'], ['NR', 'AD', 'AD', 'VA', 'AS', 'NN'], ['NN', 'VC', 'NN', 'DEG', 'NN'], ['AD', 'JJ', 'NN', 'DEG', 'NN', 'P', 'OD', 'AD', 'VV'], ['VV', 'PN', 'VV', 'VV', 'PN', 'AD', 'VV'], ['P', 'NN', 'VV', 'PN', 'VV', 'P', 'NN', 'NN', 'NN', 'NN', 'VV'], ['AD', 'VV', 'NR', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'VA'], ['NR', 'NT', 'VV', 'AS', 'NN'], ['AD', 'BA', 'NN', 'JJ', 'JJ', 'NN', 'JJ', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NN', 'AD', 'SB', 'NR', 'VV'], ['NN', 'LC', 'VV', 'NN', 'VV'], ['CD', 'VV', 'VV', 'VV', 'AS', 'VV'], ['NT', 'NN', 'VV', 'NN'], ['NR', 'OD', 'NN', 'AD', 'VV', 'NN', 'VE', 'NN', 'AD', 'VV'], ['VV', 'VV', 'VV', 'AD', 'VE', 'VV', 'VV', 'VV', 'NN', 'DEG', 'VV', 'VV', 'VV', 'VV', 'DEC', 'VV', 'VV'], ['NR', 'NN', 'LC', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VE', 'NN', 'LC', 'VV', 'NN', 'NN'], ['PN', 'VV', 'AS', 'NN', 'VV', 'AS', 'NN', 'AD', 'VV', 'AS', 'CD', 'NN', 'DEG', 'NN'], ['AD', 'AD', 'VE', 'NN', 'VV', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'AS', 'DEC', 'NN', 'VV'], ['NT', 'LC', 'NR', 'AD', 'P', 'NN', 'VV', 'NN', 'LC', 'VV', 'VV'], ['AD', 'AD', 'VV', 'JJ', 'NN', 'DEC', 'NN', 'AD', 'VV', 'P', 'DT', 'NN', 'VV', 'NN'], ['NR', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'AS', 'NR'], ['NR', 'VC', 'NR', 'NT', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['DT', 'M', 'DEG', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'SP'], ['NN', 'LC', 'NN', 'VV', 'NN', 'DEC', 'NR', 'NN', 'P', 'P', 'PN', 'DEG', 'NN', 'VV'], ['AD', 'VV', 'PN', 'SP', 'PN', 'AD', 'VV', 'NN'], ['NR', 'AD', 'DEV', 'VV', 'NN', 'VV', 'NN'], ['VC', 'VV', 'P', 'NN', 'VA', 'DEC', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'CD', 'NN', 'VV', 'AD', 'VV'], ['NR', 'NN', 'LC', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['PN', 'CC', 'AD', 'AD', 'VV', 'NN'], ['P', 'PN', 'VV', 'NN', 'DEC', 'NR', 'NN', 'NR', 'NN', 'NN', 'NR', 'AD', 'P', 'NT', 'AD', 'VV'], ['JJ', 'NN', 'VV', 'JJ', 'NN', 'LC', 'P', 'LC', 'VV', 'AD', 'AD', 'VV'], ['VV', 'NN', 'NN', 'AD', 'VV', 'DER', 'CD', 'VA'], ['NN', 'VV', 'VV', 'PN', 'VV', 'PN', 'VV', 'AD', 'VV', 'DT', 'NN', 'AD', 'SP', 'SP'], ['NN', 'NR', 'VV', 'NN', 'DEG', 'NN', 'CC', 'NN', 'AD', 'VC', 'PN', 'VV', 'DEC', 'NN'], ['NR', 'AD', 'VV', 'P', 'NN', 'ETC', 'PN', 'NN', 'VV', 'MSP', 'VV', 'VV', 'NR', 'P', 'VV', 'VV', 'PN', 'AD', 'VV', 'PN'], ['P', 'JJ', 'NN', 'VV', 'NR', 'NN', 'PN', 'AD', 'CD', 'M', 'AD', 'VV'], ['PN', 'DEG', 'NN', 'NN', 'AD', 'NN', 'PN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV'], ['VE', 'DEC', 'NN', 'VV', 'VV', 'NN', 'LC', 'VV', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'P', 'VV', 'NN'], ['AD', 'PN', 'AD', 'VV', 'VV'], ['CS', 'PN', 'NN', 'VV', 'NN', 'VC', 'NR', 'AD', 'AD', 'AD', 'P', 'PN', 'VV'], ['PN', 'VV', 'PN', 'VV', 'PN', 'VV', 'PN', 'AD', 'PN', 'VV', 'VV', 'PN', 'DEG', 'PN', 'VV', 'VV', 'AD'], ['NN', 'CS', 'VV', 'PN', 'VV', 'VV', 'PN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['P', 'VV', 'NN', 'VV', 'LC', 'NR', 'AD', 'VV', 'NN', 'NN', 'NN', 'VV'], ['P', 'NR', 'NN', 'NR', 'NN', 'NN', 'VV', 'AS', 'AD', 'NN', 'NN', 'NN'], ['NR', 'VV', 'NN', 'VV', 'VV', 'VV', 'AS', 'NN', 'LC', 'NN', 'BA', 'JJ', 'NN', 'VV', 'VV', 'NN'], ['NN', 'NN', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'AD', 'VV', 'AS', 'NN'], ['AD', 'NR', 'DEG', 'OD', 'NN', 'NN', 'AD', 'PN', 'AD', 'VV', 'NN', 'VV', 'AS', 'NR'], ['VV', 'AD', 'CD', 'NN', 'VV', 'PN', 'VA', 'DEV', 'VV', 'NN', 'VV', 'AS', 'PN', 'VV', 'NN'], ['JJ', 'NN', 'VE', 'NN', 'P', 'NR', 'P', 'NN', 'AD', 'VV', 'NN', 'CD', 'NN', 'ETC'], ['AD', 'AD', 'PN', 'NR', 'DEG', 'NN', 'VV', 'NN', 'SP'], ['AD', 'NN', 'VV', 'VV'], ['NR', 'LC', 'AD', 'VV', 'NR', 'AD', 'SB', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'NN', 'AD', 'VA'], ['NR', 'NR', 'NR', 'NN', 'NN', 'NN', 'CC', 'P', 'NN', 'VV', 'DEC', 'JJ'], ['CS', 'NN', 'AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'SP'], ['P', 'NR', 'NN', 'AD', 'VV', 'VV', 'NN', 'LC', 'VV', 'NN', 'DEG', 'NN'], ['P', 'NN', 'NN', 'AD', 'AD', 'VV', 'DT', 'NN', 'AD', 'VV'], ['PN', 'VV', 'VV', 'VA', 'VA', 'NN', 'DEG', 'NN', 'AD', 'VE', 'AD', 'VA'], ['AD', 'CD', 'NN', 'CD', 'NN', 'DT', 'NN', 'AD', 'AD', 'NN', 'DT', 'NN', 'VV', 'AS', 'NN', 'NN'], ['NN', 'NN', 'VV', 'AS', 'CD', 'M', 'AD', 'NN', 'VV', 'NN', 'VV', 'VC', 'NN', 'VV', 'VV'], ['NT', 'DT', 'NN', 'VV', 'NN', 'VV', 'NN', 'AD', 'AD', 'BA', 'PN', 'VV', 'AS'], ['PN', 'VV', 'AD', 'AD', 'VV', 'NN', 'VA'], ['VV', 'NN', 'NN', 'NN', 'ETC', 'NN', 'MSP', 'VV', 'NN', 'NT'], ['NR', 'VV', 'AD', 'VV', 'NN', 'CC', 'CD', 'CD', 'M'], ['NN', 'P', 'PN', 'NN', 'VV', 'AS', 'VV', 'AD', 'VV', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'PN', 'AD', 'VV', 'NN', 'VV', 'NN'], ['NN', 'VV', 'AS', 'CD', 'NN', 'AD', 'PN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['NN', 'LC', 'NN', 'VV', 'NN'], ['PN', 'VC', 'NN', 'VV', 'PN', 'VC', 'JJ', 'NN'], ['NR', 'NN', 'NR', 'VV'], ['PN', 'LC', 'AD', 'VE', 'AS', 'VV', 'PN', 'NN', 'VV', 'VV', 'NN', 'VV', 'NR', 'NN', 'NN'], ['DT', 'M', 'NN', 'P', 'VV', 'NN', 'NR', 'AD', 'VV', 'NN', 'NR'], ['NR', 'AD', 'VA', 'NR', 'DEG', 'NN', 'VV', 'NR', 'VV', 'AD', 'VA'], ['VV', 'VV', 'IJ', 'LC', 'NN', 'AD', 'VA', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'NN', 'LC', 'VV', 'NN', 'AD', 'AD', 'VV'], ['NN', 'VV', 'VE', 'NN', 'VV', 'NN', 'ETC', 'NN', 'AD', 'NT', 'AD', 'VV', 'VV', 'AS'], ['JJ', 'NN', 'NN', 'VV', 'VV', 'AD'], ['AD', 'NR', 'VV', 'AS', 'NN', 'VV', 'AS', 'DEC', 'NN', 'AD', 'JJ', 'NN', 'VV'], ['P', 'PN', 'VV', 'AS', 'VV', 'P', 'CD', 'JJ', 'NN', 'VV', 'LC', 'VV', 'AS'], ['AD', 'NR', 'NN', 'LC', 'DEC', 'NN', 'NN', 'AD', 'VA', 'PN', 'NN', 'VV'], ['NT', 'VV', 'NR', 'NN', 'VV', 'DT', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'VV', 'AS', 'PN', 'VV', 'AS', 'NR', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'AD', 'NN'], ['NN', 'NN', 'VV', 'NN', 'LC', 'VV', 'NR', 'NN', 'AD', 'VV'], ['PN', 'CC', 'NN', 'NN', 'VC', 'NR', 'NN', 'DEV', 'VV', 'NN', 'PN', 'P', 'PN', 'JJ', 'CD', 'NN'], ['NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'VV', 'VV', 'VV'], ['NR', 'NR', 'NN', 'VV', 'NN', 'AD', 'VV', 'VA', 'DEC', 'NN'], ['VE', 'CD', 'AD', 'AD', 'VV', 'VC', 'NN', 'VV', 'VE', 'CD', 'VC', 'VV', 'AD', 'VA'], ['VV', 'DEC', 'NN', 'VV', 'AS', 'DT', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NR', 'NN'], ['DT', 'P', 'NN', 'LC', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN', 'NN'], ['AD', 'P', 'NN', 'NN', 'AD', 'VC', 'VV', 'LC', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN'], ['PN', 'VV', 'VC', 'NN', 'P', 'NN', 'DEG', 'NN', 'LC', 'P', 'PN', 'VV', 'AS', 'VV', 'NN'], ['VV', 'NN', 'LC', 'NN', 'VV', 'CD', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV'], ['AD', 'VV', 'VV', 'AS', 'NR', 'NN', 'AD', 'CD', 'NN', 'AD', 'VV', 'AD', 'VA'], ['NN', 'NT', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'ETC', 'JJ', 'NN'], ['VV', 'VV', 'DEC', 'NN', 'NN', 'NN', 'VV', 'VV', 'AD', 'VA', 'DEV', 'VV', 'VV'], ['NN', 'NN', 'LC', 'VV', 'DEC', 'NN', 'AD', 'VC', 'NR', 'DEG', 'VV', 'AS', 'NN', 'AD', 'VC', 'JJ', 'DEG'], ['AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'PN', 'VV', 'DEC', 'NN'], ['P', 'VV', 'PN', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV', 'DT', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN', 'CC', 'NN'], ['NT', 'DEG', 'NN', 'NN', 'AD', 'SB', 'NN', 'VV', 'AS', 'NN', 'SP'], ['NR', 'CC', 'NR', 'P', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'AD', 'P', 'PN', 'NN', 'VV', 'AS', 'VV', 'DEC', 'NN'], ['PN', 'DEG', 'NN', 'LC', 'AD', 'VV', 'AS', 'NN', 'NN', 'NN', 'AD', 'VV'], ['PN', 'AD', 'VV', 'VV', 'DEC', 'NN', 'NN', 'SP', 'NN', 'NN', 'AD', 'VV', 'DEC', 'VC', 'PN'], ['NN', 'VV', 'LC', 'NN', 'VV', 'NN', 'VV', 'AD', 'NN', 'VA', 'DER', 'VV', 'AS', 'VV'], ['NN', 'VE', 'DT', 'NN', 'P', 'AD', 'VV', 'PN', 'NN', 'VV', 'PN', 'NN'], ['CD', 'NN', 'JJ', 'NN', 'NN', 'NN', 'VV', 'VV', 'VV', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'VV', 'CC', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'ETC'], ['NR', 'AD', 'VV', 'VV', 'PN', 'NN', 'VV', 'VV', 'PN', 'CD', 'NN', 'AD', 'VV', 'VV', 'NR'], ['NN', 'AD', 'AD', 'AD', 'VA', 'AD', 'VV', 'VV', 'AS', 'NN', 'AD', 'VV', 'LB', 'NN', 'VV'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'P', 'PN', 'VV', 'SP'], ['AD', 'AD', 'M', 'NN', 'NN', 'AD', 'AD', 'VA', 'VA', 'VV', 'VA', 'DEV', 'VV', 'NN'], ['NN', 'VE', 'NN', 'PN', 'AD', 'VV', 'PN', 'DEC', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['NR', 'NN', 'AD', 'VV', 'VV', 'BA', 'NN', 'VV', 'NN', 'AD', 'LB', 'NN', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'SP'], ['CS', 'NN', 'VC', 'AD', 'NN', 'DT', 'NT', 'NN', 'PN', 'AD', 'VV', 'AS'], ['PN', 'P', 'NR', 'NN', 'P', 'NR', 'NN', 'P', 'NN', 'NN', 'VV', 'VV', 'PN', 'JJ', 'DEG', 'NN'], ['NR', 'NN', 'NN', 'NN', 'VV', 'AD', 'NN', 'NN', 'VC', 'NR', 'NR', 'NR'], ['VC', 'VV', 'VV', 'AD', 'VV', 'VV', 'DEC'], ['PN', 'NN', 'DEG', 'NN', 'VV', 'PN', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'AD', 'NR', 'NT', 'NT', 'NN'], ['PN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'AD', 'VV', 'NN', 'AD', 'VE', 'AD', 'JJ', 'DEG', 'NN'], ['VV', 'DEC', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'AS'], ['AD', 'AD', 'P', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['NN', 'VV', 'LC', 'NR', 'NN', 'AD', 'VV', 'VV'], ['AD', 'NN', 'NN', 'NN', 'JJ', 'NN', 'VV', 'VA'], ['NN', 'AD', 'VC', 'NN', 'PN', 'VV', 'P', 'NN', 'VV', 'VV', 'NN', 'AD', 'VA', 'SP'], ['CD', 'NN', 'NN', 'AD', 'VV', 'AS', 'NR', 'NR', 'AD', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'VV', 'SP'], ['NT', 'PN', 'P', 'NR', 'NN', 'NN', 'VV', 'VV'], ['NR', 'CC', 'JJ', 'JJ', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN'], ['P', 'VV', 'DEC', 'NN', 'NN', 'AD', 'BA', 'PN', 'VV', 'AS'], ['NN', 'AD', 'AD', 'VV', 'DEV', 'VV', 'VV', 'OD', 'NN'], ['AD', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN'], ['P', 'AD', 'VV', 'CD', 'M', 'PN', 'AD', 'AD', 'DEV', 'VV'], ['P', 'VV', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV', 'AS', 'VV'], ['AD', 'VV', 'NN', 'NN', 'DEC', 'NN', 'NN', 'AD', 'P', 'PN', 'NN'], ['P', 'NN', 'NN', 'NN', 'VC', 'NR', 'NN', 'NR', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'PN', 'AD', 'VV', 'AD', 'AS', 'AD', 'VV', 'AD', 'VV'], ['NN', 'NN', 'LC', 'PN', 'VV', 'VV', 'VC', 'NN', 'NN', 'VE', 'NN'], ['NN', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'ETC', 'NN'], ['LB', 'VV', 'NN', 'DEC', 'NN', 'VA', 'DEV', 'VV', 'NN', 'LC'], ['AD', 'VV', 'VV', 'AS', 'VE', 'NN', 'AD', 'AD', 'AD', 'VV', 'PN', 'NN'], ['VV', 'CD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN'], ['AD', 'PN', 'AD', 'VV', 'VV', 'PN', 'BA', 'NR', 'NN', 'VV', 'NN', 'LC'], ['NT', 'VV', 'DEC', 'NN', 'ETC', 'AD', 'VV', 'NR', 'VV'], ['AD', 'JJ', 'SP', 'PN', 'AD', 'VE', 'JJ', 'DEG', 'VV', 'JJ', 'DEG', 'DEC', 'NN'], ['NN', 'VV', 'AD', 'VV', 'AD', 'NR', 'DT', 'VE', 'NN'], ['NN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'AD', 'VV'], ['VV', 'NN', 'NN', 'NN', 'VA', 'DEV', 'VV', 'AD', 'VV'], ['DT', 'VV', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VA'], ['P', 'NN', 'LC', 'NN', 'AD', 'VV', 'AD', 'VV', 'AS', 'NN', 'VV'], ['AD', 'VV', 'VV', 'NN', 'LC', 'AD', 'VV', 'DEV', 'VV', 'LC', 'VV', 'NN', 'NN'], ['JJ', 'NN', 'P', 'NR', 'VV', 'NN', 'DEC', 'NN', 'VV', 'NN', 'LC'], ['NT', 'NR', 'DEG', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN'], ['NN', 'NN', 'NN', 'VV', 'DEC', 'VV', 'CD', 'NN', 'DEC', 'NN'], ['AD', 'CS', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV', 'JJ', 'DEG', 'NN', 'NN'], ['NN', 'VV', 'P', 'NN', 'LC', 'AD', 'VV', 'NN', 'CC', 'NN', 'NN'], ['AD', 'NN', 'VV', 'VV', 'LC', 'VV', 'AS', 'NN', 'SB', 'VV', 'DEC', 'NN'], ['AD', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'M', 'NN', 'AD', 'VV', 'SP'], ['NN', 'NN', 'P', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN', 'AD', 'VA'], ['PN', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'NN', 'VE', 'NN', 'VV', 'SP'], ['AD', 'CD', 'NN', 'PN', 'AD', 'VV', 'P', 'VV', 'NN', 'LC', 'VV', 'NN'], ['AD', 'P', 'NT', 'NN', 'VV', 'AD', 'VV', 'P', 'NR', 'NN', 'VV', 'NN'], ['VV', 'DEC', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'LC', 'AD', 'AD', 'AD', 'AD', 'VV', 'VA', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'AD', 'VA', 'SP', 'AD', 'VV', 'VC', 'PN', 'DEG', 'SP', 'VA', 'SP'], ['AD', 'DT', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'AS', 'PN', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'VE', 'CD', 'NT', 'NN', 'VC', 'NR', 'NN', 'NN'], ['AD', 'NN', 'LC', 'VV', 'NN', 'NN', 'VV', 'AS', 'NN', 'AD', 'AD'], ['PN', 'AD', 'VV', 'AS', 'NN', 'SP', 'AD', 'AD', 'VC', 'AD', 'VV', 'VV', 'AD'], ['AD', 'NN', 'AD', 'AD', 'VV', 'NN', 'JJ', 'NN'], ['NR', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'DEC', 'NN'], ['NN', 'NN', 'VV', 'AS', 'NN', 'AD', 'VC', 'NN', 'JJ', 'NN', 'VV', 'DEC', 'NN'], ['AD', 'NN', 'AD', 'VV', 'CD', 'M', 'NN', 'VV', 'AD', 'VV', 'DEC', 'NN', 'MSP', 'PN', 'VV', 'PN', 'VV', 'VV', 'SP'], ['NR', 'VV', 'AD', 'AD', 'VV', 'NN', 'AD', 'NR', 'AD', 'VV', 'AD', 'VV'], ['PN', 'VV', 'PN', 'DT', 'NN', 'NN', 'CD', 'M', 'CD', 'NN', 'AD', 'VV', 'SP'], ['AD', 'VV', 'VV', 'VV', 'NN', 'AD', 'VC', 'P', 'NN', 'JJ', 'DEG', 'NN', 'AD', 'P', 'PN', 'CD', 'VV'], ['NT', 'NR', 'NR', 'NN', 'NN', 'NN', 'DEG', 'NR', 'NN', 'VV', 'AS', 'DT', 'CD', 'NN'], ['AD', 'BA', 'VA', 'DEC', 'NN', 'VV', 'VV', 'NN', 'NN', 'P', 'VV', 'DEC', 'NN', 'VV', 'VV', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'CC', 'NR', 'DT', 'AD', 'VA'], ['NN', 'SP', 'PN', 'AD', 'VV', 'AD', 'VV', 'JJ', 'NN', 'VV', 'NR', 'DEG', 'NN', 'AD', 'VV', 'SP'], ['AD', 'LC', 'PN', 'AD', 'VV'], ['PN', 'NN', 'VV', 'PN', 'P', 'PN', 'VV', 'VA', 'DEC', 'SP'], ['NN', 'LC', 'VV', 'NN', 'VV', 'VA'], ['NR', 'NN', 'VV', 'VE', 'JJ', 'NN'], ['NN', 'VA', 'AD', 'NN', 'NN', 'NN', 'VA', 'AD', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NR', 'NN'], ['AD', 'VV', 'DEC', 'NN'], ['AD', 'AS', 'PN', 'BA', 'NN', 'VV', 'PN', 'PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'SP'], ['P', 'JJ', 'NN', 'NN', 'NN', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['AD', 'CD', 'CD', 'NN', 'VV', 'VE', 'JJ', 'NN'], ['NR', 'VV', 'DEC', 'NN', 'VV', 'JJ', 'NN'], ['VV', 'NN', 'NN', 'LC', 'VV', 'DEC', 'AD', 'VV', 'VV'], ['NN', 'AD', 'AD', 'VV', 'VV', 'AS', 'NN'], ['PN', 'DEG', 'NN', 'VV', 'PN', 'NT', 'AD', 'VV', 'VV', 'VV'], ['PN', 'NN', 'VA', 'NN', 'VA', 'AD', 'NR', 'VV', 'AD', 'VA', 'DEC', 'NR', 'NN'], ['AD', 'NR', 'AD', 'NN', 'NR', 'NN'], ['P', 'JJ', 'NN', 'DEC', 'NN', 'AD', 'VV', 'AS', 'AD'], ['AD', 'NR', 'NN', 'DEG', 'NN', 'NN', 'VA'], ['NR', 'JJ', 'NN', 'NN', 'NR', 'VV', 'NT', 'DEG', 'JJ', 'NN', 'VC', 'VV', 'VV', 'SP'], ['PN', 'VV', 'NN', 'AD', 'AD', 'VV', 'SP'], ['VV', 'NT', 'NN', 'NN', 'VV'], ['JJ', 'DEG', 'LC', 'VV', 'NN', 'AD', 'VV', 'AS', 'VV'], ['NN', 'DEG', 'NN', 'AD', 'SB', 'VV', 'VV', 'CD', 'M'], ['PN', 'AD', 'VV', 'VV', 'DEC', 'AD', 'AD', 'VV', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'LC', 'VV', 'AS', 'NN'], ['NT', 'VV', 'AS', 'VV', 'NN', 'VV', 'NN', 'ETC', 'NN', 'VV', 'NR', 'NN', 'DEG', 'NN'], ['NN', 'VV', 'AS', 'VA', 'CC', 'VA', 'DEC', 'NN'], ['NR', 'VV', 'VV', 'VV', 'NN', 'VV', 'VV'], ['P', 'VV', 'NN', 'NN', 'LC', 'AD', 'VV', 'AS', 'NN'], ['AD', 'NT', 'NR', 'AD', 'VV', 'NN', 'CC', 'VV', 'AS', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['NN', 'AD', 'P', 'DT', 'NN', 'VV', 'AD', 'VV', 'VV', 'NN', 'VV'], ['NN', 'JJ', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'AD', 'VV', 'NN', 'NN', 'NN', 'VE', 'CD', 'NR', 'VV', 'DEC', 'NN'], ['VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'AD', 'PN', 'AD', 'AD', 'VV'], ['AD', 'VV', 'VV', 'DER', 'VV', 'PN', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['NN', 'NN', 'VV', 'LC', 'NN', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN'], ['P', 'NT', 'NT', 'LC', 'AD', 'VE', 'NN', 'VV', 'VV', 'VV', 'VA', 'NN'], ['P', 'NR', 'VA', 'DEC', 'NN', 'NN', 'VV', 'NN', 'VV', 'PN', 'VV', 'NN'], ['NN', 'LC', 'AD', 'AD', 'VV', 'NN', 'NN', 'ETC', 'NN'], ['NT', 'VC', 'NR', 'NN', 'NT', 'VC', 'CD', 'NN'], ['NN', 'DEG', 'NN', 'VC', 'P', 'NN', 'NN', 'NN', 'VV'], ['NN', 'VV', 'LC', 'NR', 'VV', 'DT', 'NN'], ['NR', 'NN', 'VV', 'DT', 'NN', 'VV', 'VA'], ['VV', 'NR', 'NN', 'NN', 'DEG', 'NN', 'AD', 'SB', 'VV'], ['CD', 'NN', 'SB', 'VV', 'AS', 'AD', 'CD', 'NN', 'VV', 'VV'], ['VE', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['VV', 'PN', 'VV', 'NN', 'PN', 'NN', 'VV', 'AS'], ['AD', 'PN', 'AD', 'VV', 'NN', 'CC', 'NN', 'VV', 'NN'], ['AD', 'PN', 'VV', 'VV', 'NN', 'NN', 'JJ', 'DEG', 'NN'], ['JJ', 'NR', 'NN', 'AD', 'P', 'NN', 'VV', 'NN', 'SP'], ['AD', 'VA', 'DEC', 'VC', 'NN', 'DEG', 'NN', 'NN', 'VA'], ['PN', 'NT', 'VV', 'VV', 'NN', 'P', 'NN', 'VV', 'AD', 'VA'], ['AD', 'DT', 'NN', 'AD', 'AD', 'AD', 'AD', 'AD', 'VV'], ['CS', 'P', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'DEC', 'NN', 'SP'], ['NT', 'NN', 'NR', 'DEG', 'NN', 'VE', 'AS', 'NN', 'VV'], ['VV', 'VV', 'P', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['NT', 'LC', 'PN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['NR', 'VV', 'NN', 'VV', 'AD', 'VC', 'NR'], ['NN', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'VV', 'NN', 'CC', 'NN', 'CC', 'NN', 'VV', 'AS'], ['AD', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['VV', 'SP', 'VV', 'SP', 'NT', 'VV', 'AD', 'VV', 'PN', 'VV'], ['PN', 'AD', 'VV', 'AS', 'VV', 'PN', 'AD', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VA', 'AD', 'VA', 'CD', 'NN', 'VV', 'PN'], ['VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'NT', 'NN', 'NN'], ['AD', 'VV', 'NN', 'VV', 'DEC', 'NN', 'VV'], ['VE', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN', 'NR', 'P', 'NN', 'LC', 'DEG', 'NN', 'CC', 'NN', 'NN'], ['NN', 'VV', 'NR', 'NN', 'NR', 'NN', 'VV', 'CD', 'M', 'CD', 'CD', 'M'], ['NR', 'VV', 'NN', 'VV', 'NR', 'VV', 'PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'AS'], ['VV', 'NN', 'NN', 'NN', 'NR'], ['OD', 'NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'PN', 'VV', 'NN', 'DEG', 'NN', 'NN', 'VV'], ['NR', 'NN', 'VV', 'NN', 'JJ', 'NN', 'P', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VA'], ['AD', 'VV', 'SB', 'VV', 'VV', 'NN', 'NN', 'NN', 'VV', 'DEC', 'NN', 'AD', 'AD', 'SB', 'VV'], ['AD', 'VV', 'NN', 'VV', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'CC', 'NN'], ['VV', 'DEC', 'PN', 'VV', 'CD'], ['CS', 'AD', 'P', 'DT', 'M', 'VV', 'NN', 'DEG', 'NN', 'DT', 'AD', 'NN', 'AD', 'NN', 'VV', 'AS'], ['PN', 'DEG', 'NN', 'NN', 'AD', 'VA'], ['VV', 'NN', 'NN', 'IJ', 'PN', 'AD', 'VV', 'NR'], ['NN', 'VV', 'AS', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN'], ['AD', 'NR', 'VV', 'AS', 'M', 'NN', 'DEG', 'NN', 'NN', 'VV', 'NR', 'AD', 'CD', 'NN'], ['P', 'P', 'VV', 'VV', 'DEC', 'NN', 'PN', 'AD', 'VE', 'NN', 'DEV', 'VV', 'VV', 'AD', 'NN', 'DEG', 'NN'], ['NN', 'NT', 'NN', 'NN', 'NN', 'SP'], ['NT', 'AD', 'NN', 'AD', 'VV', 'SB', 'VV', 'AS', 'SP'], ['AD', 'VV', 'AS', 'M', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'VV', 'DEC', 'NN', 'PN', 'SP'], ['AD', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN'], ['AD', 'VC', 'VV', 'AS', 'VV', 'VV', 'VA', 'SP'], ['PN', 'VV', 'NN', 'DEG', 'NN', 'PN', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NN', 'LC', 'DEG', 'CD', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV'], ['P', 'NR', 'AD', 'VV', 'VV', 'DEC', 'VC', 'JJ', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['PN', 'AD', 'AD', 'NT', 'NN', 'VV', 'AD', 'VV', 'NN', 'NN', 'MSP', 'AD', 'VV', 'NN'], ['PN', 'VV', 'NN', 'NN', 'AD', 'BA', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV'], ['AD', 'DT', 'JJ', 'JJ', 'NR', 'VV', 'PN', 'NN', 'AD', 'VA'], ['PN', 'AD', 'VC', 'VV', 'AS', 'PN', 'VV', 'DEC', 'NN', 'NN', 'DEG', 'NN'], ['P', 'NR', 'AD', 'VV', 'NN', 'VV', 'PN', 'AD', 'AD', 'VV', 'SP'], ['P', 'JJ', 'NN', 'NN', 'VC', 'VA', 'DEC', 'AD', 'AD', 'NN', 'NN', 'AD', 'VC', 'VA', 'SP'], ['NN', 'VV', 'AS', 'CD', 'NN', 'PN', 'AD', 'AD', 'VV', 'NN', 'VV'], ['NT', 'PN', 'AD', 'AD', 'VV', 'NN', 'VV'], ['VV', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['AD', 'VV', 'NN', 'AD', 'VV', 'VV'], ['PN', 'AD', 'AD', 'VV', 'NT', 'PN', 'AD', 'VV', 'AD', 'VA', 'DEC', 'NN', 'SP'], ['AD', 'NR', 'JJ', 'NN', 'NN', 'NR', 'NN', 'NN', 'AD', 'VV', 'AS'], ['NN', 'LC', 'NR', 'CC', 'NR', 'NN', 'VV', 'P', 'NN', 'VV', 'JJ', 'NN', 'NN'], ['CD', 'NN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV'], ['VV', 'VV', 'NN', 'VV', 'NN', 'AD', 'P', 'NR', 'NN', 'NN', 'VV'], ['PN', 'VC', 'VV', 'DEC', 'NN', 'NT', 'VV', 'LC', 'JJ', 'VV', 'VV', 'DEC', 'AD', 'VA'], ['CS', 'NN', 'AD', 'VA', 'CC', 'VA', 'PN', 'AD', 'VV', 'AD', 'VV', 'AD', 'CD', 'NN'], ['AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'DEG', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['NN', 'VV', 'VV', 'NN', 'AD', 'JJ', 'NR', 'DEG', 'NN', 'AD', 'VV', 'NR', 'DEC'], ['NR', 'VV', 'DEC', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'AD', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN', 'DEG', 'NR', 'NR', 'NR', 'VV', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'AD', 'AD', 'VA', 'DEC', 'NR', 'NN', 'AD', 'AD', 'VA'], ['NT', 'NN', 'NR', 'DEG', 'JJ', 'NN', 'NN', 'AD', 'VV', 'JJ', 'NN', 'VV', 'NN'], ['P', 'AD', 'VV', 'NN', 'PN', 'AD', 'P', 'NN', 'NN', 'DEG', 'NN', 'VV', 'AS', 'VA'], ['NN', 'JJ', 'VV', 'NR', 'VV', 'AD', 'PN', 'DEG', 'AD', 'NN', 'VV', 'AD', 'VV', 'VV', 'VV', 'SP'], ['AD', 'NR', 'NN', 'NN', 'VV', 'VV', 'VC', 'PN', 'NN', 'NN', 'JJ', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NR', 'SB', 'VV', 'JJ', 'NN', 'NN', 'NN', 'VV', 'P', 'NN', 'VV', 'AS', 'NN', 'NN'], ['PN', 'DT', 'NN', 'NN', 'AD', 'AD', 'VV', 'PN', 'AD', 'VV', 'SP'], ['PN', 'NN', 'AD', 'AD', 'VA', 'AD', 'AD', 'AD', 'VV', 'VV', 'PN', 'VV', 'CD', 'VV', 'NN', 'NN', 'DEC', 'NN'], ['PN', 'VV', 'VA', 'AS', 'NR', 'PN', 'VE', 'CD', 'NN', 'VV', 'VV', 'VV'], ['PN', 'VV', 'JJ', 'NN', 'VV', 'AS', 'VV', 'NN', 'P', 'NN', 'VV', 'NN'], ['VV', 'VV', 'VV', 'NN', 'SP', 'AD', 'VV', 'PN', 'VV', 'AS', 'NN', 'VV'], ['NT', 'NT', 'NR', 'AD', 'VV', 'NN', 'VV', 'NN', 'CC', 'NN', 'AD', 'VV'], ['P', 'NT', 'DEG', 'OD', 'NN', 'NN', 'LC', 'NN', 'AD', 'VV', 'NN', 'VV'], ['VA', 'DEC', 'NN', 'NN', 'NR', 'VC', 'NT', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'DT', 'SB', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'SP'], ['NN', 'LC', 'NN', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'VV', 'DEC', 'NN', 'VV'], ['AD', 'PN', 'JJ', 'JJ', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN', 'SP', 'AD', 'VA', 'SP'], ['NN', 'LC', 'NN', 'VV', 'P', 'NN', 'NN', 'LC', 'AD', 'NN', 'NN', 'AD', 'JJ', 'DEG'], ['AD', 'NR', 'AD', 'VV', 'NN', 'JJ', 'DEG', 'NN', 'NN', 'NN', 'NN', 'NN'], ['PN', 'AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'VC', 'VV', 'AS'], ['P', 'NN', 'VV', 'NN', 'NN', 'VV', 'NR', 'NN', 'NR', 'VV', 'NN'], ['P', 'NT', 'VV', 'PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN'], ['DT', 'NN', 'CS', 'NR', 'NN', 'VV', 'VV', 'NN', 'NR', 'AD', 'VA', 'VV'], ['P', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'CC', 'NN', 'ETC', 'AD', 'AD', 'AD', 'VV'], ['DT', 'NN', 'AD', 'P', 'NN', 'VV', 'VV', 'NN', 'VV', 'PN', 'P', 'PN', 'VV', 'NN', 'PN', 'AD', 'VV', 'PN', 'VV', 'NN', 'VV', 'SP'], ['AD', 'VV', 'NN', 'VV', 'VA', 'AD', 'DEV', 'DT', 'NN', 'CD', 'NN', 'VV', 'SP'], ['NN', 'NN', 'VA', 'DER', 'AD', 'VA', 'VA', 'DER', 'VV'], ['VV', 'VV', 'LC', 'AD', 'VV', 'AS', 'AD', 'VV', 'NT', 'PN', 'AD', 'AD', 'VV', 'CD', 'NN', 'VV', 'NN', 'SP'], ['VV', 'VV', 'DT', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'AD', 'VA', 'DEC', 'NN', 'CC', 'NN', 'VV', 'VV', 'NR', 'NN', 'NN'], ['NN', 'VV', 'PN', 'NT', 'NN', 'VV', 'P', 'PN', 'VV', 'VV', 'PN', 'NT', 'PN', 'VV', 'NN', 'SP'], ['NR', 'VV', 'VV', 'P', 'NN', 'NN', 'LC', 'NN'], ['VV', 'AS', 'NN', 'PN', 'NN', 'VC', 'NN'], ['CD', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'AD', 'AD', 'AD', 'VV'], ['AD', 'AD', 'NN', 'AD', 'VV', 'VV'], ['NN', 'AD', 'VV', 'VV', 'VV', 'AD', 'VA', 'DEC', 'NR', 'NR', 'AD', 'VV', 'VV'], ['P', 'NN', 'DT', 'NN', 'LC', 'NN', 'VV', 'DEC', 'NN', 'AD', 'VA'], ['PN', 'VC', 'VV', 'NN', 'P', 'PN', 'VV', 'VV'], ['VA', 'DEC', 'NN', 'AD', 'P', 'NN'], ['NR', 'VV', 'AD', 'VV', 'AD', 'JJ', 'DEG'], ['AD', 'NR', 'NR', 'NN', 'ETC', 'JJ', 'NN', 'AD', 'VV', 'VV', 'AS', 'CD'], ['NN', 'P', 'NT', 'NT', 'VV', 'LC', 'NT', 'AD', 'VC', 'OD', 'CD', 'NN'], ['IJ', 'PN', 'AD', 'VC', 'M', 'JJ', 'DEG', 'NN'], ['P', 'PN', 'AD', 'AD', 'VV', 'NR', 'PN', 'VV', 'PN'], ['VV', 'DEC', 'NN', 'VC', 'NR', 'DT', 'VV', 'DEC'], ['CC', 'NN', 'AD', 'VV', 'VV'], ['PN', 'AD', 'VC', 'CD', 'JJ', 'DEG', 'NN', 'SP'], ['PN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'PN'], ['AD', 'JJ', 'NN', 'SB', 'NN', 'VV', 'NN', 'VV', 'P', 'NN', 'VV'], ['P', 'PN', 'VV', 'VV', 'AS', 'PN', 'VV', 'VV', 'DEC', 'NN', 'PN', 'AD', 'VV', 'AS'], ['NR', 'NN', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NR', 'NN', 'P', 'NN', 'NN', 'CD', 'NN', 'NN'], ['NR', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'VV', 'PN', 'AD', 'VV', 'PN'], ['NN', 'VC', 'NN', 'NN', 'CC', 'NN'], ['NR', 'DEG', 'NN', 'VV', 'PN', 'DEG', 'NN', 'P', 'NN', 'VV', 'P', 'NR'], ['NN', 'VV', 'VA', 'NN', 'P', 'NN', 'VV', 'NN', 'VV', 'VV'], ['CC', 'VV', 'AS', 'CC', 'VA', 'AS', 'VC', 'VV', 'NN', 'VE', 'AD', 'AD', 'VA', 'SP'], ['NR', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'VE', 'NN', 'NR', 'NN', 'P', 'NN', 'NR', 'DEG', 'NN', 'SP'], ['NN', 'NN', 'VV', 'VV', 'CD', 'NN'], ['VV', 'NN', 'VC', 'P', 'PN', 'VV', 'CS', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN'], ['NR', 'NR', 'NR', 'NN'], ['JJ', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV'], ['PN', 'NN', 'NN', 'NN', 'VA'], ['AD', 'AD', 'VV', 'VC', 'AD', 'VV'], ['NN', 'LC', 'AD', 'AD', 'VA', 'VV'], ['AD', 'VC', 'VV', 'DT', 'NN', 'AD', 'VA', 'NR', 'AD', 'AD', 'VV'], ['PN', 'VV', 'PN', 'VV', 'NN', 'VV', 'VA'], ['JJ', 'VV', 'NN', 'NN'], ['AD', 'VA', 'NN', 'DT', 'NN', 'NR'], ['AD', 'AD', 'VE', 'DT', 'NN', 'SP'], ['NN', 'VC', 'JJ', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['NR', 'DEG', 'NN', 'AD', 'VV', 'AS', 'NR', 'NN', 'DEG', 'NN', 'CC', 'VA'], ['NN', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VA', 'VC', 'VV', 'VV'], ['PN', 'AD', 'VV', 'NN'], ['NN', 'NN', 'LC', 'AD', 'AD', 'P', 'NN', 'VV', 'VC', 'PN', 'NN', 'VV'], ['VV', 'PN', 'DEG', 'NN', 'VC', 'VA', 'SP'], ['PN', 'VV', 'NN', 'VC', 'VV', 'SP', 'SP'], ['NN', 'DEG', 'VV', 'AS', 'CD', 'NN'], ['NN', 'LC', 'NN', 'AD', 'SB', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN'], ['AD', 'DEV', 'AD', 'PN', 'VV', 'NN'], ['NR', 'AD', 'NR', 'VV', 'AS', 'JJ', 'NN', 'VV', 'VV'], ['NN', 'AD', 'VV', 'DEC', 'NN', 'NN', 'DEG', 'NN', 'NR', 'NN'], ['DT', 'NN', 'AD', 'VC', 'DT', 'NN', 'PN', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV', 'SP'], ['NN', 'NN', 'P', 'NN', 'JJ', 'NN', 'AD', 'P', 'JJ', 'NN', 'NN', 'VV'], ['NN', 'NN', 'VC', 'NR', 'NN', 'NR'], ['VV', 'VV', 'VV', 'NN', 'VV'], ['DT', 'NN', 'JJ', 'NN', 'NN'], ['PN', 'VC', 'NR', 'NN', 'DEG', 'NN'], ['VV', 'VC', 'NN', 'NN', 'DEC', 'VV'], ['OD', 'NN', 'NN', 'NN', 'VV', 'AS', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['AD', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'VV', 'AS', 'NN'], ['AD', 'AD', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'NR', 'NN', 'NT', 'CD'], ['PN', 'CC', 'AD', 'VV', 'AD', 'AD', 'PN', 'AD', 'VA', 'PN', 'DEG', 'NN', 'SP', 'PN', 'SP'], ['NN', 'NR', 'NR', 'P', 'NN', 'LC', 'VV', 'VV', 'PN', 'NN'], ['VE', 'NN', 'DEC', 'NN', 'AD', 'VV', 'NN', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VA', 'JJ', 'DEG', 'NN', 'AD', 'NR', 'NN'], ['NR', 'AD', 'VV', 'AS', 'CD', 'NN', 'BA', 'NN', 'VV', 'AS', 'NN'], ['NR', 'CC', 'NR', 'DT', 'VV', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN'], ['VV', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN'], ['NN', 'P', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN', 'NN'], ['AD', 'P', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VC', 'NN', 'VV', 'M', 'NN', 'SP'], ['JJ', 'SB', 'VV', 'VV', 'DEC', 'NR', 'AD', 'VC', 'VV', 'AS', 'AD', 'VV'], ['LB', 'VV', 'VV', 'VV', 'DEC', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN'], ['NN', 'VV', 'VV', 'CD', 'OD', 'NN', 'NN', 'VC', 'NR', 'NN'], ['VE', 'NN', 'VV', 'NR', 'AD', 'NR', 'DEG', 'NN', 'NN'], ['NR', 'VV', 'NR', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'DEV', 'NR', 'DEG', 'NN', 'VV', 'AS', 'AD', 'VV', 'VV'], ['NR', 'VV', 'NR', 'NN', 'VV', 'NN', 'LC', 'AD', 'P', 'JJ', 'NN', 'VV', 'AS', 'NN'], ['NN', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'VV'], ['P', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'NN'], ['PN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NR', 'VV', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NN', 'VV', 'P', 'NN', 'LC', 'AD', 'VV'], ['OD', 'AD', 'VV', 'VA'], ['AD', 'VE', 'NN', 'SP', 'AD', 'VV', 'SP'], ['VV', 'SP', 'VV', 'PN', 'VV', 'VV'], ['DT', 'NN', 'VV', 'NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'PN', 'VV', 'VV', 'PN', 'SP', 'SP'], ['JJ', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'AD', 'AD', 'AD', 'VA'], ['AD', 'AD', 'VE', 'CD', 'DEG', 'NN'], ['NR', 'VV', 'AS', 'DT', 'CD', 'NN'], ['AD', 'AD', 'VA', 'AD', 'VA'], ['NT', 'NR', 'P', 'VV', 'LC', 'NN', 'VV', 'AD', 'VV', 'VV'], ['PN', 'AD', 'PN', 'VV', 'AD', 'BA', 'NN', 'AD', 'VV', 'AS', 'VV'], ['AD', 'VA', 'DEC', 'NN', 'AD', 'PN', 'VV', 'PN', 'VV', 'DEC', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'NN', 'NN', 'VV', 'VV', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'VV'], ['NT', 'AD', 'VV', 'NT', 'AD', 'AD', 'SP'], ['DT', 'NN', 'DEG', 'NN', 'NN', 'CC', 'NN', 'DEG', 'NN', 'VV', 'CD', 'NN', 'NN', 'VV'], ['NR', 'NT', 'AD', 'VV', 'VV', 'VV', 'NN', 'NN', 'NN', 'DEG', 'NN', 'VV', 'NN', 'VV', 'VV'], ['VV', 'LC', 'VV', 'AD', 'VE', 'NR', 'CC', 'VA', 'DEC', 'NN', 'NN'], ['AD', 'VV', 'NN', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'VC', 'AD', 'VA', 'DEV', 'VV', 'NN'], ['PN', 'AD', 'VV', 'NN', 'NT', 'P', 'PN', 'CD', 'NN', 'VV'], ['P', 'VV', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN', 'VV'], ['NT', 'NN', 'P', 'NN', 'DEG', 'NN', 'VC', 'VA', 'AD', 'CD', 'NT'], ['NT', 'NR', 'NN', 'VA', 'NR', 'NN', 'PN', 'VV', 'VV', 'PN', 'NT', 'VV', 'PN'], ['P', 'VV', 'DEC', 'NN', 'CD', 'NN', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['NR', 'NT', 'NN', 'VE', 'JJ', 'DEG', 'NN', 'NN', 'CC', 'NN'], ['NN', 'NN', 'DEG', 'NN', 'VV', 'DER', 'AD', 'AD', 'NN', 'AD', 'VV'], ['NN', 'VV', 'AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['NN', 'AD', 'VE', 'NN', 'DEC', 'NN', 'NN', 'VV', 'SP', 'PN', 'VV', 'VV', 'PN'], ['NT', 'NN', 'VC', 'NN', 'CD', 'CD', 'CD', 'CD', 'NN'], ['NN', 'VV', 'PN', 'VV', 'CD', 'PN', 'VC', 'NN', 'CC', 'PN', 'VC', 'NN'], ['NR', 'VE', 'CD', 'NT', 'DEG', 'NN', 'PN', 'VE', 'CD', 'M', 'DEC', 'NN'], ['NT', 'VV', 'PN', 'VV', 'CD', 'AS', 'PN', 'AD', 'VV', 'PN', 'VV', 'VV', 'SP'], ['PN', 'AD', 'VV', 'NN', 'DEG', 'NN', 'AD', 'NN', 'VV', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN'], ['VV', 'NN', 'NN', 'AD', 'VV', 'P', 'JJ', 'NN', 'LC', 'LC', 'VV', 'AD', 'VV', 'AD', 'NN'], ['NR', 'NT', 'NT', 'PN', 'VV', 'DER', 'VV', 'PN', 'AD', 'VC', 'NN', 'VV'], ['CS', 'VE', 'NN', 'DEC', 'NN', 'AD', 'NN', 'VV', 'AD', 'VV', 'PN', 'NN', 'VV', 'SP'], ['AD', 'AD', 'P', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN'], ['VV', 'NN', 'P', 'M', 'JJ', 'NN', 'VE', 'NN', 'NN', 'NN', 'CC', 'NN'], ['SB', 'VV', 'AS', 'AD', 'VA', 'SP'], ['NN', 'P', 'NN', 'VV'], ['PN', 'VV', 'VV', 'PN'], ['PN', 'VV', 'NN', 'NN'], ['PN', 'AD', 'AD', 'VA', 'SP'], ['PN', 'DEG', 'OD', 'NN', 'VC', 'PN'], ['PN', 'VV', 'VV', 'PN', 'NN'], ['VV', 'NN', 'NN', 'NN'], ['VC', 'NR', 'NN', 'NN'], ['PN', 'AD', 'VV', 'VV', 'NN'], ['NR', 'NR', 'NN', 'NN', 'PN', 'AD', 'VV', 'SP'], ['OD', 'CD', 'CD', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN'], ['AD', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'VV', 'SP'], ['PN', 'NT', 'AD', 'VV', 'AS', 'NN', 'AD', 'VC', 'NN', 'AD', 'VA', 'DEV', 'VV', 'AS'], ['PN', 'NN', 'AD', 'VE', 'CD', 'NN', 'DEG', 'NN', 'SP'], ['PN', 'AD', 'CC', 'VV', 'NN', 'VV', 'DEC', 'NN'], ['VA', 'DEV', 'VV', 'AS', 'AD', 'VV', 'VV'], ['NR', 'NN', 'P', 'P', 'NN', 'VV', 'NN'], ['AD', 'AD', 'NN', 'AD', 'VV', 'AD', 'AD', 'BA', 'AD', 'VV', 'NN', 'VV', 'VV', 'AS', 'NN'], ['DT', 'M', 'CS', 'CC', 'PN', 'P', 'NN', 'SP', 'NR', 'VA', 'AD', 'LB', 'VA', 'AD', 'VV', 'SP'], ['AD', 'P', 'NR', 'NN', 'VV', 'DEC', 'NR', 'NN', 'NN', 'NN', 'VV', 'VV', 'AD', 'VV'], ['AD', 'VV', 'VV', 'NN', 'VA', 'DEC', 'NN', 'AD', 'VC', 'NN', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['P', 'VV', 'AD', 'VV', 'VV', 'PN', 'VV', 'DEC', 'NR', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'VV', 'AS', 'CD', 'M', 'NN', 'DEG', 'NN', 'VV', 'AD', 'VV', 'NN', 'AD', 'AD', 'AD', 'VV'], ['NT', 'NR', 'VV', 'P', 'NN', 'LC', 'DEC', 'NN', 'AD', 'VV', 'NN', 'VV', 'VA', 'CC', 'VA'], ['NN', 'VC', 'JJ', 'NN', 'AD', 'VV', 'JJ', 'NN', 'VA', 'DEC', 'NN', 'VV', 'VV'], ['AD', 'P', 'NN', 'NN', 'AD', 'VA', 'DEC', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN'], ['CD', 'NN', 'AD', 'VV', 'AS', 'NR', 'VV', 'AS', 'NR', 'VV', 'NR', 'LC', 'DEG', 'NN'], ['DT', 'NN', 'P', 'NN', 'LC', 'DEG', 'NN', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['VV', 'VV', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN'], ['AD', 'BA', 'NN', 'VV', 'VV', 'DEC', 'NN', 'P', 'NN', 'NN', 'VV', 'VV', 'NN'], ['VA', 'DEC', 'NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'CD', 'NN', 'AD', 'AD', 'VV'], ['VV', 'DEC', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'AD', 'VV', 'NN', 'DEC', 'VA'], ['CS', 'AD', 'VV', 'AS', 'AD', 'VV', 'VV', 'AS', 'AD', 'VV', 'DT', 'NN', 'VV', 'AD', 'VV', 'NN', 'SP'], ['NN', 'AD', 'VV', 'NR', 'NN', 'AD', 'AD', 'VV', 'SP', 'VV', 'PN', 'VC', 'JJ', 'DEG', 'AD'], ['NN', 'LC', 'DT', 'NN', 'AD', 'LB', 'PN', 'NN', 'CD', 'VV', 'VV', 'VV', 'AD'], ['NN', 'DEG', 'NN', 'AD', 'P', 'NN', 'VV', 'VV', 'CC', 'P', 'JJ', 'JJ', 'NN', 'VV'], ['AD', 'P', 'NN', 'VV', 'NN', 'AD', 'VV', 'JJ', 'CS', 'VV', 'AS', 'AD', 'AD', 'VV', 'AD', 'VV'], ['AD', 'VV', 'DEC', 'JJ', 'NN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VA', 'AD', 'VA'], ['PN', 'VC', 'NN', 'VV', 'NN', 'VV', 'NN', 'LC', 'AD', 'P', 'NR', 'NN', 'LC', 'NN', 'VV'], ['JJ', 'VV', 'PN', 'VV', 'DT', 'JJ', 'DEG', 'NN'], ['NN', 'NN', 'P', 'NN', 'AD', 'AD', 'VA'], ['NT', 'AD', 'VV', 'AD', 'AD', 'NN', 'DEG', 'DT', 'NN', 'LC', 'VV', 'NN'], ['VV', 'LC', 'NN', 'P', 'PN', 'VV', 'VC', 'NN', 'DEG', 'PN', 'VV', 'VV', 'AD', 'AD', 'VA'], ['JJ', 'NN', 'VV', 'AS', 'NN', 'VV', 'VV', 'VV', 'SP'], ['NR', 'NR', 'NN', 'AD', 'VV', 'NT', 'AD', 'VV', 'NN'], ['P', 'NN', 'NN', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'NN', 'CC', 'NN'], ['VA', 'DEC', 'NN', 'VV', 'NN', 'AD', 'VV', 'PN', 'CD', 'NN', 'LC', 'AD', 'VV', 'NN'], ['AD', 'VV', 'NN', 'LC', 'NN', 'VV', 'PN', 'VC', 'NR', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VV', 'AS', 'NN', 'NN', 'VV', 'AS', 'NR', 'DEG', 'NN', 'VV', 'VV', 'SP'], ['AD', 'VV', 'AS', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'AD', 'AD', 'VV', 'DT', 'NN'], ['AD', 'P', 'NT', 'AD', 'VV', 'AD', 'NN', 'AD', 'VV', 'VV', 'VV'], ['NN', 'NN', 'VV', 'AS', 'NN', 'VA', 'DEC', 'NN'], ['NN', 'VV', 'AD', 'VV', 'PN', 'VV', 'PN', 'VC', 'PN', 'VV', 'AS', 'DEC', 'NN', 'DEG', 'NN'], ['VV', 'VV', 'NN', 'LC', 'PN', 'VV', 'AD', 'VV', 'VV', 'AS', 'VE', 'DT', 'NN'], ['AD', 'AD', 'VV', 'AD', 'VV', 'AS', 'AD', 'PN', 'DEG', 'NN', 'AD', 'VV', 'JJ', 'NN'], ['VV', 'DT', 'CD', 'NN', 'NN', 'DEC', 'NN', 'SP'], ['NR', 'PN', 'P', 'NR', 'AD', 'VC', 'AD', 'NN'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['NR', 'BA', 'NN', 'VV', 'P', 'VV', 'NN', 'VA', 'DEC', 'NN'], ['NR', 'NR', 'JJ', 'NN', 'NR', 'DEG', 'NN'], ['NN', 'VV', 'NN', 'NN', 'AD', 'AD', 'VA', 'DEC', 'NN'], ['NR', 'P', 'NR', 'NN', 'DEG', 'NN', 'VV', 'NN'], ['NN', 'VV', 'VV', 'VV', 'NN', 'VA', 'NN', 'VA'], ['DT', 'NN', 'AD', 'NR', 'DEG', 'NN', 'AD', 'VV', 'PN', 'VV', 'AS'], ['VV', 'VV', 'VV', 'NR', 'NN', 'SP', 'VV', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VV', 'PN', 'SP'], ['NN', 'P', 'NR', 'VV', 'LC', 'VV', 'NN', 'AD', 'VV'], ['NT', 'NR', 'DEG', 'NR', 'AD', 'VV', 'AS', 'VV', 'NN'], ['PN', 'AD', 'VV', 'AS', 'NN', 'LB', 'NN', 'VV', 'AS', 'NR'], ['NT', 'NN', 'LC', 'CD', 'NN', 'AD', 'VV', 'P', 'NN', 'NN'], ['AD', 'LC', 'PN', 'AD', 'VV', 'AS', 'AD', 'VA', 'DEC', 'NN'], ['AD', 'NR', 'VV', 'AS', 'CD', 'NN', 'VV', 'VV', 'VA'], ['NR', 'VV', 'CD', 'DEG', 'NN', 'AD', 'VV'], ['NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV', 'P', 'NN'], ['NN', 'CC', 'NR', 'DEG', 'NN', 'NN', 'VV', 'VV'], ['PN', 'AD', 'VV', 'VV', 'CD', 'VV', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'LC', 'AD', 'P', 'NR', 'VV', 'NR', 'VV', 'NN'], ['NN', 'AD', 'VV', 'NN', 'VV', 'PN', 'VA'], ['NN', 'AD', 'AD', 'AD', 'VV', 'NN', 'VV', 'VV'], ['NN', 'P', 'VV', 'NN', 'DEG', 'NN', 'CC', 'NN', 'VV', 'AD'], ['AD', 'AD', 'LC', 'OD', 'M', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN'], ['AD', 'NR', 'AD', 'AD', 'VV', 'NN', 'AD', 'BA', 'NN', 'VV', 'AD', 'VV', 'DEV'], ['NR', 'VV', 'NN', 'LB', 'NN', 'DT', 'VV', 'NN', 'NN', 'VV', 'VV'], ['NR', 'VV', 'NN', 'DEG', 'CD', 'CD', 'CD', 'JJ', 'NN', 'JJ', 'NN'], ['AD', 'NN', 'AD', 'NN', 'NN', 'PN', 'NT', 'AD', 'VV', 'AD', 'P', 'NN', 'SP'], ['P', 'NN', 'VV', 'SB', 'VV', 'VC', 'NN', 'CC', 'P', 'LC', 'NN', 'LC', 'AD', 'VV', 'NN'], ['AD', 'PN', 'P', 'AD', 'AD', 'AD', 'JJ', 'P', 'NN', 'NN', 'VA'], ['NT', 'AD', 'VC', 'NN', 'NN', 'NN', 'DEG', 'NR', 'NR', 'VV', 'VV', 'NN', 'NN', 'P', 'NN', 'NN', 'NN', 'VV'], ['NN', 'NN', 'LC', 'NN', 'NN', 'VA'], ['PN', 'PN', 'AD', 'VV', 'VV', 'NN', 'DT', 'VV', 'NN', 'NN', 'AD', 'VE', 'JJ', 'NN'], ['NR', 'AD', 'P', 'NN', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'DT', 'NN', 'SP', 'P', 'PN', 'VV', 'AS', 'SP'], ['AD', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV', 'JJ', 'NN', 'VV'], ['NN', 'NN', 'AD', 'VV', 'AS', 'DT', 'CD', 'NN'], ['NN', 'NT', 'AD', 'VV', 'VV', 'NN', 'VV', 'NN'], ['PN', 'VV', 'AS', 'NN', 'NN', 'VV', 'DEC', 'JJ', 'NN'], ['AD', 'AD', 'VV', 'AS', 'VV', 'VA', 'DEC', 'NN'], ['NR', 'VE', 'NN', 'DEC', 'NN', 'AD', 'VC', 'NN', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VE', 'NN', 'P', 'NN', 'LC', 'VV'], ['JJ', 'PN', 'VV', 'NR', 'NR', 'AD', 'AD', 'CD'], ['NT', 'LC', 'DEG', 'NN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'VV', 'NN', 'VV', 'DEC', 'VC', 'NN', 'VV'], ['VV', 'VV', 'AD', 'VV', 'DEC', 'NN', 'VA'], ['VV', 'DT', 'NN', 'CD', 'NN', 'VV', 'NN'], ['AD', 'JJ', 'NN', 'AD', 'VV', 'NN', 'VV'], ['PN', 'AD', 'VV', 'PN', 'NN', 'AD', 'VV', 'VV', 'VV', 'PN', 'AD', 'VA', 'SP', 'VV', 'NR', 'VE', 'DT', 'VV', 'SP'], ['CD', 'NN', 'NN', 'NR', 'NN', 'VV', 'AS', 'NR', 'NN', 'DEG', 'JJ', 'NN'], ['NN', 'AD', 'VV', 'NN', 'NN', 'DEG', 'NN', 'NN', 'NN'], ['NN', 'AD', 'VV', 'AD', 'VV', 'AD', 'VV', 'P', 'NN', 'LC', 'VV', 'DEC', 'VC', 'DT', 'NN'], ['PN', 'AD', 'AD', 'VV', 'AD', 'VV', 'VA', 'AS', 'NN', 'LC', 'CD', 'NN'], ['VE', 'NN', 'VV', 'AD', 'VA', 'DEC', 'NN', 'AD', 'VA', 'AD', 'VA'], ['PN', 'PN', 'AD', 'AD', 'PN', 'AD', 'AD', 'VV'], ['NN', 'NN', 'VV', 'LC', 'AD', 'VV', 'VV'], ['AD', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV', 'VV', 'SP'], ['AD', 'JJ', 'NN', 'NR', 'CC', 'NR', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'SP'], ['NT', 'VV', 'AS', 'PN', 'NN', 'CC', 'PN', 'VV', 'SP'], ['PN', 'NT', 'VV', 'P', 'DT', 'NN', 'AD', 'VV'], ['CD', 'NN', 'VV', 'NN', 'VV', 'VV', 'VV'], ['AD', 'PN', 'PN', 'AD', 'VE', 'NN', 'VV', 'VV', 'VV', 'NN'], ['NR', 'NN', 'NN', 'P', 'VV'], ['NN', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'DEG', 'NN', 'BA', 'NR', 'DT', 'VV', 'VV'], ['VV', 'NN', 'VV', 'AD', 'VV', 'NN', 'NR'], ['AD', 'VV', 'PN', 'VV', 'NN', 'VV', 'AD', 'PN', 'VV', 'PN', 'NN', 'AD', 'VE', 'PN', 'DEG', 'NN', 'SP'], ['NR', 'SB', 'VV', 'DER', 'P', 'M', 'NN', 'NN', 'CC', 'NN', 'VV', 'VV', 'AD'], ['DT', 'NN', 'PN', 'CD', 'AD', 'AD', 'AD', 'VV', 'SP'], ['VC', 'SP', 'NR', 'VV', 'AS', 'NN', 'AD', 'AD', 'VA'], ['NN', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VE', 'NN', 'PN', 'AD', 'VV', 'AD', 'VV', 'AS', 'NN', 'DEG', 'IJ'], ['NN', 'VV', 'NN', 'VV', 'VV', 'DEC', 'NN'], ['PN', 'VV', 'VV', 'NR', 'VA', 'DEC', 'NN'], ['DT', 'DEG', 'VV', 'P', 'NN', 'NN', 'CC', 'NN'], ['AD', 'NN', 'P', 'JJ', 'NN', 'VV', 'AD', 'VV', 'AS', 'NN', 'AD', 'NR'], ['CS', 'AD', 'VV', 'DEC', 'NN', 'AD', 'VV', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'SP'], ['NT', 'AD', 'VE', 'NN', 'VV', 'AD', 'VV', 'BA', 'NN', 'NN', 'NN', 'NN', 'VV'], ['NN', 'LC', 'NR', 'AD', 'P', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'VE', 'NR', 'VV', 'DEC', 'NN', 'NN', 'NN', 'CC', 'NN', 'ETC', 'NN'], ['NN', 'VE', 'NN', 'AD', 'VA', 'AD', 'AD', 'BA', 'NR', 'VV', 'P', 'NR', 'DEG', 'NN'], ['NR', 'VV', 'AS', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'AS', 'NN'], ['VV', 'VV', 'NR', 'NN', 'VC', 'P', 'VV', 'DT', 'NN', 'NN', 'SP'], ['AD', 'VV', 'VV', 'NN', 'DEC', 'AD', 'BA', 'VV', 'NN', 'VV', 'AS'], ['AD', 'P', 'VV', 'DEC', 'NN', 'NN', 'NN', 'NN', 'VV', 'NN', 'NN'], ['DT', 'M', 'NN', 'P', 'PN', 'DEG', 'NN', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV'], ['VV', 'AD', 'VA', 'DEC', 'NN', 'NN', 'AD', 'NN', 'VV', 'VV', 'NR'], ['PN', 'AD', 'VV', 'NN', 'VV', 'AS', 'AD', 'VV', 'VV', 'AS', 'NN', 'SP', 'NN', 'SP'], ['NT', 'NN', 'VV', 'NN', 'AD', 'VV', 'CD', 'NN', 'VV', 'CD', 'M', 'CD'], ['NR', 'VV', 'NN', 'BA', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'VE', 'NN', 'PN', 'AD', 'VV', 'NN', 'P', 'NN', 'PN', 'P', 'NN'], ['P', 'AD', 'VV', 'NN', 'PN', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN'], ['AD', 'VV', 'DEC', 'JJ', 'NN', 'NN', 'VV', 'AD', 'NN', 'AD', 'AD', 'VV', 'AS'], ['NN', 'NN', 'DEG', 'NN', 'LC', 'VV', 'AS', 'CD', 'JJ', 'JJ', 'NN', 'NN', 'NN', 'VC', 'NN'], ['VV', 'DEC', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VV', 'AS', 'AD', 'VV'], ['AD', 'NN', 'VV', 'NN', 'VV', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN', 'ETC', 'NN'], ['PN', 'AD', 'P', 'NN', 'VV', 'AD', 'AD', 'VV', 'PN', 'VV'], ['VE', 'DT', 'NN', 'DEG', 'CS', 'VV', 'PN', 'SP'], ['VA', 'DEC', 'NN', 'NN', 'P', 'NN', 'LC', 'VV', 'NN', 'VV', 'AD', 'VA', 'VA', 'DEV', 'VV', 'P', 'LC'], ['CD', 'CD', 'CD', 'CD', 'CD', 'M', 'NT', 'VC', 'DT', 'NN', 'NN', 'NN', 'NN'], ['NT', 'NN', 'P', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['P', 'NN', 'NN', 'NN', 'NR', 'VC', 'CD', 'AD', 'VA', 'DEC', 'NN'], ['NN', 'VV', 'LC', 'VV', 'NR', 'DEG', 'NN', 'PN', 'VV', 'VV'], ['NN', 'LC', 'VV', 'CD', 'NN', 'NN', 'DEG', 'NN', 'NN', 'AD', 'VV', 'VV'], ['NT', 'AD', 'NN', 'NN', 'AD', 'P', 'NN', 'NN', 'VV', 'CD', 'NN'], ['NN', 'NN', 'AD', 'VV', 'AS', 'NT', 'CC', 'VV', 'AS', 'CD'], ['NT', 'NN', 'DEG', 'NN', 'AD', 'VC', 'NN', 'P', 'NN', 'VV'], ['P', 'VV', 'PN', 'DEG', 'NN', 'CC', 'NN', 'AD', 'VV', 'NN', 'VV', 'VV'], ['NR', 'NR', 'NR', 'ETC', 'JJ', 'NN', 'AD', 'VV', 'NN'], ['NT', 'NN', 'AD', 'VV', 'NN', 'AD', 'VA'], ['NN', 'VV', 'P', 'NN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV'], ['NN', 'NN', 'LC', 'NN', 'VV', 'CD', 'NN', 'AD', 'VV', 'VV'], ['AD', 'SB', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN', 'DEC', 'VV'], ['VV', 'JJ', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'SB', 'VV', 'JJ', 'NN'], ['AD', 'JJ', 'JJ', 'NN', 'DEG', 'NN', 'VC', 'VV', 'DT', 'NN'], ['AD', 'JJ', 'NN', 'VV', 'VV', 'CD', 'CD', 'M', 'NN', 'CD', 'NN'], ['AD', 'VV', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'PN', 'VV', 'VV', 'VV', 'DEC', 'NN'], ['VE', 'PN', 'P', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'AD', 'P', 'NN', 'VA', 'VA', 'SP'], ['NN', 'VV', 'VV', 'NN', 'AD', 'VV', 'VV'], ['NR', 'NN', 'JJ', 'NN', 'NN', 'VV', 'NN', 'CC', 'NN', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'VV', 'NR', 'DEG', 'NN', 'NN'], ['AD', 'NN', 'VA', 'CC', 'NN', 'VA'], ['AD', 'AD', 'VV', 'VV', 'M', 'NN', 'SP'], ['NT', 'VV', 'P', 'NR', 'NR', 'NN', 'P', 'NN', 'CC', 'NN', 'VV', 'AS', 'VV', 'NN', 'DEC', 'JJ', 'NN'], ['PN', 'VV', 'CD', 'NN', 'NN', 'PN', 'AD', 'VC', 'VV', 'VV', 'NR', 'AD', 'VE', 'VV', 'VV', 'DEC', 'NN'], ['AD', 'VV', 'NR', 'LC', 'NN', 'VV', 'DEC', 'VC', 'NN', 'NN', 'AD', 'NN', 'NN'], ['P', 'JJ', 'NN', 'NN', 'LC', 'PN', 'VV', 'PN', 'BA', 'NN', 'VV', 'VV', 'DT', 'PN', 'VV', 'VV'], ['VV', 'LC', 'NR', 'AD', 'VV', 'VV', 'JJ', 'NN', 'DEC', 'VV', 'VV', 'AS', 'NN', 'NN'], ['NN', 'NN', 'P', 'DT', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN'], ['NR', 'NN', 'DEG', 'NN', 'NR', 'AD', 'AD', 'VV', 'VV'], ['AD', 'AD', 'AD', 'VC', 'NN', 'CC', 'JJ', 'NN', 'DEG', 'NN'], ['NN', 'LC', 'VE', 'CD', 'VV', 'NN', 'CC', 'NN', 'DEG', 'NN'], ['AD', 'NT', 'NN', 'LC', 'NN', 'NN', 'NN', 'AD', 'VA'], ['NN', 'NN', 'VV', 'NN', 'ETC', 'DT', 'NN', 'VV', 'VV', 'AD'], ['P', 'CD', 'CD', 'AD', 'JJ', 'NN', 'LC', 'VV', 'AS', 'CD', 'JJ', 'DEG', 'NN', 'AD', 'VV', 'VV', 'AS'], ['P', 'NN', 'DEG', 'VV', 'LC', 'NN', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['NN', 'VV', 'NT', 'NN', 'NN', 'AD', 'AD', 'AD', 'NN', 'VV', 'NN'], ['PN', 'NN', 'VV', 'NN', 'CD', 'M', 'CD', 'CD', 'CD', 'NN'], ['AD', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'VA', 'DEC', 'NN'], ['AD', 'PN', 'AD', 'AD', 'AD', 'VV', 'PN', 'PN', 'AD', 'P', 'PN', 'VV', 'NN', 'VV'], ['PN', 'P', 'NR', 'NR', 'NN', 'NN', 'AD', 'VA', 'VV', 'VV', 'JJ', 'CD', 'DEG', 'NN'], ['AD', 'NT', 'DT', 'NN', 'AD', 'NN', 'DEG', 'NN', 'VV', 'NN', 'JJ', 'JJ', 'NN', 'NN', 'VA', 'SP'], ['AD', 'PN', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'VV', 'DER', 'NN', 'CC', 'NN', 'NN', 'AD', 'VV'], ['DT', 'NN', 'AD', 'BA', 'PN', 'VV', 'P', 'NN', 'LC', 'AD', 'DT', 'DEG', 'NN', 'AD', 'VV'], ['VV', 'MSP', 'VV', 'DEC', 'NN', 'NN', 'VC', 'VV', 'DEC', 'PN', 'VV', 'AS', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'P', 'NN', 'NN', 'PN', 'VV', 'P', 'NN', 'AD', 'VV', 'NN', 'P', 'VV', 'VV', 'NN'], ['NR', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'SB', 'NN', 'VV', 'VV', 'VV', 'NN', 'NN'], ['NN', 'AD', 'DEV', 'VV', 'AD', 'VV', 'AS', 'JJ', 'NN', 'VV', 'NN', 'VV', 'NN', 'LC'], ['NR', 'VV', 'DEC', 'AD', 'VV', 'NN', 'AD', 'VV', 'DEC', 'AD', 'AD', 'VV', 'NN', 'IJ', 'VV', 'AD'], ['AD', 'NT', 'VV', 'AS', 'AD', 'JJ', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN'], ['NT', 'NR', 'P', 'VV', 'AD', 'VV', 'NN', 'NN', 'LB', 'NN', 'VV'], ['PN', 'VV', 'VV', 'PN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'LB', 'AD', 'AD'], ['NN', 'VV', 'LC', 'VE', 'DEC', 'NN', 'VV', 'NN', 'CC', 'VV', 'NN'], ['CC', 'VV', 'VV', 'SP', 'PN', 'NN', 'VV', 'M'], ['AD', 'VV', 'NT', 'AD', 'VC', 'VA', 'DEC', 'AD'], ['AD', 'NR', 'NN', 'CC', 'NN', 'AD', 'VV'], ['NT', 'NT', 'NN', 'VV', 'NN', 'NN', 'CC', 'CD', 'NT', 'CD', 'CD', 'CD', 'CD'], ['NN', 'DEG', 'NN', 'NN', 'AD', 'VC', 'AD', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'NN', 'P', 'NR', 'AD', 'VA', 'DEC', 'NR', 'NN', 'SP'], ['PN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['PN', 'VV', 'VC', 'NN', 'NN', 'VV', 'NT', 'NR', 'VV', 'VV', 'PN', 'NN'], ['CD', 'NN', 'NN', 'NN', 'DEG', 'JJ', 'NN', 'VV', 'NR', 'PN', 'VV'], ['NT', 'NT', 'NR', 'NN', 'VV', 'VV', 'VV', 'VV', 'CD', 'CD', 'CD', 'M'], ['AD', 'P', 'NR', 'VV', 'JJ', 'DEG', 'JJ', 'NN', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV'], ['NT', 'NT', 'NN', 'AD', 'VV', 'AD', 'AD', 'P', 'NN', 'NN', 'VA'], ['AD', 'P', 'VV', 'AD', 'VV', 'NN', 'NN', 'VC', 'NR', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'NN', 'LC', 'NN', 'AD', 'AD', 'AD', 'VV', 'VV', 'DEC', 'NN'], ['AD', 'DT', 'NN', 'LC', 'AD', 'VV', 'AS', 'NR', 'DEG', 'CD', 'NN', 'CC', 'NN', 'NN'], ['P', 'NN', 'PN', 'AD', 'VV', 'PN', 'AD', 'JJ'], ['AD', 'VV', 'AS', 'P', 'NN', 'DEG', 'NN'], ['JJ', 'DEG', 'JJ', 'NN', 'NN', 'CC', 'AD', 'VA', 'DEC', 'JJ', 'NN', 'VV', 'DER', 'AD', 'VA'], ['P', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'VV', 'P', 'NN', 'LC', 'VV', 'NN'], ['AD', 'NN', 'AD', 'AD', 'VV', 'VV', 'VV', 'DT', 'CD', 'NN', 'NN'], ['PN', 'VV', 'NT', 'P', 'NN', 'NN', 'AD', 'P', 'NN', 'VV', 'NN'], ['VV', 'VV', 'PN', 'LB', 'PN', 'VV', 'VA', 'AS', 'NN', 'NN', 'DEC', 'NR', 'M', 'VV', 'VV'], ['NR', 'NN', 'DEG', 'NN', 'JJ', 'DT', 'NN', 'AD', 'AD', 'NN', 'AD', 'VV', 'VV', 'VA', 'AD', 'VV', 'NN'], ['PN', 'CC', 'CD', 'VV', 'VV', 'VV', 'AD', 'VV', 'DEC', 'NN', 'VV', 'VV', 'AD'], ['VV', 'NR', 'VV', 'LC', 'NN', 'NN', 'AD', 'VV', 'AS', 'VV', 'VV', 'NN'], ['PN', 'DEG', 'NN', 'VC', 'NN', 'NN', 'PN', 'AD', 'VV', 'PN', 'AD', 'VC', 'VV', 'NN', 'CC', 'NN', 'NN'], ['NR', 'VA', 'DEC', 'VE', 'VV'], ['VE', 'DT', 'NN', 'SP', 'VV', 'P', 'PN', 'VV', 'AD', 'SP'], ['NN', 'VV', 'DT', 'NN', 'AD', 'AD', 'VV', 'AS', 'AD', 'VV', 'PN'], ['VV', 'AD', 'NN', 'VA', 'DEV', 'P', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'AS', 'VV'], ['VV', 'CD', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'AD', 'VA'], ['DT', 'VV', 'VA', 'DEC', 'NN', 'AD', 'NN', 'VA'], ['NN', 'DEG', 'NN', 'VA', 'NN', 'AD', 'NR', 'NR', 'VV', 'AD', 'VV', 'AD', 'AD', 'VV', 'AS', 'VV'], ['P', 'NT', 'NR', 'AD', 'P', 'NN', 'NN', 'VV', 'JJ', 'NR', 'NN', 'DEG', 'NN'], ['JJ', 'NN', 'NN', 'VV', 'CC', 'PN', 'DEG', 'NN', 'AD', 'AD', 'VV'], ['AD', 'AD', 'VV', 'NN', 'VA', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['NN', 'VV', 'CC', 'AD', 'DT', 'P', 'VV', 'CC', 'VV', 'VV', 'AS', 'DT', 'DT', 'DEG', 'NN'], ['AD', 'AD', 'P', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AD', 'VV', 'SP'], ['PN', 'DT', 'AD', 'NN', 'VV', 'DEC', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN', 'VE', 'NN', 'SP'], ['VV', 'VA', 'DEC', 'NN', 'NN', 'VV', 'VV', 'M', 'NN', 'AD', 'VV'], ['VV', 'NR', 'NN', 'NT', 'P', 'NN', 'VA', 'DEV', 'VV', 'VV', 'VV', 'NN'], ['CD', 'M', 'LC', 'NR', 'VV', 'VV', 'AS', 'NR', 'NN', 'DT', 'NN'], ['CD', 'CC', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN', 'VA'], ['NN', 'VV', 'OD', 'NN', 'CD', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VA'], ['P', 'JJ', 'NN', 'PN', 'AD', 'VV', 'AS', 'VV', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'AD', 'VA', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'JJ', 'NN', 'DEC', 'VV', 'VC', 'AD', 'VV', 'SP'], ['CD', 'AD', 'VC', 'NN', 'VV', 'NN', 'SP'], ['P', 'VV', 'AD', 'VV', 'PN', 'VV', 'DEC', 'AD', 'VA'], ['AD', 'JJ', 'NN', 'VV', 'NN', 'VC', 'PN'], ['PN', 'VE', 'CD', 'VV', 'NN', 'DEC', 'NN', 'AD', 'PN', 'AD', 'VA'], ['NR', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'NN'], ['NN', 'NN', 'AD', 'VV', 'JJ', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'P', 'VV', 'LC', 'VC', 'AD', 'VV', 'P', 'NN', 'VV', 'SP'], ['NR', 'CC', 'NR', 'NR'], ['NN', 'NN', 'CC', 'LC', 'NN', 'DEG', 'JJ', 'NN', 'AD', 'AD', 'VV'], ['P', 'AD', 'VA', 'DEC', 'AD', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN'], ['NR', 'VV', 'NR', 'DEG', 'NN', 'AD', 'VC', 'NN', 'NN', 'CD', 'VV', 'NN'], ['NN', 'JJ', 'NR', 'NN', 'AD', 'JJ'], ['NN', 'AD', 'P', 'CD', 'NN', 'AD', 'NR', 'VV', 'VV', 'AS', 'VV', 'NN'], ['VV', 'NN', 'NN', 'NN', 'AD', 'AD', 'NN', 'VA'], ['AD', 'VV', 'VV', 'CD', 'CD', 'VV'], ['AD', 'VV', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'DER', 'VA'], ['NN', 'AD', 'VV', 'AD', 'VV', 'VV', 'NN', 'NN', 'DEG', 'NN'], ['AD', 'CS', 'NN', 'AD', 'VA', 'AD', 'P', 'VV', 'NN', 'AD', 'AD', 'VV'], ['PN', 'AD', 'P', 'PN', 'DEG', 'NN', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'VV'], ['NN', 'VV', 'NN', 'VV', 'VE', 'NN'], ['AD', 'NR', 'NN', 'AD', 'AD', 'P', 'NR', 'VV'], ['OD', 'VC', 'NN', 'NN', 'PN', 'NN', 'NN', 'AD', 'AD', 'VA', 'AD', 'VE', 'NN'], ['PN', 'DEG', 'NN', 'NN', 'AD', 'AD', 'VV', 'PN'], ['VV', 'AD', 'NN', 'PN', 'AD', 'AD', 'VV', 'AD', 'VV', 'SP'], ['NN', 'VV', 'JJ', 'NN', 'NN', 'NR', 'JJ', 'JJ', 'NN', 'JJ', 'NN'], ['NR', 'VV', 'NT', 'PN', 'CC', 'NN', 'NR', 'VV', 'VV', 'NN'], ['NR', 'AD', 'VV', 'AD', 'VV', 'AS', 'NN'], ['P', 'NN', 'LC', 'VV', 'VV', 'JJ', 'NN', 'AD', 'VV', 'PN', 'AD', 'VV', 'AD', 'VV', 'VV'], ['PN', 'VC', 'NN', 'CC', 'NR', 'DEG', 'NN'], ['PN', 'AD', 'AD', 'VV', 'VV', 'DT', 'NN', 'NN'], ['CS', 'PN', 'VV', 'NN', 'AD', 'AD', 'VV', 'PN'], ['NN', 'VV', 'VV', 'DT', 'NN', 'VV', 'LC', 'DT', 'JJ', 'NN', 'VV'], ['NN', 'AD', 'VV', 'AS', 'JJ', 'NN', 'DEG', 'NN'], ['VE', 'NN', 'VV', 'AD', 'NN', 'VV', 'VV'], ['AD', 'CD', 'NN', 'VA', 'DEC', 'NR', 'NN', 'DEG', 'NN', 'AD', 'SB', 'VV'], ['NN', 'NN', 'NN', 'PN', 'AD', 'VV'], ['PN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'PN', 'VA'], ['NN', 'VV', 'NN', 'VV', 'AS', 'NN'], ['AD', 'DT', 'M', 'NN', 'VV', 'AS', 'NR'], ['NR', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['PN', 'P', 'NN', 'LC', 'AD', 'VV', 'AS'], ['AD', 'AD', 'VE', 'PN', 'VV', 'VV', 'AD', 'DEV', 'VV', 'PN', 'DEG', 'NN', 'SP'], ['NT', 'NN', 'NR', 'NN'], ['AD', 'AD', 'P', 'CD', 'VV', 'VV', 'DEC', 'NN', 'VV', 'VV'], ['NN', 'LC', 'AD', 'VV', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV'], ['CS', 'VC', 'NN', 'AD', 'VV', 'P', 'NN', 'NN', 'LC', 'VV'], ['PN', 'VV', 'VV', 'NN', 'VV', 'NN', 'NN', 'NN', 'AD', 'VA'], ['PN', 'VV', 'NN', 'VV', 'PN', 'VV', 'DT', 'NN'], ['AD', 'VE', 'NN', 'VV', 'NT', 'NN', 'DEG', 'NN', 'NR'], ['NT', 'VV', 'PN', 'LB', 'VV', 'NN', 'SP'], ['VV', 'M', 'VV', 'NN'], ['NR', 'VV', 'DEC', 'NN', 'VC', 'JJ', 'NN'], ['NN', 'NN', 'VC', 'NR', 'DEG', 'NN'], ['AD', 'AD', 'AD', 'VV', 'AD', 'VA'], ['PN', 'VC', 'PN', 'VV', 'VV', 'DEC', 'AD', 'VA'], ['NN', 'LC', 'VV', 'NN', 'AD', 'P', 'NN', 'NN', 'VV'], ['NN', 'AD', 'VV', 'NN', 'VV'], ['PN', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV', 'DEC', 'NN'], ['VV', 'NR', 'VV', 'VV', 'VV', 'NN', 'NN'], ['AD', 'CD', 'CD', 'NN', 'NR', 'NN', 'AD', 'VV', 'AS', 'JJ', 'DEG', 'NN'], ['NR', 'VA', 'DEV', 'AD', 'BA', 'NN', 'VV', 'AS', 'VV'], ['NN', 'AD', 'VV', 'AS', 'NR', 'NN'], ['JJ', 'NN', 'LC', 'NR', 'AD', 'VV'], ['CD', 'NN', 'VV', 'VV', 'NN', 'VV', 'AS', 'JJ', 'NN'], ['NT', 'AD', 'VV', 'NN', 'NN', 'VV'], ['PN', 'AD', 'VV', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['P', 'DT', 'NN', 'NN', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VV', 'PN', 'AD'], ['NN', 'NN', 'VV', 'CD', 'NN'], ['LB', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'AD', 'VV', 'SP'], ['NN', 'AD', 'AD', 'VV', 'AD', 'VV', 'DEV', 'MSP', 'VV', 'VV', 'AD', 'VV'], ['NR', 'JJ', 'NN', 'DEG', 'NN', 'VV', 'AS', 'NN', 'DEG', 'NN'], ['PN', 'VV', 'PN', 'CC', 'VV', 'VV', 'SP'], ['NN', 'NN', 'NN', 'NN', 'AD', 'VV'], ['AD', 'NN', 'AD', 'SB', 'VV', 'CD', 'JJ', 'DEG', 'NN'], ['AD', 'NR', 'DEG', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'LC', 'VC', 'NN', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VV', 'AS', 'NN', 'NN', 'DEG', 'NN', 'SP'], ['AD', 'VV', 'AD', 'AD', 'PN', 'VV', 'DEC', 'AD', 'VA'], ['DT', 'NN', 'SP'], ['AD', 'VV', 'AD', 'P', 'CD', 'M'], ['PN', 'VV', 'NN', 'AD', 'VV', 'AS', 'NR', 'AD'], ['CS', 'VV', 'NN', 'DEC', 'VC', 'DT', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV'], ['DT', 'NN', 'P', 'NR', 'DEG', 'NN', 'AD', 'VA', 'CC', 'VA'], ['AD', 'NN', 'AD', 'VV', 'AD'], ['P', 'NN', 'M', 'NN', 'VV', 'NN', 'VV', 'NN', 'DEC', 'NN', 'VV'], ['AD', 'CD', 'NN', 'AD', 'VV', 'NN', 'NN', 'NN', 'NN', 'VV', 'AS', 'NN'], ['NN', 'NT', 'PN', 'AD', 'VV', 'AS', 'VV', 'AD', 'VV', 'ON'], ['P', 'PN', 'VV', 'LC', 'VV', 'PN', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['AD', 'CD', 'NN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['PN', 'VV', 'AS', 'PN', 'NN', 'VV', 'SP', 'PN', 'NN'], ['PN', 'AD', 'VV', 'PN', 'VV', 'AD', 'AD', 'P', 'PN', 'VV', 'VV', 'NN'], ['PN', 'VV', 'VV', 'AD', 'AD', 'AD', 'AD'], ['NT', 'NN', 'AD', 'AD', 'VV', 'NN'], ['NT', 'VV', 'AS', 'M', 'AD', 'VA', 'NN'], ['PN', 'VV', 'P', 'NN', 'CC', 'NN', 'LC', 'VV', 'CC', 'VV', 'LC', 'NN', 'VV', 'VV', 'DEV', 'VV'], ['NR', 'P', 'NR', 'DEG', 'NN', 'VV', 'NN'], ['PN', 'AD', 'VC', 'P', 'NN', 'NN', 'AD', 'VA', 'DEC', 'NN'], ['NT', 'NN', 'NN', 'NN', 'NN', 'AD', 'VC', 'NN', 'P', 'NN', 'NN', 'NN', 'VA'], ['VV', 'NR', 'VV', 'NN', 'P', 'NN', 'VV', 'AS', 'NR', 'DT', 'DEG', 'NN', 'CC', 'PN', 'AD', 'VV'], ['NR', 'DEG', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'MSP', 'VV', 'DEC', 'JJ', 'NN', 'VC', 'NR'], ['DT', 'NN', 'VA', 'DEC', 'NN', 'VV', 'AS', 'VA', 'JJ', 'DEG', 'NN', 'VV', 'PN', 'DEG', 'NN'], ['PN', 'AD', 'VV', 'AS', 'VV', 'CS', 'NT', 'PN', 'AD', 'VC', 'NN', 'NN', 'DEC', 'NN', 'NN', 'NN'], ['AD', 'DEV', 'VV', 'VC', 'P', 'NN', 'VV', 'NN', 'AD', 'NN', 'AD', 'VC', 'NN', 'VV', 'SP'], ['NN', 'VV', 'NT', 'VV', 'VV', 'P', 'NN', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN'], ['NN', 'NN', 'VV', 'NN', 'VE', 'AS', 'NN', 'NN', 'VA', 'DEV', 'VV', 'P', 'P', 'NR', 'PN'], ['NN', 'AD', 'P', 'NR', 'P', 'NR', 'NN', 'NN', 'NN', 'CD', 'M', 'NN', 'VV', 'NN'], ['NR', 'VV', 'P', 'NN', 'NN', 'MSP', 'VV', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['VA', 'DEC', 'NN', 'NN', 'NN', 'AD', 'AD', 'JJ', 'NN', 'LC', 'NN', 'VA'], ['AD', 'VV', 'LC', 'VV', 'NN', 'VV', 'DT', 'CC', 'PN', 'DEG', 'NN', 'NR', 'VV', 'AS', 'DT', 'NN'], ['NN', 'P', 'VV', 'VC', 'NN', 'AD', 'VV', 'NN', 'LC', 'P', 'NN', 'NN', 'NN', 'VV', 'VV'], ['AD', 'NN', 'AD', 'AD', 'VA', 'NN', 'VA', 'DEV', 'VV', 'VV', 'VV'], ['NR', 'VV', 'AS', 'CD', 'NN', 'PN', 'AD', 'VV', 'VV', 'AD', 'VV', 'SP'], ['OD', 'M', 'JJ', 'NN', 'VC', 'CD', 'M', 'NN', 'NN', 'NN'], ['NN', 'DEG', 'NN', 'VC', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'AS', 'AD', 'VV'], ['AD', 'PN', 'VV', 'VV', 'DT', 'NN', 'DEG', 'NN', 'VV', 'BA', 'PN', 'VV', 'VV'], ['PN', 'VV', 'AS', 'PN', 'NT', 'AD', 'AD', 'VV', 'DEV', 'VV', 'AS', 'CD', 'VA'], ['NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'AS', 'PN', 'VV'], ['NN', 'AD', 'VV', 'LC', 'AD', 'VV', 'AD', 'VV', 'NN', 'AD', 'SP'], ['NT', 'NR', 'NN', 'DEG', 'JJ', 'NN', 'VC', 'NR', 'NR', 'CC', 'NR'], ['NN', 'NN', 'NN', 'AD', 'VC', 'VV', 'NN', 'NN', 'DEG', 'JJ', 'NN'], ['NT', 'NT', 'VV', 'NN', 'CD', 'M', 'CD', 'NT', 'VV', 'VV', 'NN'], ['NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'CC', 'NN', 'NN', 'VA'], ['NT', 'NN', 'AD', 'P', 'PN', 'VV', 'AD', 'VA', 'DEC', 'NN', 'VV', 'NN'], ['NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['NR', 'VV', 'DEC', 'CD', 'NN', 'P', 'AD', 'VA', 'AD', 'AD', 'VV'], ['NT', 'DEG', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VA'], ['NR', 'AD', 'VV', 'AD', 'VV', 'P', 'NN', 'VV', 'CC', 'VV', 'VV'], ['NN', 'NN', 'AD', 'AD', 'VC', 'P', 'NN', 'NN', 'AD', 'AD', 'NN', 'NN'], ['NR', 'VV', 'NN', 'AD', 'VV', 'VV', 'NR', 'NN', 'CC', 'NN', 'AD', 'VV'], ['NN', 'AD', 'VV', 'VA', 'VV', 'AD', 'VV', 'NN', 'NN', 'VV', 'VV'], ['VV', 'NR', 'LC', 'NR', 'VV', 'AS', 'NN', 'VV', 'NR', 'NN'], ['NR', 'NN', 'VV', 'VV', 'AS', 'LB', 'NR', 'ETC', 'NN', 'AD', 'VV'], ['VE', 'NN', 'NR', 'VC', 'NN', 'LC', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN'], ['AD', 'NR', 'CC', 'NR', 'DEG', 'NN', 'VV'], ['AD', 'VV', 'PN', 'VV', 'JJ', 'NN', 'NN', 'VV'], ['CD', 'VV', 'NN', 'DEC', 'NN', 'VV', 'AS', 'DT', 'CD', 'NN'], ['AD', 'VA', 'DEC', 'NN', 'VV', 'PN', 'VV', 'VV'], ['AD', 'NN', 'VV', 'JJ', 'NN', 'NN', 'VV', 'JJ', 'NN'], ['NR', 'P', 'VV', 'LC', 'P', 'NN', 'VV'], ['VV', 'VV', 'NN', 'VC', 'NN', 'VV', 'LC', 'NN', 'VC', 'NN'], ['PN', 'NN', 'CD', 'M', 'NN', 'CD', 'VV', 'VV', 'NN', 'CD', 'VA', 'NN', 'VA', 'VA', 'SP'], ['AD', 'PN', 'DEG', 'NN', 'ETC', 'NN', 'NN', 'NN', 'VA'], ['NR', 'VC', 'NN', 'AD', 'VE', 'NN', 'CC', 'NN', 'DEC', 'NN'], ['PN', 'AD', 'VV', 'VV', 'P', 'PN', 'VV', 'DEC', 'VA', 'SP'], ['NN', 'NN', 'NN', 'VA', 'DEV', 'AD', 'VV', 'NN'], ['NN', 'VV', 'VV', 'NR', 'VV', 'PN', 'NN', 'NR', 'VV'], ['AD', 'NN', 'VV', 'AS', 'AD', 'VV', 'NN', 'NN'], ['AD', 'AD', 'VV', 'VV', 'JJ', 'DEG', 'NN', 'CC', 'NN'], ['AD', 'AD', 'DEV', 'VV', 'PN', 'AD', 'VV', 'PN', 'SP'], ['AD', 'NN', 'AD', 'AD', 'VA', 'AD', 'PN', 'VV', 'AS'], ['NR', 'VV', 'DEC', 'AD', 'JJ', 'NN', 'VC', 'NN', 'NN'], ['AD', 'VA', 'AD', 'AD', 'VV', 'VV', 'NN', 'NN'], ['AD', 'PN', 'VV', 'AD', 'VV', 'AD', 'AD', 'VA'], ['NN', 'DEG', 'DT', 'NN', 'VV', 'PN', 'VE', 'NN', 'NN', 'AD', 'VV', 'VV', 'PN', 'VV'], ['AD', 'PN', 'CC', 'AD', 'VV', 'NN', 'VV', 'PN', 'P', 'PN', 'NN', 'DEG', 'NN', 'AD', 'VA', 'SP'], ['PN', 'VV', 'CS', 'PN', 'DEG', 'NN', 'P', 'NN', 'AD', 'VA', 'AD', 'AD', 'VV', 'AD', 'VV', 'NN', 'VV'], ['NN', 'LC', 'AD', 'DT', 'NN', 'NN', 'AD', 'VC', 'VV', 'NN', 'AD', 'VA', 'VA', 'DEC', 'NN'], ['P', 'NN', 'LC', 'DEG', 'JJ', 'NN', 'NR', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'AD', 'VV', 'AS', 'VV', 'NN', 'NN', 'DT', 'CD', 'NN', 'NN', 'CD'], ['NR', 'NN', 'NR', 'NN', 'NN', 'JJ', 'NN', 'NN', 'NN', 'VA'], ['NR', 'NN', 'AD', 'JJ', 'AD', 'VV'], ['VV', 'VV', 'VV', 'DEC', 'NN', 'NN', 'VV', 'CS', 'VE', 'NN', 'NN', 'VV', 'VV'], ['NR', 'NN', 'NN', 'AD', 'NN', 'VA'], ['JJ', 'NN', 'NN', 'NN', 'NR', 'NN', 'P', 'NN', 'LC', 'AD', 'VV'], ['NR', 'VV', 'NN', 'CC', 'VV', 'VV', 'P', 'NR', 'NN', 'DEG', 'NN'], ['AD', 'VV', 'VV', 'AD', 'VV', 'AD', 'VV', 'VC', 'PN', 'AD', 'VV', 'AS', 'VV', 'NN'], ['NN', 'NN', 'NN', 'VV', 'NN', 'NN', 'VV', 'VV', 'NN', 'NN'], ['VV', 'NR', 'NT', 'NR', 'AD', 'AD', 'VV', 'VV', 'NN', 'VV', 'VV'], ['VV', 'DEC', 'NN', 'PN', 'AD', 'AD', 'JJ', 'NN', 'AD', 'VV', 'VV', 'M', 'NN'], ['NR', 'P', 'NN', 'NN', 'AD', 'DEV', 'P', 'PN', 'NN', 'NN', 'VV', 'PN'], ['NN', 'VV', 'DEC', 'NN', 'AD', 'VE', 'AD', 'P', 'NR', 'VV', 'DEC', 'NN'], ['NN', 'AD', 'VA', 'VV', 'VV', 'VV', 'NN', 'NR', 'NN', 'NN', 'SP'], ['VV', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['VV', 'AS', 'NR', 'SP', 'PN', 'AD', 'VV', 'AS'], ['VV', 'PN', 'AD', 'VA', 'DEC', 'CC', 'NN', 'DT', 'VA', 'DEC', 'NN'], ['AD', 'NN', 'BA', 'PN', 'AD', 'VV', 'AS'], ['NN', 'VA', 'AD', 'VV', 'NN'], ['PN', 'AD', 'VV', 'JJ', 'VV', 'CC', 'VV', 'NN', 'NN', 'NN'], ['AD', 'AD', 'VV', 'PN', 'NN', 'CC', 'NN', 'VV', 'AS', 'NN'], ['NN', 'NN', 'VV', 'AD', 'VV'], ['PN', 'VV', 'JJ', 'JJ', 'NN', 'DEG', 'NN', 'AD', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN'], ['AD', 'VV', 'NN', 'VV', 'SP'], ['NR', 'VV', 'PN', 'VV', 'PN', 'NN', 'VV', 'NN'], ['NN', 'VV', 'AS', 'AD', 'NN', 'LC', 'DEG', 'NN', 'AD', 'VC', 'CD'], ['CD', 'M', 'AD', 'NN', 'VV', 'SP'], ['NR', 'AD', 'AD', 'VV', 'AS'], ['VV', 'P', 'NN', 'DEG', 'NN', 'SP'], ['CD', 'VE', 'NN', 'DEC', 'NN', 'NN'], ['VV', 'NN', 'AD', 'VV', 'AD', 'VV', 'NN', 'VA', 'DEV', 'VV', 'NN'], ['OD', 'NN', 'VC', 'NT', 'DEG', 'NN'], ['VV', 'NR', 'NR', 'NN', 'VV', 'NN'], ['PN', 'PN', 'NN', 'VA'], ['PN', 'VV', 'VE', 'NN', 'CC', 'PN', 'DEG', 'NN', 'NN', 'AD', 'NN', 'SP'], ['NN', 'VA', 'DEC', 'P', 'NN', 'P', 'NN', 'NN', 'VV', 'VV', 'VE', 'SP'], ['NR', 'AD', 'AD', 'VV', 'AD', 'VV', 'AS', 'CD', 'NN'], ['VV', 'NN', 'VV', 'PN', 'VA'], ['NR', 'AD', 'AD', 'VV', 'VV', 'VV', 'NN', 'DEC', 'NN', 'P', 'JJ', 'NN', 'NN', 'VV', 'VV'], ['PN', 'VV', 'DEC', 'NN', 'AD', 'VA', 'NN', 'VV', 'VV', 'VV', 'NN', 'VV'], ['AD', 'NR', 'AD', 'NN', 'NN', 'VV', 'NN', 'AD', 'VV'], ['NR', 'NN', 'DEG', 'NN', 'LC', 'SB', 'VV', 'DEC', 'NN', 'CC', 'NN', 'AD', 'AD'], ['NN', 'NN', 'NR', 'NN', 'NN', 'VV', 'NN', 'NN', 'CD', 'CD', 'NN'], ['VV', 'LC', 'NR', 'CD', 'NN', 'NN', 'AD', 'AD', 'VV', 'AD', 'P', 'NN', 'LC', 'VV', 'NN'], ['NN', 'AD', 'VV', 'DEV', 'VV', 'PN', 'AD', 'P', 'PN', 'NN', 'PN', 'AD', 'DT', 'CD', 'VV'], ['PN', 'AD', 'AD', 'CD', 'NN', 'P', 'NN', 'VV'], ['NR', 'NN', 'VV', 'NN', 'AD', 'VV', 'AS', 'NT'], ['NN', 'VC', 'M', 'VV', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN'], ['AD', 'VV', 'PN', 'NN', 'VV', 'AD', 'VA', 'SP'], ['VV', 'PN', 'DEG', 'VV', 'VV', 'AD', 'VV', 'AD', 'VA'], ['CS', 'VV', 'LC', 'NN', 'AD', 'SB', 'VV', 'VV', 'SP', 'SP'], ['P', 'NN', 'CC', 'JJ', 'NN', 'AD', 'VV'], ['PN', 'VV', 'NN', 'NN', 'LC', 'VV', 'VV'], ['NN', 'VV', 'PN', 'VV', 'NN', 'DEC', 'AD', 'AD', 'VA'], ['NN', 'NN', 'VE', 'NN', 'AD', 'VV', 'VV', 'JJ', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VV', 'VV'], ['PN', 'VV', 'AD', 'NN', 'VC', 'AD', 'NN', 'DEG', 'NN'], ['NR', 'NN', 'DEG', 'NN', 'VV', 'VV'], ['NN', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN'], ['AD', 'VV', 'AD', 'NN', 'NN', 'AD', 'VA', 'SP', 'SP'], ['VA', 'VA', 'DEC', 'NN', 'NN', 'NN', 'VA', 'DEC', 'NN'], ['NN', 'VV', 'AD', 'VV', 'AD', 'P', 'VV', 'NN', 'PN', 'VV'], ['NR', 'NR', 'DEG', 'NN', 'LC', 'VV', 'NN', 'VV', 'CD', 'NN', 'LC', 'VV', 'DEC', 'NN', 'VV', 'AS', 'CD'], ['NN', 'CC', 'NN', 'NN', 'VV', 'NR', 'VV', 'NR', 'AD', 'VV', 'NN'], ['P', 'NR', 'NN', 'NN', 'NN', 'AD', 'AD', 'VV', 'AD', 'P', 'PN', 'DEG', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'AD', 'VE', 'NN', 'BA', 'AD', 'VV', 'DEC', 'NN', 'VV', 'AS', 'PN', 'DEG', 'NN'], ['NN', 'VV', 'VV', 'NN', 'AD', 'NN', 'VV', 'AD', 'P', 'PN', 'VV', 'AS', 'NN'], ['NN', 'NN', 'VV', 'NN', 'P', 'JJ', 'NN', 'DEC', 'VV', 'NN', 'AD', 'VV'], ['NN', 'NN', 'AD', 'VV', 'VV'], ['AD', 'PN', 'AD', 'AD', 'VV', 'VV', 'AD', 'VV', 'NN', 'VV', 'NN', 'PN', 'DEG'], ['AD', 'AD', 'AD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'AD', 'VA'], ['DT', 'AD', 'VV', 'NN', 'SB', 'VV', 'NR', 'NN', 'CC', 'NN', 'VV', 'VV', 'VV'], ['NN', 'LC', 'NN', 'VA', 'AD'], ['AD', 'VV', 'JJ', 'NN'], ['NN', 'NN', 'VV', 'NN'], ['NR', 'NN', 'AD', 'VV', 'AS', 'NN'], ['NR', 'AD', 'P', 'CD', 'VV', 'CD', 'NN', 'CD', 'CD', 'CD'], ['DT', 'NN', 'P', 'PN', 'VV', 'NN', 'NN'], ['PN', 'DEG', 'NN', 'AD', 'AD', 'VA', 'SP', 'SP'], ['P', 'NT', 'VV', 'NN', 'NR', 'PN', 'AD', 'AD', 'VV', 'VV', 'NN', 'AD', 'VV', 'AS', 'VV'], ['AD', 'NR', 'P', 'PN', 'DEG', 'NN', 'CC', 'NN', 'VV', 'VV'], ['PN', 'VV', 'DT', 'JJ', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'VV', 'AS', 'VV'], ['CD', 'NN', 'VV', 'P', 'VA', 'DEC', 'NN', 'VV', 'AS', 'NN', 'VV', 'PN', 'VV'], ['NN', 'VV', 'PN', 'AD', 'VV', 'DER', 'VA', 'AD', 'AD', 'VV', 'NN', 'NN'], ['VE', 'AS', 'NN', 'VV', 'VV', 'NN', 'NN', 'DEC', 'NN', 'AD', 'NN', 'VV'], ['PN', 'VV', 'VV', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'CD', 'NN', 'VV', 'AS', 'VV'], ['VV', 'AS', 'NN', 'CD', 'NN', 'VV', 'VA', 'P', 'PN', 'VV', 'VV', 'VV'], ['AD', 'BA', 'VV', 'NN', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV'], ['NT', 'LC', 'VV', 'P', 'NR', 'CD', 'NN', 'NN', 'VV', 'AS', 'NN', 'NN', 'NN'], ['CD', 'JJ', 'DEG', 'NN', 'VA', 'DEV', 'VV', 'NN', 'VV', 'CC', 'VV', 'VV'], ['NR', 'NN', 'AD', 'VA', 'VV', 'NN', 'DEC', 'NR', 'NN', 'AD', 'VV', 'NN'], ['NN', 'VV', 'VV', 'VV', 'CD', 'VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VA'], ['AD', 'VV', 'AD', 'AD', 'VC', 'NN', 'AD', 'VV', 'VV', 'NN', 'PN', 'DEG'], ['VA', 'VA', 'VV', 'AS', 'PN', 'AD', 'AD', 'VV', 'VA', 'AS', 'NN', 'PN', 'AD', 'VV', 'NN', 'SP'], ['NR', 'VV', 'NN', 'VC', 'P', 'JJ', 'NN', 'VV', 'DEC', 'CC', 'P', 'NN', 'VV', 'SP'], ['NN', 'LC', 'PN', 'SB', 'VV', 'AS', 'P', 'NR', 'AD', 'VV', 'VV', 'AS', 'AD', 'VA', 'NN'], ['VV', 'VV', 'LC', 'DEG', 'NR', 'JJ', 'NN', 'VV', 'CC', 'VV', 'NN', 'DEC', 'NN'], ['OD', 'VE', 'CD', 'CD', 'M', 'NN', 'CC', 'CD', 'CD', 'NN', 'NN', 'SB', 'VV'], ['PN', 'DT', 'JJ', 'NN', 'AD', 'NT', 'NT', 'NT', 'NN', 'VV', 'NR', 'DEG', 'NN'], ['NN', 'AD', 'VA', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'PN', 'AD', 'VV', 'ON'], ['P', 'NR', 'VV', 'NR', 'NN', 'LC', 'NR', 'AD', 'VV', 'NR', 'NN'], ['AD', 'VV', 'PN', 'AD', 'VV', 'NN', 'P', 'VV', 'NN', 'AD', 'VV', 'AS', 'PN', 'AD', 'AD', 'VV', 'VA'], ['AD', 'VV', 'AD', 'AD', 'AD', 'VV', 'NR', 'NR', 'NN', 'NN', 'DEG', 'NN', 'NN'], ['PN', 'VV', 'VV', 'AD', 'PN', 'VV', 'VV', 'AS', 'NT', 'PN', 'AD', 'VV', 'MSP', 'VV', 'VA', 'SP'], ['P', 'VV', 'LC', 'AD', 'VV', 'AS', 'NN', 'NN', 'VV', 'LC', 'DEG', 'NN', 'NN', 'VV', 'NN', 'VA'], ['NR', 'NN', 'VV', 'AD', 'NN', 'NN', 'VV', 'AS', 'P', 'NN', 'LC', 'DEC', 'NR', 'NN'], ['NT', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'VV', 'AS', 'CD', 'AD', 'VV', 'DEC', 'NN'], ['P', 'NN', 'AD', 'VV', 'NN', 'AD', 'NN', 'CC', 'NN', 'DEG', 'NN', 'AD', 'VV'], ['NN', 'NN', 'DEG', 'NR', 'NR', 'NN', 'AD', 'VV', 'NN', 'VV', 'PN', 'AD', 'NR'], ['DT', 'NN', 'VV', 'NN', 'JJ', 'NN', 'VV', 'VV', 'NN', 'NN', 'NN', 'NN', 'NN'], ['JJ', 'DEG', 'VE', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'ETC'], ['PN', 'VA', 'DEC', 'NN', 'AD', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VV', 'NN', 'VV', 'VV', 'AS', 'AD', 'NR', 'NN', 'VV', 'AS', 'NR', 'NN', 'DEG', 'NN'], ['P', 'VV', 'VV', 'VV', 'VV', 'PN', 'AD', 'VV', 'VA', 'DEC', 'NN', 'AD', 'VV', 'AS', 'VV', 'VV', 'AD', 'VV'], ['NT', 'VV', 'NN', 'PN', 'VV', 'NN', 'VC', 'PN'], ['NN', 'NN', 'AD', 'VV', 'VV', 'NT', 'NT', 'CC', 'NT', 'NT'], ['CS', 'VV', 'VV', 'AD', 'VA', 'AD', 'VV', 'NN', 'VV', 'AD', 'VA'], ['NN', 'AD', 'VV', 'DT', 'VE', 'NN', 'VV', 'DEC', 'NN'], ['VV', 'NN', 'PN', 'BA', 'JJ', 'DEG', 'NN', 'NN', 'VV', 'NN'], ['NN', 'SP', 'NN', 'VV', 'VC', 'AD', 'VA', 'SP'], ['NR', 'NR', 'VV', 'CC', 'AD', 'VV', 'VV', 'VV', 'PN', 'VV'], ['NR', 'VV', 'M', 'NN', 'VV', 'LC', 'AD', 'VV', 'NN'], ['PN', 'VV', 'VV', 'PN', 'NN'], ['NN', 'VV', 'NN', 'SP'], ['VV', 'VC', 'NN', 'VV', 'SP'], ['NN', 'VV', 'VV'], ['PN', 'VV', 'PN', 'VV'], ['VV', 'AD', 'CD'], ['NN', 'DEG', 'NN', 'AD', 'VV', 'JJ', 'NN', 'NN', 'AD', 'VV', 'PN', 'JJ', 'NN', 'LC', 'AD', 'AD', 'VV', 'VV', 'PN', 'SP'], ['NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'AD', 'BA', 'NN', 'VV', 'VV', 'NN', 'DEG', 'NR', 'PN', 'SP'], ['NN', 'NN', 'AD', 'VV', 'VV', 'VE', 'NN', 'P', 'NN', 'VV', 'DEC', 'NN', 'LC', 'VV'], ['VV', 'NR', 'NN', 'NN', 'DEG', 'NR', 'VV', 'VV', 'NN', 'NN', 'NN'], ['NR', 'P', 'NN', 'AD', 'VV', 'AD', 'AD', 'VV', 'AD', 'AD', 'VV', 'VV', 'AD', 'AD', 'VV'], ['AD', 'PN', 'NN', 'AD', 'AD', 'VV', 'NN', 'AS', 'ON', 'VV', 'PN', 'VC', 'P', 'NN', 'NN', 'SP'], ['CD', 'NN', 'VV', 'VV', 'AD', 'VV', 'PN', 'VV', 'NN', 'NN', 'SP', 'PN', 'VC', 'NN'], ['NN', 'DEG', 'NN', 'VC', 'PN', 'VV', 'DEC', 'NN', 'AD', 'VV', 'PN', 'VE', 'VV', 'NN', 'NN', 'NN'], ['P', 'NT', 'VV', 'NN', 'AD', 'VV', 'NT', 'NN', 'AD', 'VV', 'NN'], ['DT', 'NN', 'VV', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN', 'VV', 'NN', 'NN'], ['DT', 'M', 'NN', 'DEC', 'VV', 'NN', 'AD', 'VC', 'NN', 'AD', 'VV', 'NN', 'DEC', 'NN'], ['NN', 'VC', 'VA', 'PN', 'VV', 'PN', 'VC', 'P', 'CD', 'NN', 'VA', 'DEC', 'NN', 'P', 'PN', 'VV', 'DEC', 'NN'], ['NN', 'VV', 'AS', 'NN', 'VV', 'NN', 'NN', 'CC', 'NN', 'NN', 'VV'], ['AD', 'DT', 'NN', 'DEG', 'NN', 'NN', 'NN', 'AD', 'VV', 'DT', 'NN'], ['AD', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'VV'], ['AD', 'VV', 'NN', 'VV', 'AS', 'JJ', 'NN', 'NN', 'DEG', 'NN', 'LC', 'VV', 'AD', 'VV'], ['VV', 'SB', 'VV', 'NR', 'NN', 'DEG', 'NN', 'NR', 'VV', 'NN'], ['PN', 'P', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN', 'AD', 'VE', 'JJ', 'NN'], ['PN', 'NN', 'NN', 'AD', 'AD', 'AD', 'NN', 'AD', 'VA'], ['DT', 'CD', 'NN', 'NN', 'AD', 'VA', 'DEV', 'VV', 'VV', 'AS', 'AD', 'CS', 'NN'], ['AD', 'BA', 'NN', 'VV', 'VV', 'VV', 'VV', 'NN', 'NN', 'AD', 'VV', 'NN'], ['NT', 'VV', 'NN', 'CC', 'NN', 'AD', 'LB', 'NN', 'AD', 'NN', 'NN'], ['NR', 'VV', 'NR', 'VV', 'NN', 'VC', 'P', 'VV', 'NN', 'SP'], ['PN', 'VV', 'PN', 'AD', 'VV', 'AD', 'VV', 'VV', 'LC', 'AD', 'VV', 'VV', 'DER', 'PN', 'VV', 'AD', 'VV', 'NN'], ['NR', 'NR', 'P', 'VV', 'LC', 'AD', 'VV', 'AS', 'AD', 'AD', 'VV', 'NN', 'SP'], ['AD', 'NN', 'CC', 'NN', 'NN', 'VV', 'VV', 'AD', 'VV'], ['NR', 'DEG', 'NN', 'NR', 'VV', 'NN', 'VV', 'PN', 'AD', 'VA'], ['VV', 'NN', 'VV', 'NN', 'NN', 'AD', 'VC', 'NN', 'DEG', 'NN', 'NN'], ['PN', 'VV', 'NN', 'VV', 'AD', 'P', 'NN', 'LC', 'DEG', 'NN', 'VV'], ['AD', 'P', 'AD', 'VV', 'NN', 'NN', 'AD', 'VV', 'AS', 'NN', 'VV', 'NN'], ['DT', 'AD', 'NN', 'AD', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV'], ['NR', 'NR', 'NR', 'NN', 'AD', 'VV', 'AS', 'NN', 'NN'], ['NR', 'AD', 'VV', 'VV', 'VC', 'P', 'VV', 'NR', 'DEG', 'NN', 'NN'], ['NR', 'VV', 'P', 'NR', 'NR', 'NN', 'NR', 'DEG', 'NR', 'NN'], ['NR', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'NR'], ['AD', 'NT', 'NR', 'NN', 'VV', 'VV', 'AD', 'VV', 'NN'], ['P', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'VV', 'NN', 'VV'], ['AD', 'VV', 'NR', 'VV', 'DEC', 'VC', 'NN', 'DEG', 'NN', 'AD', 'VV', 'AD', 'VV', 'SP'], ['VV', 'NN', 'LC', 'VV', 'VV', 'NN', 'DEG', 'NN', 'NN', 'VV', 'NN', 'VV', 'VA'], ['PN', 'P', 'NN', 'VV', 'AS', 'VV', 'PN', 'VV', 'NN', 'DEC', 'NN', 'VV', 'VV', 'VV', 'VV', 'VV'], ['NT', 'NT', 'NN', 'VV', 'AS', 'NN', 'NN', 'VV', 'PN', 'AD', 'PN', 'VV', 'DEC', 'CD', 'NN', 'SP'], ['PN', 'AD', 'VV', 'VV', 'NN', 'VV', 'PN', 'PN', 'VC', 'AD', 'DEV', 'AD', 'AD', 'AD', 'CC', 'NN', 'VV', 'NN'], ['AD', 'VV', 'NN', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN', 'AD', 'NN', 'NN'], ['VV', 'VE', 'AD', 'JJ', 'DEG', 'NN', 'VV', 'VV', 'AD', 'VV'], ['AD', 'NN', 'AD', 'AD', 'VV', 'VV', 'DER', 'NN', 'AD', 'VA'], ['NN', 'VV', 'DEC', 'VC', 'NN', 'NN', 'VV', 'DEC', 'NN'], ['CS', 'IJ', 'NN', 'VV', 'PN', 'PN', 'AD', 'AD', 'VV', 'DEV', 'VV'], ['PN', 'VV', 'VV', 'CD', 'NN', 'AD', 'VV', 'NN', 'LC'], ['PN', 'VV', 'NN', 'AD', 'VV', 'VV', 'NN', 'DEC', 'NN'], ['AD', 'NR', 'NN', 'NN', 'VV', 'PN', 'NN'], ['NN', 'NN', 'NN', 'NN', 'AD', 'VV', 'DEC', 'VA'], ['NR', 'NN', 'NN', 'VV', 'VV', 'DEC', 'NN', 'VA'], ['DT', 'VV', 'NR', 'VV', 'VE', 'CD', 'VA', 'DEC', 'NN', 'NN'], ['AD', 'P', 'NN', 'LC', 'DEG', 'NN', 'PN', 'CD', 'AD', 'AD', 'VV'], ['JJ', 'NN', 'VV', 'NR', 'VV', 'NR', 'AD', 'VC', 'P', 'PN'], ['NN', 'VV', 'NN', 'BA', 'PN', 'NN', 'DEC', 'NR', 'NN', 'VA'], ['AD', 'AD', 'DEV', 'NN', 'NN', 'LB', 'JJ', 'NN', 'VV', 'AS'], ['NN', 'LC', 'VV', 'NN', 'NR', 'VV', 'VV', 'NN'], ['AD', 'VV', 'VV', 'NR', 'DEG', 'NN', 'NN', 'VV', 'PN'], ['NR', 'VV', 'VC', 'NN', 'VV', 'AS', 'PN', 'CC', 'NN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'VV', 'NN', 'PN', 'VV', 'AD', 'AD', 'VV'], ['NR', 'VV', 'NR', 'NR', 'NN', 'JJ', 'NN', 'VA'], ['NN', 'VV', 'AD', 'VA', 'AD', 'AD', 'NN', 'NN', 'PN', 'AD', 'VV', 'AS'], ['PN', 'NN', 'AD', 'VV', 'VV', 'NN', 'VV', 'AS', 'NT'], ['PN', 'NT', 'AD', 'VV', 'PN', 'AD', 'VV', 'VV', 'VV', 'PN', 'DEC', 'NN'], ['AD', 'NR', 'NN', 'NN', 'AD', 'VV', 'JJ', 'CD', 'NN'], ['AD', 'NR', 'VE', 'NN', 'NN', 'NN', 'DEG', 'NN'], ['NT', 'NR', 'NN', 'NN', 'NN', 'VV', 'AS', 'NR'], ['NR', 'NN', 'NN', 'AD', 'VV', 'DT', 'NN', 'NN', 'DEG', 'NN'], ['P', 'NN', 'NN', 'LC', 'LC', 'PN', 'AD', 'AD', 'VV', 'VC', 'NN'], ['NT', 'VV', 'CS', 'VV', 'NN', 'AD', 'VV', 'NN', 'NN'], ['NR', 'P', 'NN', 'NN', 'NN', 'VV', 'NN', 'JJ', 'NN'], ['AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'AD', 'VV'], ['NN', 'VV', 'NN', 'PN', 'VV', 'PN', 'NN', 'AS', 'VV', 'AD'], ['AD', 'VV', 'VV', 'LC', 'NN', 'NN', 'DEG', 'NN', 'AD', 'AD', 'VV'], ['NR', 'NN', 'NN', 'NN', 'AD', 'VV', 'NN', 'NN'], ['DT', 'NN', 'DEG', 'NN', 'CD', 'VV', 'AD', 'NR', 'NN', 'VV', 'DEC', 'NN', 'NN'], ['VV', 'NR', 'NN', 'NN', 'DEG', 'NN', 'CC', 'VC', 'NN', 'VV', 'P', 'PN', 'AD', 'VV', 'VA'], ['NR', 'JJ', 'NR', 'NR', 'NR', 'NR', 'NR', 'NR', 'NR', 'NN'], ['NR', 'SP', 'VV', 'AS', 'NR', 'NN', 'DT', 'NR', 'NN', 'DEG', 'JJ', 'NN'], ['P', 'PN', 'MSP', 'VV', 'DEC', 'NN', 'AD', 'P', 'NN', 'VV', 'AD', 'VV'], ['VV', 'NN', 'DEC', 'NN', 'NN', 'AD', 'VV', 'VV', 'NN'], ['VV', 'VV', 'NN', 'VV', 'NN', 'AD', 'NN', 'NN', 'AD', 'VV', 'PN', 'VV'], ['NN', 'NN', 'CD', 'P', 'VV', 'VV', 'AD', 'AD', 'VV'], ['AD', 'NR', 'NR', 'AD', 'NR', 'NR', 'NN', 'AD', 'NN', 'SP'], ['AD', 'PN', 'NN', 'NN', 'AD', 'P', 'PN', 'NN', 'P', 'NN', 'LC', 'NN', 'NN'], ['VV', 'NN', 'AD', 'VV', 'JJ', 'NN', 'DEG', 'NN', 'PN', 'NT', 'VV'], ['AD', 'VC', 'AD', 'VE', 'CD', 'NN', 'VV', 'DEC', 'NN', 'VV', 'VV', 'AS', 'AD', 'VA', 'AD', 'DEC', 'NN'], ['NN', 'VV', 'NN', 'PN', 'VV', 'P', 'PN', 'VV'], ['NN', 'P', 'NT', 'NR', 'NN', 'VV', 'NN'], ['AD', 'NR', 'NN', 'P', 'NR', 'NT', 'NR', 'AD', 'VA'], ['VV', 'VV', 'PN', 'VV', 'AD', 'VE', 'NN', 'DEV', 'VV', 'PN', 'AD', 'VV', 'AS'], ['NR', 'VV', 'P', 'NN', 'NN', 'LC', 'AD', 'PN', 'VV', 'VV', 'SP'], ['NR', 'DEG', 'NN', 'NN', 'VV', 'AS', 'AD', 'CD', 'M'], ['NN', 'VV', 'NT', 'NT', 'NT', 'CD', 'NT', 'CD', 'NT', 'OD', 'M', 'CD', 'JJ', 'NN'], ['NN', 'AD', 'AD', 'NN', 'DEG', 'NN', 'PN', 'NN', 'CC', 'PN', 'VV', 'AD', 'PN', 'AD', 'VV', 'VV', 'NN', 'AD', 'VA'], ['DT', 'PN', 'AD', 'VV', 'NN', 'VV', 'NN', 'NN', 'VV', 'AD', 'VV', 'VV', 'AD'], ['CC', 'JJ', 'CD', 'CD', 'M', 'PN', 'AD', 'AD', 'VV'], ['AD', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN', 'AD', 'VV', 'NN'], ['AD', 'VV', 'CD', 'LC', 'AD', 'AD', 'CD', 'NN', 'DEG', 'NN'], ['VV', 'NN', 'PN', 'AD', 'VV', 'AD', 'VV', 'DEC', 'NN'], ['AD', 'P', 'NN', 'NN', 'VV', 'AS', 'BA', 'PN', 'VV', 'AS', 'VV'], ['VE', 'CD', 'NN', 'VV', 'NR', 'PN', 'VV', 'AS', 'PN', 'NN', 'VA'], ['VV', 'AS', 'PN', 'LC', 'AD', 'VV', 'PN', 'AD', 'AD', 'VV', 'SP'], ['AD', 'VV', 'NR', 'AD', 'VV', 'AS', 'NN'], ['NN', 'LC', 'DEG', 'PN', 'VV', 'P', 'NN', 'LC', 'DEG', 'PN', 'VV', 'PN'], ['AD', 'NN', 'AD', 'VV', 'NN', 'DEC', 'VV', 'VV', 'VA'], ['PN', 'VV', 'DT', 'NN', 'AD', 'VV', 'VV', 'PN', 'DEG', 'NN'], ['NN', 'DEG', 'NN', 'AD', 'P', 'NN', 'VV', 'NN'], ['NT', 'VV', 'NN', 'VV', 'VV', 'VV', 'AS', 'JJ', 'NN', 'NN', 'VV', 'CD', 'DEC', 'NN', 'VV', 'AD'], ['NN', 'AD', 'NN', 'VV', 'LC', 'P', 'NN', 'LC', 'VV'], ['NN', 'AD', 'VV', 'NN', 'AD', 'BA', 'AD', 'VA', 'AS', 'AD', 'VV', 'VV', 'AS', 'JJ', 'NN', 'NN'], ['AD', 'NN', 'VA', 'DEV', 'VV', 'PN', 'AD', 'VV', 'VV', 'PN', 'AD', 'VV', 'AS', 'PN', 'VV', 'VV', 'SP'], ['NR', 'VV', 'VV', 'CC', 'PN', 'AD', 'NN', 'VV', 'AS', 'VV', 'VV', 'NN', 'SP'], ['AD', 'NN', 'VV', 'NN', 'VV', 'NN', 'VV', 'AD', 'NN', 'NN', 'NN'], ['NR', 'VV', 'VV', 'PN', 'NT', 'P', 'NN', 'LC', 'VV', 'AS', 'AD', 'VA', 'DEC', 'NN', 'NN'], ['DT', 'CD', 'NN', 'VV', 'AS', 'VV', 'NN', 'VV', 'AS', 'JJ', 'DEG', 'NN', 'P', 'PN', 'NN', 'VV'], ['NT', 'NN', 'NN', 'VV', 'AS', 'CD', 'NN', 'P', 'NN', 'VV', 'NR', 'VC', 'DT', 'VE', 'PN', 'NN'], ['AD', 'NN', 'NN', 'AD', 'VC', 'AD', 'VV', 'VV', 'DEC', 'AD', 'AD', 'VC', 'PN', 'VV', 'SP', 'PN'], ['P', 'NR', 'NR', 'NN', 'NN', 'P', 'PN', 'NN', 'NN'], ['P', 'NN', 'NR', 'VV', 'VV', 'VV', 'AS', 'JJ', 'NN']]
three = []
for lists in POSlist:
    zerovec = np.zeros(33)
    three = []
    lenth = len(lists)
    lists.append('PU')
    lists.insert(0,'PU')
    for i in range(0,lenth):
        vec1 = zerovec.tolist()
        vec1[int(dictPOS[lists[i]])] = 1
        vec2 = zerovec.tolist()
        vec2[int(dictPOS[lists[i + 1]])] = 1
        vec3 = zerovec.tolist()
        vec3[int(dictPOS[lists[i + 2]])] = 1
        three.append(vec1+vec2+vec3)
    feature4.append(three)
##feature5:距离上/下标点的字数
feature5 = []
for j in range(0,len(textlist)):
    a = wordlenlist[j]
    b = feature3[j]
    forwardlen = 0
    backwardlen = 0
    wcount = len(a)
    tempfeature5 = np.zeros((wcount,2))
    for i in range(0,wcount):
        forwardlen += a[i]
        backwardlen += a[-i-1]
        punforward = np.array(b[i])
        punbackward = np.array(b[wcount-i-1])
        if any(punforward):
            forwardlen = 0
        if any(punbackward):
            backwardlen = 0
        tempfeature5[i][0] = forwardlen
        tempfeature5[i][1] = backwardlen
    feature5.append(tempfeature5)

##feature6:上下词和本词的深度（后来搞短语类型试一试，见论文2）
#deepth是每个词的深度，见缓存区
feature6 = []
'''
from nltk.parse.stanford import StanfordParser as PAR
from tqdm import tqdm_notebook as tqdm

full_path=u"/home/data/liuchang/data/stanford-parser-full"
cnparser = PAR(
path_to_jar=full_path + u"/stanford-parser.jar",
path_to_models_jar=full_path +u"/stanford-parser-3.9.2-models.jar",
model_path=full_path +u'/chinesePCFG.ser.gz')
onedep = []
deepth = []
for praindex in tqdm(range(0,len(textlist))):
    sent = textlist[praindex]
    partree = list(cnparser.parse(sent.split()))
    cntree=str(partree)
    count = 0
    charnum = 0
    onedep = []
    for chara in cntree:
        if chara == '[':
            count = count + 1
        if chara == ']':
            count = count - 1
        if u'\u4e00' <= chara <= u'\u9fff':
            if (not (u'\u4e00' <= cntree[charnum+1]<= u'\u9fff')):
                onedep.append(count)
        charnum = charnum+1
    deepth.append(onedep)
'''
deepth=[]
fd = open( "/home/data/liuchang/data/deepthList.txt", "r" )
List_row = fd.readlines()

for i in range(len(List_row)):
    onedep = list(map(int, re.findall('\d+', List_row[i])))
    deepth.append(onedep)               

three = []
mix = []
for lists in deepth:
    three = []
    lenth = len(lists)
    lists.append(0)
    lists.insert(0,0)
    for i in range(0,lenth):
        three.append(lists[i])
        three.append(lists[i+1])
        three.append(lists[i+2])
    mix.append(three)
step = 3
for feature in mix:
    feature = [feature[i:i+step] for i in range(0,len(feature),step)]
    feature6.append(feature)
##feature7:word2vec
feature7 = []
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')# 忽略警告
import sys  
import gensim
fdir = '/home/data/liuchang/data/'
#model = gensim.models.Word2Vec.load(fdir + 'wikizh.model')
model = gensim.models.Word2Vec.load(fdir + 'Tinywikizh.model')
#vec0 = [ 0 for n in range(400)]
vec0 = [ 0 for n in range(len(model['西电']))]
for cnwords in tqdm(cnwordlist):
    veclist = []
    for word in cnwords:
        try:
            veclist.append(model[word])
        except:
            veclist.append(vec0)
    feature7.append(veclist)
##7 in 1 + tags
print('各个特征的长度')
print(1)
print(len(feature2[0][0]))
print(len(feature3[0][0]))
print(len(feature4[0][0]))
print(len(feature5[0][0]))
print(len(feature6[0][0]))
print(len(feature7[0][0]))
featuresum = []

    
"""for i in tqdm(range(0,len(textlist))):    
    for j in range(0,len(cnwordlist[i])):
        finalfeature=[]
        finalfeature.append(feature1[i])
        for j2 in range(0,len(feature2[i][j])):
            finalfeature.append(feature2[i][j][j2])
        for j3 in range(0,len(feature3[i][j])):
            finalfeature.append(feature3[i][j][j3])              
        for j4 in range(0,len(feature4[i][j])):
            finalfeature.append(feature4[i][j][j4])        
        for j5 in range(0,len(feature5[i][j])):
            finalfeature.append(feature5[i][j][j5])
        for j6 in range(0,len(feature6[i][j])):
            finalfeature.append(feature6[i][j][j6])
        for j7 in range(0,len(feature7[i][j])):
            finalfeature.append(feature7[i][j][j7])
        finalfeature.append(classlist[i][j])
        featuresum.append(finalfeature)   """        
#make csv
featurename = []
#featurename.append('f1_length')
#for j2 in range(0,len(feature2[0][0])):
#    featurename.append('f2_LTN_numbers'+str(j2))
for j3 in range(0,len(feature3[0][0])):
    featurename.append('f3_next_pun'+str(j3))
for j4 in range(0,len(feature4[0][0])):
    featurename.append('f4_LTN_POS'+str(j4))
#for j5 in range(0,len(feature5[0][0])):
#    featurename.append('f5_LNpun_numbers'+str(j5))
for j6 in range(0,len(feature6[0][0])):
    featurename.append('f6_LTN_deepth'+str(j6))  
#for j7 in range(0,len(feature7[0][0])):
#    featurename.append('f6_W2V'+str(j7))
featurename.append('tag')

featuresum1 = []
for i in tqdm(range(0,9900)):    
    for j in range(0,len(cnwordlist[i])):
        finalfeature1 = []
#        finalfeature1.append(feature1[i])
#        for j2 in range(0,len(feature2[i][j])):
#            finalfeature1.append(feature2[i][j][j2])
        for j3 in range(0,len(feature3[i][j])):
            finalfeature1.append(feature3[i][j][j3])              
        for j4 in range(0,len(feature4[i][j])):
            finalfeature1.append(feature4[i][j][j4])        
#        for j5 in range(0,len(feature5[i][j])):
#            finalfeature1.append(feature5[i][j][j5])
        for j6 in range(0,len(feature6[i][j])):
            finalfeature1.append(feature6[i][j][j6])
#        for j7 in range(0,len(feature7[i][j])):
#            finalfeature1.append(feature7[i][j][j7])
        finalfeature1.append(classlist[i][j])
        featuresum1.append(finalfeature1)        
featuresum2 = []
for i in tqdm(range(9900,10000)):    
    for j in range(0,len(cnwordlist[i])):
        finalfeature2 = []
#        finalfeature2.append(feature1[i])
#        for j2 in range(0,len(feature2[i][j])):
#            finalfeature2.append(feature2[i][j][j2])
        for j3 in range(0,len(feature3[i][j])):
            finalfeature2.append(feature3[i][j][j3])              
        for j4 in range(0,len(feature4[i][j])):
            finalfeature2.append(feature4[i][j][j4])        
#        for j5 in range(0,len(feature5[i][j])):
#            finalfeature2.append(feature5[i][j][j5])
        for j6 in range(0,len(feature6[i][j])):
            finalfeature2.append(feature6[i][j][j6])
#        for j7 in range(0,len(feature7[i][j])):
#            finalfeature2.append(feature7[i][j][j7])
        finalfeature2.append(classlist[i][j])
        featuresum2.append(finalfeature2)      

#test = pd.DataFrame(columns = featurename,data = featuresum )
#test.to_csv('/home/data/liuchang/data/ALLTOBI.csv',encoding = 'utf-8')
test1 = pd.DataFrame(columns = featurename,data = featuresum1 )
test1.to_csv('/home/data/liuchang/data/train2base.csv',encoding = 'utf-8',index=False)
test1 = pd.DataFrame(columns = featurename,data = featuresum2)
test1.to_csv('/home/data/liuchang/data/test2base.csv',encoding = 'utf-8',index=False)
#STEP3:CART
#训练
#测试？怎么写正确率，精确，召回，F0？
import csv
from collections import defaultdict #看起来是用来计数的
import numpy as np

t = time.time()

class Tree:
    def __init__(self, value=None, trueBranch=None, falseBranch=None, results=None, col=-1, summary=None, data=None):
        self.value = value
        self.trueBranch = trueBranch
        self.falseBranch = falseBranch
        self.results = results
        self.col = col
        self.summary = summary
        self.data = data


def calculateDiffCount(datas):
    # 将输入的数据汇总(input dataSet)
    # return results Set{type1:type1Count,type2:type2Count ... typeN:typeNCount}

    results = {}
    for data in datas:
        # data[-1] means dataType
        if data[-1] not in results:
            results[data[-1]] = 1
        else:
            results[data[-1]] += 1
    return results


def gini(rows):
    # 计算gini值(Calculate GINI)

    length = len(rows)
    results = calculateDiffCount(rows)
    imp = 0.0
    for i in results:
        imp += results[i] / length * results[i] / length
    return 1 - imp


def splitDatas(rows, value, column):
    # 根据条件分离数据集(splitDatas by value,column)
    # return 2 part(list1,list2)

    list1 = []
    list2 = []
    #if (isinstance(value, int) or isinstance(value, float)):  # for int and float type      
    for row in rows:
        if (row[column] >= value):
            list1.append(row)
        else:
            list2.append(row)
    return (list1, list2)
global time0
time0 = time.time()
global timegini
timegini = 0
global timesplit
timesplit = 0
def buildDecisionTree(rows, evaluationFunction=gini):
    # 递归建立决策树,当gain = 0 时停止递归
    # bulid decision tree by recursive function
    # stop recursive function when gain = 0
    # return tree
    global time0
    global timegini
    global timesplit
    currentGain = evaluationFunction(rows)
    column_length = len(rows[0])
    rows_length = len(rows)
    best_gain = 0.0
    best_value = None
    best_set = None

    # choose the best gain
    for col in range(column_length - 1):
        col_value_set = set([x[col] for x in rows])
        for value in col_value_set:
            list1, list2 = splitDatas(rows, value, col)
            
            timesplit += (time.time()-time0)
            time0 = time.time()
            
            p = len(list1) / rows_length
            gain = currentGain - p * evaluationFunction(list1) - (1 - p) * evaluationFunction(list2)
            
            timegini += (time.time()-time0)
            time0 = time.time()            
            
            if gain > best_gain:
                best_gain = gain
                best_value = (col, value)
                best_set = (list1, list2)

    dcY = {'impurity': '%.3f' % currentGain, 'samples': '%d' % rows_length}

    # stop or not stop
    if best_gain > 0:
        trueBranch = buildDecisionTree(best_set[0], evaluationFunction)
        falseBranch = buildDecisionTree(best_set[1], evaluationFunction)
        return Tree(col=best_value[0], value=best_value[1], trueBranch=trueBranch, falseBranch=falseBranch, summary=dcY)
    else:
        return Tree(results=calculateDiffCount(rows), summary=dcY, data=rows)


def prune(tree, miniGain, evaluationFunction=gini):
    # 剪枝, when gain < mini Gain，合并(merge the trueBranch and the falseBranch)

    if tree.trueBranch.results == None: prune(tree.trueBranch, miniGain, evaluationFunction)
    if tree.falseBranch.results == None: prune(tree.falseBranch, miniGain, evaluationFunction)

    if tree.trueBranch.results != None and tree.falseBranch.results != None:
        len1 = len(tree.trueBranch.data)
        len2 = len(tree.falseBranch.data)
        len3 = len(tree.trueBranch.data + tree.falseBranch.data)
        p = float(len1) / (len1 + len2)
        gain = evaluationFunction(tree.trueBranch.data + tree.falseBranch.data) - p * evaluationFunction(
            tree.trueBranch.data) - (1 - p) * evaluationFunction(tree.falseBranch.data)
        if (gain < miniGain):
            tree.data = tree.trueBranch.data + tree.falseBranch.data
            tree.results = calculateDiffCount(tree.data)
            tree.trueBranch = None
            tree.falseBranch = None


def classify(data, tree):
    if tree.results != None:
        return tree.results
    else:
        branch = None
        v = data[tree.col]
        if isinstance(v, int) or isinstance(v, float):
            if v >= tree.value:
                branch = tree.trueBranch
            else:
                branch = tree.falseBranch
        else:
            if v == tree.value:
                branch = tree.trueBranch
            else:
                branch = tree.falseBranch
        return classify(data, branch)

#画出决策树
#下面是辅助代码画出树
#plot tree and load data
def plot(decisionTree):
    """Plots the obtained decision tree. """

    def toString(decisionTree, indent=''):
        if decisionTree.results != None:  # leaf node
            return str(decisionTree.results)
        else:
            szCol = 'Column %s' % decisionTree.col
            if szCol in dcHeadings:
                szCol = dcHeadings[szCol]
            if isinstance(decisionTree.value, int) or isinstance(decisionTree.value, float):
                decision = '%s >= %s?' % (szCol, decisionTree.value)
            else:
                decision = '%s == %s?' % (szCol, decisionTree.value)
            trueBranch = indent + 'yes -> ' + toString(decisionTree.trueBranch, indent + '\t\t')
            falseBranch = indent + 'no  -> ' + toString(decisionTree.falseBranch, indent + '\t\t')
            return (decision + '\n' + trueBranch + '\n' + falseBranch)

    print(toString(decisionTree))
##负责读取数据
def loadCSV(file):
    """Loads a CSV file and converts all floats and ints into basic datatypes."""
    def convertTypes(s):
        s = s.strip()
        try:
            return float(s) if '.' in s else int(s)
        except ValueError:
            return s

    reader = csv.reader(open(file, 'rt'))
    dcHeader = {}
    if bHeader:
        lsHeader = next(reader)
        for i, szY in enumerate(lsHeader):
                szCol = 'Column %d' % i
                dcHeader[szCol] = str(szY)
    return dcHeader, [[convertTypes(item) for item in row] for row in reader]

bHeader = True
# the bigger example
dcHeadings, trainingData = loadCSV('/home/data/liuchang/data/train2base.csv')
basedecisionTree = buildDecisionTree(trainingData, evaluationFunction=gini)
#result = plot(basedecisionTree)
#print('剪枝后')
#prune(basedecisionTree, 0.2) # notify, when a branch is pruned (one time in this example)
#result = plot(decisionTree)

#classify
dcHeadings, testData = loadCSV('/home/data/liuchang/data/test2base.csv') 
total = len(testData)
error = 0
truth = []
for i in range(0,len(testData)):
    testres = classify(testData[i],basedecisionTree)
    try:
        a = testres[testData[i][len(trainingData[0]) - 1]]
        truth.append(len(testres))
    except:
        error += 1
accrate = 1 - (error / total)
print(accrate)
from collections import Counter
print(Counter(truth))
print(time.time( ) - t)

print('total time is',time.time()-t)
print('gini time',timegini)
print('split time',timesplit)

processing data......
processing complete!


/home/data/liuchang/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


/home/data/liuchang/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:308: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



各个特征的长度
1
3
5
99
2
3
200



0.9058823529411765
Counter({1: 584, 2: 275, 3: 142})
314.3685414791107
total time is 314.36860847473145
gini time 124.22823476791382
split time 188.10392832756042


In [6]:

for i in range(0,len(testData)):
    testres = classify(testData[i],basedecisionTree)
    try:
        a = testres[testData[i][len(trainingData[0]) - 1]]
        truth.append(len(testres))
    except:
        error += 1
accrate = 1 - (error / total)
print(accrate)
from collections import Counter
print(Counter(truth))
print(time.time( ) - t)

print('total time is',time.time()-t)
print('gini time',timegini)
print('split time',timesplit)

0.6
Counter({1: 1752, 2: 16})
411.2538411617279
total time is 411.25389289855957
gini time 132.4917185306549
split time 162.66724824905396


In [8]:
plot(basedecisionTree)

f5_LNpun_numbers0 >= 1.0?
yes -> f2_LTN_numbers1 >= 2?
		yes -> f4_LTN_POS0 >= 1.0?
				yes -> f4_LTN_POS73 >= 1.0?
						yes -> f6_W2V94 >= -1.3178025484085083?
								yes -> f6_W2V14 >= 0.4039909243583679?
										yes -> f6_W2V4 >= 2.540781021118164?
												yes -> {2: 1}
												no  -> f6_W2V18 >= -1.1096556186676025?
														yes -> {1: 42}
														no  -> f1_length >= 22?
																yes -> {1: 1}
																no  -> {2: 1}
										no  -> f6_W2V5 >= -0.8955596685409546?
												yes -> f6_W2V62 >= -0.6877578496932983?
														yes -> f6_W2V53 >= 0.04370548203587532?
																yes -> {2: 1, 1: 36}
																no  -> f6_W2V195 >= -0.2019852101802826?
																		yes -> f6_W2V41 >= 0.13136731088161469?
																				yes -> {1: 1, 2: 7}
																				no  -> f6_W2V145 >= -0.018172115087509155?
																						yes -> f6_W2V44 >= 0.031368400901556015?
																								yes -> {2: 7, 1: 1}
																						